In [ ]:
Planar data classification with one hidden layer
Welcome to your week 3 programming assignment. It's time to build your first neural network, which will have a hidden layer. You will see a big difference between this model and the one you implemented using logistic regression.

You will learn how to:

Implement a 2-class classification neural network with a single hidden layer
Use units with a non-linear activation function, such as tanh
Compute the cross entropy loss
Implement forward and backward propagation
1 - Packages
Let's first import all the packages that you will need during this assignment.

numpy is the fundamental package for scientific computing with Python.
sklearn provides simple and efficient tools for data mining and data analysis.
matplotlib is a library for plotting graphs in Python.
testCases provides some test examples to assess the correctness of your functions
planar_utils provide various useful functions used in this assignment

# Package imports
import numpy as np
import matplotlib.pyplot as plt
from testCases_v2 import *
import sklearn
import sklearn.datasets
import sklearn.linear_model
from planar_utils import plot_decision_boundary, sigmoid, load_planar_dataset, load_extra_datasets
​
%matplotlib inline
​
np.random.seed(1) # set a seed so that the results are consistent
2 - Dataset
First, let's get the dataset you will work on. The following code will load a "flower" 2-class dataset into variables X and Y.


X, Y = load_planar_dataset()
Visualize the dataset using matplotlib. The data looks like a "flower" with some red (label y=0) and some blue (y=1) points. Your goal is to build a model to fit this data.


# Visualize the data:
plt.scatter(X[0, :], X[1, :], c=Y, s=40, cmap=plt.cm.Spectral);

You have:

- a numpy-array (matrix) X that contains your features (x1, x2)
- a numpy-array (vector) Y that contains your labels (red:0, blue:1).
Lets first get a better sense of what our data is like.

Exercise: How many training examples do you have? In addition, what is the shape of the variables X and Y?

Hint: How do you get the shape of a numpy array? (help)


### START CODE HERE ### (≈ 3 lines of code)
shape_X = X.shape
shape_Y = Y.shape
m = shape_Y[1]  # training set size
### END CODE HERE ###
​
print ('The shape of X is: ' + str(shape_X))
print ('The shape of Y is: ' + str(shape_Y))
print ('I have m = %d training examples!' % (m))
The shape of X is: (2, 400)
The shape of Y is: (1, 400)
I have m = 400 training examples!
Expected Output:

shape of X	(2, 400)
shape of Y	(1, 400)
m	400
3 - Simple Logistic Regression
Before building a full neural network, lets first see how logistic regression performs on this problem. You can use sklearn's built-in functions to do that. Run the code below to train a logistic regression classifier on the dataset.


# Train the logistic regression classifier
clf = sklearn.linear_model.LogisticRegressionCV();
clf.fit(X.T, Y.T);
You can now plot the decision boundary of these models. Run the code below.


# Plot the decision boundary for logistic regression
plot_decision_boundary(lambda x: clf.predict(x), X, Y)
plt.title("Logistic Regression")
​
# Print accuracy
LR_predictions = clf.predict(X.T)
print ('Accuracy of logistic regression: %d ' % float((np.dot(Y,LR_predictions) + np.dot(1-Y,1-LR_predictions))/float(Y.size)*100) +
       '% ' + "(percentage of correctly labelled datapoints)")
Accuracy of logistic regression: 47 % (percentage of correctly labelled datapoints)

Expected Output:

Accuracy	47%
Interpretation: The dataset is not linearly separable, so logistic regression doesn't perform well. Hopefully a neural network will do better. Let's try this now!

4 - Neural Network model
Logistic regression did not work well on the "flower dataset". You are going to train a Neural Network with a single hidden layer.

Here is our model: 

Mathematically:

For one example x(i)x(i):
z[1](i)=W[1]x(i)+b[1](1)
(1)z[1](i)=W[1]x(i)+b[1]
a[1](i)=tanh(z[1](i))(2)
(2)a[1](i)=tanh⁡(z[1](i))
z[2](i)=W[2]a[1](i)+b[2](3)
(3)z[2](i)=W[2]a[1](i)+b[2]
ŷ (i)=a[2](i)=σ(z[2](i))(4)
(4)y^(i)=a[2](i)=σ(z[2](i))
y(i)prediction={10if a[2](i)>0.5otherwise (5)
(5)yprediction(i)={1if a[2](i)>0.50otherwise 
Given the predictions on all the examples, you can also compute the cost JJ as follows:
J=−1m∑i=0m(y(i)log(a[2](i))+(1−y(i))log(1−a[2](i)))(6)
(6)J=−1m∑i=0m(y(i)log⁡(a[2](i))+(1−y(i))log⁡(1−a[2](i)))
Reminder: The general methodology to build a Neural Network is to:

1. Define the neural network structure ( # of input units,  # of hidden units, etc). 
2. Initialize the model's parameters
3. Loop:
    - Implement forward propagation
    - Compute loss
    - Implement backward propagation to get the gradients
    - Update parameters (gradient descent)
You often build helper functions to compute steps 1-3 and then merge them into one function we call nn_model(). Once you've built nn_model() and learnt the right parameters, you can make predictions on new data.

4.1 - Defining the neural network structure
Exercise: Define three variables:

- n_x: the size of the input layer
- n_h: the size of the hidden layer (set this to 4) 
- n_y: the size of the output layer
Hint: Use shapes of X and Y to find n_x and n_y. Also, hard code the hidden layer size to be 4.


0
# GRADED FUNCTION: layer_sizes
​
def layer_sizes(X, Y):
    """
    Arguments:
    X -- input dataset of shape (input size, number of examples)
    Y -- labels of shape (output size, number of examples)
    
    Returns:
    n_x -- the size of the input layer
    n_h -- the size of the hidden layer
    n_y -- the size of the output layer
    """
    ### START CODE HERE ### (≈ 3 lines of code)
    n_x = X.shape[0] # size of input layer
    n_h = 4
    n_y = Y.shape[0] # size of output layer
    ### END CODE HERE ###
    return (n_x, n_h, n_y)

X_assess, Y_assess = layer_sizes_test_case()
(n_x, n_h, n_y) = layer_sizes(X_assess, Y_assess)
print("The size of the input layer is: n_x = " + str(n_x))
print("The size of the hidden layer is: n_h = " + str(n_h))
print("The size of the output layer is: n_y = " + str(n_y))
The size of the input layer is: n_x = 5
The size of the hidden layer is: n_h = 4
The size of the output layer is: n_y = 2
Expected Output (these are not the sizes you will use for your network, they are just used to assess the function you've just coded).

n_x	5
n_h	4
n_y	2
4.2 - Initialize the model's parameters
Exercise: Implement the function initialize_parameters().

Instructions:

Make sure your parameters' sizes are right. Refer to the neural network figure above if needed.
You will initialize the weights matrices with random values.
Use: np.random.randn(a,b) * 0.01 to randomly initialize a matrix of shape (a,b).
You will initialize the bias vectors as zeros.
Use: np.zeros((a,b)) to initialize a matrix of shape (a,b) with zeros.

)
# GRADED FUNCTION: initialize_parameters
​
def initialize_parameters(n_x, n_h, n_y):
    """
    Argument:
    n_x -- size of the input layer
    n_h -- size of the hidden layer
    n_y -- size of the output layer
    
    Returns:
    params -- python dictionary containing your parameters:
                    W1 -- weight matrix of shape (n_h, n_x)
                    b1 -- bias vector of shape (n_h, 1)
                    W2 -- weight matrix of shape (n_y, n_h)
                    b2 -- bias vector of shape (n_y, 1)
    """
    
    np.random.seed(2) # we set up a seed so that your output matches ours although the initialization is random.
    
    ### START CODE HERE ### (≈ 4 lines of code)
    W1 = np.random.randn(n_h,n_x)*0.01
    b1 = np.zeros((n_h, 1))
    W2 = np.random.randn(n_y,n_h)*0.01
    b2 = np.zeros((n_y, 1))
    ### END CODE HERE ###
    
    assert (W1.shape == (n_h, n_x))
    assert (b1.shape == (n_h, 1))
    assert (W2.shape == (n_y, n_h))
    assert (b2.shape == (n_y, 1))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

n_x, n_h, n_y = initialize_parameters_test_case()
​
parameters = initialize_parameters(n_x, n_h, n_y)
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))
W1 = [[-0.00416758 -0.00056267]
 [-0.02136196  0.01640271]
 [-0.01793436 -0.00841747]
 [ 0.00502881 -0.01245288]]
b1 = [[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]
W2 = [[-0.01057952 -0.00909008  0.00551454  0.02292208]]
b2 = [[ 0.]]
Expected Output:

W1	[[-0.00416758 -0.00056267] [-0.02136196 0.01640271] [-0.01793436 -0.00841747] [ 0.00502881 -0.01245288]]
b1	[[ 0.] [ 0.] [ 0.] [ 0.]]
W2	[[-0.01057952 -0.00909008 0.00551454 0.02292208]]
b2	[[ 0.]]
4.3 - The Loop
Question: Implement forward_propagation().

Instructions:

Look above at the mathematical representation of your classifier.
You can use the function sigmoid(). It is built-in (imported) in the notebook.
You can use the function np.tanh(). It is part of the numpy library.
The steps you have to implement are:
Retrieve each parameter from the dictionary "parameters" (which is the output of initialize_parameters()) by using parameters[".."].
Implement Forward Propagation. Compute Z[1],A[1],Z[2]Z[1],A[1],Z[2] and A[2]A[2] (the vector of all your predictions on all the examples in the training set).
Values needed in the backpropagation are stored in "cache". The cache will be given as an input to the backpropagation function.

# GRADED FUNCTION: forward_propagation
​
def forward_propagation(X, parameters):
    """
    Argument:
    X -- input data of size (n_x, m)
    parameters -- python dictionary containing your parameters (output of initialization function)
    
    Returns:
    A2 -- The sigmoid output of the second activation
    cache -- a dictionary containing "Z1", "A1", "Z2" and "A2"
    """
    # Retrieve each parameter from the dictionary "parameters"
    ### START CODE HERE ### (≈ 4 lines of code)
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    ### END CODE HERE ###
    
    # Implement Forward Propagation to calculate A2 (probabilities)
    ### START CODE HERE ### (≈ 4 lines of code)
    Z1 = np.matmul(W1,X)+b1
    A1 = np.tanh(Z1)
    Z2 = np.matmul(W2,A1)+b2
    A2 = sigmoid(Z2)
    ### END CODE HERE ###
    
    assert(A2.shape == (1, X.shape[1]))
    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return A2, cache

X_assess, parameters = forward_propagation_test_case()
A2, cache = forward_propagation(X_assess, parameters)
​
# Note: we use the mean here just to make sure that your output matches ours. 
print(np.mean(cache['Z1']) ,np.mean(cache['A1']),np.mean(cache['Z2']),np.mean(cache['A2']))
0.262818640198 0.091999045227 -1.30766601287 0.212877681719
Expected Output:

0.262818640198 0.091999045227 -1.30766601287 0.212877681719
Now that you have computed A[2]A[2] (in the Python variable "A2"), which contains a[2](i)a[2](i) for every example, you can compute the cost function as follows:

J=−1m∑i=0m(y(i)log(a[2](i))+(1−y(i))log(1−a[2](i)))(13)
(13)J=−1m∑i=0m(y(i)log⁡(a[2](i))+(1−y(i))log⁡(1−a[2](i)))
Exercise: Implement compute_cost() to compute the value of the cost JJ.

Instructions:

There are many ways to implement the cross-entropy loss. To help you, we give you how we would have implemented −∑i=0my(i)log(a[2](i))−∑i=0my(i)log⁡(a[2](i)):
logprobs = np.multiply(np.log(A2),Y)
cost = - np.sum(logprobs)                # no need to use a for loop!
(you can use either np.multiply() and then np.sum() or directly np.dot()).


-np.dot(np.log(A2),Y.T)
# GRADED FUNCTION: compute_cost
​
def compute_cost(A2, Y, parameters):
    """
    Computes the cross-entropy cost given in equation (13)
    
    Arguments:
    A2 -- The sigmoid output of the second activation, of shape (1, number of examples)
    Y -- "true" labels vector of shape (1, number of examples)
    parameters -- python dictionary containing your parameters W1, b1, W2 and b2
    
    Returns:
    cost -- cross-entropy cost given equation (13)
    """
    
    m = Y.shape[1] # number of example
​
    # Compute the cross-entropy cost
    ### START CODE HERE ### (≈ 2 lines of code)
    logprobs = np.multiply(np.log(A2),Y)
    cost = - np.sum(logprobs) 
    print(cost)
    ### END CODE HERE ###
    
    cost = np.squeeze(cost)     # makes sure cost is the dimension we expect. 
                                # E.g., turns [[17]] into 17 
    print(type(cost))
    assert(isinstance(cost, float))
    
    return cost

A2, Y_assess, parameters = compute_cost_test_case()
print(A2.shape)
print(Y_assess.shape)
​
print("cost = " + str(compute_cost(A2, Y_assess, parameters)))
(1, 3)
(1, 3)
0.692685886972
<class 'numpy.float64'>
cost = 0.692685886972

​
Expected Output:

cost	0.693058761...
Using the cache computed during forward propagation, you can now implement backward propagation.

Question: Implement the function backward_propagation().

Instructions: Backpropagation is usually the hardest (most mathematical) part in deep learning. To help you, here again is the slide from the lecture on backpropagation. You'll want to use the six equations on the right of this slide, since you are building a vectorized implementation.



Tips:
To compute dZ1 you'll need to compute g[1]′(Z[1])g[1]′(Z[1]). Since g[1](.)g[1](.) is the tanh activation function, if a=g[1](z)a=g[1](z) then g[1]′(z)=1−a2g[1]′(z)=1−a2. So you can compute g[1]′(Z[1])g[1]′(Z[1]) using (1 - np.power(A1, 2)).

.
# GRADED FUNCTION: backward_propagation
​
def backward_propagation(parameters, cache, X, Y):
    """
    Implement the backward propagation using the instructions above.
    
    Arguments:
    parameters -- python dictionary containing our parameters 
    cache -- a dictionary containing "Z1", "A1", "Z2" and "A2".
    X -- input data of shape (2, number of examples)
    Y -- "true" labels vector of shape (1, number of examples)
    
    Returns:
    grads -- python dictionary containing your gradients with respect to different parameters
    """
    m = X.shape[1]
    
    # First, retrieve W1 and W2 from the dictionary "parameters".
    ### START CODE HERE ### (≈ 2 lines of code)
    W1 = parameters["W1"]
    W2 = parameters["W2"]
    ### END CODE HERE ###
        
    # Retrieve also A1 and A2 from dictionary "cache".
    ### START CODE HERE ### (≈ 2 lines of code)
    A1 = cache["A1"]
    A2 = cache["A2"]
    ### END CODE HERE ###
    
    # Backward propagation: calculate dW1, db1, dW2, db2. 
    ### START CODE HERE ### (≈ 6 lines of code, corresponding to 6 equations on slide above)
    dZ2 = A2-Y
    dW2 = np.matmul(dZ2,A1.T)/m
    db2 = 1/m*np.sum(dZ2,axis=1,keepdims=True)
    dZ1 = np.matmul(W2.T,dZ2)*(1 - np.power(A1, 2))
    dW1 = np.matmul(dZ1,X.T)/m
    db1 = 1/m*np.sum(dZ1,axis=1,keepdims=True)
    ### END CODE HERE ###
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads

parameters, cache, X_assess, Y_assess = backward_propagation_test_case()
​
grads = backward_propagation(parameters, cache, X_assess, Y_assess)
print ("dW1 = "+ str(grads["dW1"]))
print ("db1 = "+ str(grads["db1"]))
print ("dW2 = "+ str(grads["dW2"]))
print ("db2 = "+ str(grads["db2"]))
dW1 = [[ 0.00301023 -0.00747267]
 [ 0.00257968 -0.00641288]
 [-0.00156892  0.003893  ]
 [-0.00652037  0.01618243]]
db1 = [[ 0.00176201]
 [ 0.00150995]
 [-0.00091736]
 [-0.00381422]]
dW2 = [[ 0.00078841  0.01765429 -0.00084166 -0.01022527]]
db2 = [[-0.16655712]]
Expected output:

dW1	[[ 0.00301023 -0.00747267] [ 0.00257968 -0.00641288] [-0.00156892 0.003893 ] [-0.00652037 0.01618243]]
db1	[[ 0.00176201] [ 0.00150995] [-0.00091736] [-0.00381422]]
dW2	[[ 0.00078841 0.01765429 -0.00084166 -0.01022527]]
db2	[[-0.16655712]]
Question: Implement the update rule. Use gradient descent. You have to use (dW1, db1, dW2, db2) in order to update (W1, b1, W2, b2).

General gradient descent rule: θ=θ−α∂J∂θθ=θ−α∂J∂θ where αα is the learning rate and θθ represents a parameter.

Illustration: The gradient descent algorithm with a good learning rate (converging) and a bad learning rate (diverging). Images courtesy of Adam Harley.




b2
# GRADED FUNCTION: update_parameters
​
def update_parameters(parameters, grads, learning_rate = 1.2):
    """
    Updates parameters using the gradient descent update rule given above
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients 
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
    """
    # Retrieve each parameter from the dictionary "parameters"
    ### START CODE HERE ### (≈ 4 lines of code)
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    ### END CODE HERE ###
    
    # Retrieve each gradient from the dictionary "grads"
    ### START CODE HERE ### (≈ 4 lines of code)
    dW1 = grads["dW1"]
    db1 = grads["db1"]
    dW2 = grads["dW2"]
    db2 = grads["db2"]
    ## END CODE HERE ###
    
    # Update rule for each parameter
    ### START CODE HERE ### (≈ 4 lines of code)
    W1 = W1-learning_rate*dW1
    b1 = b1-learning_rate*db1
    W2 = W2-learning_rate*dW2
    b2 = b2-learning_rate*db2
    ### END CODE HERE ###
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

parameters, grads = update_parameters_test_case()
parameters = update_parameters(parameters, grads)
​
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))
W1 = [[-0.00643025  0.01936718]
 [-0.02410458  0.03978052]
 [-0.01653973 -0.02096177]
 [ 0.01046864 -0.05990141]]
b1 = [[ -1.02420756e-06]
 [  1.27373948e-05]
 [  8.32996807e-07]
 [ -3.20136836e-06]]
W2 = [[-0.01041081 -0.04463285  0.01758031  0.04747113]]
b2 = [[ 0.00010457]]
Expected Output:

W1	[[-0.00643025 0.01936718] [-0.02410458 0.03978052] [-0.01653973 -0.02096177] [ 0.01046864 -0.05990141]]
b1	[[ -1.02420756e-06] [ 1.27373948e-05] [ 8.32996807e-07] [ -3.20136836e-06]]
W2	[[-0.01041081 -0.04463285 0.01758031 0.04747113]]
b2	[[ 0.00010457]]
4.4 - Integrate parts 4.1, 4.2 and 4.3 in nn_model()
Question: Build your neural network model in nn_model().

Instructions: The neural network model has to use the previous functions in the right order.


# GRADED FUNCTION: nn_model
​
def nn_model(X, Y, n_h, num_iterations = 10000, print_cost=False):
    """
    Arguments:
    X -- dataset of shape (2, number of examples)
    Y -- labels of shape (1, number of examples)
    n_h -- size of the hidden layer
    num_iterations -- Number of iterations in gradient descent loop
    print_cost -- if True, print the cost every 1000 iterations
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """
    
    np.random.seed(3)
    n_x = layer_sizes(X, Y)[0]
    n_y = layer_sizes(X, Y)[2]
    
    # Initialize parameters, then retrieve W1, b1, W2, b2. Inputs: "n_x, n_h, n_y". Outputs = "W1, b1, W2, b2, parameters".
    ### START CODE HERE ### (≈ 5 lines of code)
    parameters = initialize_parameters(n_x,n_h,n_y)
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    ### END CODE HERE ###
    
    # Loop (gradient descent)
​
    for i in range(0, num_iterations):
         
        ### START CODE HERE ### (≈ 4 lines of code)
        # Forward propagation. Inputs: "X, parameters". Outputs: "A2, cache".
        A2, cache = forward_propagation(X,parameters)
        
        # Cost function. Inputs: "A2, Y, parameters". Outputs: "cost".
        cost = compute_cost(A2,Y,parameters)
 
        # Backpropagation. Inputs: "parameters, cache, X, Y". Outputs: "grads".
        grads = backward_propagation(parameters,cache,X,Y)
 
        # Gradient descent parameter update. Inputs: "parameters, grads". Outputs: "parameters".
        parameters = update_parameters(parameters,grads)
        
        ### END CODE HERE ###
        
        # Print the cost every 1000 iterations
        if print_cost and i % 1000 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
​
    return parameters

X_assess, Y_assess = nn_model_test_case()
parameters = nn_model(X_assess, Y_assess, 4, num_iterations=10000, print_cost=True)
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))
1.38535391185
<class 'numpy.float64'>
Cost after iteration 0: 1.385354
1.1925122892
<class 'numpy.float64'>
1.06282256619
<class 'numpy.float64'>
0.958213825466
<class 'numpy.float64'>
0.83815969064
<class 'numpy.float64'>
0.66299516054
<class 'numpy.float64'>
0.453141634454
<class 'numpy.float64'>
0.287524758933
<class 'numpy.float64'>
0.188486372739
<class 'numpy.float64'>
0.133026041624
<class 'numpy.float64'>
0.100373791899
<class 'numpy.float64'>
0.0796460237499
<class 'numpy.float64'>
0.0655808035526
<class 'numpy.float64'>
0.0555141965646
<class 'numpy.float64'>
0.0480004129985
<class 'numpy.float64'>
0.0422015300761
<class 'numpy.float64'>
0.0376037109
<class 'numpy.float64'>
0.033876526904
<class 'numpy.float64'>
0.0307988342099
<class 'numpy.float64'>
0.0282175424415
<class 'numpy.float64'>
0.0260235716087
<class 'numpy.float64'>
0.0241372458177
<class 'numpy.float64'>
0.0224990980403
<class 'numpy.float64'>
0.0210638981399
<class 'numpy.float64'>
0.0197966664902
<class 'numpy.float64'>
0.0186699476421
<class 'numpy.float64'>
0.0176619050291
<class 'numpy.float64'>
0.0167549634407
<class 'numpy.float64'>
0.0159348247938
<class 'numpy.float64'>
0.0151897432228
<class 'numpy.float64'>
0.0145099834718
<class 'numpy.float64'>
0.0138874109227
<class 'numpy.float64'>
0.0133151775346
<class 'numpy.float64'>
0.0127874785997
<class 'numpy.float64'>
0.0122993624301
<class 'numpy.float64'>
0.0118465800563
<class 'numpy.float64'>
0.0114254654909
<class 'numpy.float64'>
0.0110328395668
<class 'numpy.float64'>
0.0106659321269
<class 'numpy.float64'>
0.010322318616
<class 'numpy.float64'>
0.00999986807071
<class 'numpy.float64'>
0.0096967001911
<class 'numpy.float64'>
0.00941114970421
<class 'numpy.float64'>
0.00914173661703
<class 'numpy.float64'>
0.00888714125763
<class 'numpy.float64'>
0.00864618323139
<class 'numpy.float64'>
0.00841780359587
<class 'numpy.float64'>
0.00820104969599
<class 'numpy.float64'>
0.0079950622087
<class 'numpy.float64'>
0.00779906403159
<class 'numpy.float64'>
0.00761235071688
<class 'numpy.float64'>
0.00743428220671
<class 'numpy.float64'>
0.00726427566795
<class 'numpy.float64'>
0.00710179926023
<class 'numpy.float64'>
0.00694636669866
<class 'numpy.float64'>
0.00679753249571
<class 'numpy.float64'>
0.00665488778566
<class 'numpy.float64'>
0.00651805665004
<class 'numpy.float64'>
0.00638669287577
<class 'numpy.float64'>
0.0062604770875
<class 'numpy.float64'>
0.0061391142051
<class 'numpy.float64'>
0.00602233118402
<class 'numpy.float64'>
0.00590987500255
<class 'numpy.float64'>
0.00580151086528
<class 'numpy.float64'>
0.005697020596
<class 'numpy.float64'>
0.0055962011973
<class 'numpy.float64'>
0.00549886355715
<class 'numpy.float64'>
0.00540483128507
<class 'numpy.float64'>
0.00531393966326
<class 'numpy.float64'>
0.00522603469953
<class 'numpy.float64'>
0.00514097227074
<class 'numpy.float64'>
0.00505861734689
<class 'numpy.float64'>
0.00497884328711
<class 'numpy.float64'>
0.00490153119986
<class 'numpy.float64'>
0.00482656936076
<class 'numpy.float64'>
0.00475385268195
<class 'numpy.float64'>
0.00468328222781
<class 'numpy.float64'>
0.00461476477243
<class 'numpy.float64'>
0.00454821239461
<class 'numpy.float64'>
0.00448354210682
<class 'numpy.float64'>
0.00442067551476
<class 'numpy.float64'>
0.0043595385047
<class 'numpy.float64'>
0.004300060956
<class 'numpy.float64'>
0.00424217647635
<class 'numpy.float64'>
0.00418582215785
<class 'numpy.float64'>
0.00413093835195
<class 'numpy.float64'>
0.00407746846148
<class 'numpy.float64'>
0.0040253587485
<class 'numpy.float64'>
0.00397455815639
<class 'numpy.float64'>
0.00392501814505
<class 'numpy.float64'>
0.0038766925381
<class 'numpy.float64'>
0.00382953738101
<class 'numpy.float64'>
0.0037835108094
<class 'numpy.float64'>
0.00373857292648
<class 'numpy.float64'>
0.00369468568907
<class 'numpy.float64'>
0.00365181280143
<class 'numpy.float64'>
0.00360991961622
<class 'numpy.float64'>
0.00356897304222
<class 'numpy.float64'>
0.00352894145812
<class 'numpy.float64'>
0.0034897946319
<class 'numpy.float64'>
0.00345150364558
<class 'numpy.float64'>
0.00341404082473
<class 'numpy.float64'>
0.00337737967241
<class 'numpy.float64'>
0.0033414948074
<class 'numpy.float64'>
0.00330636190615
<class 'numpy.float64'>
0.00327195764838
<class 'numpy.float64'>
0.00323825966597
<class 'numpy.float64'>
0.00320524649497
<class 'numpy.float64'>
0.00317289753042
<class 'numpy.float64'>
0.00314119298389
<class 'numpy.float64'>
0.00311011384356
<class 'numpy.float64'>
0.00307964183646
<class 'numpy.float64'>
0.00304975939309
<class 'numpy.float64'>
0.00302044961387
<class 'numpy.float64'>
0.00299169623762
<class 'numpy.float64'>
0.00296348361174
<class 'numpy.float64'>
0.00293579666401
<class 'numpy.float64'>
0.00290862087604
<class 'numpy.float64'>
0.00288194225808
<class 'numpy.float64'>
0.00285574732523
<class 'numpy.float64'>
0.00283002307486
<class 'numpy.float64'>
0.00280475696537
<class 'numpy.float64'>
0.00277993689586
<class 'numpy.float64'>
0.00275555118707
<class 'numpy.float64'>
0.00273158856313
<class 'numpy.float64'>
0.00270803813437
<class 'numpy.float64'>
0.00268488938094
<class 'numpy.float64'>
0.00266213213724
<class 'numpy.float64'>
0.00263975657719
<class 'numpy.float64'>
0.00261775320018
<class 'numpy.float64'>
0.00259611281774
<class 'numpy.float64'>
0.00257482654082
<class 'numpy.float64'>
0.00255388576773
<class 'numpy.float64'>
0.00253328217263
<class 'numpy.float64'>
0.00251300769457
<class 'numpy.float64'>
0.00249305452709
<class 'numpy.float64'>
0.00247341510821
<class 'numpy.float64'>
0.002454082111
<class 'numpy.float64'>
0.00243504843452
<class 'numpy.float64'>
0.00241630719514
<class 'numpy.float64'>
0.00239785171839
<class 'numpy.float64'>
0.002379675531
<class 'numpy.float64'>
0.00236177235343
<class 'numpy.float64'>
0.00234413609269
<class 'numpy.float64'>
0.00232676083545
<class 'numpy.float64'>
0.00230964084149
<class 'numpy.float64'>
0.00229277053742
<class 'numpy.float64'>
0.00227614451067
<class 'numpy.float64'>
0.00225975750373
<class 'numpy.float64'>
0.00224360440866
<class 'numpy.float64'>
0.0022276802618
<class 'numpy.float64'>
0.00221198023872
<class 'numpy.float64'>
0.00219649964941
<class 'numpy.float64'>
0.00218123393357
<class 'numpy.float64'>
0.00216617865624
<class 'numpy.float64'>
0.00215132950346
<class 'numpy.float64'>
0.00213668227824
<class 'numpy.float64'>
0.00212223289656
<class 'numpy.float64'>
0.00210797738367
<class 'numpy.float64'>
0.00209391187039
<class 'numpy.float64'>
0.0020800325897
<class 'numpy.float64'>
0.00206633587337
<class 'numpy.float64'>
0.0020528181487
<class 'numpy.float64'>
0.00203947593553
<class 'numpy.float64'>
0.00202630584317
<class 'numpy.float64'>
0.00201330456761
<class 'numpy.float64'>
0.00200046888874
<class 'numpy.float64'>
0.00198779566773
<class 'numpy.float64'>
0.00197528184446
<class 'numpy.float64'>
0.00196292443508
<class 'numpy.float64'>
0.00195072052967
<class 'numpy.float64'>
0.00193866728992
<class 'numpy.float64'>
0.00192676194701
<class 'numpy.float64'>
0.00191500179944
<class 'numpy.float64'>
0.00190338421102
<class 'numpy.float64'>
0.00189190660892
<class 'numpy.float64'>
0.00188056648177
<class 'numpy.float64'>
0.00186936137783
<class 'numpy.float64'>
0.00185828890326
<class 'numpy.float64'>
0.00184734672038
<class 'numpy.float64'>
0.00183653254607
<class 'numpy.float64'>
0.00182584415019
<class 'numpy.float64'>
0.00181527935402
<class 'numpy.float64'>
0.00180483602882
<class 'numpy.float64'>
0.0017945120944
<class 'numpy.float64'>
0.00178430551773
<class 'numpy.float64'>
0.00177421431163
<class 'numpy.float64'>
0.00176423653348
<class 'numpy.float64'>
0.00175437028398
<class 'numpy.float64'>
0.00174461370595
<class 'numpy.float64'>
0.00173496498315
<class 'numpy.float64'>
0.00172542233921
<class 'numpy.float64'>
0.00171598403649
<class 'numpy.float64'>
0.00170664837505
<class 'numpy.float64'>
0.00169741369166
<class 'numpy.float64'>
0.00168827835877
<class 'numpy.float64'>
0.00167924078358
<class 'numpy.float64'>
0.00167029940714
<class 'numpy.float64'>
0.00166145270342
<class 'numpy.float64'>
0.00165269917846
<class 'numpy.float64'>
0.00164403736954
<class 'numpy.float64'>
0.00163546584437
<class 'numpy.float64'>
0.0016269832003
<class 'numpy.float64'>
0.00161858806353
<class 'numpy.float64'>
0.00161027908842
<class 'numpy.float64'>
0.00160205495675
<class 'numpy.float64'>
0.00159391437701
<class 'numpy.float64'>
0.00158585608377
<class 'numpy.float64'>
0.00157787883698
<class 'numpy.float64'>
0.00156998142135
<class 'numpy.float64'>
0.00156216264577
<class 'numpy.float64'>
0.00155442134265
<class 'numpy.float64'>
0.00154675636742
<class 'numpy.float64'>
0.00153916659788
<class 'numpy.float64'>
0.00153165093372
<class 'numpy.float64'>
0.00152420829598
<class 'numpy.float64'>
0.0015168376265
<class 'numpy.float64'>
0.00150953788748
<class 'numpy.float64'>
0.00150230806091
<class 'numpy.float64'>
0.0014951471482
<class 'numpy.float64'>
0.00148805416964
<class 'numpy.float64'>
0.00148102816399
<class 'numpy.float64'>
0.00147406818805
<class 'numpy.float64'>
0.00146717331624
<class 'numpy.float64'>
0.00146034264015
<class 'numpy.float64'>
0.00145357526821
<class 'numpy.float64'>
0.00144687032526
<class 'numpy.float64'>
0.00144022695217
<class 'numpy.float64'>
0.00143364430551
<class 'numpy.float64'>
0.00142712155716
<class 'numpy.float64'>
0.00142065789396
<class 'numpy.float64'>
0.00141425251742
<class 'numpy.float64'>
0.00140790464335
<class 'numpy.float64'>
0.00140161350154
<class 'numpy.float64'>
0.00139537833547
<class 'numpy.float64'>
0.00138919840201
<class 'numpy.float64'>
0.00138307297109
<class 'numpy.float64'>
0.00137700132548
<class 'numpy.float64'>
0.00137098276043
<class 'numpy.float64'>
0.00136501658346
<class 'numpy.float64'>
0.00135910211406
<class 'numpy.float64'>
0.00135323868346
<class 'numpy.float64'>
0.00134742563436
<class 'numpy.float64'>
0.00134166232067
<class 'numpy.float64'>
0.00133594810733
<class 'numpy.float64'>
0.00133028237
<class 'numpy.float64'>
0.00132466449489
<class 'numpy.float64'>
0.00131909387852
<class 'numpy.float64'>
0.0013135699275
<class 'numpy.float64'>
0.00130809205833
<class 'numpy.float64'>
0.00130265969718
<class 'numpy.float64'>
0.00129727227972
<class 'numpy.float64'>
0.0012919292509
<class 'numpy.float64'>
0.00128663006474
<class 'numpy.float64'>
0.0012813741842
<class 'numpy.float64'>
0.00127616108096
<class 'numpy.float64'>
0.00127099023525
<class 'numpy.float64'>
0.00126586113569
<class 'numpy.float64'>
0.00126077327909
<class 'numpy.float64'>
0.00125572617033
<class 'numpy.float64'>
0.00125071932217
<class 'numpy.float64'>
0.0012457522551
<class 'numpy.float64'>
0.0012408244972
<class 'numpy.float64'>
0.00123593558396
<class 'numpy.float64'>
0.00123108505818
<class 'numpy.float64'>
0.00122627246977
<class 'numpy.float64'>
0.00122149737568
<class 'numpy.float64'>
0.00121675933971
<class 'numpy.float64'>
0.00121205793238
<class 'numpy.float64'>
0.00120739273083
<class 'numpy.float64'>
0.0012027633187
<class 'numpy.float64'>
0.00119816928596
<class 'numpy.float64'>
0.00119361022881
<class 'numpy.float64'>
0.00118908574959
<class 'numpy.float64'>
0.00118459545663
<class 'numpy.float64'>
0.00118013896414
<class 'numpy.float64'>
0.00117571589214
<class 'numpy.float64'>
0.00117132586628
<class 'numpy.float64'>
0.00116696851781
<class 'numpy.float64'>
0.00116264348342
<class 'numpy.float64'>
0.00115835040517
<class 'numpy.float64'>
0.00115408893039
<class 'numpy.float64'>
0.00114985871158
<class 'numpy.float64'>
0.00114565940629
<class 'numpy.float64'>
0.00114149067708
<class 'numpy.float64'>
0.00113735219137
<class 'numpy.float64'>
0.00113324362142
<class 'numpy.float64'>
0.00112916464417
<class 'numpy.float64'>
0.00112511494121
<class 'numpy.float64'>
0.00112109419868
<class 'numpy.float64'>
0.00111710210718
<class 'numpy.float64'>
0.00111313836171
<class 'numpy.float64'>
0.00110920266157
<class 'numpy.float64'>
0.0011052947103
<class 'numpy.float64'>
0.0011014142156
<class 'numpy.float64'>
0.00109756088926
<class 'numpy.float64'>
0.0010937344471
<class 'numpy.float64'>
0.00108993460885
<class 'numpy.float64'>
0.00108616109817
<class 'numpy.float64'>
0.00108241364249
<class 'numpy.float64'>
0.00107869197301
<class 'numpy.float64'>
0.0010749958246
<class 'numpy.float64'>
0.00107132493577
<class 'numpy.float64'>
0.00106767904857
<class 'numpy.float64'>
0.00106405790855
<class 'numpy.float64'>
0.0010604612647
<class 'numpy.float64'>
0.0010568888694
<class 'numpy.float64'>
0.00105334047836
<class 'numpy.float64'>
0.00104981585052
<class 'numpy.float64'>
0.00104631474809
<class 'numpy.float64'>
0.00104283693639
<class 'numpy.float64'>
0.00103938218389
<class 'numpy.float64'>
0.00103595026209
<class 'numpy.float64'>
0.00103254094552
<class 'numpy.float64'>
0.00102915401163
<class 'numpy.float64'>
0.00102578924083
<class 'numpy.float64'>
0.00102244641635
<class 'numpy.float64'>
0.00101912532426
<class 'numpy.float64'>
0.0010158257534
<class 'numpy.float64'>
0.00101254749531
<class 'numpy.float64'>
0.00100929034425
<class 'numpy.float64'>
0.00100605409708
<class 'numpy.float64'>
0.00100283855329
<class 'numpy.float64'>
0.00099964351491
<class 'numpy.float64'>
0.00099646878648
<class 'numpy.float64'>
0.000993314175023
<class 'numpy.float64'>
0.000990179489997
<class 'numpy.float64'>
0.000987064543254
<class 'numpy.float64'>
0.000983969149011
<class 'numpy.float64'>
0.000980893123805
<class 'numpy.float64'>
0.00097783628646
<class 'numpy.float64'>
0.000974798458052
<class 'numpy.float64'>
0.000971779461875
<class 'numpy.float64'>
0.000968779123404
<class 'numpy.float64'>
0.000965797270263
<class 'numpy.float64'>
0.000962833732192
<class 'numpy.float64'>
0.000959888341015
<class 'numpy.float64'>
0.000956960930606
<class 'numpy.float64'>
0.000954051336864
<class 'numpy.float64'>
0.000951159397673
<class 'numpy.float64'>
0.000948284952879
<class 'numpy.float64'>
0.000945427844259
<class 'numpy.float64'>
0.000942587915492
<class 'numpy.float64'>
0.000939765012127
<class 'numpy.float64'>
0.000936958981562
<class 'numpy.float64'>
0.000934169673008
<class 'numpy.float64'>
0.000931396937472
<class 'numpy.float64'>
0.00092864062772
<class 'numpy.float64'>
0.000925900598259
<class 'numpy.float64'>
0.000923176705308
<class 'numpy.float64'>
0.000920468806774
<class 'numpy.float64'>
0.000917776762225
<class 'numpy.float64'>
0.000915100432869
<class 'numpy.float64'>
0.000912439681528
<class 'numpy.float64'>
0.000909794372615
<class 'numpy.float64'>
0.000907164372113
<class 'numpy.float64'>
0.00090454954755
<class 'numpy.float64'>
0.000901949767974
<class 'numpy.float64'>
0.000899364903941
<class 'numpy.float64'>
0.000896794827482
<class 'numpy.float64'>
0.000894239412089
<class 'numpy.float64'>
0.000891698532693
<class 'numpy.float64'>
0.000889172065641
<class 'numpy.float64'>
0.000886659888681
<class 'numpy.float64'>
0.000884161880936
<class 'numpy.float64'>
0.000881677922891
<class 'numpy.float64'>
0.000879207896368
<class 'numpy.float64'>
0.000876751684512
<class 'numpy.float64'>
0.00087430917177
<class 'numpy.float64'>
0.000871880243875
<class 'numpy.float64'>
0.000869464787824
<class 'numpy.float64'>
0.000867062691867
<class 'numpy.float64'>
0.000864673845483
<class 'numpy.float64'>
0.000862298139368
<class 'numpy.float64'>
0.000859935465417
<class 'numpy.float64'>
0.000857585716705
<class 'numpy.float64'>
0.000855248787474
<class 'numpy.float64'>
0.000852924573118
<class 'numpy.float64'>
0.000850612970164
<class 'numpy.float64'>
0.000848313876258
<class 'numpy.float64'>
0.000846027190151
<class 'numpy.float64'>
0.000843752811684
<class 'numpy.float64'>
0.000841490641771
<class 'numpy.float64'>
0.000839240582387
<class 'numpy.float64'>
0.000837002536555
<class 'numpy.float64'>
0.000834776408328
<class 'numpy.float64'>
0.000832562102779
<class 'numpy.float64'>
0.000830359525985
<class 'numpy.float64'>
0.000828168585017
<class 'numpy.float64'>
0.000825989187923
<class 'numpy.float64'>
0.000823821243717
<class 'numpy.float64'>
0.000821664662367
<class 'numpy.float64'>
0.000819519354782
<class 'numpy.float64'>
0.0008173852328
<class 'numpy.float64'>
0.000815262209174
<class 'numpy.float64'>
0.000813150197563
<class 'numpy.float64'>
0.00081104911252
<class 'numpy.float64'>
0.000808958869475
<class 'numpy.float64'>
0.000806879384735
<class 'numpy.float64'>
0.000804810575461
<class 'numpy.float64'>
0.000802752359663
<class 'numpy.float64'>
0.000800704656188
<class 'numpy.float64'>
0.000798667384712
<class 'numpy.float64'>
0.000796640465724
<class 'numpy.float64'>
0.00079462382052
<class 'numpy.float64'>
0.000792617371192
<class 'numpy.float64'>
0.000790621040617
<class 'numpy.float64'>
0.000788634752446
<class 'numpy.float64'>
0.0007866584311
<class 'numpy.float64'>
0.000784692001752
<class 'numpy.float64'>
0.000782735390325
<class 'numpy.float64'>
0.000780788523477
<class 'numpy.float64'>
0.000778851328598
<class 'numpy.float64'>
0.000776923733794
<class 'numpy.float64'>
0.000775005667882
<class 'numpy.float64'>
0.000773097060384
<class 'numpy.float64'>
0.00077119784151
<class 'numpy.float64'>
0.00076930794216
<class 'numpy.float64'>
0.000767427293908
<class 'numpy.float64'>
0.000765555828993
<class 'numpy.float64'>
0.00076369348032
<class 'numpy.float64'>
0.000761840181441
<class 'numpy.float64'>
0.000759995866554
<class 'numpy.float64'>
0.000758160470493
<class 'numpy.float64'>
0.000756333928721
<class 'numpy.float64'>
0.000754516177322
<class 'numpy.float64'>
0.000752707152991
<class 'numpy.float64'>
0.000750906793034
<class 'numpy.float64'>
0.000749115035351
<class 'numpy.float64'>
0.000747331818438
<class 'numpy.float64'>
0.000745557081373
<class 'numpy.float64'>
0.000743790763814
<class 'numpy.float64'>
0.000742032805987
<class 'numpy.float64'>
0.000740283148686
<class 'numpy.float64'>
0.000738541733259
<class 'numpy.float64'>
0.000736808501609
<class 'numpy.float64'>
0.00073508339618
<class 'numpy.float64'>
0.000733366359958
<class 'numpy.float64'>
0.000731657336457
<class 'numpy.float64'>
0.00072995626972
<class 'numpy.float64'>
0.000728263104307
<class 'numpy.float64'>
0.000726577785296
<class 'numpy.float64'>
0.000724900258267
<class 'numpy.float64'>
0.000723230469307
<class 'numpy.float64'>
0.000721568364997
<class 'numpy.float64'>
0.000719913892407
<class 'numpy.float64'>
0.000718266999095
<class 'numpy.float64'>
0.000716627633096
<class 'numpy.float64'>
0.00071499574292
<class 'numpy.float64'>
0.000713371277544
<class 'numpy.float64'>
0.000711754186411
<class 'numpy.float64'>
0.000710144419418
<class 'numpy.float64'>
0.000708541926918
<class 'numpy.float64'>
0.00070694665971
<class 'numpy.float64'>
0.000705358569037
<class 'numpy.float64'>
0.000703777606577
<class 'numpy.float64'>
0.000702203724443
<class 'numpy.float64'>
0.000700636875176
<class 'numpy.float64'>
0.000699077011739
<class 'numpy.float64'>
0.000697524087513
<class 'numpy.float64'>
0.000695978056295
<class 'numpy.float64'>
0.00069443887229
<class 'numpy.float64'>
0.000692906490105
<class 'numpy.float64'>
0.000691380864753
<class 'numpy.float64'>
0.000689861951638
<class 'numpy.float64'>
0.000688349706557
<class 'numpy.float64'>
0.000686844085694
<class 'numpy.float64'>
0.000685345045618
<class 'numpy.float64'>
0.000683852543273
<class 'numpy.float64'>
0.000682366535981
<class 'numpy.float64'>
0.000680886981433
<class 'numpy.float64'>
0.000679413837687
<class 'numpy.float64'>
0.000677947063164
<class 'numpy.float64'>
0.000676486616643
<class 'numpy.float64'>
0.000675032457257
<class 'numpy.float64'>
0.000673584544494
<class 'numpy.float64'>
0.000672142838184
<class 'numpy.float64'>
0.000670707298504
<class 'numpy.float64'>
0.000669277885971
<class 'numpy.float64'>
0.000667854561435
<class 'numpy.float64'>
0.000666437286084
<class 'numpy.float64'>
0.000665026021431
<class 'numpy.float64'>
0.000663620729316
<class 'numpy.float64'>
0.000662221371901
<class 'numpy.float64'>
0.000660827911668
<class 'numpy.float64'>
0.000659440311413
<class 'numpy.float64'>
0.000658058534245
<class 'numpy.float64'>
0.000656682543583
<class 'numpy.float64'>
0.00065531230315
<class 'numpy.float64'>
0.000653947776972
<class 'numpy.float64'>
0.000652588929374
<class 'numpy.float64'>
0.00065123572498
<class 'numpy.float64'>
0.000649888128703
<class 'numpy.float64'>
0.00064854610575
<class 'numpy.float64'>
0.000647209621612
<class 'numpy.float64'>
0.000645878642067
<class 'numpy.float64'>
0.000644553133172
<class 'numpy.float64'>
0.000643233061265
<class 'numpy.float64'>
0.000641918392957
<class 'numpy.float64'>
0.000640609095133
<class 'numpy.float64'>
0.000639305134948
<class 'numpy.float64'>
0.000638006479825
<class 'numpy.float64'>
0.000636713097449
<class 'numpy.float64'>
0.000635424955771
<class 'numpy.float64'>
0.000634142022996
<class 'numpy.float64'>
0.000632864267591
<class 'numpy.float64'>
0.000631591658272
<class 'numpy.float64'>
0.00063032416401
<class 'numpy.float64'>
0.000629061754025
<class 'numpy.float64'>
0.00062780439778
<class 'numpy.float64'>
0.000626552064986
<class 'numpy.float64'>
0.000625304725592
<class 'numpy.float64'>
0.00062406234979
<class 'numpy.float64'>
0.000622824908005
<class 'numpy.float64'>
0.0006215923709
<class 'numpy.float64'>
0.000620364709367
<class 'numpy.float64'>
0.000619141894529
<class 'numpy.float64'>
0.000617923897737
<class 'numpy.float64'>
0.000616710690567
<class 'numpy.float64'>
0.000615502244819
<class 'numpy.float64'>
0.000614298532511
<class 'numpy.float64'>
0.000613099525884
<class 'numpy.float64'>
0.000611905197393
<class 'numpy.float64'>
0.000610715519707
<class 'numpy.float64'>
0.000609530465709
<class 'numpy.float64'>
0.000608350008492
<class 'numpy.float64'>
0.000607174121357
<class 'numpy.float64'>
0.000606002777812
<class 'numpy.float64'>
0.000604835951569
<class 'numpy.float64'>
0.000603673616541
<class 'numpy.float64'>
0.000602515746844
<class 'numpy.float64'>
0.000601362316791
<class 'numpy.float64'>
0.000600213300891
<class 'numpy.float64'>
0.000599068673849
<class 'numpy.float64'>
0.000597928410563
<class 'numpy.float64'>
0.000596792486122
<class 'numpy.float64'>
0.000595660875802
<class 'numpy.float64'>
0.000594533555069
<class 'numpy.float64'>
0.000593410499575
<class 'numpy.float64'>
0.000592291685153
<class 'numpy.float64'>
0.000591177087822
<class 'numpy.float64'>
0.000590066683777
<class 'numpy.float64'>
0.000588960449397
<class 'numpy.float64'>
0.000587858361233
<class 'numpy.float64'>
0.000586760396015
<class 'numpy.float64'>
0.000585666530645
<class 'numpy.float64'>
0.000584576742198
<class 'numpy.float64'>
0.00058349100792
<class 'numpy.float64'>
0.000582409305225
<class 'numpy.float64'>
0.000581331611695
<class 'numpy.float64'>
0.000580257905076
<class 'numpy.float64'>
0.000579188163283
<class 'numpy.float64'>
0.000578122364389
<class 'numpy.float64'>
0.00057706048663
<class 'numpy.float64'>
0.000576002508404
<class 'numpy.float64'>
0.000574948408264
<class 'numpy.float64'>
0.000573898164921
<class 'numpy.float64'>
0.000572851757244
<class 'numpy.float64'>
0.000571809164253
<class 'numpy.float64'>
0.000570770365121
<class 'numpy.float64'>
0.000569735339175
<class 'numpy.float64'>
0.000568704065888
<class 'numpy.float64'>
0.000567676524885
<class 'numpy.float64'>
0.000566652695937
<class 'numpy.float64'>
0.00056563255896
<class 'numpy.float64'>
0.000564616094016
<class 'numpy.float64'>
0.000563603281309
<class 'numpy.float64'>
0.000562594101187
<class 'numpy.float64'>
0.000561588534136
<class 'numpy.float64'>
0.000560586560784
<class 'numpy.float64'>
0.000559588161897
<class 'numpy.float64'>
0.000558593318375
<class 'numpy.float64'>
0.000557602011258
<class 'numpy.float64'>
0.000556614221717
<class 'numpy.float64'>
0.000555629931059
<class 'numpy.float64'>
0.000554649120723
<class 'numpy.float64'>
0.000553671772277
<class 'numpy.float64'>
0.000552697867421
<class 'numpy.float64'>
0.000551727387982
<class 'numpy.float64'>
0.000550760315917
<class 'numpy.float64'>
0.000549796633307
<class 'numpy.float64'>
0.00054883632236
<class 'numpy.float64'>
0.000547879365407
<class 'numpy.float64'>
0.000546925744905
<class 'numpy.float64'>
0.000545975443429
<class 'numpy.float64'>
0.000545028443678
<class 'numpy.float64'>
0.000544084728472
<class 'numpy.float64'>
0.000543144280746
<class 'numpy.float64'>
0.000542207083556
<class 'numpy.float64'>
0.000541273120075
<class 'numpy.float64'>
0.00054034237359
<class 'numpy.float64'>
0.000539414827506
<class 'numpy.float64'>
0.000538490465338
<class 'numpy.float64'>
0.000537569270717
<class 'numpy.float64'>
0.000536651227386
<class 'numpy.float64'>
0.000535736319197
<class 'numpy.float64'>
0.000534824530114
<class 'numpy.float64'>
0.00053391584421
<class 'numpy.float64'>
0.000533010245664
<class 'numpy.float64'>
0.000532107718766
<class 'numpy.float64'>
0.000531208247909
<class 'numpy.float64'>
0.000530311817594
<class 'numpy.float64'>
0.000529418412426
<class 'numpy.float64'>
0.000528528017112
<class 'numpy.float64'>
0.000527640616465
<class 'numpy.float64'>
0.000526756195397
<class 'numpy.float64'>
0.000525874738924
<class 'numpy.float64'>
0.00052499623216
<class 'numpy.float64'>
0.000524120660319
<class 'numpy.float64'>
0.000523248008714
<class 'numpy.float64'>
0.000522378262757
<class 'numpy.float64'>
0.000521511407954
<class 'numpy.float64'>
0.000520647429911
<class 'numpy.float64'>
0.000519786314326
<class 'numpy.float64'>
0.000518928046993
<class 'numpy.float64'>
0.000518072613801
<class 'numpy.float64'>
0.000517220000731
<class 'numpy.float64'>
0.000516370193855
<class 'numpy.float64'>
0.000515523179339
<class 'numpy.float64'>
0.000514678943438
<class 'numpy.float64'>
0.000513837472498
<class 'numpy.float64'>
0.000512998752954
<class 'numpy.float64'>
0.00051216277133
<class 'numpy.float64'>
0.000511329514237
<class 'numpy.float64'>
0.000510498968374
<class 'numpy.float64'>
0.000509671120525
<class 'numpy.float64'>
0.000508845957562
<class 'numpy.float64'>
0.00050802346644
<class 'numpy.float64'>
0.0005072036342
<class 'numpy.float64'>
0.000506386447966
<class 'numpy.float64'>
0.000505571894944
<class 'numpy.float64'>
0.000504759962423
<class 'numpy.float64'>
0.000503950637775
<class 'numpy.float64'>
0.000503143908452
<class 'numpy.float64'>
0.000502339761985
<class 'numpy.float64'>
0.000501538185987
<class 'numpy.float64'>
0.00050073916815
<class 'numpy.float64'>
0.000499942696241
<class 'numpy.float64'>
0.00049914875811
<class 'numpy.float64'>
0.00049835734168
<class 'numpy.float64'>
0.000497568434953
<class 'numpy.float64'>
0.000496782026007
<class 'numpy.float64'>
0.000495998102993
<class 'numpy.float64'>
0.000495216654139
<class 'numpy.float64'>
0.000494437667748
<class 'numpy.float64'>
0.000493661132193
<class 'numpy.float64'>
0.000492887035925
<class 'numpy.float64'>
0.000492115367463
<class 'numpy.float64'>
0.0004913461154
<class 'numpy.float64'>
0.0004905792684
<class 'numpy.float64'>
0.000489814815199
<class 'numpy.float64'>
0.000489052744602
<class 'numpy.float64'>
0.000488293045483
<class 'numpy.float64'>
0.000487535706786
<class 'numpy.float64'>
0.000486780717524
<class 'numpy.float64'>
0.000486028066778
<class 'numpy.float64'>
0.000485277743696
<class 'numpy.float64'>
0.000484529737494
<class 'numpy.float64'>
0.000483784037453
<class 'numpy.float64'>
0.00048304063292
<class 'numpy.float64'>
0.00048229951331
<class 'numpy.float64'>
0.000481560668101
<class 'numpy.float64'>
0.000480824086835
<class 'numpy.float64'>
0.00048008975912
<class 'numpy.float64'>
0.000479357674626
<class 'numpy.float64'>
0.000478627823086
<class 'numpy.float64'>
0.000477900194297
<class 'numpy.float64'>
0.000477174778116
<class 'numpy.float64'>
0.000476451564463
<class 'numpy.float64'>
0.000475730543318
<class 'numpy.float64'>
0.000475011704724
<class 'numpy.float64'>
0.000474295038782
<class 'numpy.float64'>
0.000473580535654
<class 'numpy.float64'>
0.000472868185559
<class 'numpy.float64'>
0.000472157978779
<class 'numpy.float64'>
0.000471449905651
<class 'numpy.float64'>
0.000470743956571
<class 'numpy.float64'>
0.000470040121993
<class 'numpy.float64'>
0.000469338392426
<class 'numpy.float64'>
0.000468638758441
<class 'numpy.float64'>
0.000467941210658
<class 'numpy.float64'>
0.000467245739759
<class 'numpy.float64'>
0.000466552336478
<class 'numpy.float64'>
0.000465860991605
<class 'numpy.float64'>
0.000465171695985
<class 'numpy.float64'>
0.000464484440517
<class 'numpy.float64'>
0.000463799216153
<class 'numpy.float64'>
0.0004631160139
<class 'numpy.float64'>
0.000462434824817
<class 'numpy.float64'>
0.000461755640015
<class 'numpy.float64'>
0.000461078450658
<class 'numpy.float64'>
0.000460403247963
<class 'numpy.float64'>
0.000459730023197
<class 'numpy.float64'>
0.000459058767678
<class 'numpy.float64'>
0.000458389472775
<class 'numpy.float64'>
0.000457722129909
<class 'numpy.float64'>
0.000457056730548
<class 'numpy.float64'>
0.000456393266211
<class 'numpy.float64'>
0.000455731728468
<class 'numpy.float64'>
0.000455072108935
<class 'numpy.float64'>
0.000454414399279
<class 'numpy.float64'>
0.000453758591214
<class 'numpy.float64'>
0.000453104676502
<class 'numpy.float64'>
0.000452452646951
<class 'numpy.float64'>
0.00045180249442
<class 'numpy.float64'>
0.000451154210812
<class 'numpy.float64'>
0.000450507788075
<class 'numpy.float64'>
0.000449863218208
<class 'numpy.float64'>
0.000449220493252
<class 'numpy.float64'>
0.000448579605294
<class 'numpy.float64'>
0.000447940546468
<class 'numpy.float64'>
0.00044730330895
<class 'numpy.float64'>
0.000446667884963
<class 'numpy.float64'>
0.000446034266774
<class 'numpy.float64'>
0.000445402446692
<class 'numpy.float64'>
0.000444772417072
<class 'numpy.float64'>
0.000444144170311
<class 'numpy.float64'>
0.000443517698848
<class 'numpy.float64'>
0.000442892995168
<class 'numpy.float64'>
0.000442270051794
<class 'numpy.float64'>
0.000441648861294
<class 'numpy.float64'>
0.000441029416278
<class 'numpy.float64'>
0.000440411709396
<class 'numpy.float64'>
0.00043979573334
<class 'numpy.float64'>
0.000439181480843
<class 'numpy.float64'>
0.000438568944677
<class 'numpy.float64'>
0.000437958117657
<class 'numpy.float64'>
0.000437348992637
<class 'numpy.float64'>
0.000436741562509
<class 'numpy.float64'>
0.000436135820207
<class 'numpy.float64'>
0.000435531758705
<class 'numpy.float64'>
0.000434929371011
<class 'numpy.float64'>
0.000434328650177
<class 'numpy.float64'>
0.00043372958929
<class 'numpy.float64'>
0.000433132181479
<class 'numpy.float64'>
0.000432536419906
<class 'numpy.float64'>
0.000431942297774
<class 'numpy.float64'>
0.000431349808323
<class 'numpy.float64'>
0.000430758944828
<class 'numpy.float64'>
0.000430169700604
<class 'numpy.float64'>
0.000429582069001
<class 'numpy.float64'>
0.000428996043404
<class 'numpy.float64'>
0.000428411617238
<class 'numpy.float64'>
0.000427828783959
<class 'numpy.float64'>
0.000427247537062
<class 'numpy.float64'>
0.000426667870076
<class 'numpy.float64'>
0.000426089776567
<class 'numpy.float64'>
0.000425513250133
<class 'numpy.float64'>
0.000424938284408
<class 'numpy.float64'>
0.000424364873062
<class 'numpy.float64'>
0.000423793009797
<class 'numpy.float64'>
0.000423222688349
<class 'numpy.float64'>
0.00042265390249
<class 'numpy.float64'>
0.000422086646023
<class 'numpy.float64'>
0.000421520912786
<class 'numpy.float64'>
0.000420956696649
<class 'numpy.float64'>
0.000420393991515
<class 'numpy.float64'>
0.00041983279132
<class 'numpy.float64'>
0.000419273090032
<class 'numpy.float64'>
0.000418714881652
<class 'numpy.float64'>
0.000418158160212
<class 'numpy.float64'>
0.000417602919777
<class 'numpy.float64'>
0.000417049154442
<class 'numpy.float64'>
0.000416496858334
<class 'numpy.float64'>
0.000415946025612
<class 'numpy.float64'>
0.000415396650464
<class 'numpy.float64'>
0.000414848727111
<class 'numpy.float64'>
0.000414302249803
<class 'numpy.float64'>
0.000413757212821
<class 'numpy.float64'>
0.000413213610475
<class 'numpy.float64'>
0.000412671437107
<class 'numpy.float64'>
0.000412130687086
<class 'numpy.float64'>
0.000411591354814
<class 'numpy.float64'>
0.000411053434719
<class 'numpy.float64'>
0.000410516921259
<class 'numpy.float64'>
0.000409981808923
<class 'numpy.float64'>
0.000409448092227
<class 'numpy.float64'>
0.000408915765715
<class 'numpy.float64'>
0.000408384823961
<class 'numpy.float64'>
0.000407855261566
<class 'numpy.float64'>
0.00040732707316
<class 'numpy.float64'>
0.0004068002534
<class 'numpy.float64'>
0.000406274796971
<class 'numpy.float64'>
0.000405750698585
<class 'numpy.float64'>
0.000405227952983
<class 'numpy.float64'>
0.000404706554931
<class 'numpy.float64'>
0.000404186499223
<class 'numpy.float64'>
0.000403667780679
<class 'numpy.float64'>
0.000403150394148
<class 'numpy.float64'>
0.000402634334503
<class 'numpy.float64'>
0.000402119596643
<class 'numpy.float64'>
0.000401606175495
<class 'numpy.float64'>
0.000401094066011
<class 'numpy.float64'>
0.000400583263168
<class 'numpy.float64'>
0.000400073761972
<class 'numpy.float64'>
0.000399565557448
<class 'numpy.float64'>
0.000399058644654
<class 'numpy.float64'>
0.000398553018666
<class 'numpy.float64'>
0.000398048674591
<class 'numpy.float64'>
0.000397545607556
<class 'numpy.float64'>
0.000397043812715
<class 'numpy.float64'>
0.000396543285248
<class 'numpy.float64'>
0.000396044020355
<class 'numpy.float64'>
0.000395546013264
<class 'numpy.float64'>
0.000395049259226
<class 'numpy.float64'>
0.000394553753516
<class 'numpy.float64'>
0.000394059491431
<class 'numpy.float64'>
0.000393566468293
<class 'numpy.float64'>
0.000393074679448
<class 'numpy.float64'>
0.000392584120265
<class 'numpy.float64'>
0.000392094786136
<class 'numpy.float64'>
0.000391606672475
<class 'numpy.float64'>
0.00039111977472
<class 'numpy.float64'>
0.000390634088331
<class 'numpy.float64'>
0.000390149608791
<class 'numpy.float64'>
0.000389666331606
<class 'numpy.float64'>
0.000389184252304
<class 'numpy.float64'>
0.000388703366434
<class 'numpy.float64'>
0.000388223669567
<class 'numpy.float64'>
0.000387745157299
<class 'numpy.float64'>
0.000387267825244
<class 'numpy.float64'>
0.000386791669039
<class 'numpy.float64'>
0.000386316684343
<class 'numpy.float64'>
0.000385842866836
<class 'numpy.float64'>
0.000385370212219
<class 'numpy.float64'>
0.000384898716214
<class 'numpy.float64'>
0.000384428374565
<class 'numpy.float64'>
0.000383959183036
<class 'numpy.float64'>
0.00038349113741
<class 'numpy.float64'>
0.000383024233494
<class 'numpy.float64'>
0.000382558467113
<class 'numpy.float64'>
0.000382093834114
<class 'numpy.float64'>
0.000381630330362
<class 'numpy.float64'>
0.000381167951744
<class 'numpy.float64'>
0.000380706694167
<class 'numpy.float64'>
0.000380246553556
<class 'numpy.float64'>
0.000379787525857
<class 'numpy.float64'>
0.000379329607036
<class 'numpy.float64'>
0.000378872793078
<class 'numpy.float64'>
0.000378417079987
<class 'numpy.float64'>
0.000377962463786
<class 'numpy.float64'>
0.000377508940519
<class 'numpy.float64'>
0.000377056506247
<class 'numpy.float64'>
0.00037660515705
<class 'numpy.float64'>
0.000376154889028
<class 'numpy.float64'>
0.0003757056983
<class 'numpy.float64'>
0.000375257581001
<class 'numpy.float64'>
0.000374810533287
<class 'numpy.float64'>
0.000374364551331
<class 'numpy.float64'>
0.000373919631325
<class 'numpy.float64'>
0.000373475769479
<class 'numpy.float64'>
0.00037303296202
<class 'numpy.float64'>
0.000372591205194
<class 'numpy.float64'>
0.000372150495266
<class 'numpy.float64'>
0.000371710828515
<class 'numpy.float64'>
0.00037127220124
<class 'numpy.float64'>
0.00037083460976
<class 'numpy.float64'>
0.000370398050405
<class 'numpy.float64'>
0.000369962519529
<class 'numpy.float64'>
0.000369528013498
<class 'numpy.float64'>
0.000369094528699
<class 'numpy.float64'>
0.000368662061532
<class 'numpy.float64'>
0.000368230608419
<class 'numpy.float64'>
0.000367800165793
<class 'numpy.float64'>
0.000367370730109
<class 'numpy.float64'>
0.000366942297835
<class 'numpy.float64'>
0.000366514865456
<class 'numpy.float64'>
0.000366088429476
<class 'numpy.float64'>
0.000365662986411
<class 'numpy.float64'>
0.000365238532797
<class 'numpy.float64'>
0.000364815065185
<class 'numpy.float64'>
0.00036439258014
<class 'numpy.float64'>
0.000363971074246
<class 'numpy.float64'>
0.000363550544102
<class 'numpy.float64'>
0.00036313098632
<class 'numpy.float64'>
0.000362712397532
<class 'numpy.float64'>
0.000362294774382
<class 'numpy.float64'>
0.000361878113532
<class 'numpy.float64'>
0.000361462411656
<class 'numpy.float64'>
0.000361047665448
<class 'numpy.float64'>
0.000360633871614
<class 'numpy.float64'>
0.000360221026875
<class 'numpy.float64'>
0.000359809127969
<class 'numpy.float64'>
0.000359398171647
<class 'numpy.float64'>
0.000358988154676
<class 'numpy.float64'>
0.000358579073838
<class 'numpy.float64'>
0.000358170925928
<class 'numpy.float64'>
0.000357763707757
<class 'numpy.float64'>
0.000357357416151
<class 'numpy.float64'>
0.000356952047949
<class 'numpy.float64'>
0.000356547600005
<class 'numpy.float64'>
0.000356144069188
<class 'numpy.float64'>
0.00035574145238
<class 'numpy.float64'>
0.000355339746479
<class 'numpy.float64'>
0.000354938948393
<class 'numpy.float64'>
0.000354539055049
<class 'numpy.float64'>
0.000354140063384
<class 'numpy.float64'>
0.000353741970352
<class 'numpy.float64'>
0.000353344772917
<class 'numpy.float64'>
0.00035294846806
<class 'numpy.float64'>
0.000352553052774
<class 'numpy.float64'>
0.000352158524065
<class 'numpy.float64'>
0.000351764878955
<class 'numpy.float64'>
0.000351372114475
<class 'numpy.float64'>
0.000350980227674
<class 'numpy.float64'>
0.000350589215611
<class 'numpy.float64'>
0.000350199075359
<class 'numpy.float64'>
0.000349809804004
<class 'numpy.float64'>
0.000349421398646
<class 'numpy.float64'>
0.000349033856396
<class 'numpy.float64'>
0.000348647174378
<class 'numpy.float64'>
0.000348261349732
<class 'numpy.float64'>
0.000347876379607
<class 'numpy.float64'>
0.000347492261165
<class 'numpy.float64'>
0.000347108991583
<class 'numpy.float64'>
0.000346726568048
<class 'numpy.float64'>
0.00034634498776
<class 'numpy.float64'>
0.000345964247933
<class 'numpy.float64'>
0.000345584345791
<class 'numpy.float64'>
0.000345205278571
<class 'numpy.float64'>
0.000344827043522
<class 'numpy.float64'>
0.000344449637907
<class 'numpy.float64'>
0.000344073058998
<class 'numpy.float64'>
0.000343697304079
<class 'numpy.float64'>
0.00034332237045
<class 'numpy.float64'>
0.000342948255419
<class 'numpy.float64'>
0.000342574956305
<class 'numpy.float64'>
0.000342202470443
<class 'numpy.float64'>
0.000341830795175
<class 'numpy.float64'>
0.000341459927858
<class 'numpy.float64'>
0.000341089865859
<class 'numpy.float64'>
0.000340720606555
<class 'numpy.float64'>
0.000340352147337
<class 'numpy.float64'>
0.000339984485606
<class 'numpy.float64'>
0.000339617618774
<class 'numpy.float64'>
0.000339251544266
<class 'numpy.float64'>
0.000338886259515
<class 'numpy.float64'>
0.000338521761968
<class 'numpy.float64'>
0.000338158049081
<class 'numpy.float64'>
0.000337795118322
<class 'numpy.float64'>
0.00033743296717
<class 'numpy.float64'>
0.000337071593114
<class 'numpy.float64'>
0.000336710993654
<class 'numpy.float64'>
0.000336351166302
<class 'numpy.float64'>
0.000335992108578
<class 'numpy.float64'>
0.000335633818016
<class 'numpy.float64'>
0.000335276292157
<class 'numpy.float64'>
0.000334919528555
<class 'numpy.float64'>
0.000334563524774
<class 'numpy.float64'>
0.000334208278388
<class 'numpy.float64'>
0.00033385378698
<class 'numpy.float64'>
0.000333500048147
<class 'numpy.float64'>
0.000333147059491
<class 'numpy.float64'>
0.000332794818629
<class 'numpy.float64'>
0.000332443323185
<class 'numpy.float64'>
0.000332092570795
<class 'numpy.float64'>
0.000331742559104
<class 'numpy.float64'>
0.000331393285766
<class 'numpy.float64'>
0.000331044748447
<class 'numpy.float64'>
0.000330696944821
<class 'numpy.float64'>
0.000330349872572
<class 'numpy.float64'>
0.000330003529397
<class 'numpy.float64'>
0.000329657912998
<class 'numpy.float64'>
0.000329313021088
<class 'numpy.float64'>
0.000328968851391
<class 'numpy.float64'>
0.00032862540164
<class 'numpy.float64'>
0.000328282669577
<class 'numpy.float64'>
0.000327940652953
<class 'numpy.float64'>
0.000327599349531
<class 'numpy.float64'>
0.000327258757079
<class 'numpy.float64'>
0.000326918873377
<class 'numpy.float64'>
0.000326579696215
<class 'numpy.float64'>
0.000326241223391
<class 'numpy.float64'>
0.000325903452711
<class 'numpy.float64'>
0.000325566381992
<class 'numpy.float64'>
0.00032523000906
<class 'numpy.float64'>
0.000324894331749
<class 'numpy.float64'>
0.000324559347901
<class 'numpy.float64'>
0.00032422505537
<class 'numpy.float64'>
0.000323891452017
<class 'numpy.float64'>
0.00032355853571
<class 'numpy.float64'>
0.000323226304329
<class 'numpy.float64'>
0.000322894755762
<class 'numpy.float64'>
0.000322563887903
<class 'numpy.float64'>
0.00032223369866
<class 'numpy.float64'>
0.000321904185943
<class 'numpy.float64'>
0.000321575347675
<class 'numpy.float64'>
0.000321247181787
<class 'numpy.float64'>
0.000320919686217
<class 'numpy.float64'>
0.000320592858913
<class 'numpy.float64'>
0.000320266697831
<class 'numpy.float64'>
0.000319941200933
<class 'numpy.float64'>
0.000319616366192
<class 'numpy.float64'>
0.00031929219159
<class 'numpy.float64'>
0.000318968675113
<class 'numpy.float64'>
0.000318645814761
<class 'numpy.float64'>
0.000318323608536
<class 'numpy.float64'>
0.000318002054453
<class 'numpy.float64'>
0.000317681150532
<class 'numpy.float64'>
0.000317360894802
<class 'numpy.float64'>
0.000317041285302
<class 'numpy.float64'>
Cost after iteration 1000: 0.000317
0.000316722320075
<class 'numpy.float64'>
0.000316403997174
<class 'numpy.float64'>
0.00031608631466
<class 'numpy.float64'>
0.000315769270601
<class 'numpy.float64'>
0.000315452863074
<class 'numpy.float64'>
0.000315137090164
<class 'numpy.float64'>
0.00031482194996
<class 'numpy.float64'>
0.000314507440563
<class 'numpy.float64'>
0.00031419356008
<class 'numpy.float64'>
0.000313880306625
<class 'numpy.float64'>
0.00031356767832
<class 'numpy.float64'>
0.000313255673294
<class 'numpy.float64'>
0.000312944289686
<class 'numpy.float64'>
0.000312633525637
<class 'numpy.float64'>
0.000312323379302
<class 'numpy.float64'>
0.000312013848838
<class 'numpy.float64'>
0.000311704932412
<class 'numpy.float64'>
0.000311396628199
<class 'numpy.float64'>
0.000311088934377
<class 'numpy.float64'>
0.000310781849137
<class 'numpy.float64'>
0.000310475370673
<class 'numpy.float64'>
0.000310169497188
<class 'numpy.float64'>
0.00030986422689
<class 'numpy.float64'>
0.000309559557998
<class 'numpy.float64'>
0.000309255488733
<class 'numpy.float64'>
0.000308952017327
<class 'numpy.float64'>
0.000308649142018
<class 'numpy.float64'>
0.00030834686105
<class 'numpy.float64'>
0.000308045172673
<class 'numpy.float64'>
0.000307744075147
<class 'numpy.float64'>
0.000307443566737
<class 'numpy.float64'>
0.000307143645713
<class 'numpy.float64'>
0.000306844310356
<class 'numpy.float64'>
0.000306545558949
<class 'numpy.float64'>
0.000306247389786
<class 'numpy.float64'>
0.000305949801164
<class 'numpy.float64'>
0.000305652791389
<class 'numpy.float64'>
0.000305356358773
<class 'numpy.float64'>
0.000305060501634
<class 'numpy.float64'>
0.000304765218298
<class 'numpy.float64'>
0.000304470507094
<class 'numpy.float64'>
0.000304176366363
<class 'numpy.float64'>
0.000303882794447
<class 'numpy.float64'>
0.000303589789697
<class 'numpy.float64'>
0.000303297350472
<class 'numpy.float64'>
0.000303005475133
<class 'numpy.float64'>
0.000302714162051
<class 'numpy.float64'>
0.000302423409602
<class 'numpy.float64'>
0.000302133216168
<class 'numpy.float64'>
0.000301843580138
<class 'numpy.float64'>
0.000301554499905
<class 'numpy.float64'>
0.000301265973872
<class 'numpy.float64'>
0.000300978000445
<class 'numpy.float64'>
0.000300690578038
<class 'numpy.float64'>
0.000300403705069
<class 'numpy.float64'>
0.000300117379963
<class 'numpy.float64'>
0.000299831601152
<class 'numpy.float64'>
0.000299546367073
<class 'numpy.float64'>
0.000299261676169
<class 'numpy.float64'>
0.00029897752689
<class 'numpy.float64'>
0.000298693917689
<class 'numpy.float64'>
0.000298410847029
<class 'numpy.float64'>
0.000298128313374
<class 'numpy.float64'>
0.0002978463152
<class 'numpy.float64'>
0.000297564850983
<class 'numpy.float64'>
0.000297283919207
<class 'numpy.float64'>
0.000297003518362
<class 'numpy.float64'>
0.000296723646945
<class 'numpy.float64'>
0.000296444303455
<class 'numpy.float64'>
0.000296165486401
<class 'numpy.float64'>
0.000295887194293
<class 'numpy.float64'>
0.000295609425651
<class 'numpy.float64'>
0.000295332178999
<class 'numpy.float64'>
0.000295055452864
<class 'numpy.float64'>
0.000294779245782
<class 'numpy.float64'>
0.000294503556293
<class 'numpy.float64'>
0.000294228382943
<class 'numpy.float64'>
0.000293953724283
<class 'numpy.float64'>
0.000293679578869
<class 'numpy.float64'>
0.000293405945264
<class 'numpy.float64'>
0.000293132822034
<class 'numpy.float64'>
0.000292860207752
<class 'numpy.float64'>
0.000292588100997
<class 'numpy.float64'>
0.000292316500351
<class 'numpy.float64'>
0.000292045404403
<class 'numpy.float64'>
0.000291774811747
<class 'numpy.float64'>
0.000291504720982
<class 'numpy.float64'>
0.000291235130711
<class 'numpy.float64'>
0.000290966039545
<class 'numpy.float64'>
0.000290697446098
<class 'numpy.float64'>
0.000290429348989
<class 'numpy.float64'>
0.000290161746843
<class 'numpy.float64'>
0.00028989463829
<class 'numpy.float64'>
0.000289628021965
<class 'numpy.float64'>
0.000289361896508
<class 'numpy.float64'>
0.000289096260563
<class 'numpy.float64'>
0.000288831112781
<class 'numpy.float64'>
0.000288566451816
<class 'numpy.float64'>
0.000288302276328
<class 'numpy.float64'>
0.000288038584982
<class 'numpy.float64'>
0.000287775376447
<class 'numpy.float64'>
0.000287512649398
<class 'numpy.float64'>
0.000287250402514
<class 'numpy.float64'>
0.000286988634479
<class 'numpy.float64'>
0.000286727343983
<class 'numpy.float64'>
0.000286466529719
<class 'numpy.float64'>
0.000286206190384
<class 'numpy.float64'>
0.000285946324684
<class 'numpy.float64'>
0.000285686931326
<class 'numpy.float64'>
0.000285428009021
<class 'numpy.float64'>
0.000285169556488
<class 'numpy.float64'>
0.000284911572449
<class 'numpy.float64'>
0.000284654055629
<class 'numpy.float64'>
0.000284397004761
<class 'numpy.float64'>
0.00028414041858
<class 'numpy.float64'>
0.000283884295827
<class 'numpy.float64'>
0.000283628635246
<class 'numpy.float64'>
0.000283373435587
<class 'numpy.float64'>
0.000283118695604
<class 'numpy.float64'>
0.000282864414054
<class 'numpy.float64'>
0.000282610589703
<class 'numpy.float64'>
0.000282357221315
<class 'numpy.float64'>
0.000282104307664
<class 'numpy.float64'>
0.000281851847526
<class 'numpy.float64'>
0.000281599839681
<class 'numpy.float64'>
0.000281348282913
<class 'numpy.float64'>
0.000281097176013
<class 'numpy.float64'>
0.000280846517775
<class 'numpy.float64'>
0.000280596306995
<class 'numpy.float64'>
0.000280346542477
<class 'numpy.float64'>
0.000280097223025
<class 'numpy.float64'>
0.000279848347453
<class 'numpy.float64'>
0.000279599914574
<class 'numpy.float64'>
0.000279351923208
<class 'numpy.float64'>
0.000279104372178
<class 'numpy.float64'>
0.000278857260311
<class 'numpy.float64'>
0.00027861058644
<class 'numpy.float64'>
0.0002783643494
<class 'numpy.float64'>
0.000278118548032
<class 'numpy.float64'>
0.000277873181179
<class 'numpy.float64'>
0.000277628247689
<class 'numpy.float64'>
0.000277383746416
<class 'numpy.float64'>
0.000277139676215
<class 'numpy.float64'>
0.000276896035946
<class 'numpy.float64'>
0.000276652824474
<class 'numpy.float64'>
0.000276410040667
<class 'numpy.float64'>
0.000276167683399
<class 'numpy.float64'>
0.000275925751543
<class 'numpy.float64'>
0.000275684243983
<class 'numpy.float64'>
0.0002754431596
<class 'numpy.float64'>
0.000275202497284
<class 'numpy.float64'>
0.000274962255926
<class 'numpy.float64'>
0.000274722434423
<class 'numpy.float64'>
0.000274483031673
<class 'numpy.float64'>
0.000274244046581
<class 'numpy.float64'>
0.000274005478054
<class 'numpy.float64'>
0.000273767325003
<class 'numpy.float64'>
0.000273529586343
<class 'numpy.float64'>
0.000273292260993
<class 'numpy.float64'>
0.000273055347876
<class 'numpy.float64'>
0.000272818845916
<class 'numpy.float64'>
0.000272582754046
<class 'numpy.float64'>
0.000272347071198
<class 'numpy.float64'>
0.000272111796309
<class 'numpy.float64'>
0.000271876928321
<class 'numpy.float64'>
0.000271642466178
<class 'numpy.float64'>
0.00027140840883
<class 'numpy.float64'>
0.000271174755227
<class 'numpy.float64'>
0.000270941504325
<class 'numpy.float64'>
0.000270708655085
<class 'numpy.float64'>
0.000270476206468
<class 'numpy.float64'>
0.00027024415744
<class 'numpy.float64'>
0.000270012506973
<class 'numpy.float64'>
0.000269781254038
<class 'numpy.float64'>
0.000269550397614
<class 'numpy.float64'>
0.000269319936681
<class 'numpy.float64'>
0.000269089870222
<class 'numpy.float64'>
0.000268860197226
<class 'numpy.float64'>
0.000268630916683
<class 'numpy.float64'>
0.000268402027587
<class 'numpy.float64'>
0.000268173528937
<class 'numpy.float64'>
0.000267945419733
<class 'numpy.float64'>
0.00026771769898
<class 'numpy.float64'>
0.000267490365687
<class 'numpy.float64'>
0.000267263418864
<class 'numpy.float64'>
0.000267036857526
<class 'numpy.float64'>
0.000266810680692
<class 'numpy.float64'>
0.000266584887383
<class 'numpy.float64'>
0.000266359476624
<class 'numpy.float64'>
0.000266134447442
<class 'numpy.float64'>
0.00026590979887
<class 'numpy.float64'>
0.000265685529941
<class 'numpy.float64'>
0.000265461639694
<class 'numpy.float64'>
0.00026523812717
<class 'numpy.float64'>
0.000265014991413
<class 'numpy.float64'>
0.00026479223147
<class 'numpy.float64'>
0.000264569846394
<class 'numpy.float64'>
0.000264347835236
<class 'numpy.float64'>
0.000264126197056
<class 'numpy.float64'>
0.000263904930913
<class 'numpy.float64'>
0.00026368403587
<class 'numpy.float64'>
0.000263463510994
<class 'numpy.float64'>
0.000263243355356
<class 'numpy.float64'>
0.000263023568028
<class 'numpy.float64'>
0.000262804148085
<class 'numpy.float64'>
0.000262585094607
<class 'numpy.float64'>
0.000262366406677
<class 'numpy.float64'>
0.000262148083379
<class 'numpy.float64'>
0.000261930123801
<class 'numpy.float64'>
0.000261712527036
<class 'numpy.float64'>
0.000261495292177
<class 'numpy.float64'>
0.000261278418321
<class 'numpy.float64'>
0.00026106190457
<class 'numpy.float64'>
0.000260845750026
<class 'numpy.float64'>
0.000260629953795
<class 'numpy.float64'>
0.000260414514987
<class 'numpy.float64'>
0.000260199432715
<class 'numpy.float64'>
0.000259984706093
<class 'numpy.float64'>
0.000259770334239
<class 'numpy.float64'>
0.000259556316274
<class 'numpy.float64'>
0.000259342651323
<class 'numpy.float64'>
0.000259129338512
<class 'numpy.float64'>
0.000258916376971
<class 'numpy.float64'>
0.000258703765831
<class 'numpy.float64'>
0.00025849150423
<class 'numpy.float64'>
0.000258279591304
<class 'numpy.float64'>
0.000258068026196
<class 'numpy.float64'>
0.000257856808048
<class 'numpy.float64'>
0.000257645936008
<class 'numpy.float64'>
0.000257435409224
<class 'numpy.float64'>
0.00025722522685
<class 'numpy.float64'>
0.000257015388041
<class 'numpy.float64'>
0.000256805891954
<class 'numpy.float64'>
0.000256596737749
<class 'numpy.float64'>
0.000256387924591
<class 'numpy.float64'>
0.000256179451644
<class 'numpy.float64'>
0.000255971318078
<class 'numpy.float64'>
0.000255763523065
<class 'numpy.float64'>
0.000255556065778
<class 'numpy.float64'>
0.000255348945394
<class 'numpy.float64'>
0.000255142161093
<class 'numpy.float64'>
0.000254935712057
<class 'numpy.float64'>
0.000254729597471
<class 'numpy.float64'>
0.000254523816522
<class 'numpy.float64'>
0.0002543183684
<class 'numpy.float64'>
0.000254113252299
<class 'numpy.float64'>
0.000253908467412
<class 'numpy.float64'>
0.000253704012939
<class 'numpy.float64'>
0.00025349988808
<class 'numpy.float64'>
0.000253296092037
<class 'numpy.float64'>
0.000253092624018
<class 'numpy.float64'>
0.000252889483229
<class 'numpy.float64'>
0.000252686668881
<class 'numpy.float64'>
0.000252484180188
<class 'numpy.float64'>
0.000252282016366
<class 'numpy.float64'>
0.000252080176633
<class 'numpy.float64'>
0.000251878660209
<class 'numpy.float64'>
0.000251677466319
<class 'numpy.float64'>
0.000251476594188
<class 'numpy.float64'>
0.000251276043044
<class 'numpy.float64'>
0.000251075812118
<class 'numpy.float64'>
0.000250875900643
<class 'numpy.float64'>
0.000250676307856
<class 'numpy.float64'>
0.000250477032993
<class 'numpy.float64'>
0.000250278075296
<class 'numpy.float64'>
0.000250079434008
<class 'numpy.float64'>
0.000249881108373
<class 'numpy.float64'>
0.00024968309764
<class 'numpy.float64'>
0.000249485401059
<class 'numpy.float64'>
0.000249288017883
<class 'numpy.float64'>
0.000249090947365
<class 'numpy.float64'>
0.000248894188764
<class 'numpy.float64'>
0.00024869774134
<class 'numpy.float64'>
0.000248501604354
<class 'numpy.float64'>
0.000248305777069
<class 'numpy.float64'>
0.000248110258754
<class 'numpy.float64'>
0.000247915048677
<class 'numpy.float64'>
0.000247720146109
<class 'numpy.float64'>
0.000247525550323
<class 'numpy.float64'>
0.000247331260596
<class 'numpy.float64'>
0.000247137276206
<class 'numpy.float64'>
0.000246943596433
<class 'numpy.float64'>
0.000246750220559
<class 'numpy.float64'>
0.000246557147869
<class 'numpy.float64'>
0.000246364377651
<class 'numpy.float64'>
0.000246171909194
<class 'numpy.float64'>
0.00024597974179
<class 'numpy.float64'>
0.000245787874732
<class 'numpy.float64'>
0.000245596307315
<class 'numpy.float64'>
0.00024540503884
<class 'numpy.float64'>
0.000245214068605
<class 'numpy.float64'>
0.000245023395914
<class 'numpy.float64'>
0.00024483302007
<class 'numpy.float64'>
0.000244642940382
<class 'numpy.float64'>
0.000244453156158
<class 'numpy.float64'>
0.00024426366671
<class 'numpy.float64'>
0.00024407447135
<class 'numpy.float64'>
0.000243885569394
<class 'numpy.float64'>
0.00024369696016
<class 'numpy.float64'>
0.000243508642967
<class 'numpy.float64'>
0.000243320617138
<class 'numpy.float64'>
0.000243132881996
<class 'numpy.float64'>
0.000242945436867
<class 'numpy.float64'>
0.000242758281079
<class 'numpy.float64'>
0.000242571413964
<class 'numpy.float64'>
0.000242384834851
<class 'numpy.float64'>
0.000242198543077
<class 'numpy.float64'>
0.000242012537978
<class 'numpy.float64'>
0.000241826818891
<class 'numpy.float64'>
0.000241641385158
<class 'numpy.float64'>
0.00024145623612
<class 'numpy.float64'>
0.000241271371122
<class 'numpy.float64'>
0.000241086789511
<class 'numpy.float64'>
0.000240902490635
<class 'numpy.float64'>
0.000240718473845
<class 'numpy.float64'>
0.000240534738493
<class 'numpy.float64'>
0.000240351283933
<class 'numpy.float64'>
0.000240168109522
<class 'numpy.float64'>
0.000239985214619
<class 'numpy.float64'>
0.000239802598583
<class 'numpy.float64'>
0.000239620260778
<class 'numpy.float64'>
0.000239438200567
<class 'numpy.float64'>
0.000239256417316
<class 'numpy.float64'>
0.000239074910393
<class 'numpy.float64'>
0.00023889367917
<class 'numpy.float64'>
0.000238712723016
<class 'numpy.float64'>
0.000238532041308
<class 'numpy.float64'>
0.000238351633419
<class 'numpy.float64'>
0.000238171498728
<class 'numpy.float64'>
0.000237991636615
<class 'numpy.float64'>
0.00023781204646
<class 'numpy.float64'>
0.000237632727647
<class 'numpy.float64'>
0.000237453679561
<class 'numpy.float64'>
0.00023727490159
<class 'numpy.float64'>
0.000237096393121
<class 'numpy.float64'>
0.000236918153546
<class 'numpy.float64'>
0.000236740182257
<class 'numpy.float64'>
0.000236562478649
<class 'numpy.float64'>
0.000236385042118
<class 'numpy.float64'>
0.000236207872061
<class 'numpy.float64'>
0.000236030967879
<class 'numpy.float64'>
0.000235854328973
<class 'numpy.float64'>
0.000235677954746
<class 'numpy.float64'>
0.000235501844604
<class 'numpy.float64'>
0.000235325997954
<class 'numpy.float64'>
0.000235150414204
<class 'numpy.float64'>
0.000234975092765
<class 'numpy.float64'>
0.000234800033049
<class 'numpy.float64'>
0.000234625234471
<class 'numpy.float64'>
0.000234450696445
<class 'numpy.float64'>
0.00023427641839
<class 'numpy.float64'>
0.000234102399725
<class 'numpy.float64'>
0.00023392863987
<class 'numpy.float64'>
0.000233755138249
<class 'numpy.float64'>
0.000233581894286
<class 'numpy.float64'>
0.000233408907406
<class 'numpy.float64'>
0.000233236177038
<class 'numpy.float64'>
0.000233063702612
<class 'numpy.float64'>
0.000232891483559
<class 'numpy.float64'>
0.000232719519311
<class 'numpy.float64'>
0.000232547809302
<class 'numpy.float64'>
0.000232376352971
<class 'numpy.float64'>
0.000232205149753
<class 'numpy.float64'>
0.000232034199089
<class 'numpy.float64'>
0.00023186350042
<class 'numpy.float64'>
0.000231693053189
<class 'numpy.float64'>
0.000231522856841
<class 'numpy.float64'>
0.000231352910821
<class 'numpy.float64'>
0.000231183214578
<class 'numpy.float64'>
0.00023101376756
<class 'numpy.float64'>
0.000230844569219
<class 'numpy.float64'>
0.000230675619008
<class 'numpy.float64'>
0.00023050691638
<class 'numpy.float64'>
0.000230338460792
<class 'numpy.float64'>
0.000230170251701
<class 'numpy.float64'>
0.000230002288566
<class 'numpy.float64'>
0.000229834570847
<class 'numpy.float64'>
0.000229667098008
<class 'numpy.float64'>
0.00022949986951
<class 'numpy.float64'>
0.000229332884821
<class 'numpy.float64'>
0.000229166143407
<class 'numpy.float64'>
0.000228999644736
<class 'numpy.float64'>
0.000228833388278
<class 'numpy.float64'>
0.000228667373504
<class 'numpy.float64'>
0.000228501599889
<class 'numpy.float64'>
0.000228336066905
<class 'numpy.float64'>
0.000228170774031
<class 'numpy.float64'>
0.000228005720742
<class 'numpy.float64'>
0.000227840906519
<class 'numpy.float64'>
0.000227676330842
<class 'numpy.float64'>
0.000227511993192
<class 'numpy.float64'>
0.000227347893055
<class 'numpy.float64'>
0.000227184029914
<class 'numpy.float64'>
0.000227020403258
<class 'numpy.float64'>
0.000226857012572
<class 'numpy.float64'>
0.000226693857348
<class 'numpy.float64'>
0.000226530937076
<class 'numpy.float64'>
0.00022636825125
<class 'numpy.float64'>
0.000226205799362
<class 'numpy.float64'>
0.000226043580909
<class 'numpy.float64'>
0.000225881595386
<class 'numpy.float64'>
0.000225719842294
<class 'numpy.float64'>
0.000225558321131
<class 'numpy.float64'>
0.000225397031398
<class 'numpy.float64'>
0.000225235972599
<class 'numpy.float64'>
0.000225075144237
<class 'numpy.float64'>
0.000224914545818
<class 'numpy.float64'>
0.000224754176848
<class 'numpy.float64'>
0.000224594036837
<class 'numpy.float64'>
0.000224434125294
<class 'numpy.float64'>
0.00022427444173
<class 'numpy.float64'>
0.000224114985657
<class 'numpy.float64'>
0.000223955756589
<class 'numpy.float64'>
0.000223796754042
<class 'numpy.float64'>
0.000223637977533
<class 'numpy.float64'>
0.000223479426578
<class 'numpy.float64'>
0.000223321100699
<class 'numpy.float64'>
0.000223162999415
<class 'numpy.float64'>
0.000223005122249
<class 'numpy.float64'>
0.000222847468724
<class 'numpy.float64'>
0.000222690038365
<class 'numpy.float64'>
0.000222532830699
<class 'numpy.float64'>
0.000222375845252
<class 'numpy.float64'>
0.000222219081554
<class 'numpy.float64'>
0.000222062539135
<class 'numpy.float64'>
0.000221906217526
<class 'numpy.float64'>
0.00022175011626
<class 'numpy.float64'>
0.000221594234872
<class 'numpy.float64'>
0.000221438572896
<class 'numpy.float64'>
0.00022128312987
<class 'numpy.float64'>
0.000221127905332
<class 'numpy.float64'>
0.000220972898821
<class 'numpy.float64'>
0.000220818109878
<class 'numpy.float64'>
0.000220663538045
<class 'numpy.float64'>
0.000220509182864
<class 'numpy.float64'>
0.000220355043882
<class 'numpy.float64'>
0.000220201120642
<class 'numpy.float64'>
0.000220047412693
<class 'numpy.float64'>
0.000219893919583
<class 'numpy.float64'>
0.000219740640861
<class 'numpy.float64'>
0.000219587576079
<class 'numpy.float64'>
0.000219434724788
<class 'numpy.float64'>
0.000219282086542
<class 'numpy.float64'>
0.000219129660895
<class 'numpy.float64'>
0.000218977447404
<class 'numpy.float64'>
0.000218825445624
<class 'numpy.float64'>
0.000218673655116
<class 'numpy.float64'>
0.000218522075438
<class 'numpy.float64'>
0.000218370706151
<class 'numpy.float64'>
0.000218219546817
<class 'numpy.float64'>
0.000218068596999
<class 'numpy.float64'>
0.000217917856262
<class 'numpy.float64'>
0.000217767324171
<class 'numpy.float64'>
0.000217617000294
<class 'numpy.float64'>
0.000217466884198
<class 'numpy.float64'>
0.000217316975452
<class 'numpy.float64'>
0.000217167273627
<class 'numpy.float64'>
0.000217017778294
<class 'numpy.float64'>
0.000216868489027
<class 'numpy.float64'>
0.000216719405399
<class 'numpy.float64'>
0.000216570526985
<class 'numpy.float64'>
0.000216421853362
<class 'numpy.float64'>
0.000216273384107
<class 'numpy.float64'>
0.000216125118799
<class 'numpy.float64'>
0.000215977057018
<class 'numpy.float64'>
0.000215829198344
<class 'numpy.float64'>
0.00021568154236
<class 'numpy.float64'>
0.000215534088648
<class 'numpy.float64'>
0.000215386836794
<class 'numpy.float64'>
0.000215239786383
<class 'numpy.float64'>
0.000215092937001
<class 'numpy.float64'>
0.000214946288237
<class 'numpy.float64'>
0.000214799839678
<class 'numpy.float64'>
0.000214653590916
<class 'numpy.float64'>
0.000214507541541
<class 'numpy.float64'>
0.000214361691146
<class 'numpy.float64'>
0.000214216039324
<class 'numpy.float64'>
0.000214070585669
<class 'numpy.float64'>
0.000213925329778
<class 'numpy.float64'>
0.000213780271246
<class 'numpy.float64'>
0.000213635409672
<class 'numpy.float64'>
0.000213490744654
<class 'numpy.float64'>
0.000213346275793
<class 'numpy.float64'>
0.000213202002689
<class 'numpy.float64'>
0.000213057924946
<class 'numpy.float64'>
0.000212914042165
<class 'numpy.float64'>
0.000212770353951
<class 'numpy.float64'>
0.000212626859909
<class 'numpy.float64'>
0.000212483559647
<class 'numpy.float64'>
0.000212340452771
<class 'numpy.float64'>
0.00021219753889
<class 'numpy.float64'>
0.000212054817614
<class 'numpy.float64'>
0.000211912288553
<class 'numpy.float64'>
0.000211769951319
<class 'numpy.float64'>
0.000211627805525
<class 'numpy.float64'>
0.000211485850784
<class 'numpy.float64'>
0.000211344086712
<class 'numpy.float64'>
0.000211202512924
<class 'numpy.float64'>
0.000211061129037
<class 'numpy.float64'>
0.000210919934669
<class 'numpy.float64'>
0.000210778929439
<class 'numpy.float64'>
0.000210638112968
<class 'numpy.float64'>
0.000210497484875
<class 'numpy.float64'>
0.000210357044783
<class 'numpy.float64'>
0.000210216792315
<class 'numpy.float64'>
0.000210076727095
<class 'numpy.float64'>
0.000209936848749
<class 'numpy.float64'>
0.000209797156901
<class 'numpy.float64'>
0.00020965765118
<class 'numpy.float64'>
0.000209518331213
<class 'numpy.float64'>
0.000209379196628
<class 'numpy.float64'>
0.000209240247058
<class 'numpy.float64'>
0.000209101482131
<class 'numpy.float64'>
0.00020896290148
<class 'numpy.float64'>
0.000208824504739
<class 'numpy.float64'>
0.00020868629154
<class 'numpy.float64'>
0.000208548261519
<class 'numpy.float64'>
0.000208410414312
<class 'numpy.float64'>
0.000208272749556
<class 'numpy.float64'>
0.000208135266888
<class 'numpy.float64'>
0.000207997965947
<class 'numpy.float64'>
0.000207860846373
<class 'numpy.float64'>
0.000207723907806
<class 'numpy.float64'>
0.000207587149888
<class 'numpy.float64'>
0.000207450572262
<class 'numpy.float64'>
0.000207314174571
<class 'numpy.float64'>
0.000207177956459
<class 'numpy.float64'>
0.000207041917572
<class 'numpy.float64'>
0.000206906057556
<class 'numpy.float64'>
0.000206770376059
<class 'numpy.float64'>
0.000206634872728
<class 'numpy.float64'>
0.000206499547213
<class 'numpy.float64'>
0.000206364399163
<class 'numpy.float64'>
0.00020622942823
<class 'numpy.float64'>
0.000206094634066
<class 'numpy.float64'>
0.000205960016322
<class 'numpy.float64'>
0.000205825574653
<class 'numpy.float64'>
0.000205691308713
<class 'numpy.float64'>
0.000205557218158
<class 'numpy.float64'>
0.000205423302644
<class 'numpy.float64'>
0.000205289561829
<class 'numpy.float64'>
0.00020515599537
<class 'numpy.float64'>
0.000205022602927
<class 'numpy.float64'>
0.000204889384159
<class 'numpy.float64'>
0.000204756338728
<class 'numpy.float64'>
0.000204623466295
<class 'numpy.float64'>
0.000204490766522
<class 'numpy.float64'>
0.000204358239073
<class 'numpy.float64'>
0.000204225883613
<class 'numpy.float64'>
0.000204093699807
<class 'numpy.float64'>
0.00020396168732
<class 'numpy.float64'>
0.00020382984582
<class 'numpy.float64'>
0.000203698174975
<class 'numpy.float64'>
0.000203566674452
<class 'numpy.float64'>
0.000203435343922
<class 'numpy.float64'>
0.000203304183056
<class 'numpy.float64'>
0.000203173191523
<class 'numpy.float64'>
0.000203042368997
<class 'numpy.float64'>
0.00020291171515
<class 'numpy.float64'>
0.000202781229656
<class 'numpy.float64'>
0.00020265091219
<class 'numpy.float64'>
0.000202520762427
<class 'numpy.float64'>
0.000202390780043
<class 'numpy.float64'>
0.000202260964716
<class 'numpy.float64'>
0.000202131316123
<class 'numpy.float64'>
0.000202001833943
<class 'numpy.float64'>
0.000201872517857
<class 'numpy.float64'>
0.000201743367543
<class 'numpy.float64'>
0.000201614382684
<class 'numpy.float64'>
0.000201485562961
<class 'numpy.float64'>
0.000201356908058
<class 'numpy.float64'>
0.000201228417658
<class 'numpy.float64'>
0.000201100091445
<class 'numpy.float64'>
0.000200971929106
<class 'numpy.float64'>
0.000200843930325
<class 'numpy.float64'>
0.00020071609479
<class 'numpy.float64'>
0.000200588422189
<class 'numpy.float64'>
0.00020046091221
<class 'numpy.float64'>
0.000200333564542
<class 'numpy.float64'>
0.000200206378876
<class 'numpy.float64'>
0.000200079354902
<class 'numpy.float64'>
0.000199952492312
<class 'numpy.float64'>
0.000199825790798
<class 'numpy.float64'>
0.000199699250054
<class 'numpy.float64'>
0.000199572869774
<class 'numpy.float64'>
0.000199446649652
<class 'numpy.float64'>
0.000199320589385
<class 'numpy.float64'>
0.000199194688667
<class 'numpy.float64'>
0.000199068947197
<class 'numpy.float64'>
0.000198943364672
<class 'numpy.float64'>
0.000198817940791
<class 'numpy.float64'>
0.000198692675254
<class 'numpy.float64'>
0.00019856756776
<class 'numpy.float64'>
0.00019844261801
<class 'numpy.float64'>
0.000198317825706
<class 'numpy.float64'>
0.00019819319055
<class 'numpy.float64'>
0.000198068712246
<class 'numpy.float64'>
0.000197944390498
<class 'numpy.float64'>
0.000197820225009
<class 'numpy.float64'>
0.000197696215486
<class 'numpy.float64'>
0.000197572361635
<class 'numpy.float64'>
0.000197448663162
<class 'numpy.float64'>
0.000197325119776
<class 'numpy.float64'>
0.000197201731184
<class 'numpy.float64'>
0.000197078497095
<class 'numpy.float64'>
0.000196955417219
<class 'numpy.float64'>
0.000196832491268
<class 'numpy.float64'>
0.000196709718951
<class 'numpy.float64'>
0.000196587099982
<class 'numpy.float64'>
0.000196464634072
<class 'numpy.float64'>
0.000196342320936
<class 'numpy.float64'>
0.000196220160287
<class 'numpy.float64'>
0.00019609815184
<class 'numpy.float64'>
0.00019597629531
<class 'numpy.float64'>
0.000195854590415
<class 'numpy.float64'>
0.00019573303687
<class 'numpy.float64'>
0.000195611634395
<class 'numpy.float64'>
0.000195490382706
<class 'numpy.float64'>
0.000195369281523
<class 'numpy.float64'>
0.000195248330566
<class 'numpy.float64'>
0.000195127529555
<class 'numpy.float64'>
0.000195006878211
<class 'numpy.float64'>
0.000194886376257
<class 'numpy.float64'>
0.000194766023415
<class 'numpy.float64'>
0.000194645819407
<class 'numpy.float64'>
0.000194525763959
<class 'numpy.float64'>
0.000194405856793
<class 'numpy.float64'>
0.000194286097637
<class 'numpy.float64'>
0.000194166486216
<class 'numpy.float64'>
0.000194047022256
<class 'numpy.float64'>
0.000193927705484
<class 'numpy.float64'>
0.000193808535629
<class 'numpy.float64'>
0.00019368951242
<class 'numpy.float64'>
0.000193570635584
<class 'numpy.float64'>
0.000193451904854
<class 'numpy.float64'>
0.000193333319959
<class 'numpy.float64'>
0.00019321488063
<class 'numpy.float64'>
0.0001930965866
<class 'numpy.float64'>
0.000192978437601
<class 'numpy.float64'>
0.000192860433366
<class 'numpy.float64'>
0.000192742573629
<class 'numpy.float64'>
0.000192624858125
<class 'numpy.float64'>
0.00019250728659
<class 'numpy.float64'>
0.000192389858758
<class 'numpy.float64'>
0.000192272574367
<class 'numpy.float64'>
0.000192155433154
<class 'numpy.float64'>
0.000192038434856
<class 'numpy.float64'>
0.000191921579212
<class 'numpy.float64'>
0.000191804865961
<class 'numpy.float64'>
0.000191688294843
<class 'numpy.float64'>
0.000191571865598
<class 'numpy.float64'>
0.000191455577967
<class 'numpy.float64'>
0.000191339431692
<class 'numpy.float64'>
0.000191223426514
<class 'numpy.float64'>
0.000191107562178
<class 'numpy.float64'>
0.000190991838426
<class 'numpy.float64'>
0.000190876255002
<class 'numpy.float64'>
0.000190760811652
<class 'numpy.float64'>
0.00019064550812
<class 'numpy.float64'>
0.000190530344153
<class 'numpy.float64'>
0.000190415319497
<class 'numpy.float64'>
0.000190300433899
<class 'numpy.float64'>
0.000190185687108
<class 'numpy.float64'>
0.000190071078871
<class 'numpy.float64'>
0.000189956608938
<class 'numpy.float64'>
0.000189842277058
<class 'numpy.float64'>
0.000189728082982
<class 'numpy.float64'>
0.00018961402646
<class 'numpy.float64'>
0.000189500107244
<class 'numpy.float64'>
0.000189386325086
<class 'numpy.float64'>
0.000189272679738
<class 'numpy.float64'>
0.000189159170954
<class 'numpy.float64'>
0.000189045798488
<class 'numpy.float64'>
0.000188932562094
<class 'numpy.float64'>
0.000188819461527
<class 'numpy.float64'>
0.000188706496542
<class 'numpy.float64'>
0.000188593666897
<class 'numpy.float64'>
0.000188480972347
<class 'numpy.float64'>
0.00018836841265
<class 'numpy.float64'>
0.000188255987564
<class 'numpy.float64'>
0.000188143696847
<class 'numpy.float64'>
0.000188031540259
<class 'numpy.float64'>
0.000187919517559
<class 'numpy.float64'>
0.000187807628507
<class 'numpy.float64'>
0.000187695872865
<class 'numpy.float64'>
0.000187584250394
<class 'numpy.float64'>
0.000187472760854
<class 'numpy.float64'>
0.000187361404011
<class 'numpy.float64'>
0.000187250179625
<class 'numpy.float64'>
0.000187139087461
<class 'numpy.float64'>
0.000187028127283
<class 'numpy.float64'>
0.000186917298856
<class 'numpy.float64'>
0.000186806601946
<class 'numpy.float64'>
0.000186696036317
<class 'numpy.float64'>
0.000186585601737
<class 'numpy.float64'>
0.000186475297972
<class 'numpy.float64'>
0.000186365124791
<class 'numpy.float64'>
0.00018625508196
<class 'numpy.float64'>
0.00018614516925
<class 'numpy.float64'>
0.000186035386428
<class 'numpy.float64'>
0.000185925733265
<class 'numpy.float64'>
0.000185816209532
<class 'numpy.float64'>
0.000185706814998
<class 'numpy.float64'>
0.000185597549435
<class 'numpy.float64'>
0.000185488412616
<class 'numpy.float64'>
0.000185379404312
<class 'numpy.float64'>
0.000185270524297
<class 'numpy.float64'>
0.000185161772343
<class 'numpy.float64'>
0.000185053148226
<class 'numpy.float64'>
0.00018494465172
<class 'numpy.float64'>
0.0001848362826
<class 'numpy.float64'>
0.000184728040641
<class 'numpy.float64'>
0.000184619925619
<class 'numpy.float64'>
0.000184511937312
<class 'numpy.float64'>
0.000184404075497
<class 'numpy.float64'>
0.000184296339951
<class 'numpy.float64'>
0.000184188730453
<class 'numpy.float64'>
0.000184081246781
<class 'numpy.float64'>
0.000183973888714
<class 'numpy.float64'>
0.000183866656033
<class 'numpy.float64'>
0.000183759548518
<class 'numpy.float64'>
0.000183652565949
<class 'numpy.float64'>
0.000183545708109
<class 'numpy.float64'>
0.000183438974778
<class 'numpy.float64'>
0.00018333236574
<class 'numpy.float64'>
0.000183225880776
<class 'numpy.float64'>
0.000183119519672
<class 'numpy.float64'>
0.000183013282209
<class 'numpy.float64'>
0.000182907168173
<class 'numpy.float64'>
0.000182801177349
<class 'numpy.float64'>
0.000182695309521
<class 'numpy.float64'>
0.000182589564477
<class 'numpy.float64'>
0.000182483942002
<class 'numpy.float64'>
0.000182378441883
<class 'numpy.float64'>
0.000182273063908
<class 'numpy.float64'>
0.000182167807864
<class 'numpy.float64'>
0.00018206267354
<class 'numpy.float64'>
0.000181957660725
<class 'numpy.float64'>
0.000181852769207
<class 'numpy.float64'>
0.000181747998777
<class 'numpy.float64'>
0.000181643349226
<class 'numpy.float64'>
0.000181538820343
<class 'numpy.float64'>
0.000181434411921
<class 'numpy.float64'>
0.00018133012375
<class 'numpy.float64'>
0.000181225955623
<class 'numpy.float64'>
0.000181121907334
<class 'numpy.float64'>
0.000181017978674
<class 'numpy.float64'>
0.000180914169438
<class 'numpy.float64'>
0.00018081047942
<class 'numpy.float64'>
0.000180706908415
<class 'numpy.float64'>
0.000180603456217
<class 'numpy.float64'>
0.000180500122622
<class 'numpy.float64'>
0.000180396907426
<class 'numpy.float64'>
0.000180293810426
<class 'numpy.float64'>
0.000180190831418
<class 'numpy.float64'>
0.000180087970201
<class 'numpy.float64'>
0.000179985226571
<class 'numpy.float64'>
0.000179882600328
<class 'numpy.float64'>
0.00017978009127
<class 'numpy.float64'>
0.000179677699196
<class 'numpy.float64'>
0.000179575423906
<class 'numpy.float64'>
0.000179473265201
<class 'numpy.float64'>
0.00017937122288
<class 'numpy.float64'>
0.000179269296745
<class 'numpy.float64'>
0.000179167486598
<class 'numpy.float64'>
0.00017906579224
<class 'numpy.float64'>
0.000178964213475
<class 'numpy.float64'>
0.000178862750104
<class 'numpy.float64'>
0.000178761401932
<class 'numpy.float64'>
0.000178660168761
<class 'numpy.float64'>
0.000178559050397
<class 'numpy.float64'>
0.000178458046644
<class 'numpy.float64'>
0.000178357157307
<class 'numpy.float64'>
0.000178256382192
<class 'numpy.float64'>
0.000178155721104
<class 'numpy.float64'>
0.00017805517385
<class 'numpy.float64'>
0.000177954740238
<class 'numpy.float64'>
0.000177854420073
<class 'numpy.float64'>
0.000177754213165
<class 'numpy.float64'>
0.000177654119321
<class 'numpy.float64'>
0.00017755413835
<class 'numpy.float64'>
0.000177454270061
<class 'numpy.float64'>
0.000177354514264
<class 'numpy.float64'>
0.000177254870768
<class 'numpy.float64'>
0.000177155339383
<class 'numpy.float64'>
0.000177055919921
<class 'numpy.float64'>
0.000176956612193
<class 'numpy.float64'>
0.00017685741601
<class 'numpy.float64'>
0.000176758331184
<class 'numpy.float64'>
0.000176659357529
<class 'numpy.float64'>
0.000176560494856
<class 'numpy.float64'>
0.000176461742979
<class 'numpy.float64'>
0.000176363101712
<class 'numpy.float64'>
0.000176264570868
<class 'numpy.float64'>
0.000176166150263
<class 'numpy.float64'>
0.000176067839712
<class 'numpy.float64'>
0.00017596963903
<class 'numpy.float64'>
0.000175871548032
<class 'numpy.float64'>
0.000175773566535
<class 'numpy.float64'>
0.000175675694355
<class 'numpy.float64'>
0.00017557793131
<class 'numpy.float64'>
0.000175480277217
<class 'numpy.float64'>
0.000175382731894
<class 'numpy.float64'>
0.000175285295159
<class 'numpy.float64'>
0.000175187966831
<class 'numpy.float64'>
0.000175090746728
<class 'numpy.float64'>
0.000174993634671
<class 'numpy.float64'>
0.000174896630479
<class 'numpy.float64'>
0.000174799733972
<class 'numpy.float64'>
0.000174702944971
<class 'numpy.float64'>
0.000174606263298
<class 'numpy.float64'>
0.000174509688773
<class 'numpy.float64'>
0.000174413221218
<class 'numpy.float64'>
0.000174316860456
<class 'numpy.float64'>
0.00017422060631
<class 'numpy.float64'>
0.000174124458602
<class 'numpy.float64'>
0.000174028417156
<class 'numpy.float64'>
0.000173932481796
<class 'numpy.float64'>
0.000173836652345
<class 'numpy.float64'>
0.000173740928629
<class 'numpy.float64'>
0.000173645310472
<class 'numpy.float64'>
0.0001735497977
<class 'numpy.float64'>
0.000173454390139
<class 'numpy.float64'>
0.000173359087615
<class 'numpy.float64'>
0.000173263889953
<class 'numpy.float64'>
0.000173168796982
<class 'numpy.float64'>
0.000173073808529
<class 'numpy.float64'>
0.00017297892442
<class 'numpy.float64'>
0.000172884144485
<class 'numpy.float64'>
0.000172789468551
<class 'numpy.float64'>
0.000172694896448
<class 'numpy.float64'>
0.000172600428004
<class 'numpy.float64'>
0.00017250606305
<class 'numpy.float64'>
0.000172411801414
<class 'numpy.float64'>
0.000172317642928
<class 'numpy.float64'>
0.000172223587422
<class 'numpy.float64'>
0.000172129634727
<class 'numpy.float64'>
0.000172035784674
<class 'numpy.float64'>
0.000171942037096
<class 'numpy.float64'>
0.000171848391824
<class 'numpy.float64'>
0.000171754848691
<class 'numpy.float64'>
0.00017166140753
<class 'numpy.float64'>
0.000171568068173
<class 'numpy.float64'>
0.000171474830456
<class 'numpy.float64'>
0.000171381694211
<class 'numpy.float64'>
0.000171288659273
<class 'numpy.float64'>
0.000171195725476
<class 'numpy.float64'>
0.000171102892657
<class 'numpy.float64'>
0.000171010160649
<class 'numpy.float64'>
0.000170917529289
<class 'numpy.float64'>
0.000170824998413
<class 'numpy.float64'>
0.000170732567858
<class 'numpy.float64'>
0.00017064023746
<class 'numpy.float64'>
0.000170548007056
<class 'numpy.float64'>
0.000170455876484
<class 'numpy.float64'>
0.000170363845582
<class 'numpy.float64'>
0.000170271914189
<class 'numpy.float64'>
0.000170180082142
<class 'numpy.float64'>
0.000170088349281
<class 'numpy.float64'>
0.000169996715444
<class 'numpy.float64'>
0.000169905180473
<class 'numpy.float64'>
0.000169813744206
<class 'numpy.float64'>
0.000169722406483
<class 'numpy.float64'>
0.000169631167147
<class 'numpy.float64'>
0.000169540026037
<class 'numpy.float64'>
0.000169448982995
<class 'numpy.float64'>
0.000169358037863
<class 'numpy.float64'>
0.000169267190482
<class 'numpy.float64'>
0.000169176440695
<class 'numpy.float64'>
0.000169085788345
<class 'numpy.float64'>
0.000168995233275
<class 'numpy.float64'>
0.000168904775329
<class 'numpy.float64'>
0.000168814414349
<class 'numpy.float64'>
0.00016872415018
<class 'numpy.float64'>
0.000168633982667
<class 'numpy.float64'>
0.000168543911654
<class 'numpy.float64'>
0.000168453936986
<class 'numpy.float64'>
0.000168364058509
<class 'numpy.float64'>
0.000168274276068
<class 'numpy.float64'>
0.000168184589509
<class 'numpy.float64'>
0.000168094998679
<class 'numpy.float64'>
0.000168005503426
<class 'numpy.float64'>
0.000167916103594
<class 'numpy.float64'>
0.000167826799032
<class 'numpy.float64'>
0.000167737589588
<class 'numpy.float64'>
0.00016764847511
<class 'numpy.float64'>
0.000167559455445
<class 'numpy.float64'>
0.000167470530444
<class 'numpy.float64'>
0.000167381699954
<class 'numpy.float64'>
0.000167292963824
<class 'numpy.float64'>
0.000167204321906
<class 'numpy.float64'>
0.000167115774048
<class 'numpy.float64'>
0.0001670273201
<class 'numpy.float64'>
0.000166938959914
<class 'numpy.float64'>
0.00016685069334
<class 'numpy.float64'>
0.00016676252023
<class 'numpy.float64'>
0.000166674440435
<class 'numpy.float64'>
0.000166586453807
<class 'numpy.float64'>
0.000166498560197
<class 'numpy.float64'>
0.000166410759459
<class 'numpy.float64'>
0.000166323051446
<class 'numpy.float64'>
0.000166235436009
<class 'numpy.float64'>
0.000166147913004
<class 'numpy.float64'>
0.000166060482283
<class 'numpy.float64'>
0.0001659731437
<class 'numpy.float64'>
0.000165885897111
<class 'numpy.float64'>
0.000165798742369
<class 'numpy.float64'>
0.000165711679329
<class 'numpy.float64'>
0.000165624707847
<class 'numpy.float64'>
0.000165537827778
<class 'numpy.float64'>
0.000165451038978
<class 'numpy.float64'>
0.000165364341304
<class 'numpy.float64'>
0.000165277734611
<class 'numpy.float64'>
0.000165191218756
<class 'numpy.float64'>
0.000165104793598
<class 'numpy.float64'>
0.000165018458992
<class 'numpy.float64'>
0.000164932214796
<class 'numpy.float64'>
0.00016484606087
<class 'numpy.float64'>
0.00016475999707
<class 'numpy.float64'>
0.000164674023255
<class 'numpy.float64'>
0.000164588139285
<class 'numpy.float64'>
0.000164502345017
<class 'numpy.float64'>
0.000164416640313
<class 'numpy.float64'>
0.000164331025032
<class 'numpy.float64'>
0.000164245499032
<class 'numpy.float64'>
0.000164160062176
<class 'numpy.float64'>
0.000164074714323
<class 'numpy.float64'>
0.000163989455335
<class 'numpy.float64'>
0.000163904285072
<class 'numpy.float64'>
0.000163819203396
<class 'numpy.float64'>
0.000163734210169
<class 'numpy.float64'>
0.000163649305253
<class 'numpy.float64'>
0.00016356448851
<class 'numpy.float64'>
0.000163479759803
<class 'numpy.float64'>
0.000163395118995
<class 'numpy.float64'>
0.000163310565949
<class 'numpy.float64'>
0.000163226100528
<class 'numpy.float64'>
0.000163141722596
<class 'numpy.float64'>
0.000163057432018
<class 'numpy.float64'>
0.000162973228657
<class 'numpy.float64'>
0.000162889112379
<class 'numpy.float64'>
0.000162805083048
<class 'numpy.float64'>
0.000162721140528
<class 'numpy.float64'>
0.000162637284687
<class 'numpy.float64'>
0.000162553515389
<class 'numpy.float64'>
0.000162469832501
<class 'numpy.float64'>
0.000162386235888
<class 'numpy.float64'>
0.000162302725418
<class 'numpy.float64'>
0.000162219300957
<class 'numpy.float64'>
0.000162135962372
<class 'numpy.float64'>
0.000162052709531
<class 'numpy.float64'>
0.000161969542301
<class 'numpy.float64'>
0.00016188646055
<class 'numpy.float64'>
0.000161803464146
<class 'numpy.float64'>
0.000161720552959
<class 'numpy.float64'>
0.000161637726856
<class 'numpy.float64'>
0.000161554985707
<class 'numpy.float64'>
0.00016147232938
<class 'numpy.float64'>
0.000161389757746
<class 'numpy.float64'>
0.000161307270674
<class 'numpy.float64'>
0.000161224868035
<class 'numpy.float64'>
0.000161142549698
<class 'numpy.float64'>
0.000161060315535
<class 'numpy.float64'>
0.000160978165416
<class 'numpy.float64'>
0.000160896099211
<class 'numpy.float64'>
0.000160814116794
<class 'numpy.float64'>
0.000160732218035
<class 'numpy.float64'>
0.000160650402807
<class 'numpy.float64'>
0.000160568670981
<class 'numpy.float64'>
0.00016048702243
<class 'numpy.float64'>
0.000160405457026
<class 'numpy.float64'>
0.000160323974643
<class 'numpy.float64'>
0.000160242575154
<class 'numpy.float64'>
0.000160161258433
<class 'numpy.float64'>
0.000160080024352
<class 'numpy.float64'>
0.000159998872786
<class 'numpy.float64'>
0.00015991780361
<class 'numpy.float64'>
0.000159836816698
<class 'numpy.float64'>
0.000159755911924
<class 'numpy.float64'>
0.000159675089164
<class 'numpy.float64'>
0.000159594348293
<class 'numpy.float64'>
0.000159513689186
<class 'numpy.float64'>
0.000159433111719
<class 'numpy.float64'>
0.000159352615769
<class 'numpy.float64'>
0.000159272201211
<class 'numpy.float64'>
0.000159191867922
<class 'numpy.float64'>
0.00015911161578
<class 'numpy.float64'>
0.00015903144466
<class 'numpy.float64'>
0.00015895135444
<class 'numpy.float64'>
0.000158871344998
<class 'numpy.float64'>
0.000158791416211
<class 'numpy.float64'>
0.000158711567958
<class 'numpy.float64'>
0.000158631800116
<class 'numpy.float64'>
0.000158552112565
<class 'numpy.float64'>
0.000158472505183
<class 'numpy.float64'>
0.000158392977848
<class 'numpy.float64'>
0.000158313530442
<class 'numpy.float64'>
0.000158234162841
<class 'numpy.float64'>
0.000158154874928
<class 'numpy.float64'>
0.00015807566658
<class 'numpy.float64'>
Cost after iteration 2000: 0.000158
0.00015799653768
<class 'numpy.float64'>
0.000157917488106
<class 'numpy.float64'>
0.00015783851774
<class 'numpy.float64'>
0.000157759626463
<class 'numpy.float64'>
0.000157680814156
<class 'numpy.float64'>
0.0001576020807
<class 'numpy.float64'>
0.000157523425978
<class 'numpy.float64'>
0.000157444849869
<class 'numpy.float64'>
0.000157366352258
<class 'numpy.float64'>
0.000157287933027
<class 'numpy.float64'>
0.000157209592057
<class 'numpy.float64'>
0.000157131329232
<class 'numpy.float64'>
0.000157053144434
<class 'numpy.float64'>
0.000156975037548
<class 'numpy.float64'>
0.000156897008456
<class 'numpy.float64'>
0.000156819057043
<class 'numpy.float64'>
0.000156741183192
<class 'numpy.float64'>
0.000156663386787
<class 'numpy.float64'>
0.000156585667714
<class 'numpy.float64'>
0.000156508025856
<class 'numpy.float64'>
0.000156430461099
<class 'numpy.float64'>
0.000156352973328
<class 'numpy.float64'>
0.000156275562427
<class 'numpy.float64'>
0.000156198228283
<class 'numpy.float64'>
0.000156120970782
<class 'numpy.float64'>
0.000156043789809
<class 'numpy.float64'>
0.000155966685252
<class 'numpy.float64'>
0.000155889656995
<class 'numpy.float64'>
0.000155812704927
<class 'numpy.float64'>
0.000155735828933
<class 'numpy.float64'>
0.000155659028902
<class 'numpy.float64'>
0.00015558230472
<class 'numpy.float64'>
0.000155505656275
<class 'numpy.float64'>
0.000155429083456
<class 'numpy.float64'>
0.000155352586149
<class 'numpy.float64'>
0.000155276164244
<class 'numpy.float64'>
0.000155199817628
<class 'numpy.float64'>
0.000155123546191
<class 'numpy.float64'>
0.000155047349822
<class 'numpy.float64'>
0.000154971228409
<class 'numpy.float64'>
0.000154895181842
<class 'numpy.float64'>
0.000154819210011
<class 'numpy.float64'>
0.000154743312805
<class 'numpy.float64'>
0.000154667490115
<class 'numpy.float64'>
0.000154591741831
<class 'numpy.float64'>
0.000154516067842
<class 'numpy.float64'>
0.000154440468041
<class 'numpy.float64'>
0.000154364942317
<class 'numpy.float64'>
0.000154289490562
<class 'numpy.float64'>
0.000154214112667
<class 'numpy.float64'>
0.000154138808523
<class 'numpy.float64'>
0.000154063578023
<class 'numpy.float64'>
0.000153988421059
<class 'numpy.float64'>
0.000153913337522
<class 'numpy.float64'>
0.000153838327305
<class 'numpy.float64'>
0.0001537633903
<class 'numpy.float64'>
0.000153688526401
<class 'numpy.float64'>
0.0001536137355
<class 'numpy.float64'>
0.00015353901749
<class 'numpy.float64'>
0.000153464372265
<class 'numpy.float64'>
0.000153389799719
<class 'numpy.float64'>
0.000153315299745
<class 'numpy.float64'>
0.000153240872238
<class 'numpy.float64'>
0.000153166517091
<class 'numpy.float64'>
0.000153092234199
<class 'numpy.float64'>
0.000153018023457
<class 'numpy.float64'>
0.00015294388476
<class 'numpy.float64'>
0.000152869818003
<class 'numpy.float64'>
0.00015279582308
<class 'numpy.float64'>
0.000152721899888
<class 'numpy.float64'>
0.000152648048322
<class 'numpy.float64'>
0.000152574268278
<class 'numpy.float64'>
0.000152500559652
<class 'numpy.float64'>
0.000152426922341
<class 'numpy.float64'>
0.00015235335624
<class 'numpy.float64'>
0.000152279861247
<class 'numpy.float64'>
0.000152206437258
<class 'numpy.float64'>
0.000152133084171
<class 'numpy.float64'>
0.000152059801883
<class 'numpy.float64'>
0.000151986590291
<class 'numpy.float64'>
0.000151913449293
<class 'numpy.float64'>
0.000151840378786
<class 'numpy.float64'>
0.00015176737867
<class 'numpy.float64'>
0.000151694448842
<class 'numpy.float64'>
0.0001516215892
<class 'numpy.float64'>
0.000151548799644
<class 'numpy.float64'>
0.000151476080072
<class 'numpy.float64'>
0.000151403430383
<class 'numpy.float64'>
0.000151330850477
<class 'numpy.float64'>
0.000151258340252
<class 'numpy.float64'>
0.000151185899609
<class 'numpy.float64'>
0.000151113528448
<class 'numpy.float64'>
0.000151041226667
<class 'numpy.float64'>
0.000150968994169
<class 'numpy.float64'>
0.000150896830852
<class 'numpy.float64'>
0.000150824736618
<class 'numpy.float64'>
0.000150752711367
<class 'numpy.float64'>
0.000150680755001
<class 'numpy.float64'>
0.00015060886742
<class 'numpy.float64'>
0.000150537048526
<class 'numpy.float64'>
0.00015046529822
<class 'numpy.float64'>
0.000150393616405
<class 'numpy.float64'>
0.000150322002981
<class 'numpy.float64'>
0.000150250457851
<class 'numpy.float64'>
0.000150178980919
<class 'numpy.float64'>
0.000150107572084
<class 'numpy.float64'>
0.000150036231252
<class 'numpy.float64'>
0.000149964958323
<class 'numpy.float64'>
0.000149893753202
<class 'numpy.float64'>
0.000149822615792
<class 'numpy.float64'>
0.000149751545996
<class 'numpy.float64'>
0.000149680543718
<class 'numpy.float64'>
0.000149609608861
<class 'numpy.float64'>
0.000149538741329
<class 'numpy.float64'>
0.000149467941026
<class 'numpy.float64'>
0.000149397207858
<class 'numpy.float64'>
0.000149326541727
<class 'numpy.float64'>
0.00014925594254
<class 'numpy.float64'>
0.0001491854102
<class 'numpy.float64'>
0.000149114944614
<class 'numpy.float64'>
0.000149044545685
<class 'numpy.float64'>
0.00014897421332
<class 'numpy.float64'>
0.000148903947424
<class 'numpy.float64'>
0.000148833747903
<class 'numpy.float64'>
0.000148763614662
<class 'numpy.float64'>
0.000148693547608
<class 'numpy.float64'>
0.000148623546648
<class 'numpy.float64'>
0.000148553611687
<class 'numpy.float64'>
0.000148483742632
<class 'numpy.float64'>
0.000148413939391
<class 'numpy.float64'>
0.000148344201869
<class 'numpy.float64'>
0.000148274529974
<class 'numpy.float64'>
0.000148204923615
<class 'numpy.float64'>
0.000148135382697
<class 'numpy.float64'>
0.000148065907129
<class 'numpy.float64'>
0.000147996496818
<class 'numpy.float64'>
0.000147927151674
<class 'numpy.float64'>
0.000147857871603
<class 'numpy.float64'>
0.000147788656514
<class 'numpy.float64'>
0.000147719506316
<class 'numpy.float64'>
0.000147650420918
<class 'numpy.float64'>
0.000147581400229
<class 'numpy.float64'>
0.000147512444156
<class 'numpy.float64'>
0.000147443552611
<class 'numpy.float64'>
0.000147374725502
<class 'numpy.float64'>
0.000147305962738
<class 'numpy.float64'>
0.00014723726423
<class 'numpy.float64'>
0.000147168629887
<class 'numpy.float64'>
0.00014710005962
<class 'numpy.float64'>
0.000147031553339
<class 'numpy.float64'>
0.000146963110954
<class 'numpy.float64'>
0.000146894732375
<class 'numpy.float64'>
0.000146826417514
<class 'numpy.float64'>
0.000146758166281
<class 'numpy.float64'>
0.000146689978588
<class 'numpy.float64'>
0.000146621854346
<class 'numpy.float64'>
0.000146553793466
<class 'numpy.float64'>
0.000146485795859
<class 'numpy.float64'>
0.000146417861439
<class 'numpy.float64'>
0.000146349990115
<class 'numpy.float64'>
0.000146282181801
<class 'numpy.float64'>
0.000146214436409
<class 'numpy.float64'>
0.000146146753851
<class 'numpy.float64'>
0.00014607913404
<class 'numpy.float64'>
0.000146011576888
<class 'numpy.float64'>
0.000145944082308
<class 'numpy.float64'>
0.000145876650214
<class 'numpy.float64'>
0.000145809280518
<class 'numpy.float64'>
0.000145741973135
<class 'numpy.float64'>
0.000145674727976
<class 'numpy.float64'>
0.000145607544957
<class 'numpy.float64'>
0.000145540423992
<class 'numpy.float64'>
0.000145473364992
<class 'numpy.float64'>
0.000145406367875
<class 'numpy.float64'>
0.000145339432552
<class 'numpy.float64'>
0.00014527255894
<class 'numpy.float64'>
0.000145205746953
<class 'numpy.float64'>
0.000145138996504
<class 'numpy.float64'>
0.00014507230751
<class 'numpy.float64'>
0.000145005679886
<class 'numpy.float64'>
0.000144939113547
<class 'numpy.float64'>
0.000144872608408
<class 'numpy.float64'>
0.000144806164384
<class 'numpy.float64'>
0.000144739781393
<class 'numpy.float64'>
0.000144673459348
<class 'numpy.float64'>
0.000144607198167
<class 'numpy.float64'>
0.000144540997766
<class 'numpy.float64'>
0.000144474858061
<class 'numpy.float64'>
0.000144408778968
<class 'numpy.float64'>
0.000144342760405
<class 'numpy.float64'>
0.000144276802288
<class 'numpy.float64'>
0.000144210904533
<class 'numpy.float64'>
0.000144145067059
<class 'numpy.float64'>
0.000144079289782
<class 'numpy.float64'>
0.00014401357262
<class 'numpy.float64'>
0.000143947915491
<class 'numpy.float64'>
0.000143882318311
<class 'numpy.float64'>
0.000143816780999
<class 'numpy.float64'>
0.000143751303474
<class 'numpy.float64'>
0.000143685885652
<class 'numpy.float64'>
0.000143620527453
<class 'numpy.float64'>
0.000143555228795
<class 'numpy.float64'>
0.000143489989597
<class 'numpy.float64'>
0.000143424809777
<class 'numpy.float64'>
0.000143359689254
<class 'numpy.float64'>
0.000143294627948
<class 'numpy.float64'>
0.000143229625777
<class 'numpy.float64'>
0.000143164682661
<class 'numpy.float64'>
0.00014309979852
<class 'numpy.float64'>
0.000143034973273
<class 'numpy.float64'>
0.000142970206839
<class 'numpy.float64'>
0.000142905499139
<class 'numpy.float64'>
0.000142840850093
<class 'numpy.float64'>
0.000142776259621
<class 'numpy.float64'>
0.000142711727644
<class 'numpy.float64'>
0.000142647254082
<class 'numpy.float64'>
0.000142582838855
<class 'numpy.float64'>
0.000142518481885
<class 'numpy.float64'>
0.000142454183093
<class 'numpy.float64'>
0.000142389942399
<class 'numpy.float64'>
0.000142325759724
<class 'numpy.float64'>
0.000142261634991
<class 'numpy.float64'>
0.000142197568121
<class 'numpy.float64'>
0.000142133559035
<class 'numpy.float64'>
0.000142069607655
<class 'numpy.float64'>
0.000142005713903
<class 'numpy.float64'>
0.000141941877702
<class 'numpy.float64'>
0.000141878098973
<class 'numpy.float64'>
0.000141814377639
<class 'numpy.float64'>
0.000141750713621
<class 'numpy.float64'>
0.000141687106844
<class 'numpy.float64'>
0.00014162355723
<class 'numpy.float64'>
0.000141560064701
<class 'numpy.float64'>
0.000141496629181
<class 'numpy.float64'>
0.000141433250593
<class 'numpy.float64'>
0.000141369928861
<class 'numpy.float64'>
0.000141306663907
<class 'numpy.float64'>
0.000141243455655
<class 'numpy.float64'>
0.00014118030403
<class 'numpy.float64'>
0.000141117208954
<class 'numpy.float64'>
0.000141054170353
<class 'numpy.float64'>
0.000140991188149
<class 'numpy.float64'>
0.000140928262269
<class 'numpy.float64'>
0.000140865392635
<class 'numpy.float64'>
0.000140802579172
<class 'numpy.float64'>
0.000140739821806
<class 'numpy.float64'>
0.000140677120461
<class 'numpy.float64'>
0.000140614475062
<class 'numpy.float64'>
0.000140551885534
<class 'numpy.float64'>
0.000140489351802
<class 'numpy.float64'>
0.000140426873793
<class 'numpy.float64'>
0.00014036445143
<class 'numpy.float64'>
0.00014030208464
<class 'numpy.float64'>
0.000140239773349
<class 'numpy.float64'>
0.000140177517482
<class 'numpy.float64'>
0.000140115316966
<class 'numpy.float64'>
0.000140053171726
<class 'numpy.float64'>
0.00013999108169
<class 'numpy.float64'>
0.000139929046783
<class 'numpy.float64'>
0.000139867066932
<class 'numpy.float64'>
0.000139805142064
<class 'numpy.float64'>
0.000139743272106
<class 'numpy.float64'>
0.000139681456983
<class 'numpy.float64'>
0.000139619696625
<class 'numpy.float64'>
0.000139557990957
<class 'numpy.float64'>
0.000139496339908
<class 'numpy.float64'>
0.000139434743404
<class 'numpy.float64'>
0.000139373201373
<class 'numpy.float64'>
0.000139311713743
<class 'numpy.float64'>
0.000139250280441
<class 'numpy.float64'>
0.000139188901397
<class 'numpy.float64'>
0.000139127576537
<class 'numpy.float64'>
0.000139066305791
<class 'numpy.float64'>
0.000139005089085
<class 'numpy.float64'>
0.00013894392635
<class 'numpy.float64'>
0.000138882817513
<class 'numpy.float64'>
0.000138821762504
<class 'numpy.float64'>
0.00013876076125
<class 'numpy.float64'>
0.000138699813682
<class 'numpy.float64'>
0.000138638919728
<class 'numpy.float64'>
0.000138578079317
<class 'numpy.float64'>
0.00013851729238
<class 'numpy.float64'>
0.000138456558844
<class 'numpy.float64'>
0.000138395878641
<class 'numpy.float64'>
0.000138335251699
<class 'numpy.float64'>
0.000138274677949
<class 'numpy.float64'>
0.00013821415732
<class 'numpy.float64'>
0.000138153689743
<class 'numpy.float64'>
0.000138093275148
<class 'numpy.float64'>
0.000138032913464
<class 'numpy.float64'>
0.000137972604624
<class 'numpy.float64'>
0.000137912348556
<class 'numpy.float64'>
0.000137852145193
<class 'numpy.float64'>
0.000137791994465
<class 'numpy.float64'>
0.000137731896302
<class 'numpy.float64'>
0.000137671850636
<class 'numpy.float64'>
0.000137611857398
<class 'numpy.float64'>
0.000137551916519
<class 'numpy.float64'>
0.000137492027932
<class 'numpy.float64'>
0.000137432191566
<class 'numpy.float64'>
0.000137372407355
<class 'numpy.float64'>
0.00013731267523
<class 'numpy.float64'>
0.000137252995122
<class 'numpy.float64'>
0.000137193366964
<class 'numpy.float64'>
0.000137133790688
<class 'numpy.float64'>
0.000137074266226
<class 'numpy.float64'>
0.000137014793511
<class 'numpy.float64'>
0.000136955372474
<class 'numpy.float64'>
0.00013689600305
<class 'numpy.float64'>
0.00013683668517
<class 'numpy.float64'>
0.000136777418767
<class 'numpy.float64'>
0.000136718203775
<class 'numpy.float64'>
0.000136659040126
<class 'numpy.float64'>
0.000136599927754
<class 'numpy.float64'>
0.000136540866591
<class 'numpy.float64'>
0.000136481856572
<class 'numpy.float64'>
0.00013642289763
<class 'numpy.float64'>
0.000136363989698
<class 'numpy.float64'>
0.000136305132711
<class 'numpy.float64'>
0.000136246326602
<class 'numpy.float64'>
0.000136187571305
<class 'numpy.float64'>
0.000136128866755
<class 'numpy.float64'>
0.000136070212885
<class 'numpy.float64'>
0.00013601160963
<class 'numpy.float64'>
0.000135953056925
<class 'numpy.float64'>
0.000135894554704
<class 'numpy.float64'>
0.000135836102901
<class 'numpy.float64'>
0.000135777701452
<class 'numpy.float64'>
0.000135719350292
<class 'numpy.float64'>
0.000135661049354
<class 'numpy.float64'>
0.000135602798576
<class 'numpy.float64'>
0.000135544597892
<class 'numpy.float64'>
0.000135486447236
<class 'numpy.float64'>
0.000135428346546
<class 'numpy.float64'>
0.000135370295756
<class 'numpy.float64'>
0.000135312294801
<class 'numpy.float64'>
0.000135254343619
<class 'numpy.float64'>
0.000135196442144
<class 'numpy.float64'>
0.000135138590313
<class 'numpy.float64'>
0.000135080788062
<class 'numpy.float64'>
0.000135023035328
<class 'numpy.float64'>
0.000134965332045
<class 'numpy.float64'>
0.000134907678152
<class 'numpy.float64'>
0.000134850073584
<class 'numpy.float64'>
0.000134792518278
<class 'numpy.float64'>
0.000134735012172
<class 'numpy.float64'>
0.000134677555201
<class 'numpy.float64'>
0.000134620147304
<class 'numpy.float64'>
0.000134562788416
<class 'numpy.float64'>
0.000134505478476
<class 'numpy.float64'>
0.00013444821742
<class 'numpy.float64'>
0.000134391005186
<class 'numpy.float64'>
0.000134333841712
<class 'numpy.float64'>
0.000134276726935
<class 'numpy.float64'>
0.000134219660794
<class 'numpy.float64'>
0.000134162643225
<class 'numpy.float64'>
0.000134105674167
<class 'numpy.float64'>
0.000134048753558
<class 'numpy.float64'>
0.000133991881335
<class 'numpy.float64'>
0.000133935057439
<class 'numpy.float64'>
0.000133878281806
<class 'numpy.float64'>
0.000133821554375
<class 'numpy.float64'>
0.000133764875085
<class 'numpy.float64'>
0.000133708243875
<class 'numpy.float64'>
0.000133651660683
<class 'numpy.float64'>
0.000133595125448
<class 'numpy.float64'>
0.00013353863811
<class 'numpy.float64'>
0.000133482198607
<class 'numpy.float64'>
0.000133425806879
<class 'numpy.float64'>
0.000133369462864
<class 'numpy.float64'>
0.000133313166504
<class 'numpy.float64'>
0.000133256917736
<class 'numpy.float64'>
0.0001332007165
<class 'numpy.float64'>
0.000133144562737
<class 'numpy.float64'>
0.000133088456386
<class 'numpy.float64'>
0.000133032397387
<class 'numpy.float64'>
0.00013297638568
<class 'numpy.float64'>
0.000132920421206
<class 'numpy.float64'>
0.000132864503904
<class 'numpy.float64'>
0.000132808633714
<class 'numpy.float64'>
0.000132752810578
<class 'numpy.float64'>
0.000132697034436
<class 'numpy.float64'>
0.000132641305228
<class 'numpy.float64'>
0.000132585622896
<class 'numpy.float64'>
0.000132529987379
<class 'numpy.float64'>
0.00013247439862
<class 'numpy.float64'>
0.000132418856558
<class 'numpy.float64'>
0.000132363361136
<class 'numpy.float64'>
0.000132307912294
<class 'numpy.float64'>
0.000132252509974
<class 'numpy.float64'>
0.000132197154117
<class 'numpy.float64'>
0.000132141844665
<class 'numpy.float64'>
0.000132086581559
<class 'numpy.float64'>
0.000132031364741
<class 'numpy.float64'>
0.000131976194153
<class 'numpy.float64'>
0.000131921069737
<class 'numpy.float64'>
0.000131865991435
<class 'numpy.float64'>
0.000131810959189
<class 'numpy.float64'>
0.000131755972941
<class 'numpy.float64'>
0.000131701032634
<class 'numpy.float64'>
0.000131646138209
<class 'numpy.float64'>
0.00013159128961
<class 'numpy.float64'>
0.000131536486779
<class 'numpy.float64'>
0.000131481729659
<class 'numpy.float64'>
0.000131427018192
<class 'numpy.float64'>
0.000131372352321
<class 'numpy.float64'>
0.000131317731991
<class 'numpy.float64'>
0.000131263157142
<class 'numpy.float64'>
0.000131208627719
<class 'numpy.float64'>
0.000131154143666
<class 'numpy.float64'>
0.000131099704924
<class 'numpy.float64'>
0.000131045311438
<class 'numpy.float64'>
0.000130990963152
<class 'numpy.float64'>
0.000130936660009
<class 'numpy.float64'>
0.000130882401952
<class 'numpy.float64'>
0.000130828188925
<class 'numpy.float64'>
0.000130774020874
<class 'numpy.float64'>
0.00013071989774
<class 'numpy.float64'>
0.00013066581947
<class 'numpy.float64'>
0.000130611786006
<class 'numpy.float64'>
0.000130557797293
<class 'numpy.float64'>
0.000130503853276
<class 'numpy.float64'>
0.000130449953898
<class 'numpy.float64'>
0.000130396099106
<class 'numpy.float64'>
0.000130342288842
<class 'numpy.float64'>
0.000130288523052
<class 'numpy.float64'>
0.000130234801681
<class 'numpy.float64'>
0.000130181124674
<class 'numpy.float64'>
0.000130127491976
<class 'numpy.float64'>
0.000130073903532
<class 'numpy.float64'>
0.000130020359287
<class 'numpy.float64'>
0.000129966859186
<class 'numpy.float64'>
0.000129913403175
<class 'numpy.float64'>
0.000129859991199
<class 'numpy.float64'>
0.000129806623205
<class 'numpy.float64'>
0.000129753299136
<class 'numpy.float64'>
0.000129700018941
<class 'numpy.float64'>
0.000129646782563
<class 'numpy.float64'>
0.000129593589949
<class 'numpy.float64'>
0.000129540441046
<class 'numpy.float64'>
0.000129487335798
<class 'numpy.float64'>
0.000129434274153
<class 'numpy.float64'>
0.000129381256057
<class 'numpy.float64'>
0.000129328281456
<class 'numpy.float64'>
0.000129275350296
<class 'numpy.float64'>
0.000129222462524
<class 'numpy.float64'>
0.000129169618087
<class 'numpy.float64'>
0.000129116816932
<class 'numpy.float64'>
0.000129064059004
<class 'numpy.float64'>
0.000129011344252
<class 'numpy.float64'>
0.000128958672622
<class 'numpy.float64'>
0.000128906044061
<class 'numpy.float64'>
0.000128853458517
<class 'numpy.float64'>
0.000128800915936
<class 'numpy.float64'>
0.000128748416266
<class 'numpy.float64'>
0.000128695959455
<class 'numpy.float64'>
0.000128643545449
<class 'numpy.float64'>
0.000128591174197
<class 'numpy.float64'>
0.000128538845646
<class 'numpy.float64'>
0.000128486559744
<class 'numpy.float64'>
0.000128434316438
<class 'numpy.float64'>
0.000128382115677
<class 'numpy.float64'>
0.000128329957409
<class 'numpy.float64'>
0.000128277841582
<class 'numpy.float64'>
0.000128225768143
<class 'numpy.float64'>
0.000128173737043
<class 'numpy.float64'>
0.000128121748227
<class 'numpy.float64'>
0.000128069801646
<class 'numpy.float64'>
0.000128017897247
<class 'numpy.float64'>
0.000127966034979
<class 'numpy.float64'>
0.000127914214791
<class 'numpy.float64'>
0.000127862436632
<class 'numpy.float64'>
0.00012781070045
<class 'numpy.float64'>
0.000127759006195
<class 'numpy.float64'>
0.000127707353815
<class 'numpy.float64'>
0.00012765574326
<class 'numpy.float64'>
0.000127604174478
<class 'numpy.float64'>
0.000127552647419
<class 'numpy.float64'>
0.000127501162033
<class 'numpy.float64'>
0.000127449718269
<class 'numpy.float64'>
0.000127398316075
<class 'numpy.float64'>
0.000127346955403
<class 'numpy.float64'>
0.000127295636201
<class 'numpy.float64'>
0.00012724435842
<class 'numpy.float64'>
0.000127193122009
<class 'numpy.float64'>
0.000127141926918
<class 'numpy.float64'>
0.000127090773097
<class 'numpy.float64'>
0.000127039660496
<class 'numpy.float64'>
0.000126988589065
<class 'numpy.float64'>
0.000126937558755
<class 'numpy.float64'>
0.000126886569516
<class 'numpy.float64'>
0.000126835621299
<class 'numpy.float64'>
0.000126784714053
<class 'numpy.float64'>
0.000126733847729
<class 'numpy.float64'>
0.000126683022279
<class 'numpy.float64'>
0.000126632237652
<class 'numpy.float64'>
0.0001265814938
<class 'numpy.float64'>
0.000126530790673
<class 'numpy.float64'>
0.000126480128222
<class 'numpy.float64'>
0.000126429506399
<class 'numpy.float64'>
0.000126378925154
<class 'numpy.float64'>
0.000126328384439
<class 'numpy.float64'>
0.000126277884206
<class 'numpy.float64'>
0.000126227424404
<class 'numpy.float64'>
0.000126177004986
<class 'numpy.float64'>
0.000126126625903
<class 'numpy.float64'>
0.000126076287108
<class 'numpy.float64'>
0.000126025988551
<class 'numpy.float64'>
0.000125975730184
<class 'numpy.float64'>
0.000125925511959
<class 'numpy.float64'>
0.000125875333828
<class 'numpy.float64'>
0.000125825195743
<class 'numpy.float64'>
0.000125775097656
<class 'numpy.float64'>
0.000125725039519
<class 'numpy.float64'>
0.000125675021285
<class 'numpy.float64'>
0.000125625042905
<class 'numpy.float64'>
0.000125575104332
<class 'numpy.float64'>
0.000125525205518
<class 'numpy.float64'>
0.000125475346416
<class 'numpy.float64'>
0.000125425526979
<class 'numpy.float64'>
0.000125375747159
<class 'numpy.float64'>
0.000125326006909
<class 'numpy.float64'>
0.000125276306181
<class 'numpy.float64'>
0.000125226644929
<class 'numpy.float64'>
0.000125177023105
<class 'numpy.float64'>
0.000125127440663
<class 'numpy.float64'>
0.000125077897555
<class 'numpy.float64'>
0.000125028393736
<class 'numpy.float64'>
0.000124978929157
<class 'numpy.float64'>
0.000124929503773
<class 'numpy.float64'>
0.000124880117536
<class 'numpy.float64'>
0.000124830770401
<class 'numpy.float64'>
0.000124781462321
<class 'numpy.float64'>
0.000124732193249
<class 'numpy.float64'>
0.000124682963139
<class 'numpy.float64'>
0.000124633771945
<class 'numpy.float64'>
0.00012458461962
<class 'numpy.float64'>
0.00012453550612
<class 'numpy.float64'>
0.000124486431397
<class 'numpy.float64'>
0.000124437395406
<class 'numpy.float64'>
0.0001243883981
<class 'numpy.float64'>
0.000124339439435
<class 'numpy.float64'>
0.000124290519364
<class 'numpy.float64'>
0.000124241637842
<class 'numpy.float64'>
0.000124192794822
<class 'numpy.float64'>
0.000124143990261
<class 'numpy.float64'>
0.000124095224111
<class 'numpy.float64'>
0.000124046496329
<class 'numpy.float64'>
0.000123997806868
<class 'numpy.float64'>
0.000123949155683
<class 'numpy.float64'>
0.000123900542729
<class 'numpy.float64'>
0.000123851967961
<class 'numpy.float64'>
0.000123803431335
<class 'numpy.float64'>
0.000123754932804
<class 'numpy.float64'>
0.000123706472325
<class 'numpy.float64'>
0.000123658049852
<class 'numpy.float64'>
0.000123609665341
<class 'numpy.float64'>
0.000123561318747
<class 'numpy.float64'>
0.000123513010026
<class 'numpy.float64'>
0.000123464739133
<class 'numpy.float64'>
0.000123416506023
<class 'numpy.float64'>
0.000123368310652
<class 'numpy.float64'>
0.000123320152977
<class 'numpy.float64'>
0.000123272032951
<class 'numpy.float64'>
0.000123223950533
<class 'numpy.float64'>
0.000123175905676
<class 'numpy.float64'>
0.000123127898338
<class 'numpy.float64'>
0.000123079928475
<class 'numpy.float64'>
0.000123031996042
<class 'numpy.float64'>
0.000122984100995
<class 'numpy.float64'>
0.000122936243292
<class 'numpy.float64'>
0.000122888422887
<class 'numpy.float64'>
0.000122840639739
<class 'numpy.float64'>
0.000122792893802
<class 'numpy.float64'>
0.000122745185034
<class 'numpy.float64'>
0.000122697513391
<class 'numpy.float64'>
0.00012264987883
<class 'numpy.float64'>
0.000122602281307
<class 'numpy.float64'>
0.00012255472078
<class 'numpy.float64'>
0.000122507197205
<class 'numpy.float64'>
0.000122459710539
<class 'numpy.float64'>
0.000122412260739
<class 'numpy.float64'>
0.000122364847762
<class 'numpy.float64'>
0.000122317471566
<class 'numpy.float64'>
0.000122270132107
<class 'numpy.float64'>
0.000122222829343
<class 'numpy.float64'>
0.000122175563231
<class 'numpy.float64'>
0.000122128333728
<class 'numpy.float64'>
0.000122081140792
<class 'numpy.float64'>
0.00012203398438
<class 'numpy.float64'>
0.000121986864451
<class 'numpy.float64'>
0.000121939780961
<class 'numpy.float64'>
0.000121892733869
<class 'numpy.float64'>
0.000121845723131
<class 'numpy.float64'>
0.000121798748707
<class 'numpy.float64'>
0.000121751810554
<class 'numpy.float64'>
0.000121704908629
<class 'numpy.float64'>
0.000121658042891
<class 'numpy.float64'>
0.000121611213299
<class 'numpy.float64'>
0.00012156441981
<class 'numpy.float64'>
0.000121517662381
<class 'numpy.float64'>
0.000121470940973
<class 'numpy.float64'>
0.000121424255543
<class 'numpy.float64'>
0.00012137760605
<class 'numpy.float64'>
0.000121330992451
<class 'numpy.float64'>
0.000121284414706
<class 'numpy.float64'>
0.000121237872774
<class 'numpy.float64'>
0.000121191366612
<class 'numpy.float64'>
0.00012114489618
<class 'numpy.float64'>
0.000121098461436
<class 'numpy.float64'>
0.00012105206234
<class 'numpy.float64'>
0.00012100569885
<class 'numpy.float64'>
0.000120959370925
<class 'numpy.float64'>
0.000120913078525
<class 'numpy.float64'>
0.000120866821608
<class 'numpy.float64'>
0.000120820600133
<class 'numpy.float64'>
0.000120774414061
<class 'numpy.float64'>
0.00012072826335
<class 'numpy.float64'>
0.000120682147959
<class 'numpy.float64'>
0.000120636067849
<class 'numpy.float64'>
0.000120590022978
<class 'numpy.float64'>
0.000120544013306
<class 'numpy.float64'>
0.000120498038793
<class 'numpy.float64'>
0.000120452099398
<class 'numpy.float64'>
0.000120406195082
<class 'numpy.float64'>
0.000120360325804
<class 'numpy.float64'>
0.000120314491523
<class 'numpy.float64'>
0.0001202686922
<class 'numpy.float64'>
0.000120222927795
<class 'numpy.float64'>
0.000120177198268
<class 'numpy.float64'>
0.000120131503579
<class 'numpy.float64'>
0.000120085843688
<class 'numpy.float64'>
0.000120040218556
<class 'numpy.float64'>
0.000119994628141
<class 'numpy.float64'>
0.000119949072407
<class 'numpy.float64'>
0.000119903551311
<class 'numpy.float64'>
0.000119858064816
<class 'numpy.float64'>
0.000119812612881
<class 'numpy.float64'>
0.000119767195467
<class 'numpy.float64'>
0.000119721812534
<class 'numpy.float64'>
0.000119676464045
<class 'numpy.float64'>
0.000119631149958
<class 'numpy.float64'>
0.000119585870236
<class 'numpy.float64'>
0.000119540624838
<class 'numpy.float64'>
0.000119495413727
<class 'numpy.float64'>
0.000119450236862
<class 'numpy.float64'>
0.000119405094205
<class 'numpy.float64'>
0.000119359985718
<class 'numpy.float64'>
0.000119314911361
<class 'numpy.float64'>
0.000119269871096
<class 'numpy.float64'>
0.000119224864883
<class 'numpy.float64'>
0.000119179892685
<class 'numpy.float64'>
0.000119134954463
<class 'numpy.float64'>
0.000119090050177
<class 'numpy.float64'>
0.000119045179791
<class 'numpy.float64'>
0.000119000343264
<class 'numpy.float64'>
0.00011895554056
<class 'numpy.float64'>
0.00011891077164
<class 'numpy.float64'>
0.000118866036465
<class 'numpy.float64'>
0.000118821334997
<class 'numpy.float64'>
0.000118776667199
<class 'numpy.float64'>
0.000118732033031
<class 'numpy.float64'>
0.000118687432457
<class 'numpy.float64'>
0.000118642865438
<class 'numpy.float64'>
0.000118598331937
<class 'numpy.float64'>
0.000118553831915
<class 'numpy.float64'>
0.000118509365335
<class 'numpy.float64'>
0.000118464932159
<class 'numpy.float64'>
0.00011842053235
<class 'numpy.float64'>
0.000118376165869
<class 'numpy.float64'>
0.000118331832679
<class 'numpy.float64'>
0.000118287532744
<class 'numpy.float64'>
0.000118243266025
<class 'numpy.float64'>
0.000118199032484
<class 'numpy.float64'>
0.000118154832086
<class 'numpy.float64'>
0.000118110664792
<class 'numpy.float64'>
0.000118066530564
<class 'numpy.float64'>
0.000118022429368
<class 'numpy.float64'>
0.000117978361164
<class 'numpy.float64'>
0.000117934325916
<class 'numpy.float64'>
0.000117890323587
<class 'numpy.float64'>
0.00011784635414
<class 'numpy.float64'>
0.000117802417539
<class 'numpy.float64'>
0.000117758513746
<class 'numpy.float64'>
0.000117714642724
<class 'numpy.float64'>
0.000117670804437
<class 'numpy.float64'>
0.000117626998849
<class 'numpy.float64'>
0.000117583225922
<class 'numpy.float64'>
0.00011753948562
<class 'numpy.float64'>
0.000117495777907
<class 'numpy.float64'>
0.000117452102746
<class 'numpy.float64'>
0.000117408460101
<class 'numpy.float64'>
0.000117364849936
<class 'numpy.float64'>
0.000117321272214
<class 'numpy.float64'>
0.000117277726898
<class 'numpy.float64'>
0.000117234213954
<class 'numpy.float64'>
0.000117190733345
<class 'numpy.float64'>
0.000117147285034
<class 'numpy.float64'>
0.000117103868986
<class 'numpy.float64'>
0.000117060485164
<class 'numpy.float64'>
0.000117017133534
<class 'numpy.float64'>
0.000116973814058
<class 'numpy.float64'>
0.000116930526702
<class 'numpy.float64'>
0.000116887271429
<class 'numpy.float64'>
0.000116844048204
<class 'numpy.float64'>
0.000116800856992
<class 'numpy.float64'>
0.000116757697755
<class 'numpy.float64'>
0.000116714570461
<class 'numpy.float64'>
0.000116671475071
<class 'numpy.float64'>
0.000116628411552
<class 'numpy.float64'>
0.000116585379868
<class 'numpy.float64'>
0.000116542379983
<class 'numpy.float64'>
0.000116499411863
<class 'numpy.float64'>
0.000116456475471
<class 'numpy.float64'>
0.000116413570774
<class 'numpy.float64'>
0.000116370697735
<class 'numpy.float64'>
0.00011632785632
<class 'numpy.float64'>
0.000116285046494
<class 'numpy.float64'>
0.000116242268222
<class 'numpy.float64'>
0.000116199521468
<class 'numpy.float64'>
0.000116156806199
<class 'numpy.float64'>
0.000116114122379
<class 'numpy.float64'>
0.000116071469974
<class 'numpy.float64'>
0.000116028848948
<class 'numpy.float64'>
0.000115986259268
<class 'numpy.float64'>
0.000115943700898
<class 'numpy.float64'>
0.000115901173804
<class 'numpy.float64'>
0.000115858677952
<class 'numpy.float64'>
0.000115816213307
<class 'numpy.float64'>
0.000115773779835
<class 'numpy.float64'>
0.000115731377501
<class 'numpy.float64'>
0.000115689006272
<class 'numpy.float64'>
0.000115646666112
<class 'numpy.float64'>
0.000115604356988
<class 'numpy.float64'>
0.000115562078866
<class 'numpy.float64'>
0.000115519831711
<class 'numpy.float64'>
0.000115477615489
<class 'numpy.float64'>
0.000115435430167
<class 'numpy.float64'>
0.000115393275711
<class 'numpy.float64'>
0.000115351152086
<class 'numpy.float64'>
0.000115309059259
<class 'numpy.float64'>
0.000115266997196
<class 'numpy.float64'>
0.000115224965864
<class 'numpy.float64'>
0.000115182965228
<class 'numpy.float64'>
0.000115140995255
<class 'numpy.float64'>
0.000115099055911
<class 'numpy.float64'>
0.000115057147164
<class 'numpy.float64'>
0.000115015268979
<class 'numpy.float64'>
0.000114973421322
<class 'numpy.float64'>
0.000114931604162
<class 'numpy.float64'>
0.000114889817463
<class 'numpy.float64'>
0.000114848061193
<class 'numpy.float64'>
0.000114806335319
<class 'numpy.float64'>
0.000114764639807
<class 'numpy.float64'>
0.000114722974625
<class 'numpy.float64'>
0.000114681339738
<class 'numpy.float64'>
0.000114639735115
<class 'numpy.float64'>
0.000114598160722
<class 'numpy.float64'>
0.000114556616526
<class 'numpy.float64'>
0.000114515102495
<class 'numpy.float64'>
0.000114473618594
<class 'numpy.float64'>
0.000114432164793
<class 'numpy.float64'>
0.000114390741057
<class 'numpy.float64'>
0.000114349347355
<class 'numpy.float64'>
0.000114307983652
<class 'numpy.float64'>
0.000114266649918
<class 'numpy.float64'>
0.000114225346119
<class 'numpy.float64'>
0.000114184072222
<class 'numpy.float64'>
0.000114142828196
<class 'numpy.float64'>
0.000114101614007
<class 'numpy.float64'>
0.000114060429624
<class 'numpy.float64'>
0.000114019275013
<class 'numpy.float64'>
0.000113978150144
<class 'numpy.float64'>
0.000113937054983
<class 'numpy.float64'>
0.000113895989498
<class 'numpy.float64'>
0.000113854953658
<class 'numpy.float64'>
0.000113813947429
<class 'numpy.float64'>
0.000113772970781
<class 'numpy.float64'>
0.00011373202368
<class 'numpy.float64'>
0.000113691106095
<class 'numpy.float64'>
0.000113650217995
<class 'numpy.float64'>
0.000113609359346
<class 'numpy.float64'>
0.000113568530118
<class 'numpy.float64'>
0.000113527730279
<class 'numpy.float64'>
0.000113486959796
<class 'numpy.float64'>
0.000113446218638
<class 'numpy.float64'>
0.000113405506774
<class 'numpy.float64'>
0.000113364824172
<class 'numpy.float64'>
0.0001133241708
<class 'numpy.float64'>
0.000113283546627
<class 'numpy.float64'>
0.000113242951621
<class 'numpy.float64'>
0.000113202385751
<class 'numpy.float64'>
0.000113161848985
<class 'numpy.float64'>
0.000113121341293
<class 'numpy.float64'>
0.000113080862643
<class 'numpy.float64'>
0.000113040413003
<class 'numpy.float64'>
0.000112999992343
<class 'numpy.float64'>
0.000112959600631
<class 'numpy.float64'>
0.000112919237837
<class 'numpy.float64'>
0.000112878903928
<class 'numpy.float64'>
0.000112838598875
<class 'numpy.float64'>
0.000112798322646
<class 'numpy.float64'>
0.00011275807521
<class 'numpy.float64'>
0.000112717856536
<class 'numpy.float64'>
0.000112677666594
<class 'numpy.float64'>
0.000112637505352
<class 'numpy.float64'>
0.000112597372781
<class 'numpy.float64'>
0.000112557268849
<class 'numpy.float64'>
0.000112517193525
<class 'numpy.float64'>
0.000112477146779
<class 'numpy.float64'>
0.00011243712858
<class 'numpy.float64'>
0.000112397138899
<class 'numpy.float64'>
0.000112357177703
<class 'numpy.float64'>
0.000112317244964
<class 'numpy.float64'>
0.00011227734065
<class 'numpy.float64'>
0.000112237464732
<class 'numpy.float64'>
0.000112197617178
<class 'numpy.float64'>
0.000112157797958
<class 'numpy.float64'>
0.000112118007043
<class 'numpy.float64'>
0.000112078244402
<class 'numpy.float64'>
0.000112038510005
<class 'numpy.float64'>
0.000111998803822
<class 'numpy.float64'>
0.000111959125823
<class 'numpy.float64'>
0.000111919475977
<class 'numpy.float64'>
0.000111879854255
<class 'numpy.float64'>
0.000111840260627
<class 'numpy.float64'>
0.000111800695063
<class 'numpy.float64'>
0.000111761157534
<class 'numpy.float64'>
0.000111721648008
<class 'numpy.float64'>
0.000111682166457
<class 'numpy.float64'>
0.000111642712851
<class 'numpy.float64'>
0.000111603287161
<class 'numpy.float64'>
0.000111563889355
<class 'numpy.float64'>
0.000111524519406
<class 'numpy.float64'>
0.000111485177283
<class 'numpy.float64'>
0.000111445862957
<class 'numpy.float64'>
0.000111406576399
<class 'numpy.float64'>
0.000111367317578
<class 'numpy.float64'>
0.000111328086466
<class 'numpy.float64'>
0.000111288883033
<class 'numpy.float64'>
0.00011124970725
<class 'numpy.float64'>
0.000111210559088
<class 'numpy.float64'>
0.000111171438517
<class 'numpy.float64'>
0.000111132345509
<class 'numpy.float64'>
0.000111093280034
<class 'numpy.float64'>
0.000111054242063
<class 'numpy.float64'>
0.000111015231566
<class 'numpy.float64'>
0.000110976248516
<class 'numpy.float64'>
0.000110937292883
<class 'numpy.float64'>
0.000110898364638
<class 'numpy.float64'>
0.000110859463752
<class 'numpy.float64'>
0.000110820590196
<class 'numpy.float64'>
0.000110781743943
<class 'numpy.float64'>
0.000110742924961
<class 'numpy.float64'>
0.000110704133224
<class 'numpy.float64'>
0.000110665368702
<class 'numpy.float64'>
0.000110626631367
<class 'numpy.float64'>
0.00011058792119
<class 'numpy.float64'>
0.000110549238142
<class 'numpy.float64'>
0.000110510582195
<class 'numpy.float64'>
0.000110471953321
<class 'numpy.float64'>
0.000110433351491
<class 'numpy.float64'>
0.000110394776677
<class 'numpy.float64'>
0.000110356228849
<class 'numpy.float64'>
0.000110317707981
<class 'numpy.float64'>
0.000110279214043
<class 'numpy.float64'>
0.000110240747008
<class 'numpy.float64'>
0.000110202306847
<class 'numpy.float64'>
0.000110163893532
<class 'numpy.float64'>
0.000110125507034
<class 'numpy.float64'>
0.000110087147327
<class 'numpy.float64'>
0.000110048814381
<class 'numpy.float64'>
0.000110010508169
<class 'numpy.float64'>
0.000109972228663
<class 'numpy.float64'>
0.000109933975834
<class 'numpy.float64'>
0.000109895749656
<class 'numpy.float64'>
0.000109857550099
<class 'numpy.float64'>
0.000109819377137
<class 'numpy.float64'>
0.000109781230741
<class 'numpy.float64'>
0.000109743110884
<class 'numpy.float64'>
0.000109705017537
<class 'numpy.float64'>
0.000109666950674
<class 'numpy.float64'>
0.000109628910267
<class 'numpy.float64'>
0.000109590896288
<class 'numpy.float64'>
0.000109552908709
<class 'numpy.float64'>
0.000109514947504
<class 'numpy.float64'>
0.000109477012644
<class 'numpy.float64'>
0.000109439104102
<class 'numpy.float64'>
0.000109401221851
<class 'numpy.float64'>
0.000109363365864
<class 'numpy.float64'>
0.000109325536112
<class 'numpy.float64'>
0.000109287732569
<class 'numpy.float64'>
0.000109249955208
<class 'numpy.float64'>
0.000109212204001
<class 'numpy.float64'>
0.000109174478921
<class 'numpy.float64'>
0.000109136779942
<class 'numpy.float64'>
0.000109099107035
<class 'numpy.float64'>
0.000109061460174
<class 'numpy.float64'>
0.000109023839332
<class 'numpy.float64'>
0.000108986244482
<class 'numpy.float64'>
0.000108948675596
<class 'numpy.float64'>
0.000108911132649
<class 'numpy.float64'>
0.000108873615613
<class 'numpy.float64'>
0.000108836124461
<class 'numpy.float64'>
0.000108798659167
<class 'numpy.float64'>
0.000108761219703
<class 'numpy.float64'>
0.000108723806044
<class 'numpy.float64'>
0.000108686418162
<class 'numpy.float64'>
0.00010864905603
<class 'numpy.float64'>
0.000108611719623
<class 'numpy.float64'>
0.000108574408913
<class 'numpy.float64'>
0.000108537123874
<class 'numpy.float64'>
0.00010849986448
<class 'numpy.float64'>
0.000108462630703
<class 'numpy.float64'>
0.000108425422518
<class 'numpy.float64'>
0.000108388239899
<class 'numpy.float64'>
0.000108351082818
<class 'numpy.float64'>
0.00010831395125
<class 'numpy.float64'>
0.000108276845167
<class 'numpy.float64'>
0.000108239764545
<class 'numpy.float64'>
0.000108202709357
<class 'numpy.float64'>
0.000108165679576
<class 'numpy.float64'>
0.000108128675177
<class 'numpy.float64'>
0.000108091696133
<class 'numpy.float64'>
0.000108054742418
<class 'numpy.float64'>
0.000108017814006
<class 'numpy.float64'>
0.000107980910872
<class 'numpy.float64'>
0.000107944032989
<class 'numpy.float64'>
0.000107907180332
<class 'numpy.float64'>
0.000107870352873
<class 'numpy.float64'>
0.000107833550589
<class 'numpy.float64'>
0.000107796773452
<class 'numpy.float64'>
0.000107760021437
<class 'numpy.float64'>
0.000107723294519
<class 'numpy.float64'>
0.000107686592671
<class 'numpy.float64'>
0.000107649915867
<class 'numpy.float64'>
0.000107613264084
<class 'numpy.float64'>
0.000107576637293
<class 'numpy.float64'>
0.000107540035471
<class 'numpy.float64'>
0.000107503458591
<class 'numpy.float64'>
0.000107466906628
<class 'numpy.float64'>
0.000107430379557
<class 'numpy.float64'>
0.000107393877352
<class 'numpy.float64'>
0.000107357399987
<class 'numpy.float64'>
0.000107320947438
<class 'numpy.float64'>
0.000107284519679
<class 'numpy.float64'>
0.000107248116684
<class 'numpy.float64'>
0.000107211738429
<class 'numpy.float64'>
0.000107175384888
<class 'numpy.float64'>
0.000107139056036
<class 'numpy.float64'>
0.000107102751848
<class 'numpy.float64'>
0.000107066472298
<class 'numpy.float64'>
0.000107030217362
<class 'numpy.float64'>
0.000106993987015
<class 'numpy.float64'>
0.000106957781232
<class 'numpy.float64'>
0.000106921599987
<class 'numpy.float64'>
0.000106885443255
<class 'numpy.float64'>
0.000106849311012
<class 'numpy.float64'>
0.000106813203233
<class 'numpy.float64'>
0.000106777119893
<class 'numpy.float64'>
0.000106741060967
<class 'numpy.float64'>
0.000106705026431
<class 'numpy.float64'>
0.000106669016259
<class 'numpy.float64'>
0.000106633030426
<class 'numpy.float64'>
0.00010659706891
<class 'numpy.float64'>
0.000106561131683
<class 'numpy.float64'>
0.000106525218722
<class 'numpy.float64'>
0.000106489330003
<class 'numpy.float64'>
0.000106453465501
<class 'numpy.float64'>
0.000106417625191
<class 'numpy.float64'>
0.000106381809048
<class 'numpy.float64'>
0.000106346017049
<class 'numpy.float64'>
0.000106310249169
<class 'numpy.float64'>
0.000106274505383
<class 'numpy.float64'>
0.000106238785667
<class 'numpy.float64'>
0.000106203089997
<class 'numpy.float64'>
0.000106167418349
<class 'numpy.float64'>
0.000106131770698
<class 'numpy.float64'>
0.00010609614702
<class 'numpy.float64'>
0.000106060547291
<class 'numpy.float64'>
0.000106024971487
<class 'numpy.float64'>
0.000105989419583
<class 'numpy.float64'>
0.000105953891556
<class 'numpy.float64'>
0.000105918387381
<class 'numpy.float64'>
0.000105882907035
<class 'numpy.float64'>
0.000105847450493
<class 'numpy.float64'>
0.000105812017731
<class 'numpy.float64'>
0.000105776608726
<class 'numpy.float64'>
0.000105741223453
<class 'numpy.float64'>
0.000105705861889
<class 'numpy.float64'>
0.00010567052401
<class 'numpy.float64'>
0.000105635209793
<class 'numpy.float64'>
0.000105599919212
<class 'numpy.float64'>
0.000105564652245
<class 'numpy.float64'>
0.000105529408868
<class 'numpy.float64'>
0.000105494189058
<class 'numpy.float64'>
0.000105458992789
<class 'numpy.float64'>
0.00010542382004
<class 'numpy.float64'>
0.000105388670786
<class 'numpy.float64'>
0.000105353545004
<class 'numpy.float64'>
Cost after iteration 3000: 0.000105
0.00010531844267
<class 'numpy.float64'>
0.000105283363761
<class 'numpy.float64'>
0.000105248308253
<class 'numpy.float64'>
0.000105213276123
<class 'numpy.float64'>
0.000105178267348
<class 'numpy.float64'>
0.000105143281904
<class 'numpy.float64'>
0.000105108319767
<class 'numpy.float64'>
0.000105073380916
<class 'numpy.float64'>
0.000105038465325
<class 'numpy.float64'>
0.000105003572972
<class 'numpy.float64'>
0.000104968703835
<class 'numpy.float64'>
0.000104933857889
<class 'numpy.float64'>
0.000104899035111
<class 'numpy.float64'>
0.000104864235479
<class 'numpy.float64'>
0.000104829458969
<class 'numpy.float64'>
0.000104794705559
<class 'numpy.float64'>
0.000104759975224
<class 'numpy.float64'>
0.000104725267943
<class 'numpy.float64'>
0.000104690583692
<class 'numpy.float64'>
0.000104655922449
<class 'numpy.float64'>
0.00010462128419
<class 'numpy.float64'>
0.000104586668892
<class 'numpy.float64'>
0.000104552076533
<class 'numpy.float64'>
0.00010451750709
<class 'numpy.float64'>
0.000104482960541
<class 'numpy.float64'>
0.000104448436861
<class 'numpy.float64'>
0.000104413936029
<class 'numpy.float64'>
0.000104379458023
<class 'numpy.float64'>
0.000104345002818
<class 'numpy.float64'>
0.000104310570393
<class 'numpy.float64'>
0.000104276160726
<class 'numpy.float64'>
0.000104241773793
<class 'numpy.float64'>
0.000104207409572
<class 'numpy.float64'>
0.00010417306804
<class 'numpy.float64'>
0.000104138749176
<class 'numpy.float64'>
0.000104104452955
<class 'numpy.float64'>
0.000104070179358
<class 'numpy.float64'>
0.000104035928359
<class 'numpy.float64'>
0.000104001699939
<class 'numpy.float64'>
0.000103967494073
<class 'numpy.float64'>
0.00010393331074
<class 'numpy.float64'>
0.000103899149918
<class 'numpy.float64'>
0.000103865011583
<class 'numpy.float64'>
0.000103830895715
<class 'numpy.float64'>
0.000103796802291
<class 'numpy.float64'>
0.000103762731288
<class 'numpy.float64'>
0.000103728682685
<class 'numpy.float64'>
0.000103694656459
<class 'numpy.float64'>
0.000103660652589
<class 'numpy.float64'>
0.000103626671051
<class 'numpy.float64'>
0.000103592711826
<class 'numpy.float64'>
0.00010355877489
<class 'numpy.float64'>
0.000103524860221
<class 'numpy.float64'>
0.000103490967797
<class 'numpy.float64'>
0.000103457097597
<class 'numpy.float64'>
0.000103423249599
<class 'numpy.float64'>
0.000103389423781
<class 'numpy.float64'>
0.000103355620121
<class 'numpy.float64'>
0.000103321838597
<class 'numpy.float64'>
0.000103288079188
<class 'numpy.float64'>
0.000103254341871
<class 'numpy.float64'>
0.000103220626626
<class 'numpy.float64'>
0.00010318693343
<class 'numpy.float64'>
0.000103153262262
<class 'numpy.float64'>
0.0001031196131
<class 'numpy.float64'>
0.000103085985923
<class 'numpy.float64'>
0.000103052380708
<class 'numpy.float64'>
0.000103018797435
<class 'numpy.float64'>
0.000102985236083
<class 'numpy.float64'>
0.000102951696629
<class 'numpy.float64'>
0.000102918179052
<class 'numpy.float64'>
0.000102884683331
<class 'numpy.float64'>
0.000102851209444
<class 'numpy.float64'>
0.000102817757371
<class 'numpy.float64'>
0.000102784327088
<class 'numpy.float64'>
0.000102750918577
<class 'numpy.float64'>
0.000102717531814
<class 'numpy.float64'>
0.000102684166779
<class 'numpy.float64'>
0.000102650823451
<class 'numpy.float64'>
0.000102617501808
<class 'numpy.float64'>
0.00010258420183
<class 'numpy.float64'>
0.000102550923495
<class 'numpy.float64'>
0.000102517666781
<class 'numpy.float64'>
0.000102484431669
<class 'numpy.float64'>
0.000102451218136
<class 'numpy.float64'>
0.000102418026163
<class 'numpy.float64'>
0.000102384855727
<class 'numpy.float64'>
0.000102351706808
<class 'numpy.float64'>
0.000102318579385
<class 'numpy.float64'>
0.000102285473438
<class 'numpy.float64'>
0.000102252388944
<class 'numpy.float64'>
0.000102219325884
<class 'numpy.float64'>
0.000102186284236
<class 'numpy.float64'>
0.00010215326398
<class 'numpy.float64'>
0.000102120265094
<class 'numpy.float64'>
0.000102087287559
<class 'numpy.float64'>
0.000102054331353
<class 'numpy.float64'>
0.000102021396456
<class 'numpy.float64'>
0.000101988482846
<class 'numpy.float64'>
0.000101955590505
<class 'numpy.float64'>
0.000101922719409
<class 'numpy.float64'>
0.00010188986954
<class 'numpy.float64'>
0.000101857040877
<class 'numpy.float64'>
0.000101824233399
<class 'numpy.float64'>
0.000101791447085
<class 'numpy.float64'>
0.000101758681915
<class 'numpy.float64'>
0.000101725937869
<class 'numpy.float64'>
0.000101693214926
<class 'numpy.float64'>
0.000101660513066
<class 'numpy.float64'>
0.000101627832268
<class 'numpy.float64'>
0.000101595172512
<class 'numpy.float64'>
0.000101562533778
<class 'numpy.float64'>
0.000101529916045
<class 'numpy.float64'>
0.000101497319293
<class 'numpy.float64'>
0.000101464743502
<class 'numpy.float64'>
0.000101432188652
<class 'numpy.float64'>
0.000101399654722
<class 'numpy.float64'>
0.000101367141692
<class 'numpy.float64'>
0.000101334649543
<class 'numpy.float64'>
0.000101302178253
<class 'numpy.float64'>
0.000101269727803
<class 'numpy.float64'>
0.000101237298173
<class 'numpy.float64'>
0.000101204889343
<class 'numpy.float64'>
0.000101172501292
<class 'numpy.float64'>
0.000101140134002
<class 'numpy.float64'>
0.000101107787451
<class 'numpy.float64'>
0.00010107546162
<class 'numpy.float64'>
0.000101043156488
<class 'numpy.float64'>
0.000101010872037
<class 'numpy.float64'>
0.000100978608247
<class 'numpy.float64'>
0.000100946365096
<class 'numpy.float64'>
0.000100914142566
<class 'numpy.float64'>
0.000100881940636
<class 'numpy.float64'>
0.000100849759288
<class 'numpy.float64'>
0.000100817598501
<class 'numpy.float64'>
0.000100785458255
<class 'numpy.float64'>
0.000100753338532
<class 'numpy.float64'>
0.00010072123931
<class 'numpy.float64'>
0.000100689160571
<class 'numpy.float64'>
0.000100657102295
<class 'numpy.float64'>
0.000100625064462
<class 'numpy.float64'>
0.000100593047053
<class 'numpy.float64'>
0.000100561050048
<class 'numpy.float64'>
0.000100529073428
<class 'numpy.float64'>
0.000100497117174
<class 'numpy.float64'>
0.000100465181264
<class 'numpy.float64'>
0.000100433265682
<class 'numpy.float64'>
0.000100401370406
<class 'numpy.float64'>
0.000100369495418
<class 'numpy.float64'>
0.000100337640698
<class 'numpy.float64'>
0.000100305806227
<class 'numpy.float64'>
0.000100273991985
<class 'numpy.float64'>
0.000100242197954
<class 'numpy.float64'>
0.000100210424113
<class 'numpy.float64'>
0.000100178670444
<class 'numpy.float64'>
0.000100146936928
<class 'numpy.float64'>
0.000100115223545
<class 'numpy.float64'>
0.000100083530276
<class 'numpy.float64'>
0.000100051857103
<class 'numpy.float64'>
0.000100020204004
<class 'numpy.float64'>
9.99885709635e-05
<class 'numpy.float64'>
9.99569579601e-05
<class 'numpy.float64'>
9.99253649755e-05
<class 'numpy.float64'>
9.98937919903e-05
<class 'numpy.float64'>
9.98622389862e-05
<class 'numpy.float64'>
9.98307059436e-05
<class 'numpy.float64'>
9.97991928436e-05
<class 'numpy.float64'>
9.97676996673e-05
<class 'numpy.float64'>
9.97362263961e-05
<class 'numpy.float64'>
9.97047730109e-05
<class 'numpy.float64'>
9.96733394929e-05
<class 'numpy.float64'>
9.96419258234e-05
<class 'numpy.float64'>
9.96105319833e-05
<class 'numpy.float64'>
9.95791579539e-05
<class 'numpy.float64'>
9.95478037166e-05
<class 'numpy.float64'>
9.95164692526e-05
<class 'numpy.float64'>
9.94851545432e-05
<class 'numpy.float64'>
9.94538595692e-05
<class 'numpy.float64'>
9.94225843128e-05
<class 'numpy.float64'>
9.9391328755e-05
<class 'numpy.float64'>
9.93600928772e-05
<class 'numpy.float64'>
9.93288766604e-05
<class 'numpy.float64'>
9.92976800864e-05
<class 'numpy.float64'>
9.9266503137e-05
<class 'numpy.float64'>
9.92353457932e-05
<class 'numpy.float64'>
9.92042080365e-05
<class 'numpy.float64'>
9.91730898485e-05
<class 'numpy.float64'>
9.9141991211e-05
<class 'numpy.float64'>
9.9110912105e-05
<class 'numpy.float64'>
9.9079852513e-05
<class 'numpy.float64'>
9.90488124155e-05
<class 'numpy.float64'>
9.90177917952e-05
<class 'numpy.float64'>
9.89867906325e-05
<class 'numpy.float64'>
9.89558089104e-05
<class 'numpy.float64'>
9.892484661e-05
<class 'numpy.float64'>
9.8893903713e-05
<class 'numpy.float64'>
9.88629802012e-05
<class 'numpy.float64'>
9.88320760566e-05
<class 'numpy.float64'>
9.88011912604e-05
<class 'numpy.float64'>
9.87703257953e-05
<class 'numpy.float64'>
9.87394796426e-05
<class 'numpy.float64'>
9.87086527839e-05
<class 'numpy.float64'>
9.8677845202e-05
<class 'numpy.float64'>
9.86470568778e-05
<class 'numpy.float64'>
9.86162877936e-05
<class 'numpy.float64'>
9.85855379316e-05
<class 'numpy.float64'>
9.85548072738e-05
<class 'numpy.float64'>
9.85240958019e-05
<class 'numpy.float64'>
9.8493403498e-05
<class 'numpy.float64'>
9.84627303442e-05
<class 'numpy.float64'>
9.84320763225e-05
<class 'numpy.float64'>
9.84014414155e-05
<class 'numpy.float64'>
9.83708256043e-05
<class 'numpy.float64'>
9.83402288718e-05
<class 'numpy.float64'>
9.83096512001e-05
<class 'numpy.float64'>
9.82790925711e-05
<class 'numpy.float64'>
9.82485529675e-05
<class 'numpy.float64'>
9.8218032371e-05
<class 'numpy.float64'>
9.8187530764e-05
<class 'numpy.float64'>
9.81570481288e-05
<class 'numpy.float64'>
9.81265844476e-05
<class 'numpy.float64'>
9.8096139703e-05
<class 'numpy.float64'>
9.80657138771e-05
<class 'numpy.float64'>
9.80353069525e-05
<class 'numpy.float64'>
9.8004918911e-05
<class 'numpy.float64'>
9.79745497356e-05
<class 'numpy.float64'>
9.79441994085e-05
<class 'numpy.float64'>
9.79138679124e-05
<class 'numpy.float64'>
9.78835552293e-05
<class 'numpy.float64'>
9.78532613419e-05
<class 'numpy.float64'>
9.78229862331e-05
<class 'numpy.float64'>
9.77927298845e-05
<class 'numpy.float64'>
9.77624922796e-05
<class 'numpy.float64'>
9.77322734005e-05
<class 'numpy.float64'>
9.770207323e-05
<class 'numpy.float64'>
9.76718917505e-05
<class 'numpy.float64'>
9.76417289448e-05
<class 'numpy.float64'>
9.76115847955e-05
<class 'numpy.float64'>
9.75814592853e-05
<class 'numpy.float64'>
9.75513523973e-05
<class 'numpy.float64'>
9.75212641133e-05
<class 'numpy.float64'>
9.74911944169e-05
<class 'numpy.float64'>
9.74611432905e-05
<class 'numpy.float64'>
9.74311107169e-05
<class 'numpy.float64'>
9.74010966793e-05
<class 'numpy.float64'>
9.73711011596e-05
<class 'numpy.float64'>
9.73411241417e-05
<class 'numpy.float64'>
9.73111656081e-05
<class 'numpy.float64'>
9.72812255413e-05
<class 'numpy.float64'>
9.72513039247e-05
<class 'numpy.float64'>
9.72214007413e-05
<class 'numpy.float64'>
9.71915159734e-05
<class 'numpy.float64'>
9.71616496046e-05
<class 'numpy.float64'>
9.71318016179e-05
<class 'numpy.float64'>
9.71019719962e-05
<class 'numpy.float64'>
9.70721607222e-05
<class 'numpy.float64'>
9.70423677796e-05
<class 'numpy.float64'>
9.70125931508e-05
<class 'numpy.float64'>
9.69828368196e-05
<class 'numpy.float64'>
9.69530987684e-05
<class 'numpy.float64'>
9.69233789813e-05
<class 'numpy.float64'>
9.68936774404e-05
<class 'numpy.float64'>
9.68639941295e-05
<class 'numpy.float64'>
9.6834329032e-05
<class 'numpy.float64'>
9.68046821306e-05
<class 'numpy.float64'>
9.67750534087e-05
<class 'numpy.float64'>
9.67454428498e-05
<class 'numpy.float64'>
9.67158504371e-05
<class 'numpy.float64'>
9.66862761538e-05
<class 'numpy.float64'>
9.6656719983e-05
<class 'numpy.float64'>
9.66271819087e-05
<class 'numpy.float64'>
9.6597661914e-05
<class 'numpy.float64'>
9.65681599819e-05
<class 'numpy.float64'>
9.65386760965e-05
<class 'numpy.float64'>
9.65092102404e-05
<class 'numpy.float64'>
9.6479762398e-05
<class 'numpy.float64'>
9.64503325521e-05
<class 'numpy.float64'>
9.64209206864e-05
<class 'numpy.float64'>
9.63915267844e-05
<class 'numpy.float64'>
9.63621508298e-05
<class 'numpy.float64'>
9.6332792806e-05
<class 'numpy.float64'>
9.63034526965e-05
<class 'numpy.float64'>
9.62741304849e-05
<class 'numpy.float64'>
9.62448261551e-05
<class 'numpy.float64'>
9.62155396901e-05
<class 'numpy.float64'>
9.61862710745e-05
<class 'numpy.float64'>
9.61570202914e-05
<class 'numpy.float64'>
9.61277873245e-05
<class 'numpy.float64'>
9.60985721575e-05
<class 'numpy.float64'>
9.60693747742e-05
<class 'numpy.float64'>
9.60401951587e-05
<class 'numpy.float64'>
9.60110332939e-05
<class 'numpy.float64'>
9.59818891645e-05
<class 'numpy.float64'>
9.5952762754e-05
<class 'numpy.float64'>
9.59236540459e-05
<class 'numpy.float64'>
9.58945630247e-05
<class 'numpy.float64'>
9.58654896737e-05
<class 'numpy.float64'>
9.58364339771e-05
<class 'numpy.float64'>
9.58073959186e-05
<class 'numpy.float64'>
9.57783754827e-05
<class 'numpy.float64'>
9.57493726523e-05
<class 'numpy.float64'>
9.57203874125e-05
<class 'numpy.float64'>
9.56914197466e-05
<class 'numpy.float64'>
9.56624696389e-05
<class 'numpy.float64'>
9.5633537073e-05
<class 'numpy.float64'>
9.56046220338e-05
<class 'numpy.float64'>
9.55757245044e-05
<class 'numpy.float64'>
9.55468444695e-05
<class 'numpy.float64'>
9.5517981913e-05
<class 'numpy.float64'>
9.54891368192e-05
<class 'numpy.float64'>
9.54603091719e-05
<class 'numpy.float64'>
9.54314989556e-05
<class 'numpy.float64'>
9.54027061545e-05
<class 'numpy.float64'>
9.53739307524e-05
<class 'numpy.float64'>
9.53451727339e-05
<class 'numpy.float64'>
9.5316432083e-05
<class 'numpy.float64'>
9.52877087843e-05
<class 'numpy.float64'>
9.52590028219e-05
<class 'numpy.float64'>
9.52303141798e-05
<class 'numpy.float64'>
9.52016428427e-05
<class 'numpy.float64'>
9.5172988795e-05
<class 'numpy.float64'>
9.51443520208e-05
<class 'numpy.float64'>
9.51157325046e-05
<class 'numpy.float64'>
9.50871302306e-05
<class 'numpy.float64'>
9.50585451835e-05
<class 'numpy.float64'>
9.50299773476e-05
<class 'numpy.float64'>
9.50014267074e-05
<class 'numpy.float64'>
9.49728932473e-05
<class 'numpy.float64'>
9.49443769515e-05
<class 'numpy.float64'>
9.49158778052e-05
<class 'numpy.float64'>
9.48873957922e-05
<class 'numpy.float64'>
9.48589308976e-05
<class 'numpy.float64'>
9.48304831057e-05
<class 'numpy.float64'>
9.4802052401e-05
<class 'numpy.float64'>
9.47736387681e-05
<class 'numpy.float64'>
9.4745242192e-05
<class 'numpy.float64'>
9.47168626571e-05
<class 'numpy.float64'>
9.46885001477e-05
<class 'numpy.float64'>
9.4660154649e-05
<class 'numpy.float64'>
9.46318261454e-05
<class 'numpy.float64'>
9.46035146219e-05
<class 'numpy.float64'>
9.45752200629e-05
<class 'numpy.float64'>
9.45469424534e-05
<class 'numpy.float64'>
9.45186817776e-05
<class 'numpy.float64'>
9.44904380212e-05
<class 'numpy.float64'>
9.44622111684e-05
<class 'numpy.float64'>
9.4434001204e-05
<class 'numpy.float64'>
9.44058081129e-05
<class 'numpy.float64'>
9.437763188e-05
<class 'numpy.float64'>
9.43494724906e-05
<class 'numpy.float64'>
9.4321329929e-05
<class 'numpy.float64'>
9.42932041803e-05
<class 'numpy.float64'>
9.42650952293e-05
<class 'numpy.float64'>
9.42370030611e-05
<class 'numpy.float64'>
9.42089276608e-05
<class 'numpy.float64'>
9.41808690132e-05
<class 'numpy.float64'>
9.41528271032e-05
<class 'numpy.float64'>
9.41248019159e-05
<class 'numpy.float64'>
9.40967934365e-05
<class 'numpy.float64'>
9.40688016499e-05
<class 'numpy.float64'>
9.40408265412e-05
<class 'numpy.float64'>
9.40128680955e-05
<class 'numpy.float64'>
9.39849262979e-05
<class 'numpy.float64'>
9.39570011335e-05
<class 'numpy.float64'>
9.39290925875e-05
<class 'numpy.float64'>
9.39012006448e-05
<class 'numpy.float64'>
9.38733252912e-05
<class 'numpy.float64'>
9.38454665111e-05
<class 'numpy.float64'>
9.38176242904e-05
<class 'numpy.float64'>
9.3789798614e-05
<class 'numpy.float64'>
9.37619894674e-05
<class 'numpy.float64'>
9.37341968351e-05
<class 'numpy.float64'>
9.37064207033e-05
<class 'numpy.float64'>
9.36786610568e-05
<class 'numpy.float64'>
9.36509178812e-05
<class 'numpy.float64'>
9.36231911616e-05
<class 'numpy.float64'>
9.35954808835e-05
<class 'numpy.float64'>
9.3567787032e-05
<class 'numpy.float64'>
9.35401095929e-05
<class 'numpy.float64'>
9.35124485513e-05
<class 'numpy.float64'>
9.34848038927e-05
<class 'numpy.float64'>
9.34571756027e-05
<class 'numpy.float64'>
9.34295636665e-05
<class 'numpy.float64'>
9.34019680696e-05
<class 'numpy.float64'>
9.3374388798e-05
<class 'numpy.float64'>
9.33468258366e-05
<class 'numpy.float64'>
9.33192791711e-05
<class 'numpy.float64'>
9.3291748787e-05
<class 'numpy.float64'>
9.32642346698e-05
<class 'numpy.float64'>
9.32367368054e-05
<class 'numpy.float64'>
9.32092551793e-05
<class 'numpy.float64'>
9.31817897769e-05
<class 'numpy.float64'>
9.31543405839e-05
<class 'numpy.float64'>
9.31269075861e-05
<class 'numpy.float64'>
9.30994907687e-05
<class 'numpy.float64'>
9.30720901179e-05
<class 'numpy.float64'>
9.30447056195e-05
<class 'numpy.float64'>
9.30173372587e-05
<class 'numpy.float64'>
9.29899850213e-05
<class 'numpy.float64'>
9.29626488935e-05
<class 'numpy.float64'>
9.29353288604e-05
<class 'numpy.float64'>
9.29080249086e-05
<class 'numpy.float64'>
9.28807370232e-05
<class 'numpy.float64'>
9.28534651903e-05
<class 'numpy.float64'>
9.28262093957e-05
<class 'numpy.float64'>
9.27989696252e-05
<class 'numpy.float64'>
9.27717458645e-05
<class 'numpy.float64'>
9.27445380999e-05
<class 'numpy.float64'>
9.27173463171e-05
<class 'numpy.float64'>
9.2690170502e-05
<class 'numpy.float64'>
9.26630106405e-05
<class 'numpy.float64'>
9.26358667184e-05
<class 'numpy.float64'>
9.2608738722e-05
<class 'numpy.float64'>
9.25816266372e-05
<class 'numpy.float64'>
9.25545304496e-05
<class 'numpy.float64'>
9.25274501456e-05
<class 'numpy.float64'>
9.25003857112e-05
<class 'numpy.float64'>
9.24733371324e-05
<class 'numpy.float64'>
9.24463043952e-05
<class 'numpy.float64'>
9.2419287486e-05
<class 'numpy.float64'>
9.23922863901e-05
<class 'numpy.float64'>
9.23653010946e-05
<class 'numpy.float64'>
9.2338331585e-05
<class 'numpy.float64'>
9.23113778474e-05
<class 'numpy.float64'>
9.22844398685e-05
<class 'numpy.float64'>
9.2257517634e-05
<class 'numpy.float64'>
9.223061113e-05
<class 'numpy.float64'>
9.22037203432e-05
<class 'numpy.float64'>
9.21768452594e-05
<class 'numpy.float64'>
9.21499858652e-05
<class 'numpy.float64'>
9.21231421465e-05
<class 'numpy.float64'>
9.20963140897e-05
<class 'numpy.float64'>
9.2069501681e-05
<class 'numpy.float64'>
9.20427049071e-05
<class 'numpy.float64'>
9.20159237538e-05
<class 'numpy.float64'>
9.19891582078e-05
<class 'numpy.float64'>
9.19624082553e-05
<class 'numpy.float64'>
9.19356738829e-05
<class 'numpy.float64'>
9.19089550762e-05
<class 'numpy.float64'>
9.18822518227e-05
<class 'numpy.float64'>
9.1855564108e-05
<class 'numpy.float64'>
9.18288919189e-05
<class 'numpy.float64'>
9.18022352419e-05
<class 'numpy.float64'>
9.17755940633e-05
<class 'numpy.float64'>
9.17489683694e-05
<class 'numpy.float64'>
9.17223581471e-05
<class 'numpy.float64'>
9.16957633827e-05
<class 'numpy.float64'>
9.16691840627e-05
<class 'numpy.float64'>
9.16426201736e-05
<class 'numpy.float64'>
9.16160717019e-05
<class 'numpy.float64'>
9.15895386347e-05
<class 'numpy.float64'>
9.15630209579e-05
<class 'numpy.float64'>
9.15365186587e-05
<class 'numpy.float64'>
9.1510031723e-05
<class 'numpy.float64'>
9.14835601383e-05
<class 'numpy.float64'>
9.14571038904e-05
<class 'numpy.float64'>
9.14306629665e-05
<class 'numpy.float64'>
9.14042373534e-05
<class 'numpy.float64'>
9.13778270372e-05
<class 'numpy.float64'>
9.1351432005e-05
<class 'numpy.float64'>
9.13250522438e-05
<class 'numpy.float64'>
9.12986877397e-05
<class 'numpy.float64'>
9.12723384799e-05
<class 'numpy.float64'>
9.12460044512e-05
<class 'numpy.float64'>
9.12196856402e-05
<class 'numpy.float64'>
9.11933820333e-05
<class 'numpy.float64'>
9.11670936181e-05
<class 'numpy.float64'>
9.11408203812e-05
<class 'numpy.float64'>
9.11145623094e-05
<class 'numpy.float64'>
9.10883193895e-05
<class 'numpy.float64'>
9.10620916083e-05
<class 'numpy.float64'>
9.10358789527e-05
<class 'numpy.float64'>
9.10096814099e-05
<class 'numpy.float64'>
9.09834989664e-05
<class 'numpy.float64'>
9.09573316095e-05
<class 'numpy.float64'>
9.09311793263e-05
<class 'numpy.float64'>
9.09050421032e-05
<class 'numpy.float64'>
9.08789199276e-05
<class 'numpy.float64'>
9.08528127861e-05
<class 'numpy.float64'>
9.08267206663e-05
<class 'numpy.float64'>
9.08006435548e-05
<class 'numpy.float64'>
9.07745814389e-05
<class 'numpy.float64'>
9.07485343053e-05
<class 'numpy.float64'>
9.07225021417e-05
<class 'numpy.float64'>
9.06964849345e-05
<class 'numpy.float64'>
9.06704826712e-05
<class 'numpy.float64'>
9.06444953388e-05
<class 'numpy.float64'>
9.06185229246e-05
<class 'numpy.float64'>
9.05925654154e-05
<class 'numpy.float64'>
9.05666227984e-05
<class 'numpy.float64'>
9.05406950615e-05
<class 'numpy.float64'>
9.0514782191e-05
<class 'numpy.float64'>
9.04888841743e-05
<class 'numpy.float64'>
9.04630009993e-05
<class 'numpy.float64'>
9.04371326525e-05
<class 'numpy.float64'>
9.04112791213e-05
<class 'numpy.float64'>
9.03854403929e-05
<class 'numpy.float64'>
9.03596164549e-05
<class 'numpy.float64'>
9.03338072945e-05
<class 'numpy.float64'>
9.03080128989e-05
<class 'numpy.float64'>
9.02822332551e-05
<class 'numpy.float64'>
9.02564683514e-05
<class 'numpy.float64'>
9.02307181739e-05
<class 'numpy.float64'>
9.02049827108e-05
<class 'numpy.float64'>
9.01792619497e-05
<class 'numpy.float64'>
9.01535558769e-05
<class 'numpy.float64'>
9.0127864481e-05
<class 'numpy.float64'>
9.01021877491e-05
<class 'numpy.float64'>
9.00765256679e-05
<class 'numpy.float64'>
9.00508782257e-05
<class 'numpy.float64'>
9.002524541e-05
<class 'numpy.float64'>
8.99996272076e-05
<class 'numpy.float64'>
8.99740236063e-05
<class 'numpy.float64'>
8.99484345939e-05
<class 'numpy.float64'>
8.99228601577e-05
<class 'numpy.float64'>
8.98973002854e-05
<class 'numpy.float64'>
8.98717549643e-05
<class 'numpy.float64'>
8.98462241817e-05
<class 'numpy.float64'>
8.98207079261e-05
<class 'numpy.float64'>
8.97952061843e-05
<class 'numpy.float64'>
8.97697189442e-05
<class 'numpy.float64'>
8.97442461933e-05
<class 'numpy.float64'>
8.97187879195e-05
<class 'numpy.float64'>
8.96933441103e-05
<class 'numpy.float64'>
8.96679147536e-05
<class 'numpy.float64'>
8.96424998366e-05
<class 'numpy.float64'>
8.96170993468e-05
<class 'numpy.float64'>
8.95917132728e-05
<class 'numpy.float64'>
8.95663416019e-05
<class 'numpy.float64'>
8.95409843216e-05
<class 'numpy.float64'>
8.95156414202e-05
<class 'numpy.float64'>
8.94903128849e-05
<class 'numpy.float64'>
8.94649987038e-05
<class 'numpy.float64'>
8.94396988646e-05
<class 'numpy.float64'>
8.94144133547e-05
<class 'numpy.float64'>
8.93891421627e-05
<class 'numpy.float64'>
8.93638852761e-05
<class 'numpy.float64'>
8.93386426826e-05
<class 'numpy.float64'>
8.93134143703e-05
<class 'numpy.float64'>
8.92882003267e-05
<class 'numpy.float64'>
8.92630005401e-05
<class 'numpy.float64'>
8.92378149981e-05
<class 'numpy.float64'>
8.92126436888e-05
<class 'numpy.float64'>
8.91874866003e-05
<class 'numpy.float64'>
8.91623437201e-05
<class 'numpy.float64'>
8.91372150367e-05
<class 'numpy.float64'>
8.91121005373e-05
<class 'numpy.float64'>
8.90870002109e-05
<class 'numpy.float64'>
8.90619140444e-05
<class 'numpy.float64'>
8.90368420266e-05
<class 'numpy.float64'>
8.90117841451e-05
<class 'numpy.float64'>
8.89867403883e-05
<class 'numpy.float64'>
8.89617107441e-05
<class 'numpy.float64'>
8.89366952006e-05
<class 'numpy.float64'>
8.89116937458e-05
<class 'numpy.float64'>
8.88867063677e-05
<class 'numpy.float64'>
8.88617330545e-05
<class 'numpy.float64'>
8.88367737945e-05
<class 'numpy.float64'>
8.88118285755e-05
<class 'numpy.float64'>
8.8786897386e-05
<class 'numpy.float64'>
8.87619802137e-05
<class 'numpy.float64'>
8.87370770475e-05
<class 'numpy.float64'>
8.87121878748e-05
<class 'numpy.float64'>
8.86873126842e-05
<class 'numpy.float64'>
8.86624514639e-05
<class 'numpy.float64'>
8.8637604202e-05
<class 'numpy.float64'>
8.86127708867e-05
<class 'numpy.float64'>
8.85879515067e-05
<class 'numpy.float64'>
8.85631460499e-05
<class 'numpy.float64'>
8.85383545046e-05
<class 'numpy.float64'>
8.8513576859e-05
<class 'numpy.float64'>
8.84888131016e-05
<class 'numpy.float64'>
8.84640632206e-05
<class 'numpy.float64'>
8.84393272039e-05
<class 'numpy.float64'>
8.8414605041e-05
<class 'numpy.float64'>
8.83898967193e-05
<class 'numpy.float64'>
8.83652022277e-05
<class 'numpy.float64'>
8.8340521554e-05
<class 'numpy.float64'>
8.8315854687e-05
<class 'numpy.float64'>
8.82912016153e-05
<class 'numpy.float64'>
8.82665623267e-05
<class 'numpy.float64'>
8.82419368101e-05
<class 'numpy.float64'>
8.82173250541e-05
<class 'numpy.float64'>
8.81927270466e-05
<class 'numpy.float64'>
8.81681427764e-05
<class 'numpy.float64'>
8.8143572232e-05
<class 'numpy.float64'>
8.81190154018e-05
<class 'numpy.float64'>
8.80944722744e-05
<class 'numpy.float64'>
8.80699428384e-05
<class 'numpy.float64'>
8.80454270822e-05
<class 'numpy.float64'>
8.80209249946e-05
<class 'numpy.float64'>
8.79964365636e-05
<class 'numpy.float64'>
8.79719617784e-05
<class 'numpy.float64'>
8.79475006273e-05
<class 'numpy.float64'>
8.79230530989e-05
<class 'numpy.float64'>
8.78986191818e-05
<class 'numpy.float64'>
8.78741988646e-05
<class 'numpy.float64'>
8.78497921364e-05
<class 'numpy.float64'>
8.7825398985e-05
<class 'numpy.float64'>
8.78010193997e-05
<class 'numpy.float64'>
8.77766533689e-05
<class 'numpy.float64'>
8.77523008815e-05
<class 'numpy.float64'>
8.77279619265e-05
<class 'numpy.float64'>
8.77036364918e-05
<class 'numpy.float64'>
8.76793245666e-05
<class 'numpy.float64'>
8.76550261397e-05
<class 'numpy.float64'>
8.76307411997e-05
<class 'numpy.float64'>
8.76064697355e-05
<class 'numpy.float64'>
8.75822117356e-05
<class 'numpy.float64'>
8.75579671891e-05
<class 'numpy.float64'>
8.75337360847e-05
<class 'numpy.float64'>
8.75095184111e-05
<class 'numpy.float64'>
8.74853141574e-05
<class 'numpy.float64'>
8.74611233122e-05
<class 'numpy.float64'>
8.74369458644e-05
<class 'numpy.float64'>
8.74127818032e-05
<class 'numpy.float64'>
8.7388631117e-05
<class 'numpy.float64'>
8.73644937946e-05
<class 'numpy.float64'>
8.73403698254e-05
<class 'numpy.float64'>
8.7316259198e-05
<class 'numpy.float64'>
8.72921619014e-05
<class 'numpy.float64'>
8.72680779248e-05
<class 'numpy.float64'>
8.72440072565e-05
<class 'numpy.float64'>
8.72199498862e-05
<class 'numpy.float64'>
8.71959058023e-05
<class 'numpy.float64'>
8.71718749943e-05
<class 'numpy.float64'>
8.7147857451e-05
<class 'numpy.float64'>
8.71238531611e-05
<class 'numpy.float64'>
8.7099862114e-05
<class 'numpy.float64'>
8.70758842986e-05
<class 'numpy.float64'>
8.70519197037e-05
<class 'numpy.float64'>
8.70279683188e-05
<class 'numpy.float64'>
8.70040301327e-05
<class 'numpy.float64'>
8.69801051348e-05
<class 'numpy.float64'>
8.69561933139e-05
<class 'numpy.float64'>
8.6932294659e-05
<class 'numpy.float64'>
8.69084091598e-05
<class 'numpy.float64'>
8.68845368048e-05
<class 'numpy.float64'>
8.68606775831e-05
<class 'numpy.float64'>
8.68368314846e-05
<class 'numpy.float64'>
8.68129984977e-05
<class 'numpy.float64'>
8.6789178612e-05
<class 'numpy.float64'>
8.67653718165e-05
<class 'numpy.float64'>
8.6741578101e-05
<class 'numpy.float64'>
8.67177974538e-05
<class 'numpy.float64'>
8.66940298643e-05
<class 'numpy.float64'>
8.66702753224e-05
<class 'numpy.float64'>
8.66465338169e-05
<class 'numpy.float64'>
8.66228053369e-05
<class 'numpy.float64'>
8.65990898717e-05
<class 'numpy.float64'>
8.6575387411e-05
<class 'numpy.float64'>
8.65516979438e-05
<class 'numpy.float64'>
8.65280214594e-05
<class 'numpy.float64'>
8.65043579475e-05
<class 'numpy.float64'>
8.64807073969e-05
<class 'numpy.float64'>
8.64570697971e-05
<class 'numpy.float64'>
8.64334451377e-05
<class 'numpy.float64'>
8.64098334078e-05
<class 'numpy.float64'>
8.6386234597e-05
<class 'numpy.float64'>
8.63626486944e-05
<class 'numpy.float64'>
8.63390756895e-05
<class 'numpy.float64'>
8.63155155723e-05
<class 'numpy.float64'>
8.62919683313e-05
<class 'numpy.float64'>
8.62684339567e-05
<class 'numpy.float64'>
8.62449124372e-05
<class 'numpy.float64'>
8.62214037627e-05
<class 'numpy.float64'>
8.61979079228e-05
<class 'numpy.float64'>
8.61744249069e-05
<class 'numpy.float64'>
8.61509547042e-05
<class 'numpy.float64'>
8.61274973046e-05
<class 'numpy.float64'>
8.61040526978e-05
<class 'numpy.float64'>
8.60806208723e-05
<class 'numpy.float64'>
8.60572018187e-05
<class 'numpy.float64'>
8.60337955262e-05
<class 'numpy.float64'>
8.60104019844e-05
<class 'numpy.float64'>
8.59870211828e-05
<class 'numpy.float64'>
8.59636531108e-05
<class 'numpy.float64'>
8.59402977587e-05
<class 'numpy.float64'>
8.59169551152e-05
<class 'numpy.float64'>
8.58936251703e-05
<class 'numpy.float64'>
8.5870307914e-05
<class 'numpy.float64'>
8.58470033354e-05
<class 'numpy.float64'>
8.58237114245e-05
<class 'numpy.float64'>
8.5800432171e-05
<class 'numpy.float64'>
8.57771655643e-05
<class 'numpy.float64'>
8.57539115941e-05
<class 'numpy.float64'>
8.57306702503e-05
<class 'numpy.float64'>
8.57074415228e-05
<class 'numpy.float64'>
8.56842254007e-05
<class 'numpy.float64'>
8.5661021874e-05
<class 'numpy.float64'>
8.56378309331e-05
<class 'numpy.float64'>
8.56146525665e-05
<class 'numpy.float64'>
8.5591486765e-05
<class 'numpy.float64'>
8.55683335181e-05
<class 'numpy.float64'>
8.55451928157e-05
<class 'numpy.float64'>
8.55220646472e-05
<class 'numpy.float64'>
8.54989490023e-05
<class 'numpy.float64'>
8.54758458717e-05
<class 'numpy.float64'>
8.54527552446e-05
<class 'numpy.float64'>
8.54296771111e-05
<class 'numpy.float64'>
8.54066114604e-05
<class 'numpy.float64'>
8.53835582835e-05
<class 'numpy.float64'>
8.53605175691e-05
<class 'numpy.float64'>
8.53374893079e-05
<class 'numpy.float64'>
8.53144734896e-05
<class 'numpy.float64'>
8.52914701039e-05
<class 'numpy.float64'>
8.52684791407e-05
<class 'numpy.float64'>
8.52455005903e-05
<class 'numpy.float64'>
8.52225344427e-05
<class 'numpy.float64'>
8.51995806873e-05
<class 'numpy.float64'>
8.51766393145e-05
<class 'numpy.float64'>
8.51537103141e-05
<class 'numpy.float64'>
8.51307936763e-05
<class 'numpy.float64'>
8.51078893906e-05
<class 'numpy.float64'>
8.50849974475e-05
<class 'numpy.float64'>
8.50621178369e-05
<class 'numpy.float64'>
8.50392505489e-05
<class 'numpy.float64'>
8.50163955732e-05
<class 'numpy.float64'>
8.49935529004e-05
<class 'numpy.float64'>
8.49707225201e-05
<class 'numpy.float64'>
8.49479044223e-05
<class 'numpy.float64'>
8.49250985977e-05
<class 'numpy.float64'>
8.49023050355e-05
<class 'numpy.float64'>
8.48795237266e-05
<class 'numpy.float64'>
8.4856754661e-05
<class 'numpy.float64'>
8.48339978282e-05
<class 'numpy.float64'>
8.48112532191e-05
<class 'numpy.float64'>
8.47885208236e-05
<class 'numpy.float64'>
8.47658006315e-05
<class 'numpy.float64'>
8.47430926334e-05
<class 'numpy.float64'>
8.47203968193e-05
<class 'numpy.float64'>
8.46977131795e-05
<class 'numpy.float64'>
8.46750417039e-05
<class 'numpy.float64'>
8.46523823831e-05
<class 'numpy.float64'>
8.46297352074e-05
<class 'numpy.float64'>
8.46071001664e-05
<class 'numpy.float64'>
8.45844772511e-05
<class 'numpy.float64'>
8.45618664511e-05
<class 'numpy.float64'>
8.4539267757e-05
<class 'numpy.float64'>
8.4516681159e-05
<class 'numpy.float64'>
8.44941066476e-05
<class 'numpy.float64'>
8.44715442128e-05
<class 'numpy.float64'>
8.44489938451e-05
<class 'numpy.float64'>
8.44264555346e-05
<class 'numpy.float64'>
8.44039292716e-05
<class 'numpy.float64'>
8.43814150469e-05
<class 'numpy.float64'>
8.43589128503e-05
<class 'numpy.float64'>
8.43364226725e-05
<class 'numpy.float64'>
8.43139445038e-05
<class 'numpy.float64'>
8.42914783345e-05
<class 'numpy.float64'>
8.42690241549e-05
<class 'numpy.float64'>
8.42465819558e-05
<class 'numpy.float64'>
8.4224151727e-05
<class 'numpy.float64'>
8.42017334593e-05
<class 'numpy.float64'>
8.41793271433e-05
<class 'numpy.float64'>
8.4156932769e-05
<class 'numpy.float64'>
8.4134550327e-05
<class 'numpy.float64'>
8.41121798081e-05
<class 'numpy.float64'>
8.40898212023e-05
<class 'numpy.float64'>
8.40674745003e-05
<class 'numpy.float64'>
8.40451396926e-05
<class 'numpy.float64'>
8.40228167698e-05
<class 'numpy.float64'>
8.4000505722e-05
<class 'numpy.float64'>
8.39782065403e-05
<class 'numpy.float64'>
8.39559192148e-05
<class 'numpy.float64'>
8.3933643736e-05
<class 'numpy.float64'>
8.39113800947e-05
<class 'numpy.float64'>
8.38891282814e-05
<class 'numpy.float64'>
8.38668882866e-05
<class 'numpy.float64'>
8.38446601011e-05
<class 'numpy.float64'>
8.38224437151e-05
<class 'numpy.float64'>
8.38002391194e-05
<class 'numpy.float64'>
8.37780463046e-05
<class 'numpy.float64'>
8.37558652615e-05
<class 'numpy.float64'>
8.37336959805e-05
<class 'numpy.float64'>
8.37115384525e-05
<class 'numpy.float64'>
8.36893926677e-05
<class 'numpy.float64'>
8.36672586172e-05
<class 'numpy.float64'>
8.36451362914e-05
<class 'numpy.float64'>
8.36230256812e-05
<class 'numpy.float64'>
8.3600926777e-05
<class 'numpy.float64'>
8.35788395698e-05
<class 'numpy.float64'>
8.35567640502e-05
<class 'numpy.float64'>
8.35347002087e-05
<class 'numpy.float64'>
8.35126480362e-05
<class 'numpy.float64'>
8.34906075237e-05
<class 'numpy.float64'>
8.34685786617e-05
<class 'numpy.float64'>
8.34465614409e-05
<class 'numpy.float64'>
8.3424555852e-05
<class 'numpy.float64'>
8.3402561886e-05
<class 'numpy.float64'>
8.33805795334e-05
<class 'numpy.float64'>
8.33586087859e-05
<class 'numpy.float64'>
8.33366496329e-05
<class 'numpy.float64'>
8.33147020663e-05
<class 'numpy.float64'>
8.32927660763e-05
<class 'numpy.float64'>
8.32708416542e-05
<class 'numpy.float64'>
8.32489287907e-05
<class 'numpy.float64'>
8.32270274764e-05
<class 'numpy.float64'>
8.32051377023e-05
<class 'numpy.float64'>
8.31832594596e-05
<class 'numpy.float64'>
8.31613927386e-05
<class 'numpy.float64'>
8.31395375307e-05
<class 'numpy.float64'>
8.31176938267e-05
<class 'numpy.float64'>
8.30958616175e-05
<class 'numpy.float64'>
8.30740408935e-05
<class 'numpy.float64'>
8.30522316465e-05
<class 'numpy.float64'>
8.30304338668e-05
<class 'numpy.float64'>
8.30086475457e-05
<class 'numpy.float64'>
8.29868726741e-05
<class 'numpy.float64'>
8.29651092429e-05
<class 'numpy.float64'>
8.29433572429e-05
<class 'numpy.float64'>
8.29216166654e-05
<class 'numpy.float64'>
8.28998875013e-05
<class 'numpy.float64'>
8.28781697415e-05
<class 'numpy.float64'>
8.28564633774e-05
<class 'numpy.float64'>
8.28347683992e-05
<class 'numpy.float64'>
8.28130847989e-05
<class 'numpy.float64'>
8.27914125671e-05
<class 'numpy.float64'>
8.27697516946e-05
<class 'numpy.float64'>
8.2748102173e-05
<class 'numpy.float64'>
8.27264639929e-05
<class 'numpy.float64'>
8.27048371457e-05
<class 'numpy.float64'>
8.26832216226e-05
<class 'numpy.float64'>
8.26616174143e-05
<class 'numpy.float64'>
8.26400245121e-05
<class 'numpy.float64'>
8.26184429071e-05
<class 'numpy.float64'>
8.25968725905e-05
<class 'numpy.float64'>
8.25753135533e-05
<class 'numpy.float64'>
8.25537657868e-05
<class 'numpy.float64'>
8.25322292822e-05
<class 'numpy.float64'>
8.25107040305e-05
<class 'numpy.float64'>
8.24891900232e-05
<class 'numpy.float64'>
8.24676872507e-05
<class 'numpy.float64'>
8.24461957051e-05
<class 'numpy.float64'>
8.24247153771e-05
<class 'numpy.float64'>
8.24032462581e-05
<class 'numpy.float64'>
8.23817883393e-05
<class 'numpy.float64'>
8.23603416117e-05
<class 'numpy.float64'>
8.2338906067e-05
<class 'numpy.float64'>
8.23174816961e-05
<class 'numpy.float64'>
8.22960684903e-05
<class 'numpy.float64'>
8.22746664408e-05
<class 'numpy.float64'>
8.22532755394e-05
<class 'numpy.float64'>
8.22318957768e-05
<class 'numpy.float64'>
8.22105271445e-05
<class 'numpy.float64'>
8.21891696336e-05
<class 'numpy.float64'>
8.21678232357e-05
<class 'numpy.float64'>
8.21464879422e-05
<class 'numpy.float64'>
8.21251637439e-05
<class 'numpy.float64'>
8.2103850633e-05
<class 'numpy.float64'>
8.20825485998e-05
<class 'numpy.float64'>
8.20612576364e-05
<class 'numpy.float64'>
8.20399777343e-05
<class 'numpy.float64'>
8.2018708884e-05
<class 'numpy.float64'>
8.1997451078e-05
<class 'numpy.float64'>
8.19762043065e-05
<class 'numpy.float64'>
8.19549685619e-05
<class 'numpy.float64'>
8.19337438351e-05
<class 'numpy.float64'>
8.19125301179e-05
<class 'numpy.float64'>
8.18913274014e-05
<class 'numpy.float64'>
8.1870135677e-05
<class 'numpy.float64'>
8.18489549364e-05
<class 'numpy.float64'>
8.1827785171e-05
<class 'numpy.float64'>
8.18066263723e-05
<class 'numpy.float64'>
8.17854785314e-05
<class 'numpy.float64'>
8.17643416402e-05
<class 'numpy.float64'>
8.17432156901e-05
<class 'numpy.float64'>
8.17221006725e-05
<class 'numpy.float64'>
8.1700996579e-05
<class 'numpy.float64'>
8.16799034012e-05
<class 'numpy.float64'>
8.16588211305e-05
<class 'numpy.float64'>
8.16377497584e-05
<class 'numpy.float64'>
8.16166892762e-05
<class 'numpy.float64'>
8.15956396761e-05
<class 'numpy.float64'>
8.15746009492e-05
<class 'numpy.float64'>
8.15535730872e-05
<class 'numpy.float64'>
8.15325560818e-05
<class 'numpy.float64'>
8.15115499242e-05
<class 'numpy.float64'>
8.14905546066e-05
<class 'numpy.float64'>
8.14695701199e-05
<class 'numpy.float64'>
8.14485964564e-05
<class 'numpy.float64'>
8.14276336074e-05
<class 'numpy.float64'>
8.14066815643e-05
<class 'numpy.float64'>
8.13857403191e-05
<class 'numpy.float64'>
8.1364809863e-05
<class 'numpy.float64'>
8.13438901885e-05
<class 'numpy.float64'>
8.13229812863e-05
<class 'numpy.float64'>
8.13020831489e-05
<class 'numpy.float64'>
8.12811957672e-05
<class 'numpy.float64'>
8.12603191337e-05
<class 'numpy.float64'>
8.12394532396e-05
<class 'numpy.float64'>
8.12185980767e-05
<class 'numpy.float64'>
8.11977536366e-05
<class 'numpy.float64'>
8.11769199113e-05
<class 'numpy.float64'>
8.11560968922e-05
<class 'numpy.float64'>
8.11352845712e-05
<class 'numpy.float64'>
8.11144829405e-05
<class 'numpy.float64'>
8.10936919913e-05
<class 'numpy.float64'>
8.10729117155e-05
<class 'numpy.float64'>
8.10521421047e-05
<class 'numpy.float64'>
8.10313831509e-05
<class 'numpy.float64'>
8.10106348461e-05
<class 'numpy.float64'>
8.09898971817e-05
<class 'numpy.float64'>
8.09691701497e-05
<class 'numpy.float64'>
8.09484537421e-05
<class 'numpy.float64'>
8.09277479503e-05
<class 'numpy.float64'>
8.09070527665e-05
<class 'numpy.float64'>
8.08863681823e-05
<class 'numpy.float64'>
8.08656941898e-05
<class 'numpy.float64'>
8.08450307806e-05
<class 'numpy.float64'>
8.08243779468e-05
<class 'numpy.float64'>
8.080373568e-05
<class 'numpy.float64'>
8.07831039725e-05
<class 'numpy.float64'>
8.07624828159e-05
<class 'numpy.float64'>
8.0741872202e-05
<class 'numpy.float64'>
8.07212721231e-05
<class 'numpy.float64'>
8.07006825709e-05
<class 'numpy.float64'>
8.06801035369e-05
<class 'numpy.float64'>
8.06595350138e-05
<class 'numpy.float64'>
8.06389769934e-05
<class 'numpy.float64'>
8.06184294671e-05
<class 'numpy.float64'>
8.05978924275e-05
<class 'numpy.float64'>
8.0577365866e-05
<class 'numpy.float64'>
8.05568497751e-05
<class 'numpy.float64'>
8.05363441464e-05
<class 'numpy.float64'>
8.05158489721e-05
<class 'numpy.float64'>
8.04953642443e-05
<class 'numpy.float64'>
8.04748899547e-05
<class 'numpy.float64'>
8.04544260955e-05
<class 'numpy.float64'>
8.04339726586e-05
<class 'numpy.float64'>
8.04135296361e-05
<class 'numpy.float64'>
8.039309702e-05
<class 'numpy.float64'>
8.0372674803e-05
<class 'numpy.float64'>
8.03522629759e-05
<class 'numpy.float64'>
8.03318615319e-05
<class 'numpy.float64'>
8.03114704622e-05
<class 'numpy.float64'>
8.02910897598e-05
<class 'numpy.float64'>
8.02707194162e-05
<class 'numpy.float64'>
8.02503594233e-05
<class 'numpy.float64'>
8.02300097736e-05
<class 'numpy.float64'>
8.02096704593e-05
<class 'numpy.float64'>
8.01893414723e-05
<class 'numpy.float64'>
8.01690228048e-05
<class 'numpy.float64'>
8.01487144487e-05
<class 'numpy.float64'>
8.01284163966e-05
<class 'numpy.float64'>
8.01081286402e-05
<class 'numpy.float64'>
8.0087851172e-05
<class 'numpy.float64'>
8.00675839842e-05
<class 'numpy.float64'>
8.00473270685e-05
<class 'numpy.float64'>
8.00270804176e-05
<class 'numpy.float64'>
8.00068440236e-05
<class 'numpy.float64'>
7.99866178784e-05
<class 'numpy.float64'>
7.99664019746e-05
<class 'numpy.float64'>
7.99461963044e-05
<class 'numpy.float64'>
7.99260008597e-05
<class 'numpy.float64'>
7.99058156326e-05
<class 'numpy.float64'>
7.9885640616e-05
<class 'numpy.float64'>
7.98654758017e-05
<class 'numpy.float64'>
7.98453211821e-05
<class 'numpy.float64'>
7.98251767493e-05
<class 'numpy.float64'>
7.98050424958e-05
<class 'numpy.float64'>
7.97849184136e-05
<class 'numpy.float64'>
7.97648044954e-05
<class 'numpy.float64'>
7.97447007331e-05
<class 'numpy.float64'>
7.97246071193e-05
<class 'numpy.float64'>
7.97045236461e-05
<class 'numpy.float64'>
7.96844503061e-05
<class 'numpy.float64'>
7.96643870913e-05
<class 'numpy.float64'>
7.96443339939e-05
<class 'numpy.float64'>
7.96242910068e-05
<class 'numpy.float64'>
7.96042581219e-05
<class 'numpy.float64'>
7.95842353318e-05
<class 'numpy.float64'>
7.95642226289e-05
<class 'numpy.float64'>
7.95442200051e-05
<class 'numpy.float64'>
7.95242274534e-05
<class 'numpy.float64'>
7.95042449659e-05
<class 'numpy.float64'>
7.94842725348e-05
<class 'numpy.float64'>
7.9464310153e-05
<class 'numpy.float64'>
7.94443578126e-05
<class 'numpy.float64'>
7.94244155062e-05
<class 'numpy.float64'>
7.94044832259e-05
<class 'numpy.float64'>
7.93845609642e-05
<class 'numpy.float64'>
7.93646487141e-05
<class 'numpy.float64'>
7.93447464672e-05
<class 'numpy.float64'>
7.93248542166e-05
<class 'numpy.float64'>
7.93049719544e-05
<class 'numpy.float64'>
7.92850996734e-05
<class 'numpy.float64'>
7.92652373659e-05
<class 'numpy.float64'>
7.92453850242e-05
<class 'numpy.float64'>
7.9225542641e-05
<class 'numpy.float64'>
7.92057102089e-05
<class 'numpy.float64'>
7.91858877204e-05
<class 'numpy.float64'>
7.91660751676e-05
<class 'numpy.float64'>
7.91462725434e-05
<class 'numpy.float64'>
7.91264798405e-05
<class 'numpy.float64'>
7.91066970509e-05
<class 'numpy.float64'>
7.90869241679e-05
<class 'numpy.float64'>
7.90671611834e-05
<class 'numpy.float64'>
7.90474080901e-05
<class 'numpy.float64'>
7.90276648809e-05
<class 'numpy.float64'>
Cost after iteration 4000: 0.000079
7.90079315479e-05
<class 'numpy.float64'>
7.8988208084e-05
<class 'numpy.float64'>
7.89684944818e-05
<class 'numpy.float64'>
7.89487907339e-05
<class 'numpy.float64'>
7.89290968327e-05
<class 'numpy.float64'>
7.89094127706e-05
<class 'numpy.float64'>
7.88897385409e-05
<class 'numpy.float64'>
7.88700741361e-05
<class 'numpy.float64'>
7.88504195485e-05
<class 'numpy.float64'>
7.88307747707e-05
<class 'numpy.float64'>
7.88111397956e-05
<class 'numpy.float64'>
7.87915146158e-05
<class 'numpy.float64'>
7.87718992241e-05
<class 'numpy.float64'>
7.87522936127e-05
<class 'numpy.float64'>
7.87326977748e-05
<class 'numpy.float64'>
7.87131117027e-05
<class 'numpy.float64'>
7.86935353896e-05
<class 'numpy.float64'>
7.8673968828e-05
<class 'numpy.float64'>
7.86544120103e-05
<class 'numpy.float64'>
7.86348649295e-05
<class 'numpy.float64'>
7.86153275782e-05
<class 'numpy.float64'>
7.85957999491e-05
<class 'numpy.float64'>
7.85762820351e-05
<class 'numpy.float64'>
7.85567738289e-05
<class 'numpy.float64'>
7.85372753232e-05
<class 'numpy.float64'>
7.85177865107e-05
<class 'numpy.float64'>
7.84983073846e-05
<class 'numpy.float64'>
7.8478837937e-05
<class 'numpy.float64'>
7.84593781611e-05
<class 'numpy.float64'>
7.84399280498e-05
<class 'numpy.float64'>
7.84204875955e-05
<class 'numpy.float64'>
7.84010567911e-05
<class 'numpy.float64'>
7.83816356299e-05
<class 'numpy.float64'>
7.83622241039e-05
<class 'numpy.float64'>
7.83428222069e-05
<class 'numpy.float64'>
7.83234299308e-05
<class 'numpy.float64'>
7.83040472689e-05
<class 'numpy.float64'>
7.82846742139e-05
<class 'numpy.float64'>
7.82653107589e-05
<class 'numpy.float64'>
7.82459568965e-05
<class 'numpy.float64'>
7.822661262e-05
<class 'numpy.float64'>
7.82072779215e-05
<class 'numpy.float64'>
7.81879527945e-05
<class 'numpy.float64'>
7.81686372318e-05
<class 'numpy.float64'>
7.81493312262e-05
<class 'numpy.float64'>
7.81300347706e-05
<class 'numpy.float64'>
7.81107478578e-05
<class 'numpy.float64'>
7.80914704812e-05
<class 'numpy.float64'>
7.80722026332e-05
<class 'numpy.float64'>
7.80529443067e-05
<class 'numpy.float64'>
7.80336954951e-05
<class 'numpy.float64'>
7.8014456191e-05
<class 'numpy.float64'>
7.79952263872e-05
<class 'numpy.float64'>
7.79760060774e-05
<class 'numpy.float64'>
7.79567952538e-05
<class 'numpy.float64'>
7.79375939096e-05
<class 'numpy.float64'>
7.79184020381e-05
<class 'numpy.float64'>
7.78992196317e-05
<class 'numpy.float64'>
7.78800466838e-05
<class 'numpy.float64'>
7.78608831875e-05
<class 'numpy.float64'>
7.78417291353e-05
<class 'numpy.float64'>
7.78225845207e-05
<class 'numpy.float64'>
7.78034493364e-05
<class 'numpy.float64'>
7.77843235761e-05
<class 'numpy.float64'>
7.77652072317e-05
<class 'numpy.float64'>
7.77461002971e-05
<class 'numpy.float64'>
7.77270027651e-05
<class 'numpy.float64'>
7.77079146289e-05
<class 'numpy.float64'>
7.76888358812e-05
<class 'numpy.float64'>
7.76697665156e-05
<class 'numpy.float64'>
7.76507065247e-05
<class 'numpy.float64'>
7.76316559018e-05
<class 'numpy.float64'>
7.761261464e-05
<class 'numpy.float64'>
7.75935827322e-05
<class 'numpy.float64'>
7.75745601718e-05
<class 'numpy.float64'>
7.75555469515e-05
<class 'numpy.float64'>
7.75365430652e-05
<class 'numpy.float64'>
7.7517548505e-05
<class 'numpy.float64'>
7.74985632649e-05
<class 'numpy.float64'>
7.74795873375e-05
<class 'numpy.float64'>
7.7460620716e-05
<class 'numpy.float64'>
7.74416633939e-05
<class 'numpy.float64'>
7.74227153642e-05
<class 'numpy.float64'>
7.74037766198e-05
<class 'numpy.float64'>
7.73848471542e-05
<class 'numpy.float64'>
7.73659269602e-05
<class 'numpy.float64'>
7.73470160317e-05
<class 'numpy.float64'>
7.7328114361e-05
<class 'numpy.float64'>
7.73092219418e-05
<class 'numpy.float64'>
7.72903387673e-05
<class 'numpy.float64'>
7.72714648308e-05
<class 'numpy.float64'>
7.72526001253e-05
<class 'numpy.float64'>
7.72337446438e-05
<class 'numpy.float64'>
7.72148983799e-05
<class 'numpy.float64'>
7.71960613267e-05
<class 'numpy.float64'>
7.71772334778e-05
<class 'numpy.float64'>
7.71584148259e-05
<class 'numpy.float64'>
7.71396053645e-05
<class 'numpy.float64'>
7.71208050872e-05
<class 'numpy.float64'>
7.71020139865e-05
<class 'numpy.float64'>
7.70832320564e-05
<class 'numpy.float64'>
7.70644592899e-05
<class 'numpy.float64'>
7.704569568e-05
<class 'numpy.float64'>
7.70269412208e-05
<class 'numpy.float64'>
7.70081959049e-05
<class 'numpy.float64'>
7.69894597257e-05
<class 'numpy.float64'>
7.69707326766e-05
<class 'numpy.float64'>
7.6952014751e-05
<class 'numpy.float64'>
7.6933305942e-05
<class 'numpy.float64'>
7.69146062433e-05
<class 'numpy.float64'>
7.68959156481e-05
<class 'numpy.float64'>
7.68772341496e-05
<class 'numpy.float64'>
7.68585617414e-05
<class 'numpy.float64'>
7.68398984164e-05
<class 'numpy.float64'>
7.68212441688e-05
<class 'numpy.float64'>
7.68025989909e-05
<class 'numpy.float64'>
7.67839628771e-05
<class 'numpy.float64'>
7.67653358203e-05
<class 'numpy.float64'>
7.67467178137e-05
<class 'numpy.float64'>
7.6728108851e-05
<class 'numpy.float64'>
7.67095089257e-05
<class 'numpy.float64'>
7.66909180309e-05
<class 'numpy.float64'>
7.66723361603e-05
<class 'numpy.float64'>
7.66537633071e-05
<class 'numpy.float64'>
7.66351994649e-05
<class 'numpy.float64'>
7.66166446269e-05
<class 'numpy.float64'>
7.65980987869e-05
<class 'numpy.float64'>
7.65795619382e-05
<class 'numpy.float64'>
7.65610340743e-05
<class 'numpy.float64'>
7.65425151884e-05
<class 'numpy.float64'>
7.65240052743e-05
<class 'numpy.float64'>
7.65055043254e-05
<class 'numpy.float64'>
7.64870123348e-05
<class 'numpy.float64'>
7.64685292967e-05
<class 'numpy.float64'>
7.6450055204e-05
<class 'numpy.float64'>
7.64315900505e-05
<class 'numpy.float64'>
7.64131338295e-05
<class 'numpy.float64'>
7.63946865349e-05
<class 'numpy.float64'>
7.63762481598e-05
<class 'numpy.float64'>
7.63578186981e-05
<class 'numpy.float64'>
7.63393981427e-05
<class 'numpy.float64'>
7.6320986488e-05
<class 'numpy.float64'>
7.63025837268e-05
<class 'numpy.float64'>
7.62841898531e-05
<class 'numpy.float64'>
7.62658048604e-05
<class 'numpy.float64'>
7.62474287421e-05
<class 'numpy.float64'>
7.6229061492e-05
<class 'numpy.float64'>
7.62107031033e-05
<class 'numpy.float64'>
7.619235357e-05
<class 'numpy.float64'>
7.61740128855e-05
<class 'numpy.float64'>
7.61556810434e-05
<class 'numpy.float64'>
7.61373580372e-05
<class 'numpy.float64'>
7.61190438607e-05
<class 'numpy.float64'>
7.61007385076e-05
<class 'numpy.float64'>
7.60824419713e-05
<class 'numpy.float64'>
7.60641542453e-05
<class 'numpy.float64'>
7.60458753236e-05
<class 'numpy.float64'>
7.60276051997e-05
<class 'numpy.float64'>
7.6009343867e-05
<class 'numpy.float64'>
7.59910913194e-05
<class 'numpy.float64'>
7.59728475508e-05
<class 'numpy.float64'>
7.59546125544e-05
<class 'numpy.float64'>
7.59363863238e-05
<class 'numpy.float64'>
7.59181688534e-05
<class 'numpy.float64'>
7.5899960136e-05
<class 'numpy.float64'>
7.5881760166e-05
<class 'numpy.float64'>
7.58635689367e-05
<class 'numpy.float64'>
7.58453864421e-05
<class 'numpy.float64'>
7.58272126753e-05
<class 'numpy.float64'>
7.58090476305e-05
<class 'numpy.float64'>
7.57908913013e-05
<class 'numpy.float64'>
7.57727436816e-05
<class 'numpy.float64'>
7.5754604765e-05
<class 'numpy.float64'>
7.57364745451e-05
<class 'numpy.float64'>
7.57183530157e-05
<class 'numpy.float64'>
7.57002401708e-05
<class 'numpy.float64'>
7.5682136004e-05
<class 'numpy.float64'>
7.5664040509e-05
<class 'numpy.float64'>
7.56459536795e-05
<class 'numpy.float64'>
7.56278755094e-05
<class 'numpy.float64'>
7.56098059923e-05
<class 'numpy.float64'>
7.55917451224e-05
<class 'numpy.float64'>
7.55736928932e-05
<class 'numpy.float64'>
7.55556492985e-05
<class 'numpy.float64'>
7.55376143317e-05
<class 'numpy.float64'>
7.55195879875e-05
<class 'numpy.float64'>
7.55015702592e-05
<class 'numpy.float64'>
7.54835611405e-05
<class 'numpy.float64'>
7.54655606255e-05
<class 'numpy.float64'>
7.5447568708e-05
<class 'numpy.float64'>
7.54295853818e-05
<class 'numpy.float64'>
7.54116106404e-05
<class 'numpy.float64'>
7.53936444781e-05
<class 'numpy.float64'>
7.53756868886e-05
<class 'numpy.float64'>
7.53577378659e-05
<class 'numpy.float64'>
7.53397974037e-05
<class 'numpy.float64'>
7.53218654959e-05
<class 'numpy.float64'>
7.53039421363e-05
<class 'numpy.float64'>
7.52860273189e-05
<class 'numpy.float64'>
7.52681210377e-05
<class 'numpy.float64'>
7.52502232862e-05
<class 'numpy.float64'>
7.52323340588e-05
<class 'numpy.float64'>
7.52144533489e-05
<class 'numpy.float64'>
7.51965811511e-05
<class 'numpy.float64'>
7.51787174586e-05
<class 'numpy.float64'>
7.51608622656e-05
<class 'numpy.float64'>
7.51430155661e-05
<class 'numpy.float64'>
7.51251773543e-05
<class 'numpy.float64'>
7.51073476237e-05
<class 'numpy.float64'>
7.5089526368e-05
<class 'numpy.float64'>
7.5071713582e-05
<class 'numpy.float64'>
7.50539092592e-05
<class 'numpy.float64'>
7.50361133934e-05
<class 'numpy.float64'>
7.50183259789e-05
<class 'numpy.float64'>
7.50005470092e-05
<class 'numpy.float64'>
7.49827764788e-05
<class 'numpy.float64'>
7.49650143814e-05
<class 'numpy.float64'>
7.49472607114e-05
<class 'numpy.float64'>
7.49295154622e-05
<class 'numpy.float64'>
7.49117786283e-05
<class 'numpy.float64'>
7.48940502031e-05
<class 'numpy.float64'>
7.48763301815e-05
<class 'numpy.float64'>
7.48586185567e-05
<class 'numpy.float64'>
7.48409153235e-05
<class 'numpy.float64'>
7.48232204751e-05
<class 'numpy.float64'>
7.48055340059e-05
<class 'numpy.float64'>
7.47878559102e-05
<class 'numpy.float64'>
7.4770186182e-05
<class 'numpy.float64'>
7.47525248149e-05
<class 'numpy.float64'>
7.47348718032e-05
<class 'numpy.float64'>
7.47172271415e-05
<class 'numpy.float64'>
7.46995908231e-05
<class 'numpy.float64'>
7.46819628424e-05
<class 'numpy.float64'>
7.46643431933e-05
<class 'numpy.float64'>
7.46467318703e-05
<class 'numpy.float64'>
7.46291288673e-05
<class 'numpy.float64'>
7.46115341781e-05
<class 'numpy.float64'>
7.45939477973e-05
<class 'numpy.float64'>
7.45763697186e-05
<class 'numpy.float64'>
7.45587999365e-05
<class 'numpy.float64'>
7.45412384446e-05
<class 'numpy.float64'>
7.45236852378e-05
<class 'numpy.float64'>
7.45061403096e-05
<class 'numpy.float64'>
7.44886036541e-05
<class 'numpy.float64'>
7.44710752659e-05
<class 'numpy.float64'>
7.44535551389e-05
<class 'numpy.float64'>
7.44360432673e-05
<class 'numpy.float64'>
7.4418539645e-05
<class 'numpy.float64'>
7.44010442666e-05
<class 'numpy.float64'>
7.43835571263e-05
<class 'numpy.float64'>
7.43660782177e-05
<class 'numpy.float64'>
7.43486075356e-05
<class 'numpy.float64'>
7.43311450736e-05
<class 'numpy.float64'>
7.43136908268e-05
<class 'numpy.float64'>
7.42962447883e-05
<class 'numpy.float64'>
7.42788069531e-05
<class 'numpy.float64'>
7.42613773153e-05
<class 'numpy.float64'>
7.42439558685e-05
<class 'numpy.float64'>
7.42265426079e-05
<class 'numpy.float64'>
7.42091375268e-05
<class 'numpy.float64'>
7.41917406202e-05
<class 'numpy.float64'>
7.41743518822e-05
<class 'numpy.float64'>
7.41569713061e-05
<class 'numpy.float64'>
7.41395988874e-05
<class 'numpy.float64'>
7.41222346199e-05
<class 'numpy.float64'>
7.41048784975e-05
<class 'numpy.float64'>
7.40875305149e-05
<class 'numpy.float64'>
7.40701906661e-05
<class 'numpy.float64'>
7.40528589457e-05
<class 'numpy.float64'>
7.40355353479e-05
<class 'numpy.float64'>
7.40182198669e-05
<class 'numpy.float64'>
7.40009124967e-05
<class 'numpy.float64'>
7.3983613232e-05
<class 'numpy.float64'>
7.39663220671e-05
<class 'numpy.float64'>
7.3949038996e-05
<class 'numpy.float64'>
7.39317640134e-05
<class 'numpy.float64'>
7.39144971136e-05
<class 'numpy.float64'>
7.38972382902e-05
<class 'numpy.float64'>
7.38799875385e-05
<class 'numpy.float64'>
7.38627448524e-05
<class 'numpy.float64'>
7.38455102262e-05
<class 'numpy.float64'>
7.38282836543e-05
<class 'numpy.float64'>
7.38110651309e-05
<class 'numpy.float64'>
7.37938546507e-05
<class 'numpy.float64'>
7.37766522078e-05
<class 'numpy.float64'>
7.37594577967e-05
<class 'numpy.float64'>
7.37422714116e-05
<class 'numpy.float64'>
7.37250930471e-05
<class 'numpy.float64'>
7.37079226976e-05
<class 'numpy.float64'>
7.36907603571e-05
<class 'numpy.float64'>
7.36736060205e-05
<class 'numpy.float64'>
7.3656459682e-05
<class 'numpy.float64'>
7.36393213356e-05
<class 'numpy.float64'>
7.36221909762e-05
<class 'numpy.float64'>
7.36050685984e-05
<class 'numpy.float64'>
7.35879541961e-05
<class 'numpy.float64'>
7.3570847764e-05
<class 'numpy.float64'>
7.35537492964e-05
<class 'numpy.float64'>
7.35366587878e-05
<class 'numpy.float64'>
7.35195762324e-05
<class 'numpy.float64'>
7.35025016252e-05
<class 'numpy.float64'>
7.34854349603e-05
<class 'numpy.float64'>
7.34683762321e-05
<class 'numpy.float64'>
7.34513254353e-05
<class 'numpy.float64'>
7.34342825639e-05
<class 'numpy.float64'>
7.34172476127e-05
<class 'numpy.float64'>
7.34002205767e-05
<class 'numpy.float64'>
7.33832014495e-05
<class 'numpy.float64'>
7.33661902259e-05
<class 'numpy.float64'>
7.33491869006e-05
<class 'numpy.float64'>
7.33321914676e-05
<class 'numpy.float64'>
7.33152039219e-05
<class 'numpy.float64'>
7.32982242577e-05
<class 'numpy.float64'>
7.32812524699e-05
<class 'numpy.float64'>
7.32642885527e-05
<class 'numpy.float64'>
7.32473325006e-05
<class 'numpy.float64'>
7.32303843084e-05
<class 'numpy.float64'>
7.32134439703e-05
<class 'numpy.float64'>
7.31965114809e-05
<class 'numpy.float64'>
7.31795868347e-05
<class 'numpy.float64'>
7.31626700267e-05
<class 'numpy.float64'>
7.3145761051e-05
<class 'numpy.float64'>
7.31288599022e-05
<class 'numpy.float64'>
7.31119665754e-05
<class 'numpy.float64'>
7.30950810642e-05
<class 'numpy.float64'>
7.30782033637e-05
<class 'numpy.float64'>
7.30613334686e-05
<class 'numpy.float64'>
7.30444713733e-05
<class 'numpy.float64'>
7.30276170725e-05
<class 'numpy.float64'>
7.30107705606e-05
<class 'numpy.float64'>
7.29939318327e-05
<class 'numpy.float64'>
7.2977100883e-05
<class 'numpy.float64'>
7.29602777056e-05
<class 'numpy.float64'>
7.29434622964e-05
<class 'numpy.float64'>
7.29266546487e-05
<class 'numpy.float64'>
7.29098547578e-05
<class 'numpy.float64'>
7.28930626182e-05
<class 'numpy.float64'>
7.28762782248e-05
<class 'numpy.float64'>
7.28595015719e-05
<class 'numpy.float64'>
7.28427326541e-05
<class 'numpy.float64'>
7.28259714662e-05
<class 'numpy.float64'>
7.28092180029e-05
<class 'numpy.float64'>
7.2792472259e-05
<class 'numpy.float64'>
7.27757342287e-05
<class 'numpy.float64'>
7.27590039069e-05
<class 'numpy.float64'>
7.27422812886e-05
<class 'numpy.float64'>
7.27255663679e-05
<class 'numpy.float64'>
7.27088591399e-05
<class 'numpy.float64'>
7.26921595989e-05
<class 'numpy.float64'>
7.26754677402e-05
<class 'numpy.float64'>
7.26587835579e-05
<class 'numpy.float64'>
7.26421070472e-05
<class 'numpy.float64'>
7.26254382021e-05
<class 'numpy.float64'>
7.2608777018e-05
<class 'numpy.float64'>
7.25921234896e-05
<class 'numpy.float64'>
7.25754776113e-05
<class 'numpy.float64'>
7.25588393776e-05
<class 'numpy.float64'>
7.25422087838e-05
<class 'numpy.float64'>
7.25255858245e-05
<class 'numpy.float64'>
7.2508970494e-05
<class 'numpy.float64'>
7.24923627876e-05
<class 'numpy.float64'>
7.24757626998e-05
<class 'numpy.float64'>
7.24591702253e-05
<class 'numpy.float64'>
7.24425853591e-05
<class 'numpy.float64'>
7.24260080955e-05
<class 'numpy.float64'>
7.240943843e-05
<class 'numpy.float64'>
7.23928763567e-05
<class 'numpy.float64'>
7.23763218707e-05
<class 'numpy.float64'>
7.23597749664e-05
<class 'numpy.float64'>
7.23432356393e-05
<class 'numpy.float64'>
7.23267038838e-05
<class 'numpy.float64'>
7.23101796944e-05
<class 'numpy.float64'>
7.22936630665e-05
<class 'numpy.float64'>
7.22771539943e-05
<class 'numpy.float64'>
7.2260652473e-05
<class 'numpy.float64'>
7.22441584977e-05
<class 'numpy.float64'>
7.22276720624e-05
<class 'numpy.float64'>
7.22111931628e-05
<class 'numpy.float64'>
7.2194721793e-05
<class 'numpy.float64'>
7.21782579483e-05
<class 'numpy.float64'>
7.21618016234e-05
<class 'numpy.float64'>
7.21453528128e-05
<class 'numpy.float64'>
7.21289115122e-05
<class 'numpy.float64'>
7.21124777155e-05
<class 'numpy.float64'>
7.20960514183e-05
<class 'numpy.float64'>
7.20796326151e-05
<class 'numpy.float64'>
7.20632213008e-05
<class 'numpy.float64'>
7.20468174706e-05
<class 'numpy.float64'>
7.20304211186e-05
<class 'numpy.float64'>
7.20140322404e-05
<class 'numpy.float64'>
7.19976508306e-05
<class 'numpy.float64'>
7.19812768842e-05
<class 'numpy.float64'>
7.19649103961e-05
<class 'numpy.float64'>
7.19485513612e-05
<class 'numpy.float64'>
7.19321997744e-05
<class 'numpy.float64'>
7.19158556302e-05
<class 'numpy.float64'>
7.18995189245e-05
<class 'numpy.float64'>
7.1883189651e-05
<class 'numpy.float64'>
7.18668678054e-05
<class 'numpy.float64'>
7.18505533828e-05
<class 'numpy.float64'>
7.18342463775e-05
<class 'numpy.float64'>
7.18179467846e-05
<class 'numpy.float64'>
7.18016545996e-05
<class 'numpy.float64'>
7.17853698167e-05
<class 'numpy.float64'>
7.17690924315e-05
<class 'numpy.float64'>
7.17528224383e-05
<class 'numpy.float64'>
7.17365598328e-05
<class 'numpy.float64'>
7.1720304609e-05
<class 'numpy.float64'>
7.17040567629e-05
<class 'numpy.float64'>
7.16878162887e-05
<class 'numpy.float64'>
7.1671583182e-05
<class 'numpy.float64'>
7.16553574372e-05
<class 'numpy.float64'>
7.16391390499e-05
<class 'numpy.float64'>
7.16229280146e-05
<class 'numpy.float64'>
7.16067243263e-05
<class 'numpy.float64'>
7.15905279802e-05
<class 'numpy.float64'>
7.15743389714e-05
<class 'numpy.float64'>
7.15581572949e-05
<class 'numpy.float64'>
7.15419829454e-05
<class 'numpy.float64'>
7.15258159184e-05
<class 'numpy.float64'>
7.15096562083e-05
<class 'numpy.float64'>
7.14935038107e-05
<class 'numpy.float64'>
7.14773587203e-05
<class 'numpy.float64'>
7.14612209322e-05
<class 'numpy.float64'>
7.14450904415e-05
<class 'numpy.float64'>
7.14289672434e-05
<class 'numpy.float64'>
7.14128513326e-05
<class 'numpy.float64'>
7.13967427044e-05
<class 'numpy.float64'>
7.13806413539e-05
<class 'numpy.float64'>
7.13645472762e-05
<class 'numpy.float64'>
7.1348460466e-05
<class 'numpy.float64'>
7.13323809188e-05
<class 'numpy.float64'>
7.13163086297e-05
<class 'numpy.float64'>
7.13002435932e-05
<class 'numpy.float64'>
7.12841858049e-05
<class 'numpy.float64'>
7.12681352597e-05
<class 'numpy.float64'>
7.12520919528e-05
<class 'numpy.float64'>
7.12360558795e-05
<class 'numpy.float64'>
7.12200270343e-05
<class 'numpy.float64'>
7.12040054131e-05
<class 'numpy.float64'>
7.11879910104e-05
<class 'numpy.float64'>
7.11719838212e-05
<class 'numpy.float64'>
7.11559838412e-05
<class 'numpy.float64'>
7.11399910654e-05
<class 'numpy.float64'>
7.11240054889e-05
<class 'numpy.float64'>
7.11080271065e-05
<class 'numpy.float64'>
7.10920559137e-05
<class 'numpy.float64'>
7.10760919055e-05
<class 'numpy.float64'>
7.10601350769e-05
<class 'numpy.float64'>
7.10441854234e-05
<class 'numpy.float64'>
7.10282429399e-05
<class 'numpy.float64'>
7.10123076217e-05
<class 'numpy.float64'>
7.09963794641e-05
<class 'numpy.float64'>
7.09804584618e-05
<class 'numpy.float64'>
7.09645446106e-05
<class 'numpy.float64'>
7.0948637905e-05
<class 'numpy.float64'>
7.09327383408e-05
<class 'numpy.float64'>
7.09168459128e-05
<class 'numpy.float64'>
7.09009606162e-05
<class 'numpy.float64'>
7.08850824463e-05
<class 'numpy.float64'>
7.08692113985e-05
<class 'numpy.float64'>
7.08533474678e-05
<class 'numpy.float64'>
7.08374906494e-05
<class 'numpy.float64'>
7.08216409385e-05
<class 'numpy.float64'>
7.08057983302e-05
<class 'numpy.float64'>
7.07899628203e-05
<class 'numpy.float64'>
7.07741344033e-05
<class 'numpy.float64'>
7.07583130751e-05
<class 'numpy.float64'>
7.07424988303e-05
<class 'numpy.float64'>
7.07266916643e-05
<class 'numpy.float64'>
7.07108915725e-05
<class 'numpy.float64'>
7.06950985506e-05
<class 'numpy.float64'>
7.06793125928e-05
<class 'numpy.float64'>
7.06635336953e-05
<class 'numpy.float64'>
7.06477618526e-05
<class 'numpy.float64'>
7.06319970608e-05
<class 'numpy.float64'>
7.06162393144e-05
<class 'numpy.float64'>
7.06004886092e-05
<class 'numpy.float64'>
7.05847449402e-05
<class 'numpy.float64'>
7.05690083028e-05
<class 'numpy.float64'>
7.05532786923e-05
<class 'numpy.float64'>
7.0537556104e-05
<class 'numpy.float64'>
7.05218405328e-05
<class 'numpy.float64'>
7.05061319746e-05
<class 'numpy.float64'>
7.04904304243e-05
<class 'numpy.float64'>
7.04747358776e-05
<class 'numpy.float64'>
7.04590483292e-05
<class 'numpy.float64'>
7.04433677751e-05
<class 'numpy.float64'>
7.04276942103e-05
<class 'numpy.float64'>
7.04120276298e-05
<class 'numpy.float64'>
7.03963680297e-05
<class 'numpy.float64'>
7.03807154045e-05
<class 'numpy.float64'>
7.036506975e-05
<class 'numpy.float64'>
7.03494310616e-05
<class 'numpy.float64'>
7.03337993344e-05
<class 'numpy.float64'>
7.0318174564e-05
<class 'numpy.float64'>
7.03025567455e-05
<class 'numpy.float64'>
7.02869458745e-05
<class 'numpy.float64'>
7.0271341946e-05
<class 'numpy.float64'>
7.02557449559e-05
<class 'numpy.float64'>
7.02401548992e-05
<class 'numpy.float64'>
7.02245717712e-05
<class 'numpy.float64'>
7.02089955674e-05
<class 'numpy.float64'>
7.01934262834e-05
<class 'numpy.float64'>
7.01778639143e-05
<class 'numpy.float64'>
7.01623084557e-05
<class 'numpy.float64'>
7.01467599027e-05
<class 'numpy.float64'>
7.0131218251e-05
<class 'numpy.float64'>
7.01156834957e-05
<class 'numpy.float64'>
7.01001556327e-05
<class 'numpy.float64'>
7.00846346569e-05
<class 'numpy.float64'>
7.0069120564e-05
<class 'numpy.float64'>
7.00536133493e-05
<class 'numpy.float64'>
7.00381130083e-05
<class 'numpy.float64'>
7.00226195362e-05
<class 'numpy.float64'>
7.0007132929e-05
<class 'numpy.float64'>
6.99916531815e-05
<class 'numpy.float64'>
6.99761802893e-05
<class 'numpy.float64'>
6.99607142482e-05
<class 'numpy.float64'>
6.99452550534e-05
<class 'numpy.float64'>
6.99298027e-05
<class 'numpy.float64'>
6.99143571843e-05
<class 'numpy.float64'>
6.98989185009e-05
<class 'numpy.float64'>
6.98834866457e-05
<class 'numpy.float64'>
6.9868061614e-05
<class 'numpy.float64'>
6.98526434015e-05
<class 'numpy.float64'>
6.98372320033e-05
<class 'numpy.float64'>
6.98218274158e-05
<class 'numpy.float64'>
6.98064296332e-05
<class 'numpy.float64'>
6.97910386515e-05
<class 'numpy.float64'>
6.97756544666e-05
<class 'numpy.float64'>
6.97602770739e-05
<class 'numpy.float64'>
6.97449064681e-05
<class 'numpy.float64'>
6.97295426455e-05
<class 'numpy.float64'>
6.97141856016e-05
<class 'numpy.float64'>
6.96988353316e-05
<class 'numpy.float64'>
6.9683491831e-05
<class 'numpy.float64'>
6.96681550955e-05
<class 'numpy.float64'>
6.96528251208e-05
<class 'numpy.float64'>
6.96375019019e-05
<class 'numpy.float64'>
6.96221854346e-05
<class 'numpy.float64'>
6.96068757146e-05
<class 'numpy.float64'>
6.95915727372e-05
<class 'numpy.float64'>
6.95762764981e-05
<class 'numpy.float64'>
6.95609869929e-05
<class 'numpy.float64'>
6.9545704217e-05
<class 'numpy.float64'>
6.9530428166e-05
<class 'numpy.float64'>
6.95151588353e-05
<class 'numpy.float64'>
6.94998962209e-05
<class 'numpy.float64'>
6.94846403177e-05
<class 'numpy.float64'>
6.94693911219e-05
<class 'numpy.float64'>
6.94541486289e-05
<class 'numpy.float64'>
6.94389128342e-05
<class 'numpy.float64'>
6.94236837333e-05
<class 'numpy.float64'>
6.94084613216e-05
<class 'numpy.float64'>
6.93932455955e-05
<class 'numpy.float64'>
6.93780365498e-05
<class 'numpy.float64'>
6.93628341803e-05
<class 'numpy.float64'>
6.93476384829e-05
<class 'numpy.float64'>
6.9332449453e-05
<class 'numpy.float64'>
6.9317267086e-05
<class 'numpy.float64'>
6.93020913777e-05
<class 'numpy.float64'>
6.92869223235e-05
<class 'numpy.float64'>
6.92717599197e-05
<class 'numpy.float64'>
6.92566041614e-05
<class 'numpy.float64'>
6.9241455044e-05
<class 'numpy.float64'>
6.92263125635e-05
<class 'numpy.float64'>
6.92111767157e-05
<class 'numpy.float64'>
6.91960474959e-05
<class 'numpy.float64'>
6.91809248995e-05
<class 'numpy.float64'>
6.91658089227e-05
<class 'numpy.float64'>
6.91506995611e-05
<class 'numpy.float64'>
6.91355968101e-05
<class 'numpy.float64'>
6.91205006656e-05
<class 'numpy.float64'>
6.91054111228e-05
<class 'numpy.float64'>
6.90903281777e-05
<class 'numpy.float64'>
6.90752518262e-05
<class 'numpy.float64'>
6.9060182064e-05
<class 'numpy.float64'>
6.90451188861e-05
<class 'numpy.float64'>
6.90300622884e-05
<class 'numpy.float64'>
6.90150122672e-05
<class 'numpy.float64'>
6.89999688176e-05
<class 'numpy.float64'>
6.89849319354e-05
<class 'numpy.float64'>
6.89699016166e-05
<class 'numpy.float64'>
6.89548778565e-05
<class 'numpy.float64'>
6.8939860651e-05
<class 'numpy.float64'>
6.89248499959e-05
<class 'numpy.float64'>
6.89098458867e-05
<class 'numpy.float64'>
6.88948483192e-05
<class 'numpy.float64'>
6.88798572891e-05
<class 'numpy.float64'>
6.88648727921e-05
<class 'numpy.float64'>
6.88498948239e-05
<class 'numpy.float64'>
6.88349233803e-05
<class 'numpy.float64'>
6.88199584575e-05
<class 'numpy.float64'>
6.88050000504e-05
<class 'numpy.float64'>
6.87900481552e-05
<class 'numpy.float64'>
6.87751027675e-05
<class 'numpy.float64'>
6.87601638834e-05
<class 'numpy.float64'>
6.87452314978e-05
<class 'numpy.float64'>
6.87303056075e-05
<class 'numpy.float64'>
6.87153862079e-05
<class 'numpy.float64'>
6.87004732943e-05
<class 'numpy.float64'>
6.86855668628e-05
<class 'numpy.float64'>
6.86706669095e-05
<class 'numpy.float64'>
6.86557734297e-05
<class 'numpy.float64'>
6.86408864194e-05
<class 'numpy.float64'>
6.86260058742e-05
<class 'numpy.float64'>
6.86111317902e-05
<class 'numpy.float64'>
6.85962641629e-05
<class 'numpy.float64'>
6.85814029885e-05
<class 'numpy.float64'>
6.8566548262e-05
<class 'numpy.float64'>
6.855169998e-05
<class 'numpy.float64'>
6.8536858138e-05
<class 'numpy.float64'>
6.85220227315e-05
<class 'numpy.float64'>
6.85071937571e-05
<class 'numpy.float64'>
6.84923712099e-05
<class 'numpy.float64'>
6.84775550862e-05
<class 'numpy.float64'>
6.84627453812e-05
<class 'numpy.float64'>
6.84479420914e-05
<class 'numpy.float64'>
6.84331452123e-05
<class 'numpy.float64'>
6.84183547398e-05
<class 'numpy.float64'>
6.84035706695e-05
<class 'numpy.float64'>
6.83887929977e-05
<class 'numpy.float64'>
6.83740217197e-05
<class 'numpy.float64'>
6.83592568321e-05
<class 'numpy.float64'>
6.83444983298e-05
<class 'numpy.float64'>
6.83297462093e-05
<class 'numpy.float64'>
6.83150004665e-05
<class 'numpy.float64'>
6.83002610971e-05
<class 'numpy.float64'>
6.82855280967e-05
<class 'numpy.float64'>
6.82708014613e-05
<class 'numpy.float64'>
6.82560811873e-05
<class 'numpy.float64'>
6.824136727e-05
<class 'numpy.float64'>
6.82266597053e-05
<class 'numpy.float64'>
6.82119584895e-05
<class 'numpy.float64'>
6.81972636178e-05
<class 'numpy.float64'>
6.8182575087e-05
<class 'numpy.float64'>
6.81678928923e-05
<class 'numpy.float64'>
6.81532170297e-05
<class 'numpy.float64'>
6.81385474953e-05
<class 'numpy.float64'>
6.81238842847e-05
<class 'numpy.float64'>
6.81092273942e-05
<class 'numpy.float64'>
6.80945768196e-05
<class 'numpy.float64'>
6.80799325568e-05
<class 'numpy.float64'>
6.80652946014e-05
<class 'numpy.float64'>
6.80506629495e-05
<class 'numpy.float64'>
6.80360375974e-05
<class 'numpy.float64'>
6.80214185407e-05
<class 'numpy.float64'>
6.80068057753e-05
<class 'numpy.float64'>
6.79921992971e-05
<class 'numpy.float64'>
6.79775991024e-05
<class 'numpy.float64'>
6.79630051868e-05
<class 'numpy.float64'>
6.79484175462e-05
<class 'numpy.float64'>
6.79338361768e-05
<class 'numpy.float64'>
6.79192610743e-05
<class 'numpy.float64'>
6.79046922351e-05
<class 'numpy.float64'>
6.78901296549e-05
<class 'numpy.float64'>
6.78755733293e-05
<class 'numpy.float64'>
6.78610232548e-05
<class 'numpy.float64'>
6.78464794272e-05
<class 'numpy.float64'>
6.78319418424e-05
<class 'numpy.float64'>
6.78174104962e-05
<class 'numpy.float64'>
6.78028853852e-05
<class 'numpy.float64'>
6.77883665045e-05
<class 'numpy.float64'>
6.77738538508e-05
<class 'numpy.float64'>
6.77593474204e-05
<class 'numpy.float64'>
6.77448472082e-05
<class 'numpy.float64'>
6.77303532109e-05
<class 'numpy.float64'>
6.77158654243e-05
<class 'numpy.float64'>
6.77013838447e-05
<class 'numpy.float64'>
6.76869084674e-05
<class 'numpy.float64'>
6.76724392895e-05
<class 'numpy.float64'>
6.7657976306e-05
<class 'numpy.float64'>
6.76435195135e-05
<class 'numpy.float64'>
6.7629068908e-05
<class 'numpy.float64'>
6.76146244852e-05
<class 'numpy.float64'>
6.76001862414e-05
<class 'numpy.float64'>
6.75857541724e-05
<class 'numpy.float64'>
6.75713282745e-05
<class 'numpy.float64'>
6.75569085437e-05
<class 'numpy.float64'>
6.75424949758e-05
<class 'numpy.float64'>
6.7528087567e-05
<class 'numpy.float64'>
6.75136863136e-05
<class 'numpy.float64'>
6.74992912112e-05
<class 'numpy.float64'>
6.74849022563e-05
<class 'numpy.float64'>
6.74705194447e-05
<class 'numpy.float64'>
6.74561427724e-05
<class 'numpy.float64'>
6.74417722355e-05
<class 'numpy.float64'>
6.74274078302e-05
<class 'numpy.float64'>
6.74130495528e-05
<class 'numpy.float64'>
6.73986973986e-05
<class 'numpy.float64'>
6.73843513645e-05
<class 'numpy.float64'>
6.73700114463e-05
<class 'numpy.float64'>
6.73556776397e-05
<class 'numpy.float64'>
6.73413499413e-05
<class 'numpy.float64'>
6.73270283474e-05
<class 'numpy.float64'>
6.73127128535e-05
<class 'numpy.float64'>
6.72984034558e-05
<class 'numpy.float64'>
6.72841001505e-05
<class 'numpy.float64'>
6.72698029338e-05
<class 'numpy.float64'>
6.72555118018e-05
<class 'numpy.float64'>
6.72412267507e-05
<class 'numpy.float64'>
6.72269477763e-05
<class 'numpy.float64'>
6.72126748749e-05
<class 'numpy.float64'>
6.71984080427e-05
<class 'numpy.float64'>
6.71841472756e-05
<class 'numpy.float64'>
6.71698925702e-05
<class 'numpy.float64'>
6.71556439222e-05
<class 'numpy.float64'>
6.71414013275e-05
<class 'numpy.float64'>
6.71271647829e-05
<class 'numpy.float64'>
6.71129342843e-05
<class 'numpy.float64'>
6.70987098277e-05
<class 'numpy.float64'>
6.70844914093e-05
<class 'numpy.float64'>
6.70702790252e-05
<class 'numpy.float64'>
6.70560726719e-05
<class 'numpy.float64'>
6.70418723453e-05
<class 'numpy.float64'>
6.70276780413e-05
<class 'numpy.float64'>
6.70134897567e-05
<class 'numpy.float64'>
6.6999307487e-05
<class 'numpy.float64'>
6.69851312289e-05
<class 'numpy.float64'>
6.69709609779e-05
<class 'numpy.float64'>
6.69567967311e-05
<class 'numpy.float64'>
6.69426384839e-05
<class 'numpy.float64'>
6.69284862335e-05
<class 'numpy.float64'>
6.69143399745e-05
<class 'numpy.float64'>
6.69001997044e-05
<class 'numpy.float64'>
6.68860654191e-05
<class 'numpy.float64'>
6.68719371146e-05
<class 'numpy.float64'>
6.68578147869e-05
<class 'numpy.float64'>
6.6843698433e-05
<class 'numpy.float64'>
6.6829588048e-05
<class 'numpy.float64'>
6.68154836289e-05
<class 'numpy.float64'>
6.68013851719e-05
<class 'numpy.float64'>
6.6787292673e-05
<class 'numpy.float64'>
6.67732061283e-05
<class 'numpy.float64'>
6.67591255343e-05
<class 'numpy.float64'>
6.67450508873e-05
<class 'numpy.float64'>
6.67309821831e-05
<class 'numpy.float64'>
6.67169194183e-05
<class 'numpy.float64'>
6.67028625889e-05
<class 'numpy.float64'>
6.66888116914e-05
<class 'numpy.float64'>
6.66747667219e-05
<class 'numpy.float64'>
6.66607276765e-05
<class 'numpy.float64'>
6.66466945515e-05
<class 'numpy.float64'>
6.66326673435e-05
<class 'numpy.float64'>
6.66186460486e-05
<class 'numpy.float64'>
6.66046306627e-05
<class 'numpy.float64'>
6.65906211826e-05
<class 'numpy.float64'>
6.6576617604e-05
<class 'numpy.float64'>
6.65626199235e-05
<class 'numpy.float64'>
6.65486281377e-05
<class 'numpy.float64'>
6.65346422421e-05
<class 'numpy.float64'>
6.65206622335e-05
<class 'numpy.float64'>
6.65066881081e-05
<class 'numpy.float64'>
6.64927198619e-05
<class 'numpy.float64'>
6.6478757492e-05
<class 'numpy.float64'>
6.6464800994e-05
<class 'numpy.float64'>
6.6450850364e-05
<class 'numpy.float64'>
6.64369055987e-05
<class 'numpy.float64'>
6.64229666945e-05
<class 'numpy.float64'>
6.64090336474e-05
<class 'numpy.float64'>
6.63951064539e-05
<class 'numpy.float64'>
6.63811851102e-05
<class 'numpy.float64'>
6.63672696129e-05
<class 'numpy.float64'>
6.63533599577e-05
<class 'numpy.float64'>
6.63394561413e-05
<class 'numpy.float64'>
6.63255581602e-05
<class 'numpy.float64'>
6.63116660106e-05
<class 'numpy.float64'>
6.62977796886e-05
<class 'numpy.float64'>
6.62838991907e-05
<class 'numpy.float64'>
6.62700245136e-05
<class 'numpy.float64'>
6.62561556528e-05
<class 'numpy.float64'>
6.62422926055e-05
<class 'numpy.float64'>
6.62284353673e-05
<class 'numpy.float64'>
6.6214583935e-05
<class 'numpy.float64'>
6.62007383052e-05
<class 'numpy.float64'>
6.61868984734e-05
<class 'numpy.float64'>
6.61730644369e-05
<class 'numpy.float64'>
6.61592361913e-05
<class 'numpy.float64'>
6.61454137331e-05
<class 'numpy.float64'>
6.61315970593e-05
<class 'numpy.float64'>
6.61177861657e-05
<class 'numpy.float64'>
6.61039810485e-05
<class 'numpy.float64'>
6.60901817048e-05
<class 'numpy.float64'>
6.60763881304e-05
<class 'numpy.float64'>
6.60626003216e-05
<class 'numpy.float64'>
6.6048818275e-05
<class 'numpy.float64'>
6.6035041987e-05
<class 'numpy.float64'>
6.60212714544e-05
<class 'numpy.float64'>
6.60075066727e-05
<class 'numpy.float64'>
6.59937476387e-05
<class 'numpy.float64'>
6.59799943493e-05
<class 'numpy.float64'>
6.59662468002e-05
<class 'numpy.float64'>
6.59525049881e-05
<class 'numpy.float64'>
6.59387689094e-05
<class 'numpy.float64'>
6.59250385603e-05
<class 'numpy.float64'>
6.59113139376e-05
<class 'numpy.float64'>
6.58975950376e-05
<class 'numpy.float64'>
6.58838818566e-05
<class 'numpy.float64'>
6.58701743908e-05
<class 'numpy.float64'>
6.58564726369e-05
<class 'numpy.float64'>
6.58427765916e-05
<class 'numpy.float64'>
6.58290862508e-05
<class 'numpy.float64'>
6.58154016113e-05
<class 'numpy.float64'>
6.58017226694e-05
<class 'numpy.float64'>
6.57880494215e-05
<class 'numpy.float64'>
6.57743818639e-05
<class 'numpy.float64'>
6.57607199936e-05
<class 'numpy.float64'>
6.57470638064e-05
<class 'numpy.float64'>
6.57334132992e-05
<class 'numpy.float64'>
6.57197684682e-05
<class 'numpy.float64'>
6.570612931e-05
<class 'numpy.float64'>
6.56924958211e-05
<class 'numpy.float64'>
6.56788679977e-05
<class 'numpy.float64'>
6.56652458367e-05
<class 'numpy.float64'>
6.56516293343e-05
<class 'numpy.float64'>
6.56380184867e-05
<class 'numpy.float64'>
6.56244132909e-05
<class 'numpy.float64'>
6.56108137431e-05
<class 'numpy.float64'>
6.55972198399e-05
<class 'numpy.float64'>
6.55836315774e-05
<class 'numpy.float64'>
6.55700489525e-05
<class 'numpy.float64'>
6.55564719616e-05
<class 'numpy.float64'>
6.55429006015e-05
<class 'numpy.float64'>
6.55293348679e-05
<class 'numpy.float64'>
6.55157747581e-05
<class 'numpy.float64'>
6.55022202681e-05
<class 'numpy.float64'>
6.54886713946e-05
<class 'numpy.float64'>
6.54751281341e-05
<class 'numpy.float64'>
6.54615904831e-05
<class 'numpy.float64'>
6.54480584381e-05
<class 'numpy.float64'>
6.54345319956e-05
<class 'numpy.float64'>
6.54210111523e-05
<class 'numpy.float64'>
6.54074959045e-05
<class 'numpy.float64'>
6.53939862488e-05
<class 'numpy.float64'>
6.53804821815e-05
<class 'numpy.float64'>
6.53669836995e-05
<class 'numpy.float64'>
6.53534907991e-05
<class 'numpy.float64'>
6.53400034772e-05
<class 'numpy.float64'>
6.53265217296e-05
<class 'numpy.float64'>
6.53130455538e-05
<class 'numpy.float64'>
6.52995749456e-05
<class 'numpy.float64'>
6.52861099019e-05
<class 'numpy.float64'>
6.5272650419e-05
<class 'numpy.float64'>
6.52591964934e-05
<class 'numpy.float64'>
6.52457481225e-05
<class 'numpy.float64'>
6.52323053016e-05
<class 'numpy.float64'>
6.52188680281e-05
<class 'numpy.float64'>
6.52054362985e-05
<class 'numpy.float64'>
6.51920101088e-05
<class 'numpy.float64'>
6.51785894563e-05
<class 'numpy.float64'>
6.51651743371e-05
<class 'numpy.float64'>
6.51517647483e-05
<class 'numpy.float64'>
6.51383606855e-05
<class 'numpy.float64'>
6.51249621463e-05
<class 'numpy.float64'>
6.51115691267e-05
<class 'numpy.float64'>
6.50981816236e-05
<class 'numpy.float64'>
6.50847996334e-05
<class 'numpy.float64'>
6.50714231526e-05
<class 'numpy.float64'>
6.50580521781e-05
<class 'numpy.float64'>
6.50446867063e-05
<class 'numpy.float64'>
6.50313267338e-05
<class 'numpy.float64'>
6.50179722575e-05
<class 'numpy.float64'>
6.50046232734e-05
<class 'numpy.float64'>
6.49912797785e-05
<class 'numpy.float64'>
6.49779417696e-05
<class 'numpy.float64'>
6.49646092428e-05
<class 'numpy.float64'>
6.49512821953e-05
<class 'numpy.float64'>
6.49379606233e-05
<class 'numpy.float64'>
6.49246445234e-05
<class 'numpy.float64'>
6.49113338926e-05
<class 'numpy.float64'>
6.48980287274e-05
<class 'numpy.float64'>
6.48847290241e-05
<class 'numpy.float64'>
6.48714347796e-05
<class 'numpy.float64'>
6.48581459907e-05
<class 'numpy.float64'>
6.48448626537e-05
<class 'numpy.float64'>
6.48315847655e-05
<class 'numpy.float64'>
6.48183123225e-05
<class 'numpy.float64'>
6.48050453217e-05
<class 'numpy.float64'>
6.47917837593e-05
<class 'numpy.float64'>
6.47785276324e-05
<class 'numpy.float64'>
6.47652769371e-05
<class 'numpy.float64'>
6.47520316709e-05
<class 'numpy.float64'>
6.47387918296e-05
<class 'numpy.float64'>
6.47255574104e-05
<class 'numpy.float64'>
6.471232841e-05
<class 'numpy.float64'>
6.46991048246e-05
<class 'numpy.float64'>
6.46858866512e-05
<class 'numpy.float64'>
6.46726738865e-05
<class 'numpy.float64'>
6.46594665272e-05
<class 'numpy.float64'>
6.46462645698e-05
<class 'numpy.float64'>
6.46330680108e-05
<class 'numpy.float64'>
6.46198768476e-05
<class 'numpy.float64'>
6.46066910763e-05
<class 'numpy.float64'>
6.45935106936e-05
<class 'numpy.float64'>
6.45803356965e-05
<class 'numpy.float64'>
6.45671660814e-05
<class 'numpy.float64'>
6.45540018454e-05
<class 'numpy.float64'>
6.45408429848e-05
<class 'numpy.float64'>
6.45276894964e-05
<class 'numpy.float64'>
6.45145413768e-05
<class 'numpy.float64'>
6.45013986231e-05
<class 'numpy.float64'>
6.4488261232e-05
<class 'numpy.float64'>
6.44751291996e-05
<class 'numpy.float64'>
6.44620025229e-05
<class 'numpy.float64'>
6.44488811992e-05
<class 'numpy.float64'>
6.44357652246e-05
<class 'numpy.float64'>
6.44226545958e-05
<class 'numpy.float64'>
6.440954931e-05
<class 'numpy.float64'>
6.43964493634e-05
<class 'numpy.float64'>
6.43833547531e-05
<class 'numpy.float64'>
6.43702654758e-05
<class 'numpy.float64'>
6.43571815279e-05
<class 'numpy.float64'>
6.43441029067e-05
<class 'numpy.float64'>
6.43310296085e-05
<class 'numpy.float64'>
6.43179616303e-05
<class 'numpy.float64'>
6.43048989686e-05
<class 'numpy.float64'>
6.42918416207e-05
<class 'numpy.float64'>
6.42787895825e-05
<class 'numpy.float64'>
6.42657428516e-05
<class 'numpy.float64'>
6.42527014242e-05
<class 'numpy.float64'>
6.42396652971e-05
<class 'numpy.float64'>
6.42266344676e-05
<class 'numpy.float64'>
6.42136089318e-05
<class 'numpy.float64'>
6.42005886868e-05
<class 'numpy.float64'>
6.41875737293e-05
<class 'numpy.float64'>
6.41745640564e-05
<class 'numpy.float64'>
6.41615596643e-05
<class 'numpy.float64'>
6.41485605503e-05
<class 'numpy.float64'>
6.41355667108e-05
<class 'numpy.float64'>
6.41225781427e-05
<class 'numpy.float64'>
6.4109594843e-05
<class 'numpy.float64'>
6.40966168083e-05
<class 'numpy.float64'>
6.40836440354e-05
<class 'numpy.float64'>
6.4070676521e-05
<class 'numpy.float64'>
6.40577142624e-05
<class 'numpy.float64'>
6.40447572556e-05
<class 'numpy.float64'>
6.40318054981e-05
<class 'numpy.float64'>
6.40188589864e-05
<class 'numpy.float64'>
6.40059177175e-05
<class 'numpy.float64'>
6.39929816879e-05
<class 'numpy.float64'>
6.39800508943e-05
<class 'numpy.float64'>
6.39671253343e-05
<class 'numpy.float64'>
6.39542050042e-05
<class 'numpy.float64'>
6.39412899006e-05
<class 'numpy.float64'>
6.39283800206e-05
<class 'numpy.float64'>
6.39154753614e-05
<class 'numpy.float64'>
6.39025759191e-05
<class 'numpy.float64'>
6.38896816908e-05
<class 'numpy.float64'>
6.38767926738e-05
<class 'numpy.float64'>
6.38639088642e-05
<class 'numpy.float64'>
6.38510302592e-05
<class 'numpy.float64'>
6.38381568558e-05
<class 'numpy.float64'>
6.38252886506e-05
<class 'numpy.float64'>
6.38124256407e-05
<class 'numpy.float64'>
6.37995678228e-05
<class 'numpy.float64'>
6.37867151936e-05
<class 'numpy.float64'>
6.37738677503e-05
<class 'numpy.float64'>
6.37610254894e-05
<class 'numpy.float64'>
6.37481884078e-05
<class 'numpy.float64'>
6.37353565027e-05
<class 'numpy.float64'>
6.37225297707e-05
<class 'numpy.float64'>
6.37097082089e-05
<class 'numpy.float64'>
6.36968918137e-05
<class 'numpy.float64'>
6.36840805824e-05
<class 'numpy.float64'>
6.36712745118e-05
<class 'numpy.float64'>
6.36584735989e-05
<class 'numpy.float64'>
6.36456778401e-05
<class 'numpy.float64'>
6.36328872328e-05
<class 'numpy.float64'>
6.36201017737e-05
<class 'numpy.float64'>
6.36073214598e-05
<class 'numpy.float64'>
6.35945462877e-05
<class 'numpy.float64'>
6.35817762546e-05
<class 'numpy.float64'>
6.3569011357e-05
<class 'numpy.float64'>
6.35562515923e-05
<class 'numpy.float64'>
6.35434969572e-05
<class 'numpy.float64'>
6.35307474484e-05
<class 'numpy.float64'>
6.35180030633e-05
<class 'numpy.float64'>
6.35052637984e-05
<class 'numpy.float64'>
6.34925296504e-05
<class 'numpy.float64'>
6.34798006168e-05
<class 'numpy.float64'>
6.3467076694e-05
<class 'numpy.float64'>
6.34543578793e-05
<class 'numpy.float64'>
6.34416441699e-05
<class 'numpy.float64'>
6.34289355618e-05
<class 'numpy.float64'>
6.34162320528e-05
<class 'numpy.float64'>
6.34035336389e-05
<class 'numpy.float64'>
6.33908403181e-05
<class 'numpy.float64'>
6.33781520868e-05
<class 'numpy.float64'>
6.33654689418e-05
<class 'numpy.float64'>
6.33527908803e-05
<class 'numpy.float64'>
6.33401178992e-05
<class 'numpy.float64'>
6.33274499953e-05
<class 'numpy.float64'>
6.33147871657e-05
<class 'numpy.float64'>
6.33021294072e-05
<class 'numpy.float64'>
6.32894767171e-05
<class 'numpy.float64'>
6.3276829092e-05
<class 'numpy.float64'>
6.32641865289e-05
<class 'numpy.float64'>
6.3251549025e-05
<class 'numpy.float64'>
6.32389165769e-05
<class 'numpy.float64'>
Cost after iteration 5000: 0.000063
6.32262891818e-05
<class 'numpy.float64'>
6.32136668365e-05
<class 'numpy.float64'>
6.32010495385e-05
<class 'numpy.float64'>
6.3188437284e-05
<class 'numpy.float64'>
6.31758300704e-05
<class 'numpy.float64'>
6.31632278946e-05
<class 'numpy.float64'>
6.31506307536e-05
<class 'numpy.float64'>
6.31380386443e-05
<class 'numpy.float64'>
6.31254515636e-05
<class 'numpy.float64'>
6.31128695091e-05
<class 'numpy.float64'>
6.31002924771e-05
<class 'numpy.float64'>
6.30877204651e-05
<class 'numpy.float64'>
6.30751534694e-05
<class 'numpy.float64'>
6.30625914873e-05
<class 'numpy.float64'>
6.30500345161e-05
<class 'numpy.float64'>
6.30374825529e-05
<class 'numpy.float64'>
6.30249355943e-05
<class 'numpy.float64'>
6.30123936372e-05
<class 'numpy.float64'>
6.29998566787e-05
<class 'numpy.float64'>
6.29873247163e-05
<class 'numpy.float64'>
6.29747977464e-05
<class 'numpy.float64'>
6.29622757664e-05
<class 'numpy.float64'>
6.29497587731e-05
<class 'numpy.float64'>
6.29372467638e-05
<class 'numpy.float64'>
6.29247397351e-05
<class 'numpy.float64'>
6.29122376843e-05
<class 'numpy.float64'>
6.28997406083e-05
<class 'numpy.float64'>
6.28872485043e-05
<class 'numpy.float64'>
6.28747613693e-05
<class 'numpy.float64'>
6.28622792002e-05
<class 'numpy.float64'>
6.2849801994e-05
<class 'numpy.float64'>
6.2837329748e-05
<class 'numpy.float64'>
6.28248624591e-05
<class 'numpy.float64'>
6.2812400124e-05
<class 'numpy.float64'>
6.27999427402e-05
<class 'numpy.float64'>
6.27874903051e-05
<class 'numpy.float64'>
6.27750428147e-05
<class 'numpy.float64'>
6.27626002668e-05
<class 'numpy.float64'>
6.27501626581e-05
<class 'numpy.float64'>
6.27377299863e-05
<class 'numpy.float64'>
6.27253022478e-05
<class 'numpy.float64'>
6.27128794397e-05
<class 'numpy.float64'>
6.27004615593e-05
<class 'numpy.float64'>
6.26880486037e-05
<class 'numpy.float64'>
6.26756405698e-05
<class 'numpy.float64'>
6.26632374547e-05
<class 'numpy.float64'>
6.26508392554e-05
<class 'numpy.float64'>
6.26384459693e-05
<class 'numpy.float64'>
6.2626057593e-05
<class 'numpy.float64'>
6.2613674124e-05
<class 'numpy.float64'>
6.26012955594e-05
<class 'numpy.float64'>
6.25889218957e-05
<class 'numpy.float64'>
6.25765531306e-05
<class 'numpy.float64'>
6.25641892609e-05
<class 'numpy.float64'>
6.25518302839e-05
<class 'numpy.float64'>
6.25394761967e-05
<class 'numpy.float64'>
6.25271269962e-05
<class 'numpy.float64'>
6.25147826792e-05
<class 'numpy.float64'>
6.25024432435e-05
<class 'numpy.float64'>
6.24901086859e-05
<class 'numpy.float64'>
6.24777790033e-05
<class 'numpy.float64'>
6.24654541932e-05
<class 'numpy.float64'>
6.24531342524e-05
<class 'numpy.float64'>
6.24408191781e-05
<class 'numpy.float64'>
6.24285089672e-05
<class 'numpy.float64'>
6.24162036176e-05
<class 'numpy.float64'>
6.24039031253e-05
<class 'numpy.float64'>
6.23916074883e-05
<class 'numpy.float64'>
6.23793167032e-05
<class 'numpy.float64'>
6.23670307675e-05
<class 'numpy.float64'>
6.23547496782e-05
<class 'numpy.float64'>
6.23424734325e-05
<class 'numpy.float64'>
6.23302020271e-05
<class 'numpy.float64'>
6.23179354597e-05
<class 'numpy.float64'>
6.23056737272e-05
<class 'numpy.float64'>
6.22934168267e-05
<class 'numpy.float64'>
6.22811647551e-05
<class 'numpy.float64'>
6.22689175101e-05
<class 'numpy.float64'>
6.22566750888e-05
<class 'numpy.float64'>
6.22444374876e-05
<class 'numpy.float64'>
6.22322047045e-05
<class 'numpy.float64'>
6.22199767365e-05
<class 'numpy.float64'>
6.22077535803e-05
<class 'numpy.float64'>
6.21955352332e-05
<class 'numpy.float64'>
6.2183321693e-05
<class 'numpy.float64'>
6.2171112956e-05
<class 'numpy.float64'>
6.21589090198e-05
<class 'numpy.float64'>
6.21467098817e-05
<class 'numpy.float64'>
6.21345155385e-05
<class 'numpy.float64'>
6.21223259877e-05
<class 'numpy.float64'>
6.21101412259e-05
<class 'numpy.float64'>
6.20979612511e-05
<class 'numpy.float64'>
6.20857860597e-05
<class 'numpy.float64'>
6.20736156495e-05
<class 'numpy.float64'>
6.20614500176e-05
<class 'numpy.float64'>
6.20492891609e-05
<class 'numpy.float64'>
6.20371330766e-05
<class 'numpy.float64'>
6.20249817618e-05
<class 'numpy.float64'>
6.20128352141e-05
<class 'numpy.float64'>
6.20006934306e-05
<class 'numpy.float64'>
6.19885564084e-05
<class 'numpy.float64'>
6.19764241445e-05
<class 'numpy.float64'>
6.19642966364e-05
<class 'numpy.float64'>
6.19521738811e-05
<class 'numpy.float64'>
6.19400558759e-05
<class 'numpy.float64'>
6.19279426178e-05
<class 'numpy.float64'>
6.19158341043e-05
<class 'numpy.float64'>
6.19037303326e-05
<class 'numpy.float64'>
6.18916312999e-05
<class 'numpy.float64'>
6.1879537003e-05
<class 'numpy.float64'>
6.186744744e-05
<class 'numpy.float64'>
6.1855362607e-05
<class 'numpy.float64'>
6.18432825022e-05
<class 'numpy.float64'>
6.18312071223e-05
<class 'numpy.float64'>
6.18191364645e-05
<class 'numpy.float64'>
6.18070705264e-05
<class 'numpy.float64'>
6.17950093047e-05
<class 'numpy.float64'>
6.17829527973e-05
<class 'numpy.float64'>
6.17709010007e-05
<class 'numpy.float64'>
6.17588539129e-05
<class 'numpy.float64'>
6.17468115306e-05
<class 'numpy.float64'>
6.17347738512e-05
<class 'numpy.float64'>
6.1722740872e-05
<class 'numpy.float64'>
6.17107125901e-05
<class 'numpy.float64'>
6.16986890027e-05
<class 'numpy.float64'>
6.16866701074e-05
<class 'numpy.float64'>
6.16746559011e-05
<class 'numpy.float64'>
6.16626463813e-05
<class 'numpy.float64'>
6.16506415452e-05
<class 'numpy.float64'>
6.16386413895e-05
<class 'numpy.float64'>
6.16266459127e-05
<class 'numpy.float64'>
6.16146551107e-05
<class 'numpy.float64'>
6.16026689819e-05
<class 'numpy.float64'>
6.15906875228e-05
<class 'numpy.float64'>
6.1578710731e-05
<class 'numpy.float64'>
6.15667386037e-05
<class 'numpy.float64'>
6.1554771138e-05
<class 'numpy.float64'>
6.15428083317e-05
<class 'numpy.float64'>
6.15308501811e-05
<class 'numpy.float64'>
6.15188966848e-05
<class 'numpy.float64'>
6.15069478391e-05
<class 'numpy.float64'>
6.14950036417e-05
<class 'numpy.float64'>
6.14830640896e-05
<class 'numpy.float64'>
6.14711291803e-05
<class 'numpy.float64'>
6.14591989112e-05
<class 'numpy.float64'>
6.14472732792e-05
<class 'numpy.float64'>
6.14353522819e-05
<class 'numpy.float64'>
6.14234359166e-05
<class 'numpy.float64'>
6.14115241806e-05
<class 'numpy.float64'>
6.1399617071e-05
<class 'numpy.float64'>
6.13877145853e-05
<class 'numpy.float64'>
6.13758167207e-05
<class 'numpy.float64'>
6.13639234746e-05
<class 'numpy.float64'>
6.1352034844e-05
<class 'numpy.float64'>
6.13401508265e-05
<class 'numpy.float64'>
6.13282714197e-05
<class 'numpy.float64'>
6.13163966205e-05
<class 'numpy.float64'>
6.13045264263e-05
<class 'numpy.float64'>
6.12926608343e-05
<class 'numpy.float64'>
6.12807998422e-05
<class 'numpy.float64'>
6.12689434468e-05
<class 'numpy.float64'>
6.1257091646e-05
<class 'numpy.float64'>
6.12452444366e-05
<class 'numpy.float64'>
6.12334018163e-05
<class 'numpy.float64'>
6.12215637822e-05
<class 'numpy.float64'>
6.12097303316e-05
<class 'numpy.float64'>
6.11979014622e-05
<class 'numpy.float64'>
6.1186077171e-05
<class 'numpy.float64'>
6.11742574554e-05
<class 'numpy.float64'>
6.11624423128e-05
<class 'numpy.float64'>
6.11506317406e-05
<class 'numpy.float64'>
6.11388257362e-05
<class 'numpy.float64'>
6.11270242969e-05
<class 'numpy.float64'>
6.11152274196e-05
<class 'numpy.float64'>
6.11034351021e-05
<class 'numpy.float64'>
6.10916473419e-05
<class 'numpy.float64'>
6.10798641362e-05
<class 'numpy.float64'>
6.10680854821e-05
<class 'numpy.float64'>
6.10563113772e-05
<class 'numpy.float64'>
6.10445418189e-05
<class 'numpy.float64'>
6.10327768046e-05
<class 'numpy.float64'>
6.10210163316e-05
<class 'numpy.float64'>
6.1009260397e-05
<class 'numpy.float64'>
6.09975089984e-05
<class 'numpy.float64'>
6.09857621333e-05
<class 'numpy.float64'>
6.0974019799e-05
<class 'numpy.float64'>
6.09622819928e-05
<class 'numpy.float64'>
6.09505487119e-05
<class 'numpy.float64'>
6.09388199542e-05
<class 'numpy.float64'>
6.09270957165e-05
<class 'numpy.float64'>
6.09153759966e-05
<class 'numpy.float64'>
6.09036607919e-05
<class 'numpy.float64'>
6.08919500996e-05
<class 'numpy.float64'>
6.08802439169e-05
<class 'numpy.float64'>
6.08685422415e-05
<class 'numpy.float64'>
6.08568450709e-05
<class 'numpy.float64'>
6.08451524021e-05
<class 'numpy.float64'>
6.08334642326e-05
<class 'numpy.float64'>
6.08217805602e-05
<class 'numpy.float64'>
6.08101013819e-05
<class 'numpy.float64'>
6.07984266956e-05
<class 'numpy.float64'>
6.07867564977e-05
<class 'numpy.float64'>
6.07750907866e-05
<class 'numpy.float64'>
6.07634295595e-05
<class 'numpy.float64'>
6.07517728134e-05
<class 'numpy.float64'>
6.0740120546e-05
<class 'numpy.float64'>
6.07284727548e-05
<class 'numpy.float64'>
6.07168294372e-05
<class 'numpy.float64'>
6.07051905906e-05
<class 'numpy.float64'>
6.06935562121e-05
<class 'numpy.float64'>
6.06819262996e-05
<class 'numpy.float64'>
6.06703008502e-05
<class 'numpy.float64'>
6.06586798617e-05
<class 'numpy.float64'>
6.06470633309e-05
<class 'numpy.float64'>
6.06354512558e-05
<class 'numpy.float64'>
6.06238436337e-05
<class 'numpy.float64'>
6.06122404621e-05
<class 'numpy.float64'>
6.06006417381e-05
<class 'numpy.float64'>
6.05890474594e-05
<class 'numpy.float64'>
6.05774576236e-05
<class 'numpy.float64'>
6.05658722278e-05
<class 'numpy.float64'>
6.05542912694e-05
<class 'numpy.float64'>
6.05427147465e-05
<class 'numpy.float64'>
6.05311426559e-05
<class 'numpy.float64'>
6.05195749952e-05
<class 'numpy.float64'>
6.05080117621e-05
<class 'numpy.float64'>
6.04964529539e-05
<class 'numpy.float64'>
6.04848985677e-05
<class 'numpy.float64'>
6.04733486014e-05
<class 'numpy.float64'>
6.04618030527e-05
<class 'numpy.float64'>
6.04502619185e-05
<class 'numpy.float64'>
6.04387251962e-05
<class 'numpy.float64'>
6.04271928838e-05
<class 'numpy.float64'>
6.04156649787e-05
<class 'numpy.float64'>
6.04041414779e-05
<class 'numpy.float64'>
6.03926223795e-05
<class 'numpy.float64'>
6.03811076803e-05
<class 'numpy.float64'>
6.03695973784e-05
<class 'numpy.float64'>
6.03580914709e-05
<class 'numpy.float64'>
6.03465899554e-05
<class 'numpy.float64'>
6.03350928293e-05
<class 'numpy.float64'>
6.03236000903e-05
<class 'numpy.float64'>
6.03121117356e-05
<class 'numpy.float64'>
6.03006277629e-05
<class 'numpy.float64'>
6.02891481697e-05
<class 'numpy.float64'>
6.02776729532e-05
<class 'numpy.float64'>
6.02662021115e-05
<class 'numpy.float64'>
6.02547356414e-05
<class 'numpy.float64'>
6.02432735408e-05
<class 'numpy.float64'>
6.02318158069e-05
<class 'numpy.float64'>
6.02203624379e-05
<class 'numpy.float64'>
6.02089134305e-05
<class 'numpy.float64'>
6.01974687826e-05
<class 'numpy.float64'>
6.01860284918e-05
<class 'numpy.float64'>
6.01745925551e-05
<class 'numpy.float64'>
6.01631609707e-05
<class 'numpy.float64'>
6.01517337355e-05
<class 'numpy.float64'>
6.01403108475e-05
<class 'numpy.float64'>
6.01288923041e-05
<class 'numpy.float64'>
6.01174781026e-05
<class 'numpy.float64'>
6.01060682407e-05
<class 'numpy.float64'>
6.00946627156e-05
<class 'numpy.float64'>
6.00832615256e-05
<class 'numpy.float64'>
6.00718646674e-05
<class 'numpy.float64'>
6.00604721388e-05
<class 'numpy.float64'>
6.00490839379e-05
<class 'numpy.float64'>
6.00377000613e-05
<class 'numpy.float64'>
6.0026320507e-05
<class 'numpy.float64'>
6.00149452725e-05
<class 'numpy.float64'>
6.00035743556e-05
<class 'numpy.float64'>
5.99922077532e-05
<class 'numpy.float64'>
5.99808454635e-05
<class 'numpy.float64'>
5.99694874837e-05
<class 'numpy.float64'>
5.99581338114e-05
<class 'numpy.float64'>
5.99467844439e-05
<class 'numpy.float64'>
5.99354393794e-05
<class 'numpy.float64'>
5.99240986151e-05
<class 'numpy.float64'>
5.99127621484e-05
<class 'numpy.float64'>
5.9901429977e-05
<class 'numpy.float64'>
5.98901020985e-05
<class 'numpy.float64'>
5.98787785101e-05
<class 'numpy.float64'>
5.98674592097e-05
<class 'numpy.float64'>
5.9856144195e-05
<class 'numpy.float64'>
5.98448334635e-05
<class 'numpy.float64'>
5.98335270126e-05
<class 'numpy.float64'>
5.98222248397e-05
<class 'numpy.float64'>
5.98109269426e-05
<class 'numpy.float64'>
5.97996333191e-05
<class 'numpy.float64'>
5.97883439664e-05
<class 'numpy.float64'>
5.97770588823e-05
<class 'numpy.float64'>
5.97657780642e-05
<class 'numpy.float64'>
5.97545015097e-05
<class 'numpy.float64'>
5.97432292166e-05
<class 'numpy.float64'>
5.97319611822e-05
<class 'numpy.float64'>
5.97206974043e-05
<class 'numpy.float64'>
5.97094378805e-05
<class 'numpy.float64'>
5.96981826082e-05
<class 'numpy.float64'>
5.96869315851e-05
<class 'numpy.float64'>
5.96756848088e-05
<class 'numpy.float64'>
5.96644422769e-05
<class 'numpy.float64'>
5.96532039869e-05
<class 'numpy.float64'>
5.96419699365e-05
<class 'numpy.float64'>
5.96307401235e-05
<class 'numpy.float64'>
5.9619514545e-05
<class 'numpy.float64'>
5.9608293199e-05
<class 'numpy.float64'>
5.95970760827e-05
<class 'numpy.float64'>
5.95858631943e-05
<class 'numpy.float64'>
5.95746545311e-05
<class 'numpy.float64'>
5.95634500905e-05
<class 'numpy.float64'>
5.95522498704e-05
<class 'numpy.float64'>
5.95410538684e-05
<class 'numpy.float64'>
5.95298620821e-05
<class 'numpy.float64'>
5.9518674509e-05
<class 'numpy.float64'>
5.95074911467e-05
<class 'numpy.float64'>
5.94963119931e-05
<class 'numpy.float64'>
5.94851370455e-05
<class 'numpy.float64'>
5.94739663016e-05
<class 'numpy.float64'>
5.94627997592e-05
<class 'numpy.float64'>
5.94516374156e-05
<class 'numpy.float64'>
5.94404792689e-05
<class 'numpy.float64'>
5.94293253164e-05
<class 'numpy.float64'>
5.9418175556e-05
<class 'numpy.float64'>
5.9407029985e-05
<class 'numpy.float64'>
5.9395888601e-05
<class 'numpy.float64'>
5.93847514019e-05
<class 'numpy.float64'>
5.93736183856e-05
<class 'numpy.float64'>
5.9362489549e-05
<class 'numpy.float64'>
5.93513648904e-05
<class 'numpy.float64'>
5.9340244407e-05
<class 'numpy.float64'>
5.93291280966e-05
<class 'numpy.float64'>
5.93180159571e-05
<class 'numpy.float64'>
5.9306907986e-05
<class 'numpy.float64'>
5.92958041807e-05
<class 'numpy.float64'>
5.92847045392e-05
<class 'numpy.float64'>
5.92736090588e-05
<class 'numpy.float64'>
5.92625177377e-05
<class 'numpy.float64'>
5.9251430573e-05
<class 'numpy.float64'>
5.92403475627e-05
<class 'numpy.float64'>
5.92292687043e-05
<class 'numpy.float64'>
5.92181939956e-05
<class 'numpy.float64'>
5.92071234342e-05
<class 'numpy.float64'>
5.91960570177e-05
<class 'numpy.float64'>
5.91849947439e-05
<class 'numpy.float64'>
5.91739366105e-05
<class 'numpy.float64'>
5.91628826148e-05
<class 'numpy.float64'>
5.9151832755e-05
<class 'numpy.float64'>
5.91407870282e-05
<class 'numpy.float64'>
5.91297454329e-05
<class 'numpy.float64'>
5.91187079658e-05
<class 'numpy.float64'>
5.91076746252e-05
<class 'numpy.float64'>
5.90966454091e-05
<class 'numpy.float64'>
5.90856203142e-05
<class 'numpy.float64'>
5.90745993391e-05
<class 'numpy.float64'>
5.90635824811e-05
<class 'numpy.float64'>
5.90525697378e-05
<class 'numpy.float64'>
5.9041561107e-05
<class 'numpy.float64'>
5.90305565863e-05
<class 'numpy.float64'>
5.90195561737e-05
<class 'numpy.float64'>
5.90085598667e-05
<class 'numpy.float64'>
5.89975676629e-05
<class 'numpy.float64'>
5.89865795603e-05
<class 'numpy.float64'>
5.89755955564e-05
<class 'numpy.float64'>
5.89646156489e-05
<class 'numpy.float64'>
5.89536398353e-05
<class 'numpy.float64'>
5.89426681136e-05
<class 'numpy.float64'>
5.89317004816e-05
<class 'numpy.float64'>
5.89207369367e-05
<class 'numpy.float64'>
5.8909777477e-05
<class 'numpy.float64'>
5.88988220998e-05
<class 'numpy.float64'>
5.88878708029e-05
<class 'numpy.float64'>
5.88769235846e-05
<class 'numpy.float64'>
5.88659804416e-05
<class 'numpy.float64'>
5.88550413724e-05
<class 'numpy.float64'>
5.88441063746e-05
<class 'numpy.float64'>
5.88331754457e-05
<class 'numpy.float64'>
5.88222485832e-05
<class 'numpy.float64'>
5.88113257858e-05
<class 'numpy.float64'>
5.88004070504e-05
<class 'numpy.float64'>
5.87894923747e-05
<class 'numpy.float64'>
5.87785817568e-05
<class 'numpy.float64'>
5.87676751945e-05
<class 'numpy.float64'>
5.87567726854e-05
<class 'numpy.float64'>
5.87458742268e-05
<class 'numpy.float64'>
5.87349798171e-05
<class 'numpy.float64'>
5.87240894538e-05
<class 'numpy.float64'>
5.87132031344e-05
<class 'numpy.float64'>
5.87023208568e-05
<class 'numpy.float64'>
5.86914426191e-05
<class 'numpy.float64'>
5.86805684187e-05
<class 'numpy.float64'>
5.86696982535e-05
<class 'numpy.float64'>
5.8658832121e-05
<class 'numpy.float64'>
5.8647970019e-05
<class 'numpy.float64'>
5.86371119457e-05
<class 'numpy.float64'>
5.86262578985e-05
<class 'numpy.float64'>
5.86154078749e-05
<class 'numpy.float64'>
5.86045618731e-05
<class 'numpy.float64'>
5.85937198908e-05
<class 'numpy.float64'>
5.85828819255e-05
<class 'numpy.float64'>
5.85720479754e-05
<class 'numpy.float64'>
5.8561218038e-05
<class 'numpy.float64'>
5.85503921108e-05
<class 'numpy.float64'>
5.8539570192e-05
<class 'numpy.float64'>
5.85287522794e-05
<class 'numpy.float64'>
5.85179383704e-05
<class 'numpy.float64'>
5.85071284632e-05
<class 'numpy.float64'>
5.84963225553e-05
<class 'numpy.float64'>
5.84855206442e-05
<class 'numpy.float64'>
5.84747227285e-05
<class 'numpy.float64'>
5.84639288052e-05
<class 'numpy.float64'>
5.84531388725e-05
<class 'numpy.float64'>
5.8442352928e-05
<class 'numpy.float64'>
5.84315709697e-05
<class 'numpy.float64'>
5.84207929952e-05
<class 'numpy.float64'>
5.84100190022e-05
<class 'numpy.float64'>
5.83992489886e-05
<class 'numpy.float64'>
5.83884829525e-05
<class 'numpy.float64'>
5.83777208912e-05
<class 'numpy.float64'>
5.83669628029e-05
<class 'numpy.float64'>
5.8356208685e-05
<class 'numpy.float64'>
5.83454585357e-05
<class 'numpy.float64'>
5.83347123525e-05
<class 'numpy.float64'>
5.83239701335e-05
<class 'numpy.float64'>
5.83132318762e-05
<class 'numpy.float64'>
5.83024975788e-05
<class 'numpy.float64'>
5.82917672384e-05
<class 'numpy.float64'>
5.82810408536e-05
<class 'numpy.float64'>
5.82703184217e-05
<class 'numpy.float64'>
5.82595999409e-05
<class 'numpy.float64'>
5.82488854084e-05
<class 'numpy.float64'>
5.82381748228e-05
<class 'numpy.float64'>
5.82274681814e-05
<class 'numpy.float64'>
5.82167654822e-05
<class 'numpy.float64'>
5.82060667228e-05
<class 'numpy.float64'>
5.81953719015e-05
<class 'numpy.float64'>
5.81846810157e-05
<class 'numpy.float64'>
5.81739940632e-05
<class 'numpy.float64'>
5.81633110424e-05
<class 'numpy.float64'>
5.81526319502e-05
<class 'numpy.float64'>
5.81419567854e-05
<class 'numpy.float64'>
5.8131285545e-05
<class 'numpy.float64'>
5.81206182273e-05
<class 'numpy.float64'>
5.81099548301e-05
<class 'numpy.float64'>
5.80992953514e-05
<class 'numpy.float64'>
5.80886397886e-05
<class 'numpy.float64'>
5.80779881397e-05
<class 'numpy.float64'>
5.80673404026e-05
<class 'numpy.float64'>
5.80566965753e-05
<class 'numpy.float64'>
5.80460566554e-05
<class 'numpy.float64'>
5.80354206409e-05
<class 'numpy.float64'>
5.80247885296e-05
<class 'numpy.float64'>
5.80141603193e-05
<class 'numpy.float64'>
5.80035360079e-05
<class 'numpy.float64'>
5.79929155936e-05
<class 'numpy.float64'>
5.79822990734e-05
<class 'numpy.float64'>
5.79716864455e-05
<class 'numpy.float64'>
5.79610777084e-05
<class 'numpy.float64'>
5.79504728594e-05
<class 'numpy.float64'>
5.79398718963e-05
<class 'numpy.float64'>
5.79292748173e-05
<class 'numpy.float64'>
5.79186816198e-05
<class 'numpy.float64'>
5.79080923021e-05
<class 'numpy.float64'>
5.78975068617e-05
<class 'numpy.float64'>
5.78869252968e-05
<class 'numpy.float64'>
5.78763476053e-05
<class 'numpy.float64'>
5.7865773785e-05
<class 'numpy.float64'>
5.78552038334e-05
<class 'numpy.float64'>
5.78446377489e-05
<class 'numpy.float64'>
5.7834075529e-05
<class 'numpy.float64'>
5.78235171716e-05
<class 'numpy.float64'>
5.7812962675e-05
<class 'numpy.float64'>
5.78024120365e-05
<class 'numpy.float64'>
5.77918652543e-05
<class 'numpy.float64'>
5.77813223266e-05
<class 'numpy.float64'>
5.77707832505e-05
<class 'numpy.float64'>
5.77602480243e-05
<class 'numpy.float64'>
5.77497166461e-05
<class 'numpy.float64'>
5.77391891137e-05
<class 'numpy.float64'>
5.77286654249e-05
<class 'numpy.float64'>
5.77181455775e-05
<class 'numpy.float64'>
5.77076295696e-05
<class 'numpy.float64'>
5.76971173987e-05
<class 'numpy.float64'>
5.7686609063e-05
<class 'numpy.float64'>
5.76761045607e-05
<class 'numpy.float64'>
5.76656038891e-05
<class 'numpy.float64'>
5.76551070466e-05
<class 'numpy.float64'>
5.76446140306e-05
<class 'numpy.float64'>
5.76341248396e-05
<class 'numpy.float64'>
5.7623639471e-05
<class 'numpy.float64'>
5.76131579231e-05
<class 'numpy.float64'>
5.76026801937e-05
<class 'numpy.float64'>
5.75922062803e-05
<class 'numpy.float64'>
5.75817361815e-05
<class 'numpy.float64'>
5.75712698944e-05
<class 'numpy.float64'>
5.7560807418e-05
<class 'numpy.float64'>
5.75503487493e-05
<class 'numpy.float64'>
5.75398938865e-05
<class 'numpy.float64'>
5.75294428275e-05
<class 'numpy.float64'>
5.75189955704e-05
<class 'numpy.float64'>
5.75085521129e-05
<class 'numpy.float64'>
5.74981124532e-05
<class 'numpy.float64'>
5.74876765888e-05
<class 'numpy.float64'>
5.74772445178e-05
<class 'numpy.float64'>
5.74668162386e-05
<class 'numpy.float64'>
5.74563917483e-05
<class 'numpy.float64'>
5.74459710455e-05
<class 'numpy.float64'>
5.74355541281e-05
<class 'numpy.float64'>
5.74251409936e-05
<class 'numpy.float64'>
5.74147316404e-05
<class 'numpy.float64'>
5.74043260658e-05
<class 'numpy.float64'>
5.73939242683e-05
<class 'numpy.float64'>
5.73835262461e-05
<class 'numpy.float64'>
5.73731319963e-05
<class 'numpy.float64'>
5.73627415176e-05
<class 'numpy.float64'>
5.73523548074e-05
<class 'numpy.float64'>
5.73419718641e-05
<class 'numpy.float64'>
5.73315926854e-05
<class 'numpy.float64'>
5.73212172694e-05
<class 'numpy.float64'>
5.73108456135e-05
<class 'numpy.float64'>
5.73004777167e-05
<class 'numpy.float64'>
5.72901135758e-05
<class 'numpy.float64'>
5.72797531894e-05
<class 'numpy.float64'>
5.72693965558e-05
<class 'numpy.float64'>
5.72590436722e-05
<class 'numpy.float64'>
5.72486945371e-05
<class 'numpy.float64'>
5.72383491481e-05
<class 'numpy.float64'>
5.72280075034e-05
<class 'numpy.float64'>
5.72176696008e-05
<class 'numpy.float64'>
5.72073354384e-05
<class 'numpy.float64'>
5.71970050143e-05
<class 'numpy.float64'>
5.7186678326e-05
<class 'numpy.float64'>
5.7176355372e-05
<class 'numpy.float64'>
5.71660361499e-05
<class 'numpy.float64'>
5.71557206578e-05
<class 'numpy.float64'>
5.71454088939e-05
<class 'numpy.float64'>
5.71351008557e-05
<class 'numpy.float64'>
5.71247965418e-05
<class 'numpy.float64'>
5.71144959495e-05
<class 'numpy.float64'>
5.71041990772e-05
<class 'numpy.float64'>
5.7093905923e-05
<class 'numpy.float64'>
5.70836164844e-05
<class 'numpy.float64'>
5.70733307599e-05
<class 'numpy.float64'>
5.70630487471e-05
<class 'numpy.float64'>
5.70527704443e-05
<class 'numpy.float64'>
5.7042495849e-05
<class 'numpy.float64'>
5.70322249601e-05
<class 'numpy.float64'>
5.70219577748e-05
<class 'numpy.float64'>
5.70116942914e-05
<class 'numpy.float64'>
5.70014345075e-05
<class 'numpy.float64'>
5.69911784216e-05
<class 'numpy.float64'>
5.69809260314e-05
<class 'numpy.float64'>
5.69706773353e-05
<class 'numpy.float64'>
5.69604323307e-05
<class 'numpy.float64'>
5.69501910165e-05
<class 'numpy.float64'>
5.69399533896e-05
<class 'numpy.float64'>
5.69297194488e-05
<class 'numpy.float64'>
5.69194891916e-05
<class 'numpy.float64'>
5.69092626165e-05
<class 'numpy.float64'>
5.68990397211e-05
<class 'numpy.float64'>
5.68888205037e-05
<class 'numpy.float64'>
5.6878604962e-05
<class 'numpy.float64'>
5.68683930945e-05
<class 'numpy.float64'>
5.68581848989e-05
<class 'numpy.float64'>
5.68479803732e-05
<class 'numpy.float64'>
5.68377795154e-05
<class 'numpy.float64'>
5.68275823237e-05
<class 'numpy.float64'>
5.68173887959e-05
<class 'numpy.float64'>
5.68071989301e-05
<class 'numpy.float64'>
5.67970127244e-05
<class 'numpy.float64'>
5.67868301769e-05
<class 'numpy.float64'>
5.67766512855e-05
<class 'numpy.float64'>
5.67664760483e-05
<class 'numpy.float64'>
5.67563044632e-05
<class 'numpy.float64'>
5.67461365282e-05
<class 'numpy.float64'>
5.67359722415e-05
<class 'numpy.float64'>
5.67258116012e-05
<class 'numpy.float64'>
5.67156546053e-05
<class 'numpy.float64'>
5.67055012516e-05
<class 'numpy.float64'>
5.66953515384e-05
<class 'numpy.float64'>
5.66852054635e-05
<class 'numpy.float64'>
5.66750630253e-05
<class 'numpy.float64'>
5.66649242215e-05
<class 'numpy.float64'>
5.665478905e-05
<class 'numpy.float64'>
5.66446575095e-05
<class 'numpy.float64'>
5.66345295976e-05
<class 'numpy.float64'>
5.66244053126e-05
<class 'numpy.float64'>
5.66142846518e-05
<class 'numpy.float64'>
5.66041676143e-05
<class 'numpy.float64'>
5.65940541977e-05
<class 'numpy.float64'>
5.65839443996e-05
<class 'numpy.float64'>
5.65738382189e-05
<class 'numpy.float64'>
5.65637356531e-05
<class 'numpy.float64'>
5.65536367005e-05
<class 'numpy.float64'>
5.65435413589e-05
<class 'numpy.float64'>
5.65334496268e-05
<class 'numpy.float64'>
5.65233615015e-05
<class 'numpy.float64'>
5.65132769821e-05
<class 'numpy.float64'>
5.65031960659e-05
<class 'numpy.float64'>
5.64931187511e-05
<class 'numpy.float64'>
5.6483045036e-05
<class 'numpy.float64'>
5.64729749181e-05
<class 'numpy.float64'>
5.64629083965e-05
<class 'numpy.float64'>
5.64528454685e-05
<class 'numpy.float64'>
5.64427861321e-05
<class 'numpy.float64'>
5.6432730386e-05
<class 'numpy.float64'>
5.64226782279e-05
<class 'numpy.float64'>
5.64126296554e-05
<class 'numpy.float64'>
5.64025846674e-05
<class 'numpy.float64'>
5.63925432615e-05
<class 'numpy.float64'>
5.63825054361e-05
<class 'numpy.float64'>
5.63724711891e-05
<class 'numpy.float64'>
5.63624405186e-05
<class 'numpy.float64'>
5.63524134225e-05
<class 'numpy.float64'>
5.63423898994e-05
<class 'numpy.float64'>
5.63323699469e-05
<class 'numpy.float64'>
5.6322353563e-05
<class 'numpy.float64'>
5.63123407463e-05
<class 'numpy.float64'>
5.63023314946e-05
<class 'numpy.float64'>
5.62923258062e-05
<class 'numpy.float64'>
5.62823236789e-05
<class 'numpy.float64'>
5.62723251108e-05
<class 'numpy.float64'>
5.62623301002e-05
<class 'numpy.float64'>
5.62523386451e-05
<class 'numpy.float64'>
5.62423507435e-05
<class 'numpy.float64'>
5.62323663941e-05
<class 'numpy.float64'>
5.62223855942e-05
<class 'numpy.float64'>
5.62124083422e-05
<class 'numpy.float64'>
5.62024346363e-05
<class 'numpy.float64'>
5.61924644748e-05
<class 'numpy.float64'>
5.61824978552e-05
<class 'numpy.float64'>
5.61725347763e-05
<class 'numpy.float64'>
5.61625752358e-05
<class 'numpy.float64'>
5.61526192318e-05
<class 'numpy.float64'>
5.61426667624e-05
<class 'numpy.float64'>
5.61327178261e-05
<class 'numpy.float64'>
5.61227724206e-05
<class 'numpy.float64'>
5.61128305444e-05
<class 'numpy.float64'>
5.61028921951e-05
<class 'numpy.float64'>
5.60929573713e-05
<class 'numpy.float64'>
5.60830260708e-05
<class 'numpy.float64'>
5.60730982922e-05
<class 'numpy.float64'>
5.60631740331e-05
<class 'numpy.float64'>
5.60532532916e-05
<class 'numpy.float64'>
5.60433360664e-05
<class 'numpy.float64'>
5.60334223552e-05
<class 'numpy.float64'>
5.60235121561e-05
<class 'numpy.float64'>
5.60136054672e-05
<class 'numpy.float64'>
5.60037022871e-05
<class 'numpy.float64'>
5.59938026134e-05
<class 'numpy.float64'>
5.59839064446e-05
<class 'numpy.float64'>
5.59740137787e-05
<class 'numpy.float64'>
5.59641246136e-05
<class 'numpy.float64'>
5.59542389479e-05
<class 'numpy.float64'>
5.59443567793e-05
<class 'numpy.float64'>
5.59344781062e-05
<class 'numpy.float64'>
5.5924602927e-05
<class 'numpy.float64'>
5.59147312391e-05
<class 'numpy.float64'>
5.59048630414e-05
<class 'numpy.float64'>
5.58949983315e-05
<class 'numpy.float64'>
5.5885137108e-05
<class 'numpy.float64'>
5.58752793688e-05
<class 'numpy.float64'>
5.58654251123e-05
<class 'numpy.float64'>
5.58555743359e-05
<class 'numpy.float64'>
5.58457270388e-05
<class 'numpy.float64'>
5.58358832183e-05
<class 'numpy.float64'>
5.58260428731e-05
<class 'numpy.float64'>
5.58162060014e-05
<class 'numpy.float64'>
5.58063726007e-05
<class 'numpy.float64'>
5.57965426698e-05
<class 'numpy.float64'>
5.57867162068e-05
<class 'numpy.float64'>
5.57768932093e-05
<class 'numpy.float64'>
5.57670736763e-05
<class 'numpy.float64'>
5.57572576056e-05
<class 'numpy.float64'>
5.5747444995e-05
<class 'numpy.float64'>
5.57376358433e-05
<class 'numpy.float64'>
5.57278301479e-05
<class 'numpy.float64'>
5.5718027908e-05
<class 'numpy.float64'>
5.57082291209e-05
<class 'numpy.float64'>
5.56984337848e-05
<class 'numpy.float64'>
5.56886418987e-05
<class 'numpy.float64'>
5.56788534602e-05
<class 'numpy.float64'>
5.56690684671e-05
<class 'numpy.float64'>
5.56592869183e-05
<class 'numpy.float64'>
5.56495088115e-05
<class 'numpy.float64'>
5.56397341452e-05
<class 'numpy.float64'>
5.56299629173e-05
<class 'numpy.float64'>
5.56201951263e-05
<class 'numpy.float64'>
5.561043077e-05
<class 'numpy.float64'>
5.56006698472e-05
<class 'numpy.float64'>
5.55909123553e-05
<class 'numpy.float64'>
5.5581158293e-05
<class 'numpy.float64'>
5.55714076585e-05
<class 'numpy.float64'>
5.55616604495e-05
<class 'numpy.float64'>
5.55519166647e-05
<class 'numpy.float64'>
5.55421763022e-05
<class 'numpy.float64'>
5.55324393602e-05
<class 'numpy.float64'>
5.55227058368e-05
<class 'numpy.float64'>
5.55129757303e-05
<class 'numpy.float64'>
5.55032490388e-05
<class 'numpy.float64'>
5.54935257606e-05
<class 'numpy.float64'>
5.54838058938e-05
<class 'numpy.float64'>
5.54740894367e-05
<class 'numpy.float64'>
5.54643763871e-05
<class 'numpy.float64'>
5.5454666744e-05
<class 'numpy.float64'>
5.54449605049e-05
<class 'numpy.float64'>
5.54352576684e-05
<class 'numpy.float64'>
5.54255582327e-05
<class 'numpy.float64'>
5.54158621956e-05
<class 'numpy.float64'>
5.54061695556e-05
<class 'numpy.float64'>
5.53964803112e-05
<class 'numpy.float64'>
5.53867944604e-05
<class 'numpy.float64'>
5.53771120012e-05
<class 'numpy.float64'>
5.53674329321e-05
<class 'numpy.float64'>
5.53577572509e-05
<class 'numpy.float64'>
5.53480849561e-05
<class 'numpy.float64'>
5.53384160464e-05
<class 'numpy.float64'>
5.53287505191e-05
<class 'numpy.float64'>
5.53190883731e-05
<class 'numpy.float64'>
5.53094296064e-05
<class 'numpy.float64'>
5.52997742172e-05
<class 'numpy.float64'>
5.52901222038e-05
<class 'numpy.float64'>
5.52804735642e-05
<class 'numpy.float64'>
5.52708282968e-05
<class 'numpy.float64'>
5.52611864001e-05
<class 'numpy.float64'>
5.52515478719e-05
<class 'numpy.float64'>
5.52419127107e-05
<class 'numpy.float64'>
5.52322809145e-05
<class 'numpy.float64'>
5.52226524819e-05
<class 'numpy.float64'>
5.52130274108e-05
<class 'numpy.float64'>
5.52034056997e-05
<class 'numpy.float64'>
5.51937873466e-05
<class 'numpy.float64'>
5.51841723498e-05
<class 'numpy.float64'>
5.51745607079e-05
<class 'numpy.float64'>
5.51649524184e-05
<class 'numpy.float64'>
5.51553474802e-05
<class 'numpy.float64'>
5.51457458911e-05
<class 'numpy.float64'>
5.51361476497e-05
<class 'numpy.float64'>
5.5126552754e-05
<class 'numpy.float64'>
5.51169612026e-05
<class 'numpy.float64'>
5.51073729936e-05
<class 'numpy.float64'>
5.50977881248e-05
<class 'numpy.float64'>
5.50882065952e-05
<class 'numpy.float64'>
5.50786284024e-05
<class 'numpy.float64'>
5.50690535452e-05
<class 'numpy.float64'>
5.50594820213e-05
<class 'numpy.float64'>
5.50499138294e-05
<class 'numpy.float64'>
5.50403489676e-05
<class 'numpy.float64'>
5.50307874341e-05
<class 'numpy.float64'>
5.50212292273e-05
<class 'numpy.float64'>
5.50116743453e-05
<class 'numpy.float64'>
5.50021227866e-05
<class 'numpy.float64'>
5.49925745492e-05
<class 'numpy.float64'>
5.49830296315e-05
<class 'numpy.float64'>
5.49734880318e-05
<class 'numpy.float64'>
5.49639497484e-05
<class 'numpy.float64'>
5.49544147793e-05
<class 'numpy.float64'>
5.49448831231e-05
<class 'numpy.float64'>
5.49353547778e-05
<class 'numpy.float64'>
5.49258297421e-05
<class 'numpy.float64'>
5.49163080139e-05
<class 'numpy.float64'>
5.49067895915e-05
<class 'numpy.float64'>
5.48972744732e-05
<class 'numpy.float64'>
5.48877626574e-05
<class 'numpy.float64'>
5.48782541423e-05
<class 'numpy.float64'>
5.48687489262e-05
<class 'numpy.float64'>
5.48592470072e-05
<class 'numpy.float64'>
5.48497483838e-05
<class 'numpy.float64'>
5.48402530542e-05
<class 'numpy.float64'>
5.4830761017e-05
<class 'numpy.float64'>
5.482127227e-05
<class 'numpy.float64'>
5.48117868118e-05
<class 'numpy.float64'>
5.48023046404e-05
<class 'numpy.float64'>
5.47928257543e-05
<class 'numpy.float64'>
5.4783350152e-05
<class 'numpy.float64'>
5.47738778313e-05
<class 'numpy.float64'>
5.47644087909e-05
<class 'numpy.float64'>
5.4754943029e-05
<class 'numpy.float64'>
5.47454805436e-05
<class 'numpy.float64'>
5.47360213335e-05
<class 'numpy.float64'>
5.47265653968e-05
<class 'numpy.float64'>
5.47171127318e-05
<class 'numpy.float64'>
5.47076633366e-05
<class 'numpy.float64'>
5.46982172094e-05
<class 'numpy.float64'>
5.46887743488e-05
<class 'numpy.float64'>
5.46793347533e-05
<class 'numpy.float64'>
5.46698984207e-05
<class 'numpy.float64'>
5.46604653499e-05
<class 'numpy.float64'>
5.46510355386e-05
<class 'numpy.float64'>
5.46416089857e-05
<class 'numpy.float64'>
5.4632185689e-05
<class 'numpy.float64'>
5.46227656469e-05
<class 'numpy.float64'>
5.4613348858e-05
<class 'numpy.float64'>
5.46039353203e-05
<class 'numpy.float64'>
5.45945250324e-05
<class 'numpy.float64'>
5.45851179923e-05
<class 'numpy.float64'>
5.45757141987e-05
<class 'numpy.float64'>
5.45663136494e-05
<class 'numpy.float64'>
5.45569163433e-05
<class 'numpy.float64'>
5.45475222783e-05
<class 'numpy.float64'>
5.45381314531e-05
<class 'numpy.float64'>
5.45287438656e-05
<class 'numpy.float64'>
5.45193595143e-05
<class 'numpy.float64'>
5.45099783976e-05
<class 'numpy.float64'>
5.45006005133e-05
<class 'numpy.float64'>
5.4491225861e-05
<class 'numpy.float64'>
5.44818544378e-05
<class 'numpy.float64'>
5.44724862427e-05
<class 'numpy.float64'>
5.44631212734e-05
<class 'numpy.float64'>
5.44537595289e-05
<class 'numpy.float64'>
5.44444010072e-05
<class 'numpy.float64'>
5.44350457065e-05
<class 'numpy.float64'>
5.44256936256e-05
<class 'numpy.float64'>
5.44163447625e-05
<class 'numpy.float64'>
5.44069991155e-05
<class 'numpy.float64'>
5.4397656683e-05
<class 'numpy.float64'>
5.43883174636e-05
<class 'numpy.float64'>
5.43789814553e-05
<class 'numpy.float64'>
5.43696486567e-05
<class 'numpy.float64'>
5.4360319066e-05
<class 'numpy.float64'>
5.43509926811e-05
<class 'numpy.float64'>
5.43416695012e-05
<class 'numpy.float64'>
5.43323495243e-05
<class 'numpy.float64'>
5.43230327483e-05
<class 'numpy.float64'>
5.43137191725e-05
<class 'numpy.float64'>
5.43044087943e-05
<class 'numpy.float64'>
5.42951016128e-05
<class 'numpy.float64'>
5.42857976256e-05
<class 'numpy.float64'>
5.42764968316e-05
<class 'numpy.float64'>
5.42671992291e-05
<class 'numpy.float64'>
5.42579048166e-05
<class 'numpy.float64'>
5.42486135919e-05
<class 'numpy.float64'>
5.42393255535e-05
<class 'numpy.float64'>
5.42300407004e-05
<class 'numpy.float64'>
5.42207590303e-05
<class 'numpy.float64'>
5.42114805419e-05
<class 'numpy.float64'>
5.42022052332e-05
<class 'numpy.float64'>
5.41929331028e-05
<class 'numpy.float64'>
5.41836641494e-05
<class 'numpy.float64'>
5.41743983706e-05
<class 'numpy.float64'>
5.41651357654e-05
<class 'numpy.float64'>
5.41558763322e-05
<class 'numpy.float64'>
5.41466200689e-05
<class 'numpy.float64'>
5.41373669742e-05
<class 'numpy.float64'>
5.41281170462e-05
<class 'numpy.float64'>
5.41188702837e-05
<class 'numpy.float64'>
5.41096266845e-05
<class 'numpy.float64'>
5.41003862476e-05
<class 'numpy.float64'>
5.4091148971e-05
<class 'numpy.float64'>
5.40819148529e-05
<class 'numpy.float64'>
5.40726838924e-05
<class 'numpy.float64'>
5.40634560872e-05
<class 'numpy.float64'>
5.40542314357e-05
<class 'numpy.float64'>
5.40450099368e-05
<class 'numpy.float64'>
5.40357915886e-05
<class 'numpy.float64'>
5.40265763893e-05
<class 'numpy.float64'>
5.40173643371e-05
<class 'numpy.float64'>
5.40081554312e-05
<class 'numpy.float64'>
5.39989496696e-05
<class 'numpy.float64'>
5.39897470502e-05
<class 'numpy.float64'>
5.3980547572e-05
<class 'numpy.float64'>
5.39713512331e-05
<class 'numpy.float64'>
5.39621580319e-05
<class 'numpy.float64'>
5.39529679674e-05
<class 'numpy.float64'>
5.39437810367e-05
<class 'numpy.float64'>
5.39345972394e-05
<class 'numpy.float64'>
5.3925416573e-05
<class 'numpy.float64'>
5.39162390371e-05
<class 'numpy.float64'>
5.39070646289e-05
<class 'numpy.float64'>
5.38978933473e-05
<class 'numpy.float64'>
5.38887251906e-05
<class 'numpy.float64'>
5.38795601571e-05
<class 'numpy.float64'>
5.38703982457e-05
<class 'numpy.float64'>
5.38612394543e-05
<class 'numpy.float64'>
5.38520837815e-05
<class 'numpy.float64'>
5.38429312259e-05
<class 'numpy.float64'>
5.38337817853e-05
<class 'numpy.float64'>
5.38246354585e-05
<class 'numpy.float64'>
5.38154922442e-05
<class 'numpy.float64'>
5.38063521403e-05
<class 'numpy.float64'>
5.37972151455e-05
<class 'numpy.float64'>
5.37880812581e-05
<class 'numpy.float64'>
5.37789504765e-05
<class 'numpy.float64'>
5.37698227992e-05
<class 'numpy.float64'>
5.37606982245e-05
<class 'numpy.float64'>
5.3751576751e-05
<class 'numpy.float64'>
5.3742458377e-05
<class 'numpy.float64'>
5.37333431008e-05
<class 'numpy.float64'>
5.37242309212e-05
<class 'numpy.float64'>
5.37151218362e-05
<class 'numpy.float64'>
5.37060158445e-05
<class 'numpy.float64'>
5.36969129443e-05
<class 'numpy.float64'>
5.36878131344e-05
<class 'numpy.float64'>
5.36787164126e-05
<class 'numpy.float64'>
5.36696227778e-05
<class 'numpy.float64'>
5.36605322287e-05
<class 'numpy.float64'>
5.36514447631e-05
<class 'numpy.float64'>
5.36423603795e-05
<class 'numpy.float64'>
5.36332790767e-05
<class 'numpy.float64'>
5.3624200853e-05
<class 'numpy.float64'>
5.36151257066e-05
<class 'numpy.float64'>
5.36060536362e-05
<class 'numpy.float64'>
5.35969846402e-05
<class 'numpy.float64'>
5.35879187169e-05
<class 'numpy.float64'>
5.3578855865e-05
<class 'numpy.float64'>
5.35697960828e-05
<class 'numpy.float64'>
5.35607393684e-05
<class 'numpy.float64'>
5.35516857208e-05
<class 'numpy.float64'>
5.3542635138e-05
<class 'numpy.float64'>
5.35335876189e-05
<class 'numpy.float64'>
5.35245431616e-05
<class 'numpy.float64'>
5.35155017644e-05
<class 'numpy.float64'>
5.35064634263e-05
<class 'numpy.float64'>
5.34974281451e-05
<class 'numpy.float64'>
5.34883959199e-05
<class 'numpy.float64'>
5.34793667484e-05
<class 'numpy.float64'>
5.34703406296e-05
<class 'numpy.float64'>
5.3461317562e-05
<class 'numpy.float64'>
5.34522975435e-05
<class 'numpy.float64'>
5.34432805732e-05
<class 'numpy.float64'>
5.34342666492e-05
<class 'numpy.float64'>
5.342525577e-05
<class 'numpy.float64'>
5.34162479341e-05
<class 'numpy.float64'>
5.34072431402e-05
<class 'numpy.float64'>
5.33982413862e-05
<class 'numpy.float64'>
5.3389242671e-05
<class 'numpy.float64'>
5.33802469928e-05
<class 'numpy.float64'>
5.337125435e-05
<class 'numpy.float64'>
5.33622647417e-05
<class 'numpy.float64'>
5.33532781658e-05
<class 'numpy.float64'>
5.33442946208e-05
<class 'numpy.float64'>
5.3335314105e-05
<class 'numpy.float64'>
5.33263366172e-05
<class 'numpy.float64'>
5.33173621559e-05
<class 'numpy.float64'>
5.33083907193e-05
<class 'numpy.float64'>
5.32994223063e-05
<class 'numpy.float64'>
5.32904569151e-05
<class 'numpy.float64'>
5.32814945438e-05
<class 'numpy.float64'>
5.32725351913e-05
<class 'numpy.float64'>
5.32635788562e-05
<class 'numpy.float64'>
5.32546255367e-05
<class 'numpy.float64'>
5.32456752312e-05
<class 'numpy.float64'>
5.32367279387e-05
<class 'numpy.float64'>
5.32277836571e-05
<class 'numpy.float64'>
5.3218842385e-05
<class 'numpy.float64'>
5.3209904121e-05
<class 'numpy.float64'>
5.32009688639e-05
<class 'numpy.float64'>
5.31920366112e-05
<class 'numpy.float64'>
5.31831073624e-05
<class 'numpy.float64'>
5.31741811154e-05
<class 'numpy.float64'>
5.31652578692e-05
<class 'numpy.float64'>
5.31563376217e-05
<class 'numpy.float64'>
5.31474203716e-05
<class 'numpy.float64'>
5.31385061177e-05
<class 'numpy.float64'>
5.31295948585e-05
<class 'numpy.float64'>
5.31206865917e-05
<class 'numpy.float64'>
5.31117813165e-05
<class 'numpy.float64'>
5.31028790311e-05
<class 'numpy.float64'>
5.30939797345e-05
<class 'numpy.float64'>
5.30850834244e-05
<class 'numpy.float64'>
5.30761900998e-05
<class 'numpy.float64'>
5.3067299759e-05
<class 'numpy.float64'>
5.30584124008e-05
<class 'numpy.float64'>
5.30495280234e-05
<class 'numpy.float64'>
5.30406466254e-05
<class 'numpy.float64'>
5.30317682051e-05
<class 'numpy.float64'>
5.30228927613e-05
<class 'numpy.float64'>
5.30140202926e-05
<class 'numpy.float64'>
5.3005150797e-05
<class 'numpy.float64'>
5.29962842738e-05
<class 'numpy.float64'>
5.29874207203e-05
<class 'numpy.float64'>
5.29785601362e-05
<class 'numpy.float64'>
5.29697025195e-05
<class 'numpy.float64'>
5.29608478686e-05
<class 'numpy.float64'>
5.2951996182e-05
<class 'numpy.float64'>
5.29431474586e-05
<class 'numpy.float64'>
5.29343016967e-05
<class 'numpy.float64'>
5.29254588945e-05
<class 'numpy.float64'>
5.29166190509e-05
<class 'numpy.float64'>
5.29077821643e-05
<class 'numpy.float64'>
5.28989482334e-05
<class 'numpy.float64'>
5.28901172561e-05
<class 'numpy.float64'>
5.28812892319e-05
<class 'numpy.float64'>
5.28724641586e-05
<class 'numpy.float64'>
5.28636420349e-05
<class 'numpy.float64'>
5.28548228588e-05
<class 'numpy.float64'>
5.284600663e-05
<class 'numpy.float64'>
5.28371933462e-05
<class 'numpy.float64'>
5.28283830059e-05
<class 'numpy.float64'>
5.28195756079e-05
<class 'numpy.float64'>
5.28107711504e-05
<class 'numpy.float64'>
5.28019696325e-05
<class 'numpy.float64'>
5.27931710523e-05
<class 'numpy.float64'>
5.27843754081e-05
<class 'numpy.float64'>
5.27755826993e-05
<class 'numpy.float64'>
5.27667929235e-05
<class 'numpy.float64'>
5.27580060796e-05
<class 'numpy.float64'>
5.27492221664e-05
<class 'numpy.float64'>
5.27404411818e-05
<class 'numpy.float64'>
5.2731663125e-05
<class 'numpy.float64'>
5.2722887994e-05
<class 'numpy.float64'>
5.27141157876e-05
<class 'numpy.float64'>
Cost after iteration 6000: 0.000053
5.27053465046e-05
<class 'numpy.float64'>
5.26965801428e-05
<class 'numpy.float64'>
5.26878167016e-05
<class 'numpy.float64'>
5.26790561788e-05
<class 'numpy.float64'>
5.26702985736e-05
<class 'numpy.float64'>
5.2661543884e-05
<class 'numpy.float64'>
5.26527921089e-05
<class 'numpy.float64'>
5.26440432464e-05
<class 'numpy.float64'>
5.26352972956e-05
<class 'numpy.float64'>
5.26265542548e-05
<class 'numpy.float64'>
5.26178141225e-05
<class 'numpy.float64'>
5.26090768973e-05
<class 'numpy.float64'>
5.26003425776e-05
<class 'numpy.float64'>
5.25916111622e-05
<class 'numpy.float64'>
5.25828826494e-05
<class 'numpy.float64'>
5.2574157038e-05
<class 'numpy.float64'>
5.25654343266e-05
<class 'numpy.float64'>
5.25567145134e-05
<class 'numpy.float64'>
5.25479975972e-05
<class 'numpy.float64'>
5.25392835765e-05
<class 'numpy.float64'>
5.25305724498e-05
<class 'numpy.float64'>
5.25218642158e-05
<class 'numpy.float64'>
5.2513158873e-05
<class 'numpy.float64'>
5.25044564196e-05
<class 'numpy.float64'>
5.24957568548e-05
<class 'numpy.float64'>
5.24870601768e-05
<class 'numpy.float64'>
5.24783663843e-05
<class 'numpy.float64'>
5.24696754758e-05
<class 'numpy.float64'>
5.24609874498e-05
<class 'numpy.float64'>
5.24523023048e-05
<class 'numpy.float64'>
5.24436200397e-05
<class 'numpy.float64'>
5.24349406526e-05
<class 'numpy.float64'>
5.24262641423e-05
<class 'numpy.float64'>
5.24175905075e-05
<class 'numpy.float64'>
5.24089197468e-05
<class 'numpy.float64'>
5.24002518585e-05
<class 'numpy.float64'>
5.23915868411e-05
<class 'numpy.float64'>
5.23829246936e-05
<class 'numpy.float64'>
5.23742654142e-05
<class 'numpy.float64'>
5.23656090018e-05
<class 'numpy.float64'>
5.23569554547e-05
<class 'numpy.float64'>
5.23483047716e-05
<class 'numpy.float64'>
5.23396569509e-05
<class 'numpy.float64'>
5.23310119916e-05
<class 'numpy.float64'>
5.23223698918e-05
<class 'numpy.float64'>
5.23137306504e-05
<class 'numpy.float64'>
5.23050942656e-05
<class 'numpy.float64'>
5.22964607365e-05
<class 'numpy.float64'>
5.22878300615e-05
<class 'numpy.float64'>
5.22792022389e-05
<class 'numpy.float64'>
5.22705772676e-05
<class 'numpy.float64'>
5.22619551461e-05
<class 'numpy.float64'>
5.2253335873e-05
<class 'numpy.float64'>
5.22447194468e-05
<class 'numpy.float64'>
5.2236105866e-05
<class 'numpy.float64'>
5.22274951295e-05
<class 'numpy.float64'>
5.22188872357e-05
<class 'numpy.float64'>
5.22102821834e-05
<class 'numpy.float64'>
5.22016799709e-05
<class 'numpy.float64'>
5.21930805969e-05
<class 'numpy.float64'>
5.21844840597e-05
<class 'numpy.float64'>
5.21758903587e-05
<class 'numpy.float64'>
5.21672994918e-05
<class 'numpy.float64'>
5.21587114579e-05
<class 'numpy.float64'>
5.21501262553e-05
<class 'numpy.float64'>
5.21415438827e-05
<class 'numpy.float64'>
5.21329643389e-05
<class 'numpy.float64'>
5.21243876226e-05
<class 'numpy.float64'>
5.21158137321e-05
<class 'numpy.float64'>
5.21072426659e-05
<class 'numpy.float64'>
5.20986744228e-05
<class 'numpy.float64'>
5.20901090015e-05
<class 'numpy.float64'>
5.20815464006e-05
<class 'numpy.float64'>
5.20729866184e-05
<class 'numpy.float64'>
5.20644296539e-05
<class 'numpy.float64'>
5.20558755057e-05
<class 'numpy.float64'>
5.20473241721e-05
<class 'numpy.float64'>
5.20387756515e-05
<class 'numpy.float64'>
5.20302299433e-05
<class 'numpy.float64'>
5.20216870454e-05
<class 'numpy.float64'>
5.20131469569e-05
<class 'numpy.float64'>
5.20046096759e-05
<class 'numpy.float64'>
5.19960752017e-05
<class 'numpy.float64'>
5.19875435325e-05
<class 'numpy.float64'>
5.19790146665e-05
<class 'numpy.float64'>
5.19704886033e-05
<class 'numpy.float64'>
5.19619653404e-05
<class 'numpy.float64'>
5.19534448773e-05
<class 'numpy.float64'>
5.19449272125e-05
<class 'numpy.float64'>
5.19364123443e-05
<class 'numpy.float64'>
5.19279002714e-05
<class 'numpy.float64'>
5.19193909925e-05
<class 'numpy.float64'>
5.19108845063e-05
<class 'numpy.float64'>
5.19023808111e-05
<class 'numpy.float64'>
5.1893879906e-05
<class 'numpy.float64'>
5.18853817894e-05
<class 'numpy.float64'>
5.18768864598e-05
<class 'numpy.float64'>
5.18683939159e-05
<class 'numpy.float64'>
5.18599041566e-05
<class 'numpy.float64'>
5.18514171801e-05
<class 'numpy.float64'>
5.18429329854e-05
<class 'numpy.float64'>
5.18344515708e-05
<class 'numpy.float64'>
5.18259729351e-05
<class 'numpy.float64'>
5.18174970768e-05
<class 'numpy.float64'>
5.1809023995e-05
<class 'numpy.float64'>
5.18005536878e-05
<class 'numpy.float64'>
5.17920861541e-05
<class 'numpy.float64'>
5.17836213922e-05
<class 'numpy.float64'>
5.17751594015e-05
<class 'numpy.float64'>
5.17667001798e-05
<class 'numpy.float64'>
5.17582437263e-05
<class 'numpy.float64'>
5.17497900392e-05
<class 'numpy.float64'>
5.17413391176e-05
<class 'numpy.float64'>
5.17328909597e-05
<class 'numpy.float64'>
5.17244455646e-05
<class 'numpy.float64'>
5.17160029306e-05
<class 'numpy.float64'>
5.17075630564e-05
<class 'numpy.float64'>
5.16991259406e-05
<class 'numpy.float64'>
5.16906915822e-05
<class 'numpy.float64'>
5.16822599792e-05
<class 'numpy.float64'>
5.16738311309e-05
<class 'numpy.float64'>
5.16654050356e-05
<class 'numpy.float64'>
5.16569816923e-05
<class 'numpy.float64'>
5.16485610991e-05
<class 'numpy.float64'>
5.16401432551e-05
<class 'numpy.float64'>
5.16317281587e-05
<class 'numpy.float64'>
5.16233158088e-05
<class 'numpy.float64'>
5.16149062035e-05
<class 'numpy.float64'>
5.16064993424e-05
<class 'numpy.float64'>
5.15980952231e-05
<class 'numpy.float64'>
5.15896938451e-05
<class 'numpy.float64'>
5.15812952067e-05
<class 'numpy.float64'>
5.15728993068e-05
<class 'numpy.float64'>
5.15645061434e-05
<class 'numpy.float64'>
5.15561157159e-05
<class 'numpy.float64'>
5.15477280226e-05
<class 'numpy.float64'>
5.1539343062e-05
<class 'numpy.float64'>
5.15309608331e-05
<class 'numpy.float64'>
5.1522581335e-05
<class 'numpy.float64'>
5.15142045651e-05
<class 'numpy.float64'>
5.15058305234e-05
<class 'numpy.float64'>
5.14974592076e-05
<class 'numpy.float64'>
5.14890906169e-05
<class 'numpy.float64'>
5.14807247495e-05
<class 'numpy.float64'>
5.14723616046e-05
<class 'numpy.float64'>
5.14640011807e-05
<class 'numpy.float64'>
5.14556434763e-05
<class 'numpy.float64'>
5.14472884904e-05
<class 'numpy.float64'>
5.14389362211e-05
<class 'numpy.float64'>
5.14305866677e-05
<class 'numpy.float64'>
5.14222398285e-05
<class 'numpy.float64'>
5.14138957024e-05
<class 'numpy.float64'>
5.14055542877e-05
<class 'numpy.float64'>
5.13972155836e-05
<class 'numpy.float64'>
5.13888795885e-05
<class 'numpy.float64'>
5.1380546301e-05
<class 'numpy.float64'>
5.13722157199e-05
<class 'numpy.float64'>
5.1363887844e-05
<class 'numpy.float64'>
5.13555626717e-05
<class 'numpy.float64'>
5.13472402017e-05
<class 'numpy.float64'>
5.13389204328e-05
<class 'numpy.float64'>
5.1330603364e-05
<class 'numpy.float64'>
5.13222889934e-05
<class 'numpy.float64'>
5.13139773198e-05
<class 'numpy.float64'>
5.13056683425e-05
<class 'numpy.float64'>
5.12973620597e-05
<class 'numpy.float64'>
5.12890584698e-05
<class 'numpy.float64'>
5.1280757572e-05
<class 'numpy.float64'>
5.12724593646e-05
<class 'numpy.float64'>
5.12641638466e-05
<class 'numpy.float64'>
5.12558710168e-05
<class 'numpy.float64'>
5.12475808737e-05
<class 'numpy.float64'>
5.12392934158e-05
<class 'numpy.float64'>
5.12310086418e-05
<class 'numpy.float64'>
5.12227265506e-05
<class 'numpy.float64'>
5.12144471409e-05
<class 'numpy.float64'>
5.12061704116e-05
<class 'numpy.float64'>
5.11978963612e-05
<class 'numpy.float64'>
5.1189624988e-05
<class 'numpy.float64'>
5.11813562914e-05
<class 'numpy.float64'>
5.11730902693e-05
<class 'numpy.float64'>
5.11648269212e-05
<class 'numpy.float64'>
5.11565662453e-05
<class 'numpy.float64'>
5.11483082405e-05
<class 'numpy.float64'>
5.11400529055e-05
<class 'numpy.float64'>
5.11318002392e-05
<class 'numpy.float64'>
5.11235502397e-05
<class 'numpy.float64'>
5.11153029064e-05
<class 'numpy.float64'>
5.11070582373e-05
<class 'numpy.float64'>
5.10988162318e-05
<class 'numpy.float64'>
5.10905768881e-05
<class 'numpy.float64'>
5.10823402052e-05
<class 'numpy.float64'>
5.10741061819e-05
<class 'numpy.float64'>
5.10658748165e-05
<class 'numpy.float64'>
5.10576461079e-05
<class 'numpy.float64'>
5.10494200552e-05
<class 'numpy.float64'>
5.10411966565e-05
<class 'numpy.float64'>
5.10329759109e-05
<class 'numpy.float64'>
5.1024757817e-05
<class 'numpy.float64'>
5.10165423737e-05
<class 'numpy.float64'>
5.10083295791e-05
<class 'numpy.float64'>
5.10001194327e-05
<class 'numpy.float64'>
5.09919119326e-05
<class 'numpy.float64'>
5.09837070781e-05
<class 'numpy.float64'>
5.09755048673e-05
<class 'numpy.float64'>
5.09673052994e-05
<class 'numpy.float64'>
5.09591083732e-05
<class 'numpy.float64'>
5.09509140868e-05
<class 'numpy.float64'>
5.09427224395e-05
<class 'numpy.float64'>
5.09345334297e-05
<class 'numpy.float64'>
5.09263470565e-05
<class 'numpy.float64'>
5.09181633184e-05
<class 'numpy.float64'>
5.09099822138e-05
<class 'numpy.float64'>
5.09018037419e-05
<class 'numpy.float64'>
5.08936279012e-05
<class 'numpy.float64'>
5.08854546906e-05
<class 'numpy.float64'>
5.08772841087e-05
<class 'numpy.float64'>
5.0869116154e-05
<class 'numpy.float64'>
5.08609508261e-05
<class 'numpy.float64'>
5.08527881227e-05
<class 'numpy.float64'>
5.08446280427e-05
<class 'numpy.float64'>
5.08364705855e-05
<class 'numpy.float64'>
5.08283157495e-05
<class 'numpy.float64'>
5.08201635332e-05
<class 'numpy.float64'>
5.08120139353e-05
<class 'numpy.float64'>
5.08038669551e-05
<class 'numpy.float64'>
5.07957225908e-05
<class 'numpy.float64'>
5.07875808412e-05
<class 'numpy.float64'>
5.07794417056e-05
<class 'numpy.float64'>
5.07713051818e-05
<class 'numpy.float64'>
5.07631712692e-05
<class 'numpy.float64'>
5.07550399662e-05
<class 'numpy.float64'>
5.07469112722e-05
<class 'numpy.float64'>
5.07387851853e-05
<class 'numpy.float64'>
5.0730661704e-05
<class 'numpy.float64'>
5.07225408278e-05
<class 'numpy.float64'>
5.0714422555e-05
<class 'numpy.float64'>
5.07063068848e-05
<class 'numpy.float64'>
5.06981938151e-05
<class 'numpy.float64'>
5.06900833454e-05
<class 'numpy.float64'>
5.06819754744e-05
<class 'numpy.float64'>
5.06738702003e-05
<class 'numpy.float64'>
5.06657675222e-05
<class 'numpy.float64'>
5.0657667439e-05
<class 'numpy.float64'>
5.06495699492e-05
<class 'numpy.float64'>
5.06414750518e-05
<class 'numpy.float64'>
5.06333827453e-05
<class 'numpy.float64'>
5.06252930287e-05
<class 'numpy.float64'>
5.06172059006e-05
<class 'numpy.float64'>
5.06091213599e-05
<class 'numpy.float64'>
5.06010394049e-05
<class 'numpy.float64'>
5.0592960035e-05
<class 'numpy.float64'>
5.05848832484e-05
<class 'numpy.float64'>
5.05768090443e-05
<class 'numpy.float64'>
5.05687374213e-05
<class 'numpy.float64'>
5.05606683781e-05
<class 'numpy.float64'>
5.05526019133e-05
<class 'numpy.float64'>
5.05445380263e-05
<class 'numpy.float64'>
5.05364767151e-05
<class 'numpy.float64'>
5.0528417979e-05
<class 'numpy.float64'>
5.05203618163e-05
<class 'numpy.float64'>
5.05123082264e-05
<class 'numpy.float64'>
5.05042572075e-05
<class 'numpy.float64'>
5.04962087585e-05
<class 'numpy.float64'>
5.04881628783e-05
<class 'numpy.float64'>
5.04801195656e-05
<class 'numpy.float64'>
5.0472078819e-05
<class 'numpy.float64'>
5.04640406378e-05
<class 'numpy.float64'>
5.04560050202e-05
<class 'numpy.float64'>
5.04479719651e-05
<class 'numpy.float64'>
5.04399414715e-05
<class 'numpy.float64'>
5.04319135379e-05
<class 'numpy.float64'>
5.04238881634e-05
<class 'numpy.float64'>
5.04158653467e-05
<class 'numpy.float64'>
5.04078450861e-05
<class 'numpy.float64'>
5.03998273812e-05
<class 'numpy.float64'>
5.03918122298e-05
<class 'numpy.float64'>
5.03837996315e-05
<class 'numpy.float64'>
5.03757895847e-05
<class 'numpy.float64'>
5.03677820883e-05
<class 'numpy.float64'>
5.03597771409e-05
<class 'numpy.float64'>
5.03517747418e-05
<class 'numpy.float64'>
5.03437748891e-05
<class 'numpy.float64'>
5.03357775821e-05
<class 'numpy.float64'>
5.03277828191e-05
<class 'numpy.float64'>
5.03197905995e-05
<class 'numpy.float64'>
5.03118009215e-05
<class 'numpy.float64'>
5.03038137843e-05
<class 'numpy.float64'>
5.02958291864e-05
<class 'numpy.float64'>
5.02878471266e-05
<class 'numpy.float64'>
5.0279867604e-05
<class 'numpy.float64'>
5.02718906171e-05
<class 'numpy.float64'>
5.02639161647e-05
<class 'numpy.float64'>
5.02559442456e-05
<class 'numpy.float64'>
5.0247974859e-05
<class 'numpy.float64'>
5.02400080032e-05
<class 'numpy.float64'>
5.02320436772e-05
<class 'numpy.float64'>
5.02240818795e-05
<class 'numpy.float64'>
5.02161226094e-05
<class 'numpy.float64'>
5.02081658655e-05
<class 'numpy.float64'>
5.02002116464e-05
<class 'numpy.float64'>
5.01922599511e-05
<class 'numpy.float64'>
5.01843107782e-05
<class 'numpy.float64'>
5.01763641265e-05
<class 'numpy.float64'>
5.01684199952e-05
<class 'numpy.float64'>
5.01604783829e-05
<class 'numpy.float64'>
5.01525392879e-05
<class 'numpy.float64'>
5.01446027098e-05
<class 'numpy.float64'>
5.0136668647e-05
<class 'numpy.float64'>
5.01287370984e-05
<class 'numpy.float64'>
5.01208080624e-05
<class 'numpy.float64'>
5.01128815383e-05
<class 'numpy.float64'>
5.01049575248e-05
<class 'numpy.float64'>
5.00970360209e-05
<class 'numpy.float64'>
5.00891170248e-05
<class 'numpy.float64'>
5.00812005358e-05
<class 'numpy.float64'>
5.00732865526e-05
<class 'numpy.float64'>
5.00653750741e-05
<class 'numpy.float64'>
5.0057466099e-05
<class 'numpy.float64'>
5.00495596257e-05
<class 'numpy.float64'>
5.00416556539e-05
<class 'numpy.float64'>
5.00337541819e-05
<class 'numpy.float64'>
5.00258552083e-05
<class 'numpy.float64'>
5.00179587324e-05
<class 'numpy.float64'>
5.00100647528e-05
<class 'numpy.float64'>
5.00021732683e-05
<class 'numpy.float64'>
4.99942842776e-05
<class 'numpy.float64'>
4.99863977795e-05
<class 'numpy.float64'>
4.99785137732e-05
<class 'numpy.float64'>
4.99706322572e-05
<class 'numpy.float64'>
4.99627532302e-05
<class 'numpy.float64'>
4.99548766916e-05
<class 'numpy.float64'>
4.99470026396e-05
<class 'numpy.float64'>
4.99391310732e-05
<class 'numpy.float64'>
4.99312619915e-05
<class 'numpy.float64'>
4.99233953928e-05
<class 'numpy.float64'>
4.99155312763e-05
<class 'numpy.float64'>
4.99076696407e-05
<class 'numpy.float64'>
4.9899810485e-05
<class 'numpy.float64'>
4.9891953808e-05
<class 'numpy.float64'>
4.98840996083e-05
<class 'numpy.float64'>
4.98762478848e-05
<class 'numpy.float64'>
4.98683986364e-05
<class 'numpy.float64'>
4.98605518621e-05
<class 'numpy.float64'>
4.98527075605e-05
<class 'numpy.float64'>
4.984486573e-05
<class 'numpy.float64'>
4.98370263706e-05
<class 'numpy.float64'>
4.982918948e-05
<class 'numpy.float64'>
4.98213550575e-05
<class 'numpy.float64'>
4.98135231019e-05
<class 'numpy.float64'>
4.98056936122e-05
<class 'numpy.float64'>
4.97978665868e-05
<class 'numpy.float64'>
4.97900420251e-05
<class 'numpy.float64'>
4.97822199255e-05
<class 'numpy.float64'>
4.9774400287e-05
<class 'numpy.float64'>
4.97665831085e-05
<class 'numpy.float64'>
4.97587683885e-05
<class 'numpy.float64'>
4.97509561264e-05
<class 'numpy.float64'>
4.97431463206e-05
<class 'numpy.float64'>
4.97353389701e-05
<class 'numpy.float64'>
4.97275340736e-05
<class 'numpy.float64'>
4.97197316303e-05
<class 'numpy.float64'>
4.97119316384e-05
<class 'numpy.float64'>
4.97041340976e-05
<class 'numpy.float64'>
4.9696339006e-05
<class 'numpy.float64'>
4.96885463629e-05
<class 'numpy.float64'>
4.96807561672e-05
<class 'numpy.float64'>
4.9672968417e-05
<class 'numpy.float64'>
4.9665183112e-05
<class 'numpy.float64'>
4.96574002507e-05
<class 'numpy.float64'>
4.96496198318e-05
<class 'numpy.float64'>
4.96418418545e-05
<class 'numpy.float64'>
4.96340663176e-05
<class 'numpy.float64'>
4.96262932193e-05
<class 'numpy.float64'>
4.96185225594e-05
<class 'numpy.float64'>
4.96107543364e-05
<class 'numpy.float64'>
4.96029885488e-05
<class 'numpy.float64'>
4.95952251958e-05
<class 'numpy.float64'>
4.95874642764e-05
<class 'numpy.float64'>
4.95797057891e-05
<class 'numpy.float64'>
4.95719497328e-05
<class 'numpy.float64'>
4.95641961066e-05
<class 'numpy.float64'>
4.95564449095e-05
<class 'numpy.float64'>
4.95486961395e-05
<class 'numpy.float64'>
4.95409497964e-05
<class 'numpy.float64'>
4.95332058786e-05
<class 'numpy.float64'>
4.95254643849e-05
<class 'numpy.float64'>
4.95177253145e-05
<class 'numpy.float64'>
4.95099886663e-05
<class 'numpy.float64'>
4.95022544386e-05
<class 'numpy.float64'>
4.94945226306e-05
<class 'numpy.float64'>
4.94867932412e-05
<class 'numpy.float64'>
4.94790662693e-05
<class 'numpy.float64'>
4.94713417138e-05
<class 'numpy.float64'>
4.94636195734e-05
<class 'numpy.float64'>
4.94558998468e-05
<class 'numpy.float64'>
4.94481825334e-05
<class 'numpy.float64'>
4.94404676313e-05
<class 'numpy.float64'>
4.94327551406e-05
<class 'numpy.float64'>
4.94250450591e-05
<class 'numpy.float64'>
4.94173373857e-05
<class 'numpy.float64'>
4.94096321194e-05
<class 'numpy.float64'>
4.94019292597e-05
<class 'numpy.float64'>
4.93942288047e-05
<class 'numpy.float64'>
4.93865307538e-05
<class 'numpy.float64'>
4.93788351054e-05
<class 'numpy.float64'>
4.93711418585e-05
<class 'numpy.float64'>
4.93634510124e-05
<class 'numpy.float64'>
4.93557625657e-05
<class 'numpy.float64'>
4.93480765171e-05
<class 'numpy.float64'>
4.93403928654e-05
<class 'numpy.float64'>
4.933271161e-05
<class 'numpy.float64'>
4.93250327494e-05
<class 'numpy.float64'>
4.93173562824e-05
<class 'numpy.float64'>
4.93096822081e-05
<class 'numpy.float64'>
4.93020105256e-05
<class 'numpy.float64'>
4.92943412333e-05
<class 'numpy.float64'>
4.92866743303e-05
<class 'numpy.float64'>
4.92790098151e-05
<class 'numpy.float64'>
4.92713476873e-05
<class 'numpy.float64'>
4.92636879456e-05
<class 'numpy.float64'>
4.92560305882e-05
<class 'numpy.float64'>
4.9248375615e-05
<class 'numpy.float64'>
4.92407230242e-05
<class 'numpy.float64'>
4.92330728147e-05
<class 'numpy.float64'>
4.92254249856e-05
<class 'numpy.float64'>
4.92177795359e-05
<class 'numpy.float64'>
4.92101364644e-05
<class 'numpy.float64'>
4.92024957697e-05
<class 'numpy.float64'>
4.91948574512e-05
<class 'numpy.float64'>
4.91872215071e-05
<class 'numpy.float64'>
4.91795879372e-05
<class 'numpy.float64'>
4.91719567394e-05
<class 'numpy.float64'>
4.91643279133e-05
<class 'numpy.float64'>
4.91567014576e-05
<class 'numpy.float64'>
4.91490773714e-05
<class 'numpy.float64'>
4.9141455653e-05
<class 'numpy.float64'>
4.9133836302e-05
<class 'numpy.float64'>
4.91262193166e-05
<class 'numpy.float64'>
4.91186046965e-05
<class 'numpy.float64'>
4.91109924398e-05
<class 'numpy.float64'>
4.9103382546e-05
<class 'numpy.float64'>
4.90957750132e-05
<class 'numpy.float64'>
4.90881698415e-05
<class 'numpy.float64'>
4.9080567029e-05
<class 'numpy.float64'>
4.90729665746e-05
<class 'numpy.float64'>
4.90653684774e-05
<class 'numpy.float64'>
4.90577727363e-05
<class 'numpy.float64'>
4.90501793503e-05
<class 'numpy.float64'>
4.9042588318e-05
<class 'numpy.float64'>
4.90349996386e-05
<class 'numpy.float64'>
4.90274133107e-05
<class 'numpy.float64'>
4.90198293335e-05
<class 'numpy.float64'>
4.90122477058e-05
<class 'numpy.float64'>
4.90046684266e-05
<class 'numpy.float64'>
4.89970914949e-05
<class 'numpy.float64'>
4.8989516909e-05
<class 'numpy.float64'>
4.89819446683e-05
<class 'numpy.float64'>
4.89743747719e-05
<class 'numpy.float64'>
4.89668072182e-05
<class 'numpy.float64'>
4.89592420064e-05
<class 'numpy.float64'>
4.89516791354e-05
<class 'numpy.float64'>
4.89441186042e-05
<class 'numpy.float64'>
4.89365604116e-05
<class 'numpy.float64'>
4.89290045565e-05
<class 'numpy.float64'>
4.89214510376e-05
<class 'numpy.float64'>
4.89138998544e-05
<class 'numpy.float64'>
4.89063510052e-05
<class 'numpy.float64'>
4.88988044894e-05
<class 'numpy.float64'>
4.88912603056e-05
<class 'numpy.float64'>
4.88837184529e-05
<class 'numpy.float64'>
4.88761789299e-05
<class 'numpy.float64'>
4.88686417358e-05
<class 'numpy.float64'>
4.88611068698e-05
<class 'numpy.float64'>
4.88535743301e-05
<class 'numpy.float64'>
4.88460441163e-05
<class 'numpy.float64'>
4.8838516227e-05
<class 'numpy.float64'>
4.8830990661e-05
<class 'numpy.float64'>
4.88234674175e-05
<class 'numpy.float64'>
4.88159464953e-05
<class 'numpy.float64'>
4.88084278935e-05
<class 'numpy.float64'>
4.88009116107e-05
<class 'numpy.float64'>
4.8793397646e-05
<class 'numpy.float64'>
4.87858859983e-05
<class 'numpy.float64'>
4.87783766667e-05
<class 'numpy.float64'>
4.87708696499e-05
<class 'numpy.float64'>
4.87633649467e-05
<class 'numpy.float64'>
4.87558625565e-05
<class 'numpy.float64'>
4.87483624778e-05
<class 'numpy.float64'>
4.87408647098e-05
<class 'numpy.float64'>
4.87333692513e-05
<class 'numpy.float64'>
4.87258761014e-05
<class 'numpy.float64'>
4.87183852589e-05
<class 'numpy.float64'>
4.87108967225e-05
<class 'numpy.float64'>
4.87034104915e-05
<class 'numpy.float64'>
4.86959265645e-05
<class 'numpy.float64'>
4.86884449408e-05
<class 'numpy.float64'>
4.86809656192e-05
<class 'numpy.float64'>
4.86734885985e-05
<class 'numpy.float64'>
4.86660138778e-05
<class 'numpy.float64'>
4.86585414561e-05
<class 'numpy.float64'>
4.86510713319e-05
<class 'numpy.float64'>
4.86436035048e-05
<class 'numpy.float64'>
4.86361379733e-05
<class 'numpy.float64'>
4.86286747363e-05
<class 'numpy.float64'>
4.86212137931e-05
<class 'numpy.float64'>
4.86137551421e-05
<class 'numpy.float64'>
4.8606298783e-05
<class 'numpy.float64'>
4.8598844714e-05
<class 'numpy.float64'>
4.85913929345e-05
<class 'numpy.float64'>
4.85839434434e-05
<class 'numpy.float64'>
4.85764962392e-05
<class 'numpy.float64'>
4.85690513215e-05
<class 'numpy.float64'>
4.85616086891e-05
<class 'numpy.float64'>
4.85541683403e-05
<class 'numpy.float64'>
4.85467302747e-05
<class 'numpy.float64'>
4.85392944912e-05
<class 'numpy.float64'>
4.85318609885e-05
<class 'numpy.float64'>
4.85244297659e-05
<class 'numpy.float64'>
4.85170008219e-05
<class 'numpy.float64'>
4.85095741557e-05
<class 'numpy.float64'>
4.85021497665e-05
<class 'numpy.float64'>
4.84947276526e-05
<class 'numpy.float64'>
4.84873078134e-05
<class 'numpy.float64'>
4.8479890248e-05
<class 'numpy.float64'>
4.84724749552e-05
<class 'numpy.float64'>
4.84650619337e-05
<class 'numpy.float64'>
4.84576511829e-05
<class 'numpy.float64'>
4.84502427012e-05
<class 'numpy.float64'>
4.84428364882e-05
<class 'numpy.float64'>
4.84354325425e-05
<class 'numpy.float64'>
4.8428030863e-05
<class 'numpy.float64'>
4.84206314486e-05
<class 'numpy.float64'>
4.84132342984e-05
<class 'numpy.float64'>
4.84058394116e-05
<class 'numpy.float64'>
4.83984467869e-05
<class 'numpy.float64'>
4.83910564232e-05
<class 'numpy.float64'>
4.83836683196e-05
<class 'numpy.float64'>
4.8376282475e-05
<class 'numpy.float64'>
4.83688988884e-05
<class 'numpy.float64'>
4.83615175586e-05
<class 'numpy.float64'>
4.83541384848e-05
<class 'numpy.float64'>
4.8346761666e-05
<class 'numpy.float64'>
4.83393871009e-05
<class 'numpy.float64'>
4.83320147886e-05
<class 'numpy.float64'>
4.83246447283e-05
<class 'numpy.float64'>
4.83172769186e-05
<class 'numpy.float64'>
4.83099113587e-05
<class 'numpy.float64'>
4.83025480472e-05
<class 'numpy.float64'>
4.82951869835e-05
<class 'numpy.float64'>
4.82878281666e-05
<class 'numpy.float64'>
4.82804715949e-05
<class 'numpy.float64'>
4.82731172679e-05
<class 'numpy.float64'>
4.82657651847e-05
<class 'numpy.float64'>
4.82584153438e-05
<class 'numpy.float64'>
4.82510677446e-05
<class 'numpy.float64'>
4.82437223854e-05
<class 'numpy.float64'>
4.8236379266e-05
<class 'numpy.float64'>
4.82290383847e-05
<class 'numpy.float64'>
4.82216997411e-05
<class 'numpy.float64'>
4.82143633336e-05
<class 'numpy.float64'>
4.82070291618e-05
<class 'numpy.float64'>
4.81996972241e-05
<class 'numpy.float64'>
4.81923675196e-05
<class 'numpy.float64'>
4.81850400475e-05
<class 'numpy.float64'>
4.81777148066e-05
<class 'numpy.float64'>
4.8170391796e-05
<class 'numpy.float64'>
4.81630710146e-05
<class 'numpy.float64'>
4.81557524611e-05
<class 'numpy.float64'>
4.81484361353e-05
<class 'numpy.float64'>
4.81411220352e-05
<class 'numpy.float64'>
4.81338101603e-05
<class 'numpy.float64'>
4.81265005097e-05
<class 'numpy.float64'>
4.8119193082e-05
<class 'numpy.float64'>
4.81118878767e-05
<class 'numpy.float64'>
4.81045848923e-05
<class 'numpy.float64'>
4.8097284128e-05
<class 'numpy.float64'>
4.80899855827e-05
<class 'numpy.float64'>
4.80826892557e-05
<class 'numpy.float64'>
4.80753951455e-05
<class 'numpy.float64'>
4.80681032512e-05
<class 'numpy.float64'>
4.80608135723e-05
<class 'numpy.float64'>
4.80535261069e-05
<class 'numpy.float64'>
4.80462408549e-05
<class 'numpy.float64'>
4.80389578147e-05
<class 'numpy.float64'>
4.80316769854e-05
<class 'numpy.float64'>
4.80243983666e-05
<class 'numpy.float64'>
4.80171219559e-05
<class 'numpy.float64'>
4.80098477538e-05
<class 'numpy.float64'>
4.80025757582e-05
<class 'numpy.float64'>
4.79953059689e-05
<class 'numpy.float64'>
4.79880383845e-05
<class 'numpy.float64'>
4.7980773004e-05
<class 'numpy.float64'>
4.79735098263e-05
<class 'numpy.float64'>
4.79662488507e-05
<class 'numpy.float64'>
4.79589900761e-05
<class 'numpy.float64'>
4.79517335012e-05
<class 'numpy.float64'>
4.79444791254e-05
<class 'numpy.float64'>
4.79372269475e-05
<class 'numpy.float64'>
4.79299769664e-05
<class 'numpy.float64'>
4.79227291814e-05
<class 'numpy.float64'>
4.79154835912e-05
<class 'numpy.float64'>
4.79082401951e-05
<class 'numpy.float64'>
4.79009989917e-05
<class 'numpy.float64'>
4.78937599804e-05
<class 'numpy.float64'>
4.78865231599e-05
<class 'numpy.float64'>
4.78792885297e-05
<class 'numpy.float64'>
4.78720560879e-05
<class 'numpy.float64'>
4.78648258345e-05
<class 'numpy.float64'>
4.78575977682e-05
<class 'numpy.float64'>
4.78503718873e-05
<class 'numpy.float64'>
4.78431481918e-05
<class 'numpy.float64'>
4.78359266804e-05
<class 'numpy.float64'>
4.78287073516e-05
<class 'numpy.float64'>
4.78214902051e-05
<class 'numpy.float64'>
4.78142752396e-05
<class 'numpy.float64'>
4.7807062454e-05
<class 'numpy.float64'>
4.77998518476e-05
<class 'numpy.float64'>
4.7792643419e-05
<class 'numpy.float64'>
4.77854371677e-05
<class 'numpy.float64'>
4.77782330926e-05
<class 'numpy.float64'>
4.77710311921e-05
<class 'numpy.float64'>
4.7763831466e-05
<class 'numpy.float64'>
4.77566339131e-05
<class 'numpy.float64'>
4.77494385324e-05
<class 'numpy.float64'>
4.77422453226e-05
<class 'numpy.float64'>
4.77350542831e-05
<class 'numpy.float64'>
4.7727865413e-05
<class 'numpy.float64'>
4.77206787108e-05
<class 'numpy.float64'>
4.77134941759e-05
<class 'numpy.float64'>
4.77063118073e-05
<class 'numpy.float64'>
4.76991316039e-05
<class 'numpy.float64'>
4.76919535648e-05
<class 'numpy.float64'>
4.76847776893e-05
<class 'numpy.float64'>
4.7677603976e-05
<class 'numpy.float64'>
4.76704324238e-05
<class 'numpy.float64'>
4.7663263032e-05
<class 'numpy.float64'>
4.76560957999e-05
<class 'numpy.float64'>
4.7648930726e-05
<class 'numpy.float64'>
4.76417678095e-05
<class 'numpy.float64'>
4.76346070497e-05
<class 'numpy.float64'>
4.76274484451e-05
<class 'numpy.float64'>
4.7620291995e-05
<class 'numpy.float64'>
4.76131376987e-05
<class 'numpy.float64'>
4.76059855552e-05
<class 'numpy.float64'>
4.75988355629e-05
<class 'numpy.float64'>
4.75916877212e-05
<class 'numpy.float64'>
4.75845420291e-05
<class 'numpy.float64'>
4.75773984861e-05
<class 'numpy.float64'>
4.75702570907e-05
<class 'numpy.float64'>
4.75631178419e-05
<class 'numpy.float64'>
4.75559807389e-05
<class 'numpy.float64'>
4.75488457808e-05
<class 'numpy.float64'>
4.75417129667e-05
<class 'numpy.float64'>
4.75345822951e-05
<class 'numpy.float64'>
4.75274537657e-05
<class 'numpy.float64'>
4.75203273771e-05
<class 'numpy.float64'>
4.75132031287e-05
<class 'numpy.float64'>
4.75060810191e-05
<class 'numpy.float64'>
4.74989610479e-05
<class 'numpy.float64'>
4.74918432135e-05
<class 'numpy.float64'>
4.74847275153e-05
<class 'numpy.float64'>
4.74776139523e-05
<class 'numpy.float64'>
4.74705025234e-05
<class 'numpy.float64'>
4.74633932278e-05
<class 'numpy.float64'>
4.74562860645e-05
<class 'numpy.float64'>
4.74491810325e-05
<class 'numpy.float64'>
4.74420781308e-05
<class 'numpy.float64'>
4.74349773589e-05
<class 'numpy.float64'>
4.74278787151e-05
<class 'numpy.float64'>
4.74207821986e-05
<class 'numpy.float64'>
4.74136878091e-05
<class 'numpy.float64'>
4.74065955451e-05
<class 'numpy.float64'>
4.73995054057e-05
<class 'numpy.float64'>
4.73924173897e-05
<class 'numpy.float64'>
4.73853314966e-05
<class 'numpy.float64'>
4.73782477253e-05
<class 'numpy.float64'>
4.73711660747e-05
<class 'numpy.float64'>
4.7364086544e-05
<class 'numpy.float64'>
4.73570091324e-05
<class 'numpy.float64'>
4.73499338386e-05
<class 'numpy.float64'>
4.73428606615e-05
<class 'numpy.float64'>
4.7335789601e-05
<class 'numpy.float64'>
4.73287206553e-05
<class 'numpy.float64'>
4.73216538238e-05
<class 'numpy.float64'>
4.73145891056e-05
<class 'numpy.float64'>
4.73075264996e-05
<class 'numpy.float64'>
4.73004660049e-05
<class 'numpy.float64'>
4.72934076204e-05
<class 'numpy.float64'>
4.72863513455e-05
<class 'numpy.float64'>
4.72792971791e-05
<class 'numpy.float64'>
4.72722451199e-05
<class 'numpy.float64'>
4.72651951674e-05
<class 'numpy.float64'>
4.72581473209e-05
<class 'numpy.float64'>
4.72511015786e-05
<class 'numpy.float64'>
4.72440579405e-05
<class 'numpy.float64'>
4.72370164049e-05
<class 'numpy.float64'>
4.72299769711e-05
<class 'numpy.float64'>
4.72229396384e-05
<class 'numpy.float64'>
4.72159044056e-05
<class 'numpy.float64'>
4.72088712719e-05
<class 'numpy.float64'>
4.72018402363e-05
<class 'numpy.float64'>
4.71948112977e-05
<class 'numpy.float64'>
4.71877844556e-05
<class 'numpy.float64'>
4.71807597085e-05
<class 'numpy.float64'>
4.7173737056e-05
<class 'numpy.float64'>
4.71667164968e-05
<class 'numpy.float64'>
4.71596980301e-05
<class 'numpy.float64'>
4.71526816547e-05
<class 'numpy.float64'>
4.714566737e-05
<class 'numpy.float64'>
4.71386551751e-05
<class 'numpy.float64'>
4.71316450689e-05
<class 'numpy.float64'>
4.71246370504e-05
<class 'numpy.float64'>
4.71176311187e-05
<class 'numpy.float64'>
4.71106272731e-05
<class 'numpy.float64'>
4.71036255123e-05
<class 'numpy.float64'>
4.7096625836e-05
<class 'numpy.float64'>
4.70896282425e-05
<class 'numpy.float64'>
4.70826327312e-05
<class 'numpy.float64'>
4.70756393014e-05
<class 'numpy.float64'>
4.70686479517e-05
<class 'numpy.float64'>
4.70616586816e-05
<class 'numpy.float64'>
4.705467149e-05
<class 'numpy.float64'>
4.70476863757e-05
<class 'numpy.float64'>
4.70407033381e-05
<class 'numpy.float64'>
4.70337223765e-05
<class 'numpy.float64'>
4.70267434892e-05
<class 'numpy.float64'>
4.70197666762e-05
<class 'numpy.float64'>
4.7012791936e-05
<class 'numpy.float64'>
4.70058192678e-05
<class 'numpy.float64'>
4.69988486704e-05
<class 'numpy.float64'>
4.69918801436e-05
<class 'numpy.float64'>
4.69849136858e-05
<class 'numpy.float64'>
4.69779492964e-05
<class 'numpy.float64'>
4.69709869742e-05
<class 'numpy.float64'>
4.69640267186e-05
<class 'numpy.float64'>
4.69570685289e-05
<class 'numpy.float64'>
4.69501124035e-05
<class 'numpy.float64'>
4.69431583419e-05
<class 'numpy.float64'>
4.69362063429e-05
<class 'numpy.float64'>
4.69292564057e-05
<class 'numpy.float64'>
4.69223085299e-05
<class 'numpy.float64'>
4.6915362714e-05
<class 'numpy.float64'>
4.69084189571e-05
<class 'numpy.float64'>
4.69014772582e-05
<class 'numpy.float64'>
4.6894537617e-05
<class 'numpy.float64'>
4.6887600032e-05
<class 'numpy.float64'>
4.68806645026e-05
<class 'numpy.float64'>
4.68737310277e-05
<class 'numpy.float64'>
4.68667996062e-05
<class 'numpy.float64'>
4.68598702376e-05
<class 'numpy.float64'>
4.68529429208e-05
<class 'numpy.float64'>
4.6846017655e-05
<class 'numpy.float64'>
4.68390944391e-05
<class 'numpy.float64'>
4.68321732722e-05
<class 'numpy.float64'>
4.68252541536e-05
<class 'numpy.float64'>
4.68183370821e-05
<class 'numpy.float64'>
4.6811422057e-05
<class 'numpy.float64'>
4.68045090776e-05
<class 'numpy.float64'>
4.67975981424e-05
<class 'numpy.float64'>
4.67906892509e-05
<class 'numpy.float64'>
4.67837824024e-05
<class 'numpy.float64'>
4.67768775954e-05
<class 'numpy.float64'>
4.67699748293e-05
<class 'numpy.float64'>
4.67630741034e-05
<class 'numpy.float64'>
4.67561754163e-05
<class 'numpy.float64'>
4.67492787677e-05
<class 'numpy.float64'>
4.67423841562e-05
<class 'numpy.float64'>
4.67354915812e-05
<class 'numpy.float64'>
4.67286010418e-05
<class 'numpy.float64'>
4.67217125364e-05
<class 'numpy.float64'>
4.67148260653e-05
<class 'numpy.float64'>
4.67079416267e-05
<class 'numpy.float64'>
4.670105922e-05
<class 'numpy.float64'>
4.66941788444e-05
<class 'numpy.float64'>
4.66873004989e-05
<class 'numpy.float64'>
4.66804241823e-05
<class 'numpy.float64'>
4.66735498943e-05
<class 'numpy.float64'>
4.66666776334e-05
<class 'numpy.float64'>
4.66598073994e-05
<class 'numpy.float64'>
4.66529391905e-05
<class 'numpy.float64'>
4.66460730066e-05
<class 'numpy.float64'>
4.66392088463e-05
<class 'numpy.float64'>
4.6632346709e-05
<class 'numpy.float64'>
4.66254865934e-05
<class 'numpy.float64'>
4.66186284993e-05
<class 'numpy.float64'>
4.66117724254e-05
<class 'numpy.float64'>
4.66049183709e-05
<class 'numpy.float64'>
4.65980663348e-05
<class 'numpy.float64'>
4.65912163161e-05
<class 'numpy.float64'>
4.65843683141e-05
<class 'numpy.float64'>
4.65775223277e-05
<class 'numpy.float64'>
4.65706783563e-05
<class 'numpy.float64'>
4.65638363988e-05
<class 'numpy.float64'>
4.65569964549e-05
<class 'numpy.float64'>
4.65501585227e-05
<class 'numpy.float64'>
4.65433226017e-05
<class 'numpy.float64'>
4.65364886913e-05
<class 'numpy.float64'>
4.65296567906e-05
<class 'numpy.float64'>
4.65228268984e-05
<class 'numpy.float64'>
4.6515999014e-05
<class 'numpy.float64'>
4.65091731364e-05
<class 'numpy.float64'>
4.65023492648e-05
<class 'numpy.float64'>
4.64955273982e-05
<class 'numpy.float64'>
4.64887075361e-05
<class 'numpy.float64'>
4.64818896774e-05
<class 'numpy.float64'>
4.64750738208e-05
<class 'numpy.float64'>
4.64682599659e-05
<class 'numpy.float64'>
4.64614481117e-05
<class 'numpy.float64'>
4.64546382575e-05
<class 'numpy.float64'>
4.64478304022e-05
<class 'numpy.float64'>
4.64410245445e-05
<class 'numpy.float64'>
4.64342206844e-05
<class 'numpy.float64'>
4.64274188203e-05
<class 'numpy.float64'>
4.64206189519e-05
<class 'numpy.float64'>
4.64138210781e-05
<class 'numpy.float64'>
4.64070251975e-05
<class 'numpy.float64'>
4.64002313103e-05
<class 'numpy.float64'>
4.63934394143e-05
<class 'numpy.float64'>
4.63866495099e-05
<class 'numpy.float64'>
4.63798615954e-05
<class 'numpy.float64'>
4.63730756702e-05
<class 'numpy.float64'>
4.63662917333e-05
<class 'numpy.float64'>
4.63595097841e-05
<class 'numpy.float64'>
4.63527298215e-05
<class 'numpy.float64'>
4.63459518447e-05
<class 'numpy.float64'>
4.63391758527e-05
<class 'numpy.float64'>
4.63324018445e-05
<class 'numpy.float64'>
4.63256298197e-05
<class 'numpy.float64'>
4.63188597774e-05
<class 'numpy.float64'>
4.63120917164e-05
<class 'numpy.float64'>
4.63053256358e-05
<class 'numpy.float64'>
4.6298561535e-05
<class 'numpy.float64'>
4.6291799413e-05
<class 'numpy.float64'>
4.62850392688e-05
<class 'numpy.float64'>
4.62782811019e-05
<class 'numpy.float64'>
4.62715249111e-05
<class 'numpy.float64'>
4.62647706956e-05
<class 'numpy.float64'>
4.62580184544e-05
<class 'numpy.float64'>
4.6251268187e-05
<class 'numpy.float64'>
4.62445198923e-05
<class 'numpy.float64'>
4.62377735696e-05
<class 'numpy.float64'>
4.62310292177e-05
<class 'numpy.float64'>
4.6224286836e-05
<class 'numpy.float64'>
4.62175464238e-05
<class 'numpy.float64'>
4.62108079796e-05
<class 'numpy.float64'>
4.62040715033e-05
<class 'numpy.float64'>
4.61973369935e-05
<class 'numpy.float64'>
4.61906044498e-05
<class 'numpy.float64'>
4.61838738707e-05
<class 'numpy.float64'>
4.61771452559e-05
<class 'numpy.float64'>
4.61704186044e-05
<class 'numpy.float64'>
4.61636939153e-05
<class 'numpy.float64'>
4.61569711876e-05
<class 'numpy.float64'>
4.61502504205e-05
<class 'numpy.float64'>
4.61435316134e-05
<class 'numpy.float64'>
4.61368147653e-05
<class 'numpy.float64'>
4.61300998747e-05
<class 'numpy.float64'>
4.61233869421e-05
<class 'numpy.float64'>
4.61166759654e-05
<class 'numpy.float64'>
4.61099669446e-05
<class 'numpy.float64'>
4.61032598781e-05
<class 'numpy.float64'>
4.60965547657e-05
<class 'numpy.float64'>
4.60898516064e-05
<class 'numpy.float64'>
4.60831503987e-05
<class 'numpy.float64'>
4.60764511426e-05
<class 'numpy.float64'>
4.60697538368e-05
<class 'numpy.float64'>
4.60630584808e-05
<class 'numpy.float64'>
4.60563650731e-05
<class 'numpy.float64'>
4.60496736135e-05
<class 'numpy.float64'>
4.6042984101e-05
<class 'numpy.float64'>
4.60362965344e-05
<class 'numpy.float64'>
4.60296109132e-05
<class 'numpy.float64'>
4.60229272362e-05
<class 'numpy.float64'>
4.60162455031e-05
<class 'numpy.float64'>
4.60095657129e-05
<class 'numpy.float64'>
4.60028878642e-05
<class 'numpy.float64'>
4.5996211957e-05
<class 'numpy.float64'>
4.59895379898e-05
<class 'numpy.float64'>
4.59828659619e-05
<class 'numpy.float64'>
4.59761958724e-05
<class 'numpy.float64'>
4.59695277209e-05
<class 'numpy.float64'>
4.5962861506e-05
<class 'numpy.float64'>
4.59561972271e-05
<class 'numpy.float64'>
4.59495348836e-05
<class 'numpy.float64'>
4.5942874474e-05
<class 'numpy.float64'>
4.59362159981e-05
<class 'numpy.float64'>
4.59295594548e-05
<class 'numpy.float64'>
4.59229048431e-05
<class 'numpy.float64'>
4.59162521626e-05
<class 'numpy.float64'>
4.59096014119e-05
<class 'numpy.float64'>
4.59029525906e-05
<class 'numpy.float64'>
4.58963056976e-05
<class 'numpy.float64'>
4.58896607322e-05
<class 'numpy.float64'>
4.58830176938e-05
<class 'numpy.float64'>
4.58763765812e-05
<class 'numpy.float64'>
4.58697373933e-05
<class 'numpy.float64'>
4.58631001302e-05
<class 'numpy.float64'>
4.585646479e-05
<class 'numpy.float64'>
4.58498313725e-05
<class 'numpy.float64'>
4.58431998767e-05
<class 'numpy.float64'>
4.58365703018e-05
<class 'numpy.float64'>
4.58299426469e-05
<class 'numpy.float64'>
4.58233169112e-05
<class 'numpy.float64'>
4.58166930937e-05
<class 'numpy.float64'>
4.58100711941e-05
<class 'numpy.float64'>
4.58034512109e-05
<class 'numpy.float64'>
4.57968331438e-05
<class 'numpy.float64'>
4.57902169913e-05
<class 'numpy.float64'>
4.57836027537e-05
<class 'numpy.float64'>
4.57769904292e-05
<class 'numpy.float64'>
4.57703800168e-05
<class 'numpy.float64'>
4.57637715165e-05
<class 'numpy.float64'>
4.57571649271e-05
<class 'numpy.float64'>
4.57505602479e-05
<class 'numpy.float64'>
4.57439574776e-05
<class 'numpy.float64'>
4.57373566156e-05
<class 'numpy.float64'>
4.57307576616e-05
<class 'numpy.float64'>
4.57241606141e-05
<class 'numpy.float64'>
4.57175654727e-05
<class 'numpy.float64'>
4.57109722361e-05
<class 'numpy.float64'>
4.57043809039e-05
<class 'numpy.float64'>
4.56977914752e-05
<class 'numpy.float64'>
4.56912039489e-05
<class 'numpy.float64'>
4.56846183247e-05
<class 'numpy.float64'>
4.56780346013e-05
<class 'numpy.float64'>
4.56714527778e-05
<class 'numpy.float64'>
4.56648728541e-05
<class 'numpy.float64'>
4.56582948286e-05
<class 'numpy.float64'>
4.56517187011e-05
<class 'numpy.float64'>
4.56451444699e-05
<class 'numpy.float64'>
4.56385721351e-05
<class 'numpy.float64'>
4.56320016954e-05
<class 'numpy.float64'>
4.56254331501e-05
<class 'numpy.float64'>
4.56188664985e-05
<class 'numpy.float64'>
4.56123017395e-05
<class 'numpy.float64'>
4.56057388725e-05
<class 'numpy.float64'>
4.55991778965e-05
<class 'numpy.float64'>
4.55926188109e-05
<class 'numpy.float64'>
4.55860616147e-05
<class 'numpy.float64'>
4.55795063073e-05
<class 'numpy.float64'>
4.55729528875e-05
<class 'numpy.float64'>
4.55664013551e-05
<class 'numpy.float64'>
4.55598517087e-05
<class 'numpy.float64'>
4.55533039477e-05
<class 'numpy.float64'>
4.55467580712e-05
<class 'numpy.float64'>
4.55402140789e-05
<class 'numpy.float64'>
4.55336719692e-05
<class 'numpy.float64'>
4.55271317414e-05
<class 'numpy.float64'>
4.55205933955e-05
<class 'numpy.float64'>
4.55140569298e-05
<class 'numpy.float64'>
4.55075223438e-05
<class 'numpy.float64'>
4.55009896368e-05
<class 'numpy.float64'>
4.54944588078e-05
<class 'numpy.float64'>
4.5487929856e-05
<class 'numpy.float64'>
4.54814027807e-05
<class 'numpy.float64'>
4.54748775813e-05
<class 'numpy.float64'>
4.54683542564e-05
<class 'numpy.float64'>
4.54618328058e-05
<class 'numpy.float64'>
4.54553132284e-05
<class 'numpy.float64'>
4.54487955234e-05
<class 'numpy.float64'>
4.544227969e-05
<class 'numpy.float64'>
4.54357657274e-05
<class 'numpy.float64'>
4.54292536348e-05
<class 'numpy.float64'>
4.54227434114e-05
<class 'numpy.float64'>
4.54162350565e-05
<class 'numpy.float64'>
4.54097285691e-05
<class 'numpy.float64'>
4.54032239483e-05
<class 'numpy.float64'>
4.53967211937e-05
<class 'numpy.float64'>
4.53902203042e-05
<class 'numpy.float64'>
4.53837212792e-05
<class 'numpy.float64'>
4.53772241178e-05
<class 'numpy.float64'>
4.53707288188e-05
<class 'numpy.float64'>
4.53642353821e-05
<class 'numpy.float64'>
4.53577438065e-05
<class 'numpy.float64'>
4.53512540913e-05
<class 'numpy.float64'>
4.53447662355e-05
<class 'numpy.float64'>
4.53382802385e-05
<class 'numpy.float64'>
4.53317960995e-05
<class 'numpy.float64'>
4.53253138177e-05
<class 'numpy.float64'>
4.53188333925e-05
<class 'numpy.float64'>
4.53123548229e-05
<class 'numpy.float64'>
4.53058781075e-05
<class 'numpy.float64'>
4.52994032466e-05
<class 'numpy.float64'>
4.52929302386e-05
<class 'numpy.float64'>
4.5286459083e-05
<class 'numpy.float64'>
4.5279989779e-05
<class 'numpy.float64'>
4.52735223262e-05
<class 'numpy.float64'>
4.52670567229e-05
<class 'numpy.float64'>
4.5260592969e-05
<class 'numpy.float64'>
4.52541310636e-05
<class 'numpy.float64'>
4.52476710057e-05
<class 'numpy.float64'>
4.52412127947e-05
<class 'numpy.float64'>
4.52347564297e-05
<class 'numpy.float64'>
4.52283019098e-05
<class 'numpy.float64'>
4.52218492344e-05
<class 'numpy.float64'>
4.52153984028e-05
<class 'numpy.float64'>
4.52089494142e-05
<class 'numpy.float64'>
4.52025022674e-05
<class 'numpy.float64'>
4.5196056962e-05
<class 'numpy.float64'>
Cost after iteration 7000: 0.000045
4.51896134971e-05
<class 'numpy.float64'>
4.51831718719e-05
<class 'numpy.float64'>
4.51767320856e-05
<class 'numpy.float64'>
4.51702941376e-05
<class 'numpy.float64'>
4.51638580267e-05
<class 'numpy.float64'>
4.51574237525e-05
<class 'numpy.float64'>
4.5150991314e-05
<class 'numpy.float64'>
4.51445607106e-05
<class 'numpy.float64'>
4.51381319413e-05
<class 'numpy.float64'>
4.51317050054e-05
<class 'numpy.float64'>
4.51252799023e-05
<class 'numpy.float64'>
4.5118856631e-05
<class 'numpy.float64'>
4.51124351907e-05
<class 'numpy.float64'>
4.51060155807e-05
<class 'numpy.float64'>
4.50995978002e-05
<class 'numpy.float64'>
4.50931818481e-05
<class 'numpy.float64'>
4.50867677244e-05
<class 'numpy.float64'>
4.50803554278e-05
<class 'numpy.float64'>
4.50739449574e-05
<class 'numpy.float64'>
4.50675363128e-05
<class 'numpy.float64'>
4.50611294931e-05
<class 'numpy.float64'>
4.5054724497e-05
<class 'numpy.float64'>
4.50483213243e-05
<class 'numpy.float64'>
4.50419199742e-05
<class 'numpy.float64'>
4.50355204458e-05
<class 'numpy.float64'>
4.50291227383e-05
<class 'numpy.float64'>
4.50227268509e-05
<class 'numpy.float64'>
4.50163327827e-05
<class 'numpy.float64'>
4.50099405334e-05
<class 'numpy.float64'>
4.50035501016e-05
<class 'numpy.float64'>
4.4997161487e-05
<class 'numpy.float64'>
4.49907746886e-05
<class 'numpy.float64'>
4.49843897059e-05
<class 'numpy.float64'>
4.49780065377e-05
<class 'numpy.float64'>
4.49716251835e-05
<class 'numpy.float64'>
4.49652456425e-05
<class 'numpy.float64'>
4.49588679137e-05
<class 'numpy.float64'>
4.49524919967e-05
<class 'numpy.float64'>
4.49461178906e-05
<class 'numpy.float64'>
4.49397455944e-05
<class 'numpy.float64'>
4.49333751075e-05
<class 'numpy.float64'>
4.49270064292e-05
<class 'numpy.float64'>
4.49206395587e-05
<class 'numpy.float64'>
4.49142744951e-05
<class 'numpy.float64'>
4.49079112377e-05
<class 'numpy.float64'>
4.49015497861e-05
<class 'numpy.float64'>
4.48951901388e-05
<class 'numpy.float64'>
4.48888322955e-05
<class 'numpy.float64'>
4.48824762554e-05
<class 'numpy.float64'>
4.48761220174e-05
<class 'numpy.float64'>
4.48697695813e-05
<class 'numpy.float64'>
4.48634189462e-05
<class 'numpy.float64'>
4.48570701109e-05
<class 'numpy.float64'>
4.4850723075e-05
<class 'numpy.float64'>
4.48443778377e-05
<class 'numpy.float64'>
4.4838034398e-05
<class 'numpy.float64'>
4.48316927555e-05
<class 'numpy.float64'>
4.48253529089e-05
<class 'numpy.float64'>
4.48190148584e-05
<class 'numpy.float64'>
4.48126786023e-05
<class 'numpy.float64'>
4.48063441401e-05
<class 'numpy.float64'>
4.48000114711e-05
<class 'numpy.float64'>
4.47936805948e-05
<class 'numpy.float64'>
4.47873515097e-05
<class 'numpy.float64'>
4.47810242158e-05
<class 'numpy.float64'>
4.47746987125e-05
<class 'numpy.float64'>
4.47683749979e-05
<class 'numpy.float64'>
4.47620530723e-05
<class 'numpy.float64'>
4.47557329343e-05
<class 'numpy.float64'>
4.47494145836e-05
<class 'numpy.float64'>
4.47430980193e-05
<class 'numpy.float64'>
4.47367832406e-05
<class 'numpy.float64'>
4.47304702468e-05
<class 'numpy.float64'>
4.47241590369e-05
<class 'numpy.float64'>
4.47178496105e-05
<class 'numpy.float64'>
4.47115419666e-05
<class 'numpy.float64'>
4.47052361045e-05
<class 'numpy.float64'>
4.46989320237e-05
<class 'numpy.float64'>
4.46926297229e-05
<class 'numpy.float64'>
4.46863292018e-05
<class 'numpy.float64'>
4.46800304594e-05
<class 'numpy.float64'>
4.46737334951e-05
<class 'numpy.float64'>
4.46674383081e-05
<class 'numpy.float64'>
4.46611448976e-05
<class 'numpy.float64'>
4.46548532631e-05
<class 'numpy.float64'>
4.46485634035e-05
<class 'numpy.float64'>
4.46422753182e-05
<class 'numpy.float64'>
4.46359890063e-05
<class 'numpy.float64'>
4.46297044672e-05
<class 'numpy.float64'>
4.46234217002e-05
<class 'numpy.float64'>
4.46171407045e-05
<class 'numpy.float64'>
4.46108614796e-05
<class 'numpy.float64'>
4.4604584024e-05
<class 'numpy.float64'>
4.45983083378e-05
<class 'numpy.float64'>
4.45920344198e-05
<class 'numpy.float64'>
4.45857622689e-05
<class 'numpy.float64'>
4.45794918854e-05
<class 'numpy.float64'>
4.45732232675e-05
<class 'numpy.float64'>
4.45669564151e-05
<class 'numpy.float64'>
4.45606913276e-05
<class 'numpy.float64'>
4.45544280035e-05
<class 'numpy.float64'>
4.45481664423e-05
<class 'numpy.float64'>
4.45419066438e-05
<class 'numpy.float64'>
4.45356486068e-05
<class 'numpy.float64'>
4.45293923305e-05
<class 'numpy.float64'>
4.45231378145e-05
<class 'numpy.float64'>
4.45168850577e-05
<class 'numpy.float64'>
4.45106340595e-05
<class 'numpy.float64'>
4.45043848191e-05
<class 'numpy.float64'>
4.44981373359e-05
<class 'numpy.float64'>
4.4491891609e-05
<class 'numpy.float64'>
4.44856476382e-05
<class 'numpy.float64'>
4.44794054215e-05
<class 'numpy.float64'>
4.44731649595e-05
<class 'numpy.float64'>
4.44669262507e-05
<class 'numpy.float64'>
4.44606892948e-05
<class 'numpy.float64'>
4.44544540905e-05
<class 'numpy.float64'>
4.44482206377e-05
<class 'numpy.float64'>
4.44419889351e-05
<class 'numpy.float64'>
4.44357589825e-05
<class 'numpy.float64'>
4.44295307789e-05
<class 'numpy.float64'>
4.44233043232e-05
<class 'numpy.float64'>
4.44170796153e-05
<class 'numpy.float64'>
4.44108566539e-05
<class 'numpy.float64'>
4.44046354387e-05
<class 'numpy.float64'>
4.43984159689e-05
<class 'numpy.float64'>
4.43921982437e-05
<class 'numpy.float64'>
4.4385982262e-05
<class 'numpy.float64'>
4.43797680237e-05
<class 'numpy.float64'>
4.43735555276e-05
<class 'numpy.float64'>
4.43673447733e-05
<class 'numpy.float64'>
4.43611357596e-05
<class 'numpy.float64'>
4.43549284864e-05
<class 'numpy.float64'>
4.43487229525e-05
<class 'numpy.float64'>
4.43425191573e-05
<class 'numpy.float64'>
4.43363171003e-05
<class 'numpy.float64'>
4.43301167803e-05
<class 'numpy.float64'>
4.43239181966e-05
<class 'numpy.float64'>
4.43177213494e-05
<class 'numpy.float64'>
4.43115262367e-05
<class 'numpy.float64'>
4.43053328585e-05
<class 'numpy.float64'>
4.42991412137e-05
<class 'numpy.float64'>
4.42929513018e-05
<class 'numpy.float64'>
4.42867631223e-05
<class 'numpy.float64'>
4.4280576674e-05
<class 'numpy.float64'>
4.42743919563e-05
<class 'numpy.float64'>
4.42682089688e-05
<class 'numpy.float64'>
4.42620277105e-05
<class 'numpy.float64'>
4.42558481806e-05
<class 'numpy.float64'>
4.42496703783e-05
<class 'numpy.float64'>
4.42434943035e-05
<class 'numpy.float64'>
4.42373199547e-05
<class 'numpy.float64'>
4.42311473317e-05
<class 'numpy.float64'>
4.42249764334e-05
<class 'numpy.float64'>
4.42188072595e-05
<class 'numpy.float64'>
4.42126398089e-05
<class 'numpy.float64'>
4.42064740808e-05
<class 'numpy.float64'>
4.4200310075e-05
<class 'numpy.float64'>
4.41941477904e-05
<class 'numpy.float64'>
4.41879872262e-05
<class 'numpy.float64'>
4.4181828382e-05
<class 'numpy.float64'>
4.41756712569e-05
<class 'numpy.float64'>
4.41695158501e-05
<class 'numpy.float64'>
4.41633621611e-05
<class 'numpy.float64'>
4.41572101891e-05
<class 'numpy.float64'>
4.41510599329e-05
<class 'numpy.float64'>
4.41449113929e-05
<class 'numpy.float64'>
4.41387645673e-05
<class 'numpy.float64'>
4.41326194556e-05
<class 'numpy.float64'>
4.41264760576e-05
<class 'numpy.float64'>
4.41203343721e-05
<class 'numpy.float64'>
4.41141943986e-05
<class 'numpy.float64'>
4.4108056136e-05
<class 'numpy.float64'>
4.41019195842e-05
<class 'numpy.float64'>
4.40957847422e-05
<class 'numpy.float64'>
4.40896516092e-05
<class 'numpy.float64'>
4.40835201845e-05
<class 'numpy.float64'>
4.40773904675e-05
<class 'numpy.float64'>
4.40712624575e-05
<class 'numpy.float64'>
4.40651361534e-05
<class 'numpy.float64'>
4.4059011555e-05
<class 'numpy.float64'>
4.40528886612e-05
<class 'numpy.float64'>
4.40467674718e-05
<class 'numpy.float64'>
4.40406479856e-05
<class 'numpy.float64'>
4.4034530202e-05
<class 'numpy.float64'>
4.40284141205e-05
<class 'numpy.float64'>
4.402229974e-05
<class 'numpy.float64'>
4.401618706e-05
<class 'numpy.float64'>
4.401007608e-05
<class 'numpy.float64'>
4.40039667989e-05
<class 'numpy.float64'>
4.39978592164e-05
<class 'numpy.float64'>
4.39917533314e-05
<class 'numpy.float64'>
4.39856491437e-05
<class 'numpy.float64'>
4.3979546652e-05
<class 'numpy.float64'>
4.3973445856e-05
<class 'numpy.float64'>
4.39673467546e-05
<class 'numpy.float64'>
4.39612493473e-05
<class 'numpy.float64'>
4.39551536335e-05
<class 'numpy.float64'>
4.39490596126e-05
<class 'numpy.float64'>
4.39429672838e-05
<class 'numpy.float64'>
4.39368766462e-05
<class 'numpy.float64'>
4.39307876991e-05
<class 'numpy.float64'>
4.39247004423e-05
<class 'numpy.float64'>
4.39186148742e-05
<class 'numpy.float64'>
4.39125309948e-05
<class 'numpy.float64'>
4.39064488033e-05
<class 'numpy.float64'>
4.39003682989e-05
<class 'numpy.float64'>
4.38942894808e-05
<class 'numpy.float64'>
4.38882123482e-05
<class 'numpy.float64'>
4.3882136901e-05
<class 'numpy.float64'>
4.3876063138e-05
<class 'numpy.float64'>
4.38699910583e-05
<class 'numpy.float64'>
4.38639206616e-05
<class 'numpy.float64'>
4.38578519472e-05
<class 'numpy.float64'>
4.38517849144e-05
<class 'numpy.float64'>
4.38457195623e-05
<class 'numpy.float64'>
4.38396558902e-05
<class 'numpy.float64'>
4.38335938975e-05
<class 'numpy.float64'>
4.38275335835e-05
<class 'numpy.float64'>
4.38214749476e-05
<class 'numpy.float64'>
4.38154179888e-05
<class 'numpy.float64'>
4.38093627068e-05
<class 'numpy.float64'>
4.38033091006e-05
<class 'numpy.float64'>
4.37972571697e-05
<class 'numpy.float64'>
4.37912069131e-05
<class 'numpy.float64'>
4.37851583306e-05
<class 'numpy.float64'>
4.3779111421e-05
<class 'numpy.float64'>
4.37730661841e-05
<class 'numpy.float64'>
4.37670226187e-05
<class 'numpy.float64'>
4.37609807246e-05
<class 'numpy.float64'>
4.37549405006e-05
<class 'numpy.float64'>
4.37489019463e-05
<class 'numpy.float64'>
4.37428650611e-05
<class 'numpy.float64'>
4.37368298438e-05
<class 'numpy.float64'>
4.37307962946e-05
<class 'numpy.float64'>
4.37247644121e-05
<class 'numpy.float64'>
4.37187341957e-05
<class 'numpy.float64'>
4.37127056449e-05
<class 'numpy.float64'>
4.37066787586e-05
<class 'numpy.float64'>
4.37006535367e-05
<class 'numpy.float64'>
4.36946299785e-05
<class 'numpy.float64'>
4.36886080826e-05
<class 'numpy.float64'>
4.3682587849e-05
<class 'numpy.float64'>
4.36765692767e-05
<class 'numpy.float64'>
4.36705523651e-05
<class 'numpy.float64'>
4.36645371135e-05
<class 'numpy.float64'>
4.36585235211e-05
<class 'numpy.float64'>
4.36525115873e-05
<class 'numpy.float64'>
4.36465013116e-05
<class 'numpy.float64'>
4.36404926932e-05
<class 'numpy.float64'>
4.3634485731e-05
<class 'numpy.float64'>
4.3628480425e-05
<class 'numpy.float64'>
4.36224767742e-05
<class 'numpy.float64'>
4.36164747779e-05
<class 'numpy.float64'>
4.36104744352e-05
<class 'numpy.float64'>
4.36044757459e-05
<class 'numpy.float64'>
4.35984787088e-05
<class 'numpy.float64'>
4.35924833237e-05
<class 'numpy.float64'>
4.35864895895e-05
<class 'numpy.float64'>
4.35804975059e-05
<class 'numpy.float64'>
4.35745070722e-05
<class 'numpy.float64'>
4.35685182869e-05
<class 'numpy.float64'>
4.35625311505e-05
<class 'numpy.float64'>
4.35565456616e-05
<class 'numpy.float64'>
4.35505618198e-05
<class 'numpy.float64'>
4.35445796244e-05
<class 'numpy.float64'>
4.35385990743e-05
<class 'numpy.float64'>
4.35326201692e-05
<class 'numpy.float64'>
4.35266429088e-05
<class 'numpy.float64'>
4.35206672917e-05
<class 'numpy.float64'>
4.35146933174e-05
<class 'numpy.float64'>
4.35087209855e-05
<class 'numpy.float64'>
4.35027502951e-05
<class 'numpy.float64'>
4.34967812458e-05
<class 'numpy.float64'>
4.34908138367e-05
<class 'numpy.float64'>
4.34848480672e-05
<class 'numpy.float64'>
4.34788839362e-05
<class 'numpy.float64'>
4.34729214436e-05
<class 'numpy.float64'>
4.34669605886e-05
<class 'numpy.float64'>
4.34610013703e-05
<class 'numpy.float64'>
4.34550437881e-05
<class 'numpy.float64'>
4.34490878416e-05
<class 'numpy.float64'>
4.34431335298e-05
<class 'numpy.float64'>
4.34371808522e-05
<class 'numpy.float64'>
4.34312298078e-05
<class 'numpy.float64'>
4.34252803965e-05
<class 'numpy.float64'>
4.34193326172e-05
<class 'numpy.float64'>
4.34133864694e-05
<class 'numpy.float64'>
4.34074419523e-05
<class 'numpy.float64'>
4.34014990656e-05
<class 'numpy.float64'>
4.33955578081e-05
<class 'numpy.float64'>
4.33896181792e-05
<class 'numpy.float64'>
4.33836801788e-05
<class 'numpy.float64'>
4.33777438056e-05
<class 'numpy.float64'>
4.33718090591e-05
<class 'numpy.float64'>
4.33658759389e-05
<class 'numpy.float64'>
4.3359944444e-05
<class 'numpy.float64'>
4.33540145738e-05
<class 'numpy.float64'>
4.33480863275e-05
<class 'numpy.float64'>
4.3342159705e-05
<class 'numpy.float64'>
4.3336234705e-05
<class 'numpy.float64'>
4.33303113273e-05
<class 'numpy.float64'>
4.33243895709e-05
<class 'numpy.float64'>
4.33184694351e-05
<class 'numpy.float64'>
4.33125509196e-05
<class 'numpy.float64'>
4.33066340234e-05
<class 'numpy.float64'>
4.33007187462e-05
<class 'numpy.float64'>
4.32948050868e-05
<class 'numpy.float64'>
4.32888930448e-05
<class 'numpy.float64'>
4.32829826199e-05
<class 'numpy.float64'>
4.32770738108e-05
<class 'numpy.float64'>
4.32711666173e-05
<class 'numpy.float64'>
4.32652610383e-05
<class 'numpy.float64'>
4.32593570739e-05
<class 'numpy.float64'>
4.32534547223e-05
<class 'numpy.float64'>
4.3247553984e-05
<class 'numpy.float64'>
4.32416548576e-05
<class 'numpy.float64'>
4.32357573429e-05
<class 'numpy.float64'>
4.32298614388e-05
<class 'numpy.float64'>
4.32239671447e-05
<class 'numpy.float64'>
4.32180744604e-05
<class 'numpy.float64'>
4.3212183385e-05
<class 'numpy.float64'>
4.32062939172e-05
<class 'numpy.float64'>
4.32004060575e-05
<class 'numpy.float64'>
4.31945198044e-05
<class 'numpy.float64'>
4.31886351575e-05
<class 'numpy.float64'>
4.3182752116e-05
<class 'numpy.float64'>
4.31768706796e-05
<class 'numpy.float64'>
4.31709908475e-05
<class 'numpy.float64'>
4.31651126186e-05
<class 'numpy.float64'>
4.31592359929e-05
<class 'numpy.float64'>
4.31533609693e-05
<class 'numpy.float64'>
4.31474875474e-05
<class 'numpy.float64'>
4.31416157263e-05
<class 'numpy.float64'>
4.31357455054e-05
<class 'numpy.float64'>
4.31298768843e-05
<class 'numpy.float64'>
4.31240098621e-05
<class 'numpy.float64'>
4.31181444384e-05
<class 'numpy.float64'>
4.31122806122e-05
<class 'numpy.float64'>
4.3106418383e-05
<class 'numpy.float64'>
4.31005577504e-05
<class 'numpy.float64'>
4.30946987131e-05
<class 'numpy.float64'>
4.30888412712e-05
<class 'numpy.float64'>
4.30829854237e-05
<class 'numpy.float64'>
4.30771311698e-05
<class 'numpy.float64'>
4.3071278509e-05
<class 'numpy.float64'>
4.30654274406e-05
<class 'numpy.float64'>
4.30595779643e-05
<class 'numpy.float64'>
4.30537300789e-05
<class 'numpy.float64'>
4.30478837841e-05
<class 'numpy.float64'>
4.30420390789e-05
<class 'numpy.float64'>
4.30361959633e-05
<class 'numpy.float64'>
4.30303544359e-05
<class 'numpy.float64'>
4.30245144967e-05
<class 'numpy.float64'>
4.30186761445e-05
<class 'numpy.float64'>
4.30128393791e-05
<class 'numpy.float64'>
4.30070041998e-05
<class 'numpy.float64'>
4.30011706053e-05
<class 'numpy.float64'>
4.2995338596e-05
<class 'numpy.float64'>
4.29895081705e-05
<class 'numpy.float64'>
4.29836793285e-05
<class 'numpy.float64'>
4.2977852069e-05
<class 'numpy.float64'>
4.29720263917e-05
<class 'numpy.float64'>
4.29662022961e-05
<class 'numpy.float64'>
4.29603797808e-05
<class 'numpy.float64'>
4.29545588461e-05
<class 'numpy.float64'>
4.29487394909e-05
<class 'numpy.float64'>
4.29429217143e-05
<class 'numpy.float64'>
4.29371055161e-05
<class 'numpy.float64'>
4.29312908954e-05
<class 'numpy.float64'>
4.29254778516e-05
<class 'numpy.float64'>
4.29196663843e-05
<class 'numpy.float64'>
4.29138564925e-05
<class 'numpy.float64'>
4.29080481755e-05
<class 'numpy.float64'>
4.29022414332e-05
<class 'numpy.float64'>
4.2896436265e-05
<class 'numpy.float64'>
4.28906326693e-05
<class 'numpy.float64'>
4.28848306462e-05
<class 'numpy.float64'>
4.2879030195e-05
<class 'numpy.float64'>
4.2873231315e-05
<class 'numpy.float64'>
4.28674340053e-05
<class 'numpy.float64'>
4.28616382658e-05
<class 'numpy.float64'>
4.28558440954e-05
<class 'numpy.float64'>
4.28500514937e-05
<class 'numpy.float64'>
4.28442604598e-05
<class 'numpy.float64'>
4.28384709937e-05
<class 'numpy.float64'>
4.28326830938e-05
<class 'numpy.float64'>
4.28268967602e-05
<class 'numpy.float64'>
4.28211119921e-05
<class 'numpy.float64'>
4.28153287887e-05
<class 'numpy.float64'>
4.28095471498e-05
<class 'numpy.float64'>
4.28037670741e-05
<class 'numpy.float64'>
4.27979885614e-05
<class 'numpy.float64'>
4.27922116109e-05
<class 'numpy.float64'>
4.27864362221e-05
<class 'numpy.float64'>
4.27806623942e-05
<class 'numpy.float64'>
4.27748901268e-05
<class 'numpy.float64'>
4.27691194192e-05
<class 'numpy.float64'>
4.27633502707e-05
<class 'numpy.float64'>
4.27575826802e-05
<class 'numpy.float64'>
4.27518166481e-05
<class 'numpy.float64'>
4.27460521731e-05
<class 'numpy.float64'>
4.27402892545e-05
<class 'numpy.float64'>
4.2734527892e-05
<class 'numpy.float64'>
4.27287680849e-05
<class 'numpy.float64'>
4.27230098322e-05
<class 'numpy.float64'>
4.27172531335e-05
<class 'numpy.float64'>
4.27114979887e-05
<class 'numpy.float64'>
4.27057443963e-05
<class 'numpy.float64'>
4.26999923561e-05
<class 'numpy.float64'>
4.26942418677e-05
<class 'numpy.float64'>
4.268849293e-05
<class 'numpy.float64'>
4.26827455426e-05
<class 'numpy.float64'>
4.26769997048e-05
<class 'numpy.float64'>
4.26712554162e-05
<class 'numpy.float64'>
4.26655126757e-05
<class 'numpy.float64'>
4.26597714832e-05
<class 'numpy.float64'>
4.26540318379e-05
<class 'numpy.float64'>
4.26482937391e-05
<class 'numpy.float64'>
4.26425571862e-05
<class 'numpy.float64'>
4.26368221786e-05
<class 'numpy.float64'>
4.26310887155e-05
<class 'numpy.float64'>
4.26253567965e-05
<class 'numpy.float64'>
4.26196264211e-05
<class 'numpy.float64'>
4.26138975884e-05
<class 'numpy.float64'>
4.26081702976e-05
<class 'numpy.float64'>
4.26024445484e-05
<class 'numpy.float64'>
4.25967203402e-05
<class 'numpy.float64'>
4.25909976722e-05
<class 'numpy.float64'>
4.25852765442e-05
<class 'numpy.float64'>
4.25795569548e-05
<class 'numpy.float64'>
4.2573838904e-05
<class 'numpy.float64'>
4.25681223909e-05
<class 'numpy.float64'>
4.25624074152e-05
<class 'numpy.float64'>
4.25566939759e-05
<class 'numpy.float64'>
4.25509820727e-05
<class 'numpy.float64'>
4.25452717046e-05
<class 'numpy.float64'>
4.25395628712e-05
<class 'numpy.float64'>
4.25338555721e-05
<class 'numpy.float64'>
4.25281498062e-05
<class 'numpy.float64'>
4.25224455736e-05
<class 'numpy.float64'>
4.25167428727e-05
<class 'numpy.float64'>
4.25110417037e-05
<class 'numpy.float64'>
4.25053420657e-05
<class 'numpy.float64'>
4.24996439581e-05
<class 'numpy.float64'>
4.249394738e-05
<class 'numpy.float64'>
4.24882523312e-05
<class 'numpy.float64'>
4.24825588111e-05
<class 'numpy.float64'>
4.24768668186e-05
<class 'numpy.float64'>
4.24711763535e-05
<class 'numpy.float64'>
4.24654874154e-05
<class 'numpy.float64'>
4.24598000031e-05
<class 'numpy.float64'>
4.2454114116e-05
<class 'numpy.float64'>
4.2448429754e-05
<class 'numpy.float64'>
4.24427469161e-05
<class 'numpy.float64'>
4.24370656022e-05
<class 'numpy.float64'>
4.24313858108e-05
<class 'numpy.float64'>
4.2425707542e-05
<class 'numpy.float64'>
4.24200307949e-05
<class 'numpy.float64'>
4.24143555692e-05
<class 'numpy.float64'>
4.24086818637e-05
<class 'numpy.float64'>
4.24030096784e-05
<class 'numpy.float64'>
4.23973390122e-05
<class 'numpy.float64'>
4.23916698649e-05
<class 'numpy.float64'>
4.23860022356e-05
<class 'numpy.float64'>
4.2380336124e-05
<class 'numpy.float64'>
4.23746715291e-05
<class 'numpy.float64'>
4.23690084504e-05
<class 'numpy.float64'>
4.23633468874e-05
<class 'numpy.float64'>
4.23576868395e-05
<class 'numpy.float64'>
4.23520283059e-05
<class 'numpy.float64'>
4.23463712867e-05
<class 'numpy.float64'>
4.23407157803e-05
<class 'numpy.float64'>
4.23350617864e-05
<class 'numpy.float64'>
4.23294093045e-05
<class 'numpy.float64'>
4.23237583344e-05
<class 'numpy.float64'>
4.2318108875e-05
<class 'numpy.float64'>
4.23124609257e-05
<class 'numpy.float64'>
4.23068144859e-05
<class 'numpy.float64'>
4.23011695551e-05
<class 'numpy.float64'>
4.22955261328e-05
<class 'numpy.float64'>
4.22898842185e-05
<class 'numpy.float64'>
4.2284243811e-05
<class 'numpy.float64'>
4.227860491e-05
<class 'numpy.float64'>
4.22729675153e-05
<class 'numpy.float64'>
4.22673316259e-05
<class 'numpy.float64'>
4.22616972412e-05
<class 'numpy.float64'>
4.22560643604e-05
<class 'numpy.float64'>
4.22504329834e-05
<class 'numpy.float64'>
4.22448031094e-05
<class 'numpy.float64'>
4.22391747375e-05
<class 'numpy.float64'>
4.22335478679e-05
<class 'numpy.float64'>
4.22279224987e-05
<class 'numpy.float64'>
4.22222986304e-05
<class 'numpy.float64'>
4.22166762622e-05
<class 'numpy.float64'>
4.22110553933e-05
<class 'numpy.float64'>
4.2205436023e-05
<class 'numpy.float64'>
4.21998181509e-05
<class 'numpy.float64'>
4.21942017763e-05
<class 'numpy.float64'>
4.21885868986e-05
<class 'numpy.float64'>
4.21829735173e-05
<class 'numpy.float64'>
4.21773616319e-05
<class 'numpy.float64'>
4.21717512413e-05
<class 'numpy.float64'>
4.21661423455e-05
<class 'numpy.float64'>
4.21605349435e-05
<class 'numpy.float64'>
4.21549290353e-05
<class 'numpy.float64'>
4.21493246193e-05
<class 'numpy.float64'>
4.21437216959e-05
<class 'numpy.float64'>
4.21381202636e-05
<class 'numpy.float64'>
4.21325203227e-05
<class 'numpy.float64'>
4.2126921872e-05
<class 'numpy.float64'>
4.2121324911e-05
<class 'numpy.float64'>
4.21157294392e-05
<class 'numpy.float64'>
4.21101354562e-05
<class 'numpy.float64'>
4.2104542961e-05
<class 'numpy.float64'>
4.20989519531e-05
<class 'numpy.float64'>
4.20933624324e-05
<class 'numpy.float64'>
4.20877743977e-05
<class 'numpy.float64'>
4.20821878489e-05
<class 'numpy.float64'>
4.20766027848e-05
<class 'numpy.float64'>
4.2071019205e-05
<class 'numpy.float64'>
4.20654371091e-05
<class 'numpy.float64'>
4.20598564967e-05
<class 'numpy.float64'>
4.20542773667e-05
<class 'numpy.float64'>
4.20486997192e-05
<class 'numpy.float64'>
4.20431235526e-05
<class 'numpy.float64'>
4.20375488674e-05
<class 'numpy.float64'>
4.20319756623e-05
<class 'numpy.float64'>
4.20264039368e-05
<class 'numpy.float64'>
4.20208336905e-05
<class 'numpy.float64'>
4.20152649225e-05
<class 'numpy.float64'>
4.20096976329e-05
<class 'numpy.float64'>
4.20041318205e-05
<class 'numpy.float64'>
4.19985674848e-05
<class 'numpy.float64'>
4.19930046251e-05
<class 'numpy.float64'>
4.19874432413e-05
<class 'numpy.float64'>
4.19818833323e-05
<class 'numpy.float64'>
4.19763248979e-05
<class 'numpy.float64'>
4.19707679371e-05
<class 'numpy.float64'>
4.19652124495e-05
<class 'numpy.float64'>
4.19596584348e-05
<class 'numpy.float64'>
4.19541058923e-05
<class 'numpy.float64'>
4.19485548209e-05
<class 'numpy.float64'>
4.19430052205e-05
<class 'numpy.float64'>
4.19374570906e-05
<class 'numpy.float64'>
4.19319104301e-05
<class 'numpy.float64'>
4.19263652391e-05
<class 'numpy.float64'>
4.19208215164e-05
<class 'numpy.float64'>
4.19152792619e-05
<class 'numpy.float64'>
4.19097384748e-05
<class 'numpy.float64'>
4.19041991542e-05
<class 'numpy.float64'>
4.18986613002e-05
<class 'numpy.float64'>
4.18931249117e-05
<class 'numpy.float64'>
4.18875899881e-05
<class 'numpy.float64'>
4.18820565292e-05
<class 'numpy.float64'>
4.18765245341e-05
<class 'numpy.float64'>
4.18709940022e-05
<class 'numpy.float64'>
4.18654649331e-05
<class 'numpy.float64'>
4.18599373261e-05
<class 'numpy.float64'>
4.18544111811e-05
<class 'numpy.float64'>
4.18488864968e-05
<class 'numpy.float64'>
4.18433632727e-05
<class 'numpy.float64'>
4.18378415086e-05
<class 'numpy.float64'>
4.18323212041e-05
<class 'numpy.float64'>
4.18268023578e-05
<class 'numpy.float64'>
4.18212849699e-05
<class 'numpy.float64'>
4.18157690393e-05
<class 'numpy.float64'>
4.18102545659e-05
<class 'numpy.float64'>
4.18047415484e-05
<class 'numpy.float64'>
4.17992299873e-05
<class 'numpy.float64'>
4.17937198811e-05
<class 'numpy.float64'>
4.17882112295e-05
<class 'numpy.float64'>
4.17827040323e-05
<class 'numpy.float64'>
4.1777198288e-05
<class 'numpy.float64'>
4.1771693997e-05
<class 'numpy.float64'>
4.17661911584e-05
<class 'numpy.float64'>
4.17606897711e-05
<class 'numpy.float64'>
4.17551898354e-05
<class 'numpy.float64'>
4.17496913505e-05
<class 'numpy.float64'>
4.17441943152e-05
<class 'numpy.float64'>
4.17386987295e-05
<class 'numpy.float64'>
4.17332045926e-05
<class 'numpy.float64'>
4.17277119041e-05
<class 'numpy.float64'>
4.17222206635e-05
<class 'numpy.float64'>
4.171673087e-05
<class 'numpy.float64'>
4.17112425228e-05
<class 'numpy.float64'>
4.17057556218e-05
<class 'numpy.float64'>
4.17002701664e-05
<class 'numpy.float64'>
4.16947861559e-05
<class 'numpy.float64'>
4.16893035895e-05
<class 'numpy.float64'>
4.16838224668e-05
<class 'numpy.float64'>
4.16783427876e-05
<class 'numpy.float64'>
4.16728645507e-05
<class 'numpy.float64'>
4.16673877559e-05
<class 'numpy.float64'>
4.16619124027e-05
<class 'numpy.float64'>
4.16564384904e-05
<class 'numpy.float64'>
4.16509660183e-05
<class 'numpy.float64'>
4.16454949859e-05
<class 'numpy.float64'>
4.16400253927e-05
<class 'numpy.float64'>
4.1634557238e-05
<class 'numpy.float64'>
4.16290905215e-05
<class 'numpy.float64'>
4.16236252426e-05
<class 'numpy.float64'>
4.16181614006e-05
<class 'numpy.float64'>
4.16126989946e-05
<class 'numpy.float64'>
4.16072380247e-05
<class 'numpy.float64'>
4.16017784898e-05
<class 'numpy.float64'>
4.159632039e-05
<class 'numpy.float64'>
4.15908637235e-05
<class 'numpy.float64'>
4.15854084913e-05
<class 'numpy.float64'>
4.15799546917e-05
<class 'numpy.float64'>
4.15745023244e-05
<class 'numpy.float64'>
4.15690513893e-05
<class 'numpy.float64'>
4.1563601885e-05
<class 'numpy.float64'>
4.15581538117e-05
<class 'numpy.float64'>
4.15527071682e-05
<class 'numpy.float64'>
4.15472619547e-05
<class 'numpy.float64'>
4.154181817e-05
<class 'numpy.float64'>
4.15363758135e-05
<class 'numpy.float64'>
4.15309348853e-05
<class 'numpy.float64'>
4.1525495384e-05
<class 'numpy.float64'>
4.15200573096e-05
<class 'numpy.float64'>
4.15146206617e-05
<class 'numpy.float64'>
4.15091854391e-05
<class 'numpy.float64'>
4.15037516415e-05
<class 'numpy.float64'>
4.14983192687e-05
<class 'numpy.float64'>
4.14928883197e-05
<class 'numpy.float64'>
4.14874587943e-05
<class 'numpy.float64'>
4.14820306912e-05
<class 'numpy.float64'>
4.14766040108e-05
<class 'numpy.float64'>
4.14711787519e-05
<class 'numpy.float64'>
4.14657549145e-05
<class 'numpy.float64'>
4.14603324972e-05
<class 'numpy.float64'>
4.14549115004e-05
<class 'numpy.float64'>
4.14494919226e-05
<class 'numpy.float64'>
4.14440737641e-05
<class 'numpy.float64'>
4.14386570238e-05
<class 'numpy.float64'>
4.14332417014e-05
<class 'numpy.float64'>
4.14278277964e-05
<class 'numpy.float64'>
4.14224153076e-05
<class 'numpy.float64'>
4.14170042352e-05
<class 'numpy.float64'>
4.14115945782e-05
<class 'numpy.float64'>
4.14061863367e-05
<class 'numpy.float64'>
4.14007795093e-05
<class 'numpy.float64'>
4.13953740957e-05
<class 'numpy.float64'>
4.13899700956e-05
<class 'numpy.float64'>
4.13845675085e-05
<class 'numpy.float64'>
4.13791663334e-05
<class 'numpy.float64'>
4.13737665701e-05
<class 'numpy.float64'>
4.1368368218e-05
<class 'numpy.float64'>
4.13629712763e-05
<class 'numpy.float64'>
4.13575757447e-05
<class 'numpy.float64'>
4.13521816226e-05
<class 'numpy.float64'>
4.13467889096e-05
<class 'numpy.float64'>
4.13413976047e-05
<class 'numpy.float64'>
4.13360077076e-05
<class 'numpy.float64'>
4.13306192179e-05
<class 'numpy.float64'>
4.13252321351e-05
<class 'numpy.float64'>
4.13198464582e-05
<class 'numpy.float64'>
4.1314462187e-05
<class 'numpy.float64'>
4.13090793209e-05
<class 'numpy.float64'>
4.13036978593e-05
<class 'numpy.float64'>
4.12983178016e-05
<class 'numpy.float64'>
4.12929391475e-05
<class 'numpy.float64'>
4.12875618959e-05
<class 'numpy.float64'>
4.12821860471e-05
<class 'numpy.float64'>
4.12768115997e-05
<class 'numpy.float64'>
4.12714385536e-05
<class 'numpy.float64'>
4.12660669083e-05
<class 'numpy.float64'>
4.12606966629e-05
<class 'numpy.float64'>
4.12553278173e-05
<class 'numpy.float64'>
4.12499603707e-05
<class 'numpy.float64'>
4.12445943224e-05
<class 'numpy.float64'>
4.12392296725e-05
<class 'numpy.float64'>
4.12338664196e-05
<class 'numpy.float64'>
4.12285045633e-05
<class 'numpy.float64'>
4.12231441037e-05
<class 'numpy.float64'>
4.12177850398e-05
<class 'numpy.float64'>
4.12124273711e-05
<class 'numpy.float64'>
4.12070710971e-05
<class 'numpy.float64'>
4.12017162171e-05
<class 'numpy.float64'>
4.11963627308e-05
<class 'numpy.float64'>
4.11910106374e-05
<class 'numpy.float64'>
4.11856599363e-05
<class 'numpy.float64'>
4.11803106276e-05
<class 'numpy.float64'>
4.11749627098e-05
<class 'numpy.float64'>
4.11696161832e-05
<class 'numpy.float64'>
4.11642710469e-05
<class 'numpy.float64'>
4.11589273002e-05
<class 'numpy.float64'>
4.11535849428e-05
<class 'numpy.float64'>
4.11482439741e-05
<class 'numpy.float64'>
4.11429043935e-05
<class 'numpy.float64'>
4.11375662007e-05
<class 'numpy.float64'>
4.11322293944e-05
<class 'numpy.float64'>
4.11268939753e-05
<class 'numpy.float64'>
4.11215599418e-05
<class 'numpy.float64'>
4.11162272939e-05
<class 'numpy.float64'>
4.11108960309e-05
<class 'numpy.float64'>
4.11055661521e-05
<class 'numpy.float64'>
4.11002376572e-05
<class 'numpy.float64'>
4.10949105456e-05
<class 'numpy.float64'>
4.10895848167e-05
<class 'numpy.float64'>
4.10842604699e-05
<class 'numpy.float64'>
4.10789375051e-05
<class 'numpy.float64'>
4.1073615921e-05
<class 'numpy.float64'>
4.10682957176e-05
<class 'numpy.float64'>
4.10629768944e-05
<class 'numpy.float64'>
4.10576594508e-05
<class 'numpy.float64'>
4.10523433861e-05
<class 'numpy.float64'>
4.10470286996e-05
<class 'numpy.float64'>
4.10417153914e-05
<class 'numpy.float64'>
4.103640346e-05
<class 'numpy.float64'>
4.1031092906e-05
<class 'numpy.float64'>
4.1025783728e-05
<class 'numpy.float64'>
4.1020475926e-05
<class 'numpy.float64'>
4.10151694987e-05
<class 'numpy.float64'>
4.10098644466e-05
<class 'numpy.float64'>
4.10045607685e-05
<class 'numpy.float64'>
4.09992584641e-05
<class 'numpy.float64'>
4.09939575327e-05
<class 'numpy.float64'>
4.09886579737e-05
<class 'numpy.float64'>
4.09833597869e-05
<class 'numpy.float64'>
4.09780629715e-05
<class 'numpy.float64'>
4.09727675273e-05
<class 'numpy.float64'>
4.09674734532e-05
<class 'numpy.float64'>
4.09621807491e-05
<class 'numpy.float64'>
4.09568894146e-05
<class 'numpy.float64'>
4.09515994483e-05
<class 'numpy.float64'>
4.0946310851e-05
<class 'numpy.float64'>
4.09410236211e-05
<class 'numpy.float64'>
4.09357377585e-05
<class 'numpy.float64'>
4.09304532626e-05
<class 'numpy.float64'>
4.0925170133e-05
<class 'numpy.float64'>
4.09198883688e-05
<class 'numpy.float64'>
4.091460797e-05
<class 'numpy.float64'>
4.09093289353e-05
<class 'numpy.float64'>
4.09040512651e-05
<class 'numpy.float64'>
4.08987749584e-05
<class 'numpy.float64'>
4.08935000147e-05
<class 'numpy.float64'>
4.08882264331e-05
<class 'numpy.float64'>
4.08829542137e-05
<class 'numpy.float64'>
4.08776833558e-05
<class 'numpy.float64'>
4.08724138588e-05
<class 'numpy.float64'>
4.08671457219e-05
<class 'numpy.float64'>
4.08618789451e-05
<class 'numpy.float64'>
4.08566135272e-05
<class 'numpy.float64'>
4.08513494684e-05
<class 'numpy.float64'>
4.08460867678e-05
<class 'numpy.float64'>
4.08408254251e-05
<class 'numpy.float64'>
4.08355654393e-05
<class 'numpy.float64'>
4.08303068105e-05
<class 'numpy.float64'>
4.08250495372e-05
<class 'numpy.float64'>
4.08197936201e-05
<class 'numpy.float64'>
4.0814539058e-05
<class 'numpy.float64'>
4.08092858504e-05
<class 'numpy.float64'>
4.0804033997e-05
<class 'numpy.float64'>
4.07987834971e-05
<class 'numpy.float64'>
4.07935343498e-05
<class 'numpy.float64'>
4.07882865554e-05
<class 'numpy.float64'>
4.07830401129e-05
<class 'numpy.float64'>
4.07777950217e-05
<class 'numpy.float64'>
4.07725512817e-05
<class 'numpy.float64'>
4.07673088919e-05
<class 'numpy.float64'>
4.07620678517e-05
<class 'numpy.float64'>
4.07568281614e-05
<class 'numpy.float64'>
4.07515898196e-05
<class 'numpy.float64'>
4.0746352826e-05
<class 'numpy.float64'>
4.07411171803e-05
<class 'numpy.float64'>
4.07358828818e-05
<class 'numpy.float64'>
4.07306499302e-05
<class 'numpy.float64'>
4.07254183248e-05
<class 'numpy.float64'>
4.07201880648e-05
<class 'numpy.float64'>
4.07149591504e-05
<class 'numpy.float64'>
4.07097315806e-05
<class 'numpy.float64'>
4.07045053547e-05
<class 'numpy.float64'>
4.06992804724e-05
<class 'numpy.float64'>
4.06940569335e-05
<class 'numpy.float64'>
4.06888347369e-05
<class 'numpy.float64'>
4.06836138827e-05
<class 'numpy.float64'>
4.06783943697e-05
<class 'numpy.float64'>
4.06731761982e-05
<class 'numpy.float64'>
4.06679593668e-05
<class 'numpy.float64'>
4.06627438753e-05
<class 'numpy.float64'>
4.06575297238e-05
<class 'numpy.float64'>
4.06523169109e-05
<class 'numpy.float64'>
4.06471054365e-05
<class 'numpy.float64'>
4.06418953001e-05
<class 'numpy.float64'>
4.06366865012e-05
<class 'numpy.float64'>
4.06314790389e-05
<class 'numpy.float64'>
4.06262729133e-05
<class 'numpy.float64'>
4.06210681235e-05
<class 'numpy.float64'>
4.06158646689e-05
<class 'numpy.float64'>
4.06106625494e-05
<class 'numpy.float64'>
4.06054617638e-05
<class 'numpy.float64'>
4.06002623128e-05
<class 'numpy.float64'>
4.05950641944e-05
<class 'numpy.float64'>
4.05898674091e-05
<class 'numpy.float64'>
4.05846719562e-05
<class 'numpy.float64'>
4.05794778347e-05
<class 'numpy.float64'>
4.05742850445e-05
<class 'numpy.float64'>
4.05690935853e-05
<class 'numpy.float64'>
4.05639034562e-05
<class 'numpy.float64'>
4.05587146569e-05
<class 'numpy.float64'>
4.05535271866e-05
<class 'numpy.float64'>
4.05483410453e-05
<class 'numpy.float64'>
4.0543156232e-05
<class 'numpy.float64'>
4.05379727464e-05
<class 'numpy.float64'>
4.05327905882e-05
<class 'numpy.float64'>
4.05276097563e-05
<class 'numpy.float64'>
4.05224302508e-05
<class 'numpy.float64'>
4.05172520709e-05
<class 'numpy.float64'>
4.05120752161e-05
<class 'numpy.float64'>
4.05068996856e-05
<class 'numpy.float64'>
4.05017254795e-05
<class 'numpy.float64'>
4.04965525972e-05
<class 'numpy.float64'>
4.04913810381e-05
<class 'numpy.float64'>
4.0486210801e-05
<class 'numpy.float64'>
4.04810418863e-05
<class 'numpy.float64'>
4.04758742931e-05
<class 'numpy.float64'>
4.04707080211e-05
<class 'numpy.float64'>
4.04655430694e-05
<class 'numpy.float64'>
4.04603794378e-05
<class 'numpy.float64'>
4.0455217126e-05
<class 'numpy.float64'>
4.04500561332e-05
<class 'numpy.float64'>
4.04448964588e-05
<class 'numpy.float64'>
4.04397381026e-05
<class 'numpy.float64'>
4.04345810635e-05
<class 'numpy.float64'>
4.04294253419e-05
<class 'numpy.float64'>
4.04242709366e-05
<class 'numpy.float64'>
4.04191178474e-05
<class 'numpy.float64'>
4.04139660735e-05
<class 'numpy.float64'>
4.04088156148e-05
<class 'numpy.float64'>
4.04036664707e-05
<class 'numpy.float64'>
4.03985186402e-05
<class 'numpy.float64'>
4.03933721235e-05
<class 'numpy.float64'>
4.03882269196e-05
<class 'numpy.float64'>
4.03830830285e-05
<class 'numpy.float64'>
4.0377940449e-05
<class 'numpy.float64'>
4.03727991812e-05
<class 'numpy.float64'>
4.03676592242e-05
<class 'numpy.float64'>
4.0362520578e-05
<class 'numpy.float64'>
4.03573832414e-05
<class 'numpy.float64'>
4.03522472145e-05
<class 'numpy.float64'>
4.03471124965e-05
<class 'numpy.float64'>
4.03419790871e-05
<class 'numpy.float64'>
4.03368469856e-05
<class 'numpy.float64'>
4.03317161917e-05
<class 'numpy.float64'>
4.03265867044e-05
<class 'numpy.float64'>
4.03214585238e-05
<class 'numpy.float64'>
4.03163316492e-05
<class 'numpy.float64'>
4.03112060801e-05
<class 'numpy.float64'>
4.03060818161e-05
<class 'numpy.float64'>
4.03009588562e-05
<class 'numpy.float64'>
4.02958372005e-05
<class 'numpy.float64'>
4.0290716848e-05
<class 'numpy.float64'>
4.02855977988e-05
<class 'numpy.float64'>
4.02804800523e-05
<class 'numpy.float64'>
4.02753636073e-05
<class 'numpy.float64'>
4.02702484641e-05
<class 'numpy.float64'>
4.02651346217e-05
<class 'numpy.float64'>
4.02600220799e-05
<class 'numpy.float64'>
4.02549108381e-05
<class 'numpy.float64'>
4.02498008957e-05
<class 'numpy.float64'>
4.02446922526e-05
<class 'numpy.float64'>
4.02395849078e-05
<class 'numpy.float64'>
4.02344788611e-05
<class 'numpy.float64'>
4.02293741118e-05
<class 'numpy.float64'>
4.02242706594e-05
<class 'numpy.float64'>
4.02191685039e-05
<class 'numpy.float64'>
4.02140676441e-05
<class 'numpy.float64'>
4.02089680802e-05
<class 'numpy.float64'>
4.0203869811e-05
<class 'numpy.float64'>
4.01987728366e-05
<class 'numpy.float64'>
4.0193677156e-05
<class 'numpy.float64'>
4.01885827694e-05
<class 'numpy.float64'>
4.01834896754e-05
<class 'numpy.float64'>
4.01783978742e-05
<class 'numpy.float64'>
4.01733073651e-05
<class 'numpy.float64'>
4.01682181476e-05
<class 'numpy.float64'>
4.01631302212e-05
<class 'numpy.float64'>
4.01580435855e-05
<class 'numpy.float64'>
4.01529582399e-05
<class 'numpy.float64'>
4.0147874184e-05
<class 'numpy.float64'>
4.01427914169e-05
<class 'numpy.float64'>
4.01377099388e-05
<class 'numpy.float64'>
4.01326297489e-05
<class 'numpy.float64'>
4.01275508465e-05
<class 'numpy.float64'>
4.01224732311e-05
<class 'numpy.float64'>
4.01173969025e-05
<class 'numpy.float64'>
4.01123218602e-05
<class 'numpy.float64'>
4.01072481036e-05
<class 'numpy.float64'>
4.01021756324e-05
<class 'numpy.float64'>
4.00971044457e-05
<class 'numpy.float64'>
4.00920345434e-05
<class 'numpy.float64'>
4.00869659247e-05
<class 'numpy.float64'>
4.00818985895e-05
<class 'numpy.float64'>
4.00768325368e-05
<class 'numpy.float64'>
4.00717677668e-05
<class 'numpy.float64'>
4.00667042782e-05
<class 'numpy.float64'>
4.00616420711e-05
<class 'numpy.float64'>
4.00565811447e-05
<class 'numpy.float64'>
4.00515214988e-05
<class 'numpy.float64'>
4.0046463133e-05
<class 'numpy.float64'>
4.00414060463e-05
<class 'numpy.float64'>
4.00363502386e-05
<class 'numpy.float64'>
4.00312957093e-05
<class 'numpy.float64'>
4.00262424578e-05
<class 'numpy.float64'>
4.00211904838e-05
<class 'numpy.float64'>
4.00161397867e-05
<class 'numpy.float64'>
4.00110903661e-05
<class 'numpy.float64'>
4.00060422215e-05
<class 'numpy.float64'>
4.00009953523e-05
<class 'numpy.float64'>
3.99959497584e-05
<class 'numpy.float64'>
3.99909054388e-05
<class 'numpy.float64'>
3.99858623934e-05
<class 'numpy.float64'>
3.99808206215e-05
<class 'numpy.float64'>
3.99757801227e-05
<class 'numpy.float64'>
3.99707408965e-05
<class 'numpy.float64'>
3.99657029423e-05
<class 'numpy.float64'>
3.99606662601e-05
<class 'numpy.float64'>
3.99556308485e-05
<class 'numpy.float64'>
3.9950596708e-05
<class 'numpy.float64'>
3.99455638378e-05
<class 'numpy.float64'>
3.9940532237e-05
<class 'numpy.float64'>
3.99355019054e-05
<class 'numpy.float64'>
3.99304728428e-05
<class 'numpy.float64'>
3.99254450482e-05
<class 'numpy.float64'>
3.99204185217e-05
<class 'numpy.float64'>
3.99153932622e-05
<class 'numpy.float64'>
3.99103692696e-05
<class 'numpy.float64'>
3.99053465438e-05
<class 'numpy.float64'>
3.99003250832e-05
<class 'numpy.float64'>
3.98953048884e-05
<class 'numpy.float64'>
3.98902859585e-05
<class 'numpy.float64'>
3.9885268293e-05
<class 'numpy.float64'>
3.98802518915e-05
<class 'numpy.float64'>
3.98752367532e-05
<class 'numpy.float64'>
3.98702228784e-05
<class 'numpy.float64'>
3.98652102658e-05
<class 'numpy.float64'>
3.98601989154e-05
<class 'numpy.float64'>
3.98551888263e-05
<class 'numpy.float64'>
3.98501799985e-05
<class 'numpy.float64'>
3.98451724313e-05
<class 'numpy.float64'>
3.98401661243e-05
<class 'numpy.float64'>
3.98351610769e-05
<class 'numpy.float64'>
3.98301572889e-05
<class 'numpy.float64'>
3.98251547593e-05
<class 'numpy.float64'>
3.9820153488e-05
<class 'numpy.float64'>
3.98151534747e-05
<class 'numpy.float64'>
3.98101547185e-05
<class 'numpy.float64'>
3.98051572192e-05
<class 'numpy.float64'>
3.9800160976e-05
<class 'numpy.float64'>
3.97951659891e-05
<class 'numpy.float64'>
3.97901722575e-05
<class 'numpy.float64'>
3.97851797807e-05
<class 'numpy.float64'>
3.97801885585e-05
<class 'numpy.float64'>
3.97751985898e-05
<class 'numpy.float64'>
3.9770209875e-05
<class 'numpy.float64'>
3.97652224133e-05
<class 'numpy.float64'>
3.9760236204e-05
<class 'numpy.float64'>
3.97552512468e-05
<class 'numpy.float64'>
3.97502675413e-05
<class 'numpy.float64'>
3.9745285087e-05
<class 'numpy.float64'>
3.97403038832e-05
<class 'numpy.float64'>
3.97353239296e-05
<class 'numpy.float64'>
3.97303452258e-05
<class 'numpy.float64'>
3.97253677713e-05
<class 'numpy.float64'>
3.97203915654e-05
<class 'numpy.float64'>
3.97154166081e-05
<class 'numpy.float64'>
3.97104428985e-05
<class 'numpy.float64'>
3.97054704364e-05
<class 'numpy.float64'>
3.97004992211e-05
<class 'numpy.float64'>
3.96955292521e-05
<class 'numpy.float64'>
3.96905605293e-05
<class 'numpy.float64'>
3.9685593052e-05
<class 'numpy.float64'>
3.96806268197e-05
<class 'numpy.float64'>
3.96756618319e-05
<class 'numpy.float64'>
3.96706980881e-05
<class 'numpy.float64'>
3.96657355881e-05
<class 'numpy.float64'>
3.96607743312e-05
<class 'numpy.float64'>
3.96558143171e-05
<class 'numpy.float64'>
3.96508555453e-05
<class 'numpy.float64'>
3.9645898015e-05
<class 'numpy.float64'>
3.9640941726e-05
<class 'numpy.float64'>
3.9635986678e-05
<class 'numpy.float64'>
3.96310328704e-05
<class 'numpy.float64'>
3.96260803026e-05
<class 'numpy.float64'>
3.96211289743e-05
<class 'numpy.float64'>
3.96161788847e-05
<class 'numpy.float64'>
3.96112300338e-05
<class 'numpy.float64'>
3.96062824209e-05
<class 'numpy.float64'>
3.96013360456e-05
<class 'numpy.float64'>
3.95963909076e-05
<class 'numpy.float64'>
3.95914470059e-05
<class 'numpy.float64'>
3.95865043405e-05
<class 'numpy.float64'>
3.95815629109e-05
<class 'numpy.float64'>
3.95766227166e-05
<class 'numpy.float64'>
3.9571683757e-05
<class 'numpy.float64'>
3.95667460317e-05
<class 'numpy.float64'>
3.95618095403e-05
<class 'numpy.float64'>
3.95568742822e-05
<class 'numpy.float64'>
Cost after iteration 8000: 0.000040
3.95519402572e-05
<class 'numpy.float64'>
3.95470074646e-05
<class 'numpy.float64'>
3.95420759039e-05
<class 'numpy.float64'>
3.95371455746e-05
<class 'numpy.float64'>
3.95322164769e-05
<class 'numpy.float64'>
3.95272886095e-05
<class 'numpy.float64'>
3.95223619722e-05
<class 'numpy.float64'>
3.9517436565e-05
<class 'numpy.float64'>
3.95125123866e-05
<class 'numpy.float64'>
3.95075894371e-05
<class 'numpy.float64'>
3.95026677161e-05
<class 'numpy.float64'>
3.94977472227e-05
<class 'numpy.float64'>
3.9492827957e-05
<class 'numpy.float64'>
3.94879099181e-05
<class 'numpy.float64'>
3.94829931056e-05
<class 'numpy.float64'>
3.94780775193e-05
<class 'numpy.float64'>
3.94731631584e-05
<class 'numpy.float64'>
3.94682500227e-05
<class 'numpy.float64'>
3.94633381115e-05
<class 'numpy.float64'>
3.94584274247e-05
<class 'numpy.float64'>
3.94535179615e-05
<class 'numpy.float64'>
3.94486097216e-05
<class 'numpy.float64'>
3.94437027046e-05
<class 'numpy.float64'>
3.943879691e-05
<class 'numpy.float64'>
3.9433892337e-05
<class 'numpy.float64'>
3.94289889857e-05
<class 'numpy.float64'>
3.94240868554e-05
<class 'numpy.float64'>
3.94191859457e-05
<class 'numpy.float64'>
3.94142862561e-05
<class 'numpy.float64'>
3.94093877858e-05
<class 'numpy.float64'>
3.94044905348e-05
<class 'numpy.float64'>
3.93995945025e-05
<class 'numpy.float64'>
3.93946996887e-05
<class 'numpy.float64'>
3.93898060927e-05
<class 'numpy.float64'>
3.93849137137e-05
<class 'numpy.float64'>
3.93800225517e-05
<class 'numpy.float64'>
3.93751326064e-05
<class 'numpy.float64'>
3.93702438769e-05
<class 'numpy.float64'>
3.93653563628e-05
<class 'numpy.float64'>
3.9360470064e-05
<class 'numpy.float64'>
3.93555849797e-05
<class 'numpy.float64'>
3.93507011095e-05
<class 'numpy.float64'>
3.93458184531e-05
<class 'numpy.float64'>
3.93409370102e-05
<class 'numpy.float64'>
3.93360567797e-05
<class 'numpy.float64'>
3.93311777619e-05
<class 'numpy.float64'>
3.93262999559e-05
<class 'numpy.float64'>
3.93214233612e-05
<class 'numpy.float64'>
3.93165479778e-05
<class 'numpy.float64'>
3.93116738047e-05
<class 'numpy.float64'>
3.9306800842e-05
<class 'numpy.float64'>
3.93019290888e-05
<class 'numpy.float64'>
3.92970585444e-05
<class 'numpy.float64'>
3.92921892094e-05
<class 'numpy.float64'>
3.92873210822e-05
<class 'numpy.float64'>
3.92824541631e-05
<class 'numpy.float64'>
3.92775884515e-05
<class 'numpy.float64'>
3.92727239467e-05
<class 'numpy.float64'>
3.92678606483e-05
<class 'numpy.float64'>
3.92629985561e-05
<class 'numpy.float64'>
3.92581376695e-05
<class 'numpy.float64'>
3.92532779881e-05
<class 'numpy.float64'>
3.92484195114e-05
<class 'numpy.float64'>
3.92435622389e-05
<class 'numpy.float64'>
3.923870617e-05
<class 'numpy.float64'>
3.92338513047e-05
<class 'numpy.float64'>
3.92289976423e-05
<class 'numpy.float64'>
3.92241451823e-05
<class 'numpy.float64'>
3.92192939244e-05
<class 'numpy.float64'>
3.92144438681e-05
<class 'numpy.float64'>
3.92095950129e-05
<class 'numpy.float64'>
3.92047473585e-05
<class 'numpy.float64'>
3.91999009042e-05
<class 'numpy.float64'>
3.91950556497e-05
<class 'numpy.float64'>
3.91902115946e-05
<class 'numpy.float64'>
3.91853687383e-05
<class 'numpy.float64'>
3.91805270804e-05
<class 'numpy.float64'>
3.91756866207e-05
<class 'numpy.float64'>
3.91708473584e-05
<class 'numpy.float64'>
3.91660092932e-05
<class 'numpy.float64'>
3.91611724248e-05
<class 'numpy.float64'>
3.91563367528e-05
<class 'numpy.float64'>
3.91515022763e-05
<class 'numpy.float64'>
3.91466689951e-05
<class 'numpy.float64'>
3.9141836909e-05
<class 'numpy.float64'>
3.91370060171e-05
<class 'numpy.float64'>
3.91321763196e-05
<class 'numpy.float64'>
3.91273478152e-05
<class 'numpy.float64'>
3.91225205043e-05
<class 'numpy.float64'>
3.91176943861e-05
<class 'numpy.float64'>
3.91128694598e-05
<class 'numpy.float64'>
3.91080457258e-05
<class 'numpy.float64'>
3.91032231827e-05
<class 'numpy.float64'>
3.90984018308e-05
<class 'numpy.float64'>
3.90935816693e-05
<class 'numpy.float64'>
3.90887626976e-05
<class 'numpy.float64'>
3.90839449158e-05
<class 'numpy.float64'>
3.90791283231e-05
<class 'numpy.float64'>
3.9074312919e-05
<class 'numpy.float64'>
3.90694987033e-05
<class 'numpy.float64'>
3.90646856753e-05
<class 'numpy.float64'>
3.90598738347e-05
<class 'numpy.float64'>
3.90550631813e-05
<class 'numpy.float64'>
3.90502537142e-05
<class 'numpy.float64'>
3.90454454332e-05
<class 'numpy.float64'>
3.90406383378e-05
<class 'numpy.float64'>
3.90358324275e-05
<class 'numpy.float64'>
3.90310277022e-05
<class 'numpy.float64'>
3.90262241612e-05
<class 'numpy.float64'>
3.90214218039e-05
<class 'numpy.float64'>
3.90166206302e-05
<class 'numpy.float64'>
3.90118206396e-05
<class 'numpy.float64'>
3.90070218312e-05
<class 'numpy.float64'>
3.90022242051e-05
<class 'numpy.float64'>
3.89974277608e-05
<class 'numpy.float64'>
3.89926324978e-05
<class 'numpy.float64'>
3.89878384154e-05
<class 'numpy.float64'>
3.89830455136e-05
<class 'numpy.float64'>
3.89782537914e-05
<class 'numpy.float64'>
3.89734632491e-05
<class 'numpy.float64'>
3.89686738857e-05
<class 'numpy.float64'>
3.89638857012e-05
<class 'numpy.float64'>
3.89590986945e-05
<class 'numpy.float64'>
3.89543128658e-05
<class 'numpy.float64'>
3.89495282142e-05
<class 'numpy.float64'>
3.894474474e-05
<class 'numpy.float64'>
3.89399624417e-05
<class 'numpy.float64'>
3.89351813197e-05
<class 'numpy.float64'>
3.89304013735e-05
<class 'numpy.float64'>
3.89256226022e-05
<class 'numpy.float64'>
3.89208450057e-05
<class 'numpy.float64'>
3.89160685835e-05
<class 'numpy.float64'>
3.89112933352e-05
<class 'numpy.float64'>
3.89065192602e-05
<class 'numpy.float64'>
3.89017463582e-05
<class 'numpy.float64'>
3.8896974629e-05
<class 'numpy.float64'>
3.88922040717e-05
<class 'numpy.float64'>
3.88874346862e-05
<class 'numpy.float64'>
3.88826664719e-05
<class 'numpy.float64'>
3.88778994285e-05
<class 'numpy.float64'>
3.88731335556e-05
<class 'numpy.float64'>
3.88683688524e-05
<class 'numpy.float64'>
3.8863605319e-05
<class 'numpy.float64'>
3.88588429547e-05
<class 'numpy.float64'>
3.88540817588e-05
<class 'numpy.float64'>
3.88493217316e-05
<class 'numpy.float64'>
3.88445628718e-05
<class 'numpy.float64'>
3.88398051798e-05
<class 'numpy.float64'>
3.88350486545e-05
<class 'numpy.float64'>
3.88302932958e-05
<class 'numpy.float64'>
3.88255391032e-05
<class 'numpy.float64'>
3.88207860762e-05
<class 'numpy.float64'>
3.88160342144e-05
<class 'numpy.float64'>
3.88112835178e-05
<class 'numpy.float64'>
3.88065339852e-05
<class 'numpy.float64'>
3.88017856165e-05
<class 'numpy.float64'>
3.87970384116e-05
<class 'numpy.float64'>
3.87922923698e-05
<class 'numpy.float64'>
3.87875474906e-05
<class 'numpy.float64'>
3.87828037735e-05
<class 'numpy.float64'>
3.87780612186e-05
<class 'numpy.float64'>
3.87733198246e-05
<class 'numpy.float64'>
3.87685795921e-05
<class 'numpy.float64'>
3.87638405196e-05
<class 'numpy.float64'>
3.87591026074e-05
<class 'numpy.float64'>
3.87543658551e-05
<class 'numpy.float64'>
3.87496302621e-05
<class 'numpy.float64'>
3.87448958276e-05
<class 'numpy.float64'>
3.87401625518e-05
<class 'numpy.float64'>
3.87354304338e-05
<class 'numpy.float64'>
3.87306994736e-05
<class 'numpy.float64'>
3.87259696702e-05
<class 'numpy.float64'>
3.87212410238e-05
<class 'numpy.float64'>
3.87165135335e-05
<class 'numpy.float64'>
3.87117871991e-05
<class 'numpy.float64'>
3.87070620201e-05
<class 'numpy.float64'>
3.87023379963e-05
<class 'numpy.float64'>
3.86976151268e-05
<class 'numpy.float64'>
3.86928934118e-05
<class 'numpy.float64'>
3.86881728506e-05
<class 'numpy.float64'>
3.86834534424e-05
<class 'numpy.float64'>
3.86787351869e-05
<class 'numpy.float64'>
3.86740180845e-05
<class 'numpy.float64'>
3.8669302134e-05
<class 'numpy.float64'>
3.86645873347e-05
<class 'numpy.float64'>
3.8659873687e-05
<class 'numpy.float64'>
3.86551611899e-05
<class 'numpy.float64'>
3.86504498433e-05
<class 'numpy.float64'>
3.86457396465e-05
<class 'numpy.float64'>
3.86410305992e-05
<class 'numpy.float64'>
3.8636322701e-05
<class 'numpy.float64'>
3.8631615952e-05
<class 'numpy.float64'>
3.86269103506e-05
<class 'numpy.float64'>
3.86222058975e-05
<class 'numpy.float64'>
3.86175025916e-05
<class 'numpy.float64'>
3.86128004324e-05
<class 'numpy.float64'>
3.86080994202e-05
<class 'numpy.float64'>
3.86033995541e-05
<class 'numpy.float64'>
3.85987008334e-05
<class 'numpy.float64'>
3.85940032586e-05
<class 'numpy.float64'>
3.85893068283e-05
<class 'numpy.float64'>
3.85846115426e-05
<class 'numpy.float64'>
3.85799174009e-05
<class 'numpy.float64'>
3.85752244029e-05
<class 'numpy.float64'>
3.85705325481e-05
<class 'numpy.float64'>
3.85658418363e-05
<class 'numpy.float64'>
3.85611522668e-05
<class 'numpy.float64'>
3.85564638389e-05
<class 'numpy.float64'>
3.8551776553e-05
<class 'numpy.float64'>
3.85470904083e-05
<class 'numpy.float64'>
3.85424054039e-05
<class 'numpy.float64'>
3.85377215403e-05
<class 'numpy.float64'>
3.85330388162e-05
<class 'numpy.float64'>
3.85283572319e-05
<class 'numpy.float64'>
3.85236767863e-05
<class 'numpy.float64'>
3.85189974796e-05
<class 'numpy.float64'>
3.85143193108e-05
<class 'numpy.float64'>
3.85096422803e-05
<class 'numpy.float64'>
3.8504966387e-05
<class 'numpy.float64'>
3.85002916305e-05
<class 'numpy.float64'>
3.84956180107e-05
<class 'numpy.float64'>
3.8490945527e-05
<class 'numpy.float64'>
3.84862741792e-05
<class 'numpy.float64'>
3.84816039666e-05
<class 'numpy.float64'>
3.8476934889e-05
<class 'numpy.float64'>
3.84722669457e-05
<class 'numpy.float64'>
3.84676001366e-05
<class 'numpy.float64'>
3.84629344612e-05
<class 'numpy.float64'>
3.84582699188e-05
<class 'numpy.float64'>
3.84536065097e-05
<class 'numpy.float64'>
3.84489442328e-05
<class 'numpy.float64'>
3.84442830877e-05
<class 'numpy.float64'>
3.84396230746e-05
<class 'numpy.float64'>
3.84349641926e-05
<class 'numpy.float64'>
3.84303064412e-05
<class 'numpy.float64'>
3.84256498201e-05
<class 'numpy.float64'>
3.84209943293e-05
<class 'numpy.float64'>
3.84163399678e-05
<class 'numpy.float64'>
3.84116867355e-05
<class 'numpy.float64'>
3.84070346318e-05
<class 'numpy.float64'>
3.84023836563e-05
<class 'numpy.float64'>
3.83977338092e-05
<class 'numpy.float64'>
3.83930850892e-05
<class 'numpy.float64'>
3.83884374962e-05
<class 'numpy.float64'>
3.83837910303e-05
<class 'numpy.float64'>
3.83791456903e-05
<class 'numpy.float64'>
3.83745014763e-05
<class 'numpy.float64'>
3.83698583877e-05
<class 'numpy.float64'>
3.8365216424e-05
<class 'numpy.float64'>
3.8360575585e-05
<class 'numpy.float64'>
3.83559358703e-05
<class 'numpy.float64'>
3.83512972794e-05
<class 'numpy.float64'>
3.83466598118e-05
<class 'numpy.float64'>
3.83420234674e-05
<class 'numpy.float64'>
3.83373882455e-05
<class 'numpy.float64'>
3.83327541455e-05
<class 'numpy.float64'>
3.83281211678e-05
<class 'numpy.float64'>
3.83234893108e-05
<class 'numpy.float64'>
3.83188585752e-05
<class 'numpy.float64'>
3.831422896e-05
<class 'numpy.float64'>
3.83096004651e-05
<class 'numpy.float64'>
3.83049730897e-05
<class 'numpy.float64'>
3.83003468335e-05
<class 'numpy.float64'>
3.82957216964e-05
<class 'numpy.float64'>
3.82910976779e-05
<class 'numpy.float64'>
3.82864747774e-05
<class 'numpy.float64'>
3.82818529946e-05
<class 'numpy.float64'>
3.82772323292e-05
<class 'numpy.float64'>
3.82726127806e-05
<class 'numpy.float64'>
3.82679943486e-05
<class 'numpy.float64'>
3.82633770324e-05
<class 'numpy.float64'>
3.82587608321e-05
<class 'numpy.float64'>
3.82541457468e-05
<class 'numpy.float64'>
3.82495317767e-05
<class 'numpy.float64'>
3.8244918921e-05
<class 'numpy.float64'>
3.82403071794e-05
<class 'numpy.float64'>
3.82356965514e-05
<class 'numpy.float64'>
3.82310870364e-05
<class 'numpy.float64'>
3.82264786344e-05
<class 'numpy.float64'>
3.8221871345e-05
<class 'numpy.float64'>
3.82172651675e-05
<class 'numpy.float64'>
3.82126601015e-05
<class 'numpy.float64'>
3.82080561467e-05
<class 'numpy.float64'>
3.82034533029e-05
<class 'numpy.float64'>
3.81988515697e-05
<class 'numpy.float64'>
3.81942509463e-05
<class 'numpy.float64'>
3.81896514324e-05
<class 'numpy.float64'>
3.81850530282e-05
<class 'numpy.float64'>
3.81804557324e-05
<class 'numpy.float64'>
3.8175859545e-05
<class 'numpy.float64'>
3.81712644657e-05
<class 'numpy.float64'>
3.81666704942e-05
<class 'numpy.float64'>
3.81620776297e-05
<class 'numpy.float64'>
3.8157485872e-05
<class 'numpy.float64'>
3.8152895221e-05
<class 'numpy.float64'>
3.81483056759e-05
<class 'numpy.float64'>
3.81437172361e-05
<class 'numpy.float64'>
3.81391299019e-05
<class 'numpy.float64'>
3.81345436725e-05
<class 'numpy.float64'>
3.81299585473e-05
<class 'numpy.float64'>
3.81253745263e-05
<class 'numpy.float64'>
3.81207916086e-05
<class 'numpy.float64'>
3.81162097944e-05
<class 'numpy.float64'>
3.8111629083e-05
<class 'numpy.float64'>
3.8107049474e-05
<class 'numpy.float64'>
3.81024709672e-05
<class 'numpy.float64'>
3.80978935617e-05
<class 'numpy.float64'>
3.80933172576e-05
<class 'numpy.float64'>
3.80887420541e-05
<class 'numpy.float64'>
3.80841679514e-05
<class 'numpy.float64'>
3.80795949486e-05
<class 'numpy.float64'>
3.80750230454e-05
<class 'numpy.float64'>
3.80704522417e-05
<class 'numpy.float64'>
3.80658825364e-05
<class 'numpy.float64'>
3.80613139297e-05
<class 'numpy.float64'>
3.80567464213e-05
<class 'numpy.float64'>
3.80521800103e-05
<class 'numpy.float64'>
3.80476146967e-05
<class 'numpy.float64'>
3.804305048e-05
<class 'numpy.float64'>
3.80384873596e-05
<class 'numpy.float64'>
3.80339253354e-05
<class 'numpy.float64'>
3.80293644068e-05
<class 'numpy.float64'>
3.80248045733e-05
<class 'numpy.float64'>
3.80202458352e-05
<class 'numpy.float64'>
3.80156881913e-05
<class 'numpy.float64'>
3.80111316414e-05
<class 'numpy.float64'>
3.80065761855e-05
<class 'numpy.float64'>
3.80020218227e-05
<class 'numpy.float64'>
3.79974685529e-05
<class 'numpy.float64'>
3.79929163758e-05
<class 'numpy.float64'>
3.79883652902e-05
<class 'numpy.float64'>
3.79838152969e-05
<class 'numpy.float64'>
3.79792663949e-05
<class 'numpy.float64'>
3.79747185835e-05
<class 'numpy.float64'>
3.79701718631e-05
<class 'numpy.float64'>
3.79656262327e-05
<class 'numpy.float64'>
3.79610816919e-05
<class 'numpy.float64'>
3.79565382408e-05
<class 'numpy.float64'>
3.79519958786e-05
<class 'numpy.float64'>
3.79474546049e-05
<class 'numpy.float64'>
3.79429144194e-05
<class 'numpy.float64'>
3.79383753218e-05
<class 'numpy.float64'>
3.79338373116e-05
<class 'numpy.float64'>
3.79293003885e-05
<class 'numpy.float64'>
3.79247645521e-05
<class 'numpy.float64'>
3.79202298018e-05
<class 'numpy.float64'>
3.79156961374e-05
<class 'numpy.float64'>
3.79111635586e-05
<class 'numpy.float64'>
3.79066320645e-05
<class 'numpy.float64'>
3.79021016554e-05
<class 'numpy.float64'>
3.78975723306e-05
<class 'numpy.float64'>
3.78930440898e-05
<class 'numpy.float64'>
3.78885169324e-05
<class 'numpy.float64'>
3.78839908582e-05
<class 'numpy.float64'>
3.78794658667e-05
<class 'numpy.float64'>
3.78749419575e-05
<class 'numpy.float64'>
3.78704191304e-05
<class 'numpy.float64'>
3.78658973848e-05
<class 'numpy.float64'>
3.78613767204e-05
<class 'numpy.float64'>
3.78568571366e-05
<class 'numpy.float64'>
3.78523386334e-05
<class 'numpy.float64'>
3.78478212106e-05
<class 'numpy.float64'>
3.78433048669e-05
<class 'numpy.float64'>
3.78387896027e-05
<class 'numpy.float64'>
3.78342754174e-05
<class 'numpy.float64'>
3.78297623103e-05
<class 'numpy.float64'>
3.78252502815e-05
<class 'numpy.float64'>
3.78207393304e-05
<class 'numpy.float64'>
3.78162294568e-05
<class 'numpy.float64'>
3.781172066e-05
<class 'numpy.float64'>
3.78072129398e-05
<class 'numpy.float64'>
3.78027062956e-05
<class 'numpy.float64'>
3.77982007274e-05
<class 'numpy.float64'>
3.77936962343e-05
<class 'numpy.float64'>
3.77891928163e-05
<class 'numpy.float64'>
3.7784690473e-05
<class 'numpy.float64'>
3.7780189204e-05
<class 'numpy.float64'>
3.77756890086e-05
<class 'numpy.float64'>
3.77711898871e-05
<class 'numpy.float64'>
3.77666918384e-05
<class 'numpy.float64'>
3.77621948624e-05
<class 'numpy.float64'>
3.77576989589e-05
<class 'numpy.float64'>
3.77532041271e-05
<class 'numpy.float64'>
3.77487103669e-05
<class 'numpy.float64'>
3.77442176778e-05
<class 'numpy.float64'>
3.77397260598e-05
<class 'numpy.float64'>
3.77352355121e-05
<class 'numpy.float64'>
3.77307460341e-05
<class 'numpy.float64'>
3.77262576259e-05
<class 'numpy.float64'>
3.77217702871e-05
<class 'numpy.float64'>
3.77172840171e-05
<class 'numpy.float64'>
3.77127988156e-05
<class 'numpy.float64'>
3.77083146824e-05
<class 'numpy.float64'>
3.77038316164e-05
<class 'numpy.float64'>
3.76993496181e-05
<class 'numpy.float64'>
3.76948686867e-05
<class 'numpy.float64'>
3.76903888218e-05
<class 'numpy.float64'>
3.76859100235e-05
<class 'numpy.float64'>
3.76814322906e-05
<class 'numpy.float64'>
3.76769556231e-05
<class 'numpy.float64'>
3.76724800208e-05
<class 'numpy.float64'>
3.76680054835e-05
<class 'numpy.float64'>
3.76635320101e-05
<class 'numpy.float64'>
3.76590596005e-05
<class 'numpy.float64'>
3.76545882546e-05
<class 'numpy.float64'>
3.76501179718e-05
<class 'numpy.float64'>
3.76456487523e-05
<class 'numpy.float64'>
3.76411805946e-05
<class 'numpy.float64'>
3.76367134989e-05
<class 'numpy.float64'>
3.76322474651e-05
<class 'numpy.float64'>
3.76277824926e-05
<class 'numpy.float64'>
3.76233185808e-05
<class 'numpy.float64'>
3.76188557297e-05
<class 'numpy.float64'>
3.76143939385e-05
<class 'numpy.float64'>
3.76099332072e-05
<class 'numpy.float64'>
3.76054735352e-05
<class 'numpy.float64'>
3.76010149223e-05
<class 'numpy.float64'>
3.75965573679e-05
<class 'numpy.float64'>
3.75921008718e-05
<class 'numpy.float64'>
3.75876454336e-05
<class 'numpy.float64'>
3.75831910529e-05
<class 'numpy.float64'>
3.75787377293e-05
<class 'numpy.float64'>
3.75742854623e-05
<class 'numpy.float64'>
3.75698342518e-05
<class 'numpy.float64'>
3.75653840971e-05
<class 'numpy.float64'>
3.75609349983e-05
<class 'numpy.float64'>
3.75564869547e-05
<class 'numpy.float64'>
3.75520399656e-05
<class 'numpy.float64'>
3.75475940312e-05
<class 'numpy.float64'>
3.7543149151e-05
<class 'numpy.float64'>
3.75387053245e-05
<class 'numpy.float64'>
3.75342625514e-05
<class 'numpy.float64'>
3.7529820831e-05
<class 'numpy.float64'>
3.75253801634e-05
<class 'numpy.float64'>
3.75209405478e-05
<class 'numpy.float64'>
3.75165019845e-05
<class 'numpy.float64'>
3.75120644723e-05
<class 'numpy.float64'>
3.75076280114e-05
<class 'numpy.float64'>
3.75031926011e-05
<class 'numpy.float64'>
3.74987582412e-05
<class 'numpy.float64'>
3.74943249314e-05
<class 'numpy.float64'>
3.74898926712e-05
<class 'numpy.float64'>
3.74854614601e-05
<class 'numpy.float64'>
3.7481031298e-05
<class 'numpy.float64'>
3.74766021841e-05
<class 'numpy.float64'>
3.74721741186e-05
<class 'numpy.float64'>
3.74677471009e-05
<class 'numpy.float64'>
3.74633211305e-05
<class 'numpy.float64'>
3.74588962073e-05
<class 'numpy.float64'>
3.74544723307e-05
<class 'numpy.float64'>
3.74500494999e-05
<class 'numpy.float64'>
3.74456277153e-05
<class 'numpy.float64'>
3.74412069762e-05
<class 'numpy.float64'>
3.74367872826e-05
<class 'numpy.float64'>
3.74323686333e-05
<class 'numpy.float64'>
3.74279510287e-05
<class 'numpy.float64'>
3.74235344678e-05
<class 'numpy.float64'>
3.74191189509e-05
<class 'numpy.float64'>
3.7414704477e-05
<class 'numpy.float64'>
3.74102910462e-05
<class 'numpy.float64'>
3.74058786579e-05
<class 'numpy.float64'>
3.74014673119e-05
<class 'numpy.float64'>
3.73970570077e-05
<class 'numpy.float64'>
3.73926477449e-05
<class 'numpy.float64'>
3.73882395233e-05
<class 'numpy.float64'>
3.73838323422e-05
<class 'numpy.float64'>
3.73794262017e-05
<class 'numpy.float64'>
3.73750211009e-05
<class 'numpy.float64'>
3.73706170398e-05
<class 'numpy.float64'>
3.7366214018e-05
<class 'numpy.float64'>
3.73618120351e-05
<class 'numpy.float64'>
3.73574110904e-05
<class 'numpy.float64'>
3.73530111842e-05
<class 'numpy.float64'>
3.73486123157e-05
<class 'numpy.float64'>
3.73442144846e-05
<class 'numpy.float64'>
3.73398176905e-05
<class 'numpy.float64'>
3.7335421933e-05
<class 'numpy.float64'>
3.73310272119e-05
<class 'numpy.float64'>
3.73266335267e-05
<class 'numpy.float64'>
3.73222408767e-05
<class 'numpy.float64'>
3.73178492627e-05
<class 'numpy.float64'>
3.73134586828e-05
<class 'numpy.float64'>
3.73090691378e-05
<class 'numpy.float64'>
3.73046806267e-05
<class 'numpy.float64'>
3.73002931494e-05
<class 'numpy.float64'>
3.72959067054e-05
<class 'numpy.float64'>
3.72915212946e-05
<class 'numpy.float64'>
3.72871369162e-05
<class 'numpy.float64'>
3.72827535702e-05
<class 'numpy.float64'>
3.72783712562e-05
<class 'numpy.float64'>
3.72739899734e-05
<class 'numpy.float64'>
3.72696097222e-05
<class 'numpy.float64'>
3.72652305016e-05
<class 'numpy.float64'>
3.72608523113e-05
<class 'numpy.float64'>
3.72564751516e-05
<class 'numpy.float64'>
3.72520990213e-05
<class 'numpy.float64'>
3.72477239201e-05
<class 'numpy.float64'>
3.72433498482e-05
<class 'numpy.float64'>
3.72389768052e-05
<class 'numpy.float64'>
3.72346047901e-05
<class 'numpy.float64'>
3.72302338029e-05
<class 'numpy.float64'>
3.72258638436e-05
<class 'numpy.float64'>
3.72214949108e-05
<class 'numpy.float64'>
3.72171270054e-05
<class 'numpy.float64'>
3.72127601265e-05
<class 'numpy.float64'>
3.72083942734e-05
<class 'numpy.float64'>
3.7204029446e-05
<class 'numpy.float64'>
3.71996656442e-05
<class 'numpy.float64'>
3.71953028673e-05
<class 'numpy.float64'>
3.71909411149e-05
<class 'numpy.float64'>
3.71865803873e-05
<class 'numpy.float64'>
3.71822206831e-05
<class 'numpy.float64'>
3.71778620029e-05
<class 'numpy.float64'>
3.71735043455e-05
<class 'numpy.float64'>
3.71691477114e-05
<class 'numpy.float64'>
3.71647920991e-05
<class 'numpy.float64'>
3.71604375095e-05
<class 'numpy.float64'>
3.71560839417e-05
<class 'numpy.float64'>
3.7151731395e-05
<class 'numpy.float64'>
3.71473798695e-05
<class 'numpy.float64'>
3.71430293644e-05
<class 'numpy.float64'>
3.71386798801e-05
<class 'numpy.float64'>
3.71343314156e-05
<class 'numpy.float64'>
3.71299839706e-05
<class 'numpy.float64'>
3.71256375448e-05
<class 'numpy.float64'>
3.7121292138e-05
<class 'numpy.float64'>
3.71169477498e-05
<class 'numpy.float64'>
3.71126043798e-05
<class 'numpy.float64'>
3.71082620277e-05
<class 'numpy.float64'>
3.71039206928e-05
<class 'numpy.float64'>
3.7099580375e-05
<class 'numpy.float64'>
3.70952410742e-05
<class 'numpy.float64'>
3.70909027896e-05
<class 'numpy.float64'>
3.70865655211e-05
<class 'numpy.float64'>
3.70822292683e-05
<class 'numpy.float64'>
3.70778940308e-05
<class 'numpy.float64'>
3.7073559808e-05
<class 'numpy.float64'>
3.70692266003e-05
<class 'numpy.float64'>
3.70648944063e-05
<class 'numpy.float64'>
3.70605632266e-05
<class 'numpy.float64'>
3.70562330605e-05
<class 'numpy.float64'>
3.70519039073e-05
<class 'numpy.float64'>
3.70475757669e-05
<class 'numpy.float64'>
3.70432486391e-05
<class 'numpy.float64'>
3.70389225233e-05
<class 'numpy.float64'>
3.70345974195e-05
<class 'numpy.float64'>
3.70302733268e-05
<class 'numpy.float64'>
3.70259502454e-05
<class 'numpy.float64'>
3.70216281747e-05
<class 'numpy.float64'>
3.70173071141e-05
<class 'numpy.float64'>
3.70129870636e-05
<class 'numpy.float64'>
3.70086680227e-05
<class 'numpy.float64'>
3.70043499911e-05
<class 'numpy.float64'>
3.70000329684e-05
<class 'numpy.float64'>
3.69957169543e-05
<class 'numpy.float64'>
3.69914019485e-05
<class 'numpy.float64'>
3.69870879503e-05
<class 'numpy.float64'>
3.698277496e-05
<class 'numpy.float64'>
3.69784629766e-05
<class 'numpy.float64'>
3.69741519999e-05
<class 'numpy.float64'>
3.69698420298e-05
<class 'numpy.float64'>
3.69655330658e-05
<class 'numpy.float64'>
3.69612251076e-05
<class 'numpy.float64'>
3.69569181547e-05
<class 'numpy.float64'>
3.69526122069e-05
<class 'numpy.float64'>
3.69483072636e-05
<class 'numpy.float64'>
3.69440033249e-05
<class 'numpy.float64'>
3.69397003899e-05
<class 'numpy.float64'>
3.69353984585e-05
<class 'numpy.float64'>
3.69310975307e-05
<class 'numpy.float64'>
3.69267976058e-05
<class 'numpy.float64'>
3.69224986832e-05
<class 'numpy.float64'>
3.69182007631e-05
<class 'numpy.float64'>
3.69139038448e-05
<class 'numpy.float64'>
3.69096079279e-05
<class 'numpy.float64'>
3.69053130124e-05
<class 'numpy.float64'>
3.69010190976e-05
<class 'numpy.float64'>
3.68967261834e-05
<class 'numpy.float64'>
3.68924342692e-05
<class 'numpy.float64'>
3.68881433545e-05
<class 'numpy.float64'>
3.68838534396e-05
<class 'numpy.float64'>
3.68795645238e-05
<class 'numpy.float64'>
3.68752766065e-05
<class 'numpy.float64'>
3.68709896877e-05
<class 'numpy.float64'>
3.68667037669e-05
<class 'numpy.float64'>
3.68624188438e-05
<class 'numpy.float64'>
3.68581349181e-05
<class 'numpy.float64'>
3.68538519892e-05
<class 'numpy.float64'>
3.68495700571e-05
<class 'numpy.float64'>
3.68452891215e-05
<class 'numpy.float64'>
3.68410091813e-05
<class 'numpy.float64'>
3.6836730237e-05
<class 'numpy.float64'>
3.68324522879e-05
<class 'numpy.float64'>
3.68281753338e-05
<class 'numpy.float64'>
3.68238993741e-05
<class 'numpy.float64'>
3.68196244087e-05
<class 'numpy.float64'>
3.68153504371e-05
<class 'numpy.float64'>
3.6811077459e-05
<class 'numpy.float64'>
3.68068054742e-05
<class 'numpy.float64'>
3.68025344822e-05
<class 'numpy.float64'>
3.67982644827e-05
<class 'numpy.float64'>
3.6793995475e-05
<class 'numpy.float64'>
3.67897274592e-05
<class 'numpy.float64'>
3.67854604349e-05
<class 'numpy.float64'>
3.67811944019e-05
<class 'numpy.float64'>
3.67769293593e-05
<class 'numpy.float64'>
3.67726653073e-05
<class 'numpy.float64'>
3.67684022454e-05
<class 'numpy.float64'>
3.6764140173e-05
<class 'numpy.float64'>
3.67598790901e-05
<class 'numpy.float64'>
3.67556189961e-05
<class 'numpy.float64'>
3.67513598909e-05
<class 'numpy.float64'>
3.6747101774e-05
<class 'numpy.float64'>
3.6742844645e-05
<class 'numpy.float64'>
3.6738588504e-05
<class 'numpy.float64'>
3.67343333497e-05
<class 'numpy.float64'>
3.67300791828e-05
<class 'numpy.float64'>
3.67258260025e-05
<class 'numpy.float64'>
3.67215738082e-05
<class 'numpy.float64'>
3.67173225999e-05
<class 'numpy.float64'>
3.67130723771e-05
<class 'numpy.float64'>
3.67088231396e-05
<class 'numpy.float64'>
3.67045748871e-05
<class 'numpy.float64'>
3.67003276193e-05
<class 'numpy.float64'>
3.66960813353e-05
<class 'numpy.float64'>
3.66918360357e-05
<class 'numpy.float64'>
3.66875917191e-05
<class 'numpy.float64'>
3.66833483861e-05
<class 'numpy.float64'>
3.66791060356e-05
<class 'numpy.float64'>
3.66748646678e-05
<class 'numpy.float64'>
3.66706242822e-05
<class 'numpy.float64'>
3.66663848781e-05
<class 'numpy.float64'>
3.66621464558e-05
<class 'numpy.float64'>
3.66579090143e-05
<class 'numpy.float64'>
3.66536725541e-05
<class 'numpy.float64'>
3.66494370738e-05
<class 'numpy.float64'>
3.66452025741e-05
<class 'numpy.float64'>
3.66409690537e-05
<class 'numpy.float64'>
3.66367365132e-05
<class 'numpy.float64'>
3.66325049513e-05
<class 'numpy.float64'>
3.66282743684e-05
<class 'numpy.float64'>
3.66240447642e-05
<class 'numpy.float64'>
3.66198161376e-05
<class 'numpy.float64'>
3.66155884889e-05
<class 'numpy.float64'>
3.66113618179e-05
<class 'numpy.float64'>
3.66071361234e-05
<class 'numpy.float64'>
3.66029114061e-05
<class 'numpy.float64'>
3.65986876649e-05
<class 'numpy.float64'>
3.65944648996e-05
<class 'numpy.float64'>
3.65902431102e-05
<class 'numpy.float64'>
3.65860222961e-05
<class 'numpy.float64'>
3.65818024572e-05
<class 'numpy.float64'>
3.6577583593e-05
<class 'numpy.float64'>
3.65733657029e-05
<class 'numpy.float64'>
3.6569148787e-05
<class 'numpy.float64'>
3.65649328446e-05
<class 'numpy.float64'>
3.65607178758e-05
<class 'numpy.float64'>
3.65565038798e-05
<class 'numpy.float64'>
3.65522908565e-05
<class 'numpy.float64'>
3.65480788054e-05
<class 'numpy.float64'>
3.65438677262e-05
<class 'numpy.float64'>
3.6539657619e-05
<class 'numpy.float64'>
3.65354484828e-05
<class 'numpy.float64'>
3.65312403178e-05
<class 'numpy.float64'>
3.65270331235e-05
<class 'numpy.float64'>
3.65228268993e-05
<class 'numpy.float64'>
3.65186216452e-05
<class 'numpy.float64'>
3.65144173606e-05
<class 'numpy.float64'>
3.65102140455e-05
<class 'numpy.float64'>
3.6506011699e-05
<class 'numpy.float64'>
3.65018103215e-05
<class 'numpy.float64'>
3.6497609912e-05
<class 'numpy.float64'>
3.64934104708e-05
<class 'numpy.float64'>
3.64892119971e-05
<class 'numpy.float64'>
3.64850144905e-05
<class 'numpy.float64'>
3.6480817951e-05
<class 'numpy.float64'>
3.64766223781e-05
<class 'numpy.float64'>
3.64724277716e-05
<class 'numpy.float64'>
3.64682341308e-05
<class 'numpy.float64'>
3.64640414558e-05
<class 'numpy.float64'>
3.6459849746e-05
<class 'numpy.float64'>
3.64556590011e-05
<class 'numpy.float64'>
3.64514692211e-05
<class 'numpy.float64'>
3.6447280405e-05
<class 'numpy.float64'>
3.6443092553e-05
<class 'numpy.float64'>
3.64389056648e-05
<class 'numpy.float64'>
3.64347197396e-05
<class 'numpy.float64'>
3.64305347777e-05
<class 'numpy.float64'>
3.64263507782e-05
<class 'numpy.float64'>
3.64221677409e-05
<class 'numpy.float64'>
3.64179856655e-05
<class 'numpy.float64'>
3.64138045518e-05
<class 'numpy.float64'>
3.64096243996e-05
<class 'numpy.float64'>
3.64054452085e-05
<class 'numpy.float64'>
3.64012669777e-05
<class 'numpy.float64'>
3.6397089707e-05
<class 'numpy.float64'>
3.63929133967e-05
<class 'numpy.float64'>
3.63887380457e-05
<class 'numpy.float64'>
3.63845636539e-05
<class 'numpy.float64'>
3.63803902215e-05
<class 'numpy.float64'>
3.63762177476e-05
<class 'numpy.float64'>
3.63720462321e-05
<class 'numpy.float64'>
3.63678756742e-05
<class 'numpy.float64'>
3.63637060744e-05
<class 'numpy.float64'>
3.63595374318e-05
<class 'numpy.float64'>
3.63553697461e-05
<class 'numpy.float64'>
3.63512030169e-05
<class 'numpy.float64'>
3.63470372441e-05
<class 'numpy.float64'>
3.63428724274e-05
<class 'numpy.float64'>
3.63387085666e-05
<class 'numpy.float64'>
3.6334545661e-05
<class 'numpy.float64'>
3.63303837104e-05
<class 'numpy.float64'>
3.63262227142e-05
<class 'numpy.float64'>
3.63220626727e-05
<class 'numpy.float64'>
3.63179035851e-05
<class 'numpy.float64'>
3.63137454514e-05
<class 'numpy.float64'>
3.63095882707e-05
<class 'numpy.float64'>
3.63054320433e-05
<class 'numpy.float64'>
3.63012767683e-05
<class 'numpy.float64'>
3.6297122446e-05
<class 'numpy.float64'>
3.62929690759e-05
<class 'numpy.float64'>
3.62888166574e-05
<class 'numpy.float64'>
3.628466519e-05
<class 'numpy.float64'>
3.62805146739e-05
<class 'numpy.float64'>
3.62763651086e-05
<class 'numpy.float64'>
3.62722164938e-05
<class 'numpy.float64'>
3.62680688293e-05
<class 'numpy.float64'>
3.6263922114e-05
<class 'numpy.float64'>
3.62597763484e-05
<class 'numpy.float64'>
3.62556315318e-05
<class 'numpy.float64'>
3.62514876644e-05
<class 'numpy.float64'>
3.6247344745e-05
<class 'numpy.float64'>
3.62432027739e-05
<class 'numpy.float64'>
3.62390617507e-05
<class 'numpy.float64'>
3.62349216752e-05
<class 'numpy.float64'>
3.62307825465e-05
<class 'numpy.float64'>
3.62266443647e-05
<class 'numpy.float64'>
3.62225071296e-05
<class 'numpy.float64'>
3.62183708404e-05
<class 'numpy.float64'>
3.62142354974e-05
<class 'numpy.float64'>
3.62101010997e-05
<class 'numpy.float64'>
3.62059676473e-05
<class 'numpy.float64'>
3.62018351398e-05
<class 'numpy.float64'>
3.61977035767e-05
<class 'numpy.float64'>
3.61935729579e-05
<class 'numpy.float64'>
3.61894432833e-05
<class 'numpy.float64'>
3.6185314552e-05
<class 'numpy.float64'>
3.61811867643e-05
<class 'numpy.float64'>
3.61770599193e-05
<class 'numpy.float64'>
3.6172934017e-05
<class 'numpy.float64'>
3.61688090567e-05
<class 'numpy.float64'>
3.61646850387e-05
<class 'numpy.float64'>
3.61605619624e-05
<class 'numpy.float64'>
3.61564398275e-05
<class 'numpy.float64'>
3.61523186332e-05
<class 'numpy.float64'>
3.614819838e-05
<class 'numpy.float64'>
3.61440790668e-05
<class 'numpy.float64'>
3.61399606938e-05
<class 'numpy.float64'>
3.61358432606e-05
<class 'numpy.float64'>
3.61317267669e-05
<class 'numpy.float64'>
3.61276112123e-05
<class 'numpy.float64'>
3.61234965961e-05
<class 'numpy.float64'>
3.61193829185e-05
<class 'numpy.float64'>
3.61152701791e-05
<class 'numpy.float64'>
3.61111583775e-05
<class 'numpy.float64'>
3.61070475135e-05
<class 'numpy.float64'>
3.61029375865e-05
<class 'numpy.float64'>
3.60988285962e-05
<class 'numpy.float64'>
3.60947205425e-05
<class 'numpy.float64'>
3.60906134253e-05
<class 'numpy.float64'>
3.60865072434e-05
<class 'numpy.float64'>
3.60824019974e-05
<class 'numpy.float64'>
3.60782976864e-05
<class 'numpy.float64'>
3.60741943109e-05
<class 'numpy.float64'>
3.60700918695e-05
<class 'numpy.float64'>
3.60659903623e-05
<class 'numpy.float64'>
3.60618897893e-05
<class 'numpy.float64'>
3.60577901498e-05
<class 'numpy.float64'>
3.60536914438e-05
<class 'numpy.float64'>
3.60495936707e-05
<class 'numpy.float64'>
3.60454968302e-05
<class 'numpy.float64'>
3.60414009218e-05
<class 'numpy.float64'>
3.6037305946e-05
<class 'numpy.float64'>
3.60332119017e-05
<class 'numpy.float64'>
3.60291187886e-05
<class 'numpy.float64'>
3.60250266069e-05
<class 'numpy.float64'>
3.60209353558e-05
<class 'numpy.float64'>
3.60168450353e-05
<class 'numpy.float64'>
3.60127556446e-05
<class 'numpy.float64'>
3.60086671839e-05
<class 'numpy.float64'>
3.60045796528e-05
<class 'numpy.float64'>
3.6000493051e-05
<class 'numpy.float64'>
3.59964073776e-05
<class 'numpy.float64'>
3.59923226333e-05
<class 'numpy.float64'>
3.59882388168e-05
<class 'numpy.float64'>
3.59841559285e-05
<class 'numpy.float64'>
3.59800739676e-05
<class 'numpy.float64'>
3.59759929338e-05
<class 'numpy.float64'>
3.59719128275e-05
<class 'numpy.float64'>
3.59678336473e-05
<class 'numpy.float64'>
3.59637553937e-05
<class 'numpy.float64'>
3.59596780661e-05
<class 'numpy.float64'>
3.59556016644e-05
<class 'numpy.float64'>
3.59515261877e-05
<class 'numpy.float64'>
3.59474516364e-05
<class 'numpy.float64'>
3.59433780096e-05
<class 'numpy.float64'>
3.59393053076e-05
<class 'numpy.float64'>
3.59352335292e-05
<class 'numpy.float64'>
3.59311626751e-05
<class 'numpy.float64'>
3.59270927444e-05
<class 'numpy.float64'>
3.59230237366e-05
<class 'numpy.float64'>
3.59189556521e-05
<class 'numpy.float64'>
3.59148884897e-05
<class 'numpy.float64'>
3.59108222498e-05
<class 'numpy.float64'>
3.59067569319e-05
<class 'numpy.float64'>
3.59026925356e-05
<class 'numpy.float64'>
3.58986290603e-05
<class 'numpy.float64'>
3.58945665061e-05
<class 'numpy.float64'>
3.58905048727e-05
<class 'numpy.float64'>
3.58864441597e-05
<class 'numpy.float64'>
3.58823843666e-05
<class 'numpy.float64'>
3.58783254934e-05
<class 'numpy.float64'>
3.58742675395e-05
<class 'numpy.float64'>
3.58702105047e-05
<class 'numpy.float64'>
3.58661543887e-05
<class 'numpy.float64'>
3.58620991912e-05
<class 'numpy.float64'>
3.5858044912e-05
<class 'numpy.float64'>
3.58539915504e-05
<class 'numpy.float64'>
3.58499391065e-05
<class 'numpy.float64'>
3.58458875799e-05
<class 'numpy.float64'>
3.58418369701e-05
<class 'numpy.float64'>
3.58377872771e-05
<class 'numpy.float64'>
3.58337385002e-05
<class 'numpy.float64'>
3.58296906395e-05
<class 'numpy.float64'>
3.58256436941e-05
<class 'numpy.float64'>
3.58215976646e-05
<class 'numpy.float64'>
3.58175525497e-05
<class 'numpy.float64'>
3.58135083498e-05
<class 'numpy.float64'>
3.5809465064e-05
<class 'numpy.float64'>
3.58054226927e-05
<class 'numpy.float64'>
3.58013812351e-05
<class 'numpy.float64'>
3.57973406909e-05
<class 'numpy.float64'>
3.57933010603e-05
<class 'numpy.float64'>
3.57892623423e-05
<class 'numpy.float64'>
3.57852245366e-05
<class 'numpy.float64'>
3.57811876436e-05
<class 'numpy.float64'>
3.57771516623e-05
<class 'numpy.float64'>
3.57731165927e-05
<class 'numpy.float64'>
3.57690824348e-05
<class 'numpy.float64'>
3.57650491875e-05
<class 'numpy.float64'>
3.57610168511e-05
<class 'numpy.float64'>
3.57569854249e-05
<class 'numpy.float64'>
3.57529549092e-05
<class 'numpy.float64'>
3.57489253029e-05
<class 'numpy.float64'>
3.57448966065e-05
<class 'numpy.float64'>
3.57408688186e-05
<class 'numpy.float64'>
3.57368419404e-05
<class 'numpy.float64'>
3.57328159705e-05
<class 'numpy.float64'>
3.57287909087e-05
<class 'numpy.float64'>
3.57247667547e-05
<class 'numpy.float64'>
3.57207435085e-05
<class 'numpy.float64'>
3.57167211699e-05
<class 'numpy.float64'>
3.57126997379e-05
<class 'numpy.float64'>
3.57086792129e-05
<class 'numpy.float64'>
3.5704659594e-05
<class 'numpy.float64'>
3.57006408817e-05
<class 'numpy.float64'>
3.56966230749e-05
<class 'numpy.float64'>
3.56926061737e-05
<class 'numpy.float64'>
3.56885901777e-05
<class 'numpy.float64'>
3.56845750864e-05
<class 'numpy.float64'>
3.56805608999e-05
<class 'numpy.float64'>
3.56765476174e-05
<class 'numpy.float64'>
3.56725352393e-05
<class 'numpy.float64'>
3.56685237645e-05
<class 'numpy.float64'>
3.56645131931e-05
<class 'numpy.float64'>
3.56605035248e-05
<class 'numpy.float64'>
3.56564947593e-05
<class 'numpy.float64'>
3.56524868963e-05
<class 'numpy.float64'>
3.5648479935e-05
<class 'numpy.float64'>
3.5644473876e-05
<class 'numpy.float64'>
3.56404687183e-05
<class 'numpy.float64'>
3.5636464462e-05
<class 'numpy.float64'>
3.56324611067e-05
<class 'numpy.float64'>
3.56284586517e-05
<class 'numpy.float64'>
3.56244570971e-05
<class 'numpy.float64'>
3.56204564424e-05
<class 'numpy.float64'>
3.56164566876e-05
<class 'numpy.float64'>
3.56124578323e-05
<class 'numpy.float64'>
3.56084598757e-05
<class 'numpy.float64'>
3.56044628182e-05
<class 'numpy.float64'>
3.56004666593e-05
<class 'numpy.float64'>
3.55964713983e-05
<class 'numpy.float64'>
3.55924770353e-05
<class 'numpy.float64'>
3.55884835698e-05
<class 'numpy.float64'>
3.55844910015e-05
<class 'numpy.float64'>
3.55804993304e-05
<class 'numpy.float64'>
3.5576508556e-05
<class 'numpy.float64'>
3.55725186776e-05
<class 'numpy.float64'>
3.55685296955e-05
<class 'numpy.float64'>
3.55645416091e-05
<class 'numpy.float64'>
3.55605544183e-05
<class 'numpy.float64'>
3.55565681226e-05
<class 'numpy.float64'>
3.55525827219e-05
<class 'numpy.float64'>
3.55485982155e-05
<class 'numpy.float64'>
3.55446146034e-05
<class 'numpy.float64'>
3.55406318854e-05
<class 'numpy.float64'>
3.55366500609e-05
<class 'numpy.float64'>
3.55326691298e-05
<class 'numpy.float64'>
3.55286890919e-05
<class 'numpy.float64'>
3.55247099467e-05
<class 'numpy.float64'>
3.55207316939e-05
<class 'numpy.float64'>
3.55167543329e-05
<class 'numpy.float64'>
3.55127778645e-05
<class 'numpy.float64'>
3.55088022868e-05
<class 'numpy.float64'>
3.5504827601e-05
<class 'numpy.float64'>
3.55008538061e-05
<class 'numpy.float64'>
3.54968809016e-05
<class 'numpy.float64'>
3.54929088874e-05
<class 'numpy.float64'>
3.54889377635e-05
<class 'numpy.float64'>
3.54849675291e-05
<class 'numpy.float64'>
3.54809981841e-05
<class 'numpy.float64'>
3.54770297286e-05
<class 'numpy.float64'>
3.54730621618e-05
<class 'numpy.float64'>
3.54690954836e-05
<class 'numpy.float64'>
3.54651296938e-05
<class 'numpy.float64'>
3.54611647917e-05
<class 'numpy.float64'>
3.54572007772e-05
<class 'numpy.float64'>
3.54532376501e-05
<class 'numpy.float64'>
3.54492754102e-05
<class 'numpy.float64'>
3.5445314057e-05
<class 'numpy.float64'>
3.54413535902e-05
<class 'numpy.float64'>
3.54373940098e-05
<class 'numpy.float64'>
3.54334353151e-05
<class 'numpy.float64'>
3.54294775061e-05
<class 'numpy.float64'>
3.54255205823e-05
<class 'numpy.float64'>
3.54215645433e-05
<class 'numpy.float64'>
3.54176093893e-05
<class 'numpy.float64'>
3.54136551195e-05
<class 'numpy.float64'>
3.54097017339e-05
<class 'numpy.float64'>
3.54057492321e-05
<class 'numpy.float64'>
3.54017976137e-05
<class 'numpy.float64'>
3.53978468784e-05
<class 'numpy.float64'>
3.53938970261e-05
<class 'numpy.float64'>
3.53899480565e-05
<class 'numpy.float64'>
3.53859999693e-05
<class 'numpy.float64'>
3.53820527638e-05
<class 'numpy.float64'>
3.53781064402e-05
<class 'numpy.float64'>
3.53741609979e-05
<class 'numpy.float64'>
3.5370216437e-05
<class 'numpy.float64'>
3.53662727566e-05
<class 'numpy.float64'>
3.53623299571e-05
<class 'numpy.float64'>
3.53583880378e-05
<class 'numpy.float64'>
3.53544469983e-05
<class 'numpy.float64'>
3.53505068384e-05
<class 'numpy.float64'>
3.53465675581e-05
<class 'numpy.float64'>
3.53426291566e-05
<class 'numpy.float64'>
3.53386916339e-05
<class 'numpy.float64'>
3.53347549896e-05
<class 'numpy.float64'>
3.53308192236e-05
<class 'numpy.float64'>
3.53268843357e-05
<class 'numpy.float64'>
3.53229503252e-05
<class 'numpy.float64'>
3.53190171921e-05
<class 'numpy.float64'>
3.5315084936e-05
<class 'numpy.float64'>
3.53111535565e-05
<class 'numpy.float64'>
3.53072230531e-05
<class 'numpy.float64'>
3.53032934261e-05
<class 'numpy.float64'>
3.52993646751e-05
<class 'numpy.float64'>
3.52954367997e-05
<class 'numpy.float64'>
3.52915097992e-05
<class 'numpy.float64'>
3.52875836741e-05
<class 'numpy.float64'>
3.52836584235e-05
<class 'numpy.float64'>
3.52797340472e-05
<class 'numpy.float64'>
3.52758105449e-05
<class 'numpy.float64'>
3.52718879164e-05
<class 'numpy.float64'>
3.52679661615e-05
<class 'numpy.float64'>
3.52640452799e-05
<class 'numpy.float64'>
3.52601252709e-05
<class 'numpy.float64'>
3.52562061348e-05
<class 'numpy.float64'>
3.52522878709e-05
<class 'numpy.float64'>
3.52483704791e-05
<class 'numpy.float64'>
3.5244453959e-05
<class 'numpy.float64'>
3.52405383101e-05
<class 'numpy.float64'>
3.52366235328e-05
<class 'numpy.float64'>
3.52327096262e-05
<class 'numpy.float64'>
3.52287965902e-05
<class 'numpy.float64'>
3.52248844246e-05
<class 'numpy.float64'>
3.52209731288e-05
<class 'numpy.float64'>
3.52170627026e-05
<class 'numpy.float64'>
3.5213153146e-05
<class 'numpy.float64'>
3.52092444586e-05
<class 'numpy.float64'>
3.52053366401e-05
<class 'numpy.float64'>
3.52014296899e-05
<class 'numpy.float64'>
3.5197523608e-05
<class 'numpy.float64'>
3.51936183943e-05
<class 'numpy.float64'>
3.51897140479e-05
<class 'numpy.float64'>
3.51858105694e-05
<class 'numpy.float64'>
3.51819079577e-05
<class 'numpy.float64'>
3.51780062129e-05
<class 'numpy.float64'>
3.51741053343e-05
<class 'numpy.float64'>
3.51702053223e-05
<class 'numpy.float64'>
Cost after iteration 9000: 0.000035
3.51663061762e-05
<class 'numpy.float64'>
3.51624078958e-05
<class 'numpy.float64'>
3.51585104807e-05
<class 'numpy.float64'>
3.51546139303e-05
<class 'numpy.float64'>
3.51507182454e-05
<class 'numpy.float64'>
3.51468234245e-05
<class 'numpy.float64'>
3.51429294682e-05
<class 'numpy.float64'>
3.51390363755e-05
<class 'numpy.float64'>
3.51351441465e-05
<class 'numpy.float64'>
3.51312527808e-05
<class 'numpy.float64'>
3.51273622785e-05
<class 'numpy.float64'>
3.51234726385e-05
<class 'numpy.float64'>
3.51195838612e-05
<class 'numpy.float64'>
3.51156959463e-05
<class 'numpy.float64'>
3.51118088932e-05
<class 'numpy.float64'>
3.51079227016e-05
<class 'numpy.float64'>
3.51040373714e-05
<class 'numpy.float64'>
3.51001529025e-05
<class 'numpy.float64'>
3.50962692942e-05
<class 'numpy.float64'>
3.50923865461e-05
<class 'numpy.float64'>
3.50885046586e-05
<class 'numpy.float64'>
3.5084623631e-05
<class 'numpy.float64'>
3.50807434628e-05
<class 'numpy.float64'>
3.50768641541e-05
<class 'numpy.float64'>
3.50729857043e-05
<class 'numpy.float64'>
3.50691081136e-05
<class 'numpy.float64'>
3.50652313811e-05
<class 'numpy.float64'>
3.50613555069e-05
<class 'numpy.float64'>
3.50574804903e-05
<class 'numpy.float64'>
3.50536063319e-05
<class 'numpy.float64'>
3.50497330306e-05
<class 'numpy.float64'>
3.50458605862e-05
<class 'numpy.float64'>
3.50419889988e-05
<class 'numpy.float64'>
3.50381182677e-05
<class 'numpy.float64'>
3.50342483929e-05
<class 'numpy.float64'>
3.50303793742e-05
<class 'numpy.float64'>
3.50265112109e-05
<class 'numpy.float64'>
3.50226439028e-05
<class 'numpy.float64'>
3.50187774501e-05
<class 'numpy.float64'>
3.5014911852e-05
<class 'numpy.float64'>
3.50110471085e-05
<class 'numpy.float64'>
3.50071832191e-05
<class 'numpy.float64'>
3.50033201839e-05
<class 'numpy.float64'>
3.49994580022e-05
<class 'numpy.float64'>
3.4995596674e-05
<class 'numpy.float64'>
3.49917361986e-05
<class 'numpy.float64'>
3.49878765762e-05
<class 'numpy.float64'>
3.49840178063e-05
<class 'numpy.float64'>
3.49801598886e-05
<class 'numpy.float64'>
3.4976302823e-05
<class 'numpy.float64'>
3.4972446609e-05
<class 'numpy.float64'>
3.49685912464e-05
<class 'numpy.float64'>
3.49647367348e-05
<class 'numpy.float64'>
3.49608830739e-05
<class 'numpy.float64'>
3.4957030264e-05
<class 'numpy.float64'>
3.49531783041e-05
<class 'numpy.float64'>
3.49493271941e-05
<class 'numpy.float64'>
3.49454769338e-05
<class 'numpy.float64'>
3.49416275231e-05
<class 'numpy.float64'>
3.49377789613e-05
<class 'numpy.float64'>
3.49339312488e-05
<class 'numpy.float64'>
3.49300843842e-05
<class 'numpy.float64'>
3.49262383685e-05
<class 'numpy.float64'>
3.49223932006e-05
<class 'numpy.float64'>
3.49185488805e-05
<class 'numpy.float64'>
3.4914705408e-05
<class 'numpy.float64'>
3.49108627823e-05
<class 'numpy.float64'>
3.49070210036e-05
<class 'numpy.float64'>
3.49031800716e-05
<class 'numpy.float64'>
3.4899339986e-05
<class 'numpy.float64'>
3.48955007464e-05
<class 'numpy.float64'>
3.48916623526e-05
<class 'numpy.float64'>
3.48878248042e-05
<class 'numpy.float64'>
3.48839881012e-05
<class 'numpy.float64'>
3.4880152243e-05
<class 'numpy.float64'>
3.48763172294e-05
<class 'numpy.float64'>
3.48724830602e-05
<class 'numpy.float64'>
3.48686497351e-05
<class 'numpy.float64'>
3.48648172539e-05
<class 'numpy.float64'>
3.48609856162e-05
<class 'numpy.float64'>
3.48571548215e-05
<class 'numpy.float64'>
3.48533248703e-05
<class 'numpy.float64'>
3.48494957614e-05
<class 'numpy.float64'>
3.48456674951e-05
<class 'numpy.float64'>
3.48418400709e-05
<class 'numpy.float64'>
3.48380134888e-05
<class 'numpy.float64'>
3.48341877481e-05
<class 'numpy.float64'>
3.48303628485e-05
<class 'numpy.float64'>
3.48265387901e-05
<class 'numpy.float64'>
3.48227155724e-05
<class 'numpy.float64'>
3.48188931955e-05
<class 'numpy.float64'>
3.48150716586e-05
<class 'numpy.float64'>
3.48112509616e-05
<class 'numpy.float64'>
3.48074311042e-05
<class 'numpy.float64'>
3.48036120864e-05
<class 'numpy.float64'>
3.47997939076e-05
<class 'numpy.float64'>
3.47959765675e-05
<class 'numpy.float64'>
3.47921600659e-05
<class 'numpy.float64'>
3.47883444027e-05
<class 'numpy.float64'>
3.47845295775e-05
<class 'numpy.float64'>
3.47807155899e-05
<class 'numpy.float64'>
3.477690244e-05
<class 'numpy.float64'>
3.4773090127e-05
<class 'numpy.float64'>
3.47692786511e-05
<class 'numpy.float64'>
3.47654680115e-05
<class 'numpy.float64'>
3.47616582086e-05
<class 'numpy.float64'>
3.47578492416e-05
<class 'numpy.float64'>
3.47540411102e-05
<class 'numpy.float64'>
3.47502338147e-05
<class 'numpy.float64'>
3.47464273541e-05
<class 'numpy.float64'>
3.47426217286e-05
<class 'numpy.float64'>
3.47388169378e-05
<class 'numpy.float64'>
3.47350129815e-05
<class 'numpy.float64'>
3.47312098592e-05
<class 'numpy.float64'>
3.47274075708e-05
<class 'numpy.float64'>
3.47236061159e-05
<class 'numpy.float64'>
3.47198054944e-05
<class 'numpy.float64'>
3.47160057059e-05
<class 'numpy.float64'>
3.47122067502e-05
<class 'numpy.float64'>
3.47084086268e-05
<class 'numpy.float64'>
3.47046113358e-05
<class 'numpy.float64'>
3.47008148769e-05
<class 'numpy.float64'>
3.46970192495e-05
<class 'numpy.float64'>
3.46932244537e-05
<class 'numpy.float64'>
3.46894304885e-05
<class 'numpy.float64'>
3.46856373545e-05
<class 'numpy.float64'>
3.46818450512e-05
<class 'numpy.float64'>
3.46780535781e-05
<class 'numpy.float64'>
3.4674262935e-05
<class 'numpy.float64'>
3.46704731216e-05
<class 'numpy.float64'>
3.46666841377e-05
<class 'numpy.float64'>
3.46628959829e-05
<class 'numpy.float64'>
3.46591086573e-05
<class 'numpy.float64'>
3.46553221604e-05
<class 'numpy.float64'>
3.46515364917e-05
<class 'numpy.float64'>
3.46477516513e-05
<class 'numpy.float64'>
3.46439676384e-05
<class 'numpy.float64'>
3.46401844534e-05
<class 'numpy.float64'>
3.46364020957e-05
<class 'numpy.float64'>
3.46326205648e-05
<class 'numpy.float64'>
3.4628839861e-05
<class 'numpy.float64'>
3.46250599835e-05
<class 'numpy.float64'>
3.46212809324e-05
<class 'numpy.float64'>
3.46175027067e-05
<class 'numpy.float64'>
3.46137253072e-05
<class 'numpy.float64'>
3.46099487332e-05
<class 'numpy.float64'>
3.46061729839e-05
<class 'numpy.float64'>
3.46023980598e-05
<class 'numpy.float64'>
3.45986239602e-05
<class 'numpy.float64'>
3.4594850685e-05
<class 'numpy.float64'>
3.4591078234e-05
<class 'numpy.float64'>
3.45873066065e-05
<class 'numpy.float64'>
3.45835358026e-05
<class 'numpy.float64'>
3.4579765822e-05
<class 'numpy.float64'>
3.45759966642e-05
<class 'numpy.float64'>
3.45722283294e-05
<class 'numpy.float64'>
3.45684608168e-05
<class 'numpy.float64'>
3.45646941265e-05
<class 'numpy.float64'>
3.45609282581e-05
<class 'numpy.float64'>
3.45571632111e-05
<class 'numpy.float64'>
3.45533989857e-05
<class 'numpy.float64'>
3.45496355816e-05
<class 'numpy.float64'>
3.45458729981e-05
<class 'numpy.float64'>
3.45421112352e-05
<class 'numpy.float64'>
3.45383502925e-05
<class 'numpy.float64'>
3.45345901697e-05
<class 'numpy.float64'>
3.4530830867e-05
<class 'numpy.float64'>
3.45270723835e-05
<class 'numpy.float64'>
3.45233147192e-05
<class 'numpy.float64'>
3.45195578741e-05
<class 'numpy.float64'>
3.45158018476e-05
<class 'numpy.float64'>
3.45120466392e-05
<class 'numpy.float64'>
3.45082922493e-05
<class 'numpy.float64'>
3.45045386774e-05
<class 'numpy.float64'>
3.45007859226e-05
<class 'numpy.float64'>
3.44970339854e-05
<class 'numpy.float64'>
3.44932828653e-05
<class 'numpy.float64'>
3.44895325623e-05
<class 'numpy.float64'>
3.44857830753e-05
<class 'numpy.float64'>
3.4482034405e-05
<class 'numpy.float64'>
3.44782865505e-05
<class 'numpy.float64'>
3.44745395115e-05
<class 'numpy.float64'>
3.44707932884e-05
<class 'numpy.float64'>
3.44670478805e-05
<class 'numpy.float64'>
3.44633032874e-05
<class 'numpy.float64'>
3.44595595089e-05
<class 'numpy.float64'>
3.44558165451e-05
<class 'numpy.float64'>
3.4452074395e-05
<class 'numpy.float64'>
3.44483330593e-05
<class 'numpy.float64'>
3.44445925366e-05
<class 'numpy.float64'>
3.44408528278e-05
<class 'numpy.float64'>
3.44371139317e-05
<class 'numpy.float64'>
3.44333758486e-05
<class 'numpy.float64'>
3.4429638578e-05
<class 'numpy.float64'>
3.442590212e-05
<class 'numpy.float64'>
3.44221664736e-05
<class 'numpy.float64'>
3.44184316392e-05
<class 'numpy.float64'>
3.44146976161e-05
<class 'numpy.float64'>
3.44109644043e-05
<class 'numpy.float64'>
3.44072320034e-05
<class 'numpy.float64'>
3.44035004134e-05
<class 'numpy.float64'>
3.43997696336e-05
<class 'numpy.float64'>
3.43960396639e-05
<class 'numpy.float64'>
3.43923105045e-05
<class 'numpy.float64'>
3.43885821546e-05
<class 'numpy.float64'>
3.43848546138e-05
<class 'numpy.float64'>
3.43811278823e-05
<class 'numpy.float64'>
3.43774019598e-05
<class 'numpy.float64'>
3.43736768457e-05
<class 'numpy.float64'>
3.43699525401e-05
<class 'numpy.float64'>
3.43662290424e-05
<class 'numpy.float64'>
3.43625063526e-05
<class 'numpy.float64'>
3.43587844704e-05
<class 'numpy.float64'>
3.43550633953e-05
<class 'numpy.float64'>
3.43513431272e-05
<class 'numpy.float64'>
3.43476236659e-05
<class 'numpy.float64'>
3.4343905011e-05
<class 'numpy.float64'>
3.43401871625e-05
<class 'numpy.float64'>
3.43364701198e-05
<class 'numpy.float64'>
3.43327538831e-05
<class 'numpy.float64'>
3.43290384514e-05
<class 'numpy.float64'>
3.43253238252e-05
<class 'numpy.float64'>
3.4321610004e-05
<class 'numpy.float64'>
3.43178969871e-05
<class 'numpy.float64'>
3.43141847745e-05
<class 'numpy.float64'>
3.43104733664e-05
<class 'numpy.float64'>
3.43067627622e-05
<class 'numpy.float64'>
3.43030529612e-05
<class 'numpy.float64'>
3.42993439638e-05
<class 'numpy.float64'>
3.42956357697e-05
<class 'numpy.float64'>
3.42919283781e-05
<class 'numpy.float64'>
3.42882217891e-05
<class 'numpy.float64'>
3.42845160024e-05
<class 'numpy.float64'>
3.42808110177e-05
<class 'numpy.float64'>
3.42771068349e-05
<class 'numpy.float64'>
3.42734034535e-05
<class 'numpy.float64'>
3.42697008736e-05
<class 'numpy.float64'>
3.42659990944e-05
<class 'numpy.float64'>
3.42622981159e-05
<class 'numpy.float64'>
3.42585979384e-05
<class 'numpy.float64'>
3.42548985607e-05
<class 'numpy.float64'>
3.42511999829e-05
<class 'numpy.float64'>
3.4247502205e-05
<class 'numpy.float64'>
3.42438052263e-05
<class 'numpy.float64'>
3.42401090469e-05
<class 'numpy.float64'>
3.42364136665e-05
<class 'numpy.float64'>
3.42327190847e-05
<class 'numpy.float64'>
3.42290253014e-05
<class 'numpy.float64'>
3.42253323161e-05
<class 'numpy.float64'>
3.42216401288e-05
<class 'numpy.float64'>
3.42179487391e-05
<class 'numpy.float64'>
3.42142581465e-05
<class 'numpy.float64'>
3.42105683512e-05
<class 'numpy.float64'>
3.42068793529e-05
<class 'numpy.float64'>
3.42031911511e-05
<class 'numpy.float64'>
3.41995037457e-05
<class 'numpy.float64'>
3.41958171365e-05
<class 'numpy.float64'>
3.41921313229e-05
<class 'numpy.float64'>
3.41884463046e-05
<class 'numpy.float64'>
3.41847620818e-05
<class 'numpy.float64'>
3.41810786543e-05
<class 'numpy.float64'>
3.41773960214e-05
<class 'numpy.float64'>
3.41737141831e-05
<class 'numpy.float64'>
3.4170033139e-05
<class 'numpy.float64'>
3.41663528888e-05
<class 'numpy.float64'>
3.41626734326e-05
<class 'numpy.float64'>
3.41589947699e-05
<class 'numpy.float64'>
3.41553169002e-05
<class 'numpy.float64'>
3.41516398235e-05
<class 'numpy.float64'>
3.41479635395e-05
<class 'numpy.float64'>
3.41442880485e-05
<class 'numpy.float64'>
3.41406133491e-05
<class 'numpy.float64'>
3.41369394418e-05
<class 'numpy.float64'>
3.41332663264e-05
<class 'numpy.float64'>
3.41295940025e-05
<class 'numpy.float64'>
3.41259224696e-05
<class 'numpy.float64'>
3.41222517276e-05
<class 'numpy.float64'>
3.41185817765e-05
<class 'numpy.float64'>
3.41149126156e-05
<class 'numpy.float64'>
3.41112442449e-05
<class 'numpy.float64'>
3.41075766644e-05
<class 'numpy.float64'>
3.4103909873e-05
<class 'numpy.float64'>
3.41002438715e-05
<class 'numpy.float64'>
3.40965786592e-05
<class 'numpy.float64'>
3.40929142355e-05
<class 'numpy.float64'>
3.40892506003e-05
<class 'numpy.float64'>
3.4085587754e-05
<class 'numpy.float64'>
3.40819256953e-05
<class 'numpy.float64'>
3.40782644248e-05
<class 'numpy.float64'>
3.4074603942e-05
<class 'numpy.float64'>
3.40709442463e-05
<class 'numpy.float64'>
3.40672853379e-05
<class 'numpy.float64'>
3.40636272161e-05
<class 'numpy.float64'>
3.40599698814e-05
<class 'numpy.float64'>
3.40563133326e-05
<class 'numpy.float64'>
3.405265757e-05
<class 'numpy.float64'>
3.40490025933e-05
<class 'numpy.float64'>
3.40453484022e-05
<class 'numpy.float64'>
3.40416949965e-05
<class 'numpy.float64'>
3.40380423759e-05
<class 'numpy.float64'>
3.40343905399e-05
<class 'numpy.float64'>
3.40307394888e-05
<class 'numpy.float64'>
3.40270892217e-05
<class 'numpy.float64'>
3.40234397389e-05
<class 'numpy.float64'>
3.40197910399e-05
<class 'numpy.float64'>
3.40161431243e-05
<class 'numpy.float64'>
3.4012495992e-05
<class 'numpy.float64'>
3.40088496429e-05
<class 'numpy.float64'>
3.40052040769e-05
<class 'numpy.float64'>
3.40015592932e-05
<class 'numpy.float64'>
3.39979152917e-05
<class 'numpy.float64'>
3.39942720722e-05
<class 'numpy.float64'>
3.39906296345e-05
<class 'numpy.float64'>
3.39869879786e-05
<class 'numpy.float64'>
3.39833471038e-05
<class 'numpy.float64'>
3.39797070102e-05
<class 'numpy.float64'>
3.39760676972e-05
<class 'numpy.float64'>
3.39724291647e-05
<class 'numpy.float64'>
3.39687914126e-05
<class 'numpy.float64'>
3.39651544407e-05
<class 'numpy.float64'>
3.39615182483e-05
<class 'numpy.float64'>
3.39578828357e-05
<class 'numpy.float64'>
3.39542482025e-05
<class 'numpy.float64'>
3.39506143482e-05
<class 'numpy.float64'>
3.39469812727e-05
<class 'numpy.float64'>
3.39433489756e-05
<class 'numpy.float64'>
3.39397174568e-05
<class 'numpy.float64'>
3.3936086716e-05
<class 'numpy.float64'>
3.39324567531e-05
<class 'numpy.float64'>
3.39288275675e-05
<class 'numpy.float64'>
3.39251991596e-05
<class 'numpy.float64'>
3.39215715286e-05
<class 'numpy.float64'>
3.39179446743e-05
<class 'numpy.float64'>
3.39143185966e-05
<class 'numpy.float64'>
3.3910693295e-05
<class 'numpy.float64'>
3.39070687697e-05
<class 'numpy.float64'>
3.39034450201e-05
<class 'numpy.float64'>
3.38998220458e-05
<class 'numpy.float64'>
3.38961998472e-05
<class 'numpy.float64'>
3.38925784236e-05
<class 'numpy.float64'>
3.38889577746e-05
<class 'numpy.float64'>
3.38853379e-05
<class 'numpy.float64'>
3.38817187999e-05
<class 'numpy.float64'>
3.38781004739e-05
<class 'numpy.float64'>
3.38744829216e-05
<class 'numpy.float64'>
3.38708661427e-05
<class 'numpy.float64'>
3.38672501375e-05
<class 'numpy.float64'>
3.38636349049e-05
<class 'numpy.float64'>
3.38600204454e-05
<class 'numpy.float64'>
3.38564067585e-05
<class 'numpy.float64'>
3.38527938437e-05
<class 'numpy.float64'>
3.38491817012e-05
<class 'numpy.float64'>
3.38455703304e-05
<class 'numpy.float64'>
3.3841959731e-05
<class 'numpy.float64'>
3.38383499031e-05
<class 'numpy.float64'>
3.38347408464e-05
<class 'numpy.float64'>
3.38311325601e-05
<class 'numpy.float64'>
3.38275250449e-05
<class 'numpy.float64'>
3.38239182997e-05
<class 'numpy.float64'>
3.38203123245e-05
<class 'numpy.float64'>
3.38167071194e-05
<class 'numpy.float64'>
3.38131026837e-05
<class 'numpy.float64'>
3.38094990175e-05
<class 'numpy.float64'>
3.38058961203e-05
<class 'numpy.float64'>
3.3802293992e-05
<class 'numpy.float64'>
3.37986926324e-05
<class 'numpy.float64'>
3.37950920409e-05
<class 'numpy.float64'>
3.37914922178e-05
<class 'numpy.float64'>
3.37878931623e-05
<class 'numpy.float64'>
3.37842948746e-05
<class 'numpy.float64'>
3.37806973543e-05
<class 'numpy.float64'>
3.37771006009e-05
<class 'numpy.float64'>
3.37735046147e-05
<class 'numpy.float64'>
3.37699093949e-05
<class 'numpy.float64'>
3.37663149417e-05
<class 'numpy.float64'>
3.37627212545e-05
<class 'numpy.float64'>
3.37591283332e-05
<class 'numpy.float64'>
3.37555361776e-05
<class 'numpy.float64'>
3.37519447874e-05
<class 'numpy.float64'>
3.37483541625e-05
<class 'numpy.float64'>
3.37447643022e-05
<class 'numpy.float64'>
3.37411752068e-05
<class 'numpy.float64'>
3.3737586876e-05
<class 'numpy.float64'>
3.37339993092e-05
<class 'numpy.float64'>
3.37304125063e-05
<class 'numpy.float64'>
3.37268264671e-05
<class 'numpy.float64'>
3.37232411913e-05
<class 'numpy.float64'>
3.3719656679e-05
<class 'numpy.float64'>
3.37160729293e-05
<class 'numpy.float64'>
3.37124899425e-05
<class 'numpy.float64'>
3.37089077185e-05
<class 'numpy.float64'>
3.37053262563e-05
<class 'numpy.float64'>
3.37017455563e-05
<class 'numpy.float64'>
3.36981656179e-05
<class 'numpy.float64'>
3.3694586441e-05
<class 'numpy.float64'>
3.36910080256e-05
<class 'numpy.float64'>
3.36874303711e-05
<class 'numpy.float64'>
3.36838534773e-05
<class 'numpy.float64'>
3.36802773443e-05
<class 'numpy.float64'>
3.36767019713e-05
<class 'numpy.float64'>
3.36731273584e-05
<class 'numpy.float64'>
3.36695535055e-05
<class 'numpy.float64'>
3.36659804121e-05
<class 'numpy.float64'>
3.36624080779e-05
<class 'numpy.float64'>
3.36588365031e-05
<class 'numpy.float64'>
3.36552656866e-05
<class 'numpy.float64'>
3.36516956291e-05
<class 'numpy.float64'>
3.36481263299e-05
<class 'numpy.float64'>
3.36445577888e-05
<class 'numpy.float64'>
3.36409900054e-05
<class 'numpy.float64'>
3.363742298e-05
<class 'numpy.float64'>
3.36338567119e-05
<class 'numpy.float64'>
3.36302912005e-05
<class 'numpy.float64'>
3.36267264464e-05
<class 'numpy.float64'>
3.36231624491e-05
<class 'numpy.float64'>
3.36195992079e-05
<class 'numpy.float64'>
3.3616036723e-05
<class 'numpy.float64'>
3.36124749941e-05
<class 'numpy.float64'>
3.36089140206e-05
<class 'numpy.float64'>
3.36053538031e-05
<class 'numpy.float64'>
3.36017943405e-05
<class 'numpy.float64'>
3.35982356328e-05
<class 'numpy.float64'>
3.35946776801e-05
<class 'numpy.float64'>
3.3591120482e-05
<class 'numpy.float64'>
3.35875640378e-05
<class 'numpy.float64'>
3.35840083476e-05
<class 'numpy.float64'>
3.35804534115e-05
<class 'numpy.float64'>
3.35768992288e-05
<class 'numpy.float64'>
3.35733457994e-05
<class 'numpy.float64'>
3.35697931232e-05
<class 'numpy.float64'>
3.35662411994e-05
<class 'numpy.float64'>
3.35626900287e-05
<class 'numpy.float64'>
3.35591396102e-05
<class 'numpy.float64'>
3.35555899437e-05
<class 'numpy.float64'>
3.3552041029e-05
<class 'numpy.float64'>
3.3548492866e-05
<class 'numpy.float64'>
3.35449454545e-05
<class 'numpy.float64'>
3.35413987941e-05
<class 'numpy.float64'>
3.35378528845e-05
<class 'numpy.float64'>
3.35343077256e-05
<class 'numpy.float64'>
3.35307633171e-05
<class 'numpy.float64'>
3.35272196592e-05
<class 'numpy.float64'>
3.35236767508e-05
<class 'numpy.float64'>
3.35201345923e-05
<class 'numpy.float64'>
3.35165931833e-05
<class 'numpy.float64'>
3.35130525235e-05
<class 'numpy.float64'>
3.35095126124e-05
<class 'numpy.float64'>
3.35059734504e-05
<class 'numpy.float64'>
3.35024350368e-05
<class 'numpy.float64'>
3.34988973717e-05
<class 'numpy.float64'>
3.34953604546e-05
<class 'numpy.float64'>
3.3491824285e-05
<class 'numpy.float64'>
3.34882888635e-05
<class 'numpy.float64'>
3.3484754189e-05
<class 'numpy.float64'>
3.34812202616e-05
<class 'numpy.float64'>
3.34776870811e-05
<class 'numpy.float64'>
3.34741546472e-05
<class 'numpy.float64'>
3.34706229598e-05
<class 'numpy.float64'>
3.34670920187e-05
<class 'numpy.float64'>
3.34635618233e-05
<class 'numpy.float64'>
3.34600323735e-05
<class 'numpy.float64'>
3.34565036694e-05
<class 'numpy.float64'>
3.34529757103e-05
<class 'numpy.float64'>
3.34494484965e-05
<class 'numpy.float64'>
3.34459220271e-05
<class 'numpy.float64'>
3.34423963025e-05
<class 'numpy.float64'>
3.34388713218e-05
<class 'numpy.float64'>
3.34353470853e-05
<class 'numpy.float64'>
3.34318235928e-05
<class 'numpy.float64'>
3.34283008436e-05
<class 'numpy.float64'>
3.34247788379e-05
<class 'numpy.float64'>
3.3421257575e-05
<class 'numpy.float64'>
3.34177370551e-05
<class 'numpy.float64'>
3.34142172781e-05
<class 'numpy.float64'>
3.34106982431e-05
<class 'numpy.float64'>
3.34071799503e-05
<class 'numpy.float64'>
3.34036623998e-05
<class 'numpy.float64'>
3.34001455904e-05
<class 'numpy.float64'>
3.33966295231e-05
<class 'numpy.float64'>
3.33931141965e-05
<class 'numpy.float64'>
3.33895996109e-05
<class 'numpy.float64'>
3.33860857664e-05
<class 'numpy.float64'>
3.33825726622e-05
<class 'numpy.float64'>
3.33790602982e-05
<class 'numpy.float64'>
3.33755486744e-05
<class 'numpy.float64'>
3.33720377901e-05
<class 'numpy.float64'>
3.33685276458e-05
<class 'numpy.float64'>
3.33650182405e-05
<class 'numpy.float64'>
3.33615095745e-05
<class 'numpy.float64'>
3.33580016473e-05
<class 'numpy.float64'>
3.33544944587e-05
<class 'numpy.float64'>
3.33509880085e-05
<class 'numpy.float64'>
3.33474822965e-05
<class 'numpy.float64'>
3.33439773225e-05
<class 'numpy.float64'>
3.3340473086e-05
<class 'numpy.float64'>
3.33369695872e-05
<class 'numpy.float64'>
3.33334668257e-05
<class 'numpy.float64'>
3.33299648009e-05
<class 'numpy.float64'>
3.3326463513e-05
<class 'numpy.float64'>
3.33229629618e-05
<class 'numpy.float64'>
3.33194631468e-05
<class 'numpy.float64'>
3.3315964068e-05
<class 'numpy.float64'>
3.33124657247e-05
<class 'numpy.float64'>
3.33089681173e-05
<class 'numpy.float64'>
3.33054712452e-05
<class 'numpy.float64'>
3.33019751082e-05
<class 'numpy.float64'>
3.32984797062e-05
<class 'numpy.float64'>
3.32949850391e-05
<class 'numpy.float64'>
3.32914911059e-05
<class 'numpy.float64'>
3.32879979073e-05
<class 'numpy.float64'>
3.32845054427e-05
<class 'numpy.float64'>
3.32810137118e-05
<class 'numpy.float64'>
3.32775227145e-05
<class 'numpy.float64'>
3.32740324502e-05
<class 'numpy.float64'>
3.32705429191e-05
<class 'numpy.float64'>
3.32670541209e-05
<class 'numpy.float64'>
3.32635660553e-05
<class 'numpy.float64'>
3.32600787221e-05
<class 'numpy.float64'>
3.3256592121e-05
<class 'numpy.float64'>
3.32531062516e-05
<class 'numpy.float64'>
3.32496211141e-05
<class 'numpy.float64'>
3.3246136708e-05
<class 'numpy.float64'>
3.32426530334e-05
<class 'numpy.float64'>
3.32391700895e-05
<class 'numpy.float64'>
3.32356878763e-05
<class 'numpy.float64'>
3.32322063936e-05
<class 'numpy.float64'>
3.32287256414e-05
<class 'numpy.float64'>
3.32252456192e-05
<class 'numpy.float64'>
3.32217663267e-05
<class 'numpy.float64'>
3.32182877639e-05
<class 'numpy.float64'>
3.32148099306e-05
<class 'numpy.float64'>
3.32113328264e-05
<class 'numpy.float64'>
3.32078564509e-05
<class 'numpy.float64'>
3.32043808044e-05
<class 'numpy.float64'>
3.3200905886e-05
<class 'numpy.float64'>
3.31974316959e-05
<class 'numpy.float64'>
3.31939582343e-05
<class 'numpy.float64'>
3.31904855e-05
<class 'numpy.float64'>
3.31870134935e-05
<class 'numpy.float64'>
3.31835422141e-05
<class 'numpy.float64'>
3.31800716617e-05
<class 'numpy.float64'>
3.31766018364e-05
<class 'numpy.float64'>
3.31731327374e-05
<class 'numpy.float64'>
3.31696643655e-05
<class 'numpy.float64'>
3.31661967192e-05
<class 'numpy.float64'>
3.31627297989e-05
<class 'numpy.float64'>
3.31592636044e-05
<class 'numpy.float64'>
3.31557981353e-05
<class 'numpy.float64'>
3.31523333915e-05
<class 'numpy.float64'>
3.3148869373e-05
<class 'numpy.float64'>
3.31454060787e-05
<class 'numpy.float64'>
3.31419435096e-05
<class 'numpy.float64'>
3.31384816645e-05
<class 'numpy.float64'>
3.31350205439e-05
<class 'numpy.float64'>
3.31315601466e-05
<class 'numpy.float64'>
3.31281004734e-05
<class 'numpy.float64'>
3.31246415234e-05
<class 'numpy.float64'>
3.31211832969e-05
<class 'numpy.float64'>
3.31177257931e-05
<class 'numpy.float64'>
3.31142690122e-05
<class 'numpy.float64'>
3.31108129537e-05
<class 'numpy.float64'>
3.3107357618e-05
<class 'numpy.float64'>
3.31039030038e-05
<class 'numpy.float64'>
3.31004491116e-05
<class 'numpy.float64'>
3.30969959413e-05
<class 'numpy.float64'>
3.30935434921e-05
<class 'numpy.float64'>
3.30900917642e-05
<class 'numpy.float64'>
3.30866407572e-05
<class 'numpy.float64'>
3.30831904707e-05
<class 'numpy.float64'>
3.3079740905e-05
<class 'numpy.float64'>
3.30762920596e-05
<class 'numpy.float64'>
3.30728439343e-05
<class 'numpy.float64'>
3.30693965287e-05
<class 'numpy.float64'>
3.30659498427e-05
<class 'numpy.float64'>
3.30625038761e-05
<class 'numpy.float64'>
3.30590586283e-05
<class 'numpy.float64'>
3.30556140997e-05
<class 'numpy.float64'>
3.30521702899e-05
<class 'numpy.float64'>
3.30487271987e-05
<class 'numpy.float64'>
3.30452848255e-05
<class 'numpy.float64'>
3.304184317e-05
<class 'numpy.float64'>
3.30384022326e-05
<class 'numpy.float64'>
3.30349620129e-05
<class 'numpy.float64'>
3.30315225105e-05
<class 'numpy.float64'>
3.3028083725e-05
<class 'numpy.float64'>
3.30246456568e-05
<class 'numpy.float64'>
3.30212083049e-05
<class 'numpy.float64'>
3.30177716696e-05
<class 'numpy.float64'>
3.30143357504e-05
<class 'numpy.float64'>
3.30109005472e-05
<class 'numpy.float64'>
3.30074660599e-05
<class 'numpy.float64'>
3.30040322883e-05
<class 'numpy.float64'>
3.30005992319e-05
<class 'numpy.float64'>
3.29971668907e-05
<class 'numpy.float64'>
3.29937352642e-05
<class 'numpy.float64'>
3.29903043521e-05
<class 'numpy.float64'>
3.29868741549e-05
<class 'numpy.float64'>
3.29834446718e-05
<class 'numpy.float64'>
3.29800159026e-05
<class 'numpy.float64'>
3.29765878469e-05
<class 'numpy.float64'>
3.29731605051e-05
<class 'numpy.float64'>
3.29697338768e-05
<class 'numpy.float64'>
3.29663079612e-05
<class 'numpy.float64'>
3.29628827587e-05
<class 'numpy.float64'>
3.29594582686e-05
<class 'numpy.float64'>
3.29560344911e-05
<class 'numpy.float64'>
3.2952611426e-05
<class 'numpy.float64'>
3.29491890726e-05
<class 'numpy.float64'>
3.2945767431e-05
<class 'numpy.float64'>
3.29423465012e-05
<class 'numpy.float64'>
3.29389262823e-05
<class 'numpy.float64'>
3.29355067747e-05
<class 'numpy.float64'>
3.29320879781e-05
<class 'numpy.float64'>
3.2928669892e-05
<class 'numpy.float64'>
3.29252525165e-05
<class 'numpy.float64'>
3.29218358507e-05
<class 'numpy.float64'>
3.29184198953e-05
<class 'numpy.float64'>
3.29150046498e-05
<class 'numpy.float64'>
3.29115901136e-05
<class 'numpy.float64'>
3.29081762869e-05
<class 'numpy.float64'>
3.2904763169e-05
<class 'numpy.float64'>
3.29013507602e-05
<class 'numpy.float64'>
3.28979390602e-05
<class 'numpy.float64'>
3.28945280685e-05
<class 'numpy.float64'>
3.28911177848e-05
<class 'numpy.float64'>
3.28877082092e-05
<class 'numpy.float64'>
3.28842993415e-05
<class 'numpy.float64'>
3.28808911814e-05
<class 'numpy.float64'>
3.28774837284e-05
<class 'numpy.float64'>
3.28740769828e-05
<class 'numpy.float64'>
3.28706709438e-05
<class 'numpy.float64'>
3.28672656117e-05
<class 'numpy.float64'>
3.2863860986e-05
<class 'numpy.float64'>
3.28604570667e-05
<class 'numpy.float64'>
3.28570538531e-05
<class 'numpy.float64'>
3.28536513453e-05
<class 'numpy.float64'>
3.28502495433e-05
<class 'numpy.float64'>
3.28468484464e-05
<class 'numpy.float64'>
3.28434480551e-05
<class 'numpy.float64'>
3.28400483682e-05
<class 'numpy.float64'>
3.28366493861e-05
<class 'numpy.float64'>
3.28332511087e-05
<class 'numpy.float64'>
3.28298535355e-05
<class 'numpy.float64'>
3.28264566663e-05
<class 'numpy.float64'>
3.28230605007e-05
<class 'numpy.float64'>
3.28196650391e-05
<class 'numpy.float64'>
3.28162702807e-05
<class 'numpy.float64'>
3.28128762253e-05
<class 'numpy.float64'>
3.28094828731e-05
<class 'numpy.float64'>
3.28060902235e-05
<class 'numpy.float64'>
3.28026982764e-05
<class 'numpy.float64'>
3.27993070319e-05
<class 'numpy.float64'>
3.27959164891e-05
<class 'numpy.float64'>
3.27925266482e-05
<class 'numpy.float64'>
3.27891375089e-05
<class 'numpy.float64'>
3.27857490714e-05
<class 'numpy.float64'>
3.27823613345e-05
<class 'numpy.float64'>
3.27789742993e-05
<class 'numpy.float64'>
3.27755879643e-05
<class 'numpy.float64'>
3.277220233e-05
<class 'numpy.float64'>
3.27688173961e-05
<class 'numpy.float64'>
3.27654331621e-05
<class 'numpy.float64'>
3.2762049628e-05
<class 'numpy.float64'>
3.27586667941e-05
<class 'numpy.float64'>
3.27552846592e-05
<class 'numpy.float64'>
3.27519032235e-05
<class 'numpy.float64'>
3.2748522487e-05
<class 'numpy.float64'>
3.27451424495e-05
<class 'numpy.float64'>
3.27417631104e-05
<class 'numpy.float64'>
3.27383844696e-05
<class 'numpy.float64'>
3.27350065271e-05
<class 'numpy.float64'>
3.27316292823e-05
<class 'numpy.float64'>
3.27282527357e-05
<class 'numpy.float64'>
3.27248768859e-05
<class 'numpy.float64'>
3.27215017337e-05
<class 'numpy.float64'>
3.27181272791e-05
<class 'numpy.float64'>
3.27147535208e-05
<class 'numpy.float64'>
3.27113804593e-05
<class 'numpy.float64'>
3.27080080941e-05
<class 'numpy.float64'>
3.27046364253e-05
<class 'numpy.float64'>
3.27012654523e-05
<class 'numpy.float64'>
3.26978951754e-05
<class 'numpy.float64'>
3.26945255938e-05
<class 'numpy.float64'>
3.26911567078e-05
<class 'numpy.float64'>
3.26877885165e-05
<class 'numpy.float64'>
3.26844210205e-05
<class 'numpy.float64'>
3.26810542194e-05
<class 'numpy.float64'>
3.26776881121e-05
<class 'numpy.float64'>
3.26743226997e-05
<class 'numpy.float64'>
3.26709579812e-05
<class 'numpy.float64'>
3.26675939564e-05
<class 'numpy.float64'>
3.26642306254e-05
<class 'numpy.float64'>
3.26608679875e-05
<class 'numpy.float64'>
3.26575060433e-05
<class 'numpy.float64'>
3.26541447917e-05
<class 'numpy.float64'>
3.2650784233e-05
<class 'numpy.float64'>
3.26474243669e-05
<class 'numpy.float64'>
3.2644065193e-05
<class 'numpy.float64'>
3.26407067115e-05
<class 'numpy.float64'>
3.26373489217e-05
<class 'numpy.float64'>
3.26339918237e-05
<class 'numpy.float64'>
3.2630635417e-05
<class 'numpy.float64'>
3.26272797019e-05
<class 'numpy.float64'>
3.26239246776e-05
<class 'numpy.float64'>
3.26205703443e-05
<class 'numpy.float64'>
3.26172167015e-05
<class 'numpy.float64'>
3.26138637491e-05
<class 'numpy.float64'>
3.26105114871e-05
<class 'numpy.float64'>
3.2607159915e-05
<class 'numpy.float64'>
3.26038090329e-05
<class 'numpy.float64'>
3.26004588401e-05
<class 'numpy.float64'>
3.25971093367e-05
<class 'numpy.float64'>
3.25937605224e-05
<class 'numpy.float64'>
3.25904123972e-05
<class 'numpy.float64'>
3.25870649604e-05
<class 'numpy.float64'>
3.25837182123e-05
<class 'numpy.float64'>
3.25803721526e-05
<class 'numpy.float64'>
3.2577026781e-05
<class 'numpy.float64'>
3.25736820973e-05
<class 'numpy.float64'>
3.25703381011e-05
<class 'numpy.float64'>
3.25669947922e-05
<class 'numpy.float64'>
3.25636521708e-05
<class 'numpy.float64'>
3.25603102364e-05
<class 'numpy.float64'>
3.25569689885e-05
<class 'numpy.float64'>
3.25536284277e-05
<class 'numpy.float64'>
3.25502885529e-05
<class 'numpy.float64'>
3.25469493643e-05
<class 'numpy.float64'>
3.25436108617e-05
<class 'numpy.float64'>
3.25402730449e-05
<class 'numpy.float64'>
3.25369359137e-05
<class 'numpy.float64'>
3.25335994678e-05
<class 'numpy.float64'>
3.2530263707e-05
<class 'numpy.float64'>
3.2526928631e-05
<class 'numpy.float64'>
3.25235942398e-05
<class 'numpy.float64'>
3.2520260533e-05
<class 'numpy.float64'>
3.25169275106e-05
<class 'numpy.float64'>
3.25135951721e-05
<class 'numpy.float64'>
3.25102635175e-05
<class 'numpy.float64'>
3.25069325468e-05
<class 'numpy.float64'>
3.25036022592e-05
<class 'numpy.float64'>
3.2500272655e-05
<class 'numpy.float64'>
3.24969437336e-05
<class 'numpy.float64'>
3.24936154953e-05
<class 'numpy.float64'>
3.24902879395e-05
<class 'numpy.float64'>
3.24869610658e-05
<class 'numpy.float64'>
3.24836348745e-05
<class 'numpy.float64'>
3.24803093651e-05
<class 'numpy.float64'>
3.24769845374e-05
<class 'numpy.float64'>
3.24736603914e-05
<class 'numpy.float64'>
3.24703369267e-05
<class 'numpy.float64'>
3.24670141431e-05
<class 'numpy.float64'>
3.24636920405e-05
<class 'numpy.float64'>
3.24603706182e-05
<class 'numpy.float64'>
3.24570498769e-05
<class 'numpy.float64'>
3.24537298155e-05
<class 'numpy.float64'>
3.24504104343e-05
<class 'numpy.float64'>
3.24470917331e-05
<class 'numpy.float64'>
3.24437737116e-05
<class 'numpy.float64'>
3.2440456369e-05
<class 'numpy.float64'>
3.24371397063e-05
<class 'numpy.float64'>
3.24338237224e-05
<class 'numpy.float64'>
3.24305084173e-05
<class 'numpy.float64'>
3.24271937907e-05
<class 'numpy.float64'>
3.24238798426e-05
<class 'numpy.float64'>
3.24205665727e-05
<class 'numpy.float64'>
3.24172539806e-05
<class 'numpy.float64'>
3.24139420667e-05
<class 'numpy.float64'>
3.241063083e-05
<class 'numpy.float64'>
3.24073202707e-05
<class 'numpy.float64'>
3.24040103887e-05
<class 'numpy.float64'>
3.24007011834e-05
<class 'numpy.float64'>
3.23973926552e-05
<class 'numpy.float64'>
3.23940848036e-05
<class 'numpy.float64'>
3.23907776277e-05
<class 'numpy.float64'>
3.23874711284e-05
<class 'numpy.float64'>
3.23841653049e-05
<class 'numpy.float64'>
3.23808601571e-05
<class 'numpy.float64'>
3.23775556848e-05
<class 'numpy.float64'>
3.23742518878e-05
<class 'numpy.float64'>
3.23709487658e-05
<class 'numpy.float64'>
3.23676463189e-05
<class 'numpy.float64'>
3.23643445463e-05
<class 'numpy.float64'>
3.23610434484e-05
<class 'numpy.float64'>
3.23577430248e-05
<class 'numpy.float64'>
3.23544432752e-05
<class 'numpy.float64'>
3.23511441994e-05
<class 'numpy.float64'>
3.2347845797e-05
<class 'numpy.float64'>
3.23445480684e-05
<class 'numpy.float64'>
3.23412510126e-05
<class 'numpy.float64'>
3.23379546302e-05
<class 'numpy.float64'>
3.23346589204e-05
<class 'numpy.float64'>
3.23313638836e-05
<class 'numpy.float64'>
3.23280695186e-05
<class 'numpy.float64'>
3.23247758261e-05
<class 'numpy.float64'>
3.23214828055e-05
<class 'numpy.float64'>
3.23181904569e-05
<class 'numpy.float64'>
3.23148987797e-05
<class 'numpy.float64'>
3.2311607774e-05
<class 'numpy.float64'>
3.23083174392e-05
<class 'numpy.float64'>
3.23050277753e-05
<class 'numpy.float64'>
3.23017387826e-05
<class 'numpy.float64'>
3.229845046e-05
<class 'numpy.float64'>
3.22951628081e-05
<class 'numpy.float64'>
3.2291875826e-05
<class 'numpy.float64'>
3.22885895139e-05
<class 'numpy.float64'>
3.22853038718e-05
<class 'numpy.float64'>
3.22820188987e-05
<class 'numpy.float64'>
3.22787345955e-05
<class 'numpy.float64'>
3.22754509612e-05
<class 'numpy.float64'>
3.22721679956e-05
<class 'numpy.float64'>
3.2268885699e-05
<class 'numpy.float64'>
3.22656040706e-05
<class 'numpy.float64'>
3.22623231107e-05
<class 'numpy.float64'>
3.22590428188e-05
<class 'numpy.float64'>
3.22557631946e-05
<class 'numpy.float64'>
3.22524842385e-05
<class 'numpy.float64'>
3.22492059495e-05
<class 'numpy.float64'>
3.22459283282e-05
<class 'numpy.float64'>
3.22426513737e-05
<class 'numpy.float64'>
3.2239375086e-05
<class 'numpy.float64'>
3.22360994651e-05
<class 'numpy.float64'>
3.22328245107e-05
<class 'numpy.float64'>
3.22295502223e-05
<class 'numpy.float64'>
3.22262765999e-05
<class 'numpy.float64'>
3.22230036438e-05
<class 'numpy.float64'>
3.2219731353e-05
<class 'numpy.float64'>
3.22164597279e-05
<class 'numpy.float64'>
3.22131887676e-05
<class 'numpy.float64'>
3.22099184728e-05
<class 'numpy.float64'>
3.22066488425e-05
<class 'numpy.float64'>
3.2203379877e-05
<class 'numpy.float64'>
3.22001115758e-05
<class 'numpy.float64'>
3.21968439389e-05
<class 'numpy.float64'>
3.2193576966e-05
<class 'numpy.float64'>
3.21903106569e-05
<class 'numpy.float64'>
3.21870450117e-05
<class 'numpy.float64'>
3.21837800296e-05
<class 'numpy.float64'>
3.21805157107e-05
<class 'numpy.float64'>
3.21772520551e-05
<class 'numpy.float64'>
3.2173989062e-05
<class 'numpy.float64'>
3.21707267314e-05
<class 'numpy.float64'>
3.21674650637e-05
<class 'numpy.float64'>
3.21642040578e-05
<class 'numpy.float64'>
3.21609437139e-05
<class 'numpy.float64'>
3.21576840321e-05
<class 'numpy.float64'>
3.21544250115e-05
<class 'numpy.float64'>
3.21511666527e-05
<class 'numpy.float64'>
3.21479089547e-05
<class 'numpy.float64'>
3.21446519178e-05
<class 'numpy.float64'>
3.21413955419e-05
<class 'numpy.float64'>
3.21381398267e-05
<class 'numpy.float64'>
3.21348847715e-05
<class 'numpy.float64'>
3.21316303767e-05
<class 'numpy.float64'>
3.21283766416e-05
<class 'numpy.float64'>
3.21251235666e-05
<class 'numpy.float64'>
3.21218711513e-05
<class 'numpy.float64'>
3.21186193949e-05
<class 'numpy.float64'>
3.2115368298e-05
<class 'numpy.float64'>
3.21121178602e-05
<class 'numpy.float64'>
3.21088680808e-05
<class 'numpy.float64'>
3.21056189603e-05
<class 'numpy.float64'>
3.21023704982e-05
<class 'numpy.float64'>
3.20991226941e-05
<class 'numpy.float64'>
3.20958755482e-05
<class 'numpy.float64'>
3.20926290597e-05
<class 'numpy.float64'>
3.2089383229e-05
<class 'numpy.float64'>
3.20861380558e-05
<class 'numpy.float64'>
3.20828935397e-05
<class 'numpy.float64'>
3.20796496807e-05
<class 'numpy.float64'>
3.20764064781e-05
<class 'numpy.float64'>
3.20731639322e-05
<class 'numpy.float64'>
3.20699220428e-05
<class 'numpy.float64'>
3.20666808097e-05
<class 'numpy.float64'>
3.20634402324e-05
<class 'numpy.float64'>
3.2060200311e-05
<class 'numpy.float64'>
3.20569610453e-05
<class 'numpy.float64'>
3.20537224348e-05
<class 'numpy.float64'>
3.20504844793e-05
<class 'numpy.float64'>
3.2047247179e-05
<class 'numpy.float64'>
3.20440105336e-05
<class 'numpy.float64'>
3.20407745429e-05
<class 'numpy.float64'>
3.20375392063e-05
<class 'numpy.float64'>
3.20343045239e-05
<class 'numpy.float64'>
3.20310704956e-05
<class 'numpy.float64'>
3.2027837121e-05
<class 'numpy.float64'>
3.20246044001e-05
<class 'numpy.float64'>
3.20213723324e-05
<class 'numpy.float64'>
3.20181409181e-05
<class 'numpy.float64'>
3.2014910157e-05
<class 'numpy.float64'>
3.20116800483e-05
<class 'numpy.float64'>
3.20084505925e-05
<class 'numpy.float64'>
3.2005221789e-05
<class 'numpy.float64'>
3.20019936376e-05
<class 'numpy.float64'>
3.19987661384e-05
<class 'numpy.float64'>
3.19955392912e-05
<class 'numpy.float64'>
3.19923130949e-05
<class 'numpy.float64'>
3.19890875507e-05
<class 'numpy.float64'>
3.19858626575e-05
<class 'numpy.float64'>
3.19826384154e-05
<class 'numpy.float64'>
3.19794148238e-05
<class 'numpy.float64'>
3.19761918833e-05
<class 'numpy.float64'>
3.19729695929e-05
<class 'numpy.float64'>
3.1969747953e-05
<class 'numpy.float64'>
3.19665269632e-05
<class 'numpy.float64'>
3.19633066229e-05
<class 'numpy.float64'>
3.19600869324e-05
<class 'numpy.float64'>
3.19568678915e-05
<class 'numpy.float64'>
3.19536494997e-05
<class 'numpy.float64'>
3.19504317568e-05
<class 'numpy.float64'>
3.1947214663e-05
<class 'numpy.float64'>
3.19439982178e-05
<class 'numpy.float64'>
3.19407824209e-05
<class 'numpy.float64'>
3.19375672726e-05
<class 'numpy.float64'>
3.19343527722e-05
<class 'numpy.float64'>
3.19311389198e-05
<class 'numpy.float64'>
3.19279257148e-05
<class 'numpy.float64'>
3.19247131577e-05
<class 'numpy.float64'>
3.19215012476e-05
<class 'numpy.float64'>
3.19182899848e-05
<class 'numpy.float64'>
3.19150793686e-05
<class 'numpy.float64'>
3.19118693994e-05
<class 'numpy.float64'>
3.19086600766e-05
<class 'numpy.float64'>
3.19054513999e-05
<class 'numpy.float64'>
3.19022433695e-05
<class 'numpy.float64'>
3.18990359852e-05
<class 'numpy.float64'>
3.18958292466e-05
<class 'numpy.float64'>
3.18926231535e-05
<class 'numpy.float64'>
3.18894177055e-05
<class 'numpy.float64'>
3.18862129026e-05
<class 'numpy.float64'>
3.1883008745e-05
<class 'numpy.float64'>
3.1879805232e-05
<class 'numpy.float64'>
3.18766023637e-05
<class 'numpy.float64'>
3.18734001397e-05
<class 'numpy.float64'>
3.18701985597e-05
<class 'numpy.float64'>
3.18669976238e-05
<class 'numpy.float64'>
3.18637973316e-05
<class 'numpy.float64'>
3.1860597683e-05
<class 'numpy.float64'>
3.1857398678e-05
<class 'numpy.float64'>
3.1854200316e-05
<class 'numpy.float64'>
3.18510025971e-05
<class 'numpy.float64'>
3.18478055209e-05
<class 'numpy.float64'>
3.18446090873e-05
<class 'numpy.float64'>
3.18414132962e-05
<class 'numpy.float64'>
3.18382181475e-05
<class 'numpy.float64'>
3.18350236405e-05
<class 'numpy.float64'>
3.18318297756e-05
<class 'numpy.float64'>
3.18286365525e-05
<class 'numpy.float64'>
3.18254439704e-05
<class 'numpy.float64'>
3.18222520298e-05
<class 'numpy.float64'>
3.18190607302e-05
<class 'numpy.float64'>
3.18158700715e-05
<class 'numpy.float64'>
3.18126800536e-05
<class 'numpy.float64'>
3.1809490676e-05
<class 'numpy.float64'>
3.18063019388e-05
<class 'numpy.float64'>
3.18031138418e-05
<class 'numpy.float64'>
3.17999263846e-05
<class 'numpy.float64'>
3.17967395671e-05
<class 'numpy.float64'>
3.17935533892e-05
<class 'numpy.float64'>
3.17903678506e-05
<class 'numpy.float64'>
3.17871829511e-05
<class 'numpy.float64'>
3.17839986904e-05
<class 'numpy.float64'>
3.17808150689e-05
<class 'numpy.float64'>
3.17776320855e-05
<class 'numpy.float64'>
3.17744497408e-05
<class 'numpy.float64'>
3.1771268034e-05
<class 'numpy.float64'>
3.17680869655e-05
<class 'numpy.float64'>
3.17649065345e-05
<class 'numpy.float64'>
3.17617267414e-05
<class 'numpy.float64'>
3.17585475854e-05
<class 'numpy.float64'>
3.17553690667e-05
<class 'numpy.float64'>
3.17521911854e-05
<class 'numpy.float64'>
3.17490139404e-05
<class 'numpy.float64'>
3.17458373325e-05
<class 'numpy.float64'>
3.17426613605e-05
<class 'numpy.float64'>
3.17394860252e-05
<class 'numpy.float64'>
3.1736311326e-05
<class 'numpy.float64'>
3.17331372626e-05
<class 'numpy.float64'>
3.17299638347e-05
<class 'numpy.float64'>
3.17267910423e-05
<class 'numpy.float64'>
3.17236188855e-05
<class 'numpy.float64'>
3.17204473637e-05
<class 'numpy.float64'>
3.17172764767e-05
<class 'numpy.float64'>
3.17141062245e-05
<class 'numpy.float64'>
3.1710936607e-05
<class 'numpy.float64'>
3.17077676235e-05
<class 'numpy.float64'>
3.17045992743e-05
<class 'numpy.float64'>
3.17014315591e-05
<class 'numpy.float64'>
3.16982644778e-05
<class 'numpy.float64'>
3.16950980299e-05
<class 'numpy.float64'>
3.16919322156e-05
<class 'numpy.float64'>
3.16887670346e-05
<class 'numpy.float64'>
3.16856024862e-05
<class 'numpy.float64'>
3.16824385708e-05
<class 'numpy.float64'>
3.16792752883e-05
<class 'numpy.float64'>
3.16761126381e-05
<class 'numpy.float64'>
3.167295062e-05
<class 'numpy.float64'>
3.16697892341e-05
<class 'numpy.float64'>
3.16666284802e-05
<class 'numpy.float64'>
3.16634683578e-05
<class 'numpy.float64'>
W1 = [[-0.65848169  1.21866811]
 [-0.76204273  1.39377573]
 [ 0.5792005  -1.10397703]
 [ 0.76773391 -1.41477129]]
b1 = [[ 0.287592  ]
 [ 0.3511264 ]
 [-0.2431246 ]
 [-0.35772805]]
W2 = [[-2.45566237 -3.27042274  2.00784958  3.36773273]]
b2 = [[ 0.20459656]]
Expected Output:

cost after iteration 0	0.692739
⋮⋮
⋮⋮
W1	[[-0.65848169 1.21866811] [-0.76204273 1.39377573] [ 0.5792005 -1.10397703] [ 0.76773391 -1.41477129]]
b1	[[ 0.287592 ] [ 0.3511264 ] [-0.2431246 ] [-0.35772805]]
W2	[[-2.45566237 -3.27042274 2.00784958 3.36773273]]
b2	[[ 0.20459656]]
4.5 Predictions
Question: Use your model to predict by building predict(). Use forward propagation to predict results.

Reminder: predictions = yprediction=𝟙{activation > 0.5}={10if activation>0.5otherwiseyprediction=1{activation > 0.5}={1if activation>0.50otherwise

As an example, if you would like to set the entries of a matrix X to 0 and 1 based on a threshold you would do: X_new = (X > threshold)


A2>0.5
# GRADED FUNCTION: predict
​
def predict(parameters, X):
    """
    Using the learned parameters, predicts a class for each example in X
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    X -- input data of size (n_x, m)
    
    Returns
    predictions -- vector of predictions of our model (red: 0 / blue: 1)
    """
    
    # Computes probabilities using forward propagation, and classifies to 0/1 using 0.5 as the threshold.
    ### START CODE HERE ### (≈ 2 lines of code)
    A2, cache = forward_propagation(X,parameters)
    predictions = A2>0.5
    ### END CODE HERE ###
    
    return predictions

parameters, X_assess = predict_test_case()
​
predictions = predict(parameters, X_assess)
print("predictions mean = " + str(np.mean(predictions)))
predictions mean = 0.666666666667
Expected Output:

predictions mean	0.666666666667
It is time to run the model and see how it performs on a planar dataset. Run the following code to test your model with a single hidden layer of nhnh hidden units.


# Build a model with a n_h-dimensional hidden layer
parameters = nn_model(X, Y, n_h = 4, num_iterations = 10000, print_cost=True)
​
# Plot the decision boundary
plot_decision_boundary(lambda x: predict(parameters, x.T), X, Y)
plt.title("Decision Boundary for hidden layer size " + str(4))
138.61396309
<class 'numpy.float64'>
Cost after iteration 0: 138.613963
138.602029118
<class 'numpy.float64'>
138.585063516
<class 'numpy.float64'>
138.560182249
<class 'numpy.float64'>
138.52337152
<class 'numpy.float64'>
138.469155563
<class 'numpy.float64'>
138.390444732
<class 'numpy.float64'>
138.278908812
<class 'numpy.float64'>
138.126411489
<class 'numpy.float64'>
137.927976905
<class 'numpy.float64'>
137.685938448
<class 'numpy.float64'>
137.413157553
<class 'numpy.float64'>
137.131877837
<class 'numpy.float64'>
136.866684058
<class 'numpy.float64'>
136.635183243
<class 'numpy.float64'>
136.442843824
<class 'numpy.float64'>
136.284767838
<class 'numpy.float64'>
136.15139423
<class 'numpy.float64'>
136.033496399
<class 'numpy.float64'>
135.924203976
<class 'numpy.float64'>
135.818450103
<class 'numpy.float64'>
135.711347059
<class 'numpy.float64'>
135.596801275
<class 'numpy.float64'>
135.466957977
<class 'numpy.float64'>
135.312407618
<class 'numpy.float64'>
135.122826931
<class 'numpy.float64'>
134.887846815
<class 'numpy.float64'>
134.598156339
<class 'numpy.float64'>
134.246899154
<class 'numpy.float64'>
133.831210055
<class 'numpy.float64'>
133.353417528
<class 'numpy.float64'>
132.821330375
<class 'numpy.float64'>
132.247360907
<class 'numpy.float64'>
131.64678773
<class 'numpy.float64'>
131.035751487
<class 'numpy.float64'>
130.42948025
<class 'numpy.float64'>
129.841008108
<class 'numpy.float64'>
129.280464593
<class 'numpy.float64'>
128.754883807
<class 'numpy.float64'>
128.268401965
<class 'numpy.float64'>
127.822688178
<class 'numpy.float64'>
127.417473435
<class 'numpy.float64'>
127.051081395
<class 'numpy.float64'>
126.720902995
<class 'numpy.float64'>
126.42378738
<class 'numpy.float64'>
126.156342859
<class 'numpy.float64'>
125.915154525
<class 'numpy.float64'>
125.696931594
<class 'numpy.float64'>
125.498599436
<class 'numpy.float64'>
125.317350494
<class 'numpy.float64'>
125.150666149
<class 'numpy.float64'>
124.996319086
<class 'numpy.float64'>
124.852363298
<class 'numpy.float64'>
124.717116806
<class 'numpy.float64'>
124.589140596
<class 'numpy.float64'>
124.467216011
<class 'numpy.float64'>
124.35032203
<class 'numpy.float64'>
124.237613238
<class 'numpy.float64'>
124.128398928
<class 'numpy.float64'>
124.022123544
<class 'numpy.float64'>
123.918348531
<class 'numpy.float64'>
123.816735606
<class 'numpy.float64'>
123.717031423
<class 'numpy.float64'>
123.619053592
<class 'numpy.float64'>
123.52267804
<class 'numpy.float64'>
123.427827654
<class 'numpy.float64'>
123.334462194
<class 'numpy.float64'>
123.242569412
<class 'numpy.float64'>
123.152157348
<class 'numpy.float64'>
123.06324773
<class 'numpy.float64'>
122.975870417
<class 'numpy.float64'>
122.890058811
<class 'numpy.float64'>
122.805846162
<class 'numpy.float64'>
122.723262676
<class 'numpy.float64'>
122.642333348
<class 'numpy.float64'>
122.563076428
<class 'numpy.float64'>
122.485502447
<class 'numpy.float64'>
122.409613703
<class 'numpy.float64'>
122.33540414
<class 'numpy.float64'>
122.262859533
<class 'numpy.float64'>
122.191957909
<class 'numpy.float64'>
122.122670133
<class 'numpy.float64'>
122.054960595
<class 'numpy.float64'>
121.988787934
<class 'numpy.float64'>
121.92410576
<class 'numpy.float64'>
121.860863314
<class 'numpy.float64'>
121.799006025
<class 'numpy.float64'>
121.738475949
<class 'numpy.float64'>
121.679212025
<class 'numpy.float64'>
121.62115014
<class 'numpy.float64'>
121.564222949
<class 'numpy.float64'>
121.508359417
<class 'numpy.float64'>
121.453484003
<class 'numpy.float64'>
121.399515416
<class 'numpy.float64'>
121.346364814
<class 'numpy.float64'>
121.293933253
<class 'numpy.float64'>
121.242108125
<class 'numpy.float64'>
121.190758172
<class 'numpy.float64'>
121.139726465
<class 'numpy.float64'>
121.088820381
<class 'numpy.float64'>
121.037797084
<class 'numpy.float64'>
120.986342111
<class 'numpy.float64'>
120.934037102
<class 'numpy.float64'>
120.880310034
<class 'numpy.float64'>
120.824356451
<class 'numpy.float64'>
120.765011072
<class 'numpy.float64'>
120.70053167
<class 'numpy.float64'>
120.628221821
<class 'numpy.float64'>
120.543745389
<class 'numpy.float64'>
120.439824529
<class 'numpy.float64'>
120.303647054
<class 'numpy.float64'>
120.111456652
<class 'numpy.float64'>
119.816855799
<class 'numpy.float64'>
119.325743313
<class 'numpy.float64'>
118.451608837
<class 'numpy.float64'>
116.905995239
<class 'numpy.float64'>
114.617757894
<class 'numpy.float64'>
112.179828344
<class 'numpy.float64'>
109.964098573
<class 'numpy.float64'>
107.914444068
<class 'numpy.float64'>
106.005473016
<class 'numpy.float64'>
104.216173522
<class 'numpy.float64'>
102.527681622
<class 'numpy.float64'>
100.927329943
<class 'numpy.float64'>
99.4071504724
<class 'numpy.float64'>
97.9620533385
<class 'numpy.float64'>
96.5885950209
<class 'numpy.float64'>
95.2841957326
<class 'numpy.float64'>
94.0466480585
<class 'numpy.float64'>
92.8738213289
<class 'numpy.float64'>
91.763500751
<class 'numpy.float64'>
90.7133178378
<class 'numpy.float64'>
89.7207396857
<class 'numpy.float64'>
88.7830930875
<class 'numpy.float64'>
87.8976063496
<class 'numpy.float64'>
87.0614572153
<class 'numpy.float64'>
86.2718195837
<class 'numpy.float64'>
85.5259048604
<class 'numpy.float64'>
84.8209959908
<class 'numpy.float64'>
84.1544736782
<class 'numpy.float64'>
83.5238351784
<class 'numpy.float64'>
82.926706542
<class 'numpy.float64'>
82.3608493769
<class 'numpy.float64'>
81.8241632311
<class 'numpy.float64'>
81.3146846166
<class 'numpy.float64'>
80.8305835679
<class 'numpy.float64'>
80.3701584778
<class 'numpy.float64'>
79.9318298111
<class 'numpy.float64'>
79.5141331599
<class 'numpy.float64'>
79.1157119945
<class 'numpy.float64'>
78.7353103677
<class 'numpy.float64'>
78.3717657533
<class 'numpy.float64'>
78.0240021451
<class 'numpy.float64'>
77.6910234928
<class 'numpy.float64'>
77.3719075228
<class 'numpy.float64'>
77.0657999624
<class 'numpy.float64'>
76.7719091742
<class 'numpy.float64'>
76.4895011911
<class 'numpy.float64'>
76.2178951376
<class 'numpy.float64'>
75.9564590161
<class 'numpy.float64'>
75.7046058369
<class 'numpy.float64'>
75.461790066
<class 'numpy.float64'>
75.2275043684
<class 'numpy.float64'>
75.0012766225
<class 'numpy.float64'>
74.7826671834
<class 'numpy.float64'>
74.5712663744
<class 'numpy.float64'>
74.3666921867
<class 'numpy.float64'>
74.1685881694
<class 'numpy.float64'>
73.9766214937
<class 'numpy.float64'>
73.790481175
<class 'numpy.float64'>
73.6098764399
<class 'numpy.float64'>
73.4345352264
<class 'numpy.float64'>
73.2642028027
<class 'numpy.float64'>
73.0986404987
<class 'numpy.float64'>
72.9376245365
<class 'numpy.float64'>
72.7809449545
<class 'numpy.float64'>
72.6284046154
<class 'numpy.float64'>
72.4798182921
<class 'numpy.float64'>
72.3350118239
<class 'numpy.float64'>
72.193821339
<class 'numpy.float64'>
72.0560925361
<class 'numpy.float64'>
71.921680021
<class 'numpy.float64'>
71.7904466939
<class 'numpy.float64'>
71.6622631832
<class 'numpy.float64'>
71.5370073218
<class 'numpy.float64'>
71.4145636628
<class 'numpy.float64'>
71.2948230314
<class 'numpy.float64'>
71.1776821103
<class 'numpy.float64'>
71.0630430552
<class 'numpy.float64'>
70.9508131393
<class 'numpy.float64'>
70.8409044233
<class 'numpy.float64'>
70.7332334501
<class 'numpy.float64'>
70.6277209603
<class 'numpy.float64'>
70.5242916299
<class 'numpy.float64'>
70.4228738256
<class 'numpy.float64'>
70.3233993777
<class 'numpy.float64'>
70.2258033695
<class 'numpy.float64'>
70.1300239413
<class 'numpy.float64'>
70.0360021075
<class 'numpy.float64'>
69.9436815875
<class 'numpy.float64'>
69.8530086475
<class 'numpy.float64'>
69.763931953
<class 'numpy.float64'>
69.6764024321
<class 'numpy.float64'>
69.590373147
<class 'numpy.float64'>
69.505799175
<class 'numpy.float64'>
69.4226374968
<class 'numpy.float64'>
69.3408468924
<class 'numpy.float64'>
69.2603878439
<class 'numpy.float64'>
69.1812224443
<class 'numpy.float64'>
69.1033143123
<class 'numpy.float64'>
69.0266285126
<class 'numpy.float64'>
68.9511314814
<class 'numpy.float64'>
68.8767909559
<class 'numpy.float64'>
68.8035759088
<class 'numpy.float64'>
68.7314564868
<class 'numpy.float64'>
68.6604039525
<class 'numpy.float64'>
68.5903906299
<class 'numpy.float64'>
68.5213898535
<class 'numpy.float64'>
68.4533759198
<class 'numpy.float64'>
68.3863240424
<class 'numpy.float64'>
68.3202103091
<class 'numpy.float64'>
68.2550116416
<class 'numpy.float64'>
68.1907057577
<class 'numpy.float64'>
68.1272711353
<class 'numpy.float64'>
68.0646869789
<class 'numpy.float64'>
68.0029331877
<class 'numpy.float64'>
67.9419903248
<class 'numpy.float64'>
67.8818395896
<class 'numpy.float64'>
67.8224627898
<class 'numpy.float64'>
67.7638423168
<class 'numpy.float64'>
67.7059611204
<class 'numpy.float64'>
67.6488026869
<class 'numpy.float64'>
67.5923510165
<class 'numpy.float64'>
67.5365906029
<class 'numpy.float64'>
67.4815064138
<class 'numpy.float64'>
67.4270838719
<class 'numpy.float64'>
67.3733088376
<class 'numpy.float64'>
67.3201675916
<class 'numpy.float64'>
67.2676468192
<class 'numpy.float64'>
67.2157335949
<class 'numpy.float64'>
67.1644153679
<class 'numpy.float64'>
67.1136799479
<class 'numpy.float64'>
67.0635154925
<class 'numpy.float64'>
67.0139104937
<class 'numpy.float64'>
66.9648537666
<class 'numpy.float64'>
66.9163344373
<class 'numpy.float64'>
66.8683419322
<class 'numpy.float64'>
66.8208659676
<class 'numpy.float64'>
66.7738965391
<class 'numpy.float64'>
66.7274239126
<class 'numpy.float64'>
66.6814386144
<class 'numpy.float64'>
66.635931423
<class 'numpy.float64'>
66.5908933602
<class 'numpy.float64'>
66.5463156831
<class 'numpy.float64'>
66.5021898765
<class 'numpy.float64'>
66.458507645
<class 'numpy.float64'>
66.4152609062
<class 'numpy.float64'>
66.3724417838
<class 'numpy.float64'>
66.3300426007
<class 'numpy.float64'>
66.2880558729
<class 'numpy.float64'>
66.2464743033
<class 'numpy.float64'>
66.2052907759
<class 'numpy.float64'>
66.1644983501
<class 'numpy.float64'>
66.124090255
<class 'numpy.float64'>
66.0840598846
<class 'numpy.float64'>
66.0444007928
<class 'numpy.float64'>
66.0051066878
<class 'numpy.float64'>
65.9661714285
<class 'numpy.float64'>
65.9275890189
<class 'numpy.float64'>
65.8893536047
<class 'numpy.float64'>
65.8514594684
<class 'numpy.float64'>
65.8139010259
<class 'numpy.float64'>
65.7766728218
<class 'numpy.float64'>
65.7397695266
<class 'numpy.float64'>
65.7031859322
<class 'numpy.float64'>
65.6669169489
<class 'numpy.float64'>
65.6309576017
<class 'numpy.float64'>
65.5953030276
<class 'numpy.float64'>
65.5599484716
<class 'numpy.float64'>
65.5248892844
<class 'numpy.float64'>
65.4901209189
<class 'numpy.float64'>
65.4556389277
<class 'numpy.float64'>
65.4214389603
<class 'numpy.float64'>
65.3875167603
<class 'numpy.float64'>
65.3538681627
<class 'numpy.float64'>
65.3204890918
<class 'numpy.float64'>
65.2873755585
<class 'numpy.float64'>
65.2545236579
<class 'numpy.float64'>
65.2219295675
<class 'numpy.float64'>
65.1895895442
<class 'numpy.float64'>
65.157499923
<class 'numpy.float64'>
65.1256571144
<class 'numpy.float64'>
65.0940576026
<class 'numpy.float64'>
65.0626979438
<class 'numpy.float64'>
65.0315747638
<class 'numpy.float64'>
65.0006847566
<class 'numpy.float64'>
64.9700246825
<class 'numpy.float64'>
64.9395913666
<class 'numpy.float64'>
64.9093816968
<class 'numpy.float64'>
64.8793926223
<class 'numpy.float64'>
64.8496211524
<class 'numpy.float64'>
64.8200643545
<class 'numpy.float64'>
64.7907193529
<class 'numpy.float64'>
64.7615833275
<class 'numpy.float64'>
64.7326535121
<class 'numpy.float64'>
64.7039271932
<class 'numpy.float64'>
64.6754017089
<class 'numpy.float64'>
64.6470744474
<class 'numpy.float64'>
64.6189428459
<class 'numpy.float64'>
64.5910043894
<class 'numpy.float64'>
64.5632566093
<class 'numpy.float64'>
64.535697083
<class 'numpy.float64'>
64.5083234317
<class 'numpy.float64'>
64.4811333206
<class 'numpy.float64'>
64.4541244568
<class 'numpy.float64'>
64.4272945889
<class 'numpy.float64'>
64.400641506
<class 'numpy.float64'>
64.3741630364
<class 'numpy.float64'>
64.3478570473
<class 'numpy.float64'>
64.3217214431
<class 'numpy.float64'>
64.2957541653
<class 'numpy.float64'>
64.2699531913
<class 'numpy.float64'>
64.2443165336
<class 'numpy.float64'>
64.2188422388
<class 'numpy.float64'>
64.1935283875
<class 'numpy.float64'>
64.1683730928
<class 'numpy.float64'>
64.1433745
<class 'numpy.float64'>
64.1185307857
<class 'numpy.float64'>
64.0938401571
<class 'numpy.float64'>
64.0693008516
<class 'numpy.float64'>
64.0449111358
<class 'numpy.float64'>
64.0206693051
<class 'numpy.float64'>
63.9965736829
<class 'numpy.float64'>
63.9726226201
<class 'numpy.float64'>
63.9488144945
<class 'numpy.float64'>
63.9251477102
<class 'numpy.float64'>
63.9016206972
<class 'numpy.float64'>
63.8782319104
<class 'numpy.float64'>
63.8549798297
<class 'numpy.float64'>
63.8318629589
<class 'numpy.float64'>
63.8088798258
<class 'numpy.float64'>
63.786028981
<class 'numpy.float64'>
63.7633089979
<class 'numpy.float64'>
63.7407184724
<class 'numpy.float64'>
63.7182560218
<class 'numpy.float64'>
63.6959202849
<class 'numpy.float64'>
63.6737099214
<class 'numpy.float64'>
63.6516236114
<class 'numpy.float64'>
63.6296600552
<class 'numpy.float64'>
63.6078179726
<class 'numpy.float64'>
63.5860961028
<class 'numpy.float64'>
63.5644932037
<class 'numpy.float64'>
63.543008052
<class 'numpy.float64'>
63.5216394424
<class 'numpy.float64'>
63.5003861873
<class 'numpy.float64'>
63.4792471168
<class 'numpy.float64'>
63.4582210779
<class 'numpy.float64'>
63.4373069344
<class 'numpy.float64'>
63.4165035667
<class 'numpy.float64'>
63.3958098713
<class 'numpy.float64'>
63.3752247605
<class 'numpy.float64'>
63.3547471621
<class 'numpy.float64'>
63.3343760193
<class 'numpy.float64'>
63.3141102901
<class 'numpy.float64'>
63.2939489474
<class 'numpy.float64'>
63.2738909782
<class 'numpy.float64'>
63.253935384
<class 'numpy.float64'>
63.23408118
<class 'numpy.float64'>
63.214327395
<class 'numpy.float64'>
63.1946730711
<class 'numpy.float64'>
63.1751172639
<class 'numpy.float64'>
63.1556590416
<class 'numpy.float64'>
63.1362974851
<class 'numpy.float64'>
63.1170316878
<class 'numpy.float64'>
63.0978607554
<class 'numpy.float64'>
63.0787838054
<class 'numpy.float64'>
63.0597999673
<class 'numpy.float64'>
63.040908382
<class 'numpy.float64'>
63.022108202
<class 'numpy.float64'>
63.0033985907
<class 'numpy.float64'>
62.9847787228
<class 'numpy.float64'>
62.9662477835
<class 'numpy.float64'>
62.9478049689
<class 'numpy.float64'>
62.9294494852
<class 'numpy.float64'>
62.9111805493
<class 'numpy.float64'>
62.8929973876
<class 'numpy.float64'>
62.8748992369
<class 'numpy.float64'>
62.8568853436
<class 'numpy.float64'>
62.8389549635
<class 'numpy.float64'>
62.821107362
<class 'numpy.float64'>
62.8033418137
<class 'numpy.float64'>
62.7856576022
<class 'numpy.float64'>
62.7680540204
<class 'numpy.float64'>
62.7505303696
<class 'numpy.float64'>
62.7330859599
<class 'numpy.float64'>
62.71572011
<class 'numpy.float64'>
62.698432147
<class 'numpy.float64'>
62.681221406
<class 'numpy.float64'>
62.6640872306
<class 'numpy.float64'>
62.647028972
<class 'numpy.float64'>
62.6300459894
<class 'numpy.float64'>
62.61313765
<class 'numpy.float64'>
62.5963033281
<class 'numpy.float64'>
62.5795424058
<class 'numpy.float64'>
62.5628542726
<class 'numpy.float64'>
62.5462383251
<class 'numpy.float64'>
62.5296939672
<class 'numpy.float64'>
62.5132206096
<class 'numpy.float64'>
62.4968176701
<class 'numpy.float64'>
62.4804845733
<class 'numpy.float64'>
62.4642207505
<class 'numpy.float64'>
62.4480256396
<class 'numpy.float64'>
62.4318986848
<class 'numpy.float64'>
62.4158393371
<class 'numpy.float64'>
62.3998470535
<class 'numpy.float64'>
62.3839212973
<class 'numpy.float64'>
62.368061538
<class 'numpy.float64'>
62.3522672509
<class 'numpy.float64'>
62.3365379176
<class 'numpy.float64'>
62.3208730253
<class 'numpy.float64'>
62.3052720669
<class 'numpy.float64'>
62.2897345412
<class 'numpy.float64'>
62.2742599523
<class 'numpy.float64'>
62.2588478103
<class 'numpy.float64'>
62.2434976301
<class 'numpy.float64'>
62.2282089325
<class 'numpy.float64'>
62.2129812432
<class 'numpy.float64'>
62.1978140934
<class 'numpy.float64'>
62.1827070192
<class 'numpy.float64'>
62.1676595618
<class 'numpy.float64'>
62.1526712675
<class 'numpy.float64'>
62.1377416873
<class 'numpy.float64'>
62.1228703773
<class 'numpy.float64'>
62.108056898
<class 'numpy.float64'>
62.093300815
<class 'numpy.float64'>
62.0786016984
<class 'numpy.float64'>
62.0639591226
<class 'numpy.float64'>
62.049372667
<class 'numpy.float64'>
62.0348419151
<class 'numpy.float64'>
62.0203664548
<class 'numpy.float64'>
62.0059458785
<class 'numpy.float64'>
61.9915797827
<class 'numpy.float64'>
61.9772677682
<class 'numpy.float64'>
61.9630094399
<class 'numpy.float64'>
61.9488044068
<class 'numpy.float64'>
61.9346522819
<class 'numpy.float64'>
61.9205526824
<class 'numpy.float64'>
61.9065052291
<class 'numpy.float64'>
61.8925095468
<class 'numpy.float64'>
61.8785652643
<class 'numpy.float64'>
61.864672014
<class 'numpy.float64'>
61.850829432
<class 'numpy.float64'>
61.8370371581
<class 'numpy.float64'>
61.8232948358
<class 'numpy.float64'>
61.8096021121
<class 'numpy.float64'>
61.7959586376
<class 'numpy.float64'>
61.7823640663
<class 'numpy.float64'>
61.7688180558
<class 'numpy.float64'>
61.7553202668
<class 'numpy.float64'>
61.7418703637
<class 'numpy.float64'>
61.7284680139
<class 'numpy.float64'>
61.7151128883
<class 'numpy.float64'>
61.7018046608
<class 'numpy.float64'>
61.6885430088
<class 'numpy.float64'>
61.6753276125
<class 'numpy.float64'>
61.6621581554
<class 'numpy.float64'>
61.649034324
<class 'numpy.float64'>
61.6359558079
<class 'numpy.float64'>
61.6229222995
<class 'numpy.float64'>
61.6099334943
<class 'numpy.float64'>
61.5969890908
<class 'numpy.float64'>
61.5840887901
<class 'numpy.float64'>
61.5712322964
<class 'numpy.float64'>
61.5584193166
<class 'numpy.float64'>
61.5456495603
<class 'numpy.float64'>
61.53292274
<class 'numpy.float64'>
61.5202385707
<class 'numpy.float64'>
61.5075967703
<class 'numpy.float64'>
61.4949970591
<class 'numpy.float64'>
61.4824391602
<class 'numpy.float64'>
61.4699227991
<class 'numpy.float64'>
61.4574477041
<class 'numpy.float64'>
61.4450136056
<class 'numpy.float64'>
61.432620237
<class 'numpy.float64'>
61.4202673337
<class 'numpy.float64'>
61.4079546337
<class 'numpy.float64'>
61.3956818774
<class 'numpy.float64'>
61.3834488077
<class 'numpy.float64'>
61.3712551695
<class 'numpy.float64'>
61.3591007103
<class 'numpy.float64'>
61.3469851798
<class 'numpy.float64'>
61.3349083299
<class 'numpy.float64'>
61.3228699147
<class 'numpy.float64'>
61.3108696908
<class 'numpy.float64'>
61.2989074165
<class 'numpy.float64'>
61.2869828526
<class 'numpy.float64'>
61.275095762
<class 'numpy.float64'>
61.2632459095
<class 'numpy.float64'>
61.2514330623
<class 'numpy.float64'>
61.2396569892
<class 'numpy.float64'>
61.2279174616
<class 'numpy.float64'>
61.2162142523
<class 'numpy.float64'>
61.2045471366
<class 'numpy.float64'>
61.1929158915
<class 'numpy.float64'>
61.181320296
<class 'numpy.float64'>
61.1697601309
<class 'numpy.float64'>
61.1582351792
<class 'numpy.float64'>
61.1467452254
<class 'numpy.float64'>
61.1352900562
<class 'numpy.float64'>
61.1238694599
<class 'numpy.float64'>
61.1124832267
<class 'numpy.float64'>
61.1011311485
<class 'numpy.float64'>
61.0898130191
<class 'numpy.float64'>
61.0785286341
<class 'numpy.float64'>
61.0672777907
<class 'numpy.float64'>
61.0560602878
<class 'numpy.float64'>
61.0448759261
<class 'numpy.float64'>
61.0337245079
<class 'numpy.float64'>
61.0226058372
<class 'numpy.float64'>
61.0115197197
<class 'numpy.float64'>
61.0004659626
<class 'numpy.float64'>
60.9894443748
<class 'numpy.float64'>
60.9784547668
<class 'numpy.float64'>
60.9674969505
<class 'numpy.float64'>
60.9565707396
<class 'numpy.float64'>
60.9456759493
<class 'numpy.float64'>
60.934812396
<class 'numpy.float64'>
60.9239798981
<class 'numpy.float64'>
60.9131782751
<class 'numpy.float64'>
60.9024073482
<class 'numpy.float64'>
60.8916669399
<class 'numpy.float64'>
60.8809568743
<class 'numpy.float64'>
60.8702769769
<class 'numpy.float64'>
60.8596270744
<class 'numpy.float64'>
60.8490069952
<class 'numpy.float64'>
60.838416569
<class 'numpy.float64'>
60.8278556266
<class 'numpy.float64'>
60.8173240006
<class 'numpy.float64'>
60.8068215246
<class 'numpy.float64'>
60.7963480337
<class 'numpy.float64'>
60.7859033642
<class 'numpy.float64'>
60.7754873538
<class 'numpy.float64'>
60.7650998415
<class 'numpy.float64'>
60.7547406674
<class 'numpy.float64'>
60.7444096732
<class 'numpy.float64'>
60.7341067014
<class 'numpy.float64'>
60.723831596
<class 'numpy.float64'>
60.7135842024
<class 'numpy.float64'>
60.7033643668
<class 'numpy.float64'>
60.6931719369
<class 'numpy.float64'>
60.6830067614
<class 'numpy.float64'>
60.6728686904
<class 'numpy.float64'>
60.6627575749
<class 'numpy.float64'>
60.6526732673
<class 'numpy.float64'>
60.6426156209
<class 'numpy.float64'>
60.6325844904
<class 'numpy.float64'>
60.6225797313
<class 'numpy.float64'>
60.6126012005
<class 'numpy.float64'>
60.6026487558
<class 'numpy.float64'>
60.5927222562
<class 'numpy.float64'>
60.5828215618
<class 'numpy.float64'>
60.5729465335
<class 'numpy.float64'>
60.5630970337
<class 'numpy.float64'>
60.5532729253
<class 'numpy.float64'>
60.5434740728
<class 'numpy.float64'>
60.5337003413
<class 'numpy.float64'>
60.5239515972
<class 'numpy.float64'>
60.5142277075
<class 'numpy.float64'>
60.5045285408
<class 'numpy.float64'>
60.494853966
<class 'numpy.float64'>
60.4852038536
<class 'numpy.float64'>
60.4755780747
<class 'numpy.float64'>
60.4659765014
<class 'numpy.float64'>
60.4563990068
<class 'numpy.float64'>
60.446845465
<class 'numpy.float64'>
60.4373157509
<class 'numpy.float64'>
60.4278097404
<class 'numpy.float64'>
60.4183273102
<class 'numpy.float64'>
60.4088683381
<class 'numpy.float64'>
60.3994327027
<class 'numpy.float64'>
60.3900202835
<class 'numpy.float64'>
60.3806309607
<class 'numpy.float64'>
60.3712646157
<class 'numpy.float64'>
60.3619211305
<class 'numpy.float64'>
60.352600388
<class 'numpy.float64'>
60.3433022722
<class 'numpy.float64'>
60.3340266674
<class 'numpy.float64'>
60.3247734594
<class 'numpy.float64'>
60.3155425342
<class 'numpy.float64'>
60.3063337791
<class 'numpy.float64'>
60.297147082
<class 'numpy.float64'>
60.2879823314
<class 'numpy.float64'>
60.278839417
<class 'numpy.float64'>
60.269718229
<class 'numpy.float64'>
60.2606186585
<class 'numpy.float64'>
60.2515405974
<class 'numpy.float64'>
60.2424839382
<class 'numpy.float64'>
60.2334485743
<class 'numpy.float64'>
60.2244343998
<class 'numpy.float64'>
60.2154413095
<class 'numpy.float64'>
60.2064691992
<class 'numpy.float64'>
60.197517965
<class 'numpy.float64'>
60.1885875041
<class 'numpy.float64'>
60.1796777142
<class 'numpy.float64'>
60.1707884939
<class 'numpy.float64'>
60.1619197422
<class 'numpy.float64'>
60.1530713591
<class 'numpy.float64'>
60.1442432452
<class 'numpy.float64'>
60.1354353017
<class 'numpy.float64'>
60.1266474307
<class 'numpy.float64'>
60.1178795346
<class 'numpy.float64'>
60.1091315169
<class 'numpy.float64'>
60.1004032814
<class 'numpy.float64'>
60.0916947329
<class 'numpy.float64'>
60.0830057765
<class 'numpy.float64'>
60.0743363182
<class 'numpy.float64'>
60.0656862645
<class 'numpy.float64'>
60.0570555227
<class 'numpy.float64'>
60.0484440006
<class 'numpy.float64'>
60.0398516065
<class 'numpy.float64'>
60.0312782496
<class 'numpy.float64'>
60.0227238396
<class 'numpy.float64'>
60.0141882868
<class 'numpy.float64'>
60.0056715019
<class 'numpy.float64'>
59.9971733966
<class 'numpy.float64'>
59.9886938829
<class 'numpy.float64'>
59.9802328735
<class 'numpy.float64'>
59.9717902816
<class 'numpy.float64'>
59.9633660212
<class 'numpy.float64'>
59.9549600064
<class 'numpy.float64'>
59.9465721525
<class 'numpy.float64'>
59.9382023748
<class 'numpy.float64'>
59.9298505896
<class 'numpy.float64'>
59.9215167133
<class 'numpy.float64'>
59.9132006634
<class 'numpy.float64'>
59.9049023574
<class 'numpy.float64'>
59.8966217137
<class 'numpy.float64'>
59.8883586511
<class 'numpy.float64'>
59.880113089
<class 'numpy.float64'>
59.8718849473
<class 'numpy.float64'>
59.8636741462
<class 'numpy.float64'>
59.8554806069
<class 'numpy.float64'>
59.8473042507
<class 'numpy.float64'>
59.8391449996
<class 'numpy.float64'>
59.831002776
<class 'numpy.float64'>
59.8228775029
<class 'numpy.float64'>
59.8147691037
<class 'numpy.float64'>
59.8066775024
<class 'numpy.float64'>
59.7986026235
<class 'numpy.float64'>
59.7905443918
<class 'numpy.float64'>
59.7825027327
<class 'numpy.float64'>
59.7744775722
<class 'numpy.float64'>
59.7664688367
<class 'numpy.float64'>
59.7584764528
<class 'numpy.float64'>
59.750500348
<class 'numpy.float64'>
59.74254045
<class 'numpy.float64'>
59.734596687
<class 'numpy.float64'>
59.7266689876
<class 'numpy.float64'>
59.7187572811
<class 'numpy.float64'>
59.710861497
<class 'numpy.float64'>
59.7029815652
<class 'numpy.float64'>
59.6951174164
<class 'numpy.float64'>
59.6872689812
<class 'numpy.float64'>
59.6794361912
<class 'numpy.float64'>
59.671618978
<class 'numpy.float64'>
59.6638172739
<class 'numpy.float64'>
59.6560310114
<class 'numpy.float64'>
59.6482601236
<class 'numpy.float64'>
59.6405045439
<class 'numpy.float64'>
59.6327642063
<class 'numpy.float64'>
59.6250390449
<class 'numpy.float64'>
59.6173289945
<class 'numpy.float64'>
59.6096339901
<class 'numpy.float64'>
59.6019539673
<class 'numpy.float64'>
59.5942888619
<class 'numpy.float64'>
59.5866386103
<class 'numpy.float64'>
59.579003149
<class 'numpy.float64'>
59.5713824152
<class 'numpy.float64'>
59.5637763463
<class 'numpy.float64'>
59.5561848802
<class 'numpy.float64'>
59.548607955
<class 'numpy.float64'>
59.5410455094
<class 'numpy.float64'>
59.5334974824
<class 'numpy.float64'>
59.5259638131
<class 'numpy.float64'>
59.5184444415
<class 'numpy.float64'>
59.5109393075
<class 'numpy.float64'>
59.5034483516
<class 'numpy.float64'>
59.4959715146
<class 'numpy.float64'>
59.4885087376
<class 'numpy.float64'>
59.4810599621
<class 'numpy.float64'>
59.4736251301
<class 'numpy.float64'>
59.4662041837
<class 'numpy.float64'>
59.4587970655
<class 'numpy.float64'>
59.4514037184
<class 'numpy.float64'>
59.4440240856
<class 'numpy.float64'>
59.4366581108
<class 'numpy.float64'>
59.4293057378
<class 'numpy.float64'>
59.4219669109
<class 'numpy.float64'>
59.4146415747
<class 'numpy.float64'>
59.4073296741
<class 'numpy.float64'>
59.4000311544
<class 'numpy.float64'>
59.3927459611
<class 'numpy.float64'>
59.3854740402
<class 'numpy.float64'>
59.3782153378
<class 'numpy.float64'>
59.3709698005
<class 'numpy.float64'>
59.3637373751
<class 'numpy.float64'>
59.3565180088
<class 'numpy.float64'>
59.349311649
<class 'numpy.float64'>
59.3421182436
<class 'numpy.float64'>
59.3349377406
<class 'numpy.float64'>
59.3277700884
<class 'numpy.float64'>
59.3206152356
<class 'numpy.float64'>
59.3134731314
<class 'numpy.float64'>
59.3063437249
<class 'numpy.float64'>
59.2992269658
<class 'numpy.float64'>
59.2921228039
<class 'numpy.float64'>
59.2850311893
<class 'numpy.float64'>
59.2779520727
<class 'numpy.float64'>
59.2708854046
<class 'numpy.float64'>
59.2638311361
<class 'numpy.float64'>
59.2567892185
<class 'numpy.float64'>
59.2497596035
<class 'numpy.float64'>
59.2427422428
<class 'numpy.float64'>
59.2357370886
<class 'numpy.float64'>
59.2287440934
<class 'numpy.float64'>
59.2217632098
<class 'numpy.float64'>
59.2147943908
<class 'numpy.float64'>
59.2078375896
<class 'numpy.float64'>
59.2008927598
<class 'numpy.float64'>
59.193959855
<class 'numpy.float64'>
59.1870388293
<class 'numpy.float64'>
59.1801296371
<class 'numpy.float64'>
59.1732322328
<class 'numpy.float64'>
59.1663465712
<class 'numpy.float64'>
59.1594726075
<class 'numpy.float64'>
59.1526102969
<class 'numpy.float64'>
59.145759595
<class 'numpy.float64'>
59.1389204576
<class 'numpy.float64'>
59.1320928408
<class 'numpy.float64'>
59.1252767009
<class 'numpy.float64'>
59.1184719944
<class 'numpy.float64'>
59.1116786782
<class 'numpy.float64'>
59.1048967093
<class 'numpy.float64'>
59.098126045
<class 'numpy.float64'>
59.0913666428
<class 'numpy.float64'>
59.0846184605
<class 'numpy.float64'>
59.0778814561
<class 'numpy.float64'>
59.0711555877
<class 'numpy.float64'>
59.0644408139
<class 'numpy.float64'>
59.0577370934
<class 'numpy.float64'>
59.051044385
<class 'numpy.float64'>
59.044362648
<class 'numpy.float64'>
59.0376918416
<class 'numpy.float64'>
59.0310319255
<class 'numpy.float64'>
59.0243828596
<class 'numpy.float64'>
59.0177446038
<class 'numpy.float64'>
59.0111171185
<class 'numpy.float64'>
59.004500364
<class 'numpy.float64'>
58.9978943012
<class 'numpy.float64'>
58.9912988908
<class 'numpy.float64'>
58.9847140941
<class 'numpy.float64'>
58.9781398724
<class 'numpy.float64'>
58.9715761872
<class 'numpy.float64'>
58.9650230003
<class 'numpy.float64'>
58.9584802737
<class 'numpy.float64'>
58.9519479694
<class 'numpy.float64'>
58.94542605
<class 'numpy.float64'>
58.938914478
<class 'numpy.float64'>
58.9324132161
<class 'numpy.float64'>
58.9259222274
<class 'numpy.float64'>
58.9194414751
<class 'numpy.float64'>
58.9129709225
<class 'numpy.float64'>
58.9065105332
<class 'numpy.float64'>
58.900060271
<class 'numpy.float64'>
58.8936200999
<class 'numpy.float64'>
58.887189984
<class 'numpy.float64'>
58.8807698877
<class 'numpy.float64'>
58.8743597756
<class 'numpy.float64'>
58.8679596123
<class 'numpy.float64'>
58.8615693629
<class 'numpy.float64'>
58.8551889923
<class 'numpy.float64'>
58.848818466
<class 'numpy.float64'>
58.8424577494
<class 'numpy.float64'>
58.8361068082
<class 'numpy.float64'>
58.8297656082
<class 'numpy.float64'>
58.8234341155
<class 'numpy.float64'>
58.8171122962
<class 'numpy.float64'>
58.8108001168
<class 'numpy.float64'>
58.8044975438
<class 'numpy.float64'>
58.7982045439
<class 'numpy.float64'>
58.7919210842
<class 'numpy.float64'>
58.7856471316
<class 'numpy.float64'>
58.7793826534
<class 'numpy.float64'>
58.7731276171
<class 'numpy.float64'>
58.7668819903
<class 'numpy.float64'>
58.7606457408
<class 'numpy.float64'>
58.7544188364
<class 'numpy.float64'>
58.7482012454
<class 'numpy.float64'>
58.7419929359
<class 'numpy.float64'>
58.7357938765
<class 'numpy.float64'>
58.7296040357
<class 'numpy.float64'>
58.7234233823
<class 'numpy.float64'>
58.7172518853
<class 'numpy.float64'>
58.7110895137
<class 'numpy.float64'>
58.7049362367
<class 'numpy.float64'>
58.6987920239
<class 'numpy.float64'>
58.6926568447
<class 'numpy.float64'>
58.6865306689
<class 'numpy.float64'>
58.6804134664
<class 'numpy.float64'>
58.6743052072
<class 'numpy.float64'>
58.6682058615
<class 'numpy.float64'>
58.6621153996
<class 'numpy.float64'>
58.6560337921
<class 'numpy.float64'>
58.6499610095
<class 'numpy.float64'>
58.6438970227
<class 'numpy.float64'>
58.6378418026
<class 'numpy.float64'>
58.6317953204
<class 'numpy.float64'>
58.6257575472
<class 'numpy.float64'>
58.6197284544
<class 'numpy.float64'>
58.6137080136
<class 'numpy.float64'>
58.6076961965
<class 'numpy.float64'>
58.6016929748
<class 'numpy.float64'>
58.5956983205
<class 'numpy.float64'>
58.5897122057
<class 'numpy.float64'>
58.5837346028
<class 'numpy.float64'>
58.5777654839
<class 'numpy.float64'>
58.5718048218
<class 'numpy.float64'>
58.5658525889
<class 'numpy.float64'>
58.5599087582
<class 'numpy.float64'>
58.5539733026
<class 'numpy.float64'>
58.5480461951
<class 'numpy.float64'>
58.5421274089
<class 'numpy.float64'>
58.5362169174
<class 'numpy.float64'>
58.5303146941
<class 'numpy.float64'>
58.5244207125
<class 'numpy.float64'>
58.5185349463
<class 'numpy.float64'>
58.5126573695
<class 'numpy.float64'>
58.506787956
<class 'numpy.float64'>
58.50092668
<class 'numpy.float64'>
58.4950735157
<class 'numpy.float64'>
58.4892284375
<class 'numpy.float64'>
58.4833914198
<class 'numpy.float64'>
58.4775624374
<class 'numpy.float64'>
58.4717414649
<class 'numpy.float64'>
58.4659284773
<class 'numpy.float64'>
58.4601234495
<class 'numpy.float64'>
58.4543263567
<class 'numpy.float64'>
58.4485371741
<class 'numpy.float64'>
58.4427558771
<class 'numpy.float64'>
58.4369824411
<class 'numpy.float64'>
58.4312168418
<class 'numpy.float64'>
58.425459055
<class 'numpy.float64'>
58.4197090563
<class 'numpy.float64'>
58.4139668219
<class 'numpy.float64'>
58.4082323277
<class 'numpy.float64'>
58.4025055501
<class 'numpy.float64'>
58.3967864652
<class 'numpy.float64'>
58.3910750495
<class 'numpy.float64'>
58.3853712795
<class 'numpy.float64'>
58.3796751319
<class 'numpy.float64'>
58.3739865835
<class 'numpy.float64'>
58.3683056111
<class 'numpy.float64'>
58.3626321917
<class 'numpy.float64'>
58.3569663024
<class 'numpy.float64'>
58.3513079204
<class 'numpy.float64'>
58.345657023
<class 'numpy.float64'>
58.3400135877
<class 'numpy.float64'>
58.3343775919
<class 'numpy.float64'>
58.3287490134
<class 'numpy.float64'>
58.3231278297
<class 'numpy.float64'>
58.3175140189
<class 'numpy.float64'>
58.3119075588
<class 'numpy.float64'>
58.3063084275
<class 'numpy.float64'>
58.3007166032
<class 'numpy.float64'>
58.2951320641
<class 'numpy.float64'>
58.2895547886
<class 'numpy.float64'>
58.2839847551
<class 'numpy.float64'>
58.2784219423
<class 'numpy.float64'>
58.2728663288
<class 'numpy.float64'>
58.2673178933
<class 'numpy.float64'>
58.2617766148
<class 'numpy.float64'>
58.2562424722
<class 'numpy.float64'>
58.2507154446
<class 'numpy.float64'>
58.2451955111
<class 'numpy.float64'>
58.239682651
<class 'numpy.float64'>
58.2341768436
<class 'numpy.float64'>
58.2286780685
<class 'numpy.float64'>
58.2231863051
<class 'numpy.float64'>
58.2177015331
<class 'numpy.float64'>
58.2122237323
<class 'numpy.float64'>
58.2067528824
<class 'numpy.float64'>
58.2012889634
<class 'numpy.float64'>
58.1958319553
<class 'numpy.float64'>
58.1903818383
<class 'numpy.float64'>
58.1849385924
<class 'numpy.float64'>
58.1795021981
<class 'numpy.float64'>
58.1740726356
<class 'numpy.float64'>
58.1686498855
<class 'numpy.float64'>
58.1632339283
<class 'numpy.float64'>
58.1578247446
<class 'numpy.float64'>
58.1524223153
<class 'numpy.float64'>
58.1470266211
<class 'numpy.float64'>
58.1416376429
<class 'numpy.float64'>
58.1362553617
<class 'numpy.float64'>
58.1308797586
<class 'numpy.float64'>
58.1255108148
<class 'numpy.float64'>
58.1201485116
<class 'numpy.float64'>
58.1147928302
<class 'numpy.float64'>
58.1094437521
<class 'numpy.float64'>
58.1041012588
<class 'numpy.float64'>
58.0987653319
<class 'numpy.float64'>
58.093435953
<class 'numpy.float64'>
58.0881131039
<class 'numpy.float64'>
58.0827967665
<class 'numpy.float64'>
58.0774869226
<class 'numpy.float64'>
58.0721835542
<class 'numpy.float64'>
58.0668866435
<class 'numpy.float64'>
58.0615961726
<class 'numpy.float64'>
58.0563121236
<class 'numpy.float64'>
58.051034479
<class 'numpy.float64'>
58.0457632211
<class 'numpy.float64'>
58.0404983324
<class 'numpy.float64'>
58.0352397954
<class 'numpy.float64'>
58.0299875927
<class 'numpy.float64'>
58.0247417071
<class 'numpy.float64'>
58.0195021214
<class 'numpy.float64'>
58.0142688182
<class 'numpy.float64'>
58.0090417807
<class 'numpy.float64'>
58.0038209918
<class 'numpy.float64'>
57.9986064345
<class 'numpy.float64'>
57.993398092
<class 'numpy.float64'>
57.9881959476
<class 'numpy.float64'>
57.9829999845
<class 'numpy.float64'>
57.9778101861
<class 'numpy.float64'>
57.9726265358
<class 'numpy.float64'>
57.9674490171
<class 'numpy.float64'>
57.9622776137
<class 'numpy.float64'>
57.9571123091
<class 'numpy.float64'>
57.9519530871
<class 'numpy.float64'>
57.9467999315
<class 'numpy.float64'>
57.9416528261
<class 'numpy.float64'>
57.9365117548
<class 'numpy.float64'>
57.9313767017
<class 'numpy.float64'>
57.9262476509
<class 'numpy.float64'>
57.9211245863
<class 'numpy.float64'>
57.9160074924
<class 'numpy.float64'>
57.9108963532
<class 'numpy.float64'>
57.9057911532
<class 'numpy.float64'>
57.9006918768
<class 'numpy.float64'>
57.8955985084
<class 'numpy.float64'>
57.8905110325
<class 'numpy.float64'>
57.8854294338
<class 'numpy.float64'>
57.8803536969
<class 'numpy.float64'>
57.8752838065
<class 'numpy.float64'>
57.8702197475
<class 'numpy.float64'>
57.8651615046
<class 'numpy.float64'>
57.8601090628
<class 'numpy.float64'>
57.8550624071
<class 'numpy.float64'>
57.8500215225
<class 'numpy.float64'>
57.8449863941
<class 'numpy.float64'>
57.8399570071
<class 'numpy.float64'>
57.8349333466
<class 'numpy.float64'>
57.8299153981
<class 'numpy.float64'>
57.8249031468
<class 'numpy.float64'>
57.8198965782
<class 'numpy.float64'>
57.8148956776
<class 'numpy.float64'>
57.8099004307
<class 'numpy.float64'>
57.8049108231
<class 'numpy.float64'>
57.7999268403
<class 'numpy.float64'>
57.7949484681
<class 'numpy.float64'>
57.7899756922
<class 'numpy.float64'>
57.7850084985
<class 'numpy.float64'>
57.7800468728
<class 'numpy.float64'>
57.7750908011
<class 'numpy.float64'>
57.7701402693
<class 'numpy.float64'>
57.7651952636
<class 'numpy.float64'>
57.76025577
<class 'numpy.float64'>
57.7553217747
<class 'numpy.float64'>
57.7503932638
<class 'numpy.float64'>
57.7454702237
<class 'numpy.float64'>
57.7405526408
<class 'numpy.float64'>
57.7356405012
<class 'numpy.float64'>
57.7307337916
<class 'numpy.float64'>
57.7258324984
<class 'numpy.float64'>
57.7209366082
<class 'numpy.float64'>
57.7160461074
<class 'numpy.float64'>
57.7111609829
<class 'numpy.float64'>
57.7062812212
<class 'numpy.float64'>
57.7014068092
<class 'numpy.float64'>
57.6965377336
<class 'numpy.float64'>
57.6916739814
<class 'numpy.float64'>
57.6868155393
<class 'numpy.float64'>
57.6819623944
<class 'numpy.float64'>
57.6771145337
<class 'numpy.float64'>
57.6722719443
<class 'numpy.float64'>
57.6674346132
<class 'numpy.float64'>
57.6626025277
<class 'numpy.float64'>
57.6577756749
<class 'numpy.float64'>
57.6529540421
<class 'numpy.float64'>
57.6481376166
<class 'numpy.float64'>
57.6433263858
<class 'numpy.float64'>
Cost after iteration 1000: 57.643326
57.6385203371
<class 'numpy.float64'>
57.6337194579
<class 'numpy.float64'>
57.6289237358
<class 'numpy.float64'>
57.6241331583
<class 'numpy.float64'>
57.619347713
<class 'numpy.float64'>
57.6145673876
<class 'numpy.float64'>
57.6097921697
<class 'numpy.float64'>
57.605022047
<class 'numpy.float64'>
57.6002570075
<class 'numpy.float64'>
57.5954970389
<class 'numpy.float64'>
57.5907421291
<class 'numpy.float64'>
57.585992266
<class 'numpy.float64'>
57.5812474376
<class 'numpy.float64'>
57.5765076319
<class 'numpy.float64'>
57.571772837
<class 'numpy.float64'>
57.567043041
<class 'numpy.float64'>
57.562318232
<class 'numpy.float64'>
57.5575983983
<class 'numpy.float64'>
57.552883528
<class 'numpy.float64'>
57.5481736095
<class 'numpy.float64'>
57.543468631
<class 'numpy.float64'>
57.538768581
<class 'numpy.float64'>
57.5340734479
<class 'numpy.float64'>
57.5293832201
<class 'numpy.float64'>
57.5246978861
<class 'numpy.float64'>
57.5200174344
<class 'numpy.float64'>
57.5153418537
<class 'numpy.float64'>
57.5106711326
<class 'numpy.float64'>
57.5060052596
<class 'numpy.float64'>
57.5013442237
<class 'numpy.float64'>
57.4966880133
<class 'numpy.float64'>
57.4920366175
<class 'numpy.float64'>
57.487390025
<class 'numpy.float64'>
57.4827482246
<class 'numpy.float64'>
57.4781112053
<class 'numpy.float64'>
57.473478956
<class 'numpy.float64'>
57.4688514657
<class 'numpy.float64'>
57.4642287235
<class 'numpy.float64'>
57.4596107183
<class 'numpy.float64'>
57.4549974394
<class 'numpy.float64'>
57.4503888759
<class 'numpy.float64'>
57.4457850168
<class 'numpy.float64'>
57.4411858516
<class 'numpy.float64'>
57.4365913693
<class 'numpy.float64'>
57.4320015594
<class 'numpy.float64'>
57.4274164112
<class 'numpy.float64'>
57.4228359139
<class 'numpy.float64'>
57.4182600571
<class 'numpy.float64'>
57.4136888302
<class 'numpy.float64'>
57.4091222226
<class 'numpy.float64'>
57.4045602239
<class 'numpy.float64'>
57.4000028237
<class 'numpy.float64'>
57.3954500115
<class 'numpy.float64'>
57.390901777
<class 'numpy.float64'>
57.3863581098
<class 'numpy.float64'>
57.3818189996
<class 'numpy.float64'>
57.3772844362
<class 'numpy.float64'>
57.3727544093
<class 'numpy.float64'>
57.3682289087
<class 'numpy.float64'>
57.3637079244
<class 'numpy.float64'>
57.359191446
<class 'numpy.float64'>
57.3546794637
<class 'numpy.float64'>
57.3501719672
<class 'numpy.float64'>
57.3456689466
<class 'numpy.float64'>
57.341170392
<class 'numpy.float64'>
57.3366762932
<class 'numpy.float64'>
57.3321866405
<class 'numpy.float64'>
57.3277014239
<class 'numpy.float64'>
57.3232206335
<class 'numpy.float64'>
57.3187442596
<class 'numpy.float64'>
57.3142722924
<class 'numpy.float64'>
57.309804722
<class 'numpy.float64'>
57.3053415388
<class 'numpy.float64'>
57.3008827331
<class 'numpy.float64'>
57.2964282952
<class 'numpy.float64'>
57.2919782155
<class 'numpy.float64'>
57.2875324843
<class 'numpy.float64'>
57.2830910922
<class 'numpy.float64'>
57.2786540295
<class 'numpy.float64'>
57.2742212869
<class 'numpy.float64'>
57.2697928547
<class 'numpy.float64'>
57.2653687235
<class 'numpy.float64'>
57.260948884
<class 'numpy.float64'>
57.2565333268
<class 'numpy.float64'>
57.2521220424
<class 'numpy.float64'>
57.2477150217
<class 'numpy.float64'>
57.2433122552
<class 'numpy.float64'>
57.2389137337
<class 'numpy.float64'>
57.234519448
<class 'numpy.float64'>
57.2301293888
<class 'numpy.float64'>
57.2257435471
<class 'numpy.float64'>
57.2213619136
<class 'numpy.float64'>
57.2169844793
<class 'numpy.float64'>
57.2126112349
<class 'numpy.float64'>
57.2082421716
<class 'numpy.float64'>
57.2038772801
<class 'numpy.float64'>
57.1995165516
<class 'numpy.float64'>
57.1951599771
<class 'numpy.float64'>
57.1908075476
<class 'numpy.float64'>
57.1864592541
<class 'numpy.float64'>
57.1821150878
<class 'numpy.float64'>
57.1777750398
<class 'numpy.float64'>
57.1734391012
<class 'numpy.float64'>
57.1691072633
<class 'numpy.float64'>
57.1647795173
<class 'numpy.float64'>
57.1604558544
<class 'numpy.float64'>
57.1561362658
<class 'numpy.float64'>
57.1518207428
<class 'numpy.float64'>
57.1475092768
<class 'numpy.float64'>
57.1432018592
<class 'numpy.float64'>
57.1388984811
<class 'numpy.float64'>
57.1345991342
<class 'numpy.float64'>
57.1303038097
<class 'numpy.float64'>
57.1260124991
<class 'numpy.float64'>
57.1217251939
<class 'numpy.float64'>
57.1174418856
<class 'numpy.float64'>
57.1131625657
<class 'numpy.float64'>
57.1088872257
<class 'numpy.float64'>
57.1046158572
<class 'numpy.float64'>
57.1003484518
<class 'numpy.float64'>
57.0960850011
<class 'numpy.float64'>
57.0918254968
<class 'numpy.float64'>
57.0875699304
<class 'numpy.float64'>
57.0833182938
<class 'numpy.float64'>
57.0790705785
<class 'numpy.float64'>
57.0748267763
<class 'numpy.float64'>
57.0705868791
<class 'numpy.float64'>
57.0663508785
<class 'numpy.float64'>
57.0621187663
<class 'numpy.float64'>
57.0578905344
<class 'numpy.float64'>
57.0536661747
<class 'numpy.float64'>
57.0494456789
<class 'numpy.float64'>
57.0452290391
<class 'numpy.float64'>
57.041016247
<class 'numpy.float64'>
57.0368072947
<class 'numpy.float64'>
57.032602174
<class 'numpy.float64'>
57.028400877
<class 'numpy.float64'>
57.0242033957
<class 'numpy.float64'>
57.020009722
<class 'numpy.float64'>
57.015819848
<class 'numpy.float64'>
57.0116337659
<class 'numpy.float64'>
57.0074514676
<class 'numpy.float64'>
57.0032729452
<class 'numpy.float64'>
56.9990981909
<class 'numpy.float64'>
56.9949271969
<class 'numpy.float64'>
56.9907599553
<class 'numpy.float64'>
56.9865964582
<class 'numpy.float64'>
56.9824366979
<class 'numpy.float64'>
56.9782806667
<class 'numpy.float64'>
56.9741283566
<class 'numpy.float64'>
56.9699797601
<class 'numpy.float64'>
56.9658348694
<class 'numpy.float64'>
56.9616936767
<class 'numpy.float64'>
56.9575561745
<class 'numpy.float64'>
56.953422355
<class 'numpy.float64'>
56.9492922106
<class 'numpy.float64'>
56.9451657337
<class 'numpy.float64'>
56.9410429166
<class 'numpy.float64'>
56.9369237519
<class 'numpy.float64'>
56.9328082318
<class 'numpy.float64'>
56.9286963489
<class 'numpy.float64'>
56.9245880957
<class 'numpy.float64'>
56.9204834645
<class 'numpy.float64'>
56.916382448
<class 'numpy.float64'>
56.9122850385
<class 'numpy.float64'>
56.9081912288
<class 'numpy.float64'>
56.9041010112
<class 'numpy.float64'>
56.9000143784
<class 'numpy.float64'>
56.895931323
<class 'numpy.float64'>
56.8918518375
<class 'numpy.float64'>
56.8877759147
<class 'numpy.float64'>
56.883703547
<class 'numpy.float64'>
56.8796347272
<class 'numpy.float64'>
56.8755694479
<class 'numpy.float64'>
56.8715077019
<class 'numpy.float64'>
56.8674494817
<class 'numpy.float64'>
56.8633947802
<class 'numpy.float64'>
56.85934359
<class 'numpy.float64'>
56.8552959039
<class 'numpy.float64'>
56.8512517147
<class 'numpy.float64'>
56.8472110151
<class 'numpy.float64'>
56.8431737978
<class 'numpy.float64'>
56.8391400558
<class 'numpy.float64'>
56.8351097819
<class 'numpy.float64'>
56.8310829687
<class 'numpy.float64'>
56.8270596093
<class 'numpy.float64'>
56.8230396964
<class 'numpy.float64'>
56.819023223
<class 'numpy.float64'>
56.8150101819
<class 'numpy.float64'>
56.8110005659
<class 'numpy.float64'>
56.8069943681
<class 'numpy.float64'>
56.8029915814
<class 'numpy.float64'>
56.7989921986
<class 'numpy.float64'>
56.7949962128
<class 'numpy.float64'>
56.7910036168
<class 'numpy.float64'>
56.7870144038
<class 'numpy.float64'>
56.7830285666
<class 'numpy.float64'>
56.7790460982
<class 'numpy.float64'>
56.7750669917
<class 'numpy.float64'>
56.7710912401
<class 'numpy.float64'>
56.7671188365
<class 'numpy.float64'>
56.7631497738
<class 'numpy.float64'>
56.7591840451
<class 'numpy.float64'>
56.7552216436
<class 'numpy.float64'>
56.7512625622
<class 'numpy.float64'>
56.7473067941
<class 'numpy.float64'>
56.7433543324
<class 'numpy.float64'>
56.7394051701
<class 'numpy.float64'>
56.7354593005
<class 'numpy.float64'>
56.7315167166
<class 'numpy.float64'>
56.7275774115
<class 'numpy.float64'>
56.7236413785
<class 'numpy.float64'>
56.7197086107
<class 'numpy.float64'>
56.7157791012
<class 'numpy.float64'>
56.7118528433
<class 'numpy.float64'>
56.7079298301
<class 'numpy.float64'>
56.7040100548
<class 'numpy.float64'>
56.7000935107
<class 'numpy.float64'>
56.6961801909
<class 'numpy.float64'>
56.6922700887
<class 'numpy.float64'>
56.6883631973
<class 'numpy.float64'>
56.6844595099
<class 'numpy.float64'>
56.6805590198
<class 'numpy.float64'>
56.6766617202
<class 'numpy.float64'>
56.6727676045
<class 'numpy.float64'>
56.6688766658
<class 'numpy.float64'>
56.6649888975
<class 'numpy.float64'>
56.6611042928
<class 'numpy.float64'>
56.657222845
<class 'numpy.float64'>
56.6533445475
<class 'numpy.float64'>
56.6494693935
<class 'numpy.float64'>
56.6455973763
<class 'numpy.float64'>
56.6417284893
<class 'numpy.float64'>
56.6378627258
<class 'numpy.float64'>
56.634000079
<class 'numpy.float64'>
56.6301405425
<class 'numpy.float64'>
56.6262841093
<class 'numpy.float64'>
56.622430773
<class 'numpy.float64'>
56.6185805269
<class 'numpy.float64'>
56.6147333643
<class 'numpy.float64'>
56.6108892785
<class 'numpy.float64'>
56.607048263
<class 'numpy.float64'>
56.6032103111
<class 'numpy.float64'>
56.5993754162
<class 'numpy.float64'>
56.5955435717
<class 'numpy.float64'>
56.5917147708
<class 'numpy.float64'>
56.5878890071
<class 'numpy.float64'>
56.5840662739
<class 'numpy.float64'>
56.5802465645
<class 'numpy.float64'>
56.5764298725
<class 'numpy.float64'>
56.572616191
<class 'numpy.float64'>
56.5688055137
<class 'numpy.float64'>
56.5649978338
<class 'numpy.float64'>
56.5611931447
<class 'numpy.float64'>
56.5573914399
<class 'numpy.float64'>
56.5535927128
<class 'numpy.float64'>
56.5497969567
<class 'numpy.float64'>
56.5460041651
<class 'numpy.float64'>
56.5422143314
<class 'numpy.float64'>
56.5384274489
<class 'numpy.float64'>
56.5346435112
<class 'numpy.float64'>
56.5308625115
<class 'numpy.float64'>
56.5270844433
<class 'numpy.float64'>
56.5233093
<class 'numpy.float64'>
56.5195370751
<class 'numpy.float64'>
56.5157677618
<class 'numpy.float64'>
56.5120013537
<class 'numpy.float64'>
56.5082378441
<class 'numpy.float64'>
56.5044772265
<class 'numpy.float64'>
56.5007194942
<class 'numpy.float64'>
56.4969646406
<class 'numpy.float64'>
56.4932126592
<class 'numpy.float64'>
56.4894635433
<class 'numpy.float64'>
56.4857172863
<class 'numpy.float64'>
56.4819738816
<class 'numpy.float64'>
56.4782333227
<class 'numpy.float64'>
56.4744956028
<class 'numpy.float64'>
56.4707607154
<class 'numpy.float64'>
56.4670286538
<class 'numpy.float64'>
56.4632994115
<class 'numpy.float64'>
56.4595729818
<class 'numpy.float64'>
56.455849358
<class 'numpy.float64'>
56.4521285336
<class 'numpy.float64'>
56.4484105018
<class 'numpy.float64'>
56.4446952561
<class 'numpy.float64'>
56.4409827898
<class 'numpy.float64'>
56.4372730962
<class 'numpy.float64'>
56.4335661686
<class 'numpy.float64'>
56.4298620004
<class 'numpy.float64'>
56.426160585
<class 'numpy.float64'>
56.4224619156
<class 'numpy.float64'>
56.4187659855
<class 'numpy.float64'>
56.4150727881
<class 'numpy.float64'>
56.4113823166
<class 'numpy.float64'>
56.4076945643
<class 'numpy.float64'>
56.4040095246
<class 'numpy.float64'>
56.4003271906
<class 'numpy.float64'>
56.3966475557
<class 'numpy.float64'>
56.3929706131
<class 'numpy.float64'>
56.3892963561
<class 'numpy.float64'>
56.3856247778
<class 'numpy.float64'>
56.3819558716
<class 'numpy.float64'>
56.3782896306
<class 'numpy.float64'>
56.3746260481
<class 'numpy.float64'>
56.3709651172
<class 'numpy.float64'>
56.3673068312
<class 'numpy.float64'>
56.3636511832
<class 'numpy.float64'>
56.3599981664
<class 'numpy.float64'>
56.3563477739
<class 'numpy.float64'>
56.3526999989
<class 'numpy.float64'>
56.3490548345
<class 'numpy.float64'>
56.3454122739
<class 'numpy.float64'>
56.3417723101
<class 'numpy.float64'>
56.3381349363
<class 'numpy.float64'>
56.3345001455
<class 'numpy.float64'>
56.3308679308
<class 'numpy.float64'>
56.3272382852
<class 'numpy.float64'>
56.3236112019
<class 'numpy.float64'>
56.3199866737
<class 'numpy.float64'>
56.3163646938
<class 'numpy.float64'>
56.3127452552
<class 'numpy.float64'>
56.3091283507
<class 'numpy.float64'>
56.3055139734
<class 'numpy.float64'>
56.3019021163
<class 'numpy.float64'>
56.2982927722
<class 'numpy.float64'>
56.2946859341
<class 'numpy.float64'>
56.2910815949
<class 'numpy.float64'>
56.2874797475
<class 'numpy.float64'>
56.2838803847
<class 'numpy.float64'>
56.2802834994
<class 'numpy.float64'>
56.2766890844
<class 'numpy.float64'>
56.2730971326
<class 'numpy.float64'>
56.2695076366
<class 'numpy.float64'>
56.2659205894
<class 'numpy.float64'>
56.2623359837
<class 'numpy.float64'>
56.2587538121
<class 'numpy.float64'>
56.2551740675
<class 'numpy.float64'>
56.2515967425
<class 'numpy.float64'>
56.2480218298
<class 'numpy.float64'>
56.244449322
<class 'numpy.float64'>
56.2408792118
<class 'numpy.float64'>
56.2373114917
<class 'numpy.float64'>
56.2337461545
<class 'numpy.float64'>
56.2301831926
<class 'numpy.float64'>
56.2266225985
<class 'numpy.float64'>
56.2230643649
<class 'numpy.float64'>
56.2195084842
<class 'numpy.float64'>
56.2159549488
<class 'numpy.float64'>
56.2124037512
<class 'numpy.float64'>
56.2088548839
<class 'numpy.float64'>
56.2053083393
<class 'numpy.float64'>
56.2017641096
<class 'numpy.float64'>
56.1982221873
<class 'numpy.float64'>
56.1946825647
<class 'numpy.float64'>
56.1911452341
<class 'numpy.float64'>
56.1876101877
<class 'numpy.float64'>
56.1840774178
<class 'numpy.float64'>
56.1805469165
<class 'numpy.float64'>
56.1770186762
<class 'numpy.float64'>
56.1734926888
<class 'numpy.float64'>
56.1699689466
<class 'numpy.float64'>
56.1664474417
<class 'numpy.float64'>
56.162928166
<class 'numpy.float64'>
56.1594111118
<class 'numpy.float64'>
56.1558962708
<class 'numpy.float64'>
56.1523836352
<class 'numpy.float64'>
56.1488731969
<class 'numpy.float64'>
56.1453649477
<class 'numpy.float64'>
56.1418588796
<class 'numpy.float64'>
56.1383549843
<class 'numpy.float64'>
56.1348532538
<class 'numpy.float64'>
56.1313536798
<class 'numpy.float64'>
56.127856254
<class 'numpy.float64'>
56.1243609681
<class 'numpy.float64'>
56.1208678138
<class 'numpy.float64'>
56.1173767828
<class 'numpy.float64'>
56.1138878666
<class 'numpy.float64'>
56.1104010568
<class 'numpy.float64'>
56.106916345
<class 'numpy.float64'>
56.1034337226
<class 'numpy.float64'>
56.0999531811
<class 'numpy.float64'>
56.0964747118
<class 'numpy.float64'>
56.0929983062
<class 'numpy.float64'>
56.0895239557
<class 'numpy.float64'>
56.0860516514
<class 'numpy.float64'>
56.0825813847
<class 'numpy.float64'>
56.0791131467
<class 'numpy.float64'>
56.0756469287
<class 'numpy.float64'>
56.0721827218
<class 'numpy.float64'>
56.068720517
<class 'numpy.float64'>
56.0652603054
<class 'numpy.float64'>
56.061802078
<class 'numpy.float64'>
56.0583458258
<class 'numpy.float64'>
56.0548915397
<class 'numpy.float64'>
56.0514392105
<class 'numpy.float64'>
56.0479888291
<class 'numpy.float64'>
56.0445403862
<class 'numpy.float64'>
56.0410938726
<class 'numpy.float64'>
56.037649279
<class 'numpy.float64'>
56.0342065959
<class 'numpy.float64'>
56.0307658141
<class 'numpy.float64'>
56.0273269239
<class 'numpy.float64'>
56.023889916
<class 'numpy.float64'>
56.0204547808
<class 'numpy.float64'>
56.0170215085
<class 'numpy.float64'>
56.0135900897
<class 'numpy.float64'>
56.0101605145
<class 'numpy.float64'>
56.0067327732
<class 'numpy.float64'>
56.003306856
<class 'numpy.float64'>
55.9998827529
<class 'numpy.float64'>
55.9964604542
<class 'numpy.float64'>
55.9930399498
<class 'numpy.float64'>
55.9896212296
<class 'numpy.float64'>
55.9862042835
<class 'numpy.float64'>
55.9827891015
<class 'numpy.float64'>
55.9793756733
<class 'numpy.float64'>
55.9759639887
<class 'numpy.float64'>
55.9725540372
<class 'numpy.float64'>
55.9691458087
<class 'numpy.float64'>
55.9657392925
<class 'numpy.float64'>
55.9623344782
<class 'numpy.float64'>
55.9589313552
<class 'numpy.float64'>
55.9555299129
<class 'numpy.float64'>
55.9521301407
<class 'numpy.float64'>
55.9487320277
<class 'numpy.float64'>
55.9453355632
<class 'numpy.float64'>
55.9419407363
<class 'numpy.float64'>
55.938547536
<class 'numpy.float64'>
55.9351559514
<class 'numpy.float64'>
55.9317659712
<class 'numpy.float64'>
55.9283775846
<class 'numpy.float64'>
55.9249907801
<class 'numpy.float64'>
55.9216055465
<class 'numpy.float64'>
55.9182218725
<class 'numpy.float64'>
55.9148397466
<class 'numpy.float64'>
55.9114591575
<class 'numpy.float64'>
55.9080800934
<class 'numpy.float64'>
55.9047025428
<class 'numpy.float64'>
55.9013264939
<class 'numpy.float64'>
55.8979519351
<class 'numpy.float64'>
55.8945788544
<class 'numpy.float64'>
55.8912072399
<class 'numpy.float64'>
55.8878370795
<class 'numpy.float64'>
55.8844683613
<class 'numpy.float64'>
55.8811010731
<class 'numpy.float64'>
55.8777352025
<class 'numpy.float64'>
55.8743707374
<class 'numpy.float64'>
55.8710076652
<class 'numpy.float64'>
55.8676459736
<class 'numpy.float64'>
55.8642856499
<class 'numpy.float64'>
55.8609266815
<class 'numpy.float64'>
55.8575690557
<class 'numpy.float64'>
55.8542127597
<class 'numpy.float64'>
55.8508577806
<class 'numpy.float64'>
55.8475041054
<class 'numpy.float64'>
55.8441517209
<class 'numpy.float64'>
55.8408006142
<class 'numpy.float64'>
55.8374507719
<class 'numpy.float64'>
55.8341021806
<class 'numpy.float64'>
55.830754827
<class 'numpy.float64'>
55.8274086976
<class 'numpy.float64'>
55.8240637787
<class 'numpy.float64'>
55.8207200566
<class 'numpy.float64'>
55.8173775176
<class 'numpy.float64'>
55.8140361477
<class 'numpy.float64'>
55.810695933
<class 'numpy.float64'>
55.8073568593
<class 'numpy.float64'>
55.8040189125
<class 'numpy.float64'>
55.8006820783
<class 'numpy.float64'>
55.7973463424
<class 'numpy.float64'>
55.7940116902
<class 'numpy.float64'>
55.7906781071
<class 'numpy.float64'>
55.7873455786
<class 'numpy.float64'>
55.7840140897
<class 'numpy.float64'>
55.7806836256
<class 'numpy.float64'>
55.7773541713
<class 'numpy.float64'>
55.7740257116
<class 'numpy.float64'>
55.7706982315
<class 'numpy.float64'>
55.7673717154
<class 'numpy.float64'>
55.7640461481
<class 'numpy.float64'>
55.760721514
<class 'numpy.float64'>
55.7573977973
<class 'numpy.float64'>
55.7540749824
<class 'numpy.float64'>
55.7507530533
<class 'numpy.float64'>
55.7474319941
<class 'numpy.float64'>
55.7441117886
<class 'numpy.float64'>
55.7407924205
<class 'numpy.float64'>
55.7374738737
<class 'numpy.float64'>
55.7341561315
<class 'numpy.float64'>
55.7308391773
<class 'numpy.float64'>
55.7275229946
<class 'numpy.float64'>
55.7242075663
<class 'numpy.float64'>
55.7208928756
<class 'numpy.float64'>
55.7175789053
<class 'numpy.float64'>
55.7142656383
<class 'numpy.float64'>
55.7109530572
<class 'numpy.float64'>
55.7076411445
<class 'numpy.float64'>
55.7043298827
<class 'numpy.float64'>
55.7010192539
<class 'numpy.float64'>
55.6977092403
<class 'numpy.float64'>
55.6943998239
<class 'numpy.float64'>
55.6910909866
<class 'numpy.float64'>
55.6877827101
<class 'numpy.float64'>
55.684474976
<class 'numpy.float64'>
55.6811677657
<class 'numpy.float64'>
55.6778610605
<class 'numpy.float64'>
55.6745548416
<class 'numpy.float64'>
55.67124909
<class 'numpy.float64'>
55.6679437866
<class 'numpy.float64'>
55.6646389122
<class 'numpy.float64'>
55.6613344472
<class 'numpy.float64'>
55.6580303721
<class 'numpy.float64'>
55.6547266673
<class 'numpy.float64'>
55.6514233128
<class 'numpy.float64'>
55.6481202887
<class 'numpy.float64'>
55.6448175747
<class 'numpy.float64'>
55.6415151505
<class 'numpy.float64'>
55.6382129957
<class 'numpy.float64'>
55.6349110896
<class 'numpy.float64'>
55.6316094113
<class 'numpy.float64'>
55.62830794
<class 'numpy.float64'>
55.6250066544
<class 'numpy.float64'>
55.6217055334
<class 'numpy.float64'>
55.6184045554
<class 'numpy.float64'>
55.6151036988
<class 'numpy.float64'>
55.6118029418
<class 'numpy.float64'>
55.6085022625
<class 'numpy.float64'>
55.6052016387
<class 'numpy.float64'>
55.6019010481
<class 'numpy.float64'>
55.5986004683
<class 'numpy.float64'>
55.5952998764
<class 'numpy.float64'>
55.5919992499
<class 'numpy.float64'>
55.5886985655
<class 'numpy.float64'>
55.5853978001
<class 'numpy.float64'>
55.5820969303
<class 'numpy.float64'>
55.5787959326
<class 'numpy.float64'>
55.5754947832
<class 'numpy.float64'>
55.5721934582
<class 'numpy.float64'>
55.5688919334
<class 'numpy.float64'>
55.5655901845
<class 'numpy.float64'>
55.562288187
<class 'numpy.float64'>
55.5589859162
<class 'numpy.float64'>
55.5556833472
<class 'numpy.float64'>
55.5523804549
<class 'numpy.float64'>
55.549077214
<class 'numpy.float64'>
55.545773599
<class 'numpy.float64'>
55.5424695842
<class 'numpy.float64'>
55.5391651437
<class 'numpy.float64'>
55.5358602514
<class 'numpy.float64'>
55.532554881
<class 'numpy.float64'>
55.5292490059
<class 'numpy.float64'>
55.5259425994
<class 'numpy.float64'>
55.5226356345
<class 'numpy.float64'>
55.5193280842
<class 'numpy.float64'>
55.5160199209
<class 'numpy.float64'>
55.5127111171
<class 'numpy.float64'>
55.509401645
<class 'numpy.float64'>
55.5060914765
<class 'numpy.float64'>
55.5027805834
<class 'numpy.float64'>
55.4994689372
<class 'numpy.float64'>
55.4961565091
<class 'numpy.float64'>
55.4928432702
<class 'numpy.float64'>
55.4895291914
<class 'numpy.float64'>
55.4862142431
<class 'numpy.float64'>
55.4828983959
<class 'numpy.float64'>
55.4795816197
<class 'numpy.float64'>
55.4762638844
<class 'numpy.float64'>
55.4729451598
<class 'numpy.float64'>
55.4696254152
<class 'numpy.float64'>
55.4663046197
<class 'numpy.float64'>
55.4629827423
<class 'numpy.float64'>
55.4596597516
<class 'numpy.float64'>
55.456335616
<class 'numpy.float64'>
55.4530103036
<class 'numpy.float64'>
55.4496837825
<class 'numpy.float64'>
55.4463560202
<class 'numpy.float64'>
55.4430269841
<class 'numpy.float64'>
55.4396966413
<class 'numpy.float64'>
55.4363649588
<class 'numpy.float64'>
55.4330319031
<class 'numpy.float64'>
55.4296974405
<class 'numpy.float64'>
55.4263615372
<class 'numpy.float64'>
55.4230241589
<class 'numpy.float64'>
55.4196852713
<class 'numpy.float64'>
55.4163448394
<class 'numpy.float64'>
55.4130028284
<class 'numpy.float64'>
55.4096592029
<class 'numpy.float64'>
55.4063139274
<class 'numpy.float64'>
55.4029669661
<class 'numpy.float64'>
55.3996182827
<class 'numpy.float64'>
55.3962678409
<class 'numpy.float64'>
55.3929156039
<class 'numpy.float64'>
55.3895615349
<class 'numpy.float64'>
55.3862055964
<class 'numpy.float64'>
55.3828477509
<class 'numpy.float64'>
55.3794879607
<class 'numpy.float64'>
55.3761261874
<class 'numpy.float64'>
55.3727623926
<class 'numpy.float64'>
55.3693965376
<class 'numpy.float64'>
55.3660285833
<class 'numpy.float64'>
55.3626584904
<class 'numpy.float64'>
55.3592862191
<class 'numpy.float64'>
55.3559117295
<class 'numpy.float64'>
55.3525349813
<class 'numpy.float64'>
55.3491559339
<class 'numpy.float64'>
55.3457745464
<class 'numpy.float64'>
55.3423907775
<class 'numpy.float64'>
55.3390045857
<class 'numpy.float64'>
55.3356159292
<class 'numpy.float64'>
55.3322247657
<class 'numpy.float64'>
55.3288310527
<class 'numpy.float64'>
55.3254347475
<class 'numpy.float64'>
55.3220358069
<class 'numpy.float64'>
55.3186341873
<class 'numpy.float64'>
55.315229845
<class 'numpy.float64'>
55.3118227358
<class 'numpy.float64'>
55.3084128152
<class 'numpy.float64'>
55.3050000384
<class 'numpy.float64'>
55.3015843604
<class 'numpy.float64'>
55.2981657355
<class 'numpy.float64'>
55.294744118
<class 'numpy.float64'>
55.2913194617
<class 'numpy.float64'>
55.2878917201
<class 'numpy.float64'>
55.2844608463
<class 'numpy.float64'>
55.2810267932
<class 'numpy.float64'>
55.2775895132
<class 'numpy.float64'>
55.2741489584
<class 'numpy.float64'>
55.2707050806
<class 'numpy.float64'>
55.2672578311
<class 'numpy.float64'>
55.263807161
<class 'numpy.float64'>
55.2603530211
<class 'numpy.float64'>
55.2568953617
<class 'numpy.float64'>
55.2534341326
<class 'numpy.float64'>
55.2499692837
<class 'numpy.float64'>
55.2465007641
<class 'numpy.float64'>
55.2430285228
<class 'numpy.float64'>
55.2395525083
<class 'numpy.float64'>
55.2360726688
<class 'numpy.float64'>
55.2325889521
<class 'numpy.float64'>
55.2291013057
<class 'numpy.float64'>
55.2256096767
<class 'numpy.float64'>
55.2221140117
<class 'numpy.float64'>
55.2186142572
<class 'numpy.float64'>
55.2151103592
<class 'numpy.float64'>
55.2116022632
<class 'numpy.float64'>
55.2080899145
<class 'numpy.float64'>
55.204573258
<class 'numpy.float64'>
55.2010522382
<class 'numpy.float64'>
55.1975267993
<class 'numpy.float64'>
55.1939968849
<class 'numpy.float64'>
55.1904624386
<class 'numpy.float64'>
55.1869234033
<class 'numpy.float64'>
55.1833797217
<class 'numpy.float64'>
55.179831336
<class 'numpy.float64'>
55.1762781881
<class 'numpy.float64'>
55.1727202197
<class 'numpy.float64'>
55.1691573717
<class 'numpy.float64'>
55.1655895851
<class 'numpy.float64'>
55.1620168001
<class 'numpy.float64'>
55.1584389569
<class 'numpy.float64'>
55.154855995
<class 'numpy.float64'>
55.1512678539
<class 'numpy.float64'>
55.1476744723
<class 'numpy.float64'>
55.1440757888
<class 'numpy.float64'>
55.1404717416
<class 'numpy.float64'>
55.1368622684
<class 'numpy.float64'>
55.1332473067
<class 'numpy.float64'>
55.1296267936
<class 'numpy.float64'>
55.1260006656
<class 'numpy.float64'>
55.1223688592
<class 'numpy.float64'>
55.1187313102
<class 'numpy.float64'>
55.1150879542
<class 'numpy.float64'>
55.1114387264
<class 'numpy.float64'>
55.1077835617
<class 'numpy.float64'>
55.1041223945
<class 'numpy.float64'>
55.100455159
<class 'numpy.float64'>
55.0967817889
<class 'numpy.float64'>
55.0931022175
<class 'numpy.float64'>
55.089416378
<class 'numpy.float64'>
55.085724203
<class 'numpy.float64'>
55.0820256247
<class 'numpy.float64'>
55.0783205753
<class 'numpy.float64'>
55.0746089862
<class 'numpy.float64'>
55.0708907888
<class 'numpy.float64'>
55.0671659139
<class 'numpy.float64'>
55.0634342922
<class 'numpy.float64'>
55.0596958538
<class 'numpy.float64'>
55.0559505287
<class 'numpy.float64'>
55.0521982464
<class 'numpy.float64'>
55.048438936
<class 'numpy.float64'>
55.0446725266
<class 'numpy.float64'>
55.0408989465
<class 'numpy.float64'>
55.0371181241
<class 'numpy.float64'>
55.0333299872
<class 'numpy.float64'>
55.0295344634
<class 'numpy.float64'>
55.02573148
<class 'numpy.float64'>
55.0219209637
<class 'numpy.float64'>
55.0181028414
<class 'numpy.float64'>
55.0142770392
<class 'numpy.float64'>
55.0104434831
<class 'numpy.float64'>
55.0066020989
<class 'numpy.float64'>
55.0027528119
<class 'numpy.float64'>
54.9988955472
<class 'numpy.float64'>
54.9950302297
<class 'numpy.float64'>
54.9911567837
<class 'numpy.float64'>
54.9872751335
<class 'numpy.float64'>
54.983385203
<class 'numpy.float64'>
54.9794869159
<class 'numpy.float64'>
54.9755801955
<class 'numpy.float64'>
54.9716649649
<class 'numpy.float64'>
54.9677411469
<class 'numpy.float64'>
54.9638086641
<class 'numpy.float64'>
54.9598674387
<class 'numpy.float64'>
54.9559173927
<class 'numpy.float64'>
54.951958448
<class 'numpy.float64'>
54.9479905259
<class 'numpy.float64'>
54.9440135479
<class 'numpy.float64'>
54.9400274347
<class 'numpy.float64'>
54.9360321073
<class 'numpy.float64'>
54.9320274862
<class 'numpy.float64'>
54.9280134915
<class 'numpy.float64'>
54.9239900434
<class 'numpy.float64'>
54.9199570616
<class 'numpy.float64'>
54.9159144657
<class 'numpy.float64'>
54.9118621751
<class 'numpy.float64'>
54.9078001088
<class 'numpy.float64'>
54.9037281858
<class 'numpy.float64'>
54.8996463247
<class 'numpy.float64'>
54.895554444
<class 'numpy.float64'>
54.8914524619
<class 'numpy.float64'>
54.8873402965
<class 'numpy.float64'>
54.8832178654
<class 'numpy.float64'>
54.8790850863
<class 'numpy.float64'>
54.8749418765
<class 'numpy.float64'>
54.8707881532
<class 'numpy.float64'>
54.8666238333
<class 'numpy.float64'>
54.8624488336
<class 'numpy.float64'>
54.8582630704
<class 'numpy.float64'>
54.8540664601
<class 'numpy.float64'>
54.8498589187
<class 'numpy.float64'>
54.8456403621
<class 'numpy.float64'>
54.8414107057
<class 'numpy.float64'>
54.8371698651
<class 'numpy.float64'>
54.8329177552
<class 'numpy.float64'>
54.828654291
<class 'numpy.float64'>
54.8243793871
<class 'numpy.float64'>
54.820092958
<class 'numpy.float64'>
54.8157949176
<class 'numpy.float64'>
54.8114851798
<class 'numpy.float64'>
54.8071636582
<class 'numpy.float64'>
54.8028302662
<class 'numpy.float64'>
54.7984849165
<class 'numpy.float64'>
54.794127522
<class 'numpy.float64'>
54.7897579949
<class 'numpy.float64'>
54.7853762473
<class 'numpy.float64'>
54.7809821908
<class 'numpy.float64'>
54.7765757366
<class 'numpy.float64'>
54.7721567956
<class 'numpy.float64'>
54.7677252784
<class 'numpy.float64'>
54.7632810949
<class 'numpy.float64'>
54.7588241548
<class 'numpy.float64'>
54.754354367
<class 'numpy.float64'>
54.7498716404
<class 'numpy.float64'>
54.7453758829
<class 'numpy.float64'>
54.740867002
<class 'numpy.float64'>
54.7363449047
<class 'numpy.float64'>
54.7318094972
<class 'numpy.float64'>
54.7272606853
<class 'numpy.float64'>
54.7226983739
<class 'numpy.float64'>
54.718122467
<class 'numpy.float64'>
54.7135328683
<class 'numpy.float64'>
54.7089294802
<class 'numpy.float64'>
54.7043122045
<class 'numpy.float64'>
54.6996809419
<class 'numpy.float64'>
54.6950355923
<class 'numpy.float64'>
54.6903760544
<class 'numpy.float64'>
54.6857022258
<class 'numpy.float64'>
54.6810140032
<class 'numpy.float64'>
54.6763112817
<class 'numpy.float64'>
54.6715939553
<class 'numpy.float64'>
54.6668619165
<class 'numpy.float64'>
54.6621150566
<class 'numpy.float64'>
54.657353265
<class 'numpy.float64'>
54.6525764297
<class 'numpy.float64'>
54.6477844369
<class 'numpy.float64'>
54.6429771711
<class 'numpy.float64'>
54.6381545146
<class 'numpy.float64'>
54.6333163479
<class 'numpy.float64'>
54.6284625492
<class 'numpy.float64'>
54.6235929946
<class 'numpy.float64'>
54.6187075576
<class 'numpy.float64'>
54.6138061093
<class 'numpy.float64'>
54.608888518
<class 'numpy.float64'>
54.6039546494
<class 'numpy.float64'>
54.599004366
<class 'numpy.float64'>
54.5940375272
<class 'numpy.float64'>
54.5890539891
<class 'numpy.float64'>
54.5840536045
<class 'numpy.float64'>
54.5790362221
<class 'numpy.float64'>
54.5740016872
<class 'numpy.float64'>
54.5689498406
<class 'numpy.float64'>
54.5638805189
<class 'numpy.float64'>
54.5587935545
<class 'numpy.float64'>
54.5536887745
<class 'numpy.float64'>
54.5485660012
<class 'numpy.float64'>
54.5434250518
<class 'numpy.float64'>
54.5382657376
<class 'numpy.float64'>
54.5330878641
<class 'numpy.float64'>
54.5278912308
<class 'numpy.float64'>
54.5226756305
<class 'numpy.float64'>
54.5174408493
<class 'numpy.float64'>
54.512186666
<class 'numpy.float64'>
54.5069128518
<class 'numpy.float64'>
54.5016191702
<class 'numpy.float64'>
54.4963053759
<class 'numpy.float64'>
54.4909712152
<class 'numpy.float64'>
54.4856164249
<class 'numpy.float64'>
54.4802407322
<class 'numpy.float64'>
54.4748438539
<class 'numpy.float64'>
54.4694254963
<class 'numpy.float64'>
54.4639853541
<class 'numpy.float64'>
54.4585231103
<class 'numpy.float64'>
54.4530384353
<class 'numpy.float64'>
54.4475309864
<class 'numpy.float64'>
54.4420004072
<class 'numpy.float64'>
54.4364463265
<class 'numpy.float64'>
54.430868358
<class 'numpy.float64'>
54.4252660994
<class 'numpy.float64'>
54.4196391313
<class 'numpy.float64'>
54.4139870169
<class 'numpy.float64'>
54.4083093004
<class 'numpy.float64'>
54.4026055065
<class 'numpy.float64'>
54.3968751393
<class 'numpy.float64'>
54.3911176812
<class 'numpy.float64'>
54.3853325915
<class 'numpy.float64'>
54.379519306
<class 'numpy.float64'>
54.3736772347
<class 'numpy.float64'>
54.3678057613
<class 'numpy.float64'>
54.3619042413
<class 'numpy.float64'>
54.355972001
<class 'numpy.float64'>
54.3500083355
<class 'numpy.float64'>
54.3440125073
<class 'numpy.float64'>
54.3379837444
<class 'numpy.float64'>
54.3319212387
<class 'numpy.float64'>
54.3258241438
<class 'numpy.float64'>
54.3196915732
<class 'numpy.float64'>
54.3135225979
<class 'numpy.float64'>
54.3073162444
<class 'numpy.float64'>
54.3010714917
<class 'numpy.float64'>
54.2947872694
<class 'numpy.float64'>
54.2884624545
<class 'numpy.float64'>
54.2820958687
<class 'numpy.float64'>
54.2756862753
<class 'numpy.float64'>
54.2692323758
<class 'numpy.float64'>
54.2627328065
<class 'numpy.float64'>
54.2561861351
<class 'numpy.float64'>
54.2495908562
<class 'numpy.float64'>
54.2429453876
<class 'numpy.float64'>
54.236248066
<class 'numpy.float64'>
54.2294971415
<class 'numpy.float64'>
54.2226907736
<class 'numpy.float64'>
54.2158270251
<class 'numpy.float64'>
54.2089038569
<class 'numpy.float64'>
54.2019191215
<class 'numpy.float64'>
54.1948705571
<class 'numpy.float64'>
54.1877557802
<class 'numpy.float64'>
54.1805722784
<class 'numpy.float64'>
54.1733174028
<class 'numpy.float64'>
54.1659883593
<class 'numpy.float64'>
54.1585822001
<class 'numpy.float64'>
54.1510958135
<class 'numpy.float64'>
54.1435259144
<class 'numpy.float64'>
54.135869033
<class 'numpy.float64'>
54.1281215034
<class 'numpy.float64'>
54.1202794508
<class 'numpy.float64'>
54.1123387786
<class 'numpy.float64'>
54.104295154
<class 'numpy.float64'>
54.0961439928
<class 'numpy.float64'>
54.0878804432
<class 'numpy.float64'>
54.0794993682
<class 'numpy.float64'>
54.0709953275
<class 'numpy.float64'>
54.062362557
<class 'numpy.float64'>
54.0535949481
<class 'numpy.float64'>
54.0446860249
<class 'numpy.float64'>
54.0356289198
<class 'numpy.float64'>
54.0264163481
<class 'numpy.float64'>
54.01704058
<class 'numpy.float64'>
54.0074934118
<class 'numpy.float64'>
53.9977661345
<class 'numpy.float64'>
53.9878495008
<class 'numpy.float64'>
53.9777336898
<class 'numpy.float64'>
53.9674082703
<class 'numpy.float64'>
53.9568621614
<class 'numpy.float64'>
53.9460835911
<class 'numpy.float64'>
53.9350600533
<class 'numpy.float64'>
53.9237782624
<class 'numpy.float64'>
53.9122241061
<class 'numpy.float64'>
53.900382597
<class 'numpy.float64'>
53.8882378223
<class 'numpy.float64'>
53.8757728927
<class 'numpy.float64'>
53.8629698913
<class 'numpy.float64'>
53.8498098217
<class 'numpy.float64'>
53.8362725582
<class 'numpy.float64'>
53.8223367975
<class 'numpy.float64'>
53.8079800136
<class 'numpy.float64'>
53.7931784182
<class 'numpy.float64'>
53.7779069271
<class 'numpy.float64'>
53.7621391361
<class 'numpy.float64'>
53.7458473088
<class 'numpy.float64'>
53.7290023776
<class 'numpy.float64'>
53.7115739647
<class 'numpy.float64'>
53.6935304231
<class 'numpy.float64'>
53.6748389044
<class 'numpy.float64'>
53.6554654573
<class 'numpy.float64'>
53.6353751601
<class 'numpy.float64'>
53.6145322931
<class 'numpy.float64'>
53.5929005547
<class 'numpy.float64'>
53.5704433239
<class 'numpy.float64'>
53.5471239714
<class 'numpy.float64'>
53.5229062207
<class 'numpy.float64'>
53.4977545549
<class 'numpy.float64'>
53.4716346663
<class 'numpy.float64'>
53.4445139377
<class 'numpy.float64'>
53.4163619434
<class 'numpy.float64'>
53.3871509515
<class 'numpy.float64'>
53.3568564062
<class 'numpy.float64'>
53.3254573641
<class 'numpy.float64'>
53.2929368562
<class 'numpy.float64'>
53.259282149
<class 'numpy.float64'>
53.2244848775
<class 'numpy.float64'>
53.1885410319
<class 'numpy.float64'>
53.1514507851
<class 'numpy.float64'>
53.1132181636
<class 'numpy.float64'>
53.0738505785
<class 'numpy.float64'>
53.0333582479
<class 'numpy.float64'>
52.991753565
<class 'numpy.float64'>
52.9490504795
<class 'numpy.float64'>
52.9052639824
<class 'numpy.float64'>
52.8604097964
<class 'numpy.float64'>
52.8145043937
<class 'numpy.float64'>
52.7675654715
<class 'numpy.float64'>
52.7196130224
<class 'numpy.float64'>
52.6706711349
<class 'numpy.float64'>
52.6207706365
<class 'numpy.float64'>
52.5699526405
<class 'numpy.float64'>
52.5182729703
<class 'numpy.float64'>
52.4658072869
<class 'numpy.float64'>
52.4126565487
<class 'numpy.float64'>
52.3589521903
<class 'numpy.float64'>
52.3048601648
<class 'numpy.float64'>
52.2505828248
<class 'numpy.float64'>
52.1963576072
<class 'numpy.float64'>
52.1424517198
<class 'numpy.float64'>
52.0891525154
<class 'numpy.float64'>
52.0367539302
<class 'numpy.float64'>
51.9855400869
<class 'numpy.float64'>
51.9357677653
<class 'numpy.float64'>
51.8876497721
<class 'numpy.float64'>
51.8413412411
<class 'numpy.float64'>
51.7969305814
<class 'numpy.float64'>
51.7544361893
<class 'numpy.float64'>
51.7138092408
<class 'numpy.float64'>
51.6749420035
<class 'numpy.float64'>
51.637680347
<class 'numpy.float64'>
51.6018386563
<class 'numpy.float64'>
51.5672152709
<class 'numpy.float64'>
51.5336068308
<class 'numpy.float64'>
51.5008203832
<class 'numpy.float64'>
51.4686826276
<class 'numpy.float64'>
51.4370461376
<class 'numpy.float64'>
51.4057927344
<class 'numpy.float64'>
51.3748343973
<class 'numpy.float64'>
51.344112203
<class 'numpy.float64'>
51.3135938127
<class 'numpy.float64'>
51.2832700027
<class 'numpy.float64'>
51.2531506745
<class 'numpy.float64'>
51.2232607067
<class 'numpy.float64'>
51.1936359282
<class 'numpy.float64'>
51.1643194089
<class 'numpy.float64'>
51.1353581953
<class 'numpy.float64'>
51.1068005529
<class 'numpy.float64'>
51.078693733
<class 'numpy.float64'>
51.0510822442
<class 'numpy.float64'>
51.0240065858
<class 'numpy.float64'>
50.9975023885
<class 'numpy.float64'>
50.9715999001
<class 'numpy.float64'>
50.9463237536
<class 'numpy.float64'>
50.9216929601
<class 'numpy.float64'>
50.8977210729
<class 'numpy.float64'>
50.8744164769
<class 'numpy.float64'>
50.8517827636
<class 'numpy.float64'>
50.8298191613
<class 'numpy.float64'>
50.8085209925
<class 'numpy.float64'>
50.7878801408
<class 'numpy.float64'>
50.7678855107
<class 'numpy.float64'>
50.7485234704
<class 'numpy.float64'>
50.7297782685
<class 'numpy.float64'>
50.7116324224
<class 'numpy.float64'>
50.694067072
<class 'numpy.float64'>
50.6770623009
<class 'numpy.float64'>
Cost after iteration 2000: 50.677062
50.6605974232
<class 'numpy.float64'>
50.6446512374
<class 'numpy.float64'>
50.6292022489
<class 'numpy.float64'>
50.6142288634
<class 'numpy.float64'>
50.5997095526
<class 'numpy.float64'>
50.5856229953
<class 'numpy.float64'>
50.5719481961
<class 'numpy.float64'>
50.5586645834
<class 'numpy.float64'>
50.5457520895
<class 'numpy.float64'>
50.5331912147
<class 'numpy.float64'>
50.5209630771
<class 'numpy.float64'>
50.5090494506
<class 'numpy.float64'>
50.4974327912
<class 'numpy.float64'>
50.4860962552
<class 'numpy.float64'>
50.4750237088
<class 'numpy.float64'>
50.4641997309
<class 'numpy.float64'>
50.4536096107
<class 'numpy.float64'>
50.4432393402
<class 'numpy.float64'>
50.4330756032
<class 'numpy.float64'>
50.42310576
<class 'numpy.float64'>
50.4133178313
<class 'numpy.float64'>
50.4037004782
<class 'numpy.float64'>
50.3942429822
<class 'numpy.float64'>
50.3849352231
<class 'numpy.float64'>
50.3757676564
<class 'numpy.float64'>
50.3667312902
<class 'numpy.float64'>
50.3578176619
<class 'numpy.float64'>
50.349018815
<class 'numpy.float64'>
50.3403272755
<class 'numpy.float64'>
50.3317360293
<class 'numpy.float64'>
50.3232384996
<class 'numpy.float64'>
50.3148285246
<class 'numpy.float64'>
50.3065003367
<class 'numpy.float64'>
50.2982485409
<class 'numpy.float64'>
50.2900680955
<class 'numpy.float64'>
50.2819542921
<class 'numpy.float64'>
50.2739027374
<class 'numpy.float64'>
50.265909335
<class 'numpy.float64'>
50.2579702685
<class 'numpy.float64'>
50.2500819852
<class 'numpy.float64'>
50.2422411803
<class 'numpy.float64'>
50.2344447819
<class 'numpy.float64'>
50.2266899375
<class 'numpy.float64'>
50.2189739997
<class 'numpy.float64'>
50.2112945141
<class 'numpy.float64'>
50.2036492071
<class 'numpy.float64'>
50.1960359742
<class 'numpy.float64'>
50.1884528695
<class 'numpy.float64'>
50.1808980952
<class 'numpy.float64'>
50.1733699924
<class 'numpy.float64'>
50.1658670312
<class 'numpy.float64'>
50.1583878031
<class 'numpy.float64'>
50.1509310122
<class 'numpy.float64'>
50.1434954678
<class 'numpy.float64'>
50.1360800775
<class 'numpy.float64'>
50.1286838399
<class 'numpy.float64'>
50.121305839
<class 'numpy.float64'>
50.1139452376
<class 'numpy.float64'>
50.1066012722
<class 'numpy.float64'>
50.0992732476
<class 'numpy.float64'>
50.0919605318
<class 'numpy.float64'>
50.084662552
<class 'numpy.float64'>
50.0773787896
<class 'numpy.float64'>
50.0701087766
<class 'numpy.float64'>
50.0628520918
<class 'numpy.float64'>
50.0556083571
<class 'numpy.float64'>
50.0483772346
<class 'numpy.float64'>
50.0411584229
<class 'numpy.float64'>
50.0339516549
<class 'numpy.float64'>
50.0267566947
<class 'numpy.float64'>
50.0195733351
<class 'numpy.float64'>
50.0124013958
<class 'numpy.float64'>
50.0052407203
<class 'numpy.float64'>
49.9980911747
<class 'numpy.float64'>
49.9909526455
<class 'numpy.float64'>
49.9838250379
<class 'numpy.float64'>
49.9767082741
<class 'numpy.float64'>
49.9696022917
<class 'numpy.float64'>
49.9625070426
<class 'numpy.float64'>
49.9554224913
<class 'numpy.float64'>
49.9483486143
<class 'numpy.float64'>
49.9412853981
<class 'numpy.float64'>
49.9342328389
<class 'numpy.float64'>
49.9271909414
<class 'numpy.float64'>
49.9201597176
<class 'numpy.float64'>
49.9131391867
<class 'numpy.float64'>
49.9061293734
<class 'numpy.float64'>
49.8991303079
<class 'numpy.float64'>
49.8921420252
<class 'numpy.float64'>
49.8851645641
<class 'numpy.float64'>
49.8781979668
<class 'numpy.float64'>
49.8712422788
<class 'numpy.float64'>
49.8642975478
<class 'numpy.float64'>
49.8573638237
<class 'numpy.float64'>
49.850441158
<class 'numpy.float64'>
49.8435296038
<class 'numpy.float64'>
49.8366292148
<class 'numpy.float64'>
49.8297400456
<class 'numpy.float64'>
49.8228621513
<class 'numpy.float64'>
49.815995587
<class 'numpy.float64'>
49.8091404079
<class 'numpy.float64'>
49.8022966687
<class 'numpy.float64'>
49.795464424
<class 'numpy.float64'>
49.7886437275
<class 'numpy.float64'>
49.7818346324
<class 'numpy.float64'>
49.7750371908
<class 'numpy.float64'>
49.7682514539
<class 'numpy.float64'>
49.7614774717
<class 'numpy.float64'>
49.7547152931
<class 'numpy.float64'>
49.7479649658
<class 'numpy.float64'>
49.7412265358
<class 'numpy.float64'>
49.734500048
<class 'numpy.float64'>
49.7277855458
<class 'numpy.float64'>
49.7210830709
<class 'numpy.float64'>
49.7143926637
<class 'numpy.float64'>
49.7077143627
<class 'numpy.float64'>
49.7010482052
<class 'numpy.float64'>
49.6943942265
<class 'numpy.float64'>
49.6877524605
<class 'numpy.float64'>
49.6811229393
<class 'numpy.float64'>
49.6745056937
<class 'numpy.float64'>
49.6679007523
<class 'numpy.float64'>
49.6613081426
<class 'numpy.float64'>
49.6547278902
<class 'numpy.float64'>
49.6481600191
<class 'numpy.float64'>
49.6416045516
<class 'numpy.float64'>
49.6350615088
<class 'numpy.float64'>
49.6285309098
<class 'numpy.float64'>
49.6220127724
<class 'numpy.float64'>
49.6155071127
<class 'numpy.float64'>
49.6090139454
<class 'numpy.float64'>
49.6025332838
<class 'numpy.float64'>
49.5960651396
<class 'numpy.float64'>
49.5896095231
<class 'numpy.float64'>
49.5831664433
<class 'numpy.float64'>
49.5767359076
<class 'numpy.float64'>
49.5703179224
<class 'numpy.float64'>
49.5639124924
<class 'numpy.float64'>
49.5575196214
<class 'numpy.float64'>
49.5511393116
<class 'numpy.float64'>
49.5447715642
<class 'numpy.float64'>
49.5384163792
<class 'numpy.float64'>
49.5320737551
<class 'numpy.float64'>
49.5257436898
<class 'numpy.float64'>
49.5194261797
<class 'numpy.float64'>
49.5131212203
<class 'numpy.float64'>
49.5068288058
<class 'numpy.float64'>
49.5005489298
<class 'numpy.float64'>
49.4942815846
<class 'numpy.float64'>
49.4880267617
<class 'numpy.float64'>
49.4817844514
<class 'numpy.float64'>
49.4755546434
<class 'numpy.float64'>
49.4693373264
<class 'numpy.float64'>
49.4631324883
<class 'numpy.float64'>
49.456940116
<class 'numpy.float64'>
49.4507601958
<class 'numpy.float64'>
49.4445927132
<class 'numpy.float64'>
49.4384376528
<class 'numpy.float64'>
49.4322949987
<class 'numpy.float64'>
49.4261647342
<class 'numpy.float64'>
49.4200468418
<class 'numpy.float64'>
49.4139413034
<class 'numpy.float64'>
49.4078481006
<class 'numpy.float64'>
49.4017672139
<class 'numpy.float64'>
49.3956986234
<class 'numpy.float64'>
49.3896423089
<class 'numpy.float64'>
49.3835982492
<class 'numpy.float64'>
49.377566423
<class 'numpy.float64'>
49.3715468081
<class 'numpy.float64'>
49.3655393822
<class 'numpy.float64'>
49.3595441222
<class 'numpy.float64'>
49.3535610048
<class 'numpy.float64'>
49.3475900062
<class 'numpy.float64'>
49.341631102
<class 'numpy.float64'>
49.3356842678
<class 'numpy.float64'>
49.3297494784
<class 'numpy.float64'>
49.3238267084
<class 'numpy.float64'>
49.3179159323
<class 'numpy.float64'>
49.3120171238
<class 'numpy.float64'>
49.3061302566
<class 'numpy.float64'>
49.3002553041
<class 'numpy.float64'>
49.2943922392
<class 'numpy.float64'>
49.2885410347
<class 'numpy.float64'>
49.2827016632
<class 'numpy.float64'>
49.2768740968
<class 'numpy.float64'>
49.2710583075
<class 'numpy.float64'>
49.2652542671
<class 'numpy.float64'>
49.2594619471
<class 'numpy.float64'>
49.253681319
<class 'numpy.float64'>
49.2479123539
<class 'numpy.float64'>
49.2421550226
<class 'numpy.float64'>
49.2364092961
<class 'numpy.float64'>
49.230675145
<class 'numpy.float64'>
49.2249525398
<class 'numpy.float64'>
49.2192414507
<class 'numpy.float64'>
49.2135418481
<class 'numpy.float64'>
49.2078537019
<class 'numpy.float64'>
49.2021769822
<class 'numpy.float64'>
49.1965116588
<class 'numpy.float64'>
49.1908577014
<class 'numpy.float64'>
49.1852150797
<class 'numpy.float64'>
49.1795837633
<class 'numpy.float64'>
49.1739637216
<class 'numpy.float64'>
49.1683549241
<class 'numpy.float64'>
49.1627573401
<class 'numpy.float64'>
49.157170939
<class 'numpy.float64'>
49.1515956898
<class 'numpy.float64'>
49.1460315619
<class 'numpy.float64'>
49.1404785243
<class 'numpy.float64'>
49.1349365462
<class 'numpy.float64'>
49.1294055967
<class 'numpy.float64'>
49.1238856447
<class 'numpy.float64'>
49.1183766594
<class 'numpy.float64'>
49.1128786097
<class 'numpy.float64'>
49.1073914647
<class 'numpy.float64'>
49.1019151933
<class 'numpy.float64'>
49.0964497646
<class 'numpy.float64'>
49.0909951475
<class 'numpy.float64'>
49.085551311
<class 'numpy.float64'>
49.0801182242
<class 'numpy.float64'>
49.074695856
<class 'numpy.float64'>
49.0692841756
<class 'numpy.float64'>
49.0638831519
<class 'numpy.float64'>
49.0584927542
<class 'numpy.float64'>
49.0531129513
<class 'numpy.float64'>
49.0477437126
<class 'numpy.float64'>
49.0423850073
<class 'numpy.float64'>
49.0370368044
<class 'numpy.float64'>
49.0316990733
<class 'numpy.float64'>
49.0263717833
<class 'numpy.float64'>
49.0210549036
<class 'numpy.float64'>
49.0157484038
<class 'numpy.float64'>
49.0104522532
<class 'numpy.float64'>
49.0051664213
<class 'numpy.float64'>
48.9998908777
<class 'numpy.float64'>
48.9946255919
<class 'numpy.float64'>
48.9893705336
<class 'numpy.float64'>
48.9841256725
<class 'numpy.float64'>
48.9788909785
<class 'numpy.float64'>
48.9736664212
<class 'numpy.float64'>
48.9684519707
<class 'numpy.float64'>
48.9632475969
<class 'numpy.float64'>
48.9580532698
<class 'numpy.float64'>
48.9528689595
<class 'numpy.float64'>
48.9476946363
<class 'numpy.float64'>
48.9425302703
<class 'numpy.float64'>
48.9373758319
<class 'numpy.float64'>
48.9322312915
<class 'numpy.float64'>
48.9270966195
<class 'numpy.float64'>
48.9219717865
<class 'numpy.float64'>
48.9168567632
<class 'numpy.float64'>
48.9117515202
<class 'numpy.float64'>
48.9066560283
<class 'numpy.float64'>
48.9015702584
<class 'numpy.float64'>
48.8964941814
<class 'numpy.float64'>
48.8914277684
<class 'numpy.float64'>
48.8863709904
<class 'numpy.float64'>
48.8813238187
<class 'numpy.float64'>
48.8762862246
<class 'numpy.float64'>
48.8712581794
<class 'numpy.float64'>
48.8662396546
<class 'numpy.float64'>
48.8612306216
<class 'numpy.float64'>
48.8562310522
<class 'numpy.float64'>
48.8512409181
<class 'numpy.float64'>
48.846260191
<class 'numpy.float64'>
48.8412888428
<class 'numpy.float64'>
48.8363268456
<class 'numpy.float64'>
48.8313741714
<class 'numpy.float64'>
48.8264307923
<class 'numpy.float64'>
48.8214966805
<class 'numpy.float64'>
48.8165718086
<class 'numpy.float64'>
48.8116561487
<class 'numpy.float64'>
48.8067496736
<class 'numpy.float64'>
48.8018523557
<class 'numpy.float64'>
48.7969641678
<class 'numpy.float64'>
48.7920850826
<class 'numpy.float64'>
48.7872150731
<class 'numpy.float64'>
48.7823541123
<class 'numpy.float64'>
48.7775021731
<class 'numpy.float64'>
48.7726592287
<class 'numpy.float64'>
48.7678252525
<class 'numpy.float64'>
48.7630002177
<class 'numpy.float64'>
48.7581840977
<class 'numpy.float64'>
48.7533768662
<class 'numpy.float64'>
48.7485784966
<class 'numpy.float64'>
48.7437889628
<class 'numpy.float64'>
48.7390082384
<class 'numpy.float64'>
48.7342362975
<class 'numpy.float64'>
48.7294731139
<class 'numpy.float64'>
48.7247186618
<class 'numpy.float64'>
48.7199729153
<class 'numpy.float64'>
48.7152358487
<class 'numpy.float64'>
48.7105074363
<class 'numpy.float64'>
48.7057876526
<class 'numpy.float64'>
48.701076472
<class 'numpy.float64'>
48.6963738693
<class 'numpy.float64'>
48.6916798191
<class 'numpy.float64'>
48.6869942963
<class 'numpy.float64'>
48.6823172757
<class 'numpy.float64'>
48.6776487322
<class 'numpy.float64'>
48.6729886411
<class 'numpy.float64'>
48.6683369774
<class 'numpy.float64'>
48.6636937165
<class 'numpy.float64'>
48.6590588335
<class 'numpy.float64'>
48.6544323041
<class 'numpy.float64'>
48.6498141036
<class 'numpy.float64'>
48.6452042078
<class 'numpy.float64'>
48.6406025923
<class 'numpy.float64'>
48.6360092329
<class 'numpy.float64'>
48.6314241054
<class 'numpy.float64'>
48.626847186
<class 'numpy.float64'>
48.6222784505
<class 'numpy.float64'>
48.6177178752
<class 'numpy.float64'>
48.6131654362
<class 'numpy.float64'>
48.6086211099
<class 'numpy.float64'>
48.6040848727
<class 'numpy.float64'>
48.5995567011
<class 'numpy.float64'>
48.5950365716
<class 'numpy.float64'>
48.590524461
<class 'numpy.float64'>
48.5860203459
<class 'numpy.float64'>
48.5815242032
<class 'numpy.float64'>
48.5770360098
<class 'numpy.float64'>
48.5725557427
<class 'numpy.float64'>
48.568083379
<class 'numpy.float64'>
48.5636188959
<class 'numpy.float64'>
48.5591622706
<class 'numpy.float64'>
48.5547134804
<class 'numpy.float64'>
48.5502725029
<class 'numpy.float64'>
48.5458393154
<class 'numpy.float64'>
48.5414138956
<class 'numpy.float64'>
48.5369962212
<class 'numpy.float64'>
48.5325862698
<class 'numpy.float64'>
48.5281840194
<class 'numpy.float64'>
48.5237894478
<class 'numpy.float64'>
48.5194025331
<class 'numpy.float64'>
48.5150232533
<class 'numpy.float64'>
48.5106515866
<class 'numpy.float64'>
48.5062875113
<class 'numpy.float64'>
48.5019310055
<class 'numpy.float64'>
48.4975820479
<class 'numpy.float64'>
48.4932406167
<class 'numpy.float64'>
48.4889066906
<class 'numpy.float64'>
48.4845802483
<class 'numpy.float64'>
48.4802612684
<class 'numpy.float64'>
48.4759497297
<class 'numpy.float64'>
48.4716456111
<class 'numpy.float64'>
48.4673488915
<class 'numpy.float64'>
48.46305955
<class 'numpy.float64'>
48.4587775657
<class 'numpy.float64'>
48.4545029178
<class 'numpy.float64'>
48.4502355855
<class 'numpy.float64'>
48.4459755481
<class 'numpy.float64'>
48.441722785
<class 'numpy.float64'>
48.4374772758
<class 'numpy.float64'>
48.433239
<class 'numpy.float64'>
48.4290079372
<class 'numpy.float64'>
48.4247840672
<class 'numpy.float64'>
48.4205673696
<class 'numpy.float64'>
48.4163578244
<class 'numpy.float64'>
48.4121554115
<class 'numpy.float64'>
48.407960111
<class 'numpy.float64'>
48.4037719027
<class 'numpy.float64'>
48.399590767
<class 'numpy.float64'>
48.3954166841
<class 'numpy.float64'>
48.3912496341
<class 'numpy.float64'>
48.3870895976
<class 'numpy.float64'>
48.3829365549
<class 'numpy.float64'>
48.3787904865
<class 'numpy.float64'>
48.374651373
<class 'numpy.float64'>
48.3705191951
<class 'numpy.float64'>
48.3663939334
<class 'numpy.float64'>
48.3622755688
<class 'numpy.float64'>
48.3581640821
<class 'numpy.float64'>
48.3540594543
<class 'numpy.float64'>
48.3499616663
<class 'numpy.float64'>
48.3458706991
<class 'numpy.float64'>
48.341786534
<class 'numpy.float64'>
48.3377091521
<class 'numpy.float64'>
48.3336385347
<class 'numpy.float64'>
48.3295746631
<class 'numpy.float64'>
48.3255175187
<class 'numpy.float64'>
48.3214670831
<class 'numpy.float64'>
48.3174233376
<class 'numpy.float64'>
48.313386264
<class 'numpy.float64'>
48.3093558438
<class 'numpy.float64'>
48.3053320589
<class 'numpy.float64'>
48.301314891
<class 'numpy.float64'>
48.297304322
<class 'numpy.float64'>
48.2933003338
<class 'numpy.float64'>
48.2893029084
<class 'numpy.float64'>
48.2853120278
<class 'numpy.float64'>
48.2813276743
<class 'numpy.float64'>
48.2773498299
<class 'numpy.float64'>
48.2733784769
<class 'numpy.float64'>
48.2694135976
<class 'numpy.float64'>
48.2654551745
<class 'numpy.float64'>
48.2615031898
<class 'numpy.float64'>
48.2575576262
<class 'numpy.float64'>
48.2536184662
<class 'numpy.float64'>
48.2496856924
<class 'numpy.float64'>
48.2457592875
<class 'numpy.float64'>
48.2418392342
<class 'numpy.float64'>
48.2379255154
<class 'numpy.float64'>
48.2340181139
<class 'numpy.float64'>
48.2301170126
<class 'numpy.float64'>
48.2262221946
<class 'numpy.float64'>
48.2223336428
<class 'numpy.float64'>
48.2184513405
<class 'numpy.float64'>
48.2145752706
<class 'numpy.float64'>
48.2107054166
<class 'numpy.float64'>
48.2068417616
<class 'numpy.float64'>
48.202984289
<class 'numpy.float64'>
48.1991329823
<class 'numpy.float64'>
48.1952878247
<class 'numpy.float64'>
48.1914488
<class 'numpy.float64'>
48.1876158916
<class 'numpy.float64'>
48.1837890833
<class 'numpy.float64'>
48.1799683586
<class 'numpy.float64'>
48.1761537013
<class 'numpy.float64'>
48.1723450952
<class 'numpy.float64'>
48.1685425243
<class 'numpy.float64'>
48.1647459723
<class 'numpy.float64'>
48.1609554234
<class 'numpy.float64'>
48.1571708614
<class 'numpy.float64'>
48.1533922705
<class 'numpy.float64'>
48.1496196349
<class 'numpy.float64'>
48.1458529386
<class 'numpy.float64'>
48.142092166
<class 'numpy.float64'>
48.1383373014
<class 'numpy.float64'>
48.1345883292
<class 'numpy.float64'>
48.1308452336
<class 'numpy.float64'>
48.1271079993
<class 'numpy.float64'>
48.1233766106
<class 'numpy.float64'>
48.1196510523
<class 'numpy.float64'>
48.1159313088
<class 'numpy.float64'>
48.112217365
<class 'numpy.float64'>
48.1085092055
<class 'numpy.float64'>
48.104806815
<class 'numpy.float64'>
48.1011101785
<class 'numpy.float64'>
48.0974192809
<class 'numpy.float64'>
48.0937341069
<class 'numpy.float64'>
48.0900546418
<class 'numpy.float64'>
48.0863808704
<class 'numpy.float64'>
48.0827127779
<class 'numpy.float64'>
48.0790503494
<class 'numpy.float64'>
48.0753935701
<class 'numpy.float64'>
48.0717424253
<class 'numpy.float64'>
48.0680969003
<class 'numpy.float64'>
48.0644569803
<class 'numpy.float64'>
48.0608226508
<class 'numpy.float64'>
48.0571938973
<class 'numpy.float64'>
48.0535707052
<class 'numpy.float64'>
48.0499530601
<class 'numpy.float64'>
48.0463409475
<class 'numpy.float64'>
48.0427343531
<class 'numpy.float64'>
48.0391332627
<class 'numpy.float64'>
48.0355376618
<class 'numpy.float64'>
48.0319475363
<class 'numpy.float64'>
48.0283628721
<class 'numpy.float64'>
48.0247836551
<class 'numpy.float64'>
48.021209871
<class 'numpy.float64'>
48.017641506
<class 'numpy.float64'>
48.0140785461
<class 'numpy.float64'>
48.0105209772
<class 'numpy.float64'>
48.0069687856
<class 'numpy.float64'>
48.0034219574
<class 'numpy.float64'>
47.9998804788
<class 'numpy.float64'>
47.9963443361
<class 'numpy.float64'>
47.9928135155
<class 'numpy.float64'>
47.9892880035
<class 'numpy.float64'>
47.9857677863
<class 'numpy.float64'>
47.9822528505
<class 'numpy.float64'>
47.9787431825
<class 'numpy.float64'>
47.9752387689
<class 'numpy.float64'>
47.9717395962
<class 'numpy.float64'>
47.968245651
<class 'numpy.float64'>
47.9647569201
<class 'numpy.float64'>
47.9612733901
<class 'numpy.float64'>
47.9577950477
<class 'numpy.float64'>
47.9543218798
<class 'numpy.float64'>
47.9508538732
<class 'numpy.float64'>
47.9473910147
<class 'numpy.float64'>
47.9439332914
<class 'numpy.float64'>
47.9404806902
<class 'numpy.float64'>
47.937033198
<class 'numpy.float64'>
47.933590802
<class 'numpy.float64'>
47.9301534892
<class 'numpy.float64'>
47.9267212468
<class 'numpy.float64'>
47.923294062
<class 'numpy.float64'>
47.9198719219
<class 'numpy.float64'>
47.9164548139
<class 'numpy.float64'>
47.9130427253
<class 'numpy.float64'>
47.9096356434
<class 'numpy.float64'>
47.9062335555
<class 'numpy.float64'>
47.9028364493
<class 'numpy.float64'>
47.899444312
<class 'numpy.float64'>
47.8960571313
<class 'numpy.float64'>
47.8926748946
<class 'numpy.float64'>
47.8892975896
<class 'numpy.float64'>
47.8859252039
<class 'numpy.float64'>
47.8825577252
<class 'numpy.float64'>
47.8791951413
<class 'numpy.float64'>
47.8758374398
<class 'numpy.float64'>
47.8724846085
<class 'numpy.float64'>
47.8691366354
<class 'numpy.float64'>
47.8657935083
<class 'numpy.float64'>
47.862455215
<class 'numpy.float64'>
47.8591217436
<class 'numpy.float64'>
47.855793082
<class 'numpy.float64'>
47.8524692183
<class 'numpy.float64'>
47.8491501406
<class 'numpy.float64'>
47.8458358369
<class 'numpy.float64'>
47.8425262955
<class 'numpy.float64'>
47.8392215044
<class 'numpy.float64'>
47.835921452
<class 'numpy.float64'>
47.8326261264
<class 'numpy.float64'>
47.829335516
<class 'numpy.float64'>
47.8260496092
<class 'numpy.float64'>
47.8227683943
<class 'numpy.float64'>
47.8194918596
<class 'numpy.float64'>
47.8162199938
<class 'numpy.float64'>
47.8129527852
<class 'numpy.float64'>
47.8096902224
<class 'numpy.float64'>
47.8064322939
<class 'numpy.float64'>
47.8031789884
<class 'numpy.float64'>
47.7999302945
<class 'numpy.float64'>
47.7966862008
<class 'numpy.float64'>
47.793446696
<class 'numpy.float64'>
47.790211769
<class 'numpy.float64'>
47.7869814085
<class 'numpy.float64'>
47.7837556032
<class 'numpy.float64'>
47.7805343421
<class 'numpy.float64'>
47.777317614
<class 'numpy.float64'>
47.7741054079
<class 'numpy.float64'>
47.7708977128
<class 'numpy.float64'>
47.7676945175
<class 'numpy.float64'>
47.7644958112
<class 'numpy.float64'>
47.7613015828
<class 'numpy.float64'>
47.7581118216
<class 'numpy.float64'>
47.7549265166
<class 'numpy.float64'>
47.751745657
<class 'numpy.float64'>
47.748569232
<class 'numpy.float64'>
47.7453972308
<class 'numpy.float64'>
47.7422296427
<class 'numpy.float64'>
47.739066457
<class 'numpy.float64'>
47.7359076631
<class 'numpy.float64'>
47.7327532503
<class 'numpy.float64'>
47.729603208
<class 'numpy.float64'>
47.7264575256
<class 'numpy.float64'>
47.7233161927
<class 'numpy.float64'>
47.7201791986
<class 'numpy.float64'>
47.7170465331
<class 'numpy.float64'>
47.7139181856
<class 'numpy.float64'>
47.7107941457
<class 'numpy.float64'>
47.7076744031
<class 'numpy.float64'>
47.7045589474
<class 'numpy.float64'>
47.7014477683
<class 'numpy.float64'>
47.6983408557
<class 'numpy.float64'>
47.6952381991
<class 'numpy.float64'>
47.6921397885
<class 'numpy.float64'>
47.6890456136
<class 'numpy.float64'>
47.6859556644
<class 'numpy.float64'>
47.6828699307
<class 'numpy.float64'>
47.6797884023
<class 'numpy.float64'>
47.6767110694
<class 'numpy.float64'>
47.6736379219
<class 'numpy.float64'>
47.6705689497
<class 'numpy.float64'>
47.667504143
<class 'numpy.float64'>
47.6644434918
<class 'numpy.float64'>
47.6613869862
<class 'numpy.float64'>
47.6583346164
<class 'numpy.float64'>
47.6552863725
<class 'numpy.float64'>
47.6522422447
<class 'numpy.float64'>
47.6492022233
<class 'numpy.float64'>
47.6461662986
<class 'numpy.float64'>
47.6431344608
<class 'numpy.float64'>
47.6401067002
<class 'numpy.float64'>
47.6370830072
<class 'numpy.float64'>
47.6340633722
<class 'numpy.float64'>
47.6310477856
<class 'numpy.float64'>
47.6280362378
<class 'numpy.float64'>
47.6250287193
<class 'numpy.float64'>
47.6220252207
<class 'numpy.float64'>
47.6190257323
<class 'numpy.float64'>
47.6160302449
<class 'numpy.float64'>
47.6130387489
<class 'numpy.float64'>
47.610051235
<class 'numpy.float64'>
47.6070676939
<class 'numpy.float64'>
47.6040881162
<class 'numpy.float64'>
47.6011124926
<class 'numpy.float64'>
47.5981408138
<class 'numpy.float64'>
47.5951730707
<class 'numpy.float64'>
47.592209254
<class 'numpy.float64'>
47.5892493544
<class 'numpy.float64'>
47.5862933629
<class 'numpy.float64'>
47.5833412704
<class 'numpy.float64'>
47.5803930676
<class 'numpy.float64'>
47.5774487457
<class 'numpy.float64'>
47.5745082954
<class 'numpy.float64'>
47.5715717078
<class 'numpy.float64'>
47.568638974
<class 'numpy.float64'>
47.5657100848
<class 'numpy.float64'>
47.5627850315
<class 'numpy.float64'>
47.5598638051
<class 'numpy.float64'>
47.5569463967
<class 'numpy.float64'>
47.5540327974
<class 'numpy.float64'>
47.5511229985
<class 'numpy.float64'>
47.5482169911
<class 'numpy.float64'>
47.5453147665
<class 'numpy.float64'>
47.5424163158
<class 'numpy.float64'>
47.5395216305
<class 'numpy.float64'>
47.5366307018
<class 'numpy.float64'>
47.533743521
<class 'numpy.float64'>
47.5308600795
<class 'numpy.float64'>
47.5279803686
<class 'numpy.float64'>
47.5251043799
<class 'numpy.float64'>
47.5222321046
<class 'numpy.float64'>
47.5193635344
<class 'numpy.float64'>
47.5164986606
<class 'numpy.float64'>
47.5136374748
<class 'numpy.float64'>
47.5107799685
<class 'numpy.float64'>
47.5079261332
<class 'numpy.float64'>
47.5050759607
<class 'numpy.float64'>
47.5022294424
<class 'numpy.float64'>
47.4993865701
<class 'numpy.float64'>
47.4965473354
<class 'numpy.float64'>
47.4937117299
<class 'numpy.float64'>
47.4908797454
<class 'numpy.float64'>
47.4880513737
<class 'numpy.float64'>
47.4852266065
<class 'numpy.float64'>
47.4824054355
<class 'numpy.float64'>
47.4795878527
<class 'numpy.float64'>
47.4767738498
<class 'numpy.float64'>
47.4739634187
<class 'numpy.float64'>
47.4711565513
<class 'numpy.float64'>
47.4683532395
<class 'numpy.float64'>
47.4655534752
<class 'numpy.float64'>
47.4627572504
<class 'numpy.float64'>
47.4599645571
<class 'numpy.float64'>
47.4571753872
<class 'numpy.float64'>
47.4543897329
<class 'numpy.float64'>
47.4516075861
<class 'numpy.float64'>
47.448828939
<class 'numpy.float64'>
47.4460537836
<class 'numpy.float64'>
47.4432821121
<class 'numpy.float64'>
47.4405139166
<class 'numpy.float64'>
47.4377491893
<class 'numpy.float64'>
47.4349879224
<class 'numpy.float64'>
47.432230108
<class 'numpy.float64'>
47.4294757385
<class 'numpy.float64'>
47.4267248061
<class 'numpy.float64'>
47.4239773031
<class 'numpy.float64'>
47.4212332217
<class 'numpy.float64'>
47.4184925544
<class 'numpy.float64'>
47.4157552935
<class 'numpy.float64'>
47.4130214313
<class 'numpy.float64'>
47.4102909603
<class 'numpy.float64'>
47.4075638728
<class 'numpy.float64'>
47.4048401614
<class 'numpy.float64'>
47.4021198184
<class 'numpy.float64'>
47.3994028364
<class 'numpy.float64'>
47.3966892079
<class 'numpy.float64'>
47.3939789254
<class 'numpy.float64'>
47.3912719815
<class 'numpy.float64'>
47.3885683687
<class 'numpy.float64'>
47.3858680796
<class 'numpy.float64'>
47.3831711069
<class 'numpy.float64'>
47.3804774432
<class 'numpy.float64'>
47.3777870811
<class 'numpy.float64'>
47.3751000134
<class 'numpy.float64'>
47.3724162327
<class 'numpy.float64'>
47.3697357318
<class 'numpy.float64'>
47.3670585035
<class 'numpy.float64'>
47.3643845404
<class 'numpy.float64'>
47.3617138353
<class 'numpy.float64'>
47.3590463812
<class 'numpy.float64'>
47.3563821708
<class 'numpy.float64'>
47.3537211969
<class 'numpy.float64'>
47.3510634525
<class 'numpy.float64'>
47.3484089304
<class 'numpy.float64'>
47.3457576235
<class 'numpy.float64'>
47.3431095249
<class 'numpy.float64'>
47.3404646273
<class 'numpy.float64'>
47.3378229239
<class 'numpy.float64'>
47.3351844076
<class 'numpy.float64'>
47.3325490714
<class 'numpy.float64'>
47.3299169084
<class 'numpy.float64'>
47.3272879116
<class 'numpy.float64'>
47.3246620741
<class 'numpy.float64'>
47.3220393889
<class 'numpy.float64'>
47.3194198493
<class 'numpy.float64'>
47.3168034483
<class 'numpy.float64'>
47.3141901791
<class 'numpy.float64'>
47.3115800348
<class 'numpy.float64'>
47.3089730088
<class 'numpy.float64'>
47.3063690941
<class 'numpy.float64'>
47.303768284
<class 'numpy.float64'>
47.3011705718
<class 'numpy.float64'>
47.2985759507
<class 'numpy.float64'>
47.2959844141
<class 'numpy.float64'>
47.2933959552
<class 'numpy.float64'>
47.2908105674
<class 'numpy.float64'>
47.288228244
<class 'numpy.float64'>
47.2856489784
<class 'numpy.float64'>
47.2830727639
<class 'numpy.float64'>
47.2804995941
<class 'numpy.float64'>
47.2779294623
<class 'numpy.float64'>
47.2753623619
<class 'numpy.float64'>
47.2727982864
<class 'numpy.float64'>
47.2702372294
<class 'numpy.float64'>
47.2676791843
<class 'numpy.float64'>
47.2651241445
<class 'numpy.float64'>
47.2625721037
<class 'numpy.float64'>
47.2600230555
<class 'numpy.float64'>
47.2574769933
<class 'numpy.float64'>
47.2549339108
<class 'numpy.float64'>
47.2523938016
<class 'numpy.float64'>
47.2498566593
<class 'numpy.float64'>
47.2473224776
<class 'numpy.float64'>
47.2447912501
<class 'numpy.float64'>
47.2422629705
<class 'numpy.float64'>
47.2397376325
<class 'numpy.float64'>
47.2372152299
<class 'numpy.float64'>
47.2346957563
<class 'numpy.float64'>
47.2321792055
<class 'numpy.float64'>
47.2296655714
<class 'numpy.float64'>
47.2271548476
<class 'numpy.float64'>
47.224647028
<class 'numpy.float64'>
47.2221421064
<class 'numpy.float64'>
47.2196400766
<class 'numpy.float64'>
47.2171409326
<class 'numpy.float64'>
47.2146446681
<class 'numpy.float64'>
47.2121512772
<class 'numpy.float64'>
47.2096607536
<class 'numpy.float64'>
47.2071730913
<class 'numpy.float64'>
47.2046882843
<class 'numpy.float64'>
47.2022063265
<class 'numpy.float64'>
47.199727212
<class 'numpy.float64'>
47.1972509346
<class 'numpy.float64'>
47.1947774884
<class 'numpy.float64'>
47.1923068675
<class 'numpy.float64'>
47.1898390659
<class 'numpy.float64'>
47.1873740776
<class 'numpy.float64'>
47.1849118967
<class 'numpy.float64'>
47.1824525174
<class 'numpy.float64'>
47.1799959337
<class 'numpy.float64'>
47.1775421398
<class 'numpy.float64'>
47.1750911298
<class 'numpy.float64'>
47.1726428978
<class 'numpy.float64'>
47.1701974381
<class 'numpy.float64'>
47.1677547448
<class 'numpy.float64'>
47.1653148122
<class 'numpy.float64'>
47.1628776345
<class 'numpy.float64'>
47.1604432059
<class 'numpy.float64'>
47.1580115207
<class 'numpy.float64'>
47.1555825731
<class 'numpy.float64'>
47.1531563575
<class 'numpy.float64'>
47.1507328681
<class 'numpy.float64'>
47.1483120993
<class 'numpy.float64'>
47.1458940454
<class 'numpy.float64'>
47.1434787008
<class 'numpy.float64'>
47.1410660598
<class 'numpy.float64'>
47.1386561168
<class 'numpy.float64'>
47.1362488663
<class 'numpy.float64'>
47.1338443026
<class 'numpy.float64'>
47.1314424201
<class 'numpy.float64'>
47.1290432133
<class 'numpy.float64'>
47.1266466768
<class 'numpy.float64'>
47.1242528048
<class 'numpy.float64'>
47.1218615921
<class 'numpy.float64'>
47.1194730329
<class 'numpy.float64'>
47.117087122
<class 'numpy.float64'>
47.1147038537
<class 'numpy.float64'>
47.1123232227
<class 'numpy.float64'>
47.1099452235
<class 'numpy.float64'>
47.1075698508
<class 'numpy.float64'>
47.105197099
<class 'numpy.float64'>
47.1028269629
<class 'numpy.float64'>
47.1004594371
<class 'numpy.float64'>
47.0980945161
<class 'numpy.float64'>
47.0957321946
<class 'numpy.float64'>
47.0933724674
<class 'numpy.float64'>
47.0910153291
<class 'numpy.float64'>
47.0886607744
<class 'numpy.float64'>
47.086308798
<class 'numpy.float64'>
47.0839593947
<class 'numpy.float64'>
47.0816125592
<class 'numpy.float64'>
47.0792682862
<class 'numpy.float64'>
47.0769265705
<class 'numpy.float64'>
47.074587407
<class 'numpy.float64'>
47.0722507903
<class 'numpy.float64'>
47.0699167154
<class 'numpy.float64'>
47.0675851771
<class 'numpy.float64'>
47.0652561701
<class 'numpy.float64'>
47.0629296894
<class 'numpy.float64'>
47.0606057298
<class 'numpy.float64'>
47.0582842862
<class 'numpy.float64'>
47.0559653536
<class 'numpy.float64'>
47.0536489268
<class 'numpy.float64'>
47.0513350007
<class 'numpy.float64'>
47.0490235703
<class 'numpy.float64'>
47.0467146306
<class 'numpy.float64'>
47.0444081764
<class 'numpy.float64'>
47.0421042029
<class 'numpy.float64'>
47.0398027049
<class 'numpy.float64'>
47.0375036776
<class 'numpy.float64'>
47.0352071159
<class 'numpy.float64'>
47.0329130148
<class 'numpy.float64'>
47.0306213695
<class 'numpy.float64'>
47.0283321749
<class 'numpy.float64'>
47.0260454261
<class 'numpy.float64'>
47.0237611183
<class 'numpy.float64'>
47.0214792465
<class 'numpy.float64'>
47.0191998059
<class 'numpy.float64'>
47.0169227915
<class 'numpy.float64'>
47.0146481986
<class 'numpy.float64'>
47.0123760222
<class 'numpy.float64'>
47.0101062576
<class 'numpy.float64'>
47.0078388999
<class 'numpy.float64'>
47.0055739443
<class 'numpy.float64'>
47.0033113861
<class 'numpy.float64'>
47.0010512204
<class 'numpy.float64'>
46.9987934424
<class 'numpy.float64'>
46.9965380475
<class 'numpy.float64'>
46.9942850309
<class 'numpy.float64'>
46.9920343878
<class 'numpy.float64'>
46.9897861135
<class 'numpy.float64'>
46.9875402034
<class 'numpy.float64'>
46.9852966527
<class 'numpy.float64'>
46.9830554567
<class 'numpy.float64'>
46.9808166109
<class 'numpy.float64'>
46.9785801105
<class 'numpy.float64'>
46.9763459508
<class 'numpy.float64'>
46.9741141274
<class 'numpy.float64'>
46.9718846354
<class 'numpy.float64'>
46.9696574705
<class 'numpy.float64'>
46.9674326278
<class 'numpy.float64'>
46.965210103
<class 'numpy.float64'>
46.9629898913
<class 'numpy.float64'>
46.9607719883
<class 'numpy.float64'>
46.9585563893
<class 'numpy.float64'>
46.95634309
<class 'numpy.float64'>
46.9541320856
<class 'numpy.float64'>
46.9519233719
<class 'numpy.float64'>
46.9497169441
<class 'numpy.float64'>
46.9475127979
<class 'numpy.float64'>
46.9453109288
<class 'numpy.float64'>
46.9431113324
<class 'numpy.float64'>
46.9409140041
<class 'numpy.float64'>
46.9387189395
<class 'numpy.float64'>
46.9365261343
<class 'numpy.float64'>
46.9343355839
<class 'numpy.float64'>
46.9321472841
<class 'numpy.float64'>
46.9299612304
<class 'numpy.float64'>
46.9277774184
<class 'numpy.float64'>
46.9255958437
<class 'numpy.float64'>
46.9234165021
<class 'numpy.float64'>
46.9212393891
<class 'numpy.float64'>
46.9190645004
<class 'numpy.float64'>
46.9168918318
<class 'numpy.float64'>
46.9147213789
<class 'numpy.float64'>
46.9125531373
<class 'numpy.float64'>
46.9103871029
<class 'numpy.float64'>
46.9082232713
<class 'numpy.float64'>
46.9060616382
<class 'numpy.float64'>
46.9039021995
<class 'numpy.float64'>
46.9017449509
<class 'numpy.float64'>
46.8995898881
<class 'numpy.float64'>
46.8974370069
<class 'numpy.float64'>
46.8952863031
<class 'numpy.float64'>
46.8931377725
<class 'numpy.float64'>
46.890991411
<class 'numpy.float64'>
46.8888472143
<class 'numpy.float64'>
46.8867051783
<class 'numpy.float64'>
46.8845652988
<class 'numpy.float64'>
46.8824275717
<class 'numpy.float64'>
46.8802919928
<class 'numpy.float64'>
46.8781585581
<class 'numpy.float64'>
46.8760272634
<class 'numpy.float64'>
46.8738981046
<class 'numpy.float64'>
46.8717710776
<class 'numpy.float64'>
46.8696461783
<class 'numpy.float64'>
46.8675234028
<class 'numpy.float64'>
46.8654027468
<class 'numpy.float64'>
46.8632842064
<class 'numpy.float64'>
46.8611677776
<class 'numpy.float64'>
46.8590534562
<class 'numpy.float64'>
46.8569412383
<class 'numpy.float64'>
46.8548311199
<class 'numpy.float64'>
46.852723097
<class 'numpy.float64'>
46.8506171656
<class 'numpy.float64'>
46.8485133217
<class 'numpy.float64'>
46.8464115613
<class 'numpy.float64'>
46.8443118806
<class 'numpy.float64'>
46.8422142755
<class 'numpy.float64'>
46.8401187421
<class 'numpy.float64'>
46.8380252765
<class 'numpy.float64'>
46.8359338748
<class 'numpy.float64'>
46.8338445331
<class 'numpy.float64'>
46.8317572474
<class 'numpy.float64'>
46.829672014
<class 'numpy.float64'>
46.8275888288
<class 'numpy.float64'>
46.8255076882
<class 'numpy.float64'>
46.8234285881
<class 'numpy.float64'>
46.8213515248
<class 'numpy.float64'>
46.8192764944
<class 'numpy.float64'>
46.8172034931
<class 'numpy.float64'>
46.8151325171
<class 'numpy.float64'>
46.8130635626
<class 'numpy.float64'>
46.8109966258
<class 'numpy.float64'>
46.8089317029
<class 'numpy.float64'>
46.8068687901
<class 'numpy.float64'>
46.8048078836
<class 'numpy.float64'>
46.8027489797
<class 'numpy.float64'>
46.8006920747
<class 'numpy.float64'>
46.7986371648
<class 'numpy.float64'>
46.7965842463
<class 'numpy.float64'>
46.7945333155
<class 'numpy.float64'>
46.7924843686
<class 'numpy.float64'>
46.790437402
<class 'numpy.float64'>
46.7883924119
<class 'numpy.float64'>
46.7863493947
<class 'numpy.float64'>
46.7843083467
<class 'numpy.float64'>
46.7822692643
<class 'numpy.float64'>
46.7802321438
<class 'numpy.float64'>
46.7781969816
<class 'numpy.float64'>
46.7761637739
<class 'numpy.float64'>
46.7741325173
<class 'numpy.float64'>
46.772103208
<class 'numpy.float64'>
46.7700758425
<class 'numpy.float64'>
46.7680504171
<class 'numpy.float64'>
46.7660269284
<class 'numpy.float64'>
46.7640053727
<class 'numpy.float64'>
46.7619857463
<class 'numpy.float64'>
46.7599680459
<class 'numpy.float64'>
46.7579522678
<class 'numpy.float64'>
46.7559384084
<class 'numpy.float64'>
46.7539264643
<class 'numpy.float64'>
46.7519164319
<class 'numpy.float64'>
46.7499083078
<class 'numpy.float64'>
46.7479020882
<class 'numpy.float64'>
46.7458977699
<class 'numpy.float64'>
46.7438953493
<class 'numpy.float64'>
46.7418948229
<class 'numpy.float64'>
46.7398961873
<class 'numpy.float64'>
46.7378994389
<class 'numpy.float64'>
46.7359045743
<class 'numpy.float64'>
46.7339115902
<class 'numpy.float64'>
46.7319204829
<class 'numpy.float64'>
46.7299312492
<class 'numpy.float64'>
46.7279438855
<class 'numpy.float64'>
46.7259583886
<class 'numpy.float64'>
46.7239747549
<class 'numpy.float64'>
46.7219929811
<class 'numpy.float64'>
46.7200130637
<class 'numpy.float64'>
46.7180349995
<class 'numpy.float64'>
46.716058785
<class 'numpy.float64'>
46.7140844169
<class 'numpy.float64'>
46.7121118919
<class 'numpy.float64'>
46.7101412065
<class 'numpy.float64'>
46.7081723574
<class 'numpy.float64'>
46.7062053414
<class 'numpy.float64'>
46.704240155
<class 'numpy.float64'>
46.702276795
<class 'numpy.float64'>
46.7003152582
<class 'numpy.float64'>
46.698355541
<class 'numpy.float64'>
46.6963976404
<class 'numpy.float64'>
46.694441553
<class 'numpy.float64'>
46.6924872756
<class 'numpy.float64'>
46.6905348048
<class 'numpy.float64'>
46.6885841375
<class 'numpy.float64'>
46.6866352703
<class 'numpy.float64'>
46.6846882001
<class 'numpy.float64'>
46.6827429236
<class 'numpy.float64'>
46.6807994376
<class 'numpy.float64'>
46.6788577389
<class 'numpy.float64'>
46.6769178242
<class 'numpy.float64'>
46.6749796904
<class 'numpy.float64'>
46.6730433343
<class 'numpy.float64'>
46.6711087527
<class 'numpy.float64'>
46.6691759424
<class 'numpy.float64'>
46.6672449002
<class 'numpy.float64'>
46.6653156231
<class 'numpy.float64'>
46.6633881077
<class 'numpy.float64'>
46.6614623511
<class 'numpy.float64'>
46.6595383501
<class 'numpy.float64'>
46.6576161014
<class 'numpy.float64'>
46.6556956021
<class 'numpy.float64'>
46.653776849
<class 'numpy.float64'>
46.651859839
<class 'numpy.float64'>
46.6499445689
<class 'numpy.float64'>
46.6480310358
<class 'numpy.float64'>
46.6461192365
<class 'numpy.float64'>
46.6442091679
<class 'numpy.float64'>
46.642300827
<class 'numpy.float64'>
46.6403942107
<class 'numpy.float64'>
46.638489316
<class 'numpy.float64'>
46.6365861398
<class 'numpy.float64'>
46.6346846791
<class 'numpy.float64'>
46.6327849308
<class 'numpy.float64'>
Cost after iteration 3000: 46.632785
46.630886892
<class 'numpy.float64'>
46.6289905595
<class 'numpy.float64'>
46.6270959305
<class 'numpy.float64'>
46.6252030019
<class 'numpy.float64'>
46.6233117707
<class 'numpy.float64'>
46.6214222339
<class 'numpy.float64'>
46.6195343885
<class 'numpy.float64'>
46.6176482316
<class 'numpy.float64'>
46.6157637602
<class 'numpy.float64'>
46.6138809714
<class 'numpy.float64'>
46.6119998621
<class 'numpy.float64'>
46.6101204295
<class 'numpy.float64'>
46.6082426706
<class 'numpy.float64'>
46.6063665825
<class 'numpy.float64'>
46.6044921623
<class 'numpy.float64'>
46.602619407
<class 'numpy.float64'>
46.6007483138
<class 'numpy.float64'>
46.5988788797
<class 'numpy.float64'>
46.5970111019
<class 'numpy.float64'>
46.5951449774
<class 'numpy.float64'>
46.5932805034
<class 'numpy.float64'>
46.591417677
<class 'numpy.float64'>
46.5895564953
<class 'numpy.float64'>
46.5876969556
<class 'numpy.float64'>
46.5858390548
<class 'numpy.float64'>
46.5839827903
<class 'numpy.float64'>
46.5821281591
<class 'numpy.float64'>
46.5802751584
<class 'numpy.float64'>
46.5784237854
<class 'numpy.float64'>
46.5765740372
<class 'numpy.float64'>
46.5747259112
<class 'numpy.float64'>
46.5728794043
<class 'numpy.float64'>
46.571034514
<class 'numpy.float64'>
46.5691912373
<class 'numpy.float64'>
46.5673495715
<class 'numpy.float64'>
46.5655095138
<class 'numpy.float64'>
46.5636710614
<class 'numpy.float64'>
46.5618342116
<class 'numpy.float64'>
46.5599989616
<class 'numpy.float64'>
46.5581653087
<class 'numpy.float64'>
46.5563332501
<class 'numpy.float64'>
46.554502783
<class 'numpy.float64'>
46.5526739048
<class 'numpy.float64'>
46.5508466128
<class 'numpy.float64'>
46.5490209041
<class 'numpy.float64'>
46.5471967761
<class 'numpy.float64'>
46.5453742261
<class 'numpy.float64'>
46.5435532514
<class 'numpy.float64'>
46.5417338493
<class 'numpy.float64'>
46.539916017
<class 'numpy.float64'>
46.538099752
<class 'numpy.float64'>
46.5362850516
<class 'numpy.float64'>
46.534471913
<class 'numpy.float64'>
46.5326603337
<class 'numpy.float64'>
46.5308503109
<class 'numpy.float64'>
46.5290418421
<class 'numpy.float64'>
46.5272349245
<class 'numpy.float64'>
46.5254295556
<class 'numpy.float64'>
46.5236257326
<class 'numpy.float64'>
46.5218234531
<class 'numpy.float64'>
46.5200227143
<class 'numpy.float64'>
46.5182235137
<class 'numpy.float64'>
46.5164258487
<class 'numpy.float64'>
46.5146297166
<class 'numpy.float64'>
46.5128351148
<class 'numpy.float64'>
46.5110420409
<class 'numpy.float64'>
46.5092504921
<class 'numpy.float64'>
46.507460466
<class 'numpy.float64'>
46.5056719599
<class 'numpy.float64'>
46.5038849713
<class 'numpy.float64'>
46.5020994976
<class 'numpy.float64'>
46.5003155363
<class 'numpy.float64'>
46.4985330849
<class 'numpy.float64'>
46.4967521408
<class 'numpy.float64'>
46.4949727014
<class 'numpy.float64'>
46.4931947643
<class 'numpy.float64'>
46.4914183269
<class 'numpy.float64'>
46.4896433867
<class 'numpy.float64'>
46.4878699412
<class 'numpy.float64'>
46.4860979879
<class 'numpy.float64'>
46.4843275243
<class 'numpy.float64'>
46.4825585479
<class 'numpy.float64'>
46.4807910562
<class 'numpy.float64'>
46.4790250468
<class 'numpy.float64'>
46.4772605171
<class 'numpy.float64'>
46.4754974648
<class 'numpy.float64'>
46.4737358873
<class 'numpy.float64'>
46.4719757821
<class 'numpy.float64'>
46.4702171469
<class 'numpy.float64'>
46.4684599792
<class 'numpy.float64'>
46.4667042766
<class 'numpy.float64'>
46.4649500365
<class 'numpy.float64'>
46.4631972566
<class 'numpy.float64'>
46.4614459345
<class 'numpy.float64'>
46.4596960678
<class 'numpy.float64'>
46.457947654
<class 'numpy.float64'>
46.4562006907
<class 'numpy.float64'>
46.4544551755
<class 'numpy.float64'>
46.4527111061
<class 'numpy.float64'>
46.45096848
<class 'numpy.float64'>
46.4492272949
<class 'numpy.float64'>
46.4474875484
<class 'numpy.float64'>
46.4457492381
<class 'numpy.float64'>
46.4440123616
<class 'numpy.float64'>
46.4422769167
<class 'numpy.float64'>
46.4405429008
<class 'numpy.float64'>
46.4388103118
<class 'numpy.float64'>
46.4370791471
<class 'numpy.float64'>
46.4353494046
<class 'numpy.float64'>
46.4336210818
<class 'numpy.float64'>
46.4318941765
<class 'numpy.float64'>
46.4301686862
<class 'numpy.float64'>
46.4284446088
<class 'numpy.float64'>
46.4267219418
<class 'numpy.float64'>
46.425000683
<class 'numpy.float64'>
46.42328083
<class 'numpy.float64'>
46.4215623807
<class 'numpy.float64'>
46.4198453326
<class 'numpy.float64'>
46.4181296835
<class 'numpy.float64'>
46.4164154311
<class 'numpy.float64'>
46.4147025731
<class 'numpy.float64'>
46.4129911073
<class 'numpy.float64'>
46.4112810315
<class 'numpy.float64'>
46.4095723432
<class 'numpy.float64'>
46.4078650403
<class 'numpy.float64'>
46.4061591206
<class 'numpy.float64'>
46.4044545817
<class 'numpy.float64'>
46.4027514215
<class 'numpy.float64'>
46.4010496377
<class 'numpy.float64'>
46.3993492281
<class 'numpy.float64'>
46.3976501905
<class 'numpy.float64'>
46.3959525226
<class 'numpy.float64'>
46.3942562222
<class 'numpy.float64'>
46.3925612871
<class 'numpy.float64'>
46.3908677151
<class 'numpy.float64'>
46.3891755041
<class 'numpy.float64'>
46.3874846517
<class 'numpy.float64'>
46.3857951559
<class 'numpy.float64'>
46.3841070145
<class 'numpy.float64'>
46.3824202252
<class 'numpy.float64'>
46.3807347859
<class 'numpy.float64'>
46.3790506944
<class 'numpy.float64'>
46.3773679486
<class 'numpy.float64'>
46.3756865462
<class 'numpy.float64'>
46.3740064852
<class 'numpy.float64'>
46.3723277634
<class 'numpy.float64'>
46.3706503787
<class 'numpy.float64'>
46.3689743288
<class 'numpy.float64'>
46.3672996117
<class 'numpy.float64'>
46.3656262253
<class 'numpy.float64'>
46.3639541674
<class 'numpy.float64'>
46.3622834358
<class 'numpy.float64'>
46.3606140286
<class 'numpy.float64'>
46.3589459435
<class 'numpy.float64'>
46.3572791784
<class 'numpy.float64'>
46.3556137314
<class 'numpy.float64'>
46.3539496001
<class 'numpy.float64'>
46.3522867827
<class 'numpy.float64'>
46.3506252769
<class 'numpy.float64'>
46.3489650807
<class 'numpy.float64'>
46.347306192
<class 'numpy.float64'>
46.3456486088
<class 'numpy.float64'>
46.343992329
<class 'numpy.float64'>
46.3423373504
<class 'numpy.float64'>
46.3406836711
<class 'numpy.float64'>
46.339031289
<class 'numpy.float64'>
46.3373802021
<class 'numpy.float64'>
46.3357304082
<class 'numpy.float64'>
46.3340819054
<class 'numpy.float64'>
46.3324346916
<class 'numpy.float64'>
46.3307887648
<class 'numpy.float64'>
46.329144123
<class 'numpy.float64'>
46.3275007641
<class 'numpy.float64'>
46.3258586861
<class 'numpy.float64'>
46.324217887
<class 'numpy.float64'>
46.3225783648
<class 'numpy.float64'>
46.3209401174
<class 'numpy.float64'>
46.319303143
<class 'numpy.float64'>
46.3176674395
<class 'numpy.float64'>
46.3160330048
<class 'numpy.float64'>
46.3143998371
<class 'numpy.float64'>
46.3127679343
<class 'numpy.float64'>
46.3111372944
<class 'numpy.float64'>
46.3095079155
<class 'numpy.float64'>
46.3078797956
<class 'numpy.float64'>
46.3062529327
<class 'numpy.float64'>
46.3046273249
<class 'numpy.float64'>
46.3030029703
<class 'numpy.float64'>
46.3013798668
<class 'numpy.float64'>
46.2997580125
<class 'numpy.float64'>
46.2981374055
<class 'numpy.float64'>
46.2965180438
<class 'numpy.float64'>
46.2948999255
<class 'numpy.float64'>
46.2932830487
<class 'numpy.float64'>
46.2916674114
<class 'numpy.float64'>
46.2900530117
<class 'numpy.float64'>
46.2884398477
<class 'numpy.float64'>
46.2868279175
<class 'numpy.float64'>
46.2852172192
<class 'numpy.float64'>
46.2836077509
<class 'numpy.float64'>
46.2819995106
<class 'numpy.float64'>
46.2803924964
<class 'numpy.float64'>
46.2787867066
<class 'numpy.float64'>
46.2771821391
<class 'numpy.float64'>
46.2755787922
<class 'numpy.float64'>
46.2739766638
<class 'numpy.float64'>
46.2723757522
<class 'numpy.float64'>
46.2707760555
<class 'numpy.float64'>
46.2691775718
<class 'numpy.float64'>
46.2675802992
<class 'numpy.float64'>
46.2659842358
<class 'numpy.float64'>
46.2643893799
<class 'numpy.float64'>
46.2627957296
<class 'numpy.float64'>
46.261203283
<class 'numpy.float64'>
46.2596120382
<class 'numpy.float64'>
46.2580219935
<class 'numpy.float64'>
46.256433147
<class 'numpy.float64'>
46.2548454969
<class 'numpy.float64'>
46.2532590413
<class 'numpy.float64'>
46.2516737784
<class 'numpy.float64'>
46.2500897064
<class 'numpy.float64'>
46.2485068235
<class 'numpy.float64'>
46.2469251278
<class 'numpy.float64'>
46.2453446176
<class 'numpy.float64'>
46.243765291
<class 'numpy.float64'>
46.2421871463
<class 'numpy.float64'>
46.2406101817
<class 'numpy.float64'>
46.2390343953
<class 'numpy.float64'>
46.2374597854
<class 'numpy.float64'>
46.2358863502
<class 'numpy.float64'>
46.2343140878
<class 'numpy.float64'>
46.2327429967
<class 'numpy.float64'>
46.2311730748
<class 'numpy.float64'>
46.2296043206
<class 'numpy.float64'>
46.2280367321
<class 'numpy.float64'>
46.2264703078
<class 'numpy.float64'>
46.2249050457
<class 'numpy.float64'>
46.2233409442
<class 'numpy.float64'>
46.2217780014
<class 'numpy.float64'>
46.2202162157
<class 'numpy.float64'>
46.2186555853
<class 'numpy.float64'>
46.2170961085
<class 'numpy.float64'>
46.2155377835
<class 'numpy.float64'>
46.2139806086
<class 'numpy.float64'>
46.2124245821
<class 'numpy.float64'>
46.2108697022
<class 'numpy.float64'>
46.2093159673
<class 'numpy.float64'>
46.2077633755
<class 'numpy.float64'>
46.2062119253
<class 'numpy.float64'>
46.2046616148
<class 'numpy.float64'>
46.2031124424
<class 'numpy.float64'>
46.2015644064
<class 'numpy.float64'>
46.2000175051
<class 'numpy.float64'>
46.1984717367
<class 'numpy.float64'>
46.1969270997
<class 'numpy.float64'>
46.1953835922
<class 'numpy.float64'>
46.1938412127
<class 'numpy.float64'>
46.1922999593
<class 'numpy.float64'>
46.1907598306
<class 'numpy.float64'>
46.1892208247
<class 'numpy.float64'>
46.1876829401
<class 'numpy.float64'>
46.186146175
<class 'numpy.float64'>
46.1846105278
<class 'numpy.float64'>
46.1830759968
<class 'numpy.float64'>
46.1815425804
<class 'numpy.float64'>
46.1800102769
<class 'numpy.float64'>
46.1784790847
<class 'numpy.float64'>
46.1769490021
<class 'numpy.float64'>
46.1754200275
<class 'numpy.float64'>
46.1738921592
<class 'numpy.float64'>
46.1723653957
<class 'numpy.float64'>
46.1708397352
<class 'numpy.float64'>
46.1693151762
<class 'numpy.float64'>
46.167791717
<class 'numpy.float64'>
46.166269356
<class 'numpy.float64'>
46.1647480916
<class 'numpy.float64'>
46.1632279222
<class 'numpy.float64'>
46.1617088462
<class 'numpy.float64'>
46.1601908619
<class 'numpy.float64'>
46.1586739677
<class 'numpy.float64'>
46.1571581621
<class 'numpy.float64'>
46.1556434435
<class 'numpy.float64'>
46.1541298102
<class 'numpy.float64'>
46.1526172607
<class 'numpy.float64'>
46.1511057934
<class 'numpy.float64'>
46.1495954067
<class 'numpy.float64'>
46.148086099
<class 'numpy.float64'>
46.1465778687
<class 'numpy.float64'>
46.1450707143
<class 'numpy.float64'>
46.1435646342
<class 'numpy.float64'>
46.1420596268
<class 'numpy.float64'>
46.1405556906
<class 'numpy.float64'>
46.1390528241
<class 'numpy.float64'>
46.1375510255
<class 'numpy.float64'>
46.1360502935
<class 'numpy.float64'>
46.1345506263
<class 'numpy.float64'>
46.1330520226
<class 'numpy.float64'>
46.1315544808
<class 'numpy.float64'>
46.1300579992
<class 'numpy.float64'>
46.1285625765
<class 'numpy.float64'>
46.1270682109
<class 'numpy.float64'>
46.1255749011
<class 'numpy.float64'>
46.1240826454
<class 'numpy.float64'>
46.1225914424
<class 'numpy.float64'>
46.1211012905
<class 'numpy.float64'>
46.1196121883
<class 'numpy.float64'>
46.1181241341
<class 'numpy.float64'>
46.1166371265
<class 'numpy.float64'>
46.115151164
<class 'numpy.float64'>
46.1136662451
<class 'numpy.float64'>
46.1121823682
<class 'numpy.float64'>
46.1106995319
<class 'numpy.float64'>
46.1092177347
<class 'numpy.float64'>
46.107736975
<class 'numpy.float64'>
46.1062572514
<class 'numpy.float64'>
46.1047785624
<class 'numpy.float64'>
46.1033009066
<class 'numpy.float64'>
46.1018242823
<class 'numpy.float64'>
46.1003486882
<class 'numpy.float64'>
46.0988741228
<class 'numpy.float64'>
46.0974005846
<class 'numpy.float64'>
46.0959280721
<class 'numpy.float64'>
46.0944565839
<class 'numpy.float64'>
46.0929861185
<class 'numpy.float64'>
46.0915166745
<class 'numpy.float64'>
46.0900482503
<class 'numpy.float64'>
46.0885808446
<class 'numpy.float64'>
46.0871144558
<class 'numpy.float64'>
46.0856490826
<class 'numpy.float64'>
46.0841847235
<class 'numpy.float64'>
46.082721377
<class 'numpy.float64'>
46.0812590418
<class 'numpy.float64'>
46.0797977163
<class 'numpy.float64'>
46.0783373991
<class 'numpy.float64'>
46.0768780889
<class 'numpy.float64'>
46.0754197841
<class 'numpy.float64'>
46.0739624834
<class 'numpy.float64'>
46.0725061854
<class 'numpy.float64'>
46.0710508885
<class 'numpy.float64'>
46.0695965915
<class 'numpy.float64'>
46.0681432928
<class 'numpy.float64'>
46.0666909911
<class 'numpy.float64'>
46.065239685
<class 'numpy.float64'>
46.063789373
<class 'numpy.float64'>
46.0623400539
<class 'numpy.float64'>
46.0608917261
<class 'numpy.float64'>
46.0594443882
<class 'numpy.float64'>
46.057998039
<class 'numpy.float64'>
46.0565526769
<class 'numpy.float64'>
46.0551083006
<class 'numpy.float64'>
46.0536649088
<class 'numpy.float64'>
46.0522224999
<class 'numpy.float64'>
46.0507810728
<class 'numpy.float64'>
46.0493406259
<class 'numpy.float64'>
46.0479011579
<class 'numpy.float64'>
46.0464626675
<class 'numpy.float64'>
46.0450251532
<class 'numpy.float64'>
46.0435886138
<class 'numpy.float64'>
46.0421530477
<class 'numpy.float64'>
46.0407184538
<class 'numpy.float64'>
46.0392848306
<class 'numpy.float64'>
46.0378521768
<class 'numpy.float64'>
46.036420491
<class 'numpy.float64'>
46.0349897718
<class 'numpy.float64'>
46.033560018
<class 'numpy.float64'>
46.0321312282
<class 'numpy.float64'>
46.0307034011
<class 'numpy.float64'>
46.0292765352
<class 'numpy.float64'>
46.0278506293
<class 'numpy.float64'>
46.0264256821
<class 'numpy.float64'>
46.0250016921
<class 'numpy.float64'>
46.0235786582
<class 'numpy.float64'>
46.0221565789
<class 'numpy.float64'>
46.020735453
<class 'numpy.float64'>
46.0193152791
<class 'numpy.float64'>
46.0178960559
<class 'numpy.float64'>
46.016477782
<class 'numpy.float64'>
46.0150604563
<class 'numpy.float64'>
46.0136440773
<class 'numpy.float64'>
46.0122286439
<class 'numpy.float64'>
46.0108141545
<class 'numpy.float64'>
46.0094006081
<class 'numpy.float64'>
46.0079880032
<class 'numpy.float64'>
46.0065763386
<class 'numpy.float64'>
46.0051656129
<class 'numpy.float64'>
46.003755825
<class 'numpy.float64'>
46.0023469734
<class 'numpy.float64'>
46.000939057
<class 'numpy.float64'>
45.9995320744
<class 'numpy.float64'>
45.9981260244
<class 'numpy.float64'>
45.9967209057
<class 'numpy.float64'>
45.9953167169
<class 'numpy.float64'>
45.9939134569
<class 'numpy.float64'>
45.9925111244
<class 'numpy.float64'>
45.991109718
<class 'numpy.float64'>
45.9897092366
<class 'numpy.float64'>
45.9883096789
<class 'numpy.float64'>
45.9869110435
<class 'numpy.float64'>
45.9855133294
<class 'numpy.float64'>
45.9841165351
<class 'numpy.float64'>
45.9827206595
<class 'numpy.float64'>
45.9813257012
<class 'numpy.float64'>
45.9799316592
<class 'numpy.float64'>
45.978538532
<class 'numpy.float64'>
45.9771463185
<class 'numpy.float64'>
45.9757550174
<class 'numpy.float64'>
45.9743646276
<class 'numpy.float64'>
45.9729751476
<class 'numpy.float64'>
45.9715865764
<class 'numpy.float64'>
45.9701989127
<class 'numpy.float64'>
45.9688121553
<class 'numpy.float64'>
45.9674263029
<class 'numpy.float64'>
45.9660413543
<class 'numpy.float64'>
45.9646573083
<class 'numpy.float64'>
45.9632741637
<class 'numpy.float64'>
45.9618919193
<class 'numpy.float64'>
45.9605105738
<class 'numpy.float64'>
45.959130126
<class 'numpy.float64'>
45.9577505748
<class 'numpy.float64'>
45.956371919
<class 'numpy.float64'>
45.9549941572
<class 'numpy.float64'>
45.9536172884
<class 'numpy.float64'>
45.9522413113
<class 'numpy.float64'>
45.9508662247
<class 'numpy.float64'>
45.9494920275
<class 'numpy.float64'>
45.9481187184
<class 'numpy.float64'>
45.9467462962
<class 'numpy.float64'>
45.9453747599
<class 'numpy.float64'>
45.9440041081
<class 'numpy.float64'>
45.9426343397
<class 'numpy.float64'>
45.9412654535
<class 'numpy.float64'>
45.9398974484
<class 'numpy.float64'>
45.9385303232
<class 'numpy.float64'>
45.9371640766
<class 'numpy.float64'>
45.9357987076
<class 'numpy.float64'>
45.9344342149
<class 'numpy.float64'>
45.9330705974
<class 'numpy.float64'>
45.9317078539
<class 'numpy.float64'>
45.9303459833
<class 'numpy.float64'>
45.9289849845
<class 'numpy.float64'>
45.9276248561
<class 'numpy.float64'>
45.9262655972
<class 'numpy.float64'>
45.9249072064
<class 'numpy.float64'>
45.9235496828
<class 'numpy.float64'>
45.9221930251
<class 'numpy.float64'>
45.9208372323
<class 'numpy.float64'>
45.919482303
<class 'numpy.float64'>
45.9181282363
<class 'numpy.float64'>
45.9167750309
<class 'numpy.float64'>
45.9154226858
<class 'numpy.float64'>
45.9140711998
<class 'numpy.float64'>
45.9127205718
<class 'numpy.float64'>
45.9113708005
<class 'numpy.float64'>
45.910021885
<class 'numpy.float64'>
45.9086738241
<class 'numpy.float64'>
45.9073266166
<class 'numpy.float64'>
45.9059802615
<class 'numpy.float64'>
45.9046347576
<class 'numpy.float64'>
45.9032901038
<class 'numpy.float64'>
45.9019462989
<class 'numpy.float64'>
45.9006033419
<class 'numpy.float64'>
45.8992612317
<class 'numpy.float64'>
45.8979199672
<class 'numpy.float64'>
45.8965795472
<class 'numpy.float64'>
45.8952399706
<class 'numpy.float64'>
45.8939012363
<class 'numpy.float64'>
45.8925633433
<class 'numpy.float64'>
45.8912262905
<class 'numpy.float64'>
45.8898900766
<class 'numpy.float64'>
45.8885547008
<class 'numpy.float64'>
45.8872201617
<class 'numpy.float64'>
45.8858864585
<class 'numpy.float64'>
45.8845535899
<class 'numpy.float64'>
45.8832215549
<class 'numpy.float64'>
45.8818903524
<class 'numpy.float64'>
45.8805599814
<class 'numpy.float64'>
45.8792304407
<class 'numpy.float64'>
45.8779017292
<class 'numpy.float64'>
45.876573846
<class 'numpy.float64'>
45.8752467899
<class 'numpy.float64'>
45.8739205598
<class 'numpy.float64'>
45.8725951547
<class 'numpy.float64'>
45.8712705736
<class 'numpy.float64'>
45.8699468152
<class 'numpy.float64'>
45.8686238787
<class 'numpy.float64'>
45.8673017629
<class 'numpy.float64'>
45.8659804667
<class 'numpy.float64'>
45.8646599892
<class 'numpy.float64'>
45.8633403292
<class 'numpy.float64'>
45.8620214857
<class 'numpy.float64'>
45.8607034576
<class 'numpy.float64'>
45.859386244
<class 'numpy.float64'>
45.8580698437
<class 'numpy.float64'>
45.8567542557
<class 'numpy.float64'>
45.855439479
<class 'numpy.float64'>
45.8541255125
<class 'numpy.float64'>
45.8528123552
<class 'numpy.float64'>
45.851500006
<class 'numpy.float64'>
45.850188464
<class 'numpy.float64'>
45.848877728
<class 'numpy.float64'>
45.847567797
<class 'numpy.float64'>
45.8462586701
<class 'numpy.float64'>
45.8449503462
<class 'numpy.float64'>
45.8436428243
<class 'numpy.float64'>
45.8423361032
<class 'numpy.float64'>
45.8410301821
<class 'numpy.float64'>
45.8397250599
<class 'numpy.float64'>
45.8384207356
<class 'numpy.float64'>
45.8371172082
<class 'numpy.float64'>
45.8358144766
<class 'numpy.float64'>
45.8345125398
<class 'numpy.float64'>
45.8332113969
<class 'numpy.float64'>
45.8319110468
<class 'numpy.float64'>
45.8306114885
<class 'numpy.float64'>
45.829312721
<class 'numpy.float64'>
45.8280147433
<class 'numpy.float64'>
45.8267175545
<class 'numpy.float64'>
45.8254211534
<class 'numpy.float64'>
45.8241255391
<class 'numpy.float64'>
45.8228307107
<class 'numpy.float64'>
45.8215366671
<class 'numpy.float64'>
45.8202434073
<class 'numpy.float64'>
45.8189509304
<class 'numpy.float64'>
45.8176592353
<class 'numpy.float64'>
45.8163683211
<class 'numpy.float64'>
45.8150781867
<class 'numpy.float64'>
45.8137888313
<class 'numpy.float64'>
45.8125002538
<class 'numpy.float64'>
45.8112124532
<class 'numpy.float64'>
45.8099254286
<class 'numpy.float64'>
45.808639179
<class 'numpy.float64'>
45.8073537034
<class 'numpy.float64'>
45.8060690009
<class 'numpy.float64'>
45.8047850704
<class 'numpy.float64'>
45.8035019111
<class 'numpy.float64'>
45.8022195219
<class 'numpy.float64'>
45.8009379018
<class 'numpy.float64'>
45.79965705
<class 'numpy.float64'>
45.7983769655
<class 'numpy.float64'>
45.7970976472
<class 'numpy.float64'>
45.7958190943
<class 'numpy.float64'>
45.7945413058
<class 'numpy.float64'>
45.7932642807
<class 'numpy.float64'>
45.7919880181
<class 'numpy.float64'>
45.790712517
<class 'numpy.float64'>
45.7894377765
<class 'numpy.float64'>
45.7881637956
<class 'numpy.float64'>
45.7868905734
<class 'numpy.float64'>
45.785618109
<class 'numpy.float64'>
45.7843464014
<class 'numpy.float64'>
45.7830754496
<class 'numpy.float64'>
45.7818052528
<class 'numpy.float64'>
45.78053581
<class 'numpy.float64'>
45.7792671202
<class 'numpy.float64'>
45.7779991826
<class 'numpy.float64'>
45.7767319962
<class 'numpy.float64'>
45.77546556
<class 'numpy.float64'>
45.7741998732
<class 'numpy.float64'>
45.7729349347
<class 'numpy.float64'>
45.7716707438
<class 'numpy.float64'>
45.7704072995
<class 'numpy.float64'>
45.7691446008
<class 'numpy.float64'>
45.7678826468
<class 'numpy.float64'>
45.7666214366
<class 'numpy.float64'>
45.7653609693
<class 'numpy.float64'>
45.7641012441
<class 'numpy.float64'>
45.7628422598
<class 'numpy.float64'>
45.7615840158
<class 'numpy.float64'>
45.760326511
<class 'numpy.float64'>
45.7590697445
<class 'numpy.float64'>
45.7578137155
<class 'numpy.float64'>
45.756558423
<class 'numpy.float64'>
45.7553038661
<class 'numpy.float64'>
45.754050044
<class 'numpy.float64'>
45.7527969557
<class 'numpy.float64'>
45.7515446004
<class 'numpy.float64'>
45.750292977
<class 'numpy.float64'>
45.7490420849
<class 'numpy.float64'>
45.7477919229
<class 'numpy.float64'>
45.7465424904
<class 'numpy.float64'>
45.7452937863
<class 'numpy.float64'>
45.7440458098
<class 'numpy.float64'>
45.74279856
<class 'numpy.float64'>
45.741552036
<class 'numpy.float64'>
45.7403062369
<class 'numpy.float64'>
45.7390611619
<class 'numpy.float64'>
45.73781681
<class 'numpy.float64'>
45.7365731805
<class 'numpy.float64'>
45.7353302724
<class 'numpy.float64'>
45.7340880848
<class 'numpy.float64'>
45.7328466168
<class 'numpy.float64'>
45.7316058677
<class 'numpy.float64'>
45.7303658365
<class 'numpy.float64'>
45.7291265224
<class 'numpy.float64'>
45.7278879244
<class 'numpy.float64'>
45.7266500418
<class 'numpy.float64'>
45.7254128736
<class 'numpy.float64'>
45.7241764191
<class 'numpy.float64'>
45.7229406772
<class 'numpy.float64'>
45.7217056473
<class 'numpy.float64'>
45.7204713284
<class 'numpy.float64'>
45.7192377196
<class 'numpy.float64'>
45.7180048202
<class 'numpy.float64'>
45.7167726292
<class 'numpy.float64'>
45.7155411458
<class 'numpy.float64'>
45.7143103691
<class 'numpy.float64'>
45.7130802984
<class 'numpy.float64'>
45.7118509328
<class 'numpy.float64'>
45.7106222713
<class 'numpy.float64'>
45.7093943132
<class 'numpy.float64'>
45.7081670577
<class 'numpy.float64'>
45.7069405038
<class 'numpy.float64'>
45.7057146508
<class 'numpy.float64'>
45.7044894978
<class 'numpy.float64'>
45.703265044
<class 'numpy.float64'>
45.7020412885
<class 'numpy.float64'>
45.7008182306
<class 'numpy.float64'>
45.6995958693
<class 'numpy.float64'>
45.6983742038
<class 'numpy.float64'>
45.6971532334
<class 'numpy.float64'>
45.6959329571
<class 'numpy.float64'>
45.6947133742
<class 'numpy.float64'>
45.6934944839
<class 'numpy.float64'>
45.6922762853
<class 'numpy.float64'>
45.6910587775
<class 'numpy.float64'>
45.6898419598
<class 'numpy.float64'>
45.6886258314
<class 'numpy.float64'>
45.6874103914
<class 'numpy.float64'>
45.6861956391
<class 'numpy.float64'>
45.6849815735
<class 'numpy.float64'>
45.683768194
<class 'numpy.float64'>
45.6825554996
<class 'numpy.float64'>
45.6813434896
<class 'numpy.float64'>
45.6801321631
<class 'numpy.float64'>
45.6789215195
<class 'numpy.float64'>
45.6777115577
<class 'numpy.float64'>
45.6765022771
<class 'numpy.float64'>
45.6752936769
<class 'numpy.float64'>
45.6740857562
<class 'numpy.float64'>
45.6728785142
<class 'numpy.float64'>
45.6716719502
<class 'numpy.float64'>
45.6704660634
<class 'numpy.float64'>
45.6692608528
<class 'numpy.float64'>
45.6680563179
<class 'numpy.float64'>
45.6668524577
<class 'numpy.float64'>
45.6656492714
<class 'numpy.float64'>
45.6644467584
<class 'numpy.float64'>
45.6632449177
<class 'numpy.float64'>
45.6620437487
<class 'numpy.float64'>
45.6608432505
<class 'numpy.float64'>
45.6596434223
<class 'numpy.float64'>
45.6584442633
<class 'numpy.float64'>
45.6572457728
<class 'numpy.float64'>
45.65604795
<class 'numpy.float64'>
45.6548507942
<class 'numpy.float64'>
45.6536543044
<class 'numpy.float64'>
45.65245848
<class 'numpy.float64'>
45.6512633202
<class 'numpy.float64'>
45.6500688242
<class 'numpy.float64'>
45.6488749912
<class 'numpy.float64'>
45.6476818205
<class 'numpy.float64'>
45.6464893113
<class 'numpy.float64'>
45.6452974628
<class 'numpy.float64'>
45.6441062743
<class 'numpy.float64'>
45.6429157449
<class 'numpy.float64'>
45.641725874
<class 'numpy.float64'>
45.6405366608
<class 'numpy.float64'>
45.6393481044
<class 'numpy.float64'>
45.6381602042
<class 'numpy.float64'>
45.6369729594
<class 'numpy.float64'>
45.6357863692
<class 'numpy.float64'>
45.6346004328
<class 'numpy.float64'>
45.6334151496
<class 'numpy.float64'>
45.6322305188
<class 'numpy.float64'>
45.6310465395
<class 'numpy.float64'>
45.6298632111
<class 'numpy.float64'>
45.6286805328
<class 'numpy.float64'>
45.6274985039
<class 'numpy.float64'>
45.6263171236
<class 'numpy.float64'>
45.6251363912
<class 'numpy.float64'>
45.6239563058
<class 'numpy.float64'>
45.6227768669
<class 'numpy.float64'>
45.6215980736
<class 'numpy.float64'>
45.6204199251
<class 'numpy.float64'>
45.6192424209
<class 'numpy.float64'>
45.61806556
<class 'numpy.float64'>
45.6168893418
<class 'numpy.float64'>
45.6157137656
<class 'numpy.float64'>
45.6145388306
<class 'numpy.float64'>
45.613364536
<class 'numpy.float64'>
45.6121908812
<class 'numpy.float64'>
45.6110178654
<class 'numpy.float64'>
45.6098454879
<class 'numpy.float64'>
45.608673748
<class 'numpy.float64'>
45.6075026449
<class 'numpy.float64'>
45.6063321779
<class 'numpy.float64'>
45.6051623462
<class 'numpy.float64'>
45.6039931492
<class 'numpy.float64'>
45.6028245862
<class 'numpy.float64'>
45.6016566564
<class 'numpy.float64'>
45.600489359
<class 'numpy.float64'>
45.5993226934
<class 'numpy.float64'>
45.5981566589
<class 'numpy.float64'>
45.5969912548
<class 'numpy.float64'>
45.5958264802
<class 'numpy.float64'>
45.5946623346
<class 'numpy.float64'>
45.5934988172
<class 'numpy.float64'>
45.5923359273
<class 'numpy.float64'>
45.5911736641
<class 'numpy.float64'>
45.590012027
<class 'numpy.float64'>
45.5888510153
<class 'numpy.float64'>
45.5876906283
<class 'numpy.float64'>
45.5865308652
<class 'numpy.float64'>
45.5853717254
<class 'numpy.float64'>
45.5842132081
<class 'numpy.float64'>
45.5830553126
<class 'numpy.float64'>
45.5818980383
<class 'numpy.float64'>
45.5807413845
<class 'numpy.float64'>
45.5795853504
<class 'numpy.float64'>
45.5784299354
<class 'numpy.float64'>
45.5772751387
<class 'numpy.float64'>
45.5761209597
<class 'numpy.float64'>
45.5749673977
<class 'numpy.float64'>
45.5738144519
<class 'numpy.float64'>
45.5726621217
<class 'numpy.float64'>
45.5715104065
<class 'numpy.float64'>
45.5703593054
<class 'numpy.float64'>
45.5692088179
<class 'numpy.float64'>
45.5680589432
<class 'numpy.float64'>
45.5669096807
<class 'numpy.float64'>
45.5657610296
<class 'numpy.float64'>
45.5646129893
<class 'numpy.float64'>
45.5634655592
<class 'numpy.float64'>
45.5623187384
<class 'numpy.float64'>
45.5611725265
<class 'numpy.float64'>
45.5600269225
<class 'numpy.float64'>
45.558881926
<class 'numpy.float64'>
45.5577375362
<class 'numpy.float64'>
45.5565937525
<class 'numpy.float64'>
45.5554505741
<class 'numpy.float64'>
45.5543080004
<class 'numpy.float64'>
45.5531660307
<class 'numpy.float64'>
45.5520246644
<class 'numpy.float64'>
45.5508839008
<class 'numpy.float64'>
45.5497437392
<class 'numpy.float64'>
45.548604179
<class 'numpy.float64'>
45.5474652194
<class 'numpy.float64'>
45.5463268599
<class 'numpy.float64'>
45.5451890997
<class 'numpy.float64'>
45.5440519383
<class 'numpy.float64'>
45.5429153748
<class 'numpy.float64'>
45.5417794088
<class 'numpy.float64'>
45.5406440395
<class 'numpy.float64'>
45.5395092662
<class 'numpy.float64'>
45.5383750883
<class 'numpy.float64'>
45.5372415052
<class 'numpy.float64'>
45.5361085162
<class 'numpy.float64'>
45.5349761206
<class 'numpy.float64'>
45.5338443178
<class 'numpy.float64'>
45.5327131072
<class 'numpy.float64'>
45.5315824881
<class 'numpy.float64'>
45.5304524597
<class 'numpy.float64'>
45.5293230216
<class 'numpy.float64'>
45.528194173
<class 'numpy.float64'>
45.5270659134
<class 'numpy.float64'>
45.5259382419
<class 'numpy.float64'>
45.5248111581
<class 'numpy.float64'>
45.5236846613
<class 'numpy.float64'>
45.5225587508
<class 'numpy.float64'>
45.521433426
<class 'numpy.float64'>
45.5203086862
<class 'numpy.float64'>
45.5191845308
<class 'numpy.float64'>
45.5180609592
<class 'numpy.float64'>
45.5169379708
<class 'numpy.float64'>
45.5158155648
<class 'numpy.float64'>
45.5146937407
<class 'numpy.float64'>
45.5135724979
<class 'numpy.float64'>
45.5124518356
<class 'numpy.float64'>
45.5113317534
<class 'numpy.float64'>
45.5102122504
<class 'numpy.float64'>
45.5090933262
<class 'numpy.float64'>
45.5079749801
<class 'numpy.float64'>
45.5068572114
<class 'numpy.float64'>
45.5057400195
<class 'numpy.float64'>
45.5046234039
<class 'numpy.float64'>
45.5035073638
<class 'numpy.float64'>
45.5023918988
<class 'numpy.float64'>
45.501277008
<class 'numpy.float64'>
45.500162691
<class 'numpy.float64'>
45.499048947
<class 'numpy.float64'>
45.4979357756
<class 'numpy.float64'>
45.496823176
<class 'numpy.float64'>
45.4957111477
<class 'numpy.float64'>
45.49459969
<class 'numpy.float64'>
45.4934888023
<class 'numpy.float64'>
45.492378484
<class 'numpy.float64'>
45.4912687345
<class 'numpy.float64'>
45.4901595532
<class 'numpy.float64'>
45.4890509395
<class 'numpy.float64'>
45.4879428927
<class 'numpy.float64'>
45.4868354122
<class 'numpy.float64'>
45.4857284976
<class 'numpy.float64'>
45.484622148
<class 'numpy.float64'>
45.483516363
<class 'numpy.float64'>
45.4824111419
<class 'numpy.float64'>
45.4813064842
<class 'numpy.float64'>
45.4802023891
<class 'numpy.float64'>
45.4790988562
<class 'numpy.float64'>
45.4779958848
<class 'numpy.float64'>
45.4768934743
<class 'numpy.float64'>
45.4757916242
<class 'numpy.float64'>
45.4746903337
<class 'numpy.float64'>
45.4735896024
<class 'numpy.float64'>
45.4724894297
<class 'numpy.float64'>
45.4713898148
<class 'numpy.float64'>
45.4702907574
<class 'numpy.float64'>
45.4691922566
<class 'numpy.float64'>
45.468094312
<class 'numpy.float64'>
45.466996923
<class 'numpy.float64'>
45.465900089
<class 'numpy.float64'>
45.4648038093
<class 'numpy.float64'>
45.4637080835
<class 'numpy.float64'>
45.4626129109
<class 'numpy.float64'>
45.4615182909
<class 'numpy.float64'>
45.4604242229
<class 'numpy.float64'>
45.4593307064
<class 'numpy.float64'>
45.4582377408
<class 'numpy.float64'>
45.4571453254
<class 'numpy.float64'>
45.4560534598
<class 'numpy.float64'>
45.4549621433
<class 'numpy.float64'>
45.4538713754
<class 'numpy.float64'>
45.4527811554
<class 'numpy.float64'>
45.4516914828
<class 'numpy.float64'>
45.4506023571
<class 'numpy.float64'>
45.4495137776
<class 'numpy.float64'>
45.4484257437
<class 'numpy.float64'>
45.447338255
<class 'numpy.float64'>
45.4462513107
<class 'numpy.float64'>
45.4451649105
<class 'numpy.float64'>
45.4440790536
<class 'numpy.float64'>
45.4429937395
<class 'numpy.float64'>
45.4419089676
<class 'numpy.float64'>
45.4408247374
<class 'numpy.float64'>
45.4397410483
<class 'numpy.float64'>
45.4386578998
<class 'numpy.float64'>
45.4375752912
<class 'numpy.float64'>
45.436493222
<class 'numpy.float64'>
45.4354116917
<class 'numpy.float64'>
45.4343306996
<class 'numpy.float64'>
45.4332502453
<class 'numpy.float64'>
45.4321703281
<class 'numpy.float64'>
45.4310909475
<class 'numpy.float64'>
45.430012103
<class 'numpy.float64'>
45.4289337939
<class 'numpy.float64'>
45.4278560197
<class 'numpy.float64'>
45.42677878
<class 'numpy.float64'>
45.425702074
<class 'numpy.float64'>
45.4246259012
<class 'numpy.float64'>
45.4235502612
<class 'numpy.float64'>
45.4224751533
<class 'numpy.float64'>
45.421400577
<class 'numpy.float64'>
45.4203265317
<class 'numpy.float64'>
45.419253017
<class 'numpy.float64'>
45.4181800321
<class 'numpy.float64'>
45.4171075767
<class 'numpy.float64'>
45.4160356501
<class 'numpy.float64'>
45.4149642518
<class 'numpy.float64'>
45.4138933812
<class 'numpy.float64'>
45.4128230379
<class 'numpy.float64'>
45.4117532212
<class 'numpy.float64'>
45.4106839306
<class 'numpy.float64'>
45.4096151656
<class 'numpy.float64'>
45.4085469257
<class 'numpy.float64'>
45.4074792102
<class 'numpy.float64'>
45.4064120187
<class 'numpy.float64'>
45.4053453506
<class 'numpy.float64'>
45.4042792053
<class 'numpy.float64'>
45.4032135824
<class 'numpy.float64'>
45.4021484813
<class 'numpy.float64'>
45.4010839015
<class 'numpy.float64'>
45.4000198424
<class 'numpy.float64'>
45.3989563035
<class 'numpy.float64'>
45.3978932842
<class 'numpy.float64'>
45.3968307841
<class 'numpy.float64'>
45.3957688026
<class 'numpy.float64'>
45.3947073391
<class 'numpy.float64'>
45.3936463932
<class 'numpy.float64'>
45.3925859642
<class 'numpy.float64'>
45.3915260518
<class 'numpy.float64'>
45.3904666553
<class 'numpy.float64'>
45.3894077742
<class 'numpy.float64'>
45.388349408
<class 'numpy.float64'>
45.3872915562
<class 'numpy.float64'>
45.3862342183
<class 'numpy.float64'>
45.3851773936
<class 'numpy.float64'>
45.3841210818
<class 'numpy.float64'>
45.3830652823
<class 'numpy.float64'>
45.3820099945
<class 'numpy.float64'>
45.3809552179
<class 'numpy.float64'>
45.3799009521
<class 'numpy.float64'>
45.3788471964
<class 'numpy.float64'>
45.3777939505
<class 'numpy.float64'>
45.3767412137
<class 'numpy.float64'>
45.3756889856
<class 'numpy.float64'>
45.3746372656
<class 'numpy.float64'>
45.3735860532
<class 'numpy.float64'>
45.3725353479
<class 'numpy.float64'>
45.3714851492
<class 'numpy.float64'>
45.3704354566
<class 'numpy.float64'>
45.3693862696
<class 'numpy.float64'>
45.3683375876
<class 'numpy.float64'>
45.3672894102
<class 'numpy.float64'>
45.3662417368
<class 'numpy.float64'>
45.365194567
<class 'numpy.float64'>
45.3641479002
<class 'numpy.float64'>
45.3631017359
<class 'numpy.float64'>
45.3620560736
<class 'numpy.float64'>
45.3610109128
<class 'numpy.float64'>
45.359966253
<class 'numpy.float64'>
45.3589220937
<class 'numpy.float64'>
45.3578784345
<class 'numpy.float64'>
45.3568352747
<class 'numpy.float64'>
45.3557926139
<class 'numpy.float64'>
45.3547504516
<class 'numpy.float64'>
45.3537087873
<class 'numpy.float64'>
45.3526676205
<class 'numpy.float64'>
45.3516269507
<class 'numpy.float64'>
45.3505867774
<class 'numpy.float64'>
45.3495471001
<class 'numpy.float64'>
45.3485079183
<class 'numpy.float64'>
45.3474692315
<class 'numpy.float64'>
45.3464310392
<class 'numpy.float64'>
45.345393341
<class 'numpy.float64'>
45.3443561363
<class 'numpy.float64'>
45.3433194247
<class 'numpy.float64'>
45.3422832055
<class 'numpy.float64'>
45.3412474785
<class 'numpy.float64'>
45.340212243
<class 'numpy.float64'>
45.3391774986
<class 'numpy.float64'>
45.3381432448
<class 'numpy.float64'>
45.3371094811
<class 'numpy.float64'>
45.336076207
<class 'numpy.float64'>
45.3350434221
<class 'numpy.float64'>
45.3340111258
<class 'numpy.float64'>
45.3329793176
<class 'numpy.float64'>
45.3319479971
<class 'numpy.float64'>
45.3309171639
<class 'numpy.float64'>
45.3298868173
<class 'numpy.float64'>
45.328856957
<class 'numpy.float64'>
45.3278275824
<class 'numpy.float64'>
45.3267986932
<class 'numpy.float64'>
45.3257702887
<class 'numpy.float64'>
45.3247423685
<class 'numpy.float64'>
45.3237149321
<class 'numpy.float64'>
45.3226879791
<class 'numpy.float64'>
45.321661509
<class 'numpy.float64'>
45.3206355213
<class 'numpy.float64'>
45.3196100155
<class 'numpy.float64'>
45.3185849912
<class 'numpy.float64'>
45.3175604479
<class 'numpy.float64'>
45.3165363851
<class 'numpy.float64'>
45.3155128023
<class 'numpy.float64'>
45.3144896991
<class 'numpy.float64'>
45.313467075
<class 'numpy.float64'>
45.3124449295
<class 'numpy.float64'>
45.3114232622
<class 'numpy.float64'>
45.3104020726
<class 'numpy.float64'>
45.3093813601
<class 'numpy.float64'>
45.3083611245
<class 'numpy.float64'>
45.3073413651
<class 'numpy.float64'>
45.3063220816
<class 'numpy.float64'>
45.3053032734
<class 'numpy.float64'>
45.3042849401
<class 'numpy.float64'>
45.3032670812
<class 'numpy.float64'>
45.3022496964
<class 'numpy.float64'>
45.301232785
<class 'numpy.float64'>
45.3002163466
<class 'numpy.float64'>
45.2992003809
<class 'numpy.float64'>
45.2981848872
<class 'numpy.float64'>
45.2971698653
<class 'numpy.float64'>
45.2961553145
<class 'numpy.float64'>
45.2951412345
<class 'numpy.float64'>
45.2941276248
<class 'numpy.float64'>
45.2931144849
<class 'numpy.float64'>
45.2921018143
<class 'numpy.float64'>
45.2910896127
<class 'numpy.float64'>
45.2900778796
<class 'numpy.float64'>
45.2890666145
<class 'numpy.float64'>
45.288055817
<class 'numpy.float64'>
45.2870454865
<class 'numpy.float64'>
45.2860356227
<class 'numpy.float64'>
45.2850262251
<class 'numpy.float64'>
45.2840172933
<class 'numpy.float64'>
45.2830088268
<class 'numpy.float64'>
45.2820008251
<class 'numpy.float64'>
45.2809932879
<class 'numpy.float64'>
45.2799862146
<class 'numpy.float64'>
45.2789796048
<class 'numpy.float64'>
45.2779734581
<class 'numpy.float64'>
45.276967774
<class 'numpy.float64'>
Cost after iteration 4000: 45.276968
45.275962552
<class 'numpy.float64'>
45.2749577918
<class 'numpy.float64'>
45.2739534929
<class 'numpy.float64'>
45.2729496549
<class 'numpy.float64'>
45.2719462772
<class 'numpy.float64'>
45.2709433595
<class 'numpy.float64'>
45.2699409013
<class 'numpy.float64'>
45.2689389022
<class 'numpy.float64'>
45.2679373617
<class 'numpy.float64'>
45.2669362795
<class 'numpy.float64'>
45.265935655
<class 'numpy.float64'>
45.2649354878
<class 'numpy.float64'>
45.2639357775
<class 'numpy.float64'>
45.2629365236
<class 'numpy.float64'>
45.2619377258
<class 'numpy.float64'>
45.2609393835
<class 'numpy.float64'>
45.2599414964
<class 'numpy.float64'>
45.258944064
<class 'numpy.float64'>
45.2579470858
<class 'numpy.float64'>
45.2569505615
<class 'numpy.float64'>
45.2559544906
<class 'numpy.float64'>
45.2549588727
<class 'numpy.float64'>
45.2539637074
<class 'numpy.float64'>
45.2529689941
<class 'numpy.float64'>
45.2519747326
<class 'numpy.float64'>
45.2509809223
<class 'numpy.float64'>
45.2499875628
<class 'numpy.float64'>
45.2489946538
<class 'numpy.float64'>
45.2480021947
<class 'numpy.float64'>
45.2470101851
<class 'numpy.float64'>
45.2460186247
<class 'numpy.float64'>
45.245027513
<class 'numpy.float64'>
45.2440368496
<class 'numpy.float64'>
45.243046634
<class 'numpy.float64'>
45.2420568658
<class 'numpy.float64'>
45.2410675447
<class 'numpy.float64'>
45.2400786701
<class 'numpy.float64'>
45.2390902417
<class 'numpy.float64'>
45.238102259
<class 'numpy.float64'>
45.2371147217
<class 'numpy.float64'>
45.2361276292
<class 'numpy.float64'>
45.2351409813
<class 'numpy.float64'>
45.2341547774
<class 'numpy.float64'>
45.2331690171
<class 'numpy.float64'>
45.2321837001
<class 'numpy.float64'>
45.2311988258
<class 'numpy.float64'>
45.230214394
<class 'numpy.float64'>
45.2292304042
<class 'numpy.float64'>
45.2282468559
<class 'numpy.float64'>
45.2272637488
<class 'numpy.float64'>
45.2262810824
<class 'numpy.float64'>
45.2252988563
<class 'numpy.float64'>
45.2243170702
<class 'numpy.float64'>
45.2233357236
<class 'numpy.float64'>
45.222354816
<class 'numpy.float64'>
45.2213743471
<class 'numpy.float64'>
45.2203943165
<class 'numpy.float64'>
45.2194147238
<class 'numpy.float64'>
45.2184355685
<class 'numpy.float64'>
45.2174568503
<class 'numpy.float64'>
45.2164785687
<class 'numpy.float64'>
45.2155007233
<class 'numpy.float64'>
45.2145233137
<class 'numpy.float64'>
45.2135463396
<class 'numpy.float64'>
45.2125698005
<class 'numpy.float64'>
45.211593696
<class 'numpy.float64'>
45.2106180257
<class 'numpy.float64'>
45.2096427891
<class 'numpy.float64'>
45.208667986
<class 'numpy.float64'>
45.2076936159
<class 'numpy.float64'>
45.2067196784
<class 'numpy.float64'>
45.2057461731
<class 'numpy.float64'>
45.2047730995
<class 'numpy.float64'>
45.2038004574
<class 'numpy.float64'>
45.2028282463
<class 'numpy.float64'>
45.2018564657
<class 'numpy.float64'>
45.2008851153
<class 'numpy.float64'>
45.1999141948
<class 'numpy.float64'>
45.1989437036
<class 'numpy.float64'>
45.1979736415
<class 'numpy.float64'>
45.1970040079
<class 'numpy.float64'>
45.1960348026
<class 'numpy.float64'>
45.1950660251
<class 'numpy.float64'>
45.194097675
<class 'numpy.float64'>
45.193129752
<class 'numpy.float64'>
45.1921622556
<class 'numpy.float64'>
45.1911951854
<class 'numpy.float64'>
45.1902285411
<class 'numpy.float64'>
45.1892623223
<class 'numpy.float64'>
45.1882965285
<class 'numpy.float64'>
45.1873311594
<class 'numpy.float64'>
45.1863662146
<class 'numpy.float64'>
45.1854016937
<class 'numpy.float64'>
45.1844375963
<class 'numpy.float64'>
45.183473922
<class 'numpy.float64'>
45.1825106705
<class 'numpy.float64'>
45.1815478414
<class 'numpy.float64'>
45.1805854342
<class 'numpy.float64'>
45.1796234485
<class 'numpy.float64'>
45.1786618841
<class 'numpy.float64'>
45.1777007405
<class 'numpy.float64'>
45.1767400173
<class 'numpy.float64'>
45.1757797142
<class 'numpy.float64'>
45.1748198307
<class 'numpy.float64'>
45.1738603665
<class 'numpy.float64'>
45.1729013212
<class 'numpy.float64'>
45.1719426943
<class 'numpy.float64'>
45.1709844857
<class 'numpy.float64'>
45.1700266948
<class 'numpy.float64'>
45.1690693212
<class 'numpy.float64'>
45.1681123647
<class 'numpy.float64'>
45.1671558247
<class 'numpy.float64'>
45.166199701
<class 'numpy.float64'>
45.1652439932
<class 'numpy.float64'>
45.1642887008
<class 'numpy.float64'>
45.1633338235
<class 'numpy.float64'>
45.162379361
<class 'numpy.float64'>
45.1614253128
<class 'numpy.float64'>
45.1604716786
<class 'numpy.float64'>
45.159518458
<class 'numpy.float64'>
45.1585656507
<class 'numpy.float64'>
45.1576132562
<class 'numpy.float64'>
45.1566612741
<class 'numpy.float64'>
45.1557097042
<class 'numpy.float64'>
45.154758546
<class 'numpy.float64'>
45.1538077992
<class 'numpy.float64'>
45.1528574634
<class 'numpy.float64'>
45.1519075382
<class 'numpy.float64'>
45.1509580233
<class 'numpy.float64'>
45.1500089183
<class 'numpy.float64'>
45.1490602227
<class 'numpy.float64'>
45.1481119364
<class 'numpy.float64'>
45.1471640588
<class 'numpy.float64'>
45.1462165896
<class 'numpy.float64'>
45.1452695285
<class 'numpy.float64'>
45.1443228751
<class 'numpy.float64'>
45.1433766289
<class 'numpy.float64'>
45.1424307898
<class 'numpy.float64'>
45.1414853572
<class 'numpy.float64'>
45.1405403308
<class 'numpy.float64'>
45.1395957104
<class 'numpy.float64'>
45.1386514954
<class 'numpy.float64'>
45.1377076855
<class 'numpy.float64'>
45.1367642804
<class 'numpy.float64'>
45.1358212797
<class 'numpy.float64'>
45.1348786831
<class 'numpy.float64'>
45.1339364902
<class 'numpy.float64'>
45.1329947006
<class 'numpy.float64'>
45.1320533139
<class 'numpy.float64'>
45.1311123299
<class 'numpy.float64'>
45.1301717481
<class 'numpy.float64'>
45.1292315682
<class 'numpy.float64'>
45.1282917899
<class 'numpy.float64'>
45.1273524127
<class 'numpy.float64'>
45.1264134364
<class 'numpy.float64'>
45.1254748605
<class 'numpy.float64'>
45.1245366848
<class 'numpy.float64'>
45.1235989088
<class 'numpy.float64'>
45.1226615322
<class 'numpy.float64'>
45.1217245546
<class 'numpy.float64'>
45.1207879758
<class 'numpy.float64'>
45.1198517953
<class 'numpy.float64'>
45.1189160128
<class 'numpy.float64'>
45.1179806279
<class 'numpy.float64'>
45.1170456403
<class 'numpy.float64'>
45.1161110496
<class 'numpy.float64'>
45.1151768555
<class 'numpy.float64'>
45.1142430577
<class 'numpy.float64'>
45.1133096557
<class 'numpy.float64'>
45.1123766493
<class 'numpy.float64'>
45.111444038
<class 'numpy.float64'>
45.1105118216
<class 'numpy.float64'>
45.1095799997
<class 'numpy.float64'>
45.108648572
<class 'numpy.float64'>
45.107717538
<class 'numpy.float64'>
45.1067868975
<class 'numpy.float64'>
45.10585665
<class 'numpy.float64'>
45.1049267954
<class 'numpy.float64'>
45.1039973331
<class 'numpy.float64'>
45.103068263
<class 'numpy.float64'>
45.1021395845
<class 'numpy.float64'>
45.1012112974
<class 'numpy.float64'>
45.1002834014
<class 'numpy.float64'>
45.0993558961
<class 'numpy.float64'>
45.0984287811
<class 'numpy.float64'>
45.0975020561
<class 'numpy.float64'>
45.0965757208
<class 'numpy.float64'>
45.0956497749
<class 'numpy.float64'>
45.0947242179
<class 'numpy.float64'>
45.0937990496
<class 'numpy.float64'>
45.0928742696
<class 'numpy.float64'>
45.0919498775
<class 'numpy.float64'>
45.0910258731
<class 'numpy.float64'>
45.090102256
<class 'numpy.float64'>
45.0891790259
<class 'numpy.float64'>
45.0882561824
<class 'numpy.float64'>
45.0873337251
<class 'numpy.float64'>
45.0864116539
<class 'numpy.float64'>
45.0854899682
<class 'numpy.float64'>
45.0845686678
<class 'numpy.float64'>
45.0836477524
<class 'numpy.float64'>
45.0827272215
<class 'numpy.float64'>
45.081807075
<class 'numpy.float64'>
45.0808873124
<class 'numpy.float64'>
45.0799679334
<class 'numpy.float64'>
45.0790489377
<class 'numpy.float64'>
45.0781303249
<class 'numpy.float64'>
45.0772120947
<class 'numpy.float64'>
45.0762942469
<class 'numpy.float64'>
45.0753767809
<class 'numpy.float64'>
45.0744596966
<class 'numpy.float64'>
45.0735429937
<class 'numpy.float64'>
45.0726266716
<class 'numpy.float64'>
45.0717107302
<class 'numpy.float64'>
45.0707951691
<class 'numpy.float64'>
45.069879988
<class 'numpy.float64'>
45.0689651866
<class 'numpy.float64'>
45.0680507644
<class 'numpy.float64'>
45.0671367213
<class 'numpy.float64'>
45.0662230569
<class 'numpy.float64'>
45.0653097708
<class 'numpy.float64'>
45.0643968627
<class 'numpy.float64'>
45.0634843323
<class 'numpy.float64'>
45.0625721793
<class 'numpy.float64'>
45.0616604033
<class 'numpy.float64'>
45.0607490041
<class 'numpy.float64'>
45.0598379812
<class 'numpy.float64'>
45.0589273345
<class 'numpy.float64'>
45.0580170635
<class 'numpy.float64'>
45.0571071679
<class 'numpy.float64'>
45.0561976474
<class 'numpy.float64'>
45.0552885018
<class 'numpy.float64'>
45.0543797306
<class 'numpy.float64'>
45.0534713336
<class 'numpy.float64'>
45.0525633104
<class 'numpy.float64'>
45.0516556607
<class 'numpy.float64'>
45.0507483842
<class 'numpy.float64'>
45.0498414806
<class 'numpy.float64'>
45.0489349496
<class 'numpy.float64'>
45.0480287908
<class 'numpy.float64'>
45.0471230039
<class 'numpy.float64'>
45.0462175887
<class 'numpy.float64'>
45.0453125447
<class 'numpy.float64'>
45.0444078717
<class 'numpy.float64'>
45.0435035694
<class 'numpy.float64'>
45.0425996374
<class 'numpy.float64'>
45.0416960754
<class 'numpy.float64'>
45.0407928832
<class 'numpy.float64'>
45.0398900604
<class 'numpy.float64'>
45.0389876066
<class 'numpy.float64'>
45.0380855217
<class 'numpy.float64'>
45.0371838052
<class 'numpy.float64'>
45.0362824568
<class 'numpy.float64'>
45.0353814763
<class 'numpy.float64'>
45.0344808634
<class 'numpy.float64'>
45.0335806176
<class 'numpy.float64'>
45.0326807388
<class 'numpy.float64'>
45.0317812266
<class 'numpy.float64'>
45.0308820806
<class 'numpy.float64'>
45.0299833007
<class 'numpy.float64'>
45.0290848864
<class 'numpy.float64'>
45.0281868375
<class 'numpy.float64'>
45.0272891536
<class 'numpy.float64'>
45.0263918345
<class 'numpy.float64'>
45.0254948798
<class 'numpy.float64'>
45.0245982893
<class 'numpy.float64'>
45.0237020626
<class 'numpy.float64'>
45.0228061994
<class 'numpy.float64'>
45.0219106995
<class 'numpy.float64'>
45.0210155624
<class 'numpy.float64'>
45.020120788
<class 'numpy.float64'>
45.0192263759
<class 'numpy.float64'>
45.0183323257
<class 'numpy.float64'>
45.0174386373
<class 'numpy.float64'>
45.0165453103
<class 'numpy.float64'>
45.0156523443
<class 'numpy.float64'>
45.0147597392
<class 'numpy.float64'>
45.0138674945
<class 'numpy.float64'>
45.01297561
<class 'numpy.float64'>
45.0120840855
<class 'numpy.float64'>
45.0111929205
<class 'numpy.float64'>
45.0103021147
<class 'numpy.float64'>
45.009411668
<class 'numpy.float64'>
45.00852158
<class 'numpy.float64'>
45.0076318503
<class 'numpy.float64'>
45.0067424787
<class 'numpy.float64'>
45.0058534649
<class 'numpy.float64'>
45.0049648087
<class 'numpy.float64'>
45.0040765096
<class 'numpy.float64'>
45.0031885674
<class 'numpy.float64'>
45.0023009818
<class 'numpy.float64'>
45.0014137525
<class 'numpy.float64'>
45.0005268792
<class 'numpy.float64'>
44.9996403616
<class 'numpy.float64'>
44.9987541995
<class 'numpy.float64'>
44.9978683925
<class 'numpy.float64'>
44.9969829403
<class 'numpy.float64'>
44.9960978426
<class 'numpy.float64'>
44.9952130992
<class 'numpy.float64'>
44.9943287097
<class 'numpy.float64'>
44.9934446739
<class 'numpy.float64'>
44.9925609915
<class 'numpy.float64'>
44.9916776621
<class 'numpy.float64'>
44.9907946855
<class 'numpy.float64'>
44.9899120614
<class 'numpy.float64'>
44.9890297894
<class 'numpy.float64'>
44.9881478694
<class 'numpy.float64'>
44.987266301
<class 'numpy.float64'>
44.986385084
<class 'numpy.float64'>
44.9855042179
<class 'numpy.float64'>
44.9846237027
<class 'numpy.float64'>
44.9837435378
<class 'numpy.float64'>
44.9828637232
<class 'numpy.float64'>
44.9819842584
<class 'numpy.float64'>
44.9811051433
<class 'numpy.float64'>
44.9802263774
<class 'numpy.float64'>
44.9793479606
<class 'numpy.float64'>
44.9784698925
<class 'numpy.float64'>
44.9775921728
<class 'numpy.float64'>
44.9767148013
<class 'numpy.float64'>
44.9758377777
<class 'numpy.float64'>
44.9749611017
<class 'numpy.float64'>
44.974084773
<class 'numpy.float64'>
44.9732087913
<class 'numpy.float64'>
44.9723331564
<class 'numpy.float64'>
44.9714578679
<class 'numpy.float64'>
44.9705829256
<class 'numpy.float64'>
44.9697083291
<class 'numpy.float64'>
44.9688340783
<class 'numpy.float64'>
44.9679601729
<class 'numpy.float64'>
44.9670866124
<class 'numpy.float64'>
44.9662133967
<class 'numpy.float64'>
44.9653405256
<class 'numpy.float64'>
44.9644679986
<class 'numpy.float64'>
44.9635958155
<class 'numpy.float64'>
44.9627239761
<class 'numpy.float64'>
44.96185248
<class 'numpy.float64'>
44.9609813271
<class 'numpy.float64'>
44.9601105169
<class 'numpy.float64'>
44.9592400493
<class 'numpy.float64'>
44.9583699239
<class 'numpy.float64'>
44.9575001405
<class 'numpy.float64'>
44.9566306987
<class 'numpy.float64'>
44.9557615984
<class 'numpy.float64'>
44.9548928393
<class 'numpy.float64'>
44.954024421
<class 'numpy.float64'>
44.9531563432
<class 'numpy.float64'>
44.9522886058
<class 'numpy.float64'>
44.9514212085
<class 'numpy.float64'>
44.9505541509
<class 'numpy.float64'>
44.9496874328
<class 'numpy.float64'>
44.9488210539
<class 'numpy.float64'>
44.9479550139
<class 'numpy.float64'>
44.9470893126
<class 'numpy.float64'>
44.9462239497
<class 'numpy.float64'>
44.9453589249
<class 'numpy.float64'>
44.944494238
<class 'numpy.float64'>
44.9436298887
<class 'numpy.float64'>
44.9427658766
<class 'numpy.float64'>
44.9419022016
<class 'numpy.float64'>
44.9410388633
<class 'numpy.float64'>
44.9401758616
<class 'numpy.float64'>
44.939313196
<class 'numpy.float64'>
44.9384508665
<class 'numpy.float64'>
44.9375888726
<class 'numpy.float64'>
44.9367272141
<class 'numpy.float64'>
44.9358658907
<class 'numpy.float64'>
44.9350049022
<class 'numpy.float64'>
44.9341442484
<class 'numpy.float64'>
44.9332839288
<class 'numpy.float64'>
44.9324239434
<class 'numpy.float64'>
44.9315642917
<class 'numpy.float64'>
44.9307049736
<class 'numpy.float64'>
44.9298459888
<class 'numpy.float64'>
44.9289873369
<class 'numpy.float64'>
44.9281290178
<class 'numpy.float64'>
44.9272710312
<class 'numpy.float64'>
44.9264133768
<class 'numpy.float64'>
44.9255560543
<class 'numpy.float64'>
44.9246990635
<class 'numpy.float64'>
44.9238424041
<class 'numpy.float64'>
44.9229860758
<class 'numpy.float64'>
44.9221300785
<class 'numpy.float64'>
44.9212744117
<class 'numpy.float64'>
44.9204190754
<class 'numpy.float64'>
44.9195640691
<class 'numpy.float64'>
44.9187093927
<class 'numpy.float64'>
44.9178550458
<class 'numpy.float64'>
44.9170010282
<class 'numpy.float64'>
44.9161473397
<class 'numpy.float64'>
44.91529398
<class 'numpy.float64'>
44.9144409488
<class 'numpy.float64'>
44.9135882459
<class 'numpy.float64'>
44.912735871
<class 'numpy.float64'>
44.9118838238
<class 'numpy.float64'>
44.9110321042
<class 'numpy.float64'>
44.9101807117
<class 'numpy.float64'>
44.9093296462
<class 'numpy.float64'>
44.9084789075
<class 'numpy.float64'>
44.9076284952
<class 'numpy.float64'>
44.9067784091
<class 'numpy.float64'>
44.9059286489
<class 'numpy.float64'>
44.9050792144
<class 'numpy.float64'>
44.9042301054
<class 'numpy.float64'>
44.9033813215
<class 'numpy.float64'>
44.9025328625
<class 'numpy.float64'>
44.9016847282
<class 'numpy.float64'>
44.9008369183
<class 'numpy.float64'>
44.8999894326
<class 'numpy.float64'>
44.8991422707
<class 'numpy.float64'>
44.8982954325
<class 'numpy.float64'>
44.8974489176
<class 'numpy.float64'>
44.8966027259
<class 'numpy.float64'>
44.8957568571
<class 'numpy.float64'>
44.8949113109
<class 'numpy.float64'>
44.8940660871
<class 'numpy.float64'>
44.8932211853
<class 'numpy.float64'>
44.8923766055
<class 'numpy.float64'>
44.8915323473
<class 'numpy.float64'>
44.8906884104
<class 'numpy.float64'>
44.8898447946
<class 'numpy.float64'>
44.8890014997
<class 'numpy.float64'>
44.8881585254
<class 'numpy.float64'>
44.8873158715
<class 'numpy.float64'>
44.8864735377
<class 'numpy.float64'>
44.8856315237
<class 'numpy.float64'>
44.8847898294
<class 'numpy.float64'>
44.8839484544
<class 'numpy.float64'>
44.8831073985
<class 'numpy.float64'>
44.8822666615
<class 'numpy.float64'>
44.8814262431
<class 'numpy.float64'>
44.880586143
<class 'numpy.float64'>
44.8797463611
<class 'numpy.float64'>
44.8789068971
<class 'numpy.float64'>
44.8780677507
<class 'numpy.float64'>
44.8772289216
<class 'numpy.float64'>
44.8763904097
<class 'numpy.float64'>
44.8755522147
<class 'numpy.float64'>
44.8747143363
<class 'numpy.float64'>
44.8738767743
<class 'numpy.float64'>
44.8730395285
<class 'numpy.float64'>
44.8722025986
<class 'numpy.float64'>
44.8713659843
<class 'numpy.float64'>
44.8705296854
<class 'numpy.float64'>
44.8696937018
<class 'numpy.float64'>
44.868858033
<class 'numpy.float64'>
44.8680226789
<class 'numpy.float64'>
44.8671876393
<class 'numpy.float64'>
44.8663529139
<class 'numpy.float64'>
44.8655185024
<class 'numpy.float64'>
44.8646844047
<class 'numpy.float64'>
44.8638506204
<class 'numpy.float64'>
44.8630171493
<class 'numpy.float64'>
44.8621839912
<class 'numpy.float64'>
44.8613511459
<class 'numpy.float64'>
44.8605186131
<class 'numpy.float64'>
44.8596863926
<class 'numpy.float64'>
44.8588544841
<class 'numpy.float64'>
44.8580228873
<class 'numpy.float64'>
44.8571916022
<class 'numpy.float64'>
44.8563606283
<class 'numpy.float64'>
44.8555299655
<class 'numpy.float64'>
44.8546996135
<class 'numpy.float64'>
44.8538695722
<class 'numpy.float64'>
44.8530398411
<class 'numpy.float64'>
44.8522104202
<class 'numpy.float64'>
44.8513813092
<class 'numpy.float64'>
44.8505525078
<class 'numpy.float64'>
44.8497240158
<class 'numpy.float64'>
44.848895833
<class 'numpy.float64'>
44.8480679592
<class 'numpy.float64'>
44.847240394
<class 'numpy.float64'>
44.8464131373
<class 'numpy.float64'>
44.8455861888
<class 'numpy.float64'>
44.8447595483
<class 'numpy.float64'>
44.8439332155
<class 'numpy.float64'>
44.8431071903
<class 'numpy.float64'>
44.8422814723
<class 'numpy.float64'>
44.8414560614
<class 'numpy.float64'>
44.8406309574
<class 'numpy.float64'>
44.8398061599
<class 'numpy.float64'>
44.8389816687
<class 'numpy.float64'>
44.8381574837
<class 'numpy.float64'>
44.8373336046
<class 'numpy.float64'>
44.8365100311
<class 'numpy.float64'>
44.8356867631
<class 'numpy.float64'>
44.8348638002
<class 'numpy.float64'>
44.8340411423
<class 'numpy.float64'>
44.8332187891
<class 'numpy.float64'>
44.8323967404
<class 'numpy.float64'>
44.831574996
<class 'numpy.float64'>
44.8307535556
<class 'numpy.float64'>
44.829932419
<class 'numpy.float64'>
44.829111586
<class 'numpy.float64'>
44.8282910563
<class 'numpy.float64'>
44.8274708298
<class 'numpy.float64'>
44.8266509061
<class 'numpy.float64'>
44.8258312851
<class 'numpy.float64'>
44.8250119666
<class 'numpy.float64'>
44.8241929502
<class 'numpy.float64'>
44.8233742358
<class 'numpy.float64'>
44.8225558231
<class 'numpy.float64'>
44.821737712
<class 'numpy.float64'>
44.8209199022
<class 'numpy.float64'>
44.8201023934
<class 'numpy.float64'>
44.8192851855
<class 'numpy.float64'>
44.8184682781
<class 'numpy.float64'>
44.8176516712
<class 'numpy.float64'>
44.8168353644
<class 'numpy.float64'>
44.8160193576
<class 'numpy.float64'>
44.8152036504
<class 'numpy.float64'>
44.8143882428
<class 'numpy.float64'>
44.8135731344
<class 'numpy.float64'>
44.812758325
<class 'numpy.float64'>
44.8119438145
<class 'numpy.float64'>
44.8111296025
<class 'numpy.float64'>
44.810315689
<class 'numpy.float64'>
44.8095020735
<class 'numpy.float64'>
44.808688756
<class 'numpy.float64'>
44.8078757361
<class 'numpy.float64'>
44.8070630138
<class 'numpy.float64'>
44.8062505887
<class 'numpy.float64'>
44.8054384606
<class 'numpy.float64'>
44.8046266293
<class 'numpy.float64'>
44.8038150946
<class 'numpy.float64'>
44.8030038563
<class 'numpy.float64'>
44.8021929141
<class 'numpy.float64'>
44.8013822678
<class 'numpy.float64'>
44.8005719173
<class 'numpy.float64'>
44.7997618622
<class 'numpy.float64'>
44.7989521024
<class 'numpy.float64'>
44.7981426376
<class 'numpy.float64'>
44.7973334676
<class 'numpy.float64'>
44.7965245923
<class 'numpy.float64'>
44.7957160113
<class 'numpy.float64'>
44.7949077245
<class 'numpy.float64'>
44.7940997317
<class 'numpy.float64'>
44.7932920325
<class 'numpy.float64'>
44.7924846269
<class 'numpy.float64'>
44.7916775146
<class 'numpy.float64'>
44.7908706954
<class 'numpy.float64'>
44.790064169
<class 'numpy.float64'>
44.7892579353
<class 'numpy.float64'>
44.788451994
<class 'numpy.float64'>
44.7876463449
<class 'numpy.float64'>
44.7868409878
<class 'numpy.float64'>
44.7860359225
<class 'numpy.float64'>
44.7852311487
<class 'numpy.float64'>
44.7844266663
<class 'numpy.float64'>
44.7836224751
<class 'numpy.float64'>
44.7828185747
<class 'numpy.float64'>
44.7820149651
<class 'numpy.float64'>
44.7812116459
<class 'numpy.float64'>
44.780408617
<class 'numpy.float64'>
44.7796058782
<class 'numpy.float64'>
44.7788034292
<class 'numpy.float64'>
44.7780012699
<class 'numpy.float64'>
44.7771994
<class 'numpy.float64'>
44.7763978193
<class 'numpy.float64'>
44.7755965276
<class 'numpy.float64'>
44.7747955247
<class 'numpy.float64'>
44.7739948103
<class 'numpy.float64'>
44.7731943843
<class 'numpy.float64'>
44.7723942465
<class 'numpy.float64'>
44.7715943966
<class 'numpy.float64'>
44.7707948345
<class 'numpy.float64'>
44.7699955598
<class 'numpy.float64'>
44.7691965725
<class 'numpy.float64'>
44.7683978722
<class 'numpy.float64'>
44.7675994589
<class 'numpy.float64'>
44.7668013322
<class 'numpy.float64'>
44.766003492
<class 'numpy.float64'>
44.7652059381
<class 'numpy.float64'>
44.7644086702
<class 'numpy.float64'>
44.7636116881
<class 'numpy.float64'>
44.7628149917
<class 'numpy.float64'>
44.7620185807
<class 'numpy.float64'>
44.7612224549
<class 'numpy.float64'>
44.7604266141
<class 'numpy.float64'>
44.7596310581
<class 'numpy.float64'>
44.7588357868
<class 'numpy.float64'>
44.7580407998
<class 'numpy.float64'>
44.7572460969
<class 'numpy.float64'>
44.7564516781
<class 'numpy.float64'>
44.755657543
<class 'numpy.float64'>
44.7548636915
<class 'numpy.float64'>
44.7540701234
<class 'numpy.float64'>
44.7532768383
<class 'numpy.float64'>
44.7524838363
<class 'numpy.float64'>
44.751691117
<class 'numpy.float64'>
44.7508986802
<class 'numpy.float64'>
44.7501065257
<class 'numpy.float64'>
44.7493146534
<class 'numpy.float64'>
44.7485230629
<class 'numpy.float64'>
44.7477317542
<class 'numpy.float64'>
44.7469407271
<class 'numpy.float64'>
44.7461499812
<class 'numpy.float64'>
44.7453595164
<class 'numpy.float64'>
44.7445693326
<class 'numpy.float64'>
44.7437794294
<class 'numpy.float64'>
44.7429898067
<class 'numpy.float64'>
44.7422004644
<class 'numpy.float64'>
44.7414114021
<class 'numpy.float64'>
44.7406226197
<class 'numpy.float64'>
44.739834117
<class 'numpy.float64'>
44.7390458938
<class 'numpy.float64'>
44.7382579499
<class 'numpy.float64'>
44.7374702851
<class 'numpy.float64'>
44.7366828991
<class 'numpy.float64'>
44.7358957919
<class 'numpy.float64'>
44.7351089631
<class 'numpy.float64'>
44.7343224126
<class 'numpy.float64'>
44.7335361402
<class 'numpy.float64'>
44.7327501457
<class 'numpy.float64'>
44.7319644289
<class 'numpy.float64'>
44.7311789896
<class 'numpy.float64'>
44.7303938276
<class 'numpy.float64'>
44.7296089426
<class 'numpy.float64'>
44.7288243346
<class 'numpy.float64'>
44.7280400032
<class 'numpy.float64'>
44.7272559483
<class 'numpy.float64'>
44.7264721698
<class 'numpy.float64'>
44.7256886673
<class 'numpy.float64'>
44.7249054408
<class 'numpy.float64'>
44.7241224899
<class 'numpy.float64'>
44.7233398146
<class 'numpy.float64'>
44.7225574146
<class 'numpy.float64'>
44.7217752897
<class 'numpy.float64'>
44.7209934397
<class 'numpy.float64'>
44.7202118645
<class 'numpy.float64'>
44.7194305638
<class 'numpy.float64'>
44.7186495374
<class 'numpy.float64'>
44.7178687851
<class 'numpy.float64'>
44.7170883068
<class 'numpy.float64'>
44.7163081023
<class 'numpy.float64'>
44.7155281713
<class 'numpy.float64'>
44.7147485137
<class 'numpy.float64'>
44.7139691292
<class 'numpy.float64'>
44.7131900178
<class 'numpy.float64'>
44.7124111791
<class 'numpy.float64'>
44.711632613
<class 'numpy.float64'>
44.7108543192
<class 'numpy.float64'>
44.7100762977
<class 'numpy.float64'>
44.7092985482
<class 'numpy.float64'>
44.7085210705
<class 'numpy.float64'>
44.7077438645
<class 'numpy.float64'>
44.7069669298
<class 'numpy.float64'>
44.7061902664
<class 'numpy.float64'>
44.7054138741
<class 'numpy.float64'>
44.7046377526
<class 'numpy.float64'>
44.7038619018
<class 'numpy.float64'>
44.7030863215
<class 'numpy.float64'>
44.7023110114
<class 'numpy.float64'>
44.7015359714
<class 'numpy.float64'>
44.7007612014
<class 'numpy.float64'>
44.699986701
<class 'numpy.float64'>
44.6992124702
<class 'numpy.float64'>
44.6984385087
<class 'numpy.float64'>
44.6976648164
<class 'numpy.float64'>
44.696891393
<class 'numpy.float64'>
44.6961182384
<class 'numpy.float64'>
44.6953453523
<class 'numpy.float64'>
44.6945727347
<class 'numpy.float64'>
44.6938003852
<class 'numpy.float64'>
44.6930283038
<class 'numpy.float64'>
44.6922564902
<class 'numpy.float64'>
44.6914849442
<class 'numpy.float64'>
44.6907136656
<class 'numpy.float64'>
44.6899426544
<class 'numpy.float64'>
44.6891719102
<class 'numpy.float64'>
44.6884014329
<class 'numpy.float64'>
44.6876312223
<class 'numpy.float64'>
44.6868612782
<class 'numpy.float64'>
44.6860916004
<class 'numpy.float64'>
44.6853221888
<class 'numpy.float64'>
44.6845530432
<class 'numpy.float64'>
44.6837841633
<class 'numpy.float64'>
44.683015549
<class 'numpy.float64'>
44.6822472001
<class 'numpy.float64'>
44.6814791164
<class 'numpy.float64'>
44.6807112978
<class 'numpy.float64'>
44.679943744
<class 'numpy.float64'>
44.6791764548
<class 'numpy.float64'>
44.6784094302
<class 'numpy.float64'>
44.6776426698
<class 'numpy.float64'>
44.6768761736
<class 'numpy.float64'>
44.6761099412
<class 'numpy.float64'>
44.6753439727
<class 'numpy.float64'>
44.6745782676
<class 'numpy.float64'>
44.673812826
<class 'numpy.float64'>
44.6730476476
<class 'numpy.float64'>
44.6722827321
<class 'numpy.float64'>
44.6715180795
<class 'numpy.float64'>
44.6707536896
<class 'numpy.float64'>
44.6699895621
<class 'numpy.float64'>
44.6692256969
<class 'numpy.float64'>
44.6684620938
<class 'numpy.float64'>
44.6676987526
<class 'numpy.float64'>
44.6669356732
<class 'numpy.float64'>
44.6661728553
<class 'numpy.float64'>
44.6654102988
<class 'numpy.float64'>
44.6646480035
<class 'numpy.float64'>
44.6638859693
<class 'numpy.float64'>
44.6631241958
<class 'numpy.float64'>
44.6623626831
<class 'numpy.float64'>
44.6616014308
<class 'numpy.float64'>
44.6608404388
<class 'numpy.float64'>
44.6600797069
<class 'numpy.float64'>
44.659319235
<class 'numpy.float64'>
44.6585590228
<class 'numpy.float64'>
44.6577990702
<class 'numpy.float64'>
44.6570393771
<class 'numpy.float64'>
44.6562799431
<class 'numpy.float64'>
44.6555207682
<class 'numpy.float64'>
44.6547618522
<class 'numpy.float64'>
44.6540031949
<class 'numpy.float64'>
44.6532447961
<class 'numpy.float64'>
44.6524866556
<class 'numpy.float64'>
44.6517287733
<class 'numpy.float64'>
44.650971149
<class 'numpy.float64'>
44.6502137825
<class 'numpy.float64'>
44.6494566737
<class 'numpy.float64'>
44.6486998223
<class 'numpy.float64'>
44.6479432281
<class 'numpy.float64'>
44.6471868911
<class 'numpy.float64'>
44.646430811
<class 'numpy.float64'>
44.6456749877
<class 'numpy.float64'>
44.6449194209
<class 'numpy.float64'>
44.6441641106
<class 'numpy.float64'>
44.6434090564
<class 'numpy.float64'>
44.6426542584
<class 'numpy.float64'>
44.6418997162
<class 'numpy.float64'>
44.6411454297
<class 'numpy.float64'>
44.6403913988
<class 'numpy.float64'>
44.6396376232
<class 'numpy.float64'>
44.6388841028
<class 'numpy.float64'>
44.6381308374
<class 'numpy.float64'>
44.6373778268
<class 'numpy.float64'>
44.6366250709
<class 'numpy.float64'>
44.6358725695
<class 'numpy.float64'>
44.6351203225
<class 'numpy.float64'>
44.6343683295
<class 'numpy.float64'>
44.6336165906
<class 'numpy.float64'>
44.6328651054
<class 'numpy.float64'>
44.6321138739
<class 'numpy.float64'>
44.6313628958
<class 'numpy.float64'>
44.6306121711
<class 'numpy.float64'>
44.6298616994
<class 'numpy.float64'>
44.6291114807
<class 'numpy.float64'>
44.6283615148
<class 'numpy.float64'>
44.6276118014
<class 'numpy.float64'>
44.6268623405
<class 'numpy.float64'>
44.6261131319
<class 'numpy.float64'>
44.6253641753
<class 'numpy.float64'>
44.6246154707
<class 'numpy.float64'>
44.6238670178
<class 'numpy.float64'>
44.6231188165
<class 'numpy.float64'>
44.6223708666
<class 'numpy.float64'>
44.621623168
<class 'numpy.float64'>
44.6208757204
<class 'numpy.float64'>
44.6201285238
<class 'numpy.float64'>
44.6193815779
<class 'numpy.float64'>
44.6186348825
<class 'numpy.float64'>
44.6178884376
<class 'numpy.float64'>
44.6171422428
<class 'numpy.float64'>
44.6163962982
<class 'numpy.float64'>
44.6156506034
<class 'numpy.float64'>
44.6149051584
<class 'numpy.float64'>
44.6141599629
<class 'numpy.float64'>
44.6134150169
<class 'numpy.float64'>
44.61267032
<class 'numpy.float64'>
44.6119258722
<class 'numpy.float64'>
44.6111816733
<class 'numpy.float64'>
44.6104377232
<class 'numpy.float64'>
44.6096940216
<class 'numpy.float64'>
44.6089505683
<class 'numpy.float64'>
44.6082073634
<class 'numpy.float64'>
44.6074644064
<class 'numpy.float64'>
44.6067216974
<class 'numpy.float64'>
44.6059792361
<class 'numpy.float64'>
44.6052370224
<class 'numpy.float64'>
44.6044950561
<class 'numpy.float64'>
44.603753337
<class 'numpy.float64'>
44.603011865
<class 'numpy.float64'>
44.6022706399
<class 'numpy.float64'>
44.6015296615
<class 'numpy.float64'>
44.6007889297
<class 'numpy.float64'>
44.6000484444
<class 'numpy.float64'>
44.5993082053
<class 'numpy.float64'>
44.5985682122
<class 'numpy.float64'>
44.5978284651
<class 'numpy.float64'>
44.5970889638
<class 'numpy.float64'>
44.5963497081
<class 'numpy.float64'>
44.5956106978
<class 'numpy.float64'>
44.5948719327
<class 'numpy.float64'>
44.5941334128
<class 'numpy.float64'>
44.5933951378
<class 'numpy.float64'>
44.5926571076
<class 'numpy.float64'>
44.5919193221
<class 'numpy.float64'>
44.591181781
<class 'numpy.float64'>
44.5904444842
<class 'numpy.float64'>
44.5897074315
<class 'numpy.float64'>
44.5889706228
<class 'numpy.float64'>
44.588234058
<class 'numpy.float64'>
44.5874977368
<class 'numpy.float64'>
44.5867616591
<class 'numpy.float64'>
44.5860258247
<class 'numpy.float64'>
44.5852902335
<class 'numpy.float64'>
44.5845548853
<class 'numpy.float64'>
44.58381978
<class 'numpy.float64'>
44.5830849173
<class 'numpy.float64'>
44.5823502972
<class 'numpy.float64'>
44.5816159195
<class 'numpy.float64'>
44.580881784
<class 'numpy.float64'>
44.5801478905
<class 'numpy.float64'>
44.579414239
<class 'numpy.float64'>
44.5786808292
<class 'numpy.float64'>
44.5779476609
<class 'numpy.float64'>
44.5772147341
<class 'numpy.float64'>
44.5764820486
<class 'numpy.float64'>
44.5757496041
<class 'numpy.float64'>
44.5750174007
<class 'numpy.float64'>
44.574285438
<class 'numpy.float64'>
44.5735537159
<class 'numpy.float64'>
44.5728222343
<class 'numpy.float64'>
44.572090993
<class 'numpy.float64'>
44.5713599919
<class 'numpy.float64'>
44.5706292308
<class 'numpy.float64'>
44.5698987095
<class 'numpy.float64'>
44.569168428
<class 'numpy.float64'>
44.5684383859
<class 'numpy.float64'>
44.5677085833
<class 'numpy.float64'>
44.5669790198
<class 'numpy.float64'>
44.5662496955
<class 'numpy.float64'>
44.56552061
<class 'numpy.float64'>
44.5647917633
<class 'numpy.float64'>
44.5640631552
<class 'numpy.float64'>
44.5633347855
<class 'numpy.float64'>
44.5626066542
<class 'numpy.float64'>
44.561878761
<class 'numpy.float64'>
44.5611511057
<class 'numpy.float64'>
44.5604236883
<class 'numpy.float64'>
44.5596965085
<class 'numpy.float64'>
44.5589695663
<class 'numpy.float64'>
44.5582428614
<class 'numpy.float64'>
44.5575163938
<class 'numpy.float64'>
44.5567901632
<class 'numpy.float64'>
44.5560641695
<class 'numpy.float64'>
44.5553384125
<class 'numpy.float64'>
44.5546128921
<class 'numpy.float64'>
44.5538876082
<class 'numpy.float64'>
44.5531625606
<class 'numpy.float64'>
44.5524377491
<class 'numpy.float64'>
44.5517131736
<class 'numpy.float64'>
44.550988834
<class 'numpy.float64'>
44.55026473
<class 'numpy.float64'>
44.5495408615
<class 'numpy.float64'>
44.5488172284
<class 'numpy.float64'>
44.5480938306
<class 'numpy.float64'>
44.5473706678
<class 'numpy.float64'>
44.5466477399
<class 'numpy.float64'>
44.5459250469
<class 'numpy.float64'>
44.5452025884
<class 'numpy.float64'>
44.5444803644
<class 'numpy.float64'>
44.5437583747
<class 'numpy.float64'>
44.5430366192
<class 'numpy.float64'>
44.5423150977
<class 'numpy.float64'>
44.5415938101
<class 'numpy.float64'>
44.5408727562
<class 'numpy.float64'>
44.5401519359
<class 'numpy.float64'>
44.5394313489
<class 'numpy.float64'>
44.5387109953
<class 'numpy.float64'>
44.5379908748
<class 'numpy.float64'>
44.5372709872
<class 'numpy.float64'>
44.5365513325
<class 'numpy.float64'>
44.5358319104
<class 'numpy.float64'>
44.5351127209
<class 'numpy.float64'>
44.5343937637
<class 'numpy.float64'>
44.5336750388
<class 'numpy.float64'>
44.5329565459
<class 'numpy.float64'>
44.532238285
<class 'numpy.float64'>
44.5315202558
<class 'numpy.float64'>
44.5308024583
<class 'numpy.float64'>
44.5300848923
<class 'numpy.float64'>
44.5293675576
<class 'numpy.float64'>
44.5286504541
<class 'numpy.float64'>
44.5279335817
<class 'numpy.float64'>
44.5272169401
<class 'numpy.float64'>
44.5265005293
<class 'numpy.float64'>
44.5257843491
<class 'numpy.float64'>
44.5250683994
<class 'numpy.float64'>
44.52435268
<class 'numpy.float64'>
44.5236371907
<class 'numpy.float64'>
44.5229219315
<class 'numpy.float64'>
44.5222069021
<class 'numpy.float64'>
44.5214921025
<class 'numpy.float64'>
44.5207775324
<class 'numpy.float64'>
44.5200631918
<class 'numpy.float64'>
44.5193490805
<class 'numpy.float64'>
44.5186351983
<class 'numpy.float64'>
44.5179215452
<class 'numpy.float64'>
44.5172081209
<class 'numpy.float64'>
44.5164949253
<class 'numpy.float64'>
44.5157819582
<class 'numpy.float64'>
44.5150692196
<class 'numpy.float64'>
44.5143567093
<class 'numpy.float64'>
44.5136444271
<class 'numpy.float64'>
44.5129323729
<class 'numpy.float64'>
44.5122205466
<class 'numpy.float64'>
44.511508948
<class 'numpy.float64'>
44.5107975769
<class 'numpy.float64'>
44.5100864332
<class 'numpy.float64'>
44.5093755168
<class 'numpy.float64'>
44.5086648276
<class 'numpy.float64'>
44.5079543653
<class 'numpy.float64'>
44.5072441299
<class 'numpy.float64'>
44.5065341212
<class 'numpy.float64'>
44.5058243391
<class 'numpy.float64'>
44.5051147834
<class 'numpy.float64'>
44.504405454
<class 'numpy.float64'>
44.5036963507
<class 'numpy.float64'>
44.5029874733
<class 'numpy.float64'>
44.5022788219
<class 'numpy.float64'>
44.5015703962
<class 'numpy.float64'>
44.500862196
<class 'numpy.float64'>
44.5001542213
<class 'numpy.float64'>
44.4994464718
<class 'numpy.float64'>
44.4987389475
<class 'numpy.float64'>
44.4980316483
<class 'numpy.float64'>
44.4973245738
<class 'numpy.float64'>
44.4966177242
<class 'numpy.float64'>
44.4959110991
<class 'numpy.float64'>
44.4952046984
<class 'numpy.float64'>
44.4944985221
<class 'numpy.float64'>
44.4937925699
<class 'numpy.float64'>
44.4930868417
<class 'numpy.float64'>
44.4923813375
<class 'numpy.float64'>
44.491676057
<class 'numpy.float64'>
44.4909710001
<class 'numpy.float64'>
44.4902661667
<class 'numpy.float64'>
44.4895615566
<class 'numpy.float64'>
44.4888571697
<class 'numpy.float64'>
44.4881530059
<class 'numpy.float64'>
44.487449065
<class 'numpy.float64'>
44.4867453468
<class 'numpy.float64'>
44.4860418513
<class 'numpy.float64'>
44.4853385783
<class 'numpy.float64'>
44.4846355277
<class 'numpy.float64'>
44.4839326993
<class 'numpy.float64'>
44.483230093
<class 'numpy.float64'>
44.4825277087
<class 'numpy.float64'>
44.4818255462
<class 'numpy.float64'>
44.4811236053
<class 'numpy.float64'>
44.480421886
<class 'numpy.float64'>
44.4797203881
<class 'numpy.float64'>
44.4790191115
<class 'numpy.float64'>
44.478318056
<class 'numpy.float64'>
44.4776172215
<class 'numpy.float64'>
44.4769166079
<class 'numpy.float64'>
44.476216215
<class 'numpy.float64'>
44.4755160427
<class 'numpy.float64'>
44.4748160908
<class 'numpy.float64'>
44.4741163593
<class 'numpy.float64'>
44.4734168479
<class 'numpy.float64'>
44.4727175566
<class 'numpy.float64'>
44.4720184852
<class 'numpy.float64'>
44.4713196336
<class 'numpy.float64'>
44.4706210016
<class 'numpy.float64'>
44.4699225891
<class 'numpy.float64'>
44.469224396
<class 'numpy.float64'>
44.4685264222
<class 'numpy.float64'>
44.4678286674
<class 'numpy.float64'>
44.4671311317
<class 'numpy.float64'>
44.4664338147
<class 'numpy.float64'>
44.4657367165
<class 'numpy.float64'>
44.4650398368
<class 'numpy.float64'>
44.4643431756
<class 'numpy.float64'>
44.4636467327
<class 'numpy.float64'>
44.462950508
<class 'numpy.float64'>
44.4622545012
<class 'numpy.float64'>
44.4615587124
<class 'numpy.float64'>
44.4608631414
<class 'numpy.float64'>
44.460167788
<class 'numpy.float64'>
44.4594726521
<class 'numpy.float64'>
44.4587777336
<class 'numpy.float64'>
44.4580830323
<class 'numpy.float64'>
44.4573885482
<class 'numpy.float64'>
44.456694281
<class 'numpy.float64'>
44.4560002307
<class 'numpy.float64'>
44.4553063971
<class 'numpy.float64'>
44.45461278
<class 'numpy.float64'>
44.4539193794
<class 'numpy.float64'>
44.4532261952
<class 'numpy.float64'>
44.4525332271
<class 'numpy.float64'>
44.4518404751
<class 'numpy.float64'>
44.451147939
<class 'numpy.float64'>
44.4504556187
<class 'numpy.float64'>
44.4497635141
<class 'numpy.float64'>
44.449071625
<class 'numpy.float64'>
44.4483799513
<class 'numpy.float64'>
Cost after iteration 5000: 44.448380
44.4476884929
<class 'numpy.float64'>
44.4469972497
<class 'numpy.float64'>
44.4463062215
<class 'numpy.float64'>
44.4456154081
<class 'numpy.float64'>
44.4449248095
<class 'numpy.float64'>
44.4442344256
<class 'numpy.float64'>
44.4435442561
<class 'numpy.float64'>
44.442854301
<class 'numpy.float64'>
44.4421645602
<class 'numpy.float64'>
44.4414750334
<class 'numpy.float64'>
44.4407857207
<class 'numpy.float64'>
44.4400966218
<class 'numpy.float64'>
44.4394077366
<class 'numpy.float64'>
44.438719065
<class 'numpy.float64'>
44.4380306069
<class 'numpy.float64'>
44.4373423622
<class 'numpy.float64'>
44.4366543306
<class 'numpy.float64'>
44.4359665122
<class 'numpy.float64'>
44.4352789067
<class 'numpy.float64'>
44.434591514
<class 'numpy.float64'>
44.433904334
<class 'numpy.float64'>
44.4332173666
<class 'numpy.float64'>
44.4325306117
<class 'numpy.float64'>
44.4318440691
<class 'numpy.float64'>
44.4311577387
<class 'numpy.float64'>
44.4304716203
<class 'numpy.float64'>
44.4297857139
<class 'numpy.float64'>
44.4291000194
<class 'numpy.float64'>
44.4284145365
<class 'numpy.float64'>
44.4277292651
<class 'numpy.float64'>
44.4270442052
<class 'numpy.float64'>
44.4263593567
<class 'numpy.float64'>
44.4256747193
<class 'numpy.float64'>
44.424990293
<class 'numpy.float64'>
44.4243060776
<class 'numpy.float64'>
44.423622073
<class 'numpy.float64'>
44.4229382791
<class 'numpy.float64'>
44.4222546958
<class 'numpy.float64'>
44.4215713229
<class 'numpy.float64'>
44.4208881603
<class 'numpy.float64'>
44.4202052079
<class 'numpy.float64'>
44.4195224656
<class 'numpy.float64'>
44.4188399332
<class 'numpy.float64'>
44.4181576106
<class 'numpy.float64'>
44.4174754978
<class 'numpy.float64'>
44.4167935944
<class 'numpy.float64'>
44.4161119006
<class 'numpy.float64'>
44.415430416
<class 'numpy.float64'>
44.4147491406
<class 'numpy.float64'>
44.4140680744
<class 'numpy.float64'>
44.413387217
<class 'numpy.float64'>
44.4127065685
<class 'numpy.float64'>
44.4120261287
<class 'numpy.float64'>
44.4113458975
<class 'numpy.float64'>
44.4106658747
<class 'numpy.float64'>
44.4099860603
<class 'numpy.float64'>
44.409306454
<class 'numpy.float64'>
44.4086270559
<class 'numpy.float64'>
44.4079478657
<class 'numpy.float64'>
44.4072688834
<class 'numpy.float64'>
44.4065901088
<class 'numpy.float64'>
44.4059115418
<class 'numpy.float64'>
44.4052331823
<class 'numpy.float64'>
44.4045550301
<class 'numpy.float64'>
44.4038770852
<class 'numpy.float64'>
44.4031993474
<class 'numpy.float64'>
44.4025218166
<class 'numpy.float64'>
44.4018444927
<class 'numpy.float64'>
44.4011673755
<class 'numpy.float64'>
44.4004904649
<class 'numpy.float64'>
44.3998137609
<class 'numpy.float64'>
44.3991372632
<class 'numpy.float64'>
44.3984609719
<class 'numpy.float64'>
44.3977848866
<class 'numpy.float64'>
44.3971090074
<class 'numpy.float64'>
44.3964333342
<class 'numpy.float64'>
44.3957578667
<class 'numpy.float64'>
44.3950826049
<class 'numpy.float64'>
44.3944075486
<class 'numpy.float64'>
44.3937326977
<class 'numpy.float64'>
44.3930580522
<class 'numpy.float64'>
44.3923836119
<class 'numpy.float64'>
44.3917093766
<class 'numpy.float64'>
44.3910353463
<class 'numpy.float64'>
44.3903615209
<class 'numpy.float64'>
44.3896879001
<class 'numpy.float64'>
44.389014484
<class 'numpy.float64'>
44.3883412723
<class 'numpy.float64'>
44.387668265
<class 'numpy.float64'>
44.3869954619
<class 'numpy.float64'>
44.386322863
<class 'numpy.float64'>
44.3856504681
<class 'numpy.float64'>
44.384978277
<class 'numpy.float64'>
44.3843062898
<class 'numpy.float64'>
44.3836345062
<class 'numpy.float64'>
44.3829629261
<class 'numpy.float64'>
44.3822915495
<class 'numpy.float64'>
44.3816203761
<class 'numpy.float64'>
44.380949406
<class 'numpy.float64'>
44.3802786389
<class 'numpy.float64'>
44.3796080748
<class 'numpy.float64'>
44.3789377135
<class 'numpy.float64'>
44.3782675549
<class 'numpy.float64'>
44.3775975989
<class 'numpy.float64'>
44.3769278454
<class 'numpy.float64'>
44.3762582943
<class 'numpy.float64'>
44.3755889455
<class 'numpy.float64'>
44.3749197987
<class 'numpy.float64'>
44.3742508541
<class 'numpy.float64'>
44.3735821113
<class 'numpy.float64'>
44.3729135703
<class 'numpy.float64'>
44.372245231
<class 'numpy.float64'>
44.3715770932
<class 'numpy.float64'>
44.3709091569
<class 'numpy.float64'>
44.3702414219
<class 'numpy.float64'>
44.3695738881
<class 'numpy.float64'>
44.3689065555
<class 'numpy.float64'>
44.3682394238
<class 'numpy.float64'>
44.367572493
<class 'numpy.float64'>
44.366905763
<class 'numpy.float64'>
44.3662392336
<class 'numpy.float64'>
44.3655729047
<class 'numpy.float64'>
44.3649067763
<class 'numpy.float64'>
44.3642408481
<class 'numpy.float64'>
44.3635751201
<class 'numpy.float64'>
44.3629095923
<class 'numpy.float64'>
44.3622442643
<class 'numpy.float64'>
44.3615791363
<class 'numpy.float64'>
44.3609142079
<class 'numpy.float64'>
44.3602494792
<class 'numpy.float64'>
44.35958495
<class 'numpy.float64'>
44.3589206201
<class 'numpy.float64'>
44.3582564896
<class 'numpy.float64'>
44.3575925582
<class 'numpy.float64'>
44.3569288259
<class 'numpy.float64'>
44.3562652926
<class 'numpy.float64'>
44.355601958
<class 'numpy.float64'>
44.3549388222
<class 'numpy.float64'>
44.354275885
<class 'numpy.float64'>
44.3536131463
<class 'numpy.float64'>
44.352950606
<class 'numpy.float64'>
44.3522882639
<class 'numpy.float64'>
44.35162612
<class 'numpy.float64'>
44.3509641742
<class 'numpy.float64'>
44.3503024263
<class 'numpy.float64'>
44.3496408762
<class 'numpy.float64'>
44.3489795238
<class 'numpy.float64'>
44.3483183691
<class 'numpy.float64'>
44.3476574118
<class 'numpy.float64'>
44.346996652
<class 'numpy.float64'>
44.3463360893
<class 'numpy.float64'>
44.3456757239
<class 'numpy.float64'>
44.3450155555
<class 'numpy.float64'>
44.3443555841
<class 'numpy.float64'>
44.3436958094
<class 'numpy.float64'>
44.3430362315
<class 'numpy.float64'>
44.3423768503
<class 'numpy.float64'>
44.3417176655
<class 'numpy.float64'>
44.3410586771
<class 'numpy.float64'>
44.340399885
<class 'numpy.float64'>
44.339741289
<class 'numpy.float64'>
44.3390828892
<class 'numpy.float64'>
44.3384246852
<class 'numpy.float64'>
44.3377666772
<class 'numpy.float64'>
44.3371088648
<class 'numpy.float64'>
44.3364512481
<class 'numpy.float64'>
44.3357938269
<class 'numpy.float64'>
44.3351366012
<class 'numpy.float64'>
44.3344795707
<class 'numpy.float64'>
44.3338227354
<class 'numpy.float64'>
44.3331660952
<class 'numpy.float64'>
44.33250965
<class 'numpy.float64'>
44.3318533996
<class 'numpy.float64'>
44.331197344
<class 'numpy.float64'>
44.3305414831
<class 'numpy.float64'>
44.3298858167
<class 'numpy.float64'>
44.3292303448
<class 'numpy.float64'>
44.3285750671
<class 'numpy.float64'>
44.3279199837
<class 'numpy.float64'>
44.3272650945
<class 'numpy.float64'>
44.3266103992
<class 'numpy.float64'>
44.3259558978
<class 'numpy.float64'>
44.3253015902
<class 'numpy.float64'>
44.3246474764
<class 'numpy.float64'>
44.3239935561
<class 'numpy.float64'>
44.3233398292
<class 'numpy.float64'>
44.3226862958
<class 'numpy.float64'>
44.3220329556
<class 'numpy.float64'>
44.3213798085
<class 'numpy.float64'>
44.3207268546
<class 'numpy.float64'>
44.3200740935
<class 'numpy.float64'>
44.3194215253
<class 'numpy.float64'>
44.3187691499
<class 'numpy.float64'>
44.318116967
<class 'numpy.float64'>
44.3174649767
<class 'numpy.float64'>
44.3168131788
<class 'numpy.float64'>
44.3161615732
<class 'numpy.float64'>
44.3155101598
<class 'numpy.float64'>
44.3148589386
<class 'numpy.float64'>
44.3142079093
<class 'numpy.float64'>
44.3135570719
<class 'numpy.float64'>
44.3129064263
<class 'numpy.float64'>
44.3122559724
<class 'numpy.float64'>
44.31160571
<class 'numpy.float64'>
44.3109556392
<class 'numpy.float64'>
44.3103057597
<class 'numpy.float64'>
44.3096560715
<class 'numpy.float64'>
44.3090065744
<class 'numpy.float64'>
44.3083572684
<class 'numpy.float64'>
44.3077081533
<class 'numpy.float64'>
44.3070592291
<class 'numpy.float64'>
44.3064104957
<class 'numpy.float64'>
44.3057619529
<class 'numpy.float64'>
44.3051136006
<class 'numpy.float64'>
44.3044654388
<class 'numpy.float64'>
44.3038174673
<class 'numpy.float64'>
44.303169686
<class 'numpy.float64'>
44.3025220949
<class 'numpy.float64'>
44.3018746938
<class 'numpy.float64'>
44.3012274826
<class 'numpy.float64'>
44.3005804612
<class 'numpy.float64'>
44.2999336296
<class 'numpy.float64'>
44.2992869876
<class 'numpy.float64'>
44.2986405351
<class 'numpy.float64'>
44.297994272
<class 'numpy.float64'>
44.2973481982
<class 'numpy.float64'>
44.2967023136
<class 'numpy.float64'>
44.2960566181
<class 'numpy.float64'>
44.2954111117
<class 'numpy.float64'>
44.2947657941
<class 'numpy.float64'>
44.2941206653
<class 'numpy.float64'>
44.2934757253
<class 'numpy.float64'>
44.2928309738
<class 'numpy.float64'>
44.2921864108
<class 'numpy.float64'>
44.2915420363
<class 'numpy.float64'>
44.29089785
<class 'numpy.float64'>
44.2902538519
<class 'numpy.float64'>
44.289610042
<class 'numpy.float64'>
44.28896642
<class 'numpy.float64'>
44.2883229859
<class 'numpy.float64'>
44.2876797396
<class 'numpy.float64'>
44.287036681
<class 'numpy.float64'>
44.28639381
<class 'numpy.float64'>
44.2857511265
<class 'numpy.float64'>
44.2851086304
<class 'numpy.float64'>
44.2844663216
<class 'numpy.float64'>
44.2838242
<class 'numpy.float64'>
44.2831822655
<class 'numpy.float64'>
44.2825405179
<class 'numpy.float64'>
44.2818989573
<class 'numpy.float64'>
44.2812575835
<class 'numpy.float64'>
44.2806163963
<class 'numpy.float64'>
44.2799753958
<class 'numpy.float64'>
44.2793345818
<class 'numpy.float64'>
44.2786939542
<class 'numpy.float64'>
44.2780535128
<class 'numpy.float64'>
44.2774132577
<class 'numpy.float64'>
44.2767731887
<class 'numpy.float64'>
44.2761333057
<class 'numpy.float64'>
44.2754936086
<class 'numpy.float64'>
44.2748540973
<class 'numpy.float64'>
44.2742147718
<class 'numpy.float64'>
44.2735756318
<class 'numpy.float64'>
44.2729366774
<class 'numpy.float64'>
44.2722979084
<class 'numpy.float64'>
44.2716593247
<class 'numpy.float64'>
44.2710209263
<class 'numpy.float64'>
44.2703827129
<class 'numpy.float64'>
44.2697446846
<class 'numpy.float64'>
44.2691068413
<class 'numpy.float64'>
44.2684691828
<class 'numpy.float64'>
44.267831709
<class 'numpy.float64'>
44.2671944198
<class 'numpy.float64'>
44.2665573152
<class 'numpy.float64'>
44.2659203951
<class 'numpy.float64'>
44.2652836593
<class 'numpy.float64'>
44.2646471078
<class 'numpy.float64'>
44.2640107404
<class 'numpy.float64'>
44.2633745571
<class 'numpy.float64'>
44.2627385578
<class 'numpy.float64'>
44.2621027423
<class 'numpy.float64'>
44.2614671107
<class 'numpy.float64'>
44.2608316627
<class 'numpy.float64'>
44.2601963983
<class 'numpy.float64'>
44.2595613173
<class 'numpy.float64'>
44.2589264198
<class 'numpy.float64'>
44.2582917056
<class 'numpy.float64'>
44.2576571746
<class 'numpy.float64'>
44.2570228267
<class 'numpy.float64'>
44.2563886618
<class 'numpy.float64'>
44.2557546798
<class 'numpy.float64'>
44.2551208806
<class 'numpy.float64'>
44.2544872642
<class 'numpy.float64'>
44.2538538304
<class 'numpy.float64'>
44.2532205792
<class 'numpy.float64'>
44.2525875104
<class 'numpy.float64'>
44.2519546239
<class 'numpy.float64'>
44.2513219198
<class 'numpy.float64'>
44.2506893977
<class 'numpy.float64'>
44.2500570578
<class 'numpy.float64'>
44.2494248998
<class 'numpy.float64'>
44.2487929237
<class 'numpy.float64'>
44.2481611294
<class 'numpy.float64'>
44.2475295168
<class 'numpy.float64'>
44.2468980858
<class 'numpy.float64'>
44.2462668363
<class 'numpy.float64'>
44.2456357683
<class 'numpy.float64'>
44.2450048815
<class 'numpy.float64'>
44.244374176
<class 'numpy.float64'>
44.2437436516
<class 'numpy.float64'>
44.2431133083
<class 'numpy.float64'>
44.2424831459
<class 'numpy.float64'>
44.2418531643
<class 'numpy.float64'>
44.2412233636
<class 'numpy.float64'>
44.2405937435
<class 'numpy.float64'>
44.239964304
<class 'numpy.float64'>
44.239335045
<class 'numpy.float64'>
44.2387059663
<class 'numpy.float64'>
44.238077068
<class 'numpy.float64'>
44.2374483499
<class 'numpy.float64'>
44.2368198119
<class 'numpy.float64'>
44.236191454
<class 'numpy.float64'>
44.235563276
<class 'numpy.float64'>
44.2349352778
<class 'numpy.float64'>
44.2343074594
<class 'numpy.float64'>
44.2336798206
<class 'numpy.float64'>
44.2330523615
<class 'numpy.float64'>
44.2324250818
<class 'numpy.float64'>
44.2317979815
<class 'numpy.float64'>
44.2311710605
<class 'numpy.float64'>
44.2305443187
<class 'numpy.float64'>
44.229917756
<class 'numpy.float64'>
44.2292913724
<class 'numpy.float64'>
44.2286651677
<class 'numpy.float64'>
44.2280391418
<class 'numpy.float64'>
44.2274132947
<class 'numpy.float64'>
44.2267876263
<class 'numpy.float64'>
44.2261621365
<class 'numpy.float64'>
44.2255368251
<class 'numpy.float64'>
44.2249116921
<class 'numpy.float64'>
44.2242867375
<class 'numpy.float64'>
44.223661961
<class 'numpy.float64'>
44.2230373627
<class 'numpy.float64'>
44.2224129424
<class 'numpy.float64'>
44.2217887001
<class 'numpy.float64'>
44.2211646356
<class 'numpy.float64'>
44.2205407489
<class 'numpy.float64'>
44.2199170399
<class 'numpy.float64'>
44.2192935085
<class 'numpy.float64'>
44.2186701546
<class 'numpy.float64'>
44.2180469781
<class 'numpy.float64'>
44.2174239789
<class 'numpy.float64'>
44.216801157
<class 'numpy.float64'>
44.2161785122
<class 'numpy.float64'>
44.2155560445
<class 'numpy.float64'>
44.2149337537
<class 'numpy.float64'>
44.2143116399
<class 'numpy.float64'>
44.2136897028
<class 'numpy.float64'>
44.2130679424
<class 'numpy.float64'>
44.2124463587
<class 'numpy.float64'>
44.2118249515
<class 'numpy.float64'>
44.2112037207
<class 'numpy.float64'>
44.2105826663
<class 'numpy.float64'>
44.2099617882
<class 'numpy.float64'>
44.2093410862
<class 'numpy.float64'>
44.2087205604
<class 'numpy.float64'>
44.2081002105
<class 'numpy.float64'>
44.2074800366
<class 'numpy.float64'>
44.2068600385
<class 'numpy.float64'>
44.2062402162
<class 'numpy.float64'>
44.2056205695
<class 'numpy.float64'>
44.2050010984
<class 'numpy.float64'>
44.2043818028
<class 'numpy.float64'>
44.2037626826
<class 'numpy.float64'>
44.2031437377
<class 'numpy.float64'>
44.202524968
<class 'numpy.float64'>
44.2019063735
<class 'numpy.float64'>
44.201287954
<class 'numpy.float64'>
44.2006697095
<class 'numpy.float64'>
44.2000516399
<class 'numpy.float64'>
44.1994337451
<class 'numpy.float64'>
44.198816025
<class 'numpy.float64'>
44.1981984795
<class 'numpy.float64'>
44.1975811086
<class 'numpy.float64'>
44.1969639121
<class 'numpy.float64'>
44.19634689
<class 'numpy.float64'>
44.1957300421
<class 'numpy.float64'>
44.1951133685
<class 'numpy.float64'>
44.194496869
<class 'numpy.float64'>
44.1938805435
<class 'numpy.float64'>
44.193264392
<class 'numpy.float64'>
44.1926484143
<class 'numpy.float64'>
44.1920326104
<class 'numpy.float64'>
44.1914169802
<class 'numpy.float64'>
44.1908015236
<class 'numpy.float64'>
44.1901862405
<class 'numpy.float64'>
44.1895711309
<class 'numpy.float64'>
44.1889561946
<class 'numpy.float64'>
44.1883414316
<class 'numpy.float64'>
44.1877268417
<class 'numpy.float64'>
44.187112425
<class 'numpy.float64'>
44.1864981813
<class 'numpy.float64'>
44.1858841105
<class 'numpy.float64'>
44.1852702126
<class 'numpy.float64'>
44.1846564874
<class 'numpy.float64'>
44.1840429349
<class 'numpy.float64'>
44.1834295551
<class 'numpy.float64'>
44.1828163477
<class 'numpy.float64'>
44.1822033128
<class 'numpy.float64'>
44.1815904502
<class 'numpy.float64'>
44.1809777599
<class 'numpy.float64'>
44.1803652418
<class 'numpy.float64'>
44.1797528958
<class 'numpy.float64'>
44.1791407217
<class 'numpy.float64'>
44.1785287197
<class 'numpy.float64'>
44.1779168895
<class 'numpy.float64'>
44.177305231
<class 'numpy.float64'>
44.1766937443
<class 'numpy.float64'>
44.1760824291
<class 'numpy.float64'>
44.1754712855
<class 'numpy.float64'>
44.1748603133
<class 'numpy.float64'>
44.1742495125
<class 'numpy.float64'>
44.173638883
<class 'numpy.float64'>
44.1730284246
<class 'numpy.float64'>
44.1724181374
<class 'numpy.float64'>
44.1718080212
<class 'numpy.float64'>
44.171198076
<class 'numpy.float64'>
44.1705883016
<class 'numpy.float64'>
44.1699786981
<class 'numpy.float64'>
44.1693692652
<class 'numpy.float64'>
44.1687600029
<class 'numpy.float64'>
44.1681509112
<class 'numpy.float64'>
44.16754199
<class 'numpy.float64'>
44.1669332392
<class 'numpy.float64'>
44.1663246586
<class 'numpy.float64'>
44.1657162483
<class 'numpy.float64'>
44.1651080081
<class 'numpy.float64'>
44.164499938
<class 'numpy.float64'>
44.1638920379
<class 'numpy.float64'>
44.1632843076
<class 'numpy.float64'>
44.1626767472
<class 'numpy.float64'>
44.1620693566
<class 'numpy.float64'>
44.1614621355
<class 'numpy.float64'>
44.1608550841
<class 'numpy.float64'>
44.1602482022
<class 'numpy.float64'>
44.1596414897
<class 'numpy.float64'>
44.1590349465
<class 'numpy.float64'>
44.1584285726
<class 'numpy.float64'>
44.1578223679
<class 'numpy.float64'>
44.1572163322
<class 'numpy.float64'>
44.1566104656
<class 'numpy.float64'>
44.156004768
<class 'numpy.float64'>
44.1553992392
<class 'numpy.float64'>
44.1547938792
<class 'numpy.float64'>
44.1541886879
<class 'numpy.float64'>
44.1535836652
<class 'numpy.float64'>
44.1529788111
<class 'numpy.float64'>
44.1523741254
<class 'numpy.float64'>
44.1517696082
<class 'numpy.float64'>
44.1511652592
<class 'numpy.float64'>
44.1505610785
<class 'numpy.float64'>
44.149957066
<class 'numpy.float64'>
44.1493532215
<class 'numpy.float64'>
44.148749545
<class 'numpy.float64'>
44.1481460365
<class 'numpy.float64'>
44.1475426957
<class 'numpy.float64'>
44.1469395228
<class 'numpy.float64'>
44.1463365175
<class 'numpy.float64'>
44.1457336799
<class 'numpy.float64'>
44.1451310098
<class 'numpy.float64'>
44.1445285071
<class 'numpy.float64'>
44.1439261718
<class 'numpy.float64'>
44.1433240038
<class 'numpy.float64'>
44.142722003
<class 'numpy.float64'>
44.1421201694
<class 'numpy.float64'>
44.1415185029
<class 'numpy.float64'>
44.1409170033
<class 'numpy.float64'>
44.1403156707
<class 'numpy.float64'>
44.1397145049
<class 'numpy.float64'>
44.1391135058
<class 'numpy.float64'>
44.1385126734
<class 'numpy.float64'>
44.1379120077
<class 'numpy.float64'>
44.1373115085
<class 'numpy.float64'>
44.1367111757
<class 'numpy.float64'>
44.1361110093
<class 'numpy.float64'>
44.1355110092
<class 'numpy.float64'>
44.1349111753
<class 'numpy.float64'>
44.1343115076
<class 'numpy.float64'>
44.133712006
<class 'numpy.float64'>
44.1331126703
<class 'numpy.float64'>
44.1325135006
<class 'numpy.float64'>
44.1319144967
<class 'numpy.float64'>
44.1313156586
<class 'numpy.float64'>
44.1307169861
<class 'numpy.float64'>
44.1301184793
<class 'numpy.float64'>
44.1295201381
<class 'numpy.float64'>
44.1289219623
<class 'numpy.float64'>
44.1283239519
<class 'numpy.float64'>
44.1277261068
<class 'numpy.float64'>
44.127128427
<class 'numpy.float64'>
44.1265309123
<class 'numpy.float64'>
44.1259335627
<class 'numpy.float64'>
44.1253363781
<class 'numpy.float64'>
44.1247393585
<class 'numpy.float64'>
44.1241425038
<class 'numpy.float64'>
44.1235458138
<class 'numpy.float64'>
44.1229492886
<class 'numpy.float64'>
44.122352928
<class 'numpy.float64'>
44.1217567319
<class 'numpy.float64'>
44.1211607004
<class 'numpy.float64'>
44.1205648333
<class 'numpy.float64'>
44.1199691305
<class 'numpy.float64'>
44.119373592
<class 'numpy.float64'>
44.1187782177
<class 'numpy.float64'>
44.1181830076
<class 'numpy.float64'>
44.1175879614
<class 'numpy.float64'>
44.1169930793
<class 'numpy.float64'>
44.1163983611
<class 'numpy.float64'>
44.1158038067
<class 'numpy.float64'>
44.115209416
<class 'numpy.float64'>
44.1146151891
<class 'numpy.float64'>
44.1140211257
<class 'numpy.float64'>
44.1134272259
<class 'numpy.float64'>
44.1128334895
<class 'numpy.float64'>
44.1122399166
<class 'numpy.float64'>
44.1116465069
<class 'numpy.float64'>
44.1110532605
<class 'numpy.float64'>
44.1104601773
<class 'numpy.float64'>
44.1098672572
<class 'numpy.float64'>
44.1092745001
<class 'numpy.float64'>
44.108681906
<class 'numpy.float64'>
44.1080894747
<class 'numpy.float64'>
44.1074972062
<class 'numpy.float64'>
44.1069051005
<class 'numpy.float64'>
44.1063131575
<class 'numpy.float64'>
44.105721377
<class 'numpy.float64'>
44.1051297591
<class 'numpy.float64'>
44.1045383036
<class 'numpy.float64'>
44.1039470105
<class 'numpy.float64'>
44.1033558797
<class 'numpy.float64'>
44.1027649111
<class 'numpy.float64'>
44.1021741047
<class 'numpy.float64'>
44.1015834604
<class 'numpy.float64'>
44.1009929781
<class 'numpy.float64'>
44.1004026578
<class 'numpy.float64'>
44.0998124993
<class 'numpy.float64'>
44.0992225026
<class 'numpy.float64'>
44.0986326677
<class 'numpy.float64'>
44.0980429944
<class 'numpy.float64'>
44.0974534827
<class 'numpy.float64'>
44.0968641325
<class 'numpy.float64'>
44.0962749438
<class 'numpy.float64'>
44.0956859165
<class 'numpy.float64'>
44.0950970505
<class 'numpy.float64'>
44.0945083457
<class 'numpy.float64'>
44.093919802
<class 'numpy.float64'>
44.0933314195
<class 'numpy.float64'>
44.092743198
<class 'numpy.float64'>
44.0921551375
<class 'numpy.float64'>
44.0915672378
<class 'numpy.float64'>
44.0909794989
<class 'numpy.float64'>
44.0903919208
<class 'numpy.float64'>
44.0898045034
<class 'numpy.float64'>
44.0892172466
<class 'numpy.float64'>
44.0886301503
<class 'numpy.float64'>
44.0880432145
<class 'numpy.float64'>
44.087456439
<class 'numpy.float64'>
44.0868698239
<class 'numpy.float64'>
44.0862833691
<class 'numpy.float64'>
44.0856970744
<class 'numpy.float64'>
44.0851109399
<class 'numpy.float64'>
44.0845249654
<class 'numpy.float64'>
44.0839391508
<class 'numpy.float64'>
44.0833534962
<class 'numpy.float64'>
44.0827680014
<class 'numpy.float64'>
44.0821826664
<class 'numpy.float64'>
44.0815974911
<class 'numpy.float64'>
44.0810124755
<class 'numpy.float64'>
44.0804276194
<class 'numpy.float64'>
44.0798429228
<class 'numpy.float64'>
44.0792583856
<class 'numpy.float64'>
44.0786740078
<class 'numpy.float64'>
44.0780897892
<class 'numpy.float64'>
44.0775057299
<class 'numpy.float64'>
44.0769218298
<class 'numpy.float64'>
44.0763380887
<class 'numpy.float64'>
44.0757545066
<class 'numpy.float64'>
44.0751710835
<class 'numpy.float64'>
44.0745878193
<class 'numpy.float64'>
44.0740047139
<class 'numpy.float64'>
44.0734217672
<class 'numpy.float64'>
44.0728389792
<class 'numpy.float64'>
44.0722563498
<class 'numpy.float64'>
44.071673879
<class 'numpy.float64'>
44.0710915666
<class 'numpy.float64'>
44.0705094126
<class 'numpy.float64'>
44.069927417
<class 'numpy.float64'>
44.0693455796
<class 'numpy.float64'>
44.0687639005
<class 'numpy.float64'>
44.0681823794
<class 'numpy.float64'>
44.0676010165
<class 'numpy.float64'>
44.0670198115
<class 'numpy.float64'>
44.0664387645
<class 'numpy.float64'>
44.0658578754
<class 'numpy.float64'>
44.065277144
<class 'numpy.float64'>
44.0646965704
<class 'numpy.float64'>
44.0641161544
<class 'numpy.float64'>
44.0635358961
<class 'numpy.float64'>
44.0629557953
<class 'numpy.float64'>
44.0623758519
<class 'numpy.float64'>
44.061796066
<class 'numpy.float64'>
44.0612164374
<class 'numpy.float64'>
44.060636966
<class 'numpy.float64'>
44.0600576519
<class 'numpy.float64'>
44.0594784949
<class 'numpy.float64'>
44.0588994949
<class 'numpy.float64'>
44.058320652
<class 'numpy.float64'>
44.057741966
<class 'numpy.float64'>
44.0571634369
<class 'numpy.float64'>
44.0565850645
<class 'numpy.float64'>
44.056006849
<class 'numpy.float64'>
44.0554287901
<class 'numpy.float64'>
44.0548508878
<class 'numpy.float64'>
44.054273142
<class 'numpy.float64'>
44.0536955527
<class 'numpy.float64'>
44.0531181198
<class 'numpy.float64'>
44.0525408433
<class 'numpy.float64'>
44.0519637231
<class 'numpy.float64'>
44.0513867591
<class 'numpy.float64'>
44.0508099512
<class 'numpy.float64'>
44.0502332994
<class 'numpy.float64'>
44.0496568036
<class 'numpy.float64'>
44.0490804638
<class 'numpy.float64'>
44.0485042799
<class 'numpy.float64'>
44.0479282518
<class 'numpy.float64'>
44.0473523795
<class 'numpy.float64'>
44.0467766628
<class 'numpy.float64'>
44.0462011018
<class 'numpy.float64'>
44.0456256964
<class 'numpy.float64'>
44.0450504465
<class 'numpy.float64'>
44.044475352
<class 'numpy.float64'>
44.0439004128
<class 'numpy.float64'>
44.043325629
<class 'numpy.float64'>
44.0427510005
<class 'numpy.float64'>
44.0421765271
<class 'numpy.float64'>
44.0416022088
<class 'numpy.float64'>
44.0410280456
<class 'numpy.float64'>
44.0404540374
<class 'numpy.float64'>
44.0398801841
<class 'numpy.float64'>
44.0393064856
<class 'numpy.float64'>
44.038732942
<class 'numpy.float64'>
44.0381595531
<class 'numpy.float64'>
44.0375863189
<class 'numpy.float64'>
44.0370132393
<class 'numpy.float64'>
44.0364403142
<class 'numpy.float64'>
44.0358675436
<class 'numpy.float64'>
44.0352949274
<class 'numpy.float64'>
44.0347224656
<class 'numpy.float64'>
44.034150158
<class 'numpy.float64'>
44.0335780047
<class 'numpy.float64'>
44.0330060056
<class 'numpy.float64'>
44.0324341605
<class 'numpy.float64'>
44.0318624695
<class 'numpy.float64'>
44.0312909325
<class 'numpy.float64'>
44.0307195494
<class 'numpy.float64'>
44.0301483201
<class 'numpy.float64'>
44.0295772447
<class 'numpy.float64'>
44.0290063229
<class 'numpy.float64'>
44.0284355548
<class 'numpy.float64'>
44.0278649403
<class 'numpy.float64'>
44.0272944794
<class 'numpy.float64'>
44.0267241719
<class 'numpy.float64'>
44.0261540178
<class 'numpy.float64'>
44.0255840171
<class 'numpy.float64'>
44.0250141697
<class 'numpy.float64'>
44.0244444755
<class 'numpy.float64'>
44.0238749344
<class 'numpy.float64'>
44.0233055465
<class 'numpy.float64'>
44.0227363116
<class 'numpy.float64'>
44.0221672297
<class 'numpy.float64'>
44.0215983007
<class 'numpy.float64'>
44.0210295246
<class 'numpy.float64'>
44.0204609012
<class 'numpy.float64'>
44.0198924306
<class 'numpy.float64'>
44.0193241126
<class 'numpy.float64'>
44.0187559473
<class 'numpy.float64'>
44.0181879345
<class 'numpy.float64'>
44.0176200742
<class 'numpy.float64'>
44.0170523663
<class 'numpy.float64'>
44.0164848108
<class 'numpy.float64'>
44.0159174076
<class 'numpy.float64'>
44.0153501566
<class 'numpy.float64'>
44.0147830578
<class 'numpy.float64'>
44.0142161111
<class 'numpy.float64'>
44.0136493165
<class 'numpy.float64'>
44.0130826739
<class 'numpy.float64'>
44.0125161832
<class 'numpy.float64'>
44.0119498444
<class 'numpy.float64'>
44.0113836574
<class 'numpy.float64'>
44.0108176221
<class 'numpy.float64'>
44.0102517386
<class 'numpy.float64'>
44.0096860067
<class 'numpy.float64'>
44.0091204263
<class 'numpy.float64'>
44.0085549975
<class 'numpy.float64'>
44.0079897201
<class 'numpy.float64'>
44.0074245941
<class 'numpy.float64'>
44.0068596195
<class 'numpy.float64'>
44.0062947961
<class 'numpy.float64'>
44.0057301239
<class 'numpy.float64'>
44.0051656029
<class 'numpy.float64'>
44.0046012329
<class 'numpy.float64'>
44.004037014
<class 'numpy.float64'>
44.0034729461
<class 'numpy.float64'>
44.0029090291
<class 'numpy.float64'>
44.0023452629
<class 'numpy.float64'>
44.0017816475
<class 'numpy.float64'>
44.0012181829
<class 'numpy.float64'>
44.0006548689
<class 'numpy.float64'>
44.0000917056
<class 'numpy.float64'>
43.9995286928
<class 'numpy.float64'>
43.9989658305
<class 'numpy.float64'>
43.9984031186
<class 'numpy.float64'>
43.9978405571
<class 'numpy.float64'>
43.9972781459
<class 'numpy.float64'>
43.996715885
<class 'numpy.float64'>
43.9961537743
<class 'numpy.float64'>
43.9955918137
<class 'numpy.float64'>
43.9950300032
<class 'numpy.float64'>
43.9944683427
<class 'numpy.float64'>
43.9939068322
<class 'numpy.float64'>
43.9933454715
<class 'numpy.float64'>
43.9927842608
<class 'numpy.float64'>
43.9922231998
<class 'numpy.float64'>
43.9916622885
<class 'numpy.float64'>
43.9911015269
<class 'numpy.float64'>
43.9905409149
<class 'numpy.float64'>
43.9899804525
<class 'numpy.float64'>
43.9894201396
<class 'numpy.float64'>
43.9888599761
<class 'numpy.float64'>
43.9882999619
<class 'numpy.float64'>
43.9877400971
<class 'numpy.float64'>
43.9871803816
<class 'numpy.float64'>
43.9866208152
<class 'numpy.float64'>
43.986061398
<class 'numpy.float64'>
43.9855021299
<class 'numpy.float64'>
43.9849430108
<class 'numpy.float64'>
43.9843840407
<class 'numpy.float64'>
43.9838252195
<class 'numpy.float64'>
43.9832665471
<class 'numpy.float64'>
43.9827080236
<class 'numpy.float64'>
43.9821496488
<class 'numpy.float64'>
43.9815914226
<class 'numpy.float64'>
43.9810333451
<class 'numpy.float64'>
43.9804754161
<class 'numpy.float64'>
43.9799176357
<class 'numpy.float64'>
43.9793600037
<class 'numpy.float64'>
43.9788025201
<class 'numpy.float64'>
43.9782451848
<class 'numpy.float64'>
43.9776879979
<class 'numpy.float64'>
43.9771309591
<class 'numpy.float64'>
43.9765740685
<class 'numpy.float64'>
43.976017326
<class 'numpy.float64'>
43.9754607316
<class 'numpy.float64'>
43.9749042851
<class 'numpy.float64'>
43.9743479866
<class 'numpy.float64'>
43.9737918359
<class 'numpy.float64'>
43.9732358331
<class 'numpy.float64'>
43.9726799781
<class 'numpy.float64'>
43.9721242707
<class 'numpy.float64'>
43.971568711
<class 'numpy.float64'>
43.9710132989
<class 'numpy.float64'>
43.9704580344
<class 'numpy.float64'>
43.9699029173
<class 'numpy.float64'>
43.9693479476
<class 'numpy.float64'>
43.9687931254
<class 'numpy.float64'>
43.9682384504
<class 'numpy.float64'>
43.9676839227
<class 'numpy.float64'>
43.9671295421
<class 'numpy.float64'>
43.9665753088
<class 'numpy.float64'>
43.9660212225
<class 'numpy.float64'>
43.9654672832
<class 'numpy.float64'>
43.9649134909
<class 'numpy.float64'>
43.9643598455
<class 'numpy.float64'>
43.963806347
<class 'numpy.float64'>
43.9632529953
<class 'numpy.float64'>
43.9626997904
<class 'numpy.float64'>
43.9621467321
<class 'numpy.float64'>
43.9615938205
<class 'numpy.float64'>
43.9610410555
<class 'numpy.float64'>
43.960488437
<class 'numpy.float64'>
43.9599359649
<class 'numpy.float64'>
43.9593836393
<class 'numpy.float64'>
43.95883146
<class 'numpy.float64'>
43.9582794271
<class 'numpy.float64'>
43.9577275403
<class 'numpy.float64'>
43.9571757998
<class 'numpy.float64'>
43.9566242054
<class 'numpy.float64'>
43.9560727571
<class 'numpy.float64'>
43.9555214549
<class 'numpy.float64'>
43.9549702986
<class 'numpy.float64'>
43.9544192882
<class 'numpy.float64'>
43.9538684236
<class 'numpy.float64'>
43.9533177049
<class 'numpy.float64'>
43.9527671319
<class 'numpy.float64'>
43.9522167047
<class 'numpy.float64'>
43.951666423
<class 'numpy.float64'>
43.951116287
<class 'numpy.float64'>
43.9505662965
<class 'numpy.float64'>
43.9500164514
<class 'numpy.float64'>
43.9494667518
<class 'numpy.float64'>
43.9489171975
<class 'numpy.float64'>
43.9483677886
<class 'numpy.float64'>
43.9478185249
<class 'numpy.float64'>
43.9472694064
<class 'numpy.float64'>
43.9467204331
<class 'numpy.float64'>
43.9461716048
<class 'numpy.float64'>
43.9456229216
<class 'numpy.float64'>
43.9450743834
<class 'numpy.float64'>
43.9445259901
<class 'numpy.float64'>
43.9439777417
<class 'numpy.float64'>
43.9434296381
<class 'numpy.float64'>
43.9428816793
<class 'numpy.float64'>
43.9423338652
<class 'numpy.float64'>
43.9417861957
<class 'numpy.float64'>
43.9412386708
<class 'numpy.float64'>
43.9406912905
<class 'numpy.float64'>
43.9401440547
<class 'numpy.float64'>
43.9395969634
<class 'numpy.float64'>
43.9390500164
<class 'numpy.float64'>
43.9385032137
<class 'numpy.float64'>
43.9379565554
<class 'numpy.float64'>
43.9374100412
<class 'numpy.float64'>
43.9368636712
<class 'numpy.float64'>
43.9363174454
<class 'numpy.float64'>
43.9357713636
<class 'numpy.float64'>
43.9352254258
<class 'numpy.float64'>
43.934679632
<class 'numpy.float64'>
43.9341339821
<class 'numpy.float64'>
43.933588476
<class 'numpy.float64'>
43.9330431137
<class 'numpy.float64'>
43.9324978952
<class 'numpy.float64'>
43.9319528204
<class 'numpy.float64'>
43.9314078892
<class 'numpy.float64'>
43.9308631015
<class 'numpy.float64'>
43.9303184574
<class 'numpy.float64'>
43.9297739568
<class 'numpy.float64'>
43.9292295996
<class 'numpy.float64'>
43.9286853858
<class 'numpy.float64'>
43.9281413153
<class 'numpy.float64'>
43.9275973881
<class 'numpy.float64'>
43.927053604
<class 'numpy.float64'>
43.9265099631
<class 'numpy.float64'>
43.9259664654
<class 'numpy.float64'>
43.9254231106
<class 'numpy.float64'>
43.9248798989
<class 'numpy.float64'>
43.9243368301
<class 'numpy.float64'>
43.9237939043
<class 'numpy.float64'>
43.9232511212
<class 'numpy.float64'>
43.9227084809
<class 'numpy.float64'>
43.9221659834
<class 'numpy.float64'>
43.9216236286
<class 'numpy.float64'>
43.9210814164
<class 'numpy.float64'>
43.9205393467
<class 'numpy.float64'>
43.9199974196
<class 'numpy.float64'>
43.9194556349
<class 'numpy.float64'>
43.9189139927
<class 'numpy.float64'>
43.9183724929
<class 'numpy.float64'>
43.9178311353
<class 'numpy.float64'>
43.91728992
<class 'numpy.float64'>
43.9167488469
<class 'numpy.float64'>
43.916207916
<class 'numpy.float64'>
43.9156671272
<class 'numpy.float64'>
43.9151264804
<class 'numpy.float64'>
43.9145859757
<class 'numpy.float64'>
43.9140456128
<class 'numpy.float64'>
43.9135053919
<class 'numpy.float64'>
43.9129653128
<class 'numpy.float64'>
43.9124253755
<class 'numpy.float64'>
43.91188558
<class 'numpy.float64'>
43.9113459261
<class 'numpy.float64'>
43.9108064139
<class 'numpy.float64'>
43.9102670432
<class 'numpy.float64'>
43.9097278141
<class 'numpy.float64'>
43.9091887265
<class 'numpy.float64'>
43.9086497803
<class 'numpy.float64'>
43.9081109754
<class 'numpy.float64'>
43.9075723119
<class 'numpy.float64'>
43.9070337897
<class 'numpy.float64'>
43.9064954087
<class 'numpy.float64'>
43.9059571688
<class 'numpy.float64'>
43.9054190701
<class 'numpy.float64'>
43.9048811124
<class 'numpy.float64'>
43.9043432958
<class 'numpy.float64'>
43.9038056201
<class 'numpy.float64'>
43.9032680853
<class 'numpy.float64'>
43.9027306914
<class 'numpy.float64'>
43.9021934383
<class 'numpy.float64'>
43.901656326
<class 'numpy.float64'>
43.9011193543
<class 'numpy.float64'>
43.9005825233
<class 'numpy.float64'>
43.9000458329
<class 'numpy.float64'>
43.8995092831
<class 'numpy.float64'>
43.8989728738
<class 'numpy.float64'>
43.8984366049
<class 'numpy.float64'>
43.8979004764
<class 'numpy.float64'>
43.8973644883
<class 'numpy.float64'>
43.8968286404
<class 'numpy.float64'>
43.8962929328
<class 'numpy.float64'>
43.8957573654
<class 'numpy.float64'>
43.8952219382
<class 'numpy.float64'>
43.894686651
<class 'numpy.float64'>
43.8941515039
<class 'numpy.float64'>
43.8936164967
<class 'numpy.float64'>
43.8930816295
<class 'numpy.float64'>
43.8925469022
<class 'numpy.float64'>
43.8920123148
<class 'numpy.float64'>
43.8914778671
<class 'numpy.float64'>
43.8909435591
<class 'numpy.float64'>
43.8904093909
<class 'numpy.float64'>
43.8898753623
<class 'numpy.float64'>
43.8893414732
<class 'numpy.float64'>
43.8888077237
<class 'numpy.float64'>
43.8882741137
<class 'numpy.float64'>
43.8877406431
<class 'numpy.float64'>
43.8872073119
<class 'numpy.float64'>
43.88667412
<class 'numpy.float64'>
43.8861410674
<class 'numpy.float64'>
43.8856081541
<class 'numpy.float64'>
43.8850753799
<class 'numpy.float64'>
43.8845427448
<class 'numpy.float64'>
43.8840102488
<class 'numpy.float64'>
43.8834778919
<class 'numpy.float64'>
43.8829456739
<class 'numpy.float64'>
43.8824135949
<class 'numpy.float64'>
43.8818816547
<class 'numpy.float64'>
43.8813498534
<class 'numpy.float64'>
43.8808181909
<class 'numpy.float64'>
43.8802866671
<class 'numpy.float64'>
43.8797552819
<class 'numpy.float64'>
43.8792240354
<class 'numpy.float64'>
43.8786929275
<class 'numpy.float64'>
43.8781619581
<class 'numpy.float64'>
43.8776311271
<class 'numpy.float64'>
43.8771004346
<class 'numpy.float64'>
43.8765698805
<class 'numpy.float64'>
43.8760394647
<class 'numpy.float64'>
43.8755091872
<class 'numpy.float64'>
43.8749790479
<class 'numpy.float64'>
43.8744490468
<class 'numpy.float64'>
43.8739191838
<class 'numpy.float64'>
43.8733894589
<class 'numpy.float64'>
43.872859872
<class 'numpy.float64'>
43.8723304231
<class 'numpy.float64'>
43.8718011122
<class 'numpy.float64'>
43.8712719391
<class 'numpy.float64'>
43.8707429038
<class 'numpy.float64'>
43.8702140064
<class 'numpy.float64'>
43.8696852466
<class 'numpy.float64'>
43.8691566246
<class 'numpy.float64'>
43.8686281402
<class 'numpy.float64'>
43.8680997933
<class 'numpy.float64'>
43.867571584
<class 'numpy.float64'>
43.8670435122
<class 'numpy.float64'>
43.8665155778
<class 'numpy.float64'>
43.8659877808
<class 'numpy.float64'>
43.8654601211
<class 'numpy.float64'>
43.8649325988
<class 'numpy.float64'>
43.8644052136
<class 'numpy.float64'>
43.8638779657
<class 'numpy.float64'>
43.8633508548
<class 'numpy.float64'>
43.8628238811
<class 'numpy.float64'>
43.8622970444
<class 'numpy.float64'>
43.8617703447
<class 'numpy.float64'>
43.861243782
<class 'numpy.float64'>
43.8607173561
<class 'numpy.float64'>
43.8601910671
<class 'numpy.float64'>
43.8596649149
<class 'numpy.float64'>
43.8591388994
<class 'numpy.float64'>
43.8586130206
<class 'numpy.float64'>
43.8580872785
<class 'numpy.float64'>
43.8575616729
<class 'numpy.float64'>
43.8570362039
<class 'numpy.float64'>
43.8565108714
<class 'numpy.float64'>
43.8559856754
<class 'numpy.float64'>
43.8554606158
<class 'numpy.float64'>
43.8549356925
<class 'numpy.float64'>
43.8544109055
<class 'numpy.float64'>
43.8538862548
<class 'numpy.float64'>
43.8533617403
<class 'numpy.float64'>
43.8528373619
<class 'numpy.float64'>
43.8523131197
<class 'numpy.float64'>
43.8517890135
<class 'numpy.float64'>
43.8512650433
<class 'numpy.float64'>
43.8507412091
<class 'numpy.float64'>
43.8502175108
<class 'numpy.float64'>
43.8496939483
<class 'numpy.float64'>
43.8491705217
<class 'numpy.float64'>
43.8486472309
<class 'numpy.float64'>
43.8481240758
<class 'numpy.float64'>
43.8476010563
<class 'numpy.float64'>
Cost after iteration 6000: 43.847601
43.8470781725
<class 'numpy.float64'>
43.8465554242
<class 'numpy.float64'>
43.8460328115
<class 'numpy.float64'>
43.8455103342
<class 'numpy.float64'>
43.8449879924
<class 'numpy.float64'>
43.8444657859
<class 'numpy.float64'>
43.8439437148
<class 'numpy.float64'>
43.843421779
<class 'numpy.float64'>
43.8428999784
<class 'numpy.float64'>
43.842378313
<class 'numpy.float64'>
43.8418567827
<class 'numpy.float64'>
43.8413353875
<class 'numpy.float64'>
43.8408141274
<class 'numpy.float64'>
43.8402930022
<class 'numpy.float64'>
43.839772012
<class 'numpy.float64'>
43.8392511567
<class 'numpy.float64'>
43.8387304362
<class 'numpy.float64'>
43.8382098506
<class 'numpy.float64'>
43.8376893997
<class 'numpy.float64'>
43.8371690835
<class 'numpy.float64'>
43.8366489019
<class 'numpy.float64'>
43.836128855
<class 'numpy.float64'>
43.8356089426
<class 'numpy.float64'>
43.8350891647
<class 'numpy.float64'>
43.8345695213
<class 'numpy.float64'>
43.8340500123
<class 'numpy.float64'>
43.8335306377
<class 'numpy.float64'>
43.8330113974
<class 'numpy.float64'>
43.8324922914
<class 'numpy.float64'>
43.8319733195
<class 'numpy.float64'>
43.8314544819
<class 'numpy.float64'>
43.8309357784
<class 'numpy.float64'>
43.830417209
<class 'numpy.float64'>
43.8298987736
<class 'numpy.float64'>
43.8293804722
<class 'numpy.float64'>
43.8288623047
<class 'numpy.float64'>
43.8283442711
<class 'numpy.float64'>
43.8278263713
<class 'numpy.float64'>
43.8273086054
<class 'numpy.float64'>
43.8267909732
<class 'numpy.float64'>
43.8262734746
<class 'numpy.float64'>
43.8257561098
<class 'numpy.float64'>
43.8252388785
<class 'numpy.float64'>
43.8247217808
<class 'numpy.float64'>
43.8242048166
<class 'numpy.float64'>
43.8236879858
<class 'numpy.float64'>
43.8231712885
<class 'numpy.float64'>
43.8226547245
<class 'numpy.float64'>
43.8221382938
<class 'numpy.float64'>
43.8216219964
<class 'numpy.float64'>
43.8211058322
<class 'numpy.float64'>
43.8205898012
<class 'numpy.float64'>
43.8200739033
<class 'numpy.float64'>
43.8195581385
<class 'numpy.float64'>
43.8190425067
<class 'numpy.float64'>
43.8185270079
<class 'numpy.float64'>
43.8180116421
<class 'numpy.float64'>
43.8174964091
<class 'numpy.float64'>
43.816981309
<class 'numpy.float64'>
43.8164663416
<class 'numpy.float64'>
43.815951507
<class 'numpy.float64'>
43.8154368051
<class 'numpy.float64'>
43.8149222358
<class 'numpy.float64'>
43.8144077992
<class 'numpy.float64'>
43.8138934951
<class 'numpy.float64'>
43.8133793235
<class 'numpy.float64'>
43.8128652843
<class 'numpy.float64'>
43.8123513776
<class 'numpy.float64'>
43.8118376033
<class 'numpy.float64'>
43.8113239612
<class 'numpy.float64'>
43.8108104514
<class 'numpy.float64'>
43.8102970739
<class 'numpy.float64'>
43.8097838285
<class 'numpy.float64'>
43.8092707153
<class 'numpy.float64'>
43.8087577341
<class 'numpy.float64'>
43.808244885
<class 'numpy.float64'>
43.8077321678
<class 'numpy.float64'>
43.8072195826
<class 'numpy.float64'>
43.8067071293
<class 'numpy.float64'>
43.8061948079
<class 'numpy.float64'>
43.8056826182
<class 'numpy.float64'>
43.8051705603
<class 'numpy.float64'>
43.8046586341
<class 'numpy.float64'>
43.8041468395
<class 'numpy.float64'>
43.8036351765
<class 'numpy.float64'>
43.8031236452
<class 'numpy.float64'>
43.8026122453
<class 'numpy.float64'>
43.8021009769
<class 'numpy.float64'>
43.8015898399
<class 'numpy.float64'>
43.8010788343
<class 'numpy.float64'>
43.80056796
<class 'numpy.float64'>
43.800057217
<class 'numpy.float64'>
43.7995466052
<class 'numpy.float64'>
43.7990361246
<class 'numpy.float64'>
43.7985257752
<class 'numpy.float64'>
43.7980155568
<class 'numpy.float64'>
43.7975054695
<class 'numpy.float64'>
43.7969955132
<class 'numpy.float64'>
43.7964856879
<class 'numpy.float64'>
43.7959759934
<class 'numpy.float64'>
43.7954664298
<class 'numpy.float64'>
43.794956997
<class 'numpy.float64'>
43.794447695
<class 'numpy.float64'>
43.7939385237
<class 'numpy.float64'>
43.793429483
<class 'numpy.float64'>
43.792920573
<class 'numpy.float64'>
43.7924117936
<class 'numpy.float64'>
43.7919031447
<class 'numpy.float64'>
43.7913946262
<class 'numpy.float64'>
43.7908862382
<class 'numpy.float64'>
43.7903779806
<class 'numpy.float64'>
43.7898698534
<class 'numpy.float64'>
43.7893618564
<class 'numpy.float64'>
43.7888539897
<class 'numpy.float64'>
43.7883462531
<class 'numpy.float64'>
43.7878386468
<class 'numpy.float64'>
43.7873311705
<class 'numpy.float64'>
43.7868238243
<class 'numpy.float64'>
43.7863166081
<class 'numpy.float64'>
43.7858095219
<class 'numpy.float64'>
43.7853025656
<class 'numpy.float64'>
43.7847957392
<class 'numpy.float64'>
43.7842890426
<class 'numpy.float64'>
43.7837824758
<class 'numpy.float64'>
43.7832760387
<class 'numpy.float64'>
43.7827697314
<class 'numpy.float64'>
43.7822635536
<class 'numpy.float64'>
43.7817575055
<class 'numpy.float64'>
43.7812515869
<class 'numpy.float64'>
43.7807457978
<class 'numpy.float64'>
43.7802401381
<class 'numpy.float64'>
43.7797346079
<class 'numpy.float64'>
43.779229207
<class 'numpy.float64'>
43.7787239354
<class 'numpy.float64'>
43.7782187932
<class 'numpy.float64'>
43.7777137801
<class 'numpy.float64'>
43.7772088962
<class 'numpy.float64'>
43.7767041414
<class 'numpy.float64'>
43.7761995157
<class 'numpy.float64'>
43.7756950191
<class 'numpy.float64'>
43.7751906514
<class 'numpy.float64'>
43.7746864127
<class 'numpy.float64'>
43.7741823029
<class 'numpy.float64'>
43.773678322
<class 'numpy.float64'>
43.7731744698
<class 'numpy.float64'>
43.7726707464
<class 'numpy.float64'>
43.7721671517
<class 'numpy.float64'>
43.7716636857
<class 'numpy.float64'>
43.7711603483
<class 'numpy.float64'>
43.7706571394
<class 'numpy.float64'>
43.7701540591
<class 'numpy.float64'>
43.7696511073
<class 'numpy.float64'>
43.7691482839
<class 'numpy.float64'>
43.7686455889
<class 'numpy.float64'>
43.7681430222
<class 'numpy.float64'>
43.7676405838
<class 'numpy.float64'>
43.7671382737
<class 'numpy.float64'>
43.7666360918
<class 'numpy.float64'>
43.766134038
<class 'numpy.float64'>
43.7656321123
<class 'numpy.float64'>
43.7651303147
<class 'numpy.float64'>
43.7646286451
<class 'numpy.float64'>
43.7641271035
<class 'numpy.float64'>
43.7636256898
<class 'numpy.float64'>
43.763124404
<class 'numpy.float64'>
43.762623246
<class 'numpy.float64'>
43.7621222158
<class 'numpy.float64'>
43.7616213134
<class 'numpy.float64'>
43.7611205386
<class 'numpy.float64'>
43.7606198915
<class 'numpy.float64'>
43.7601193719
<class 'numpy.float64'>
43.75961898
<class 'numpy.float64'>
43.7591187155
<class 'numpy.float64'>
43.7586185785
<class 'numpy.float64'>
43.7581185689
<class 'numpy.float64'>
43.7576186867
<class 'numpy.float64'>
43.7571189318
<class 'numpy.float64'>
43.7566193041
<class 'numpy.float64'>
43.7561198038
<class 'numpy.float64'>
43.7556204305
<class 'numpy.float64'>
43.7551211845
<class 'numpy.float64'>
43.7546220655
<class 'numpy.float64'>
43.7541230736
<class 'numpy.float64'>
43.7536242086
<class 'numpy.float64'>
43.7531254707
<class 'numpy.float64'>
43.7526268596
<class 'numpy.float64'>
43.7521283754
<class 'numpy.float64'>
43.751630018
<class 'numpy.float64'>
43.7511317873
<class 'numpy.float64'>
43.7506336834
<class 'numpy.float64'>
43.7501357062
<class 'numpy.float64'>
43.7496378556
<class 'numpy.float64'>
43.7491401316
<class 'numpy.float64'>
43.7486425341
<class 'numpy.float64'>
43.7481450632
<class 'numpy.float64'>
43.7476477186
<class 'numpy.float64'>
43.7471505005
<class 'numpy.float64'>
43.7466534087
<class 'numpy.float64'>
43.7461564433
<class 'numpy.float64'>
43.745659604
<class 'numpy.float64'>
43.7451628911
<class 'numpy.float64'>
43.7446663042
<class 'numpy.float64'>
43.7441698436
<class 'numpy.float64'>
43.7436735089
<class 'numpy.float64'>
43.7431773004
<class 'numpy.float64'>
43.7426812178
<class 'numpy.float64'>
43.7421852611
<class 'numpy.float64'>
43.7416894304
<class 'numpy.float64'>
43.7411937254
<class 'numpy.float64'>
43.7406981463
<class 'numpy.float64'>
43.740202693
<class 'numpy.float64'>
43.7397073654
<class 'numpy.float64'>
43.7392121634
<class 'numpy.float64'>
43.738717087
<class 'numpy.float64'>
43.7382221362
<class 'numpy.float64'>
43.737727311
<class 'numpy.float64'>
43.7372326112
<class 'numpy.float64'>
43.7367380369
<class 'numpy.float64'>
43.736243588
<class 'numpy.float64'>
43.7357492644
<class 'numpy.float64'>
43.7352550661
<class 'numpy.float64'>
43.734760993
<class 'numpy.float64'>
43.7342670452
<class 'numpy.float64'>
43.7337732225
<class 'numpy.float64'>
43.7332795249
<class 'numpy.float64'>
43.7327859524
<class 'numpy.float64'>
43.7322925049
<class 'numpy.float64'>
43.7317991824
<class 'numpy.float64'>
43.7313059849
<class 'numpy.float64'>
43.7308129122
<class 'numpy.float64'>
43.7303199644
<class 'numpy.float64'>
43.7298271413
<class 'numpy.float64'>
43.729334443
<class 'numpy.float64'>
43.7288418694
<class 'numpy.float64'>
43.7283494205
<class 'numpy.float64'>
43.7278570962
<class 'numpy.float64'>
43.7273648964
<class 'numpy.float64'>
43.7268728212
<class 'numpy.float64'>
43.7263808704
<class 'numpy.float64'>
43.7258890441
<class 'numpy.float64'>
43.7253973421
<class 'numpy.float64'>
43.7249057645
<class 'numpy.float64'>
43.7244143112
<class 'numpy.float64'>
43.7239229821
<class 'numpy.float64'>
43.7234317772
<class 'numpy.float64'>
43.7229406965
<class 'numpy.float64'>
43.7224497398
<class 'numpy.float64'>
43.7219589073
<class 'numpy.float64'>
43.7214681987
<class 'numpy.float64'>
43.7209776142
<class 'numpy.float64'>
43.7204871535
<class 'numpy.float64'>
43.7199968167
<class 'numpy.float64'>
43.7195066038
<class 'numpy.float64'>
43.7190165147
<class 'numpy.float64'>
43.7185265493
<class 'numpy.float64'>
43.7180367076
<class 'numpy.float64'>
43.7175469895
<class 'numpy.float64'>
43.7170573951
<class 'numpy.float64'>
43.7165679242
<class 'numpy.float64'>
43.7160785768
<class 'numpy.float64'>
43.7155893529
<class 'numpy.float64'>
43.7151002524
<class 'numpy.float64'>
43.7146112753
<class 'numpy.float64'>
43.7141224215
<class 'numpy.float64'>
43.713633691
<class 'numpy.float64'>
43.7131450838
<class 'numpy.float64'>
43.7126565997
<class 'numpy.float64'>
43.7121682388
<class 'numpy.float64'>
43.711680001
<class 'numpy.float64'>
43.7111918863
<class 'numpy.float64'>
43.7107038945
<class 'numpy.float64'>
43.7102160258
<class 'numpy.float64'>
43.7097282799
<class 'numpy.float64'>
43.709240657
<class 'numpy.float64'>
43.7087531568
<class 'numpy.float64'>
43.7082657795
<class 'numpy.float64'>
43.7077785249
<class 'numpy.float64'>
43.7072913929
<class 'numpy.float64'>
43.7068043837
<class 'numpy.float64'>
43.706317497
<class 'numpy.float64'>
43.7058307329
<class 'numpy.float64'>
43.7053440913
<class 'numpy.float64'>
43.7048575722
<class 'numpy.float64'>
43.7043711754
<class 'numpy.float64'>
43.7038849011
<class 'numpy.float64'>
43.7033987491
<class 'numpy.float64'>
43.7029127194
<class 'numpy.float64'>
43.7024268119
<class 'numpy.float64'>
43.7019410266
<class 'numpy.float64'>
43.7014553634
<class 'numpy.float64'>
43.7009698224
<class 'numpy.float64'>
43.7004844034
<class 'numpy.float64'>
43.6999991064
<class 'numpy.float64'>
43.6995139314
<class 'numpy.float64'>
43.6990288783
<class 'numpy.float64'>
43.6985439471
<class 'numpy.float64'>
43.6980591377
<class 'numpy.float64'>
43.6975744501
<class 'numpy.float64'>
43.6970898842
<class 'numpy.float64'>
43.6966054401
<class 'numpy.float64'>
43.6961211175
<class 'numpy.float64'>
43.6956369166
<class 'numpy.float64'>
43.6951528372
<class 'numpy.float64'>
43.6946688794
<class 'numpy.float64'>
43.694185043
<class 'numpy.float64'>
43.693701328
<class 'numpy.float64'>
43.6932177344
<class 'numpy.float64'>
43.6927342622
<class 'numpy.float64'>
43.6922509112
<class 'numpy.float64'>
43.6917676814
<class 'numpy.float64'>
43.6912845729
<class 'numpy.float64'>
43.6908015855
<class 'numpy.float64'>
43.6903187192
<class 'numpy.float64'>
43.6898359739
<class 'numpy.float64'>
43.6893533497
<class 'numpy.float64'>
43.6888708464
<class 'numpy.float64'>
43.6883884641
<class 'numpy.float64'>
43.6879062026
<class 'numpy.float64'>
43.687424062
<class 'numpy.float64'>
43.6869420421
<class 'numpy.float64'>
43.686460143
<class 'numpy.float64'>
43.6859783646
<class 'numpy.float64'>
43.6854967068
<class 'numpy.float64'>
43.6850151697
<class 'numpy.float64'>
43.6845337531
<class 'numpy.float64'>
43.684052457
<class 'numpy.float64'>
43.6835712814
<class 'numpy.float64'>
43.6830902261
<class 'numpy.float64'>
43.6826092913
<class 'numpy.float64'>
43.6821284768
<class 'numpy.float64'>
43.6816477826
<class 'numpy.float64'>
43.6811672087
<class 'numpy.float64'>
43.6806867549
<class 'numpy.float64'>
43.6802064213
<class 'numpy.float64'>
43.6797262077
<class 'numpy.float64'>
43.6792461143
<class 'numpy.float64'>
43.6787661409
<class 'numpy.float64'>
43.6782862874
<class 'numpy.float64'>
43.6778065538
<class 'numpy.float64'>
43.6773269402
<class 'numpy.float64'>
43.6768474463
<class 'numpy.float64'>
43.6763680723
<class 'numpy.float64'>
43.675888818
<class 'numpy.float64'>
43.6754096834
<class 'numpy.float64'>
43.6749306685
<class 'numpy.float64'>
43.6744517731
<class 'numpy.float64'>
43.6739729973
<class 'numpy.float64'>
43.6734943411
<class 'numpy.float64'>
43.6730158043
<class 'numpy.float64'>
43.6725373869
<class 'numpy.float64'>
43.672059089
<class 'numpy.float64'>
43.6715809103
<class 'numpy.float64'>
43.671102851
<class 'numpy.float64'>
43.6706249109
<class 'numpy.float64'>
43.67014709
<class 'numpy.float64'>
43.6696693882
<class 'numpy.float64'>
43.6691918056
<class 'numpy.float64'>
43.668714342
<class 'numpy.float64'>
43.6682369975
<class 'numpy.float64'>
43.6677597719
<class 'numpy.float64'>
43.6672826653
<class 'numpy.float64'>
43.6668056775
<class 'numpy.float64'>
43.6663288086
<class 'numpy.float64'>
43.6658520585
<class 'numpy.float64'>
43.6653754271
<class 'numpy.float64'>
43.6648989145
<class 'numpy.float64'>
43.6644225205
<class 'numpy.float64'>
43.6639462451
<class 'numpy.float64'>
43.6634700883
<class 'numpy.float64'>
43.66299405
<class 'numpy.float64'>
43.6625181301
<class 'numpy.float64'>
43.6620423288
<class 'numpy.float64'>
43.6615666458
<class 'numpy.float64'>
43.6610910811
<class 'numpy.float64'>
43.6606156348
<class 'numpy.float64'>
43.6601403067
<class 'numpy.float64'>
43.6596650968
<class 'numpy.float64'>
43.6591900051
<class 'numpy.float64'>
43.6587150315
<class 'numpy.float64'>
43.6582401759
<class 'numpy.float64'>
43.6577654384
<class 'numpy.float64'>
43.6572908189
<class 'numpy.float64'>
43.6568163174
<class 'numpy.float64'>
43.6563419337
<class 'numpy.float64'>
43.6558676678
<class 'numpy.float64'>
43.6553935198
<class 'numpy.float64'>
43.6549194895
<class 'numpy.float64'>
43.654445577
<class 'numpy.float64'>
43.6539717821
<class 'numpy.float64'>
43.6534981048
<class 'numpy.float64'>
43.6530245452
<class 'numpy.float64'>
43.652551103
<class 'numpy.float64'>
43.6520777783
<class 'numpy.float64'>
43.6516045711
<class 'numpy.float64'>
43.6511314813
<class 'numpy.float64'>
43.6506585088
<class 'numpy.float64'>
43.6501856537
<class 'numpy.float64'>
43.6497129158
<class 'numpy.float64'>
43.6492402951
<class 'numpy.float64'>
43.6487677916
<class 'numpy.float64'>
43.6482954053
<class 'numpy.float64'>
43.647823136
<class 'numpy.float64'>
43.6473509838
<class 'numpy.float64'>
43.6468789485
<class 'numpy.float64'>
43.6464070302
<class 'numpy.float64'>
43.6459352288
<class 'numpy.float64'>
43.6454635443
<class 'numpy.float64'>
43.6449919766
<class 'numpy.float64'>
43.6445205256
<class 'numpy.float64'>
43.6440491914
<class 'numpy.float64'>
43.6435779739
<class 'numpy.float64'>
43.643106873
<class 'numpy.float64'>
43.6426358887
<class 'numpy.float64'>
43.6421650209
<class 'numpy.float64'>
43.6416942697
<class 'numpy.float64'>
43.6412236349
<class 'numpy.float64'>
43.6407531165
<class 'numpy.float64'>
43.6402827144
<class 'numpy.float64'>
43.6398124287
<class 'numpy.float64'>
43.6393422593
<class 'numpy.float64'>
43.6388722061
<class 'numpy.float64'>
43.6384022691
<class 'numpy.float64'>
43.6379324483
<class 'numpy.float64'>
43.6374627435
<class 'numpy.float64'>
43.6369931548
<class 'numpy.float64'>
43.6365236821
<class 'numpy.float64'>
43.6360543254
<class 'numpy.float64'>
43.6355850846
<class 'numpy.float64'>
43.6351159596
<class 'numpy.float64'>
43.6346469505
<class 'numpy.float64'>
43.6341780572
<class 'numpy.float64'>
43.6337092796
<class 'numpy.float64'>
43.6332406177
<class 'numpy.float64'>
43.6327720715
<class 'numpy.float64'>
43.6323036409
<class 'numpy.float64'>
43.6318353258
<class 'numpy.float64'>
43.6313671262
<class 'numpy.float64'>
43.6308990422
<class 'numpy.float64'>
43.6304310735
<class 'numpy.float64'>
43.6299632203
<class 'numpy.float64'>
43.6294954823
<class 'numpy.float64'>
43.6290278597
<class 'numpy.float64'>
43.6285603523
<class 'numpy.float64'>
43.6280929601
<class 'numpy.float64'>
43.6276256831
<class 'numpy.float64'>
43.6271585212
<class 'numpy.float64'>
43.6266914744
<class 'numpy.float64'>
43.6262245426
<class 'numpy.float64'>
43.6257577257
<class 'numpy.float64'>
43.6252910239
<class 'numpy.float64'>
43.6248244369
<class 'numpy.float64'>
43.6243579647
<class 'numpy.float64'>
43.6238916074
<class 'numpy.float64'>
43.6234253648
<class 'numpy.float64'>
43.6229592369
<class 'numpy.float64'>
43.6224932237
<class 'numpy.float64'>
43.6220273251
<class 'numpy.float64'>
43.6215615411
<class 'numpy.float64'>
43.6210958717
<class 'numpy.float64'>
43.6206303167
<class 'numpy.float64'>
43.6201648762
<class 'numpy.float64'>
43.6196995501
<class 'numpy.float64'>
43.6192343383
<class 'numpy.float64'>
43.6187692409
<class 'numpy.float64'>
43.6183042577
<class 'numpy.float64'>
43.6178393887
<class 'numpy.float64'>
43.617374634
<class 'numpy.float64'>
43.6169099933
<class 'numpy.float64'>
43.6164454668
<class 'numpy.float64'>
43.6159810543
<class 'numpy.float64'>
43.6155167558
<class 'numpy.float64'>
43.6150525712
<class 'numpy.float64'>
43.6145885006
<class 'numpy.float64'>
43.6141245439
<class 'numpy.float64'>
43.6136607009
<class 'numpy.float64'>
43.6131969718
<class 'numpy.float64'>
43.6127333564
<class 'numpy.float64'>
43.6122698546
<class 'numpy.float64'>
43.6118064666
<class 'numpy.float64'>
43.6113431921
<class 'numpy.float64'>
43.6108800312
<class 'numpy.float64'>
43.6104169838
<class 'numpy.float64'>
43.6099540498
<class 'numpy.float64'>
43.6094912293
<class 'numpy.float64'>
43.6090285222
<class 'numpy.float64'>
43.6085659284
<class 'numpy.float64'>
43.6081034479
<class 'numpy.float64'>
43.6076410807
<class 'numpy.float64'>
43.6071788266
<class 'numpy.float64'>
43.6067166858
<class 'numpy.float64'>
43.606254658
<class 'numpy.float64'>
43.6057927433
<class 'numpy.float64'>
43.6053309416
<class 'numpy.float64'>
43.6048692529
<class 'numpy.float64'>
43.6044076772
<class 'numpy.float64'>
43.6039462143
<class 'numpy.float64'>
43.6034848643
<class 'numpy.float64'>
43.6030236271
<class 'numpy.float64'>
43.6025625027
<class 'numpy.float64'>
43.602101491
<class 'numpy.float64'>
43.6016405919
<class 'numpy.float64'>
43.6011798055
<class 'numpy.float64'>
43.6007191317
<class 'numpy.float64'>
43.6002585704
<class 'numpy.float64'>
43.5997981216
<class 'numpy.float64'>
43.5993377852
<class 'numpy.float64'>
43.5988775613
<class 'numpy.float64'>
43.5984174497
<class 'numpy.float64'>
43.5979574505
<class 'numpy.float64'>
43.5974975635
<class 'numpy.float64'>
43.5970377888
<class 'numpy.float64'>
43.5965781262
<class 'numpy.float64'>
43.5961185758
<class 'numpy.float64'>
43.5956591375
<class 'numpy.float64'>
43.5951998112
<class 'numpy.float64'>
43.594740597
<class 'numpy.float64'>
43.5942814947
<class 'numpy.float64'>
43.5938225044
<class 'numpy.float64'>
43.5933636259
<class 'numpy.float64'>
43.5929048592
<class 'numpy.float64'>
43.5924462044
<class 'numpy.float64'>
43.5919876613
<class 'numpy.float64'>
43.5915292299
<class 'numpy.float64'>
43.5910709102
<class 'numpy.float64'>
43.590612702
<class 'numpy.float64'>
43.5901546055
<class 'numpy.float64'>
43.5896966205
<class 'numpy.float64'>
43.5892387469
<class 'numpy.float64'>
43.5887809848
<class 'numpy.float64'>
43.5883233341
<class 'numpy.float64'>
43.5878657948
<class 'numpy.float64'>
43.5874083667
<class 'numpy.float64'>
43.5869510499
<class 'numpy.float64'>
43.5864938444
<class 'numpy.float64'>
43.58603675
<class 'numpy.float64'>
43.5855797667
<class 'numpy.float64'>
43.5851228946
<class 'numpy.float64'>
43.5846661335
<class 'numpy.float64'>
43.5842094833
<class 'numpy.float64'>
43.5837529442
<class 'numpy.float64'>
43.5832965159
<class 'numpy.float64'>
43.5828401985
<class 'numpy.float64'>
43.582383992
<class 'numpy.float64'>
43.5819278962
<class 'numpy.float64'>
43.5814719112
<class 'numpy.float64'>
43.5810160368
<class 'numpy.float64'>
43.5805602732
<class 'numpy.float64'>
43.5801046201
<class 'numpy.float64'>
43.5796490776
<class 'numpy.float64'>
43.5791936456
<class 'numpy.float64'>
43.578738324
<class 'numpy.float64'>
43.5782831129
<class 'numpy.float64'>
43.5778280122
<class 'numpy.float64'>
43.5773730219
<class 'numpy.float64'>
43.5769181418
<class 'numpy.float64'>
43.576463372
<class 'numpy.float64'>
43.5760087124
<class 'numpy.float64'>
43.575554163
<class 'numpy.float64'>
43.5750997237
<class 'numpy.float64'>
43.5746453945
<class 'numpy.float64'>
43.5741911753
<class 'numpy.float64'>
43.5737370662
<class 'numpy.float64'>
43.5732830669
<class 'numpy.float64'>
43.5728291776
<class 'numpy.float64'>
43.5723753982
<class 'numpy.float64'>
43.5719217285
<class 'numpy.float64'>
43.5714681687
<class 'numpy.float64'>
43.5710147186
<class 'numpy.float64'>
43.5705613782
<class 'numpy.float64'>
43.5701081474
<class 'numpy.float64'>
43.5696550262
<class 'numpy.float64'>
43.5692020146
<class 'numpy.float64'>
43.5687491125
<class 'numpy.float64'>
43.5682963199
<class 'numpy.float64'>
43.5678436367
<class 'numpy.float64'>
43.5673910629
<class 'numpy.float64'>
43.5669385984
<class 'numpy.float64'>
43.5664862432
<class 'numpy.float64'>
43.5660339973
<class 'numpy.float64'>
43.5655818606
<class 'numpy.float64'>
43.5651298331
<class 'numpy.float64'>
43.5646779147
<class 'numpy.float64'>
43.5642261054
<class 'numpy.float64'>
43.5637744051
<class 'numpy.float64'>
43.5633228138
<class 'numpy.float64'>
43.5628713314
<class 'numpy.float64'>
43.562419958
<class 'numpy.float64'>
43.5619686935
<class 'numpy.float64'>
43.5615175377
<class 'numpy.float64'>
43.5610664908
<class 'numpy.float64'>
43.5606155525
<class 'numpy.float64'>
43.560164723
<class 'numpy.float64'>
43.5597140021
<class 'numpy.float64'>
43.5592633899
<class 'numpy.float64'>
43.5588128862
<class 'numpy.float64'>
43.558362491
<class 'numpy.float64'>
43.5579122043
<class 'numpy.float64'>
43.557462026
<class 'numpy.float64'>
43.5570119561
<class 'numpy.float64'>
43.5565619945
<class 'numpy.float64'>
43.5561121413
<class 'numpy.float64'>
43.5556623963
<class 'numpy.float64'>
43.5552127596
<class 'numpy.float64'>
43.554763231
<class 'numpy.float64'>
43.5543138105
<class 'numpy.float64'>
43.5538644981
<class 'numpy.float64'>
43.5534152938
<class 'numpy.float64'>
43.5529661975
<class 'numpy.float64'>
43.5525172091
<class 'numpy.float64'>
43.5520683287
<class 'numpy.float64'>
43.5516195561
<class 'numpy.float64'>
43.5511708914
<class 'numpy.float64'>
43.5507223344
<class 'numpy.float64'>
43.5502738852
<class 'numpy.float64'>
43.5498255437
<class 'numpy.float64'>
43.5493773098
<class 'numpy.float64'>
43.5489291836
<class 'numpy.float64'>
43.5484811649
<class 'numpy.float64'>
43.5480332538
<class 'numpy.float64'>
43.5475854502
<class 'numpy.float64'>
43.547137754
<class 'numpy.float64'>
43.5466901651
<class 'numpy.float64'>
43.5462426837
<class 'numpy.float64'>
43.5457953096
<class 'numpy.float64'>
43.5453480427
<class 'numpy.float64'>
43.5449008831
<class 'numpy.float64'>
43.5444538306
<class 'numpy.float64'>
43.5440068853
<class 'numpy.float64'>
43.5435600471
<class 'numpy.float64'>
43.543113316
<class 'numpy.float64'>
43.5426666918
<class 'numpy.float64'>
43.5422201747
<class 'numpy.float64'>
43.5417737644
<class 'numpy.float64'>
43.5413274611
<class 'numpy.float64'>
43.5408812646
<class 'numpy.float64'>
43.5404351749
<class 'numpy.float64'>
43.539989192
<class 'numpy.float64'>
43.5395433157
<class 'numpy.float64'>
43.5390975462
<class 'numpy.float64'>
43.5386518833
<class 'numpy.float64'>
43.5382063269
<class 'numpy.float64'>
43.5377608771
<class 'numpy.float64'>
43.5373155339
<class 'numpy.float64'>
43.536870297
<class 'numpy.float64'>
43.5364251666
<class 'numpy.float64'>
43.5359801426
<class 'numpy.float64'>
43.5355352249
<class 'numpy.float64'>
43.5350904135
<class 'numpy.float64'>
43.5346457083
<class 'numpy.float64'>
43.5342011094
<class 'numpy.float64'>
43.5337566166
<class 'numpy.float64'>
43.5333122299
<class 'numpy.float64'>
43.5328679493
<class 'numpy.float64'>
43.5324237747
<class 'numpy.float64'>
43.5319797061
<class 'numpy.float64'>
43.5315357435
<class 'numpy.float64'>
43.5310918868
<class 'numpy.float64'>
43.5306481359
<class 'numpy.float64'>
43.5302044909
<class 'numpy.float64'>
43.5297609516
<class 'numpy.float64'>
43.5293175181
<class 'numpy.float64'>
43.5288741903
<class 'numpy.float64'>
43.5284309681
<class 'numpy.float64'>
43.5279878516
<class 'numpy.float64'>
43.5275448406
<class 'numpy.float64'>
43.5271019351
<class 'numpy.float64'>
43.5266591352
<class 'numpy.float64'>
43.5262164406
<class 'numpy.float64'>
43.5257738515
<class 'numpy.float64'>
43.5253313677
<class 'numpy.float64'>
43.5248889892
<class 'numpy.float64'>
43.5244467161
<class 'numpy.float64'>
43.5240045481
<class 'numpy.float64'>
43.5235624853
<class 'numpy.float64'>
43.5231205277
<class 'numpy.float64'>
43.5226786752
<class 'numpy.float64'>
43.5222369277
<class 'numpy.float64'>
43.5217952852
<class 'numpy.float64'>
43.5213537478
<class 'numpy.float64'>
43.5209123152
<class 'numpy.float64'>
43.5204709876
<class 'numpy.float64'>
43.5200297648
<class 'numpy.float64'>
43.5195886468
<class 'numpy.float64'>
43.5191476336
<class 'numpy.float64'>
43.5187067251
<class 'numpy.float64'>
43.5182659213
<class 'numpy.float64'>
43.5178252221
<class 'numpy.float64'>
43.5173846276
<class 'numpy.float64'>
43.5169441376
<class 'numpy.float64'>
43.5165037521
<class 'numpy.float64'>
43.516063471
<class 'numpy.float64'>
43.5156232944
<class 'numpy.float64'>
43.5151832222
<class 'numpy.float64'>
43.5147432544
<class 'numpy.float64'>
43.5143033908
<class 'numpy.float64'>
43.5138636315
<class 'numpy.float64'>
43.5134239765
<class 'numpy.float64'>
43.5129844256
<class 'numpy.float64'>
43.5125449788
<class 'numpy.float64'>
43.5121056362
<class 'numpy.float64'>
43.5116663976
<class 'numpy.float64'>
43.511227263
<class 'numpy.float64'>
43.5107882324
<class 'numpy.float64'>
43.5103493057
<class 'numpy.float64'>
43.5099104829
<class 'numpy.float64'>
43.5094717639
<class 'numpy.float64'>
43.5090331488
<class 'numpy.float64'>
43.5085946374
<class 'numpy.float64'>
43.5081562297
<class 'numpy.float64'>
43.5077179257
<class 'numpy.float64'>
43.5072797253
<class 'numpy.float64'>
43.5068416286
<class 'numpy.float64'>
43.5064036354
<class 'numpy.float64'>
43.5059657457
<class 'numpy.float64'>
43.5055279594
<class 'numpy.float64'>
43.5050902766
<class 'numpy.float64'>
43.5046526972
<class 'numpy.float64'>
43.5042152212
<class 'numpy.float64'>
43.5037778484
<class 'numpy.float64'>
43.5033405789
<class 'numpy.float64'>
43.5029034126
<class 'numpy.float64'>
43.5024663495
<class 'numpy.float64'>
43.5020293896
<class 'numpy.float64'>
43.5015925327
<class 'numpy.float64'>
43.5011557789
<class 'numpy.float64'>
43.5007191281
<class 'numpy.float64'>
43.5002825803
<class 'numpy.float64'>
43.4998461354
<class 'numpy.float64'>
43.4994097934
<class 'numpy.float64'>
43.4989735542
<class 'numpy.float64'>
43.4985374179
<class 'numpy.float64'>
43.4981013843
<class 'numpy.float64'>
43.4976654534
<class 'numpy.float64'>
43.4972296253
<class 'numpy.float64'>
43.4967938997
<class 'numpy.float64'>
43.4963582768
<class 'numpy.float64'>
43.4959227564
<class 'numpy.float64'>
43.4954873386
<class 'numpy.float64'>
43.4950520232
<class 'numpy.float64'>
43.4946168103
<class 'numpy.float64'>
43.4941816997
<class 'numpy.float64'>
43.4937466915
<class 'numpy.float64'>
43.4933117856
<class 'numpy.float64'>
43.492876982
<class 'numpy.float64'>
43.4924422807
<class 'numpy.float64'>
43.4920076815
<class 'numpy.float64'>
43.4915731844
<class 'numpy.float64'>
43.4911387895
<class 'numpy.float64'>
43.4907044966
<class 'numpy.float64'>
43.4902703058
<class 'numpy.float64'>
43.4898362169
<class 'numpy.float64'>
43.48940223
<class 'numpy.float64'>
43.4889683449
<class 'numpy.float64'>
43.4885345618
<class 'numpy.float64'>
43.4881008804
<class 'numpy.float64'>
43.4876673009
<class 'numpy.float64'>
43.4872338231
<class 'numpy.float64'>
43.4868004469
<class 'numpy.float64'>
43.4863671725
<class 'numpy.float64'>
43.4859339996
<class 'numpy.float64'>
43.4855009283
<class 'numpy.float64'>
43.4850679586
<class 'numpy.float64'>
43.4846350903
<class 'numpy.float64'>
43.4842023235
<class 'numpy.float64'>
43.4837696581
<class 'numpy.float64'>
43.4833370941
<class 'numpy.float64'>
43.4829046314
<class 'numpy.float64'>
43.48247227
<class 'numpy.float64'>
43.4820400098
<class 'numpy.float64'>
43.4816078509
<class 'numpy.float64'>
43.4811757931
<class 'numpy.float64'>
43.4807438364
<class 'numpy.float64'>
43.4803119808
<class 'numpy.float64'>
43.4798802263
<class 'numpy.float64'>
43.4794485728
<class 'numpy.float64'>
43.4790170202
<class 'numpy.float64'>
43.4785855686
<class 'numpy.float64'>
43.4781542178
<class 'numpy.float64'>
43.4777229679
<class 'numpy.float64'>
43.4772918188
<class 'numpy.float64'>
43.4768607704
<class 'numpy.float64'>
43.4764298228
<class 'numpy.float64'>
43.4759989758
<class 'numpy.float64'>
43.4755682295
<class 'numpy.float64'>
43.4751375838
<class 'numpy.float64'>
43.4747070386
<class 'numpy.float64'>
43.474276594
<class 'numpy.float64'>
43.4738462498
<class 'numpy.float64'>
43.4734160061
<class 'numpy.float64'>
43.4729858628
<class 'numpy.float64'>
43.4725558198
<class 'numpy.float64'>
43.4721258771
<class 'numpy.float64'>
43.4716960347
<class 'numpy.float64'>
43.4712662926
<class 'numpy.float64'>
43.4708366506
<class 'numpy.float64'>
43.4704071088
<class 'numpy.float64'>
43.4699776671
<class 'numpy.float64'>
43.4695483255
<class 'numpy.float64'>
43.4691190839
<class 'numpy.float64'>
43.4686899423
<class 'numpy.float64'>
43.4682609007
<class 'numpy.float64'>
43.467831959
<class 'numpy.float64'>
43.4674031171
<class 'numpy.float64'>
43.4669743751
<class 'numpy.float64'>
43.4665457329
<class 'numpy.float64'>
43.4661171904
<class 'numpy.float64'>
43.4656887477
<class 'numpy.float64'>
43.4652604046
<class 'numpy.float64'>
43.4648321612
<class 'numpy.float64'>
43.4644040173
<class 'numpy.float64'>
43.463975973
<class 'numpy.float64'>
43.4635480282
<class 'numpy.float64'>
43.4631201829
<class 'numpy.float64'>
43.462692437
<class 'numpy.float64'>
43.4622647905
<class 'numpy.float64'>
43.4618372434
<class 'numpy.float64'>
43.4614097955
<class 'numpy.float64'>
43.460982447
<class 'numpy.float64'>
43.4605551976
<class 'numpy.float64'>
43.4601280475
<class 'numpy.float64'>
43.4597009965
<class 'numpy.float64'>
43.4592740447
<class 'numpy.float64'>
43.4588471919
<class 'numpy.float64'>
43.4584204381
<class 'numpy.float64'>
43.4579937834
<class 'numpy.float64'>
43.4575672276
<class 'numpy.float64'>
43.4571407707
<class 'numpy.float64'>
43.4567144127
<class 'numpy.float64'>
43.4562881535
<class 'numpy.float64'>
43.4558619931
<class 'numpy.float64'>
43.4554359315
<class 'numpy.float64'>
43.4550099685
<class 'numpy.float64'>
43.4545841043
<class 'numpy.float64'>
43.4541583387
<class 'numpy.float64'>
43.4537326717
<class 'numpy.float64'>
43.4533071033
<class 'numpy.float64'>
43.4528816333
<class 'numpy.float64'>
43.4524562619
<class 'numpy.float64'>
43.4520309889
<class 'numpy.float64'>
43.4516058143
<class 'numpy.float64'>
43.451180738
<class 'numpy.float64'>
43.4507557601
<class 'numpy.float64'>
43.4503308805
<class 'numpy.float64'>
43.4499060991
<class 'numpy.float64'>
43.4494814159
<class 'numpy.float64'>
43.4490568308
<class 'numpy.float64'>
43.4486323439
<class 'numpy.float64'>
43.4482079551
<class 'numpy.float64'>
43.4477836643
<class 'numpy.float64'>
43.4473594715
<class 'numpy.float64'>
43.4469353767
<class 'numpy.float64'>
43.4465113798
<class 'numpy.float64'>
43.4460874808
<class 'numpy.float64'>
43.4456636797
<class 'numpy.float64'>
43.4452399763
<class 'numpy.float64'>
43.4448163708
<class 'numpy.float64'>
43.4443928629
<class 'numpy.float64'>
43.4439694528
<class 'numpy.float64'>
43.4435461403
<class 'numpy.float64'>
43.4431229254
<class 'numpy.float64'>
43.4426998081
<class 'numpy.float64'>
43.4422767884
<class 'numpy.float64'>
43.4418538661
<class 'numpy.float64'>
43.4414310413
<class 'numpy.float64'>
43.4410083139
<class 'numpy.float64'>
43.4405856839
<class 'numpy.float64'>
43.4401631512
<class 'numpy.float64'>
43.4397407158
<class 'numpy.float64'>
43.4393183777
<class 'numpy.float64'>
43.4388961368
<class 'numpy.float64'>
43.4384739931
<class 'numpy.float64'>
43.4380519466
<class 'numpy.float64'>
43.4376299971
<class 'numpy.float64'>
43.4372081447
<class 'numpy.float64'>
43.4367863894
<class 'numpy.float64'>
43.436364731
<class 'numpy.float64'>
43.4359431696
<class 'numpy.float64'>
43.4355217051
<class 'numpy.float64'>
43.4351003375
<class 'numpy.float64'>
43.4346790667
<class 'numpy.float64'>
43.4342578927
<class 'numpy.float64'>
43.4338368154
<class 'numpy.float64'>
43.4334158349
<class 'numpy.float64'>
43.4329949511
<class 'numpy.float64'>
43.4325741639
<class 'numpy.float64'>
43.4321534733
<class 'numpy.float64'>
43.4317328792
<class 'numpy.float64'>
43.4313123817
<class 'numpy.float64'>
43.4308919807
<class 'numpy.float64'>
43.4304716761
<class 'numpy.float64'>
43.430051468
<class 'numpy.float64'>
43.4296313562
<class 'numpy.float64'>
43.4292113407
<class 'numpy.float64'>
43.4287914216
<class 'numpy.float64'>
43.4283715987
<class 'numpy.float64'>
43.427951872
<class 'numpy.float64'>
43.4275322414
<class 'numpy.float64'>
43.4271127071
<class 'numpy.float64'>
43.4266932688
<class 'numpy.float64'>
43.4262739266
<class 'numpy.float64'>
43.4258546804
<class 'numpy.float64'>
43.4254355302
<class 'numpy.float64'>
43.425016476
<class 'numpy.float64'>
43.4245975176
<class 'numpy.float64'>
43.4241786552
<class 'numpy.float64'>
43.4237598885
<class 'numpy.float64'>
43.4233412177
<class 'numpy.float64'>
43.4229226426
<class 'numpy.float64'>
43.4225041632
<class 'numpy.float64'>
43.4220857795
<class 'numpy.float64'>
43.4216674915
<class 'numpy.float64'>
43.4212492991
<class 'numpy.float64'>
43.4208312022
<class 'numpy.float64'>
43.4204132008
<class 'numpy.float64'>
43.419995295
<class 'numpy.float64'>
43.4195774846
<class 'numpy.float64'>
43.4191597696
<class 'numpy.float64'>
43.41874215
<class 'numpy.float64'>
43.4183246257
<class 'numpy.float64'>
43.4179071967
<class 'numpy.float64'>
43.417489863
<class 'numpy.float64'>
43.4170726245
<class 'numpy.float64'>
43.4166554812
<class 'numpy.float64'>
43.4162384331
<class 'numpy.float64'>
43.41582148
<class 'numpy.float64'>
43.415404622
<class 'numpy.float64'>
43.4149878591
<class 'numpy.float64'>
43.4145711911
<class 'numpy.float64'>
43.4141546181
<class 'numpy.float64'>
43.4137381401
<class 'numpy.float64'>
43.4133217569
<class 'numpy.float64'>
43.4129054685
<class 'numpy.float64'>
43.412489275
<class 'numpy.float64'>
43.4120731762
<class 'numpy.float64'>
43.4116571722
<class 'numpy.float64'>
43.4112412628
<class 'numpy.float64'>
43.4108254482
<class 'numpy.float64'>
43.4104097281
<class 'numpy.float64'>
43.4099941026
<class 'numpy.float64'>
43.4095785716
<class 'numpy.float64'>
43.4091631352
<class 'numpy.float64'>
43.4087477932
<class 'numpy.float64'>
43.4083325457
<class 'numpy.float64'>
43.4079173925
<class 'numpy.float64'>
43.4075023337
<class 'numpy.float64'>
43.4070873692
<class 'numpy.float64'>
43.406672499
<class 'numpy.float64'>
43.4062577231
<class 'numpy.float64'>
43.4058430413
<class 'numpy.float64'>
43.4054284538
<class 'numpy.float64'>
43.4050139603
<class 'numpy.float64'>
43.4045995609
<class 'numpy.float64'>
43.4041852556
<class 'numpy.float64'>
43.4037710443
<class 'numpy.float64'>
43.403356927
<class 'numpy.float64'>
43.4029429037
<class 'numpy.float64'>
43.4025289742
<class 'numpy.float64'>
43.4021151386
<class 'numpy.float64'>
43.4017013968
<class 'numpy.float64'>
43.4012877489
<class 'numpy.float64'>
43.4008741947
<class 'numpy.float64'>
43.4004607342
<class 'numpy.float64'>
43.4000473674
<class 'numpy.float64'>
43.3996340942
<class 'numpy.float64'>
43.3992209146
<class 'numpy.float64'>
43.3988078287
<class 'numpy.float64'>
43.3983948362
<class 'numpy.float64'>
43.3979819373
<class 'numpy.float64'>
43.3975691318
<class 'numpy.float64'>
43.3971564197
<class 'numpy.float64'>
43.396743801
<class 'numpy.float64'>
43.3963312757
<class 'numpy.float64'>
43.3959188436
<class 'numpy.float64'>
43.3955065049
<class 'numpy.float64'>
43.3950942594
<class 'numpy.float64'>
43.394682107
<class 'numpy.float64'>
43.3942700479
<class 'numpy.float64'>
43.3938580819
<class 'numpy.float64'>
43.3934462089
<class 'numpy.float64'>
43.393034429
<class 'numpy.float64'>
43.3926227422
<class 'numpy.float64'>
43.3922111483
<class 'numpy.float64'>
43.3917996473
<class 'numpy.float64'>
43.3913882393
<class 'numpy.float64'>
43.3909769241
<class 'numpy.float64'>
43.3905657018
<class 'numpy.float64'>
43.3901545722
<class 'numpy.float64'>
43.3897435354
<class 'numpy.float64'>
43.3893325914
<class 'numpy.float64'>
43.38892174
<class 'numpy.float64'>
43.3885109812
<class 'numpy.float64'>
43.3881003151
<class 'numpy.float64'>
43.3876897416
<class 'numpy.float64'>
43.3872792606
<class 'numpy.float64'>
43.3868688721
<class 'numpy.float64'>
43.386458576
<class 'numpy.float64'>
43.3860483724
<class 'numpy.float64'>
43.3856382612
<class 'numpy.float64'>
43.3852282423
<class 'numpy.float64'>
43.3848183158
<class 'numpy.float64'>
Cost after iteration 7000: 43.384818
43.3844084815
<class 'numpy.float64'>
43.3839987395
<class 'numpy.float64'>
43.3835890897
<class 'numpy.float64'>
43.383179532
<class 'numpy.float64'>
43.3827700666
<class 'numpy.float64'>
43.3823606932
<class 'numpy.float64'>
43.3819514118
<class 'numpy.float64'>
43.3815422225
<class 'numpy.float64'>
43.3811331252
<class 'numpy.float64'>
43.3807241198
<class 'numpy.float64'>
43.3803152064
<class 'numpy.float64'>
43.3799063848
<class 'numpy.float64'>
43.3794976551
<class 'numpy.float64'>
43.3790890172
<class 'numpy.float64'>
43.3786804711
<class 'numpy.float64'>
43.3782720167
<class 'numpy.float64'>
43.377863654
<class 'numpy.float64'>
43.377455383
<class 'numpy.float64'>
43.3770472036
<class 'numpy.float64'>
43.3766391158
<class 'numpy.float64'>
43.3762311196
<class 'numpy.float64'>
43.3758232149
<class 'numpy.float64'>
43.3754154016
<class 'numpy.float64'>
43.3750076798
<class 'numpy.float64'>
43.3746000495
<class 'numpy.float64'>
43.3741925105
<class 'numpy.float64'>
43.3737850628
<class 'numpy.float64'>
43.3733777065
<class 'numpy.float64'>
43.3729704414
<class 'numpy.float64'>
43.3725632675
<class 'numpy.float64'>
43.3721561849
<class 'numpy.float64'>
43.3717491934
<class 'numpy.float64'>
43.371342293
<class 'numpy.float64'>
43.3709354838
<class 'numpy.float64'>
43.3705287656
<class 'numpy.float64'>
43.3701221384
<class 'numpy.float64'>
43.3697156021
<class 'numpy.float64'>
43.3693091569
<class 'numpy.float64'>
43.3689028025
<class 'numpy.float64'>
43.368496539
<class 'numpy.float64'>
43.3680903664
<class 'numpy.float64'>
43.3676842845
<class 'numpy.float64'>
43.3672782935
<class 'numpy.float64'>
43.3668723931
<class 'numpy.float64'>
43.3664665835
<class 'numpy.float64'>
43.3660608645
<class 'numpy.float64'>
43.3656552361
<class 'numpy.float64'>
43.3652496984
<class 'numpy.float64'>
43.3648442512
<class 'numpy.float64'>
43.3644388945
<class 'numpy.float64'>
43.3640336282
<class 'numpy.float64'>
43.3636284525
<class 'numpy.float64'>
43.3632233671
<class 'numpy.float64'>
43.3628183722
<class 'numpy.float64'>
43.3624134675
<class 'numpy.float64'>
43.3620086532
<class 'numpy.float64'>
43.3616039291
<class 'numpy.float64'>
43.3611992953
<class 'numpy.float64'>
43.3607947517
<class 'numpy.float64'>
43.3603902983
<class 'numpy.float64'>
43.3599859349
<class 'numpy.float64'>
43.3595816617
<class 'numpy.float64'>
43.3591774785
<class 'numpy.float64'>
43.3587733854
<class 'numpy.float64'>
43.3583693822
<class 'numpy.float64'>
43.357965469
<class 'numpy.float64'>
43.3575616457
<class 'numpy.float64'>
43.3571579123
<class 'numpy.float64'>
43.3567542688
<class 'numpy.float64'>
43.356350715
<class 'numpy.float64'>
43.3559472511
<class 'numpy.float64'>
43.3555438768
<class 'numpy.float64'>
43.3551405923
<class 'numpy.float64'>
43.3547373975
<class 'numpy.float64'>
43.3543342923
<class 'numpy.float64'>
43.3539312766
<class 'numpy.float64'>
43.3535283506
<class 'numpy.float64'>
43.3531255141
<class 'numpy.float64'>
43.3527227671
<class 'numpy.float64'>
43.3523201095
<class 'numpy.float64'>
43.3519175414
<class 'numpy.float64'>
43.3515150627
<class 'numpy.float64'>
43.3511126733
<class 'numpy.float64'>
43.3507103733
<class 'numpy.float64'>
43.3503081625
<class 'numpy.float64'>
43.349906041
<class 'numpy.float64'>
43.3495040087
<class 'numpy.float64'>
43.3491020656
<class 'numpy.float64'>
43.3487002116
<class 'numpy.float64'>
43.3482984468
<class 'numpy.float64'>
43.347896771
<class 'numpy.float64'>
43.3474951843
<class 'numpy.float64'>
43.3470936866
<class 'numpy.float64'>
43.3466922778
<class 'numpy.float64'>
43.346290958
<class 'numpy.float64'>
43.3458897271
<class 'numpy.float64'>
43.3454885851
<class 'numpy.float64'>
43.3450875319
<class 'numpy.float64'>
43.3446865676
<class 'numpy.float64'>
43.3442856919
<class 'numpy.float64'>
43.3438849051
<class 'numpy.float64'>
43.3434842069
<class 'numpy.float64'>
43.3430835973
<class 'numpy.float64'>
43.3426830765
<class 'numpy.float64'>
43.3422826442
<class 'numpy.float64'>
43.3418823004
<class 'numpy.float64'>
43.3414820452
<class 'numpy.float64'>
43.3410818785
<class 'numpy.float64'>
43.3406818002
<class 'numpy.float64'>
43.3402818104
<class 'numpy.float64'>
43.3398819089
<class 'numpy.float64'>
43.3394820958
<class 'numpy.float64'>
43.339082371
<class 'numpy.float64'>
43.3386827345
<class 'numpy.float64'>
43.3382831863
<class 'numpy.float64'>
43.3378837263
<class 'numpy.float64'>
43.3374843544
<class 'numpy.float64'>
43.3370850707
<class 'numpy.float64'>
43.3366858752
<class 'numpy.float64'>
43.3362867676
<class 'numpy.float64'>
43.3358877482
<class 'numpy.float64'>
43.3354888167
<class 'numpy.float64'>
43.3350899733
<class 'numpy.float64'>
43.3346912178
<class 'numpy.float64'>
43.3342925501
<class 'numpy.float64'>
43.3338939704
<class 'numpy.float64'>
43.3334954785
<class 'numpy.float64'>
43.3330970744
<class 'numpy.float64'>
43.3326987581
<class 'numpy.float64'>
43.3323005295
<class 'numpy.float64'>
43.3319023886
<class 'numpy.float64'>
43.3315043354
<class 'numpy.float64'>
43.3311063699
<class 'numpy.float64'>
43.3307084919
<class 'numpy.float64'>
43.3303107015
<class 'numpy.float64'>
43.3299129987
<class 'numpy.float64'>
43.3295153833
<class 'numpy.float64'>
43.3291178554
<class 'numpy.float64'>
43.328720415
<class 'numpy.float64'>
43.3283230619
<class 'numpy.float64'>
43.3279257962
<class 'numpy.float64'>
43.3275286179
<class 'numpy.float64'>
43.3271315268
<class 'numpy.float64'>
43.326734523
<class 'numpy.float64'>
43.3263376065
<class 'numpy.float64'>
43.3259407771
<class 'numpy.float64'>
43.3255440349
<class 'numpy.float64'>
43.3251473798
<class 'numpy.float64'>
43.3247508119
<class 'numpy.float64'>
43.324354331
<class 'numpy.float64'>
43.3239579371
<class 'numpy.float64'>
43.3235616302
<class 'numpy.float64'>
43.3231654103
<class 'numpy.float64'>
43.3227692773
<class 'numpy.float64'>
43.3223732312
<class 'numpy.float64'>
43.321977272
<class 'numpy.float64'>
43.3215813996
<class 'numpy.float64'>
43.321185614
<class 'numpy.float64'>
43.3207899151
<class 'numpy.float64'>
43.320394303
<class 'numpy.float64'>
43.3199987776
<class 'numpy.float64'>
43.3196033388
<class 'numpy.float64'>
43.3192079867
<class 'numpy.float64'>
43.3188127212
<class 'numpy.float64'>
43.3184175422
<class 'numpy.float64'>
43.3180224497
<class 'numpy.float64'>
43.3176274438
<class 'numpy.float64'>
43.3172325243
<class 'numpy.float64'>
43.3168376912
<class 'numpy.float64'>
43.3164429446
<class 'numpy.float64'>
43.3160482843
<class 'numpy.float64'>
43.3156537103
<class 'numpy.float64'>
43.3152592226
<class 'numpy.float64'>
43.3148648212
<class 'numpy.float64'>
43.314470506
<class 'numpy.float64'>
43.314076277
<class 'numpy.float64'>
43.3136821342
<class 'numpy.float64'>
43.3132880775
<class 'numpy.float64'>
43.3128941069
<class 'numpy.float64'>
43.3125002223
<class 'numpy.float64'>
43.3121064238
<class 'numpy.float64'>
43.3117127113
<class 'numpy.float64'>
43.3113190848
<class 'numpy.float64'>
43.3109255441
<class 'numpy.float64'>
43.3105320894
<class 'numpy.float64'>
43.3101387206
<class 'numpy.float64'>
43.3097454376
<class 'numpy.float64'>
43.3093522404
<class 'numpy.float64'>
43.3089591289
<class 'numpy.float64'>
43.3085661032
<class 'numpy.float64'>
43.3081731632
<class 'numpy.float64'>
43.3077803088
<class 'numpy.float64'>
43.3073875401
<class 'numpy.float64'>
43.306994857
<class 'numpy.float64'>
43.3066022595
<class 'numpy.float64'>
43.3062097475
<class 'numpy.float64'>
43.305817321
<class 'numpy.float64'>
43.30542498
<class 'numpy.float64'>
43.3050327244
<class 'numpy.float64'>
43.3046405542
<class 'numpy.float64'>
43.3042484694
<class 'numpy.float64'>
43.30385647
<class 'numpy.float64'>
43.3034645558
<class 'numpy.float64'>
43.303072727
<class 'numpy.float64'>
43.3026809834
<class 'numpy.float64'>
43.3022893249
<class 'numpy.float64'>
43.3018977517
<class 'numpy.float64'>
43.3015062636
<class 'numpy.float64'>
43.3011148606
<class 'numpy.float64'>
43.3007235427
<class 'numpy.float64'>
43.3003323099
<class 'numpy.float64'>
43.2999411621
<class 'numpy.float64'>
43.2995500992
<class 'numpy.float64'>
43.2991591213
<class 'numpy.float64'>
43.2987682284
<class 'numpy.float64'>
43.2983774203
<class 'numpy.float64'>
43.297986697
<class 'numpy.float64'>
43.2975960586
<class 'numpy.float64'>
43.297205505
<class 'numpy.float64'>
43.2968150362
<class 'numpy.float64'>
43.296424652
<class 'numpy.float64'>
43.2960343526
<class 'numpy.float64'>
43.2956441378
<class 'numpy.float64'>
43.2952540076
<class 'numpy.float64'>
43.2948639621
<class 'numpy.float64'>
43.2944740011
<class 'numpy.float64'>
43.2940841246
<class 'numpy.float64'>
43.2936943327
<class 'numpy.float64'>
43.2933046252
<class 'numpy.float64'>
43.2929150021
<class 'numpy.float64'>
43.2925254635
<class 'numpy.float64'>
43.2921360092
<class 'numpy.float64'>
43.2917466392
<class 'numpy.float64'>
43.2913573536
<class 'numpy.float64'>
43.2909681522
<class 'numpy.float64'>
43.2905790351
<class 'numpy.float64'>
43.2901900022
<class 'numpy.float64'>
43.2898010535
<class 'numpy.float64'>
43.2894121889
<class 'numpy.float64'>
43.2890234085
<class 'numpy.float64'>
43.2886347121
<class 'numpy.float64'>
43.2882460997
<class 'numpy.float64'>
43.2878575714
<class 'numpy.float64'>
43.2874691271
<class 'numpy.float64'>
43.2870807668
<class 'numpy.float64'>
43.2866924903
<class 'numpy.float64'>
43.2863042978
<class 'numpy.float64'>
43.2859161891
<class 'numpy.float64'>
43.2855281642
<class 'numpy.float64'>
43.2851402232
<class 'numpy.float64'>
43.2847523659
<class 'numpy.float64'>
43.2843645923
<class 'numpy.float64'>
43.2839769025
<class 'numpy.float64'>
43.2835892963
<class 'numpy.float64'>
43.2832017737
<class 'numpy.float64'>
43.2828143348
<class 'numpy.float64'>
43.2824269794
<class 'numpy.float64'>
43.2820397076
<class 'numpy.float64'>
43.2816525193
<class 'numpy.float64'>
43.2812654145
<class 'numpy.float64'>
43.2808783931
<class 'numpy.float64'>
43.2804914552
<class 'numpy.float64'>
43.2801046006
<class 'numpy.float64'>
43.2797178294
<class 'numpy.float64'>
43.2793311415
<class 'numpy.float64'>
43.2789445369
<class 'numpy.float64'>
43.2785580155
<class 'numpy.float64'>
43.2781715774
<class 'numpy.float64'>
43.2777852225
<class 'numpy.float64'>
43.2773989508
<class 'numpy.float64'>
43.2770127621
<class 'numpy.float64'>
43.2766266566
<class 'numpy.float64'>
43.2762406342
<class 'numpy.float64'>
43.2758546948
<class 'numpy.float64'>
43.2754688384
<class 'numpy.float64'>
43.275083065
<class 'numpy.float64'>
43.2746973746
<class 'numpy.float64'>
43.274311767
<class 'numpy.float64'>
43.2739262424
<class 'numpy.float64'>
43.2735408006
<class 'numpy.float64'>
43.2731554416
<class 'numpy.float64'>
43.2727701654
<class 'numpy.float64'>
43.272384972
<class 'numpy.float64'>
43.2719998613
<class 'numpy.float64'>
43.2716148333
<class 'numpy.float64'>
43.2712298879
<class 'numpy.float64'>
43.2708450252
<class 'numpy.float64'>
43.2704602451
<class 'numpy.float64'>
43.2700755476
<class 'numpy.float64'>
43.2696909326
<class 'numpy.float64'>
43.2693064002
<class 'numpy.float64'>
43.2689219502
<class 'numpy.float64'>
43.2685375826
<class 'numpy.float64'>
43.2681532975
<class 'numpy.float64'>
43.2677690948
<class 'numpy.float64'>
43.2673849744
<class 'numpy.float64'>
43.2670009364
<class 'numpy.float64'>
43.2666169806
<class 'numpy.float64'>
43.2662331071
<class 'numpy.float64'>
43.2658493159
<class 'numpy.float64'>
43.2654656068
<class 'numpy.float64'>
43.2650819799
<class 'numpy.float64'>
43.2646984352
<class 'numpy.float64'>
43.2643149726
<class 'numpy.float64'>
43.263931592
<class 'numpy.float64'>
43.2635482935
<class 'numpy.float64'>
43.2631650771
<class 'numpy.float64'>
43.2627819426
<class 'numpy.float64'>
43.26239889
<class 'numpy.float64'>
43.2620159194
<class 'numpy.float64'>
43.2616330307
<class 'numpy.float64'>
43.2612502239
<class 'numpy.float64'>
43.2608674988
<class 'numpy.float64'>
43.2604848556
<class 'numpy.float64'>
43.2601022942
<class 'numpy.float64'>
43.2597198145
<class 'numpy.float64'>
43.2593374165
<class 'numpy.float64'>
43.2589551002
<class 'numpy.float64'>
43.2585728656
<class 'numpy.float64'>
43.2581907125
<class 'numpy.float64'>
43.2578086411
<class 'numpy.float64'>
43.2574266512
<class 'numpy.float64'>
43.2570447429
<class 'numpy.float64'>
43.256662916
<class 'numpy.float64'>
43.2562811706
<class 'numpy.float64'>
43.2558995067
<class 'numpy.float64'>
43.2555179241
<class 'numpy.float64'>
43.255136423
<class 'numpy.float64'>
43.2547550032
<class 'numpy.float64'>
43.2543736647
<class 'numpy.float64'>
43.2539924075
<class 'numpy.float64'>
43.2536112315
<class 'numpy.float64'>
43.2532301368
<class 'numpy.float64'>
43.2528491232
<class 'numpy.float64'>
43.2524681909
<class 'numpy.float64'>
43.2520873396
<class 'numpy.float64'>
43.2517065695
<class 'numpy.float64'>
43.2513258805
<class 'numpy.float64'>
43.2509452725
<class 'numpy.float64'>
43.2505647455
<class 'numpy.float64'>
43.2501842995
<class 'numpy.float64'>
43.2498039344
<class 'numpy.float64'>
43.2494236503
<class 'numpy.float64'>
43.249043447
<class 'numpy.float64'>
43.2486633247
<class 'numpy.float64'>
43.2482832831
<class 'numpy.float64'>
43.2479033224
<class 'numpy.float64'>
43.2475234425
<class 'numpy.float64'>
43.2471436432
<class 'numpy.float64'>
43.2467639247
<class 'numpy.float64'>
43.2463842869
<class 'numpy.float64'>
43.2460047298
<class 'numpy.float64'>
43.2456252532
<class 'numpy.float64'>
43.2452458573
<class 'numpy.float64'>
43.2448665419
<class 'numpy.float64'>
43.2444873071
<class 'numpy.float64'>
43.2441081528
<class 'numpy.float64'>
43.2437290789
<class 'numpy.float64'>
43.2433500855
<class 'numpy.float64'>
43.2429711725
<class 'numpy.float64'>
43.2425923399
<class 'numpy.float64'>
43.2422135876
<class 'numpy.float64'>
43.2418349157
<class 'numpy.float64'>
43.2414563241
<class 'numpy.float64'>
43.2410778127
<class 'numpy.float64'>
43.2406993816
<class 'numpy.float64'>
43.2403210306
<class 'numpy.float64'>
43.2399427599
<class 'numpy.float64'>
43.2395645693
<class 'numpy.float64'>
43.2391864588
<class 'numpy.float64'>
43.2388084284
<class 'numpy.float64'>
43.2384304781
<class 'numpy.float64'>
43.2380526078
<class 'numpy.float64'>
43.2376748175
<class 'numpy.float64'>
43.2372971072
<class 'numpy.float64'>
43.2369194768
<class 'numpy.float64'>
43.2365419264
<class 'numpy.float64'>
43.2361644558
<class 'numpy.float64'>
43.235787065
<class 'numpy.float64'>
43.2354097541
<class 'numpy.float64'>
43.235032523
<class 'numpy.float64'>
43.2346553717
<class 'numpy.float64'>
43.2342783001
<class 'numpy.float64'>
43.2339013081
<class 'numpy.float64'>
43.2335243959
<class 'numpy.float64'>
43.2331475633
<class 'numpy.float64'>
43.2327708104
<class 'numpy.float64'>
43.232394137
<class 'numpy.float64'>
43.2320175432
<class 'numpy.float64'>
43.2316410289
<class 'numpy.float64'>
43.2312645941
<class 'numpy.float64'>
43.2308882388
<class 'numpy.float64'>
43.2305119629
<class 'numpy.float64'>
43.2301357665
<class 'numpy.float64'>
43.2297596494
<class 'numpy.float64'>
43.2293836117
<class 'numpy.float64'>
43.2290076533
<class 'numpy.float64'>
43.2286317742
<class 'numpy.float64'>
43.2282559743
<class 'numpy.float64'>
43.2278802538
<class 'numpy.float64'>
43.2275046124
<class 'numpy.float64'>
43.2271290502
<class 'numpy.float64'>
43.2267535671
<class 'numpy.float64'>
43.2263781632
<class 'numpy.float64'>
43.2260028384
<class 'numpy.float64'>
43.2256275926
<class 'numpy.float64'>
43.2252524259
<class 'numpy.float64'>
43.2248773382
<class 'numpy.float64'>
43.2245023294
<class 'numpy.float64'>
43.2241273996
<class 'numpy.float64'>
43.2237525487
<class 'numpy.float64'>
43.2233777767
<class 'numpy.float64'>
43.2230030836
<class 'numpy.float64'>
43.2226284693
<class 'numpy.float64'>
43.2222539338
<class 'numpy.float64'>
43.2218794771
<class 'numpy.float64'>
43.2215050991
<class 'numpy.float64'>
43.2211307999
<class 'numpy.float64'>
43.2207565793
<class 'numpy.float64'>
43.2203824374
<class 'numpy.float64'>
43.2200083742
<class 'numpy.float64'>
43.2196343895
<class 'numpy.float64'>
43.2192604834
<class 'numpy.float64'>
43.2188866558
<class 'numpy.float64'>
43.2185129068
<class 'numpy.float64'>
43.2181392363
<class 'numpy.float64'>
43.2177656442
<class 'numpy.float64'>
43.2173921305
<class 'numpy.float64'>
43.2170186953
<class 'numpy.float64'>
43.2166453384
<class 'numpy.float64'>
43.2162720599
<class 'numpy.float64'>
43.2158988597
<class 'numpy.float64'>
43.2155257377
<class 'numpy.float64'>
43.2151526941
<class 'numpy.float64'>
43.2147797286
<class 'numpy.float64'>
43.2144068414
<class 'numpy.float64'>
43.2140340323
<class 'numpy.float64'>
43.2136613014
<class 'numpy.float64'>
43.2132886486
<class 'numpy.float64'>
43.2129160739
<class 'numpy.float64'>
43.2125435772
<class 'numpy.float64'>
43.2121711586
<class 'numpy.float64'>
43.211798818
<class 'numpy.float64'>
43.2114265553
<class 'numpy.float64'>
43.2110543706
<class 'numpy.float64'>
43.2106822639
<class 'numpy.float64'>
43.210310235
<class 'numpy.float64'>
43.209938284
<class 'numpy.float64'>
43.2095664108
<class 'numpy.float64'>
43.2091946154
<class 'numpy.float64'>
43.2088228978
<class 'numpy.float64'>
43.2084512579
<class 'numpy.float64'>
43.2080796957
<class 'numpy.float64'>
43.2077082113
<class 'numpy.float64'>
43.2073368045
<class 'numpy.float64'>
43.2069654754
<class 'numpy.float64'>
43.2065942238
<class 'numpy.float64'>
43.2062230499
<class 'numpy.float64'>
43.2058519534
<class 'numpy.float64'>
43.2054809346
<class 'numpy.float64'>
43.2051099932
<class 'numpy.float64'>
43.2047391292
<class 'numpy.float64'>
43.2043683427
<class 'numpy.float64'>
43.2039976337
<class 'numpy.float64'>
43.203627002
<class 'numpy.float64'>
43.2032564476
<class 'numpy.float64'>
43.2028859706
<class 'numpy.float64'>
43.2025155709
<class 'numpy.float64'>
43.2021452484
<class 'numpy.float64'>
43.2017750033
<class 'numpy.float64'>
43.2014048353
<class 'numpy.float64'>
43.2010347445
<class 'numpy.float64'>
43.2006647308
<class 'numpy.float64'>
43.2002947943
<class 'numpy.float64'>
43.1999249349
<class 'numpy.float64'>
43.1995551526
<class 'numpy.float64'>
43.1991854473
<class 'numpy.float64'>
43.1988158191
<class 'numpy.float64'>
43.1984462678
<class 'numpy.float64'>
43.1980767935
<class 'numpy.float64'>
43.1977073961
<class 'numpy.float64'>
43.1973380757
<class 'numpy.float64'>
43.1969688321
<class 'numpy.float64'>
43.1965996654
<class 'numpy.float64'>
43.1962305754
<class 'numpy.float64'>
43.1958615624
<class 'numpy.float64'>
43.195492626
<class 'numpy.float64'>
43.1951237665
<class 'numpy.float64'>
43.1947549836
<class 'numpy.float64'>
43.1943862774
<class 'numpy.float64'>
43.1940176479
<class 'numpy.float64'>
43.193649095
<class 'numpy.float64'>
43.1932806187
<class 'numpy.float64'>
43.192912219
<class 'numpy.float64'>
43.1925438958
<class 'numpy.float64'>
43.1921756492
<class 'numpy.float64'>
43.191807479
<class 'numpy.float64'>
43.1914393854
<class 'numpy.float64'>
43.1910713681
<class 'numpy.float64'>
43.1907034273
<class 'numpy.float64'>
43.1903355628
<class 'numpy.float64'>
43.1899677748
<class 'numpy.float64'>
43.189600063
<class 'numpy.float64'>
43.1892324276
<class 'numpy.float64'>
43.1888648684
<class 'numpy.float64'>
43.1884973855
<class 'numpy.float64'>
43.1881299787
<class 'numpy.float64'>
43.1877626483
<class 'numpy.float64'>
43.1873953939
<class 'numpy.float64'>
43.1870282158
<class 'numpy.float64'>
43.1866611136
<class 'numpy.float64'>
43.1862940877
<class 'numpy.float64'>
43.1859271376
<class 'numpy.float64'>
43.1855602638
<class 'numpy.float64'>
43.1851934657
<class 'numpy.float64'>
43.1848267439
<class 'numpy.float64'>
43.1844600978
<class 'numpy.float64'>
43.1840935278
<class 'numpy.float64'>
43.1837270335
<class 'numpy.float64'>
43.1833606153
<class 'numpy.float64'>
43.1829942727
<class 'numpy.float64'>
43.1826280062
<class 'numpy.float64'>
43.1822618151
<class 'numpy.float64'>
43.1818957002
<class 'numpy.float64'>
43.1815296606
<class 'numpy.float64'>
43.1811636971
<class 'numpy.float64'>
43.1807978088
<class 'numpy.float64'>
43.1804319967
<class 'numpy.float64'>
43.1800662597
<class 'numpy.float64'>
43.1797005988
<class 'numpy.float64'>
43.179335013
<class 'numpy.float64'>
43.1789695032
<class 'numpy.float64'>
43.1786040684
<class 'numpy.float64'>
43.1782387096
<class 'numpy.float64'>
43.1778734258
<class 'numpy.float64'>
43.1775082179
<class 'numpy.float64'>
43.1771430849
<class 'numpy.float64'>
43.1767780278
<class 'numpy.float64'>
43.1764130455
<class 'numpy.float64'>
43.1760481392
<class 'numpy.float64'>
43.1756833074
<class 'numpy.float64'>
43.1753185517
<class 'numpy.float64'>
43.1749538704
<class 'numpy.float64'>
43.1745892652
<class 'numpy.float64'>
43.1742247343
<class 'numpy.float64'>
43.1738602795
<class 'numpy.float64'>
43.1734958988
<class 'numpy.float64'>
43.1731315944
<class 'numpy.float64'>
43.1727673638
<class 'numpy.float64'>
43.1724032096
<class 'numpy.float64'>
43.1720391291
<class 'numpy.float64'>
43.1716751249
<class 'numpy.float64'>
43.1713111943
<class 'numpy.float64'>
43.1709473401
<class 'numpy.float64'>
43.1705835593
<class 'numpy.float64'>
43.1702198551
<class 'numpy.float64'>
43.169856224
<class 'numpy.float64'>
43.1694926695
<class 'numpy.float64'>
43.169129188
<class 'numpy.float64'>
43.1687657833
<class 'numpy.float64'>
43.1684024512
<class 'numpy.float64'>
43.1680391961
<class 'numpy.float64'>
43.1676760133
<class 'numpy.float64'>
43.1673129077
<class 'numpy.float64'>
43.1669498742
<class 'numpy.float64'>
43.166586918
<class 'numpy.float64'>
43.1662240336
<class 'numpy.float64'>
43.1658612268
<class 'numpy.float64'>
43.1654984913
<class 'numpy.float64'>
43.1651358338
<class 'numpy.float64'>
43.1647732471
<class 'numpy.float64'>
43.1644107387
<class 'numpy.float64'>
43.1640483008
<class 'numpy.float64'>
43.1636859415
<class 'numpy.float64'>
43.1633236522
<class 'numpy.float64'>
43.1629614419
<class 'numpy.float64'>
43.1625993011
<class 'numpy.float64'>
43.1622372397
<class 'numpy.float64'>
43.1618752472
<class 'numpy.float64'>
43.1615133347
<class 'numpy.float64'>
43.1611514903
<class 'numpy.float64'>
43.1607897266
<class 'numpy.float64'>
43.1604280303
<class 'numpy.float64'>
43.1600664154
<class 'numpy.float64'>
43.1597048669
<class 'numpy.float64'>
43.1593434006
<class 'numpy.float64'>
43.1589819999
<class 'numpy.float64'>
43.1586206822
<class 'numpy.float64'>
43.1582594291
<class 'numpy.float64'>
43.15789826
<class 'numpy.float64'>
43.1575371543
<class 'numpy.float64'>
43.1571761337
<class 'numpy.float64'>
43.1568151753
<class 'numpy.float64'>
43.1564543032
<class 'numpy.float64'>
43.1560934918
<class 'numpy.float64'>
43.1557327682
<class 'numpy.float64'>
43.1553721037
<class 'numpy.float64'>
43.1550115285
<class 'numpy.float64'>
43.1546510106
<class 'numpy.float64'>
43.1542905839
<class 'numpy.float64'>
43.1539302126
<class 'numpy.float64'>
43.1535699343
<class 'numpy.float64'>
43.1532097092
<class 'numpy.float64'>
43.1528495794
<class 'numpy.float64'>
43.1524895003
<class 'numpy.float64'>
43.1521295191
<class 'numpy.float64'>
43.1517695856
<class 'numpy.float64'>
43.1514097531
<class 'numpy.float64'>
43.1510499651
<class 'numpy.float64'>
43.1506902812
<class 'numpy.float64'>
43.1503306383
<class 'numpy.float64'>
43.1499711032
<class 'numpy.float64'>
43.1496116052
<class 'numpy.float64'>
43.1492522191
<class 'numpy.float64'>
43.1488928655
<class 'numpy.float64'>
43.1485336284
<class 'numpy.float64'>
43.1481744189
<class 'numpy.float64'>
43.1478153312
<class 'numpy.float64'>
43.1474562653
<class 'numpy.float64'>
43.1470973272
<class 'numpy.float64'>
43.1467384044
<class 'numpy.float64'>
43.1463796161
<class 'numpy.float64'>
43.146020836
<class 'numpy.float64'>
43.1456621979
<class 'numpy.float64'>
43.1453035599
<class 'numpy.float64'>
43.1449450724
<class 'numpy.float64'>
43.1445865757
<class 'numpy.float64'>
43.1442282393
<class 'numpy.float64'>
43.1438698834
<class 'numpy.float64'>
43.1435116986
<class 'numpy.float64'>
43.1431534826
<class 'numpy.float64'>
43.1427954501
<class 'numpy.float64'>
43.142437373
<class 'numpy.float64'>
43.1420794935
<class 'numpy.float64'>
43.1417215545
<class 'numpy.float64'>
43.1413638289
<class 'numpy.float64'>
43.1410060267
<class 'numpy.float64'>
43.1406484559
<class 'numpy.float64'>
43.1402907894
<class 'numpy.float64'>
43.1399333746
<class 'numpy.float64'>
43.1395758423
<class 'numpy.float64'>
43.1392185847
<class 'numpy.float64'>
43.1388611851
<class 'numpy.float64'>
43.1385040863
<class 'numpy.float64'>
43.1381468175
<class 'numpy.float64'>
43.1377898791
<class 'numpy.float64'>
43.1374327392
<class 'numpy.float64'>
43.1370759632
<class 'numpy.float64'>
43.1367189498
<class 'numpy.float64'>
43.1363623384
<class 'numpy.float64'>
43.1360054489
<class 'numpy.float64'>
43.1356490048
<class 'numpy.float64'>
43.1352922363
<class 'numpy.float64'>
43.1349359623
<class 'numpy.float64'>
43.1345793114
<class 'numpy.float64'>
43.134223211
<class 'numpy.float64'>
43.1338666739
<class 'numpy.float64'>
43.1335107508
<class 'numpy.float64'>
43.1331543232
<class 'numpy.float64'>
43.132798582
<class 'numpy.float64'>
43.1324422589
<class 'numpy.float64'>
43.1320867045
<class 'numpy.float64'>
43.1317304804
<class 'numpy.float64'>
43.1313751187
<class 'numpy.float64'>
43.1310189871
<class 'numpy.float64'>
43.1306638247
<class 'numpy.float64'>
43.1303077783
<class 'numpy.float64'>
43.1299528228
<class 'numpy.float64'>
43.1295968533
<class 'numpy.float64'>
43.1292421134
<class 'numpy.float64'>
43.1288862113
<class 'numpy.float64'>
43.128531697
<class 'numpy.float64'>
43.1281758514
<class 'numpy.float64'>
43.1278215741
<class 'numpy.float64'>
43.1274657726
<class 'numpy.float64'>
43.1271117454
<class 'numpy.float64'>
43.1267559737
<class 'numpy.float64'>
43.1264022116
<class 'numpy.float64'>
43.1260464535
<class 'numpy.float64'>
43.1256929738
<class 'numpy.float64'>
43.1253372106
<class 'numpy.float64'>
43.124984033
<class 'numpy.float64'>
43.1246282433
<class 'numpy.float64'>
43.1242753905
<class 'numpy.float64'>
43.1239195499
<class 'numpy.float64'>
43.123567048
<class 'numpy.float64'>
43.1232111282
<class 'numpy.float64'>
43.1228590071
<class 'numpy.float64'>
43.122502976
<class 'numpy.float64'>
43.1221512701
<class 'numpy.float64'>
43.1217950905
<class 'numpy.float64'>
43.1214438393
<class 'numpy.float64'>
43.1210874686
<class 'numpy.float64'>
43.1207367177
<class 'numpy.float64'>
43.120380107
<class 'numpy.float64'>
43.1200299086
<class 'numpy.float64'>
43.1196730016
<class 'numpy.float64'>
43.1193234159
<class 'numpy.float64'>
43.1189661477
<class 'numpy.float64'>
43.1186172441
<class 'numpy.float64'>
43.1182595402
<class 'numpy.float64'>
43.1179113984
<class 'numpy.float64'>
43.1175531731
<class 'numpy.float64'>
43.117205885
<class 'numpy.float64'>
43.1168470392
<class 'numpy.float64'>
43.1165007108
<class 'numpy.float64'>
43.1161411307
<class 'numpy.float64'>
43.1157958843
<class 'numpy.float64'>
43.1154354382
<class 'numpy.float64'>
43.1150914148
<class 'numpy.float64'>
43.1147299512
<class 'numpy.float64'>
43.1143873134
<class 'numpy.float64'>
43.1140246573
<class 'numpy.float64'>
43.1136835929
<class 'numpy.float64'>
43.1133195423
<class 'numpy.float64'>
43.1129802684
<class 'numpy.float64'>
43.1126145897
<class 'numpy.float64'>
43.112277357
<class 'numpy.float64'>
43.1119097806
<class 'numpy.float64'>
43.1115748789
<class 'numpy.float64'>
43.1112050929
<class 'numpy.float64'>
43.1108728573
<class 'numpy.float64'>
43.1105005011
<class 'numpy.float64'>
43.1101713194
<class 'numpy.float64'>
43.1097959755
<class 'numpy.float64'>
43.1094702966
<class 'numpy.float64'>
43.109091482
<class 'numpy.float64'>
43.1087698255
<class 'numpy.float64'>
43.1083869806
<class 'numpy.float64'>
43.1080699486
<class 'numpy.float64'>
43.1076824251
<class 'numpy.float64'>
43.1073707154
<class 'numpy.float64'>
43.1069777619
<class 'numpy.float64'>
43.1066721834
<class 'numpy.float64'>
43.1062729284
<class 'numpy.float64'>
43.1059744195
<class 'numpy.float64'>
43.1055678522
<class 'numpy.float64'>
43.1052775016
<class 'numpy.float64'>
43.1048624487
<class 'numpy.float64'>
43.1045815204
<class 'numpy.float64'>
43.1041566196
<class 'numpy.float64'>
43.1038865817
<class 'numpy.float64'>
43.1034502506
<class 'numpy.float64'>
43.1031928085
<class 'numpy.float64'>
43.1027432082
<class 'numpy.float64'>
43.1025003445
<class 'numpy.float64'>
43.102035337
<class 'numpy.float64'>
43.1018093572
<class 'numpy.float64'>
43.1013264558
<class 'numpy.float64'>
43.1011200418
<class 'numpy.float64'>
43.1006163531
<class 'numpy.float64'>
43.1004326264
<class 'numpy.float64'>
43.0999047823
<class 'numpy.float64'>
43.0997473773
<class 'numpy.float64'>
43.0991914551
<class 'numpy.float64'>
43.0990646055
<class 'numpy.float64'>
43.0984760351
<class 'numpy.float64'>
43.0983846743
<class 'numpy.float64'>
43.097758129
<class 'numpy.float64'>
43.0977080088
<class 'numpy.float64'>
43.097037277
<class 'numpy.float64'>
43.0970351057
<class 'numpy.float64'>
43.0963129413
<class 'numpy.float64'>
43.0963665464
<class 'numpy.float64'>
43.0955844927
<class 'numpy.float64'>
43.0957030115
<class 'numpy.float64'>
43.0948511946
<class 'numpy.float64'>
43.0950452978
<class 'numpy.float64'>
43.0941121845
<class 'numpy.float64'>
43.0943943387
<class 'numpy.float64'>
43.0933664515
<class 'numpy.float64'>
43.0937512282
<class 'numpy.float64'>
43.0926128114
<class 'numpy.float64'>
43.0931172489
<class 'numpy.float64'>
43.091849875
<class 'numpy.float64'>
43.0924939052
<class 'numpy.float64'>
43.0910760131
<class 'numpy.float64'>
43.0918829623
<class 'numpy.float64'>
43.0902893137
<class 'numpy.float64'>
43.0912864926
<class 'numpy.float64'>
43.0894875318
<class 'numpy.float64'>
43.0907069295
<class 'numpy.float64'>
43.0886680308
<class 'numpy.float64'>
43.0901471325
<class 'numpy.float64'>
43.0878277127
<class 'numpy.float64'>
43.0896104623
<class 'numpy.float64'>
43.0869629358
<class 'numpy.float64'>
43.0891008707
<class 'numpy.float64'>
43.0860694174
<class 'numpy.float64'>
43.0886230069
<class 'numpy.float64'>
43.0851421194
<class 'numpy.float64'>
43.0881823422
<class 'numpy.float64'>
43.084175112
<class 'numpy.float64'>
43.0877853186
<class 'numpy.float64'>
43.0831614137
<class 'numpy.float64'>
43.0874395245
<class 'numpy.float64'>
43.0820928008
<class 'numpy.float64'>
43.0871539018
<class 'numpy.float64'>
43.0809595835
<class 'numpy.float64'>
43.0869389929
<class 'numpy.float64'>
43.0797503388
<class 'numpy.float64'>
43.0868072323
<class 'numpy.float64'>
43.0784515964
<class 'numpy.float64'>
43.0867732924
<class 'numpy.float64'>
43.0770474649
<class 'numpy.float64'>
43.0868544945
<class 'numpy.float64'>
43.07551919
<class 'numpy.float64'>
43.0870712966
<class 'numpy.float64'>
43.0738446303
<class 'numpy.float64'>
43.0874478722
<class 'numpy.float64'>
43.0719976367
<class 'numpy.float64'>
43.0880127989
<class 'numpy.float64'>
43.0699473152
<class 'numpy.float64'>
43.0887998771
<class 'numpy.float64'>
43.0676571549
<class 'numpy.float64'>
43.0898491044
<class 'numpy.float64'>
43.0650839922
<class 'numpy.float64'>
43.0912078359
<class 'numpy.float64'>
43.0621767833
<class 'numpy.float64'>
43.0929321668
<class 'numpy.float64'>
43.0588751492
<class 'numpy.float64'>
43.0950885813
<class 'numpy.float64'>
43.0551076502
<class 'numpy.float64'>
43.0977559211
<class 'numpy.float64'>
43.0507897421
<class 'numpy.float64'>
43.1010277365
<class 'numpy.float64'>
43.0458213545
<class 'numpy.float64'>
43.1050150969
<class 'numpy.float64'>
43.0400840259
<class 'numpy.float64'>
43.1098499556
<class 'numpy.float64'>
43.0334375153
<class 'numpy.float64'>
43.1156891787
<class 'numpy.float64'>
43.0257158028
<class 'numpy.float64'>
43.1227193769
<class 'numpy.float64'>
43.0167223755
<class 'numpy.float64'>
43.1311627039
<class 'numpy.float64'>
43.0062246867
<class 'numpy.float64'>
43.1412838196
<class 'numpy.float64'>
42.9939476637
<class 'numpy.float64'>
43.1533982611
<class 'numpy.float64'>
42.9795661367
<class 'numpy.float64'>
43.1678825047
<class 'numpy.float64'>
42.9626960628
<class 'numpy.float64'>
43.1851860633
<class 'numpy.float64'>
42.9428844405
<class 'numpy.float64'>
43.2058460146
<class 'numpy.float64'>
42.9195978564
<class 'numpy.float64'>
43.2305044115
<class 'numpy.float64'>
42.8922097007
<class 'numpy.float64'>
43.2599290659
<class 'numpy.float64'>
42.8599862582
<class 'numpy.float64'>
43.295038186
<class 'numpy.float64'>
42.8220721735
<class 'numpy.float64'>
43.336929258
<class 'numpy.float64'>
42.7774762717
<class 'numpy.float64'>
43.3869122771
<class 'numpy.float64'>
42.725059493
<class 'numpy.float64'>
43.4465468278
<class 'numpy.float64'>
42.663527913
<class 'numpy.float64'>
43.5176812927
<class 'numpy.float64'>
42.5914356596
<class 'numpy.float64'>
43.602490228
<class 'numpy.float64'>
42.5072052004
<class 'numpy.float64'>
43.703502001
<class 'numpy.float64'>
42.4091761018
<class 'numpy.float64'>
43.8236022033
<class 'numpy.float64'>
42.2956977776
<class 'numpy.float64'>
43.9659880749
<class 'numpy.float64'>
42.1652860139
<class 'numpy.float64'>
44.1340345449
<class 'numpy.float64'>
42.0168645986
<class 'numpy.float64'>
44.3310150095
<class 'numpy.float64'>
41.8501068593
<class 'numpy.float64'>
44.5596067175
<class 'numpy.float64'>
41.665868622
<class 'numpy.float64'>
44.8211199374
<class 'numpy.float64'>
41.4666543397
<class 'numpy.float64'>
45.1144554819
<class 'numpy.float64'>
41.256980822
<class 'numpy.float64'>
45.4349551825
<class 'numpy.float64'>
41.0434244904
<class 'numpy.float64'>
45.7735615445
<class 'numpy.float64'>
40.8341276906
<class 'numpy.float64'>
46.1169185097
<class 'numpy.float64'>
40.6376840281
<class 'numpy.float64'>
46.4489461316
<class 'numpy.float64'>
40.4616292675
<class 'numpy.float64'>
46.7537892109
<class 'numpy.float64'>
40.3110632436
<class 'numpy.float64'>
47.0191273832
<class 'numpy.float64'>
40.1879525114
<class 'numpy.float64'>
47.238408488
<class 'numpy.float64'>
40.0913287509
<class 'numpy.float64'>
47.4111420945
<class 'numpy.float64'>
40.01814997
<class 'numpy.float64'>
47.5415096081
<class 'numpy.float64'>
39.964363555
<class 'numpy.float64'>
47.6362692382
<class 'numpy.float64'>
39.9257903162
<class 'numpy.float64'>
47.7028753681
<class 'numpy.float64'>
39.8986726783
<class 'numpy.float64'>
47.748246034
<class 'numpy.float64'>
39.8799126808
<class 'numpy.float64'>
47.7781782803
<class 'numpy.float64'>
39.8671017242
<class 'numpy.float64'>
47.7972153042
<class 'numpy.float64'>
39.8584427104
<class 'numpy.float64'>
47.8087562689
<class 'numpy.float64'>
39.8526340876
<class 'numpy.float64'>
47.8152602516
<class 'numpy.float64'>
39.8487539166
<class 'numpy.float64'>
47.8184610535
<class 'numpy.float64'>
39.8461603803
<class 'numpy.float64'>
47.819555593
<class 'numpy.float64'>
39.8444130327
<class 'numpy.float64'>
47.8193546821
<class 'numpy.float64'>
39.8432134307
<class 'numpy.float64'>
47.8183973814
<class 'numpy.float64'>
39.8423617793
<class 'numpy.float64'>
47.8170347832
<class 'numpy.float64'>
39.8417259623
<class 'numpy.float64'>
47.8154899464
<class 'numpy.float64'>
39.8412197752
<class 'numpy.float64'>
47.8139000195
<class 'numpy.float64'>
39.8407878209
<class 'numpy.float64'>
47.8123454246
<class 'numpy.float64'>
39.8403951419
<class 'numpy.float64'>
47.8108698239
<class 'numpy.float64'>
39.8400201743
<class 'numpy.float64'>
47.809493613
<class 'numpy.float64'>
39.8396500069
<class 'numpy.float64'>
47.808222921
<class 'numpy.float64'>
39.8392772244
<class 'numpy.float64'>
47.807055526
<class 'numpy.float64'>
39.8388978293
<class 'numpy.float64'>
47.8059846719
<class 'numpy.float64'>
39.8385098928
<class 'numpy.float64'>
47.8050014757
<class 'numpy.float64'>
39.8381126922
<class 'numpy.float64'>
47.8040963971
<class 'numpy.float64'>
39.8377061706
<class 'numpy.float64'>
47.803260095
<class 'numpy.float64'>
39.8372906075
<class 'numpy.float64'>
47.8024838899
<class 'numpy.float64'>
39.8368664238
<class 'numpy.float64'>
47.8017599778
<class 'numpy.float64'>
39.8364340734
<class 'numpy.float64'>
47.8010814955
<class 'numpy.float64'>
39.8359939868
<class 'numpy.float64'>
47.800442497
<class 'numpy.float64'>
39.8355465467
<class 'numpy.float64'>
47.7998378848
<class 'numpy.float64'>
39.8350920811
<class 'numpy.float64'>
47.7992633194
<class 'numpy.float64'>
39.8346308661
<class 'numpy.float64'>
47.7987151231
<class 'numpy.float64'>
39.8341631338
<class 'numpy.float64'>
47.7981901873
<class 'numpy.float64'>
39.8336890801
<class 'numpy.float64'>
47.7976858861
<class 'numpy.float64'>
39.8332088741
<class 'numpy.float64'>
47.7972000009
<class 'numpy.float64'>
39.8327226651
<class 'numpy.float64'>
47.7967306539
<class 'numpy.float64'>
39.8322305889
<class 'numpy.float64'>
47.7962762517
<class 'numpy.float64'>
39.8317327728
<class 'numpy.float64'>
47.7958354382
<class 'numpy.float64'>
39.8312293393
<class 'numpy.float64'>
47.7954070539
<class 'numpy.float64'>
39.8307204085
<class 'numpy.float64'>
47.7949901035
<class 'numpy.float64'>
39.8302061003
<class 'numpy.float64'>
47.7945837281
<class 'numpy.float64'>
39.8296865349
<class 'numpy.float64'>
47.7941871826
<class 'numpy.float64'>
39.8291618338
<class 'numpy.float64'>
47.7937998169
<class 'numpy.float64'>
39.8286321201
<class 'numpy.float64'>
47.7934210607
<class 'numpy.float64'>
39.828097518
<class 'numpy.float64'>
47.7930504108
<class 'numpy.float64'>
39.8275581531
<class 'numpy.float64'>
47.7926874204
<class 'numpy.float64'>
39.8270141516
<class 'numpy.float64'>
47.7923316905
<class 'numpy.float64'>
39.8264656402
<class 'numpy.float64'>
47.791982863
<class 'numpy.float64'>
39.8259127457
<class 'numpy.float64'>
47.7916406146
<class 'numpy.float64'>
39.8253555943
<class 'numpy.float64'>
47.7913046517
<class 'numpy.float64'>
39.8247943115
<class 'numpy.float64'>
47.7909747064
<class 'numpy.float64'>
39.8242290216
<class 'numpy.float64'>
47.7906505334
<class 'numpy.float64'>
39.8236598476
<class 'numpy.float64'>
47.7903319064
<class 'numpy.float64'>
39.8230869103
<class 'numpy.float64'>
47.7900186164
<class 'numpy.float64'>
Cost after iteration 8000: 47.790019
39.8225103289
<class 'numpy.float64'>
47.7897104691
<class 'numpy.float64'>
39.82193022
<class 'numpy.float64'>
47.7894072834
<class 'numpy.float64'>
39.8213466979
<class 'numpy.float64'>
47.7891088901
<class 'numpy.float64'>
39.8207598745
<class 'numpy.float64'>
47.7888151302
<class 'numpy.float64'>
39.8201698588
<class 'numpy.float64'>
47.788525854
<class 'numpy.float64'>
39.8195767571
<class 'numpy.float64'>
47.7882409206
<class 'numpy.float64'>
39.8189806729
<class 'numpy.float64'>
47.7879601963
<class 'numpy.float64'>
39.8183817068
<class 'numpy.float64'>
47.7876835545
<class 'numpy.float64'>
39.8177799567
<class 'numpy.float64'>
47.7874108749
<class 'numpy.float64'>
39.8171755174
<class 'numpy.float64'>
47.7871420431
<class 'numpy.float64'>
39.8165684811
<class 'numpy.float64'>
47.7868769498
<class 'numpy.float64'>
39.8159589368
<class 'numpy.float64'>
47.7866154908
<class 'numpy.float64'>
39.815346971
<class 'numpy.float64'>
47.7863575664
<class 'numpy.float64'>
39.8147326674
<class 'numpy.float64'>
47.7861030811
<class 'numpy.float64'>
39.8141161069
<class 'numpy.float64'>
47.7858519434
<class 'numpy.float64'>
39.8134973679
<class 'numpy.float64'>
47.7856040655
<class 'numpy.float64'>
39.812876526
<class 'numpy.float64'>
47.7853593631
<class 'numpy.float64'>
39.8122536544
<class 'numpy.float64'>
47.785117755
<class 'numpy.float64'>
39.8116288239
<class 'numpy.float64'>
47.7848791633
<class 'numpy.float64'>
39.8110021027
<class 'numpy.float64'>
47.7846435129
<class 'numpy.float64'>
39.8103735569
<class 'numpy.float64'>
47.7844107313
<class 'numpy.float64'>
39.8097432501
<class 'numpy.float64'>
47.784180749
<class 'numpy.float64'>
39.8091112441
<class 'numpy.float64'>
47.7839534985
<class 'numpy.float64'>
39.8084775981
<class 'numpy.float64'>
47.7837289151
<class 'numpy.float64'>
39.8078423695
<class 'numpy.float64'>
47.7835069359
<class 'numpy.float64'>
39.8072056138
<class 'numpy.float64'>
47.7832875005
<class 'numpy.float64'>
39.8065673844
<class 'numpy.float64'>
47.7830705504
<class 'numpy.float64'>
39.805927733
<class 'numpy.float64'>
47.782856029
<class 'numpy.float64'>
39.8052867093
<class 'numpy.float64'>
47.7826438816
<class 'numpy.float64'>
39.8046443615
<class 'numpy.float64'>
47.7824340554
<class 'numpy.float64'>
39.804000736
<class 'numpy.float64'>
47.7822264992
<class 'numpy.float64'>
39.8033558776
<class 'numpy.float64'>
47.7820211633
<class 'numpy.float64'>
39.8027098297
<class 'numpy.float64'>
47.7818179999
<class 'numpy.float64'>
39.8020626339
<class 'numpy.float64'>
47.7816169625
<class 'numpy.float64'>
39.8014143308
<class 'numpy.float64'>
47.7814180061
<class 'numpy.float64'>
39.8007649591
<class 'numpy.float64'>
47.781221087
<class 'numpy.float64'>
39.8001145567
<class 'numpy.float64'>
47.7810261629
<class 'numpy.float64'>
39.7994631597
<class 'numpy.float64'>
47.7808331929
<class 'numpy.float64'>
39.7988108033
<class 'numpy.float64'>
47.7806421372
<class 'numpy.float64'>
39.7981575214
<class 'numpy.float64'>
47.7804529572
<class 'numpy.float64'>
39.7975033468
<class 'numpy.float64'>
47.7802656156
<class 'numpy.float64'>
39.7968483111
<class 'numpy.float64'>
47.7800800759
<class 'numpy.float64'>
39.7961924449
<class 'numpy.float64'>
47.7798963029
<class 'numpy.float64'>
39.7955357777
<class 'numpy.float64'>
47.7797142624
<class 'numpy.float64'>
39.7948783382
<class 'numpy.float64'>
47.7795339212
<class 'numpy.float64'>
39.794220154
<class 'numpy.float64'>
47.7793552469
<class 'numpy.float64'>
39.7935612518
<class 'numpy.float64'>
47.7791782082
<class 'numpy.float64'>
39.7929016573
<class 'numpy.float64'>
47.7790027745
<class 'numpy.float64'>
39.7922413957
<class 'numpy.float64'>
47.7788289162
<class 'numpy.float64'>
39.791580491
<class 'numpy.float64'>
47.7786566046
<class 'numpy.float64'>
39.7909189666
<class 'numpy.float64'>
47.7784858114
<class 'numpy.float64'>
39.7902568452
<class 'numpy.float64'>
47.7783165096
<class 'numpy.float64'>
39.7895941486
<class 'numpy.float64'>
47.7781486725
<class 'numpy.float64'>
39.788930898
<class 'numpy.float64'>
47.7779822743
<class 'numpy.float64'>
39.788267114
<class 'numpy.float64'>
47.7778172898
<class 'numpy.float64'>
39.7876028163
<class 'numpy.float64'>
47.7776536947
<class 'numpy.float64'>
39.7869380243
<class 'numpy.float64'>
47.777491465
<class 'numpy.float64'>
39.7862727566
<class 'numpy.float64'>
47.7773305776
<class 'numpy.float64'>
39.7856070311
<class 'numpy.float64'>
47.7771710097
<class 'numpy.float64'>
39.7849408655
<class 'numpy.float64'>
47.7770127394
<class 'numpy.float64'>
39.7842742766
<class 'numpy.float64'>
47.7768557452
<class 'numpy.float64'>
39.7836072808
<class 'numpy.float64'>
47.7767000061
<class 'numpy.float64'>
39.7829398942
<class 'numpy.float64'>
47.7765455017
<class 'numpy.float64'>
39.7822721322
<class 'numpy.float64'>
47.7763922119
<class 'numpy.float64'>
39.7816040096
<class 'numpy.float64'>
47.7762401175
<class 'numpy.float64'>
39.7809355412
<class 'numpy.float64'>
47.7760891994
<class 'numpy.float64'>
39.7802667408
<class 'numpy.float64'>
47.775939439
<class 'numpy.float64'>
39.7795976224
<class 'numpy.float64'>
47.7757908183
<class 'numpy.float64'>
39.778928199
<class 'numpy.float64'>
47.7756433196
<class 'numpy.float64'>
39.7782584837
<class 'numpy.float64'>
47.7754969257
<class 'numpy.float64'>
39.7775884889
<class 'numpy.float64'>
47.7753516196
<class 'numpy.float64'>
39.7769182268
<class 'numpy.float64'>
47.7752073849
<class 'numpy.float64'>
39.7762477093
<class 'numpy.float64'>
47.7750642055
<class 'numpy.float64'>
39.7755769477
<class 'numpy.float64'>
47.7749220656
<class 'numpy.float64'>
39.7749059533
<class 'numpy.float64'>
47.7747809499
<class 'numpy.float64'>
39.7742347369
<class 'numpy.float64'>
47.7746408431
<class 'numpy.float64'>
39.7735633091
<class 'numpy.float64'>
47.7745017307
<class 'numpy.float64'>
39.77289168
<class 'numpy.float64'>
47.7743635982
<class 'numpy.float64'>
39.7722198597
<class 'numpy.float64'>
47.7742264314
<class 'numpy.float64'>
39.7715478579
<class 'numpy.float64'>
47.7740902165
<class 'numpy.float64'>
39.7708756841
<class 'numpy.float64'>
47.77395494
<class 'numpy.float64'>
39.7702033473
<class 'numpy.float64'>
47.7738205885
<class 'numpy.float64'>
39.7695308566
<class 'numpy.float64'>
47.7736871492
<class 'numpy.float64'>
39.7688582207
<class 'numpy.float64'>
47.7735546093
<class 'numpy.float64'>
39.7681854481
<class 'numpy.float64'>
47.7734229562
<class 'numpy.float64'>
39.767512547
<class 'numpy.float64'>
47.7732921777
<class 'numpy.float64'>
39.7668395256
<class 'numpy.float64'>
47.7731622619
<class 'numpy.float64'>
39.7661663915
<class 'numpy.float64'>
47.7730331969
<class 'numpy.float64'>
39.7654931526
<class 'numpy.float64'>
47.7729049713
<class 'numpy.float64'>
39.7648198163
<class 'numpy.float64'>
47.7727775735
<class 'numpy.float64'>
39.7641463899
<class 'numpy.float64'>
47.7726509927
<class 'numpy.float64'>
39.7634728804
<class 'numpy.float64'>
47.7725252177
<class 'numpy.float64'>
39.7627992948
<class 'numpy.float64'>
47.772400238
<class 'numpy.float64'>
39.7621256399
<class 'numpy.float64'>
47.7722760429
<class 'numpy.float64'>
39.7614519222
<class 'numpy.float64'>
47.7721526222
<class 'numpy.float64'>
39.7607781483
<class 'numpy.float64'>
47.7720299656
<class 'numpy.float64'>
39.7601043243
<class 'numpy.float64'>
47.7719080632
<class 'numpy.float64'>
39.7594304566
<class 'numpy.float64'>
47.7717869052
<class 'numpy.float64'>
39.758756551
<class 'numpy.float64'>
47.7716664819
<class 'numpy.float64'>
39.7580826134
<class 'numpy.float64'>
47.7715467838
<class 'numpy.float64'>
39.7574086497
<class 'numpy.float64'>
47.7714278017
<class 'numpy.float64'>
39.7567346654
<class 'numpy.float64'>
47.7713095262
<class 'numpy.float64'>
39.7560606659
<class 'numpy.float64'>
47.7711919485
<class 'numpy.float64'>
39.7553866568
<class 'numpy.float64'>
47.7710750596
<class 'numpy.float64'>
39.7547126431
<class 'numpy.float64'>
47.7709588508
<class 'numpy.float64'>
39.7540386301
<class 'numpy.float64'>
47.7708433135
<class 'numpy.float64'>
39.7533646228
<class 'numpy.float64'>
47.7707284392
<class 'numpy.float64'>
39.7526906261
<class 'numpy.float64'>
47.7706142197
<class 'numpy.float64'>
39.7520166449
<class 'numpy.float64'>
47.7705006465
<class 'numpy.float64'>
39.7513426838
<class 'numpy.float64'>
47.7703877118
<class 'numpy.float64'>
39.7506687475
<class 'numpy.float64'>
47.7702754075
<class 'numpy.float64'>
39.7499948406
<class 'numpy.float64'>
47.7701637258
<class 'numpy.float64'>
39.7493209674
<class 'numpy.float64'>
47.770052659
<class 'numpy.float64'>
39.7486471323
<class 'numpy.float64'>
47.7699421993
<class 'numpy.float64'>
39.7479733397
<class 'numpy.float64'>
47.7698323394
<class 'numpy.float64'>
39.7472995936
<class 'numpy.float64'>
47.7697230718
<class 'numpy.float64'>
39.7466258981
<class 'numpy.float64'>
47.7696143892
<class 'numpy.float64'>
39.7459522574
<class 'numpy.float64'>
47.7695062844
<class 'numpy.float64'>
39.7452786754
<class 'numpy.float64'>
47.7693987504
<class 'numpy.float64'>
39.7446051559
<class 'numpy.float64'>
47.76929178
<class 'numpy.float64'>
39.7439317027
<class 'numpy.float64'>
47.7691853665
<class 'numpy.float64'>
39.7432583196
<class 'numpy.float64'>
47.7690795029
<class 'numpy.float64'>
39.7425850103
<class 'numpy.float64'>
47.7689741826
<class 'numpy.float64'>
39.7419117783
<class 'numpy.float64'>
47.7688693989
<class 'numpy.float64'>
39.7412386272
<class 'numpy.float64'>
47.7687651453
<class 'numpy.float64'>
39.7405655605
<class 'numpy.float64'>
47.7686614153
<class 'numpy.float64'>
39.7398925815
<class 'numpy.float64'>
47.7685582025
<class 'numpy.float64'>
39.7392196937
<class 'numpy.float64'>
47.7684555007
<class 'numpy.float64'>
39.7385469003
<class 'numpy.float64'>
47.7683533035
<class 'numpy.float64'>
39.7378742046
<class 'numpy.float64'>
47.7682516049
<class 'numpy.float64'>
39.7372016097
<class 'numpy.float64'>
47.7681503988
<class 'numpy.float64'>
39.7365291189
<class 'numpy.float64'>
47.7680496792
<class 'numpy.float64'>
39.7358567351
<class 'numpy.float64'>
47.7679494401
<class 'numpy.float64'>
39.7351844614
<class 'numpy.float64'>
47.7678496758
<class 'numpy.float64'>
39.7345123008
<class 'numpy.float64'>
47.7677503804
<class 'numpy.float64'>
39.7338402562
<class 'numpy.float64'>
47.7676515482
<class 'numpy.float64'>
39.7331683305
<class 'numpy.float64'>
47.7675531736
<class 'numpy.float64'>
39.7324965266
<class 'numpy.float64'>
47.767455251
<class 'numpy.float64'>
39.7318248473
<class 'numpy.float64'>
47.7673577749
<class 'numpy.float64'>
39.7311532952
<class 'numpy.float64'>
47.7672607398
<class 'numpy.float64'>
39.7304818732
<class 'numpy.float64'>
47.7671641404
<class 'numpy.float64'>
39.7298105838
<class 'numpy.float64'>
47.7670679713
<class 'numpy.float64'>
39.7291394298
<class 'numpy.float64'>
47.7669722272
<class 'numpy.float64'>
39.7284684136
<class 'numpy.float64'>
47.766876903
<class 'numpy.float64'>
39.727797538
<class 'numpy.float64'>
47.7667819935
<class 'numpy.float64'>
39.7271268053
<class 'numpy.float64'>
47.7666874936
<class 'numpy.float64'>
39.726456218
<class 'numpy.float64'>
47.7665933982
<class 'numpy.float64'>
39.7257857786
<class 'numpy.float64'>
47.7664997025
<class 'numpy.float64'>
39.7251154896
<class 'numpy.float64'>
47.7664064014
<class 'numpy.float64'>
39.7244453532
<class 'numpy.float64'>
47.7663134901
<class 'numpy.float64'>
39.7237753718
<class 'numpy.float64'>
47.7662209638
<class 'numpy.float64'>
39.7231055477
<class 'numpy.float64'>
47.7661288176
<class 'numpy.float64'>
39.7224358831
<class 'numpy.float64'>
47.766037047
<class 'numpy.float64'>
39.7217663804
<class 'numpy.float64'>
47.7659456471
<class 'numpy.float64'>
39.7210970417
<class 'numpy.float64'>
47.7658546134
<class 'numpy.float64'>
39.7204278692
<class 'numpy.float64'>
47.7657639414
<class 'numpy.float64'>
39.7197588651
<class 'numpy.float64'>
47.7656736264
<class 'numpy.float64'>
39.7190900314
<class 'numpy.float64'>
47.7655836641
<class 'numpy.float64'>
39.7184213703
<class 'numpy.float64'>
47.7654940499
<class 'numpy.float64'>
39.7177528838
<class 'numpy.float64'>
47.7654047795
<class 'numpy.float64'>
39.717084574
<class 'numpy.float64'>
47.7653158486
<class 'numpy.float64'>
39.7164164428
<class 'numpy.float64'>
47.7652272528
<class 'numpy.float64'>
39.7157484923
<class 'numpy.float64'>
47.765138988
<class 'numpy.float64'>
39.7150807244
<class 'numpy.float64'>
47.7650510498
<class 'numpy.float64'>
39.714413141
<class 'numpy.float64'>
47.7649634342
<class 'numpy.float64'>
39.713745744
<class 'numpy.float64'>
47.7648761369
<class 'numpy.float64'>
39.7130785354
<class 'numpy.float64'>
47.764789154
<class 'numpy.float64'>
39.7124115169
<class 'numpy.float64'>
47.7647024813
<class 'numpy.float64'>
39.7117446904
<class 'numpy.float64'>
47.7646161149
<class 'numpy.float64'>
39.7110780577
<class 'numpy.float64'>
47.7645300507
<class 'numpy.float64'>
39.7104116207
<class 'numpy.float64'>
47.764444285
<class 'numpy.float64'>
39.7097453809
<class 'numpy.float64'>
47.7643588137
<class 'numpy.float64'>
39.7090793403
<class 'numpy.float64'>
47.7642736331
<class 'numpy.float64'>
39.7084135005
<class 'numpy.float64'>
47.7641887392
<class 'numpy.float64'>
39.7077478632
<class 'numpy.float64'>
47.7641041284
<class 'numpy.float64'>
39.7070824301
<class 'numpy.float64'>
47.7640197969
<class 'numpy.float64'>
39.7064172029
<class 'numpy.float64'>
47.763935741
<class 'numpy.float64'>
39.7057521831
<class 'numpy.float64'>
47.763851957
<class 'numpy.float64'>
39.7050873724
<class 'numpy.float64'>
47.7637684413
<class 'numpy.float64'>
39.7044227724
<class 'numpy.float64'>
47.7636851903
<class 'numpy.float64'>
39.7037583847
<class 'numpy.float64'>
47.7636022004
<class 'numpy.float64'>
39.7030942108
<class 'numpy.float64'>
47.7635194682
<class 'numpy.float64'>
39.7024302523
<class 'numpy.float64'>
47.76343699
<class 'numpy.float64'>
39.7017665107
<class 'numpy.float64'>
47.7633547624
<class 'numpy.float64'>
39.7011029874
<class 'numpy.float64'>
47.7632727821
<class 'numpy.float64'>
39.700439684
<class 'numpy.float64'>
47.7631910456
<class 'numpy.float64'>
39.6997766019
<class 'numpy.float64'>
47.7631095495
<class 'numpy.float64'>
39.6991137426
<class 'numpy.float64'>
47.7630282905
<class 'numpy.float64'>
39.6984511075
<class 'numpy.float64'>
47.7629472653
<class 'numpy.float64'>
39.6977886981
<class 'numpy.float64'>
47.7628664707
<class 'numpy.float64'>
39.6971265156
<class 'numpy.float64'>
47.7627859033
<class 'numpy.float64'>
39.6964645615
<class 'numpy.float64'>
47.76270556
<class 'numpy.float64'>
39.6958028372
<class 'numpy.float64'>
47.7626254376
<class 'numpy.float64'>
39.695141344
<class 'numpy.float64'>
47.7625455329
<class 'numpy.float64'>
39.6944800833
<class 'numpy.float64'>
47.7624658428
<class 'numpy.float64'>
39.6938190563
<class 'numpy.float64'>
47.7623863643
<class 'numpy.float64'>
39.6931582643
<class 'numpy.float64'>
47.7623070941
<class 'numpy.float64'>
39.6924977087
<class 'numpy.float64'>
47.7622280294
<class 'numpy.float64'>
39.6918373906
<class 'numpy.float64'>
47.7621491671
<class 'numpy.float64'>
39.6911773115
<class 'numpy.float64'>
47.7620705043
<class 'numpy.float64'>
39.6905174724
<class 'numpy.float64'>
47.7619920378
<class 'numpy.float64'>
39.6898578747
<class 'numpy.float64'>
47.7619137649
<class 'numpy.float64'>
39.6891985195
<class 'numpy.float64'>
47.7618356827
<class 'numpy.float64'>
39.688539408
<class 'numpy.float64'>
47.7617577881
<class 'numpy.float64'>
39.6878805414
<class 'numpy.float64'>
47.7616800785
<class 'numpy.float64'>
39.687221921
<class 'numpy.float64'>
47.761602551
<class 'numpy.float64'>
39.6865635477
<class 'numpy.float64'>
47.7615252028
<class 'numpy.float64'>
39.6859054229
<class 'numpy.float64'>
47.7614480311
<class 'numpy.float64'>
39.6852475475
<class 'numpy.float64'>
47.7613710331
<class 'numpy.float64'>
39.6845899228
<class 'numpy.float64'>
47.7612942062
<class 'numpy.float64'>
39.6839325498
<class 'numpy.float64'>
47.7612175477
<class 'numpy.float64'>
39.6832754296
<class 'numpy.float64'>
47.7611410548
<class 'numpy.float64'>
39.6826185632
<class 'numpy.float64'>
47.7610647249
<class 'numpy.float64'>
39.6819619519
<class 'numpy.float64'>
47.7609885555
<class 'numpy.float64'>
39.6813055965
<class 'numpy.float64'>
47.7609125439
<class 'numpy.float64'>
39.6806494982
<class 'numpy.float64'>
47.7608366875
<class 'numpy.float64'>
39.679993658
<class 'numpy.float64'>
47.7607609838
<class 'numpy.float64'>
39.6793380768
<class 'numpy.float64'>
47.7606854302
<class 'numpy.float64'>
39.6786827558
<class 'numpy.float64'>
47.7606100243
<class 'numpy.float64'>
39.6780276958
<class 'numpy.float64'>
47.7605347636
<class 'numpy.float64'>
39.6773728978
<class 'numpy.float64'>
47.7604596455
<class 'numpy.float64'>
39.6767183629
<class 'numpy.float64'>
47.7603846677
<class 'numpy.float64'>
39.676064092
<class 'numpy.float64'>
47.7603098278
<class 'numpy.float64'>
39.675410086
<class 'numpy.float64'>
47.7602351232
<class 'numpy.float64'>
39.6747563459
<class 'numpy.float64'>
47.7601605518
<class 'numpy.float64'>
39.6741028725
<class 'numpy.float64'>
47.7600861111
<class 'numpy.float64'>
39.6734496668
<class 'numpy.float64'>
47.7600117988
<class 'numpy.float64'>
39.6727967298
<class 'numpy.float64'>
47.7599376125
<class 'numpy.float64'>
39.6721440622
<class 'numpy.float64'>
47.75986355
<class 'numpy.float64'>
39.671491665
<class 'numpy.float64'>
47.7597896091
<class 'numpy.float64'>
39.670839539
<class 'numpy.float64'>
47.7597157874
<class 'numpy.float64'>
39.6701876851
<class 'numpy.float64'>
47.7596420828
<class 'numpy.float64'>
39.6695361042
<class 'numpy.float64'>
47.7595684931
<class 'numpy.float64'>
39.668884797
<class 'numpy.float64'>
47.7594950159
<class 'numpy.float64'>
39.6682337645
<class 'numpy.float64'>
47.7594216493
<class 'numpy.float64'>
39.6675830074
<class 'numpy.float64'>
47.759348391
<class 'numpy.float64'>
39.6669325265
<class 'numpy.float64'>
47.7592752389
<class 'numpy.float64'>
39.6662823226
<class 'numpy.float64'>
47.7592021909
<class 'numpy.float64'>
39.6656323966
<class 'numpy.float64'>
47.7591292449
<class 'numpy.float64'>
39.6649827492
<class 'numpy.float64'>
47.7590563989
<class 'numpy.float64'>
39.6643333811
<class 'numpy.float64'>
47.7589836507
<class 'numpy.float64'>
39.6636842932
<class 'numpy.float64'>
47.7589109984
<class 'numpy.float64'>
39.6630354862
<class 'numpy.float64'>
47.7588384399
<class 'numpy.float64'>
39.6623869608
<class 'numpy.float64'>
47.7587659733
<class 'numpy.float64'>
39.6617387178
<class 'numpy.float64'>
47.7586935965
<class 'numpy.float64'>
39.6610907579
<class 'numpy.float64'>
47.7586213076
<class 'numpy.float64'>
39.6604430817
<class 'numpy.float64'>
47.7585491047
<class 'numpy.float64'>
39.6597956901
<class 'numpy.float64'>
47.7584769858
<class 'numpy.float64'>
39.6591485836
<class 'numpy.float64'>
47.758404949
<class 'numpy.float64'>
39.6585017631
<class 'numpy.float64'>
47.7583329925
<class 'numpy.float64'>
39.6578552292
<class 'numpy.float64'>
47.7582611143
<class 'numpy.float64'>
39.6572089824
<class 'numpy.float64'>
47.7581893126
<class 'numpy.float64'>
39.6565630236
<class 'numpy.float64'>
47.7581175856
<class 'numpy.float64'>
39.6559173534
<class 'numpy.float64'>
47.7580459314
<class 'numpy.float64'>
39.6552719724
<class 'numpy.float64'>
47.7579743482
<class 'numpy.float64'>
39.6546268812
<class 'numpy.float64'>
47.7579028343
<class 'numpy.float64'>
39.6539820805
<class 'numpy.float64'>
47.7578313878
<class 'numpy.float64'>
39.6533375709
<class 'numpy.float64'>
47.7577600071
<class 'numpy.float64'>
39.652693353
<class 'numpy.float64'>
47.7576886903
<class 'numpy.float64'>
39.6520494274
<class 'numpy.float64'>
47.7576174357
<class 'numpy.float64'>
39.6514057948
<class 'numpy.float64'>
47.7575462417
<class 'numpy.float64'>
39.6507624556
<class 'numpy.float64'>
47.7574751065
<class 'numpy.float64'>
39.6501194106
<class 'numpy.float64'>
47.7574040285
<class 'numpy.float64'>
39.6494766602
<class 'numpy.float64'>
47.757333006
<class 'numpy.float64'>
39.648834205
<class 'numpy.float64'>
47.7572620373
<class 'numpy.float64'>
39.6481920457
<class 'numpy.float64'>
47.7571911209
<class 'numpy.float64'>
39.6475501827
<class 'numpy.float64'>
47.757120255
<class 'numpy.float64'>
39.6469086166
<class 'numpy.float64'>
47.7570494381
<class 'numpy.float64'>
39.6462673479
<class 'numpy.float64'>
47.7569786687
<class 'numpy.float64'>
39.6456263772
<class 'numpy.float64'>
47.756907945
<class 'numpy.float64'>
39.644985705
<class 'numpy.float64'>
47.7568372657
<class 'numpy.float64'>
39.6443453318
<class 'numpy.float64'>
47.756766629
<class 'numpy.float64'>
39.6437052581
<class 'numpy.float64'>
47.7566960336
<class 'numpy.float64'>
39.6430654844
<class 'numpy.float64'>
47.7566254778
<class 'numpy.float64'>
39.6424260113
<class 'numpy.float64'>
47.7565549602
<class 'numpy.float64'>
39.6417868391
<class 'numpy.float64'>
47.7564844793
<class 'numpy.float64'>
39.6411479684
<class 'numpy.float64'>
47.7564140336
<class 'numpy.float64'>
39.6405093997
<class 'numpy.float64'>
47.7563436217
<class 'numpy.float64'>
39.6398711334
<class 'numpy.float64'>
47.756273242
<class 'numpy.float64'>
39.63923317
<class 'numpy.float64'>
47.7562028933
<class 'numpy.float64'>
39.63859551
<class 'numpy.float64'>
47.756132574
<class 'numpy.float64'>
39.6379581537
<class 'numpy.float64'>
47.7560622827
<class 'numpy.float64'>
39.6373211017
<class 'numpy.float64'>
47.7559920181
<class 'numpy.float64'>
39.6366843544
<class 'numpy.float64'>
47.7559217787
<class 'numpy.float64'>
39.6360479122
<class 'numpy.float64'>
47.7558515632
<class 'numpy.float64'>
39.6354117755
<class 'numpy.float64'>
47.7557813703
<class 'numpy.float64'>
39.6347759447
<class 'numpy.float64'>
47.7557111986
<class 'numpy.float64'>
39.6341404203
<class 'numpy.float64'>
47.7556410467
<class 'numpy.float64'>
39.6335052027
<class 'numpy.float64'>
47.7555709134
<class 'numpy.float64'>
39.6328702923
<class 'numpy.float64'>
47.7555007973
<class 'numpy.float64'>
39.6322356894
<class 'numpy.float64'>
47.7554306972
<class 'numpy.float64'>
39.6316013944
<class 'numpy.float64'>
47.7553606117
<class 'numpy.float64'>
39.6309674078
<class 'numpy.float64'>
47.7552905397
<class 'numpy.float64'>
39.6303337299
<class 'numpy.float64'>
47.7552204797
<class 'numpy.float64'>
39.629700361
<class 'numpy.float64'>
47.7551504307
<class 'numpy.float64'>
39.6290673016
<class 'numpy.float64'>
47.7550803913
<class 'numpy.float64'>
39.628434552
<class 'numpy.float64'>
47.7550103604
<class 'numpy.float64'>
39.6278021126
<class 'numpy.float64'>
47.7549403367
<class 'numpy.float64'>
39.6271699836
<class 'numpy.float64'>
47.754870319
<class 'numpy.float64'>
39.6265381655
<class 'numpy.float64'>
47.7548003062
<class 'numpy.float64'>
39.6259066586
<class 'numpy.float64'>
47.754730297
<class 'numpy.float64'>
39.6252754632
<class 'numpy.float64'>
47.7546602903
<class 'numpy.float64'>
39.6246445796
<class 'numpy.float64'>
47.754590285
<class 'numpy.float64'>
39.6240140082
<class 'numpy.float64'>
47.7545202798
<class 'numpy.float64'>
39.6233837492
<class 'numpy.float64'>
47.7544502738
<class 'numpy.float64'>
39.622753803
<class 'numpy.float64'>
47.7543802656
<class 'numpy.float64'>
39.62212417
<class 'numpy.float64'>
47.7543102543
<class 'numpy.float64'>
39.6214948503
<class 'numpy.float64'>
47.7542402388
<class 'numpy.float64'>
39.6208658442
<class 'numpy.float64'>
47.7541702179
<class 'numpy.float64'>
39.6202371522
<class 'numpy.float64'>
47.7541001905
<class 'numpy.float64'>
39.6196087744
<class 'numpy.float64'>
47.7540301557
<class 'numpy.float64'>
39.6189807112
<class 'numpy.float64'>
47.7539601123
<class 'numpy.float64'>
39.6183529627
<class 'numpy.float64'>
47.7538900593
<class 'numpy.float64'>
39.6177255293
<class 'numpy.float64'>
47.7538199956
<class 'numpy.float64'>
39.6170984113
<class 'numpy.float64'>
47.7537499203
<class 'numpy.float64'>
39.6164716089
<class 'numpy.float64'>
47.7536798322
<class 'numpy.float64'>
39.6158451224
<class 'numpy.float64'>
47.7536097305
<class 'numpy.float64'>
39.6152189519
<class 'numpy.float64'>
47.753539614
<class 'numpy.float64'>
39.6145930979
<class 'numpy.float64'>
47.7534694819
<class 'numpy.float64'>
39.6139675604
<class 'numpy.float64'>
47.7533993331
<class 'numpy.float64'>
39.6133423398
<class 'numpy.float64'>
47.7533291667
<class 'numpy.float64'>
39.6127174362
<class 'numpy.float64'>
47.7532589817
<class 'numpy.float64'>
39.6120928499
<class 'numpy.float64'>
47.7531887771
<class 'numpy.float64'>
39.6114685812
<class 'numpy.float64'>
47.753118552
<class 'numpy.float64'>
39.6108446302
<class 'numpy.float64'>
47.7530483055
<class 'numpy.float64'>
39.6102209971
<class 'numpy.float64'>
47.7529780367
<class 'numpy.float64'>
39.6095976823
<class 'numpy.float64'>
47.7529077446
<class 'numpy.float64'>
39.6089746858
<class 'numpy.float64'>
47.7528374283
<class 'numpy.float64'>
39.6083520078
<class 'numpy.float64'>
47.752767087
<class 'numpy.float64'>
39.6077296487
<class 'numpy.float64'>
47.7526967197
<class 'numpy.float64'>
39.6071076085
<class 'numpy.float64'>
47.7526263256
<class 'numpy.float64'>
39.6064858874
<class 'numpy.float64'>
47.7525559038
<class 'numpy.float64'>
39.6058644857
<class 'numpy.float64'>
47.7524854535
<class 'numpy.float64'>
39.6052434036
<class 'numpy.float64'>
47.7524149737
<class 'numpy.float64'>
39.6046226411
<class 'numpy.float64'>
47.7523444637
<class 'numpy.float64'>
39.6040021985
<class 'numpy.float64'>
47.7522739226
<class 'numpy.float64'>
39.6033820759
<class 'numpy.float64'>
47.7522033496
<class 'numpy.float64'>
39.6027622736
<class 'numpy.float64'>
47.7521327439
<class 'numpy.float64'>
39.6021427916
<class 'numpy.float64'>
47.7520621046
<class 'numpy.float64'>
39.6015236301
<class 'numpy.float64'>
47.751991431
<class 'numpy.float64'>
39.6009047893
<class 'numpy.float64'>
47.7519207222
<class 'numpy.float64'>
39.6002862694
<class 'numpy.float64'>
47.7518499775
<class 'numpy.float64'>
39.5996680704
<class 'numpy.float64'>
47.7517791961
<class 'numpy.float64'>
39.5990501925
<class 'numpy.float64'>
47.7517083772
<class 'numpy.float64'>
39.5984326359
<class 'numpy.float64'>
47.7516375201
<class 'numpy.float64'>
39.5978154007
<class 'numpy.float64'>
47.751566624
<class 'numpy.float64'>
39.597198487
<class 'numpy.float64'>
47.7514956882
<class 'numpy.float64'>
39.596581895
<class 'numpy.float64'>
47.7514247119
<class 'numpy.float64'>
39.5959656247
<class 'numpy.float64'>
47.7513536944
<class 'numpy.float64'>
39.5953496763
<class 'numpy.float64'>
47.7512826349
<class 'numpy.float64'>
39.59473405
<class 'numpy.float64'>
47.7512115328
<class 'numpy.float64'>
39.5941187457
<class 'numpy.float64'>
47.7511403874
<class 'numpy.float64'>
39.5935037637
<class 'numpy.float64'>
47.7510691979
<class 'numpy.float64'>
39.5928891041
<class 'numpy.float64'>
47.7509979636
<class 'numpy.float64'>
39.5922747669
<class 'numpy.float64'>
47.7509266839
<class 'numpy.float64'>
39.5916607522
<class 'numpy.float64'>
47.750855358
<class 'numpy.float64'>
39.5910470602
<class 'numpy.float64'>
47.7507839854
<class 'numpy.float64'>
39.5904336909
<class 'numpy.float64'>
47.7507125653
<class 'numpy.float64'>
39.5898206444
<class 'numpy.float64'>
47.7506410971
<class 'numpy.float64'>
39.5892079209
<class 'numpy.float64'>
47.7505695801
<class 'numpy.float64'>
39.5885955203
<class 'numpy.float64'>
47.7504980137
<class 'numpy.float64'>
39.5879834429
<class 'numpy.float64'>
47.7504263973
<class 'numpy.float64'>
39.5873716886
<class 'numpy.float64'>
47.7503547301
<class 'numpy.float64'>
39.5867602575
<class 'numpy.float64'>
47.7502830116
<class 'numpy.float64'>
39.5861491497
<class 'numpy.float64'>
47.7502112413
<class 'numpy.float64'>
39.5855383653
<class 'numpy.float64'>
47.7501394183
<class 'numpy.float64'>
39.5849279043
<class 'numpy.float64'>
47.7500675422
<class 'numpy.float64'>
39.5843177669
<class 'numpy.float64'>
47.7499956124
<class 'numpy.float64'>
39.583707953
<class 'numpy.float64'>
47.7499236282
<class 'numpy.float64'>
39.5830984627
<class 'numpy.float64'>
47.7498515891
<class 'numpy.float64'>
39.582489296
<class 'numpy.float64'>
47.7497794945
<class 'numpy.float64'>
39.5818804531
<class 'numpy.float64'>
47.7497073438
<class 'numpy.float64'>
39.581271934
<class 'numpy.float64'>
47.7496351365
<class 'numpy.float64'>
39.5806637386
<class 'numpy.float64'>
47.7495628719
<class 'numpy.float64'>
39.5800558671
<class 'numpy.float64'>
47.7494905496
<class 'numpy.float64'>
39.5794483195
<class 'numpy.float64'>
47.749418169
<class 'numpy.float64'>
39.5788410959
<class 'numpy.float64'>
47.7493457295
<class 'numpy.float64'>
39.5782341961
<class 'numpy.float64'>
47.7492732306
<class 'numpy.float64'>
39.5776276204
<class 'numpy.float64'>
47.7492006718
<class 'numpy.float64'>
39.5770213687
<class 'numpy.float64'>
47.7491280526
<class 'numpy.float64'>
39.5764154411
<class 'numpy.float64'>
47.7490553724
<class 'numpy.float64'>
39.5758098375
<class 'numpy.float64'>
47.7489826307
<class 'numpy.float64'>
39.575204558
<class 'numpy.float64'>
47.748909827
<class 'numpy.float64'>
39.5745996026
<class 'numpy.float64'>
47.7488369608
<class 'numpy.float64'>
39.5739949714
<class 'numpy.float64'>
47.7487640317
<class 'numpy.float64'>
39.5733906642
<class 'numpy.float64'>
47.748691039
<class 'numpy.float64'>
39.5727866813
<class 'numpy.float64'>
47.7486179824
<class 'numpy.float64'>
39.5721830224
<class 'numpy.float64'>
47.7485448614
<class 'numpy.float64'>
39.5715796878
<class 'numpy.float64'>
47.7484716754
<class 'numpy.float64'>
39.5709766773
<class 'numpy.float64'>
47.7483984241
<class 'numpy.float64'>
39.5703739909
<class 'numpy.float64'>
47.7483251069
<class 'numpy.float64'>
39.5697716287
<class 'numpy.float64'>
47.7482517233
<class 'numpy.float64'>
39.5691695907
<class 'numpy.float64'>
47.7481782731
<class 'numpy.float64'>
39.5685678768
<class 'numpy.float64'>
47.7481047556
<class 'numpy.float64'>
39.5679664871
<class 'numpy.float64'>
47.7480311704
<class 'numpy.float64'>
39.5673654214
<class 'numpy.float64'>
47.7479575172
<class 'numpy.float64'>
39.5667646799
<class 'numpy.float64'>
47.7478837954
<class 'numpy.float64'>
39.5661642625
<class 'numpy.float64'>
47.7478100047
<class 'numpy.float64'>
39.5655641692
<class 'numpy.float64'>
47.7477361446
<class 'numpy.float64'>
39.5649643999
<class 'numpy.float64'>
47.7476622147
<class 'numpy.float64'>
39.5643649547
<class 'numpy.float64'>
47.7475882147
<class 'numpy.float64'>
39.5637658335
<class 'numpy.float64'>
47.747514144
<class 'numpy.float64'>
39.5631670362
<class 'numpy.float64'>
47.7474400022
<class 'numpy.float64'>
39.5625685629
<class 'numpy.float64'>
47.7473657891
<class 'numpy.float64'>
39.5619704135
<class 'numpy.float64'>
47.7472915042
<class 'numpy.float64'>
39.5613725881
<class 'numpy.float64'>
47.747217147
<class 'numpy.float64'>
39.5607750864
<class 'numpy.float64'>
47.7471427173
<class 'numpy.float64'>
39.5601779086
<class 'numpy.float64'>
47.7470682147
<class 'numpy.float64'>
39.5595810546
<class 'numpy.float64'>
47.7469936387
<class 'numpy.float64'>
39.5589845243
<class 'numpy.float64'>
47.746918989
<class 'numpy.float64'>
39.5583883177
<class 'numpy.float64'>
47.7468442652
<class 'numpy.float64'>
39.5577924347
<class 'numpy.float64'>
47.746769467
<class 'numpy.float64'>
39.5571968753
<class 'numpy.float64'>
47.746694594
<class 'numpy.float64'>
39.5566016395
<class 'numpy.float64'>
47.7466196458
<class 'numpy.float64'>
39.5560067271
<class 'numpy.float64'>
47.7465446222
<class 'numpy.float64'>
39.5554121383
<class 'numpy.float64'>
47.7464695228
<class 'numpy.float64'>
39.5548178727
<class 'numpy.float64'>
47.7463943471
<class 'numpy.float64'>
39.5542239306
<class 'numpy.float64'>
47.746319095
<class 'numpy.float64'>
39.5536303117
<class 'numpy.float64'>
47.746243766
<class 'numpy.float64'>
39.5530370159
<class 'numpy.float64'>
47.7461683599
<class 'numpy.float64'>
39.5524440434
<class 'numpy.float64'>
47.7460928763
<class 'numpy.float64'>
39.5518513939
<class 'numpy.float64'>
47.7460173149
<class 'numpy.float64'>
39.5512590674
<class 'numpy.float64'>
47.7459416753
<class 'numpy.float64'>
39.5506670639
<class 'numpy.float64'>
47.7458659573
<class 'numpy.float64'>
39.5500753833
<class 'numpy.float64'>
47.7457901606
<class 'numpy.float64'>
39.5494840254
<class 'numpy.float64'>
47.7457142849
<class 'numpy.float64'>
39.5488929903
<class 'numpy.float64'>
47.7456383298
<class 'numpy.float64'>
39.5483022778
<class 'numpy.float64'>
47.7455622951
<class 'numpy.float64'>
39.5477118879
<class 'numpy.float64'>
47.7454861805
<class 'numpy.float64'>
39.5471218206
<class 'numpy.float64'>
47.7454099857
<class 'numpy.float64'>
39.5465320756
<class 'numpy.float64'>
47.7453337103
<class 'numpy.float64'>
39.545942653
<class 'numpy.float64'>
47.7452573543
<class 'numpy.float64'>
39.5453535526
<class 'numpy.float64'>
47.7451809171
<class 'numpy.float64'>
39.5447647744
<class 'numpy.float64'>
47.7451043987
<class 'numpy.float64'>
39.5441763182
<class 'numpy.float64'>
47.7450277986
<class 'numpy.float64'>
39.5435881841
<class 'numpy.float64'>
47.7449511168
<class 'numpy.float64'>
39.5430003718
<class 'numpy.float64'>
47.7448743528
<class 'numpy.float64'>
39.5424128814
<class 'numpy.float64'>
47.7447975064
<class 'numpy.float64'>
39.5418257127
<class 'numpy.float64'>
47.7447205775
<class 'numpy.float64'>
39.5412388656
<class 'numpy.float64'>
47.7446435657
<class 'numpy.float64'>
39.54065234
<class 'numpy.float64'>
47.7445664707
<class 'numpy.float64'>
39.5400661359
<class 'numpy.float64'>
47.7444892924
<class 'numpy.float64'>
39.5394802531
<class 'numpy.float64'>
47.7444120305
<class 'numpy.float64'>
39.5388946915
<class 'numpy.float64'>
47.7443346848
<class 'numpy.float64'>
39.538309451
<class 'numpy.float64'>
47.744257255
<class 'numpy.float64'>
39.5377245316
<class 'numpy.float64'>
47.7441797409
<class 'numpy.float64'>
39.5371399331
<class 'numpy.float64'>
47.7441021423
<class 'numpy.float64'>
39.5365556554
<class 'numpy.float64'>
47.744024459
<class 'numpy.float64'>
39.5359716984
<class 'numpy.float64'>
47.7439466907
<class 'numpy.float64'>
39.535388062
<class 'numpy.float64'>
47.7438688372
<class 'numpy.float64'>
39.5348047461
<class 'numpy.float64'>
47.7437908983
<class 'numpy.float64'>
39.5342217506
<class 'numpy.float64'>
47.7437128739
<class 'numpy.float64'>
39.5336390753
<class 'numpy.float64'>
47.7436347636
<class 'numpy.float64'>
39.5330567202
<class 'numpy.float64'>
47.7435565673
<class 'numpy.float64'>
39.5324746851
<class 'numpy.float64'>
47.7434782848
<class 'numpy.float64'>
39.5318929699
<class 'numpy.float64'>
47.7433999159
<class 'numpy.float64'>
39.5313115745
<class 'numpy.float64'>
47.7433214604
<class 'numpy.float64'>
39.5307304988
<class 'numpy.float64'>
47.7432429181
<class 'numpy.float64'>
39.5301497426
<class 'numpy.float64'>
47.7431642889
<class 'numpy.float64'>
39.5295693059
<class 'numpy.float64'>
47.7430855725
<class 'numpy.float64'>
39.5289891885
<class 'numpy.float64'>
47.7430067687
<class 'numpy.float64'>
39.5284093903
<class 'numpy.float64'>
47.7429278774
<class 'numpy.float64'>
39.5278299111
<class 'numpy.float64'>
47.7428488984
<class 'numpy.float64'>
39.5272507509
<class 'numpy.float64'>
47.7427698315
<class 'numpy.float64'>
39.5266719095
<class 'numpy.float64'>
47.7426906766
<class 'numpy.float64'>
39.5260933867
<class 'numpy.float64'>
47.7426114335
<class 'numpy.float64'>
39.5255151825
<class 'numpy.float64'>
47.742532102
<class 'numpy.float64'>
39.5249372967
<class 'numpy.float64'>
47.7424526819
<class 'numpy.float64'>
39.5243597292
<class 'numpy.float64'>
47.7423731732
<class 'numpy.float64'>
39.5237824798
<class 'numpy.float64'>
47.7422935756
<class 'numpy.float64'>
39.5232055485
<class 'numpy.float64'>
47.742213889
<class 'numpy.float64'>
39.522628935
<class 'numpy.float64'>
47.7421341132
<class 'numpy.float64'>
39.5220526393
<class 'numpy.float64'>
47.7420542482
<class 'numpy.float64'>
39.5214766612
<class 'numpy.float64'>
47.7419742936
<class 'numpy.float64'>
39.5209010005
<class 'numpy.float64'>
47.7418942495
<class 'numpy.float64'>
39.5203256571
<class 'numpy.float64'>
47.7418141157
<class 'numpy.float64'>
39.519750631
<class 'numpy.float64'>
47.7417338919
<class 'numpy.float64'>
39.5191759218
<class 'numpy.float64'>
47.7416535782
<class 'numpy.float64'>
39.5186015296
<class 'numpy.float64'>
47.7415731743
<class 'numpy.float64'>
39.5180274541
<class 'numpy.float64'>
47.7414926801
<class 'numpy.float64'>
39.5174536952
<class 'numpy.float64'>
47.7414120956
<class 'numpy.float64'>
39.5168802528
<class 'numpy.float64'>
47.7413314205
<class 'numpy.float64'>
39.5163071267
<class 'numpy.float64'>
47.7412506547
<class 'numpy.float64'>
39.5157343168
<class 'numpy.float64'>
47.7411697982
<class 'numpy.float64'>
39.5151618228
<class 'numpy.float64'>
47.7410888508
<class 'numpy.float64'>
39.5145896448
<class 'numpy.float64'>
47.7410078124
<class 'numpy.float64'>
39.5140177824
<class 'numpy.float64'>
47.7409266829
<class 'numpy.float64'>
39.5134462356
<class 'numpy.float64'>
47.7408454621
<class 'numpy.float64'>
39.5128750042
<class 'numpy.float64'>
47.74076415
<class 'numpy.float64'>
39.5123040881
<class 'numpy.float64'>
47.7406827465
<class 'numpy.float64'>
39.511733487
<class 'numpy.float64'>
47.7406012514
<class 'numpy.float64'>
39.511163201
<class 'numpy.float64'>
47.7405196646
<class 'numpy.float64'>
39.5105932296
<class 'numpy.float64'>
47.7404379861
<class 'numpy.float64'>
39.510023573
<class 'numpy.float64'>
47.7403562158
<class 'numpy.float64'>
39.5094542308
<class 'numpy.float64'>
47.7402743535
<class 'numpy.float64'>
39.5088852029
<class 'numpy.float64'>
47.7401923992
<class 'numpy.float64'>
39.5083164891
<class 'numpy.float64'>
47.7401103527
<class 'numpy.float64'>
39.5077480894
<class 'numpy.float64'>
47.740028214
<class 'numpy.float64'>
39.5071800035
<class 'numpy.float64'>
47.739945983
<class 'numpy.float64'>
39.5066122312
<class 'numpy.float64'>
47.7398636597
<class 'numpy.float64'>
Cost after iteration 9000: 47.739864
39.5060447724
<class 'numpy.float64'>
47.7397812438
<class 'numpy.float64'>
39.505477627
<class 'numpy.float64'>
47.7396987354
<class 'numpy.float64'>
39.5049107948
<class 'numpy.float64'>
47.7396161344
<class 'numpy.float64'>
39.5043442756
<class 'numpy.float64'>
47.7395334407
<class 'numpy.float64'>
39.5037780692
<class 'numpy.float64'>
47.7394506542
<class 'numpy.float64'>
39.5032121755
<class 'numpy.float64'>
47.7393677748
<class 'numpy.float64'>
39.5026465943
<class 'numpy.float64'>
47.7392848025
<class 'numpy.float64'>
39.5020813254
<class 'numpy.float64'>
47.7392017373
<class 'numpy.float64'>
39.5015163687
<class 'numpy.float64'>
47.7391185789
<class 'numpy.float64'>
39.500951724
<class 'numpy.float64'>
47.7390353275
<class 'numpy.float64'>
39.5003873912
<class 'numpy.float64'>
47.7389519828
<class 'numpy.float64'>
39.49982337
<class 'numpy.float64'>
47.7388685449
<class 'numpy.float64'>
39.4992596603
<class 'numpy.float64'>
47.7387850137
<class 'numpy.float64'>
39.4986962619
<class 'numpy.float64'>
47.7387013892
<class 'numpy.float64'>
39.4981331746
<class 'numpy.float64'>
47.7386176712
<class 'numpy.float64'>
39.4975703983
<class 'numpy.float64'>
47.7385338597
<class 'numpy.float64'>
39.4970079328
<class 'numpy.float64'>
47.7384499547
<class 'numpy.float64'>
39.4964457779
<class 'numpy.float64'>
47.7383659562
<class 'numpy.float64'>
39.4958839335
<class 'numpy.float64'>
47.738281864
<class 'numpy.float64'>
39.4953223993
<class 'numpy.float64'>
47.7381976782
<class 'numpy.float64'>
39.4947611752
<class 'numpy.float64'>
47.7381133986
<class 'numpy.float64'>
39.494200261
<class 'numpy.float64'>
47.7380290253
<class 'numpy.float64'>
39.4936396565
<class 'numpy.float64'>
47.7379445582
<class 'numpy.float64'>
39.4930793616
<class 'numpy.float64'>
47.7378599973
<class 'numpy.float64'>
39.492519376
<class 'numpy.float64'>
47.7377753425
<class 'numpy.float64'>
39.4919596997
<class 'numpy.float64'>
47.7376905937
<class 'numpy.float64'>
39.4914003323
<class 'numpy.float64'>
47.7376057511
<class 'numpy.float64'>
39.4908412738
<class 'numpy.float64'>
47.7375208144
<class 'numpy.float64'>
39.4902825239
<class 'numpy.float64'>
47.7374357838
<class 'numpy.float64'>
39.4897240824
<class 'numpy.float64'>
47.7373506591
<class 'numpy.float64'>
39.4891659493
<class 'numpy.float64'>
47.7372654403
<class 'numpy.float64'>
39.4886081242
<class 'numpy.float64'>
47.7371801275
<class 'numpy.float64'>
39.4880506071
<class 'numpy.float64'>
47.7370947205
<class 'numpy.float64'>
39.4874933977
<class 'numpy.float64'>
47.7370092194
<class 'numpy.float64'>
39.4869364958
<class 'numpy.float64'>
47.7369236242
<class 'numpy.float64'>
39.4863799012
<class 'numpy.float64'>
47.7368379347
<class 'numpy.float64'>
39.4858236139
<class 'numpy.float64'>
47.7367521511
<class 'numpy.float64'>
39.4852676335
<class 'numpy.float64'>
47.7366662732
<class 'numpy.float64'>
39.4847119599
<class 'numpy.float64'>
47.7365803011
<class 'numpy.float64'>
39.4841565929
<class 'numpy.float64'>
47.7364942348
<class 'numpy.float64'>
39.4836015323
<class 'numpy.float64'>
47.7364080741
<class 'numpy.float64'>
39.4830467779
<class 'numpy.float64'>
47.7363218193
<class 'numpy.float64'>
39.4824923296
<class 'numpy.float64'>
47.7362354701
<class 'numpy.float64'>
39.4819381871
<class 'numpy.float64'>
47.7361490266
<class 'numpy.float64'>
39.4813843503
<class 'numpy.float64'>
47.7360624889
<class 'numpy.float64'>
39.4808308189
<class 'numpy.float64'>
47.7359758568
<class 'numpy.float64'>
39.4802775929
<class 'numpy.float64'>
47.7358891305
<class 'numpy.float64'>
39.4797246718
<class 'numpy.float64'>
47.7358023098
<class 'numpy.float64'>
39.4791720557
<class 'numpy.float64'>
47.7357153948
<class 'numpy.float64'>
39.4786197443
<class 'numpy.float64'>
47.7356283855
<class 'numpy.float64'>
39.4780677374
<class 'numpy.float64'>
47.7355412819
<class 'numpy.float64'>
39.4775160348
<class 'numpy.float64'>
47.7354540839
<class 'numpy.float64'>
39.4769646363
<class 'numpy.float64'>
47.7353667917
<class 'numpy.float64'>
39.4764135417
<class 'numpy.float64'>
47.7352794051
<class 'numpy.float64'>
39.4758627508
<class 'numpy.float64'>
47.7351919243
<class 'numpy.float64'>
39.4753122635
<class 'numpy.float64'>
47.7351043492
<class 'numpy.float64'>
39.4747620795
<class 'numpy.float64'>
47.7350166798
<class 'numpy.float64'>
39.4742121987
<class 'numpy.float64'>
47.7349289161
<class 'numpy.float64'>
39.4736626208
<class 'numpy.float64'>
47.7348410581
<class 'numpy.float64'>
39.4731133456
<class 'numpy.float64'>
47.734753106
<class 'numpy.float64'>
39.472564373
<class 'numpy.float64'>
47.7346650595
<class 'numpy.float64'>
39.4720157027
<class 'numpy.float64'>
47.7345769189
<class 'numpy.float64'>
39.4714673346
<class 'numpy.float64'>
47.7344886841
<class 'numpy.float64'>
39.4709192684
<class 'numpy.float64'>
47.734400355
<class 'numpy.float64'>
39.470371504
<class 'numpy.float64'>
47.7343119318
<class 'numpy.float64'>
39.4698240412
<class 'numpy.float64'>
47.7342234145
<class 'numpy.float64'>
39.4692768797
<class 'numpy.float64'>
47.734134803
<class 'numpy.float64'>
39.4687300193
<class 'numpy.float64'>
47.7340460974
<class 'numpy.float64'>
39.46818346
<class 'numpy.float64'>
47.7339572977
<class 'numpy.float64'>
39.4676372013
<class 'numpy.float64'>
47.733868404
<class 'numpy.float64'>
39.4670912433
<class 'numpy.float64'>
47.7337794162
<class 'numpy.float64'>
39.4665455856
<class 'numpy.float64'>
47.7336903345
<class 'numpy.float64'>
39.466000228
<class 'numpy.float64'>
47.7336011587
<class 'numpy.float64'>
39.4654551704
<class 'numpy.float64'>
47.733511889
<class 'numpy.float64'>
39.4649104125
<class 'numpy.float64'>
47.7334225253
<class 'numpy.float64'>
39.4643659542
<class 'numpy.float64'>
47.7333330678
<class 'numpy.float64'>
39.4638217953
<class 'numpy.float64'>
47.7332435164
<class 'numpy.float64'>
39.4632779354
<class 'numpy.float64'>
47.7331538712
<class 'numpy.float64'>
39.4627343746
<class 'numpy.float64'>
47.7330641321
<class 'numpy.float64'>
39.4621911124
<class 'numpy.float64'>
47.7329742993
<class 'numpy.float64'>
39.4616481488
<class 'numpy.float64'>
47.7328843728
<class 'numpy.float64'>
39.4611054835
<class 'numpy.float64'>
47.7327943526
<class 'numpy.float64'>
39.4605631164
<class 'numpy.float64'>
47.7327042387
<class 'numpy.float64'>
39.4600210471
<class 'numpy.float64'>
47.7326140312
<class 'numpy.float64'>
39.4594792756
<class 'numpy.float64'>
47.7325237301
<class 'numpy.float64'>
39.4589378016
<class 'numpy.float64'>
47.7324333355
<class 'numpy.float64'>
39.4583966249
<class 'numpy.float64'>
47.7323428474
<class 'numpy.float64'>
39.4578557453
<class 'numpy.float64'>
47.7322522658
<class 'numpy.float64'>
39.4573151626
<class 'numpy.float64'>
47.7321615909
<class 'numpy.float64'>
39.4567748766
<class 'numpy.float64'>
47.7320708225
<class 'numpy.float64'>
39.456234887
<class 'numpy.float64'>
47.7319799608
<class 'numpy.float64'>
39.4556951938
<class 'numpy.float64'>
47.7318890059
<class 'numpy.float64'>
39.4551557966
<class 'numpy.float64'>
47.7317979577
<class 'numpy.float64'>
39.4546166952
<class 'numpy.float64'>
47.7317068163
<class 'numpy.float64'>
39.4540778896
<class 'numpy.float64'>
47.7316155818
<class 'numpy.float64'>
39.4535393793
<class 'numpy.float64'>
47.7315242541
<class 'numpy.float64'>
39.4530011643
<class 'numpy.float64'>
47.7314328335
<class 'numpy.float64'>
39.4524632444
<class 'numpy.float64'>
47.7313413199
<class 'numpy.float64'>
39.4519256193
<class 'numpy.float64'>
47.7312497133
<class 'numpy.float64'>
39.4513882888
<class 'numpy.float64'>
47.7311580138
<class 'numpy.float64'>
39.4508512527
<class 'numpy.float64'>
47.7310662215
<class 'numpy.float64'>
39.4503145108
<class 'numpy.float64'>
47.7309743365
<class 'numpy.float64'>
39.4497780628
<class 'numpy.float64'>
47.7308823587
<class 'numpy.float64'>
39.4492419087
<class 'numpy.float64'>
47.7307902882
<class 'numpy.float64'>
39.4487060482
<class 'numpy.float64'>
47.7306981251
<class 'numpy.float64'>
39.448170481
<class 'numpy.float64'>
47.7306058695
<class 'numpy.float64'>
39.447635207
<class 'numpy.float64'>
47.7305135214
<class 'numpy.float64'>
39.4471002259
<class 'numpy.float64'>
47.7304210808
<class 'numpy.float64'>
39.4465655375
<class 'numpy.float64'>
47.7303285478
<class 'numpy.float64'>
39.4460311417
<class 'numpy.float64'>
47.7302359226
<class 'numpy.float64'>
39.4454970382
<class 'numpy.float64'>
47.730143205
<class 'numpy.float64'>
39.4449632268
<class 'numpy.float64'>
47.7300503953
<class 'numpy.float64'>
39.4444297073
<class 'numpy.float64'>
47.7299574935
<class 'numpy.float64'>
39.4438964795
<class 'numpy.float64'>
47.7298644995
<class 'numpy.float64'>
39.4433635432
<class 'numpy.float64'>
47.7297714136
<class 'numpy.float64'>
39.4428308981
<class 'numpy.float64'>
47.7296782357
<class 'numpy.float64'>
39.4422985441
<class 'numpy.float64'>
47.7295849659
<class 'numpy.float64'>
39.4417664809
<class 'numpy.float64'>
47.7294916043
<class 'numpy.float64'>
39.4412347084
<class 'numpy.float64'>
47.729398151
<class 'numpy.float64'>
39.4407032262
<class 'numpy.float64'>
47.729304606
<class 'numpy.float64'>
39.4401720343
<class 'numpy.float64'>
47.7292109694
<class 'numpy.float64'>
39.4396411324
<class 'numpy.float64'>
47.7291172412
<class 'numpy.float64'>
39.4391105203
<class 'numpy.float64'>
47.7290234216
<class 'numpy.float64'>
39.4385801978
<class 'numpy.float64'>
47.7289295105
<class 'numpy.float64'>
39.4380501646
<class 'numpy.float64'>
47.7288355081
<class 'numpy.float64'>
39.4375204206
<class 'numpy.float64'>
47.7287414145
<class 'numpy.float64'>
39.4369909655
<class 'numpy.float64'>
47.7286472296
<class 'numpy.float64'>
39.4364617991
<class 'numpy.float64'>
47.7285529537
<class 'numpy.float64'>
39.4359329213
<class 'numpy.float64'>
47.7284585866
<class 'numpy.float64'>
39.4354043317
<class 'numpy.float64'>
47.7283641286
<class 'numpy.float64'>
39.4348760303
<class 'numpy.float64'>
47.7282695797
<class 'numpy.float64'>
39.4343480167
<class 'numpy.float64'>
47.72817494
<class 'numpy.float64'>
39.4338202908
<class 'numpy.float64'>
47.7280802095
<class 'numpy.float64'>
39.4332928523
<class 'numpy.float64'>
47.7279853884
<class 'numpy.float64'>
39.4327657011
<class 'numpy.float64'>
47.7278904766
<class 'numpy.float64'>
39.4322388369
<class 'numpy.float64'>
47.7277954743
<class 'numpy.float64'>
39.4317122596
<class 'numpy.float64'>
47.7277003816
<class 'numpy.float64'>
39.4311859688
<class 'numpy.float64'>
47.7276051985
<class 'numpy.float64'>
39.4306599644
<class 'numpy.float64'>
47.7275099251
<class 'numpy.float64'>
39.4301342462
<class 'numpy.float64'>
47.7274145615
<class 'numpy.float64'>
39.429608814
<class 'numpy.float64'>
47.7273191077
<class 'numpy.float64'>
39.4290836675
<class 'numpy.float64'>
47.7272235639
<class 'numpy.float64'>
39.4285588066
<class 'numpy.float64'>
47.7271279302
<class 'numpy.float64'>
39.4280342309
<class 'numpy.float64'>
47.7270322065
<class 'numpy.float64'>
39.4275099404
<class 'numpy.float64'>
47.726936393
<class 'numpy.float64'>
39.4269859348
<class 'numpy.float64'>
47.7268404898
<class 'numpy.float64'>
39.4264622139
<class 'numpy.float64'>
47.726744497
<class 'numpy.float64'>
39.4259387774
<class 'numpy.float64'>
47.7266484146
<class 'numpy.float64'>
39.4254156252
<class 'numpy.float64'>
47.7265522427
<class 'numpy.float64'>
39.424892757
<class 'numpy.float64'>
47.7264559814
<class 'numpy.float64'>
39.4243701727
<class 'numpy.float64'>
47.7263596308
<class 'numpy.float64'>
39.423847872
<class 'numpy.float64'>
47.7262631909
<class 'numpy.float64'>
39.4233258547
<class 'numpy.float64'>
47.726166662
<class 'numpy.float64'>
39.4228041206
<class 'numpy.float64'>
47.7260700439
<class 'numpy.float64'>
39.4222826694
<class 'numpy.float64'>
47.7259733369
<class 'numpy.float64'>
39.421761501
<class 'numpy.float64'>
47.725876541
<class 'numpy.float64'>
39.4212406152
<class 'numpy.float64'>
47.7257796564
<class 'numpy.float64'>
39.4207200117
<class 'numpy.float64'>
47.725682683
<class 'numpy.float64'>
39.4201996903
<class 'numpy.float64'>
47.7255856209
<class 'numpy.float64'>
39.4196796509
<class 'numpy.float64'>
47.7254884704
<class 'numpy.float64'>
39.4191598931
<class 'numpy.float64'>
47.7253912314
<class 'numpy.float64'>
39.4186404168
<class 'numpy.float64'>
47.725293904
<class 'numpy.float64'>
39.4181212217
<class 'numpy.float64'>
47.7251964884
<class 'numpy.float64'>
39.4176023078
<class 'numpy.float64'>
47.7250989846
<class 'numpy.float64'>
39.4170836746
<class 'numpy.float64'>
47.7250013928
<class 'numpy.float64'>
39.4165653221
<class 'numpy.float64'>
47.7249037129
<class 'numpy.float64'>
39.41604725
<class 'numpy.float64'>
47.7248059451
<class 'numpy.float64'>
39.4155294581
<class 'numpy.float64'>
47.7247080895
<class 'numpy.float64'>
39.4150119461
<class 'numpy.float64'>
47.7246101462
<class 'numpy.float64'>
39.414494714
<class 'numpy.float64'>
47.7245121153
<class 'numpy.float64'>
39.4139777614
<class 'numpy.float64'>
47.7244139968
<class 'numpy.float64'>
39.4134610881
<class 'numpy.float64'>
47.7243157909
<class 'numpy.float64'>
39.4129446939
<class 'numpy.float64'>
47.7242174976
<class 'numpy.float64'>
39.4124285787
<class 'numpy.float64'>
47.7241191171
<class 'numpy.float64'>
39.4119127422
<class 'numpy.float64'>
47.7240206494
<class 'numpy.float64'>
39.4113971841
<class 'numpy.float64'>
47.7239220946
<class 'numpy.float64'>
39.4108819044
<class 'numpy.float64'>
47.7238234529
<class 'numpy.float64'>
39.4103669027
<class 'numpy.float64'>
47.7237247243
<class 'numpy.float64'>
39.4098521788
<class 'numpy.float64'>
47.7236259089
<class 'numpy.float64'>
39.4093377326
<class 'numpy.float64'>
47.7235270068
<class 'numpy.float64'>
39.4088235638
<class 'numpy.float64'>
47.7234280181
<class 'numpy.float64'>
39.4083096722
<class 'numpy.float64'>
47.723328943
<class 'numpy.float64'>
39.4077960575
<class 'numpy.float64'>
47.7232297814
<class 'numpy.float64'>
39.4072827197
<class 'numpy.float64'>
47.7231305335
<class 'numpy.float64'>
39.4067696584
<class 'numpy.float64'>
47.7230311995
<class 'numpy.float64'>
39.4062568735
<class 'numpy.float64'>
47.7229317793
<class 'numpy.float64'>
39.4057443647
<class 'numpy.float64'>
47.7228322731
<class 'numpy.float64'>
39.4052321318
<class 'numpy.float64'>
47.722732681
<class 'numpy.float64'>
39.4047201747
<class 'numpy.float64'>
47.7226330031
<class 'numpy.float64'>
39.404208493
<class 'numpy.float64'>
47.7225332395
<class 'numpy.float64'>
39.4036970866
<class 'numpy.float64'>
47.7224333903
<class 'numpy.float64'>
39.4031859553
<class 'numpy.float64'>
47.7223334555
<class 'numpy.float64'>
39.4026750989
<class 'numpy.float64'>
47.7222334354
<class 'numpy.float64'>
39.4021645171
<class 'numpy.float64'>
47.7221333299
<class 'numpy.float64'>
39.4016542097
<class 'numpy.float64'>
47.7220331392
<class 'numpy.float64'>
39.4011441766
<class 'numpy.float64'>
47.7219328634
<class 'numpy.float64'>
39.4006344174
<class 'numpy.float64'>
47.7218325026
<class 'numpy.float64'>
39.4001249321
<class 'numpy.float64'>
47.7217320568
<class 'numpy.float64'>
39.3996157203
<class 'numpy.float64'>
47.7216315263
<class 'numpy.float64'>
39.3991067819
<class 'numpy.float64'>
47.721530911
<class 'numpy.float64'>
39.3985981166
<class 'numpy.float64'>
47.7214302112
<class 'numpy.float64'>
39.3980897243
<class 'numpy.float64'>
47.7213294268
<class 'numpy.float64'>
39.3975816047
<class 'numpy.float64'>
47.721228558
<class 'numpy.float64'>
39.3970737577
<class 'numpy.float64'>
47.7211276049
<class 'numpy.float64'>
39.3965661829
<class 'numpy.float64'>
47.7210265676
<class 'numpy.float64'>
39.3960588802
<class 'numpy.float64'>
47.7209254463
<class 'numpy.float64'>
39.3955518494
<class 'numpy.float64'>
47.7208242409
<class 'numpy.float64'>
39.3950450903
<class 'numpy.float64'>
47.7207229516
<class 'numpy.float64'>
39.3945386026
<class 'numpy.float64'>
47.7206215786
<class 'numpy.float64'>
39.3940323862
<class 'numpy.float64'>
47.7205201218
<class 'numpy.float64'>
39.3935264408
<class 'numpy.float64'>
47.7204185815
<class 'numpy.float64'>
39.3930207662
<class 'numpy.float64'>
47.7203169577
<class 'numpy.float64'>
39.3925153623
<class 'numpy.float64'>
47.7202152505
<class 'numpy.float64'>
39.3920102287
<class 'numpy.float64'>
47.7201134601
<class 'numpy.float64'>
39.3915053653
<class 'numpy.float64'>
47.7200115865
<class 'numpy.float64'>
39.3910007719
<class 'numpy.float64'>
47.7199096298
<class 'numpy.float64'>
39.3904964483
<class 'numpy.float64'>
47.7198075902
<class 'numpy.float64'>
39.3899923941
<class 'numpy.float64'>
47.7197054678
<class 'numpy.float64'>
39.3894886094
<class 'numpy.float64'>
47.7196032626
<class 'numpy.float64'>
39.3889850937
<class 'numpy.float64'>
47.7195009747
<class 'numpy.float64'>
39.388481847
<class 'numpy.float64'>
47.7193986043
<class 'numpy.float64'>
39.387978869
<class 'numpy.float64'>
47.7192961515
<class 'numpy.float64'>
39.3874761595
<class 'numpy.float64'>
47.7191936164
<class 'numpy.float64'>
39.3869737182
<class 'numpy.float64'>
47.7190909991
<class 'numpy.float64'>
39.3864715451
<class 'numpy.float64'>
47.7189882997
<class 'numpy.float64'>
39.3859696397
<class 'numpy.float64'>
47.7188855182
<class 'numpy.float64'>
39.3854680021
<class 'numpy.float64'>
47.7187826549
<class 'numpy.float64'>
39.3849666319
<class 'numpy.float64'>
47.7186797098
<class 'numpy.float64'>
39.3844655289
<class 'numpy.float64'>
47.718576683
<class 'numpy.float64'>
39.3839646929
<class 'numpy.float64'>
47.7184735746
<class 'numpy.float64'>
39.3834641237
<class 'numpy.float64'>
47.7183703848
<class 'numpy.float64'>
39.3829638212
<class 'numpy.float64'>
47.7182671136
<class 'numpy.float64'>
39.382463785
<class 'numpy.float64'>
47.7181637612
<class 'numpy.float64'>
39.381964015
<class 'numpy.float64'>
47.7180603276
<class 'numpy.float64'>
39.381464511
<class 'numpy.float64'>
47.717956813
<class 'numpy.float64'>
39.3809652727
<class 'numpy.float64'>
47.7178532175
<class 'numpy.float64'>
39.3804663
<class 'numpy.float64'>
47.7177495411
<class 'numpy.float64'>
39.3799675926
<class 'numpy.float64'>
47.7176457841
<class 'numpy.float64'>
39.3794691504
<class 'numpy.float64'>
47.7175419464
<class 'numpy.float64'>
39.3789709731
<class 'numpy.float64'>
47.7174380283
<class 'numpy.float64'>
39.3784730605
<class 'numpy.float64'>
47.7173340297
<class 'numpy.float64'>
39.3779754124
<class 'numpy.float64'>
47.7172299509
<class 'numpy.float64'>
39.3774780286
<class 'numpy.float64'>
47.717125792
<class 'numpy.float64'>
39.3769809088
<class 'numpy.float64'>
47.7170215529
<class 'numpy.float64'>
39.376484053
<class 'numpy.float64'>
47.7169172339
<class 'numpy.float64'>
39.3759874608
<class 'numpy.float64'>
47.7168128351
<class 'numpy.float64'>
39.3754911321
<class 'numpy.float64'>
47.7167083566
<class 'numpy.float64'>
39.3749950666
<class 'numpy.float64'>
47.7166037984
<class 'numpy.float64'>
39.3744992642
<class 'numpy.float64'>
47.7164991608
<class 'numpy.float64'>
39.3740037246
<class 'numpy.float64'>
47.7163944437
<class 'numpy.float64'>
39.3735084477
<class 'numpy.float64'>
47.7162896473
<class 'numpy.float64'>
39.3730134331
<class 'numpy.float64'>
47.7161847718
<class 'numpy.float64'>
39.3725186808
<class 'numpy.float64'>
47.7160798172
<class 'numpy.float64'>
39.3720241905
<class 'numpy.float64'>
47.7159747836
<class 'numpy.float64'>
39.3715299619
<class 'numpy.float64'>
47.7158696712
<class 'numpy.float64'>
39.371035995
<class 'numpy.float64'>
47.71576448
<class 'numpy.float64'>
39.3705422895
<class 'numpy.float64'>
47.7156592102
<class 'numpy.float64'>
39.3700488451
<class 'numpy.float64'>
47.7155538619
<class 'numpy.float64'>
39.3695556617
<class 'numpy.float64'>
47.7154484352
<class 'numpy.float64'>
39.369062739
<class 'numpy.float64'>
47.7153429302
<class 'numpy.float64'>
39.3685700769
<class 'numpy.float64'>
47.715237347
<class 'numpy.float64'>
39.3680776752
<class 'numpy.float64'>
47.7151316857
<class 'numpy.float64'>
39.3675855336
<class 'numpy.float64'>
47.7150259464
<class 'numpy.float64'>
39.3670936519
<class 'numpy.float64'>
47.7149201293
<class 'numpy.float64'>
39.36660203
<class 'numpy.float64'>
47.7148142345
<class 'numpy.float64'>
39.3661106675
<class 'numpy.float64'>
47.714708262
<class 'numpy.float64'>
39.3656195644
<class 'numpy.float64'>
47.714602212
<class 'numpy.float64'>
39.3651287205
<class 'numpy.float64'>
47.7144960846
<class 'numpy.float64'>
39.3646381354
<class 'numpy.float64'>
47.7143898798
<class 'numpy.float64'>
39.364147809
<class 'numpy.float64'>
47.7142835979
<class 'numpy.float64'>
39.3636577411
<class 'numpy.float64'>
47.7141772389
<class 'numpy.float64'>
39.3631679315
<class 'numpy.float64'>
47.714070803
<class 'numpy.float64'>
39.36267838
<class 'numpy.float64'>
47.7139642902
<class 'numpy.float64'>
39.3621890864
<class 'numpy.float64'>
47.7138577006
<class 'numpy.float64'>
39.3617000504
<class 'numpy.float64'>
47.7137510344
<class 'numpy.float64'>
39.3612112719
<class 'numpy.float64'>
47.7136442917
<class 'numpy.float64'>
39.3607227507
<class 'numpy.float64'>
47.7135374726
<class 'numpy.float64'>
39.3602344865
<class 'numpy.float64'>
47.7134305772
<class 'numpy.float64'>
39.3597464792
<class 'numpy.float64'>
47.7133236056
<class 'numpy.float64'>
39.3592587286
<class 'numpy.float64'>
47.7132165579
<class 'numpy.float64'>
39.3587712344
<class 'numpy.float64'>
47.7131094343
<class 'numpy.float64'>
39.3582839964
<class 'numpy.float64'>
47.7130022348
<class 'numpy.float64'>
39.3577970145
<class 'numpy.float64'>
47.7128949596
<class 'numpy.float64'>
39.3573102884
<class 'numpy.float64'>
47.7127876087
<class 'numpy.float64'>
39.3568238179
<class 'numpy.float64'>
47.7126801823
<class 'numpy.float64'>
39.3563376029
<class 'numpy.float64'>
47.7125726805
<class 'numpy.float64'>
39.3558516431
<class 'numpy.float64'>
47.7124651035
<class 'numpy.float64'>
39.3553659384
<class 'numpy.float64'>
47.7123574512
<class 'numpy.float64'>
39.3548804884
<class 'numpy.float64'>
47.7122497239
<class 'numpy.float64'>
39.3543952931
<class 'numpy.float64'>
47.7121419216
<class 'numpy.float64'>
39.3539103522
<class 'numpy.float64'>
47.7120340445
<class 'numpy.float64'>
39.3534256655
<class 'numpy.float64'>
47.7119260926
<class 'numpy.float64'>
39.3529412328
<class 'numpy.float64'>
47.7118180661
<class 'numpy.float64'>
39.3524570539
<class 'numpy.float64'>
47.7117099651
<class 'numpy.float64'>
39.3519731287
<class 'numpy.float64'>
47.7116017897
<class 'numpy.float64'>
39.3514894568
<class 'numpy.float64'>
47.71149354
<class 'numpy.float64'>
39.3510060382
<class 'numpy.float64'>
47.7113852161
<class 'numpy.float64'>
39.3505228725
<class 'numpy.float64'>
47.7112768182
<class 'numpy.float64'>
39.3500399597
<class 'numpy.float64'>
47.7111683464
<class 'numpy.float64'>
39.3495572994
<class 'numpy.float64'>
47.7110598006
<class 'numpy.float64'>
39.3490748916
<class 'numpy.float64'>
47.7109511812
<class 'numpy.float64'>
39.3485927359
<class 'numpy.float64'>
47.7108424881
<class 'numpy.float64'>
39.3481108323
<class 'numpy.float64'>
47.7107337216
<class 'numpy.float64'>
39.3476291804
<class 'numpy.float64'>
47.7106248816
<class 'numpy.float64'>
39.3471477802
<class 'numpy.float64'>
47.7105159684
<class 'numpy.float64'>
39.3466666313
<class 'numpy.float64'>
47.710406982
<class 'numpy.float64'>
39.3461857337
<class 'numpy.float64'>
47.7102979226
<class 'numpy.float64'>
39.345705087
<class 'numpy.float64'>
47.7101887902
<class 'numpy.float64'>
39.3452246912
<class 'numpy.float64'>
47.7100795849
<class 'numpy.float64'>
39.3447445459
<class 'numpy.float64'>
47.709970307
<class 'numpy.float64'>
39.344264651
<class 'numpy.float64'>
47.7098609564
<class 'numpy.float64'>
39.3437850064
<class 'numpy.float64'>
47.7097515334
<class 'numpy.float64'>
39.3433056117
<class 'numpy.float64'>
47.709642038
<class 'numpy.float64'>
39.3428264669
<class 'numpy.float64'>
47.7095324703
<class 'numpy.float64'>
39.3423475716
<class 'numpy.float64'>
47.7094228304
<class 'numpy.float64'>
39.3418689257
<class 'numpy.float64'>
47.7093131185
<class 'numpy.float64'>
39.3413905291
<class 'numpy.float64'>
47.7092033346
<class 'numpy.float64'>
39.3409123814
<class 'numpy.float64'>
47.7090934789
<class 'numpy.float64'>
39.3404344826
<class 'numpy.float64'>
47.7089835515
<class 'numpy.float64'>
39.3399568324
<class 'numpy.float64'>
47.7088735525
<class 'numpy.float64'>
39.3394794306
<class 'numpy.float64'>
47.708763482
<class 'numpy.float64'>
39.339002277
<class 'numpy.float64'>
47.7086533402
<class 'numpy.float64'>
39.3385253714
<class 'numpy.float64'>
47.708543127
<class 'numpy.float64'>
39.3380487137
<class 'numpy.float64'>
47.7084328428
<class 'numpy.float64'>
39.3375723035
<class 'numpy.float64'>
47.7083224874
<class 'numpy.float64'>
39.3370961408
<class 'numpy.float64'>
47.7082120611
<class 'numpy.float64'>
39.3366202254
<class 'numpy.float64'>
47.7081015641
<class 'numpy.float64'>
39.3361445569
<class 'numpy.float64'>
47.7079909963
<class 'numpy.float64'>
39.3356691353
<class 'numpy.float64'>
47.7078803579
<class 'numpy.float64'>
39.3351939604
<class 'numpy.float64'>
47.707769649
<class 'numpy.float64'>
39.3347190319
<class 'numpy.float64'>
47.7076588697
<class 'numpy.float64'>
39.3342443496
<class 'numpy.float64'>
47.7075480202
<class 'numpy.float64'>
39.3337699134
<class 'numpy.float64'>
47.7074371005
<class 'numpy.float64'>
39.3332957231
<class 'numpy.float64'>
47.7073261108
<class 'numpy.float64'>
39.3328217784
<class 'numpy.float64'>
47.7072150511
<class 'numpy.float64'>
39.3323480791
<class 'numpy.float64'>
47.7071039216
<class 'numpy.float64'>
39.3318746252
<class 'numpy.float64'>
47.7069927225
<class 'numpy.float64'>
39.3314014163
<class 'numpy.float64'>
47.7068814537
<class 'numpy.float64'>
39.3309284523
<class 'numpy.float64'>
47.7067701154
<class 'numpy.float64'>
39.330455733
<class 'numpy.float64'>
47.7066587077
<class 'numpy.float64'>
39.3299832582
<class 'numpy.float64'>
47.7065472308
<class 'numpy.float64'>
39.3295110277
<class 'numpy.float64'>
47.7064356848
<class 'numpy.float64'>
39.3290390413
<class 'numpy.float64'>
47.7063240696
<class 'numpy.float64'>
39.3285672988
<class 'numpy.float64'>
47.7062123856
<class 'numpy.float64'>
39.3280958
<class 'numpy.float64'>
47.7061006327
<class 'numpy.float64'>
39.3276245448
<class 'numpy.float64'>
47.7059888111
<class 'numpy.float64'>
39.3271535328
<class 'numpy.float64'>
47.705876921
<class 'numpy.float64'>
39.326682764
<class 'numpy.float64'>
47.7057649623
<class 'numpy.float64'>
39.3262122382
<class 'numpy.float64'>
47.7056529352
<class 'numpy.float64'>
39.3257419551
<class 'numpy.float64'>
47.7055408399
<class 'numpy.float64'>
39.3252719146
<class 'numpy.float64'>
47.7054286764
<class 'numpy.float64'>
39.3248021164
<class 'numpy.float64'>
47.7053164449
<class 'numpy.float64'>
39.3243325604
<class 'numpy.float64'>
47.7052041454
<class 'numpy.float64'>
39.3238632464
<class 'numpy.float64'>
47.7050917781
<class 'numpy.float64'>
39.3233941742
<class 'numpy.float64'>
47.704979343
<class 'numpy.float64'>
39.3229253435
<class 'numpy.float64'>
47.7048668404
<class 'numpy.float64'>
39.3224567543
<class 'numpy.float64'>
47.7047542703
<class 'numpy.float64'>
39.3219884063
<class 'numpy.float64'>
47.7046416327
<class 'numpy.float64'>
39.3215202993
<class 'numpy.float64'>
47.7045289279
<class 'numpy.float64'>
39.3210524332
<class 'numpy.float64'>
47.7044161559
<class 'numpy.float64'>
39.3205848076
<class 'numpy.float64'>
47.7043033169
<class 'numpy.float64'>
39.3201174226
<class 'numpy.float64'>
47.7041904109
<class 'numpy.float64'>
39.3196502778
<class 'numpy.float64'>
47.704077438
<class 'numpy.float64'>
39.319183373
<class 'numpy.float64'>
47.7039643985
<class 'numpy.float64'>
39.3187167082
<class 'numpy.float64'>
47.7038512923
<class 'numpy.float64'>
39.318250283
<class 'numpy.float64'>
47.7037381195
<class 'numpy.float64'>
39.3177840973
<class 'numpy.float64'>
47.7036248804
<class 'numpy.float64'>
39.317318151
<class 'numpy.float64'>
47.703511575
<class 'numpy.float64'>
39.3168524437
<class 'numpy.float64'>
47.7033982034
<class 'numpy.float64'>
39.3163869754
<class 'numpy.float64'>
47.7032847657
<class 'numpy.float64'>
39.3159217458
<class 'numpy.float64'>
47.7031712621
<class 'numpy.float64'>
39.3154567548
<class 'numpy.float64'>
47.7030576926
<class 'numpy.float64'>
39.3149920022
<class 'numpy.float64'>
47.7029440573
<class 'numpy.float64'>
39.3145274877
<class 'numpy.float64'>
47.7028303564
<class 'numpy.float64'>
39.3140632112
<class 'numpy.float64'>
47.70271659
<class 'numpy.float64'>
39.3135991725
<class 'numpy.float64'>
47.7026027581
<class 'numpy.float64'>
39.3131353714
<class 'numpy.float64'>
47.7024888609
<class 'numpy.float64'>
39.3126718077
<class 'numpy.float64'>
47.7023748985
<class 'numpy.float64'>
39.3122084813
<class 'numpy.float64'>
47.702260871
<class 'numpy.float64'>
39.3117453919
<class 'numpy.float64'>
47.7021467786
<class 'numpy.float64'>
39.3112825394
<class 'numpy.float64'>
47.7020326212
<class 'numpy.float64'>
39.3108199235
<class 'numpy.float64'>
47.701918399
<class 'numpy.float64'>
39.3103575441
<class 'numpy.float64'>
47.7018041122
<class 'numpy.float64'>
39.309895401
<class 'numpy.float64'>
47.7016897608
<class 'numpy.float64'>
39.3094334941
<class 'numpy.float64'>
47.701575345
<class 'numpy.float64'>
39.308971823
<class 'numpy.float64'>
47.7014608648
<class 'numpy.float64'>
39.3085103877
<class 'numpy.float64'>
47.7013463204
<class 'numpy.float64'>
39.3080491879
<class 'numpy.float64'>
47.7012317119
<class 'numpy.float64'>
39.3075882235
<class 'numpy.float64'>
47.7011170393
<class 'numpy.float64'>
39.3071274943
<class 'numpy.float64'>
47.7010023028
<class 'numpy.float64'>
39.3066670001
<class 'numpy.float64'>
47.7008875025
<class 'numpy.float64'>
39.3062067407
<class 'numpy.float64'>
47.7007726385
<class 'numpy.float64'>
39.3057467159
<class 'numpy.float64'>
47.7006577109
<class 'numpy.float64'>
39.3052869256
<class 'numpy.float64'>
47.7005427198
<class 'numpy.float64'>
39.3048273695
<class 'numpy.float64'>
47.7004276654
<class 'numpy.float64'>
39.3043680474
<class 'numpy.float64'>
47.7003125476
<class 'numpy.float64'>
39.3039089593
<class 'numpy.float64'>
47.7001973667
<class 'numpy.float64'>
39.3034501048
<class 'numpy.float64'>
47.7000821228
<class 'numpy.float64'>
39.3029914839
<class 'numpy.float64'>
47.6999668158
<class 'numpy.float64'>
39.3025330963
<class 'numpy.float64'>
47.6998514461
<class 'numpy.float64'>
39.3020749419
<class 'numpy.float64'>
47.6997360135
<class 'numpy.float64'>
39.3016170204
<class 'numpy.float64'>
47.6996205184
<class 'numpy.float64'>
39.3011593317
<class 'numpy.float64'>
47.6995049607
<class 'numpy.float64'>
39.3007018755
<class 'numpy.float64'>
47.6993893406
<class 'numpy.float64'>
39.3002446518
<class 'numpy.float64'>
47.6992736582
<class 'numpy.float64'>
39.2997876604
<class 'numpy.float64'>
47.6991579136
<class 'numpy.float64'>
39.299330901
<class 'numpy.float64'>
47.6990421068
<class 'numpy.float64'>
39.2988743734
<class 'numpy.float64'>
47.6989262381
<class 'numpy.float64'>
39.2984180775
<class 'numpy.float64'>
47.6988103074
<class 'numpy.float64'>
39.2979620132
<class 'numpy.float64'>
47.698694315
<class 'numpy.float64'>
39.2975061801
<class 'numpy.float64'>
47.6985782609
<class 'numpy.float64'>
39.2970505782
<class 'numpy.float64'>
47.6984621452
<class 'numpy.float64'>
39.2965952072
<class 'numpy.float64'>
47.698345968
<class 'numpy.float64'>
39.296140067
<class 'numpy.float64'>
47.6982297295
<class 'numpy.float64'>
39.2956851574
<class 'numpy.float64'>
47.6981134297
<class 'numpy.float64'>
39.2952304783
<class 'numpy.float64'>
47.6979970687
<class 'numpy.float64'>
39.2947760293
<class 'numpy.float64'>
47.6978806467
<class 'numpy.float64'>
39.2943218104
<class 'numpy.float64'>
47.6977641637
<class 'numpy.float64'>
39.2938678214
<class 'numpy.float64'>
47.6976476199
<class 'numpy.float64'>
39.293414062
<class 'numpy.float64'>
47.6975310153
<class 'numpy.float64'>
39.2929605322
<class 'numpy.float64'>
47.6974143501
<class 'numpy.float64'>
39.2925072317
<class 'numpy.float64'>
47.6972976244
<class 'numpy.float64'>
39.2920541603
<class 'numpy.float64'>
47.6971808382
<class 'numpy.float64'>
39.2916013179
<class 'numpy.float64'>
47.6970639917
<class 'numpy.float64'>
39.2911487043
<class 'numpy.float64'>
47.696947085
<class 'numpy.float64'>
39.2906963193
<class 'numpy.float64'>
47.6968301181
<class 'numpy.float64'>
39.2902441628
<class 'numpy.float64'>
47.6967130913
<class 'numpy.float64'>
39.2897922344
<class 'numpy.float64'>
47.6965960045
<class 'numpy.float64'>
39.2893405342
<class 'numpy.float64'>
47.696478858
<class 'numpy.float64'>
39.2888890619
<class 'numpy.float64'>
47.6963616517
<class 'numpy.float64'>
39.2884378172
<class 'numpy.float64'>
47.6962443858
<class 'numpy.float64'>
39.2879868001
<class 'numpy.float64'>
47.6961270604
<class 'numpy.float64'>
39.2875360104
<class 'numpy.float64'>
47.6960096756
<class 'numpy.float64'>
39.2870854479
<class 'numpy.float64'>
47.6958922316
<class 'numpy.float64'>
39.2866351123
<class 'numpy.float64'>
47.6957747283
<class 'numpy.float64'>
39.2861850036
<class 'numpy.float64'>
47.695657166
<class 'numpy.float64'>
39.2857351215
<class 'numpy.float64'>
47.6955395446
<class 'numpy.float64'>
39.2852854659
<class 'numpy.float64'>
47.6954218644
<class 'numpy.float64'>
39.2848360366
<class 'numpy.float64'>
47.6953041254
<class 'numpy.float64'>
39.2843868334
<class 'numpy.float64'>
47.6951863277
<class 'numpy.float64'>
39.2839378562
<class 'numpy.float64'>
47.6950684714
<class 'numpy.float64'>
39.2834891047
<class 'numpy.float64'>
47.6949505567
<class 'numpy.float64'>
39.2830405788
<class 'numpy.float64'>
47.6948325836
<class 'numpy.float64'>
39.2825922783
<class 'numpy.float64'>
47.6947145522
<class 'numpy.float64'>
39.2821442031
<class 'numpy.float64'>
47.6945964627
<class 'numpy.float64'>
39.2816963529
<class 'numpy.float64'>
47.6944783151
<class 'numpy.float64'>
39.2812487276
<class 'numpy.float64'>
47.6943601095
<class 'numpy.float64'>
39.280801327
<class 'numpy.float64'>
47.694241846
<class 'numpy.float64'>
39.280354151
<class 'numpy.float64'>
47.6941235248
<class 'numpy.float64'>
39.2799071993
<class 'numpy.float64'>
47.6940051459
<class 'numpy.float64'>
39.2794604718
<class 'numpy.float64'>
47.6938867095
<class 'numpy.float64'>
39.2790139683
<class 'numpy.float64'>
47.6937682156
<class 'numpy.float64'>
39.2785676886
<class 'numpy.float64'>
47.6936496643
<class 'numpy.float64'>
39.2781216326
<class 'numpy.float64'>
47.6935310557
<class 'numpy.float64'>
39.2776758001
<class 'numpy.float64'>
47.69341239
<class 'numpy.float64'>
39.2772301909
<class 'numpy.float64'>
47.6932936673
<class 'numpy.float64'>
39.2767848049
<class 'numpy.float64'>
47.6931748876
<class 'numpy.float64'>
39.2763396418
<class 'numpy.float64'>
47.693056051
<class 'numpy.float64'>
39.2758947015
<class 'numpy.float64'>
47.6929371577
<class 'numpy.float64'>
39.2754499838
<class 'numpy.float64'>
47.6928182077
<class 'numpy.float64'>
39.2750054886
<class 'numpy.float64'>
47.6926992011
<class 'numpy.float64'>
39.2745612156
<class 'numpy.float64'>
47.6925801381
<class 'numpy.float64'>
39.2741171647
<class 'numpy.float64'>
47.6924610187
<class 'numpy.float64'>
39.2736733358
<class 'numpy.float64'>
47.692341843
<class 'numpy.float64'>
39.2732297286
<class 'numpy.float64'>
47.6922226112
<class 'numpy.float64'>
39.2727863431
<class 'numpy.float64'>
47.6921033233
<class 'numpy.float64'>
39.2723431789
<class 'numpy.float64'>
47.6919839795
<class 'numpy.float64'>
39.271900236
<class 'numpy.float64'>
47.6918645798
<class 'numpy.float64'>
39.2714575141
<class 'numpy.float64'>
47.6917451243
<class 'numpy.float64'>
39.2710150131
<class 'numpy.float64'>
47.6916256131
<class 'numpy.float64'>
39.2705727329
<class 'numpy.float64'>
47.6915060464
<class 'numpy.float64'>
39.2701306732
<class 'numpy.float64'>
47.6913864242
<class 'numpy.float64'>
39.2696888339
<class 'numpy.float64'>
47.6912667466
<class 'numpy.float64'>
39.2692472148
<class 'numpy.float64'>
47.6911470138
<class 'numpy.float64'>
39.2688058158
<class 'numpy.float64'>
47.6910272257
<class 'numpy.float64'>
39.2683646366
<class 'numpy.float64'>
47.6909073826
<class 'numpy.float64'>
39.2679236771
<class 'numpy.float64'>
47.6907874845
<class 'numpy.float64'>
39.2674829372
<class 'numpy.float64'>
47.6906675316
<class 'numpy.float64'>
39.2670424166
<class 'numpy.float64'>
47.6905475238
<class 'numpy.float64'>
39.2666021153
<class 'numpy.float64'>
47.6904274613
<class 'numpy.float64'>
39.2661620329
<class 'numpy.float64'>
47.6903073443
<class 'numpy.float64'>
39.2657221694
<class 'numpy.float64'>
47.6901871727
<class 'numpy.float64'>
39.2652825246
<class 'numpy.float64'>
47.6900669468
<class 'numpy.float64'>
39.2648430983
<class 'numpy.float64'>
47.6899466665
<class 'numpy.float64'>
39.2644038903
<class 'numpy.float64'>
47.6898263321
<class 'numpy.float64'>
39.2639649006
<class 'numpy.float64'>
47.6897059435
<class 'numpy.float64'>
39.2635261288
<class 'numpy.float64'>
47.6895855009
<class 'numpy.float64'>
39.2630875749
<class 'numpy.float64'>
47.6894650044
<class 'numpy.float64'>
39.2626492387
<class 'numpy.float64'>
47.6893444541
<class 'numpy.float64'>
39.2622111199
<class 'numpy.float64'>
47.68922385
<class 'numpy.float64'>
39.2617732185
<class 'numpy.float64'>
47.6891031924
<class 'numpy.float64'>
39.2613355343
<class 'numpy.float64'>
47.6889824812
<class 'numpy.float64'>
39.2608980671
<class 'numpy.float64'>
47.6888617165
<class 'numpy.float64'>
39.2604608167
<class 'numpy.float64'>
47.6887408985
<class 'numpy.float64'>
39.260023783
<class 'numpy.float64'>
47.6886200273
<class 'numpy.float64'>
39.2595869658
<class 'numpy.float64'>
47.6884991029
<class 'numpy.float64'>
39.2591503649
<class 'numpy.float64'>
47.6883781255
<class 'numpy.float64'>
39.2587139802
<class 'numpy.float64'>
47.6882570951
<class 'numpy.float64'>
39.2582778115
<class 'numpy.float64'>
47.6881360118
<class 'numpy.float64'>
39.2578418587
<class 'numpy.float64'>
<matplotlib.text.Text at 0x7f0e1993ccc0>

Expected Output:

Cost after iteration 9000	0.218607

# Print accuracy
predictions = predict(parameters, X)
print ('Accuracy: %d' % float((np.dot(Y,predictions.T) + np.dot(1-Y,1-predictions.T))/float(Y.size)*100) + '%')
Accuracy: 90%
Expected Output:

Accuracy	90%
Accuracy is really high compared to Logistic Regression. The model has learnt the leaf patterns of the flower! Neural networks are able to learn even highly non-linear decision boundaries, unlike logistic regression.

Now, let's try out several hidden layer sizes.

4.6 - Tuning hidden layer size (optional/ungraded exercise)
Run the following code. It may take 1-2 minutes. You will observe different behaviors of the model for various hidden layer sizes.


# This may take about 2 minutes to run
​
plt.figure(figsize=(16, 32))
hidden_layer_sizes = [1, 2, 3, 4, 5, 20, 50]
for i, n_h in enumerate(hidden_layer_sizes):
    plt.subplot(5, 2, i+1)
    plt.title('Hidden Layer of size %d' % n_h)
    parameters = nn_model(X, Y, n_h, num_iterations = 5000)
    plot_decision_boundary(lambda x: predict(parameters, x.T), X, Y)
    predictions = predict(parameters, X)
    accuracy = float((np.dot(Y,predictions.T) + np.dot(1-Y,1-predictions.T))/float(Y.size)*100)
    print ("Accuracy for {} hidden units: {} %".format(n_h, accuracy))
Interpretation:

The larger models (with more hidden units) are able to fit the training set better, until eventually the largest models overfit the data.
The best hidden layer size seems to be around n_h = 5. Indeed, a value around here seems to fits the data well without also incurring noticable overfitting.
You will also learn later about regularization, which lets you use very large models (such as n_h = 50) without much overfitting.
Optional questions:

Note: Remember to submit the assignment but clicking the blue "Submit Assignment" button at the upper-right.

Some optional/ungraded questions that you can explore if you wish:

What happens when you change the tanh activation for a sigmoid activation or a ReLU activation?
Play with the learning_rate. What happens?
What if we change the dataset? (See part 5 below!)
You've learnt to:

Build a complete neural network with a hidden layer
Make a good use of a non-linear unit
Implemented forward propagation and backpropagation, and trained a neural network
See the impact of varying the hidden layer size, including overfitting.
Nice work!

5) Performance on other datasets
If you want, you can rerun the whole notebook (minus the dataset part) for each of the following datasets.


# Datasets
noisy_circles, noisy_moons, blobs, gaussian_quantiles, no_structure = load_extra_datasets()
​
datasets = {"noisy_circles": noisy_circles,
            "noisy_moons": noisy_moons,
            "blobs": blobs,
            "gaussian_quantiles": gaussian_quantiles}
​
### START CODE HERE ### (choose your dataset)
dataset = "noisy_moons"
### END CODE HERE ###
​
X, Y = datasets[dataset]
X, Y = X.T, Y.reshape(1, Y.shape[0])
​
# make blobs binary
if dataset == "blobs":
    Y = Y%2
​
# Visualize the data
plt.scatter(X[0, :], X[1, :], c=Y, s=40, cmap=plt.cm.Spectral);

Congrats on finishing this Programming Assignment!

Reference:

http://scs.ryerson.ca/~aharley/neural-networks/
http://cs231n.github.io/neural-networks-case-study/